In [ ]:
!pip install imgaug==0.2.5
!pip install Pillow==5.0.0
!pip install torch==1.3.0+cu100 torchvision==0.4.1+cu100 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import shutil
import random
import sys

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

plt.ion()  

archivo = open("/content/drive/My Drive/Datos_Train_VGG13_BN.txt","w")

**BASE DE DATOS ALEATORIA**

In [ ]:
#Definición clases (nombres de las carpetas en la base de datos)
clase1="2-Incendio"
clase2="1-No incendio"

#Directorio con la base de datos original
dir_origen="/content/drive/MyDrive/base de datos final color/"
size=len(os.listdir(dir_origen + str(clase1))) # Retorna una lista del directorio deseado (listdir). con (len) la cantidad de archivos que tiene el directorio
#porc_train=int(0.9*size) # Porcentaje de datos para entrenamiento normalizado a 1 (80% en este ejemplo)
porc_val=int(0.1*size)

#Ciclo para K=10 en validación cruzada, crea las 10 bases de datos 
for i in range(1,11):
    print("base",i)
    dir_base="/content/drive/MyDrive/Base" + str(i) +"/"
    count = 0 #Iniciaclización contador clase 1
    count1 = 0 #Iniciaclización contador clase 2


    if os.path.exists(dir_base):                        #Condicional pregunta si directorio 'base de datos' existe
        shutil.rmtree(dir_base, ignore_errors=False)    #Si existe elimina directorio 
    else:                                                    #Si no existe se crea uno nuevo
        print("directorio no existe, se creara uno nuevo")

    #Creación de los directorios de las bases de datos
    os.mkdir(dir_base) 
    os.mkdir(dir_base + "train") 
    os.mkdir(dir_base + "val")
    os.mkdir(dir_base + "train/2-Incendio" ) 
    os.mkdir(dir_base + "train/1-No incendio")
    os.mkdir(dir_base + "val/2-Incendio" ) 
    os.mkdir(dir_base + "val/1-No incendio")
    
    #límite para que las bases de datos sean distintas durante la validación cruzada.
    lim_sup=porc_val*i
    
    ############## Proceso de asignación de imágenes en val y train de la clase 1 ####################### 
    
    if i==1:
        fires=os.listdir(dir_origen + str(clase1))
        #print("fires")
        #print(fires)
        random.shuffle(fires) 
  
    for file in fires: 
        
        if i==1:
            if count >=lim_sup-porc_val and count < lim_sup:       
                dir_destino=dir_base + "val/" + str(clase1) + "/"
            if count >= lim_sup:       
                dir_destino=dir_base + "train/" + str(clase1) + "/"
        
        if i==10:
            if count >=lim_sup-porc_val and count < lim_sup:                 
                dir_destino=dir_base + "val/" + str(clase1) + "/"
            if count < lim_sup-porc_val: 
                dir_destino=dir_base + "train/" + str(clase1) + "/"

        if i>=2 and i<=9:
            if count >=lim_sup-porc_val and count < lim_sup:                
                dir_destino=dir_base + "val/" + str(clase1) + "/"
            if count < lim_sup-porc_val or count>=lim_sup: 
                dir_destino=dir_base + "train/" + str(clase1) + "/"

        print("fires ",count)
        shutil.copy(dir_origen + str(clase1) + "/" + str(file),dir_destino + str(file))    
        count += 1

    #####################################################################################################

    ############## Proceso de asignación de imágenes en val y train de la clase 2 ####################### 
    
    if i==1:
        no_fires=os.listdir(dir_origen + str(clase2))
        #print("no_fires")
        #print(no_fires)
        random.shuffle(no_fires) 


    for file in no_fires: 
            
        if i==1:
            if count1 >=lim_sup-porc_val and count1 < lim_sup:             
                dir_destino=dir_base + "val/" + str(clase2) + "/"
            if count1 >= lim_sup:    
                dir_destino=dir_base + "train/" + str(clase2) + "/"
        if i==10:
            if count1 >=lim_sup-porc_val and count1 < lim_sup:              
                dir_destino=dir_base + "val/" + str(clase2) + "/"
            if count1 < lim_sup-porc_val:      
                dir_destino=dir_base + "train/" + str(clase2) + "/"

        if i>=2 and i<=9:
            if count1 >=lim_sup-porc_val and count1 < lim_sup:                
                dir_destino=dir_base + "val/" + str(clase2) + "/"
            if count1 < lim_sup-porc_val or count1>=lim_sup:      
                dir_destino=dir_base + "train/" + str(clase2) + "/"
        print("no_fires ",count1)
        shutil.copy(dir_origen + str(clase2) + "/" + str(file),dir_destino + str(file))    
        count1 += 1

    #####################################################################################################


In [ ]:
name_model='VGG13_BN'
k_cont=1         #Número de base a seleccionar
mod_cont=9

print("Modelo = ",name_model)
archivo.write("\n###################################################################")

while(k_cont<11):

  ###############################CARGA DE DATASET Y APLICACIÓN DE TRANSFORMACIONES################################

  data_transforms = {
      'train': transforms.Compose([
          transforms.RandomResizedCrop(224), #recorta la imagen en forma aleatoria (224x224 pixeles)
          transforms.RandomHorizontalFlip(), #rota imagen horizontalmente 
          transforms.ToTensor(),             #convierte la imagen a formato matricial 
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #normaliza cada pixel segun la media y desviacion estandar 
      ]),
      'val': transforms.Compose([
          transforms.Resize(256), #reduce la imagen por submuestreo 
          transforms.CenterCrop(224), #recorta las imagenes,toma punto de referencia el centro
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
      ]),
  }
  
  if(k_cont==1):
    data_dir = '/content/drive/My Drive/Base1' #dirección de la carpeta de base de datos
    save_model=name_model +' model_It1.pth'

  if(k_cont==2):
    data_dir = '/content/drive/My Drive/Base2' #dirección de la carpeta de base de datos 
    save_model=name_model +' model_It2.pth'

  if(k_cont==3):
    data_dir = '/content/drive/My Drive/Base3' #dirección de la carpeta de base de datos 
    save_model=name_model +' model_It3.pth'
  
  if(k_cont==4):
    data_dir = '/content/drive/My Drive/Base4' #dirección de la carpeta de base de datos 
    save_model=name_model +' model_It4.pth'

  if(k_cont==5):
    data_dir = '/content/drive/My Drive/Base5' #dirección de la carpeta de base de datos 
    save_model=name_model +' model_It5.pth'

  if(k_cont==6):
    data_dir = '/content/drive/My Drive/Base6' #dirección de la carpeta de base de datos
    save_model=name_model +' model_It6.pth'

  if(k_cont==7):
    data_dir = '/content/drive/My Drive/Base7' #dirección de la carpeta de base de datos 
    save_model=name_model +' model_It7.pth'

  if(k_cont==8):
    data_dir = '/content/drive/My Drive/Base8' #dirección de la carpeta de base de datos 
    save_model=name_model +' model_It8.pth'
  
  if(k_cont==9):
    data_dir = '/content/drive/My Drive/Base9' #dirección de la carpeta de base de datos 
    save_model=name_model +' model_It9.pth'

  if(k_cont==10):
    data_dir = '/content/drive/My Drive/Base10' #dirección de la carpeta de base de datos 
    save_model=name_model +' model_It10.pth'
  
  image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),  #carga las imagenes de train y val
                                            data_transforms[x])         #de la base de datos y les aplica las transformaciones
                    for x in ['train', 'val']}
  
  print(data_dir) 
  #batch_size = número de imagenes por lote
  #shuffle = true , reorganizacion de datos en cada epoca  

                
  dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, #reorganiza las imagenes para cada iteracion 
                                              shuffle=True, num_workers=4)
                for x in ['train', 'val']}

  dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}  #tamaño del conjunto de datos 
  class_names = image_datasets['train'].classes  #clases de la base de datos 
  #print(class_names)

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #identifica si el codigo se ba a correr en GPU  o CPU
  #print(device)

  class_to_idx = image_datasets['train'].class_to_idx
  print(class_to_idx)

  print ("Dataset Size: "+ str(dataset_sizes) + "\n")
  n_class = len(class_names)
  print ("Number of classes: "+ str(n_class) + "\n")
  print ("Classes: "+ str(class_names) + "\n")
  print (len(dataloaders))

  ###############################FUNCIÓN DE ENTRENAMIENTO###############################

  def train_model(model, criterion, optimizer, scheduler, num_epochs):
      since = time.time()

      yy_train=[]
      xx_train=[]
      yy_val=[]
      xx_val=[]
      sen_train=[]
      sen_val=[]
      esp_train=[]
      esp_val=[]
      prec_train=[]
      prec_val=[]
      fpr_train=[]
      fpr_val=[]
      acc_train=[]
      acc_val=[]
      predict=torch.cuda.FloatTensor(0)
      best_predict=torch.cuda.FloatTensor(0)
      labels_real=torch.cuda.FloatTensor(0)
      best_labels=torch.cuda.FloatTensor(0)
      prob=torch.cuda.FloatTensor(0)
      best_prob=torch.cuda.FloatTensor(0)    
      cont_epoch = 0
      epoch_min = 0
      #Inicialización en cero de variables Verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos
      VP=0
      FP=0
      VN=0
      FN=0
      #archivo = open("/content/drive/My Drive/Datos_Resnet.txt", 'w')
      #archivo.write("Resnet18\n")
      best_model_wts = copy.deepcopy(model.state_dict())
      best_acc = 0.0
      acc_aux = 0.0
      best_sen = 0.0
      specificity = 0.0
      precision= 0.0

      for epoch in range(num_epochs):
          print('Epoch {}/{}'.format(epoch, num_epochs - 1))
          print('-' * 10)

          # Cada época tiene una fase de entrenamiento y validación
          for phase in ['train', 'val']:
              if phase == 'train':
                  #scheduler.step()
                  model.train()  # Set model to training mode  # Poner el modelo en modo de entrenamiento
              else:
                  model.eval()   # Set model to evaluate mode

              running_loss = 0.0
              running_corrects = 0

              # Iterate over data. # Iterar sobre los datos.
            
              predict=torch.cuda.FloatTensor(0)
              labels_real=torch.cuda.FloatTensor(0)
              prob=torch.cuda.FloatTensor(0)
              for inputs, labels in dataloaders[phase]:
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  # zero the parameter gradients
                  optimizer.zero_grad()

                  # forward
                  # track history if only in train
                  with torch.set_grad_enabled(phase == 'train'):
                      outputs = model(inputs)

                      sm = torch.nn.Softmax(dim=1)
                      probabilities = sm(outputs)
                      probabilities = probabilities[:,1]
                      valores, preds = torch.max(outputs, 1)
                      #print(type(phase))
                      loss = criterion(outputs, labels)

                      if phase == 'val':
                        prob = torch.cat((prob,probabilities),0)
                        predict=torch.cat((predict.float(),preds.float()),0)
                        labels_real=torch.cat((labels_real.float(),labels.data.float()),0)


                      # backward + optimize only if in training phase
                      if phase == 'train':
                          loss.backward()
                          optimizer.step()

                  # statistics
                  running_loss += loss.item() * inputs.size(0)
                  running_corrects += torch.sum(preds == labels.data)
                  
                  #Prueba Verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos

                  for i in range (len(labels.data)):
                    if (labels.data[i]== preds[i]):
                    #print(i)
                      if (labels.data[i]== 1):
                        VP+=1;
                      else:
                        VN+=1;
                    else:
                      if(labels.data[i]== 1):
                        FP+=1;
                      else:
                        FN+=1;

              #px1=1-especificidad
              epoch_loss = running_loss / dataset_sizes[phase]
              epoch_acc = running_corrects.double() / dataset_sizes[phase]
              
              if acc_aux<epoch_acc:
                acc_aux=epoch_acc
              
              #33333 Condicional durante entrenamiento
              if phase == 'train':
                scheduler.step()

                acc_train.append(epoch_acc)
                sen_train.append(VP/(VP+FN))
                esp_train.append(VN/(VN+FP))
                prec_train.append(VP/(VP+FP))
                fpr_train.append(1- esp_train[epoch])
                print("VP= " + str(VP) + " VN= " + str(VN) + " FP= " + str(FP) + " FN= " + str(FN))
                print('Sensibilidad: {:.4f} Especificidad: {:.4f} Precisión: {:.4f}'.format(sen_train[epoch], esp_train[epoch], prec_train[epoch]))
                yy_train.append(epoch_loss)
                xx_train.append(epoch)                  
                writer.add_scalars("Loss", {"Loss/train": epoch_loss}, epoch)  
                  #archivo.close() 

              ###### Condicional durante validación
              if phase == 'val':
                
                #Calcula sensibilidad y especificidad en validación
                acc_val.append(epoch_acc)
                sen_val.append(VP/(VP+FN))
                esp_val.append(VN/(VN+FP))
                prec_val.append(VP/(VP+FP))
                fpr_val.append(1- esp_train[epoch])
                print("VP= " + str(VP) + " VN= " + str(VN) + " FP= " + str(FP) + " FN= " + str(FN))
                print('Sensibilidad: {:.4f} Especificidad: {:.4f} Precisión: {:.4f}'.format(sen_val[epoch], esp_val[epoch], prec_val[epoch]))
                if epoch == 0:
                  stop_val=10

                yy_val.append(epoch_loss)
                xx_val.append(epoch)
                writer.add_scalars("Loss", {"Loss/val":epoch_loss}, epoch)             

                if epoch_loss>stop_val:
                  cont_epoch+=1
                  print("cont= "+str(cont_epoch))
                  
                else:
                  if cont_epoch < 30:
                    dif_val = stop_val-epoch_loss
                    porc_val_back=0.1*stop_val
                    if dif_val>porc_val_back:
                      cont_epoch=0
                      stop_val=epoch_loss 
                      epoch_min=epoch
                      print("perdida disminuyendo ",epoch_min)
                      best_sen=sen_val[epoch]
                      best_acc = epoch_acc
                      specificity = esp_val[epoch]
                      precision = prec_val[epoch] 
                      best_model_wts = copy.deepcopy(model.state_dict())
                      best_predict=predict
                      best_labels=labels_real
                      best_prob=prob                     
                    else:
                      cont_epoch+=1
                      print("cont= "+str(cont_epoch))
                    
              VP=0
              FP=0
              VN=0
              FN=0
              print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                  phase, epoch_loss, epoch_acc))


      #print((predict))    
      time_elapsed = time.time() - since
      print('Training complete in {:.0f}m {:.0f}s'.format(
          time_elapsed // 60, time_elapsed % 60))
      print('Best val Acc: {:4f} Sen: {:.4f} Esp: {:.4f}'.format(best_acc, best_sen, specificity))

      # load best model weights
      model.load_state_dict(best_model_wts)
      #writer.flush()
      writer.close()
      return model, yy_train, xx_train, yy_val, xx_val, epoch_min, stop_val, acc_train, acc_val, best_predict, best_labels, best_prob, best_acc, best_sen, specificity, precision, time_elapsed, acc_aux #retorna el mejor modelo 

      ############################### CARGA DE PARAMETROS Y MODELO PREENTRENADO ############################### 

  model_ft = models.vgg13_bn(pretrained=True)

  num_ftrs = model_ft.classifier[6].in_features
  model_ft.classifier[6] = nn.Linear(num_ftrs,2)

  if torch.cuda.is_available():
      model_ft.cuda() #.cuda() will move everything to the GPU side
      
  criterion = nn.CrossEntropyLoss()

  optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.005)
  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.05)

  ############################### ENTRENAMIENTO DEL MODELO ###############################

  model_ft, yy_train, xx_train, yy_val, xx_val, epoch_min, stop_val, acc_train, acc_val, best_predict, best_labels, best_prob, best_acc, best_sen, specificity, precision, time_train, acc_aux = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=25)

  ############################### IMPRESIÓN DE PARAMETROS ###############################

  print("Datos perdidas")
  print("xx_train = " + str(xx_train))
  print("yy_train = "+ str(yy_train))
  print("xx_val = "+ str(xx_val))
  print("yy_val = "+ str(yy_val))
  print("epoch_min = "+ str(epoch_min))
  print("stop_val = "+ str(stop_val))
  print("\n")

  print("Datos accuracy")

  for i in range(len(acc_val)):
    acc_val[i]=acc_val[i].tolist()
    acc_train[i]=acc_train[i].tolist()
  print("acc_train = " + str(acc_train))
  #print("xx_train = " + str(xx_train))
  print("acc_val = "+ str(acc_val))
  #print("xx_val = "+ str(xx_val))
  #print(len(best_prob))

  print("\nDatos curva ROC")
  best_labels=best_labels.tolist()
  best_prob=best_prob.tolist()
  print("best_labels = "+ str(best_labels))
  print("best_prob = "+ str(best_prob))

  print("\nMetricas de validación")
  best_acc=best_acc.tolist()
  acc_aux=acc_aux.tolist()
  print("Best_acc = "+ str(best_acc))
  print("acc_aux = "+ str(acc_aux))
  print("Recall = "+ str(best_sen))
  print("Precisión = "+ str(precision))
  print("Especificidad = "+ str(specificity))
  print('Tiempo = {:.0f}s'.format(time_train))




  ############################### GUARDADO DE PARAMETROS EN TXT ###############################
  archivo.write("\n")
  archivo.write("\n######## Base" + str(k_cont) + " ########")
  archivo.write("\n#MODELO = " + name_model )
  archivo.write("\n#Datos perdidas")
  archivo.write("\nxx_train" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(xx_train))
  archivo.write("\nyy_train" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(yy_train))
  archivo.write("\nxx_val" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(xx_val))
  archivo.write("\nyy_val" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(yy_val))
  archivo.write("\nepoch_min" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(epoch_min))
  archivo.write("\nstop_val" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(stop_val))
  archivo.write("\n")

  archivo.write("\n#Datos accuracy")
  archivo.write("\nacc_train" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(acc_train))
  archivo.write("\nacc_val" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(acc_val))
  archivo.write("\n")

  archivo.write("\n#Datos curva ROC")
  archivo.write("\nbest_labels" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(best_labels))
  archivo.write("\nbest_prob" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(best_prob))
  archivo.write("\n")

  archivo.write("\n#Metricas de validación")
  archivo.write("\nBest_acc" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(best_acc))
  archivo.write("\nacc_aux" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(acc_aux))
  archivo.write("\nRecall" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(best_sen))
  archivo.write("\nPrecisión" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(precision))
  archivo.write("\nEspecificidad" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(specificity))
  archivo.write("\nTiempo" + str(k_cont)  + "_"  + str(mod_cont) + " = " + str(time_train))


  ############################### GUARDADO DE MODELO ###############################

  #print(torch.__version__)

  model_ft.class_to_idx = image_datasets['train'].class_to_idx
  model_ft.cpu()
  torch.save({'arch': name_model,
              'state_dict': model_ft.state_dict(), 
              'class_to_idx': model_ft.class_to_idx}, 
              save_model)
  k_cont+=1

In [ ]:
archivo.close() 

**DATOS GUARDADOS**

In [ ]:

###################################################################

######## Base1 ########
#MODELO = VGG11
#Datos perdidas
xx_train1_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train1_1 = [0.1878362270417037, 0.09336583367836328, 0.07970881807031455, 0.07198531228451081, 0.07350429083094184, 0.054108509402952074, 0.048146076141684147, 0.05093820185205083, 0.04124741548741305, 0.042688725732358886, 0.040636898650799266, 0.04404540061950683, 0.04741571754401113, 0.04408280387704755, 0.039243449445491956, 0.038272018473089475, 0.03708515089051223, 0.04804131531237084, 0.04590075658059414, 0.0388596888585591, 0.04439934057953917, 0.0475093251761095, 0.04246840445715704, 0.03888875407937132, 0.04006822786213439]
xx_val1_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val1_1 = [0.10909084743923611, 0.05133866137928433, 0.04672201093700197, 0.45418066788050865, 0.05933474020825492, 0.0483840243683921, 0.04717734307050705, 0.04583940497703022, 0.044392519030306075, 0.04397523750861486, 0.04396187894874149, 0.043966003590159945, 0.043973942514922884, 0.04392910309963756, 0.043913216557767656, 0.04391387858324581, 0.04391487199399206, 0.04391281447476811, 0.043911229074001315, 0.04390780031681061, 0.0439077529642317, 0.04390760287642479, 0.043907512393262654, 0.04390745187799136, 0.043907475305928124]
epoch_min1_1 = 1
stop_val1_1 = 0.05133866137928433

#Datos accuracy
acc_train1_1 = [0.9262345679012346, 0.9641975308641975, 0.9712962962962963, 0.9734567901234568, 0.9731481481481481, 0.9808641975308642, 0.983641975308642, 0.9817901234567901, 0.9848765432098765, 0.9845679012345678, 0.9854938271604938, 0.9830246913580246, 0.9824074074074074, 0.983641975308642, 0.986111111111111, 0.9848765432098765, 0.9873456790123457, 0.9827160493827161, 0.983641975308642, 0.9854938271604938, 0.9848765432098765, 0.9817901234567901, 0.983641975308642, 0.986111111111111, 0.986111111111111]
acc_val1_1 = [0.9694444444444444, 0.9805555555555556, 0.9833333333333334, 0.8916666666666667, 0.9777777777777779, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779]

#Datos curva ROC
best_labels1_1 = [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]
best_prob1_1 = [0.004291684832423925, 0.9933885335922241, 0.006430231500416994, 0.021692637354135513, 0.9857489466667175, 0.0050787110812962055, 0.9990038275718689, 0.10026199370622635, 0.9999574422836304, 0.9999567270278931, 0.00048192913527600467, 0.9999916553497314, 0.00017389978165738285, 0.00867031142115593, 0.9983381032943726, 0.017562100663781166, 2.6786901798914187e-05, 0.99174964427948, 0.0025749539490789175, 0.9999151229858398, 0.9999992847442627, 0.7524816393852234, 0.0023547469172626734, 0.007149866782128811, 4.5125078031560406e-05, 0.01653279736638069, 0.00011986812751274556, 0.9687697291374207, 0.024711808189749718, 0.9996767044067383, 0.9997689127922058, 0.0008039189269766212, 0.0002172985696233809, 0.0004702308215200901, 0.9999984502792358, 0.9971393346786499, 0.6957339644432068, 0.9999665021896362, 0.999423623085022, 0.004014939069747925, 0.9999889135360718, 0.0017725012730807066, 0.9999998807907104, 7.810031092958525e-05, 0.9998717308044434, 0.9994019269943237, 0.0019266944145783782, 0.003630717983469367, 0.9996576309204102, 0.9999933242797852, 0.0012382977874949574, 0.9998136162757874, 0.9999997615814209, 0.9999098777770996, 1.0, 0.010834329761564732, 0.9767045974731445, 0.04520099610090256, 0.004169066436588764, 1.0, 1.0, 0.0028744859155267477, 0.99980229139328, 0.008245669305324554, 0.0005185758345760405, 0.006528911180794239, 0.00461115688085556, 0.001236730138771236, 0.8886697888374329, 0.9999948740005493, 0.0037139691412448883, 5.118141052662395e-05, 0.0035075745545327663, 0.0003437912673689425, 0.00037285228609107435, 0.2243661880493164, 0.000434684770880267, 0.016204005107283592, 0.00037209014408290386, 0.0018177068559452891, 0.018596315756440163, 0.9999996423721313, 0.010036579333245754, 0.9709237217903137, 0.999761164188385, 0.00019465290824882686, 4.220718255965039e-05, 0.9999594688415527, 0.000269395939540118, 0.00038606434827670455, 1.3724484233534895e-06, 1.0, 0.000219860696233809, 0.9999024868011475, 1.5388557585538365e-05, 0.01388588361442089, 0.9999819993972778, 0.9999998807907104, 0.00037285228609107435, 5.722223431803286e-05, 0.006874840706586838, 0.10228896141052246, 0.004827114753425121, 0.9999573230743408, 0.003880312666296959, 1.0, 0.9738924503326416, 0.9856266379356384, 0.9999786615371704, 0.006277676206082106, 0.008901071734726429, 0.009477697312831879, 0.9999821186065674, 0.9901444911956787, 0.9996874332427979, 0.9981710910797119, 0.9999005794525146, 0.8621259331703186, 0.9999873638153076, 0.0013275537639856339, 0.0001752138341544196, 0.0055053457617759705, 0.9902679324150085, 0.9999866485595703, 0.9999850988388062, 0.0007568049477413297, 0.9986566305160522, 0.0009734138729982078, 0.9999926090240479, 0.1744455248117447, 0.9996905326843262, 0.00600780826061964, 0.00026721745962277055, 0.9999997615814209, 0.9999997615814209, 0.00022742424334865063, 0.9999856948852539, 2.3402921215165406e-05, 6.865816976642236e-05, 0.9997723698616028, 0.2154594510793686, 0.9999994039535522, 0.00031493717688135803, 0.27247437834739685, 0.0012253770837560296, 0.9992700219154358, 0.9986833930015564, 0.0008396550547331572, 4.806498964171624e-06, 0.9978513717651367, 1.0, 0.0701771229505539, 0.01645178720355034, 0.9999998807907104, 0.061734169721603394, 0.818513035774231, 8.625171903986484e-05, 0.0003816021198872477, 0.9911527037620544, 0.9998569488525391, 0.0008902004337869585, 0.00017714535351842642, 0.9999973773956299, 0.9998366832733154, 0.5014757513999939, 0.000569716386962682, 0.9999929666519165, 0.00763331912457943, 0.9999794960021973, 0.9999909400939941, 0.9994460940361023, 0.0033144792541861534, 0.0015682695666328073, 0.9939822554588318, 0.9976024031639099, 0.7255277633666992, 0.013965372927486897, 0.005563671234995127, 0.00022557629563380033, 0.0019272096687927842, 0.9912115931510925, 0.9967305660247803, 0.08193813264369965, 0.9999748468399048, 0.9999996423721313, 0.9999563694000244, 0.06484784930944443, 0.9999897480010986, 0.9999972581863403, 0.9999995231628418, 0.9999477863311768, 0.0002587987401057035, 0.00011607198393903673, 0.999929666519165, 0.9997585415840149, 0.0004291826917324215, 0.9999998807907104, 0.9999576807022095, 0.9996362924575806, 0.998615026473999, 0.9997201561927795, 0.0001547857973491773, 0.03983514755964279, 0.9997283816337585, 0.001220966107212007, 0.003614949993789196, 0.00018724153051152825, 0.9979472756385803, 0.9941492080688477, 0.9997679591178894, 0.01468848530203104, 0.17759296298027039, 0.9999164342880249, 0.9999943971633911, 0.0069909365847706795, 0.984332799911499, 1.0, 0.9999679327011108, 0.00034685866557992995, 0.9959883093833923, 1.0, 0.9999877214431763, 0.9989029169082642, 0.025426072999835014, 0.0004173634806647897, 0.9997314810752869, 0.9901949763298035, 8.41960427351296e-05, 0.027973206713795662, 0.008569499477744102, 0.00284743495285511, 0.0032821621280163527, 1.0, 0.43861863017082214, 0.1108216643333435, 0.9999749660491943, 0.999982476234436, 0.04698267951607704, 0.9999982118606567, 0.9087396264076233, 0.9982752799987793, 0.9999927282333374, 0.002466056263074279, 0.9997106194496155, 0.012455569580197334, 0.00385463098064065, 1.0, 0.00010438399476697668, 0.3097555637359619, 0.004986554849892855, 0.9998565912246704, 0.001510322792455554, 0.010953552089631557, 0.2101014256477356, 0.02565830387175083, 0.9999983310699463, 1.0, 0.9999905824661255, 0.043696362525224686, 0.005166495684534311, 0.00042841362301260233, 0.9901425838470459, 0.015218004584312439, 0.0065005021169781685, 0.9999589920043945, 0.9999960660934448, 1.0, 0.9511755108833313, 0.0021841730922460556, 0.9999319314956665, 0.00018760950479190797, 0.009227653965353966, 0.013175408355891705, 0.9976536631584167, 0.0004257232358213514, 0.024608595296740532, 0.9999998807907104, 0.999915599822998, 0.9999417066574097, 0.008840098045766354, 0.003417029045522213, 0.9999988079071045, 0.976321816444397, 0.999539852142334, 0.9999957084655762, 0.9999812841415405, 1.0, 0.004792418796569109, 0.9990745782852173, 0.9685648083686829, 0.0002203143812948838, 0.00015040740254335105, 1.0, 0.0007887979154475033, 0.00016887288074940443, 1.0, 0.9999984502792358, 0.9999998807907104, 2.5211662432411686e-05, 0.9647163152694702, 0.6232188940048218, 0.9997370839118958, 0.9988635778427124, 0.9999698400497437, 0.0006073562544770539, 0.9858230352401733, 0.9996500015258789, 0.0013913811417296529, 0.9994575381278992, 0.9999927282333374, 0.006171445827931166, 0.0002915404038503766, 0.9999923706054688, 0.9999966621398926, 0.0004395226715132594, 0.28708416223526, 0.9999995231628418, 0.9999905824661255, 7.83045106800273e-06, 0.0019258087268099189, 0.9999468326568604, 0.0010444248327985406, 0.9996024966239929, 2.754025990725495e-05, 0.0012988044181838632, 0.002919387072324753, 0.012464669533073902, 0.9999797344207764, 0.0022541217040270567, 0.03804831951856613, 0.127248153090477, 0.9999805688858032, 1.0, 0.9909425377845764, 0.9994540810585022, 0.9999998807907104, 0.982887864112854, 0.005845718551427126, 0.0009867976186797023, 0.6107458472251892, 0.9998528957366943, 0.9999998807907104, 0.9980043768882751, 0.9957428574562073, 1.0, 0.11859782040119171, 0.000586847891099751, 0.009552296251058578, 0.0019930503331124783, 0.014887945726513863, 0.012903072871267796, 0.9939365983009338, 0.9999552965164185, 0.9999560117721558, 0.004599819891154766, 5.849852823303081e-05, 0.0031944122165441513, 0.0002207997313234955, 0.9988296627998352, 0.9998639822006226]

#Metricas de validación
Best_acc1_1 = 0.9805555555555556
acc_aux1_1 = 0.9873456790123457
Recall1_1 = 0.9726775956284153
Especificidad1_1 = 0.9887005649717514
Tiempo1_1 = 850.619968175888

######## Base2 ########
#MODELO = VGG11
#Datos perdidas
xx_train2_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train2_1 = [0.10779881512309299, 0.07226002308321589, 0.06372362405613617, 0.06062134492360515, 0.05664354921491058, 0.06496231886155811, 0.0435882368573436, 0.04212542282946316, 0.0452328849353908, 0.045392514121385265, 0.03295279013705842, 0.042300752421588074, 0.042395097927914724, 0.04032427239933132, 0.03886805840848405, 0.042498293022314705, 0.0430470237761368, 0.039439370152023104, 0.03956701466698705, 0.03641573176891715, 0.041554879811075, 0.036577420755300995, 0.03668302868803342, 0.034589418962045954, 0.03423592622081439]
xx_val2_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val2_1 = [0.07402369164758259, 0.07754721111721463, 0.06798235724369685, 0.06498941654960315, 0.1441781280769242, 0.0742945028675927, 0.07711672807733218, 0.07760144819815953, 0.07621375636921988, 0.07814758337206311, 0.07820763091246287, 0.07786328825685714, 0.07805490295092264, 0.07795857025517358, 0.0781272636519538, 0.07812503063016468, 0.07813321070538627, 0.078139272166623, 0.07813055432505078, 0.07812585681676865, 0.07812569323513242, 0.07812564588255352, 0.07812541094091204, 0.07812541259659661, 0.07812517219119602]
epoch_min2_1 = 3
stop_val2_1 = 0.06498941654960315

#Datos accuracy
acc_train2_1 = [0.9583333333333333, 0.975, 0.9774691358024691, 0.9796296296296296, 0.9796296296296296, 0.9725308641975309, 0.9848765432098765, 0.9854938271604938, 0.9839506172839506, 0.9845679012345678, 0.9882716049382716, 0.9851851851851852, 0.986111111111111, 0.9854938271604938, 0.9858024691358025, 0.9824074074074074, 0.983641975308642, 0.9867283950617284, 0.987037037037037, 0.9876543209876543, 0.9851851851851852, 0.9854938271604938, 0.9885802469135803, 0.9891975308641975, 0.9888888888888888]
acc_val2_1 = [0.9694444444444444, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9333333333333333, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001, 0.9750000000000001]

#Datos curva ROC
best_labels2_1 = [0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
best_prob2_1 = [0.002056501340121031, 0.0022970528807491064, 1.0, 0.08417576551437378, 0.9999988079071045, 0.9999980926513672, 0.9917689561843872, 0.000561133143492043, 1.0, 0.9999927282333374, 0.009606153704226017, 0.08032989501953125, 0.9998358488082886, 0.018670767545700073, 0.9999948740005493, 1.0, 0.011127631179988384, 0.9953112006187439, 0.0013175412314012647, 1.0, 0.0033799821976572275, 0.1265285462141037, 0.9999997615814209, 0.0077632940374314785, 0.9998694658279419, 0.9999642372131348, 0.9999983310699463, 0.9999983310699463, 0.9999985694885254, 0.9999881982803345, 0.00020448837312869728, 0.001264033024199307, 0.9999998807907104, 0.9999994039535522, 0.9999780654907227, 0.004186257719993591, 0.0066817705519497395, 0.0005067344754934311, 0.9999703168869019, 0.9999514818191528, 0.999983549118042, 0.2719353437423706, 0.004533617291599512, 0.9999990463256836, 0.0059722112491726875, 0.9999816417694092, 0.9953473210334778, 0.0043333531357347965, 0.006631179712712765, 0.00023365256492979825, 0.1203533262014389, 0.999981164932251, 0.006066534668207169, 0.0004671153728850186, 0.0017431230517104268, 0.9998493194580078, 1.0, 0.0009095537243410945, 0.07042043656110764, 1.0, 0.008204442448914051, 0.11767800897359848, 0.9390391707420349, 0.9999880790710449, 0.9999996423721313, 0.999997615814209, 0.016440626233816147, 1.0, 0.0010231054620817304, 0.9996322393417358, 0.0007482703658752143, 0.9962524771690369, 0.9999279975891113, 0.19688446819782257, 0.9999649524688721, 0.9997209906578064, 0.9999997615814209, 1.0, 0.9833868145942688, 0.9993283748626709, 0.0280314851552248, 0.9999808073043823, 1.0, 1.0, 0.03272553160786629, 0.045512471348047256, 0.9999983310699463, 1.0, 0.04044758528470993, 0.025556471198797226, 0.8945071697235107, 0.001135210390202701, 0.0004217961977701634, 1.0, 0.0025635198689997196, 0.9995805621147156, 0.9990782737731934, 0.16588056087493896, 0.0031734532676637173, 0.999951958656311, 0.011867818422615528, 0.9994099140167236, 0.0020592587534338236, 1.0, 1.0, 0.013095633126795292, 5.405632691690698e-05, 0.9999936819076538, 0.0034779629204422235, 0.9977591037750244, 0.015587573871016502, 0.003011963563039899, 0.998871386051178, 0.0024798456579446793, 1.0, 1.0, 0.9999985694885254, 0.0008740374469198287, 0.0007060549105517566, 0.9999985694885254, 1.0, 0.07590682804584503, 0.0022567228879779577, 0.020680438727140427, 0.06066499650478363, 0.0015790233155712485, 0.46459999680519104, 0.9999986886978149, 0.005178739782422781, 0.9999997615814209, 1.0, 1.0, 0.01926281303167343, 0.999872088432312, 0.013857508078217506, 0.0022009769454598427, 0.010401299223303795, 0.9999957084655762, 0.0015813735080882907, 1.0, 1.0, 0.9990142583847046, 0.00032435357570648193, 0.0026711432728916407, 0.9999961853027344, 0.014040696434676647, 0.9999798536300659, 0.022579044103622437, 0.998403012752533, 0.001131897559389472, 0.004604490473866463, 0.00771840987727046, 0.9998218417167664, 0.0014866265701130033, 0.9999979734420776, 1.0, 0.9999921321868896, 0.10465781390666962, 0.999863862991333, 0.005106192082166672, 0.9999262094497681, 0.0018294149776920676, 0.01697363145649433, 0.015612654387950897, 1.0, 0.9999970197677612, 0.0030019443947821856, 0.091545470058918, 0.9999858140945435, 0.00245240842923522, 0.9999996423721313, 0.0020673826802521944, 0.9999979734420776, 0.0021503230091184378, 1.0, 0.03296337649226189, 0.00022696198720950633, 0.03490195423364639, 0.9962130784988403, 0.999990701675415, 0.0006040593725629151, 6.5172316681128e-05, 0.9999986886978149, 0.0008651313837617636, 0.9999973773956299, 0.0017423591343685985, 0.009099012240767479, 0.9984760880470276, 0.0002833520120475441, 0.9999749660491943, 0.9913511276245117, 0.9999970197677612, 0.007543620653450489, 0.9998094439506531, 0.00150457548443228, 0.002695027505978942, 0.9999977350234985, 0.9999998807907104, 0.000777583452872932, 0.0127646429464221, 8.334307494806126e-05, 0.9999998807907104, 0.9999967813491821, 0.03773694112896919, 1.0, 0.9559075832366943, 1.0, 0.049770038574934006, 0.0011054022470489144, 0.999998927116394, 0.005337885580956936, 0.9868467450141907, 0.9999668598175049, 0.9999873638153076, 1.0, 0.001541393343359232, 0.052050359547138214, 0.004713337868452072, 0.005280527286231518, 0.9999897480010986, 0.00031168138957582414, 0.9992438554763794, 0.015645083039999008, 0.0026645075995475054, 0.000573859375435859, 0.0033882276620715857, 1.0, 0.8344895839691162, 0.9999942779541016, 0.002693816786631942, 0.0002042771375272423, 0.006457378156483173, 1.0, 1.0, 0.9999922513961792, 0.0018337623914703727, 0.9993098974227905, 0.02107834629714489, 0.9999996423721313, 0.20905953645706177, 0.9997579455375671, 0.8908393979072571, 0.9999973773956299, 0.1372731775045395, 0.005089316517114639, 0.01023948285728693, 0.0008097787504084408, 1.0, 0.9998613595962524, 0.004298526328057051, 0.9191831350326538, 1.0, 0.9999998807907104, 0.9999439716339111, 0.0028204701375216246, 0.6981041431427002, 1.0, 0.00033768374123610556, 1.0, 0.0018009851919487119, 0.0004923299420624971, 0.9999980926513672, 0.9999858140945435, 0.004542509559541941, 0.9999994039535522, 0.9999997615814209, 0.00022681854898110032, 0.35838818550109863, 0.5337297320365906, 0.9999955892562866, 5.5236938351299614e-05, 0.0013010547263547778, 0.9999772310256958, 0.006182281766086817, 1.0, 0.9997240900993347, 0.9999862909317017, 1.0, 0.0028452156111598015, 0.04961287975311279, 0.00037208161666058004, 0.9999983310699463, 0.9994953870773315, 0.9999868869781494, 0.9999874830245972, 0.03569253906607628, 0.9999998807907104, 0.4788840115070343, 0.9999804496765137, 0.20901590585708618, 0.9998871088027954, 0.0007878645556047559, 0.010520082898437977, 7.412188278976828e-05, 0.0028165876865386963, 0.0043623922392725945, 0.002565162954851985, 0.998809814453125, 0.005363811738789082, 0.0002167692146031186, 0.9999994039535522, 0.017276812344789505, 0.9999263286590576, 0.000813193793874234, 0.026346508413553238, 0.9998855590820312, 0.9964208602905273, 0.0023994753137230873, 3.333223139634356e-05, 0.9999735355377197, 0.9998788833618164, 0.0005969321355223656, 0.004474907647818327, 0.002897411584854126, 0.9997056126594543, 1.0, 0.00152898661326617, 0.9999997615814209, 0.012486313469707966, 0.9999996423721313, 1.0, 0.013422936201095581, 0.9999984502792358, 0.00020838437194470316, 0.9999918937683105, 0.0006171292043291032, 0.0021313061006367207, 0.9999220371246338, 0.9999995231628418, 0.01816282980144024, 0.9999951124191284, 0.9999512434005737, 0.9999991655349731, 0.0005373944295570254, 0.999974250793457, 0.0006767961313016713, 1.0, 1.0, 0.9972742199897766, 0.9999974966049194, 0.001368832541629672, 0.04502221196889877, 0.9999998807907104, 0.9999880790710449, 0.00803406536579132, 0.9999935626983643, 0.00018921752052847296, 0.9999985694885254, 0.9985872507095337, 1.0, 0.0012829583138227463, 0.005000490229576826, 0.009799789637327194, 0.0001490501017542556, 0.9989206790924072, 1.0, 0.012073970399796963, 0.0003500115417409688, 0.002081975806504488, 0.010228634811937809]

#Metricas de validación
Best_acc2_1 = 0.9750000000000001
acc_aux2_1 = 0.9891975308641975
Recall2_1 = 0.9672131147540983
Especificidad2_1 = 0.9830508474576272
Tiempo2_1 = 828.3318169116974

######## Base3 ########
#MODELO = VGG11
#Datos perdidas
xx_train3_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train3_1 = [0.0814465386999978, 0.06363319933966353, 0.052779948830972485, 0.05296403097885626, 0.04803087300172559, 0.036580307919670034, 0.03778483083034739, 0.03455797165264318, 0.032298869823600036, 0.0301653136625702, 0.03584576390775633, 0.029996812729923814, 0.027169301388440308, 0.025403143575897923, 0.03042920286088814, 0.0287720347168269, 0.03317872070603901, 0.028125722182017785, 0.03555087777383534, 0.03673946430653702, 0.03189007270115393, 0.038832148670414346, 0.025992594737513565, 0.03650760782169707, 0.037707899216516516]
xx_val3_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val3_1 = [0.023297661377324, 0.023774970405631594, 0.021978906790415444, 0.017003630929523044, 0.037931072215239205, 0.021213836471239724, 0.023604875140719943, 0.022117545041773053, 0.026080420944425795, 0.018103802618053225, 0.018454439441363015, 0.01841434207227495, 0.018511468006504907, 0.01842878626452552, 0.018488578663931953, 0.018502584927611882, 0.018506113522582582, 0.018517648345894283, 0.01853566649887297, 0.018550191819667817, 0.01854999777343538, 0.018550273527701697, 0.01855040126376682, 0.01855048156446881, 0.018550850119855668]
epoch_min3_1 = 3
stop_val3_1 = 0.017003630929523044

#Datos accuracy
acc_train3_1 = [0.9657407407407407, 0.9780864197530864, 0.9814814814814815, 0.9799382716049383, 0.9824074074074074, 0.9879629629629629, 0.9858024691358025, 0.987037037037037, 0.9891975308641975, 0.9888888888888888, 0.9882716049382716, 0.9876543209876543, 0.9882716049382716, 0.991358024691358, 0.9895061728395061, 0.9901234567901235, 0.9882716049382716, 0.9895061728395061, 0.986111111111111, 0.9873456790123457, 0.9888888888888888, 0.987037037037037, 0.9898148148148148, 0.9873456790123457, 0.986111111111111]
acc_val3_1 = [0.9916666666666667, 0.9916666666666667, 0.9888888888888889, 0.9888888888888889, 0.9861111111111112, 0.9916666666666667, 0.9861111111111112, 0.9861111111111112, 0.9833333333333334, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667]

#Datos curva ROC
best_labels3_1 = [0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0]
best_prob3_1 = [0.04407758265733719, 1.4191905393090565e-05, 7.957125490065664e-05, 0.9991287589073181, 8.18151020212099e-05, 0.9999966621398926, 0.00030964190955273807, 1.9453713321126997e-05, 0.000582648441195488, 0.9985135197639465, 1.1632502037173253e-06, 0.021509060636162758, 0.05552441254258156, 1.0, 0.9991993308067322, 1.0, 9.6054223831743e-05, 6.950500051061681e-07, 0.990575909614563, 7.405208452837542e-05, 3.714826846135111e-07, 1.0, 0.9999959468841553, 0.9999935626983643, 1.0, 0.9999914169311523, 9.899853466777131e-05, 0.00023543735733255744, 0.999998927116394, 1.491005036768911e-06, 0.9999988079071045, 0.997859537601471, 1.0, 0.00011638797877822071, 0.9994902610778809, 1.6223561033257283e-05, 0.10932286083698273, 0.9999324083328247, 1.0, 1.0, 2.033717646554578e-05, 8.758731564739719e-05, 0.9999995231628418, 0.06708184629678726, 1.0, 8.94607583177276e-05, 1.379299646941945e-05, 1.270114751150686e-07, 2.6007826818386093e-05, 1.1803629604401067e-05, 1.0, 0.9999803304672241, 0.9999896287918091, 0.0009721793467178941, 1.0, 0.9999994039535522, 2.6729743694886565e-05, 0.0033972046803683043, 2.5354012905154377e-05, 1.0, 0.999774158000946, 3.1341212434199406e-06, 0.9998990297317505, 1.0, 0.004793872125446796, 4.437592906469945e-07, 0.9999998807907104, 3.370080094100558e-06, 0.9999854564666748, 0.9999992847442627, 1.0, 0.9999785423278809, 0.007149633951485157, 3.904403456544969e-06, 0.0001468560949433595, 0.9983624815940857, 0.9998763799667358, 0.0024381366092711687, 2.6186748982581776e-06, 0.9937722086906433, 0.9999890327453613, 8.500185685988981e-06, 0.9999998807907104, 1.1016628377547022e-05, 0.9999747276306152, 6.420994509426237e-07, 2.632966470628162e-06, 1.0, 0.999907374382019, 0.9999990463256836, 0.9999990463256836, 0.00010595061758067459, 0.9999994039535522, 0.018773192539811134, 0.002968996297568083, 0.9999463558197021, 1.0, 0.9983880519866943, 1.0307270713383332e-05, 0.0003111466357950121, 9.875321120489389e-05, 0.9998308420181274, 0.529169499874115, 4.30465915997047e-05, 4.904688012175029e-06, 0.9999392032623291, 4.9744554416975006e-05, 0.9999971389770508, 0.008096995763480663, 0.9999829530715942, 0.7399236559867859, 0.0004744558536913246, 3.423615271458402e-05, 0.9999978542327881, 1.0, 2.646019936491939e-08, 0.9999988079071045, 0.9999446868896484, 1.0, 0.9999059438705444, 0.010435185395181179, 0.00014791432477068156, 3.6306346373748966e-08, 0.9999889135360718, 0.9999996423721313, 0.9999997615814209, 3.2797036055853823e-06, 6.330528776743449e-06, 1.0, 0.9991835951805115, 4.647423338610679e-06, 0.9999982118606567, 0.00017664993356447667, 0.04917861148715019, 0.00043937694863416255, 0.9625530242919922, 0.999648928642273, 1.8281330085301306e-06, 1.0, 0.9999911785125732, 1.0, 0.9999969005584717, 0.004028747323900461, 0.9997243285179138, 0.02338271774351597, 0.999776303768158, 3.526338332449086e-05, 0.0012206161627545953, 0.9999984502792358, 0.9999804496765137, 1.0, 0.002138417912647128, 0.0001746553898556158, 0.9999480247497559, 0.0004046542744617909, 0.9999995231628418, 0.9999864101409912, 0.004456037189811468, 0.9999998807907104, 9.472650930320015e-08, 1.0, 0.9999982118606567, 7.041047069833439e-07, 0.000146757549373433, 7.275561983988155e-06, 0.9999996423721313, 0.999935507774353, 0.9999996423721313, 8.617138519184664e-06, 0.9999998807907104, 7.4349100032122806e-06, 0.999996542930603, 1.0, 1.843035533966031e-05, 1.0, 3.5805335301120067e-06, 0.9998781681060791, 0.0005195294506847858, 0.9924522042274475, 1.6972752518995549e-06, 5.1601425354874664e-08, 1.0, 1.0, 0.9993081092834473, 1.0, 7.288423148565926e-06, 0.9999932050704956, 0.999916672706604, 0.004252839833498001, 9.71802364801988e-07, 0.9999821186065674, 0.9998317956924438, 0.9696940779685974, 0.9999874830245972, 1.8217614297100226e-06, 0.00012636705650947988, 0.9993182420730591, 9.392538777319714e-05, 8.492220331390854e-06, 0.001326926751062274, 0.9999971389770508, 0.9999449253082275, 0.9994506239891052, 1.0, 0.9999971389770508, 0.0002645451168064028, 0.07327466458082199, 8.86560337676201e-06, 0.00040622326196171343, 0.0472746416926384, 0.9999991655349731, 3.5081129112768394e-07, 0.008808553218841553, 0.8305491805076599, 1.0, 0.007752124685794115, 0.999998927116394, 0.9999163150787354, 0.9999465942382812, 8.220604649977759e-05, 0.9999974966049194, 0.00019175482157152146, 3.08906237478368e-05, 1.0, 1.0, 0.016963787376880646, 0.9999926090240479, 0.9999550580978394, 0.9995405673980713, 0.0010922008659690619, 0.9993958473205566, 9.459774901188212e-07, 1.4382326298800763e-05, 1.1375546819181181e-05, 0.0004907436668872833, 0.0005374766769818962, 7.683436706429347e-05, 0.9999953508377075, 0.9999973773956299, 0.9999953508377075, 0.00020965267322026193, 1.8877817637985572e-06, 1.0, 1.5147901649470441e-05, 3.0007277018739842e-05, 0.00013108279381413013, 0.0149718401953578, 6.35043704733107e-07, 0.9999773502349854, 0.9999980926513672, 4.0980370386023424e-07, 9.554392454447225e-05, 5.281072299112566e-05, 0.9999957084655762, 4.603594425134361e-05, 0.9999953508377075, 2.76810496870894e-05, 9.233344826498069e-06, 0.9999892711639404, 0.99997878074646, 0.9999998807907104, 0.9999097585678101, 0.005335165187716484, 4.5326764848141465e-06, 8.011190948309377e-05, 0.9999281167984009, 0.9999983310699463, 0.9998902082443237, 0.9999827146530151, 6.310011667665094e-05, 1.0, 1.0, 0.0035236997064203024, 0.00013258500257506967, 1.0, 0.9998140931129456, 0.00014191803347785026, 0.0005331109277904034, 2.1045982066425495e-05, 1.1621497833402827e-05, 0.11105427145957947, 2.3910595700726844e-05, 1.0, 0.9999964237213135, 2.4719605789869092e-05, 1.0, 0.001115292077884078, 1.4289302896486333e-07, 0.007876080460846424, 1.0479506045157905e-06, 0.004017510451376438, 5.605118076346116e-07, 0.9997839331626892, 5.2790986956097186e-05, 3.437290069996379e-05, 8.728596185392234e-06, 0.9999929666519165, 0.9999285936355591, 1.0, 0.00016152793250512332, 1.0, 1.0, 5.727832217417017e-07, 0.999962329864502, 0.9999803304672241, 0.00016853059059940279, 1.0, 0.9999995231628418, 0.0008036228828132153, 0.0007562821847386658, 2.4750831926212413e-06, 0.9999982118606567, 0.9999983310699463, 1.0, 8.16638646483625e-07, 1.6907881217775866e-05, 2.7715468604583293e-05, 0.9999926090240479, 0.9999974966049194, 0.9999990463256836, 9.32208786252886e-06, 0.9999908208847046, 1.0, 0.9999973773956299, 1.0, 1.0, 0.00013702837168239057, 1.552330650156364e-05, 6.2386698118643835e-06, 0.9999707937240601, 0.9999821186065674, 0.006066799163818359, 3.1042298360262066e-05, 1.9636114302556962e-05, 0.9999996423721313, 3.319327390727267e-07, 0.9999990463256836, 0.9999994039535522, 3.776019002543762e-05, 0.001251978799700737, 0.9504560232162476, 0.9999982118606567, 0.9999982118606567, 0.7026145458221436, 2.3555805455544032e-05, 1.0, 0.9995984435081482, 0.002687745029106736, 5.399423343988019e-07, 6.030001532053575e-05, 0.017909307032823563, 0.9999986886978149, 0.9999995231628418, 2.1554144495894434e-06, 2.326658659512759e-06, 0.9953770637512207, 0.9522039294242859, 0.9946135878562927, 1.0, 6.168958498165011e-05]

#Metricas de validación
Best_acc3_1 = 0.9888888888888889
acc_aux3_1 = 0.9916666666666667
Recall3_1 = 0.9782608695652174
Especificidad3_1 = 1.0
Tiempo3_1 = 837.4268851280212

######## Base4 ########
#MODELO = VGG11
#Datos perdidas
xx_train4_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train4_1 = [0.07113618455183358, 0.04016128127221708, 0.05335778104670254, 0.03501411795064255, 0.047144835893018745, 0.03394865232669277, 0.03315737221711947, 0.027458352301223777, 0.029099126987987094, 0.027647787461310257, 0.028217084549459412, 0.03632244690332884, 0.03203841320894383, 0.02931690478214511, 0.031151622708564922, 0.035798656388565345, 0.024668393394461384, 0.032838831161275325, 0.03770281802173014, 0.03166029004403102, 0.026575934022297094, 0.03212016751000911, 0.03163905227441847, 0.02788083885370949, 0.03414781945355144]
xx_val4_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val4_1 = [0.02011667291323344, 0.009706943233807882, 0.016425051954057482, 0.008012245843807856, 0.006286757853296068, 0.005685516364044614, 0.0058220899767345855, 0.005853468759192361, 0.0052536681294441225, 0.00532359199391471, 0.005291378001372019, 0.005293217301368713, 0.005278390314843919, 0.005270129028293821, 0.005256988604863485, 0.005258363319767846, 0.00525715665684806, 0.005256552331977421, 0.005256304475996229, 0.005256873534785377, 0.00525688264105055, 0.005256894396411048, 0.00525691525803672, 0.005256908800866869, 0.0052569227086173165]
epoch_min4_1 = 8
stop_val4_1 = 0.0052536681294441225

#Datos accuracy
acc_train4_1 = [0.9725308641975309, 0.9833333333333333, 0.9814814814814815, 0.9879629629629629, 0.9824074074074074, 0.9879629629629629, 0.9891975308641975, 0.9879629629629629, 0.990432098765432, 0.9907407407407407, 0.9891975308641975, 0.986111111111111, 0.9876543209876543, 0.9876543209876543, 0.9901234567901235, 0.9882716049382716, 0.991358024691358, 0.9895061728395061, 0.9876543209876543, 0.9882716049382716, 0.9888888888888888, 0.987037037037037, 0.9895061728395061, 0.991358024691358, 0.9854938271604938]
acc_val4_1 = [0.9916666666666667, 0.9972222222222222, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels4_1 = [1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0]
best_prob4_1 = [0.9999971389770508, 4.632567288354039e-05, 0.9999961853027344, 3.86290423648461e-07, 1.0, 1.0, 0.9999665021896362, 0.0018178349127992988, 1.0, 7.004453550507606e-08, 0.0003315051144454628, 0.0008638823055662215, 0.0001448405528208241, 1.0, 0.9999803304672241, 1.0, 1.8088739395238917e-08, 0.9995341300964355, 2.6740517569123767e-06, 3.9879137148091104e-07, 0.9998867511749268, 0.9999990463256836, 1.0, 0.9999988079071045, 0.9999902248382568, 0.0004046845715492964, 1.388509872413124e-06, 0.9999992847442627, 0.001455692807212472, 7.799997547408566e-05, 1.0, 1.0, 1.8570244719740003e-06, 1.1772982361435425e-05, 6.329390203063667e-07, 0.9542204141616821, 0.9999063014984131, 5.772795219627369e-08, 0.00016761533333919942, 0.9994674324989319, 0.9999790191650391, 9.046298146131448e-07, 1.0, 7.98051686956569e-08, 0.0006412513321265578, 1.0, 0.9999443292617798, 8.530999018319108e-09, 0.9999998807907104, 2.110221553053293e-09, 0.0001532904279883951, 1.0, 6.12689746049e-07, 1.0, 0.08267205953598022, 0.00011949665349675342, 0.9998623132705688, 1.0, 9.707365097710863e-05, 0.999818742275238, 1.375317879137583e-06, 1.257385520148091e-05, 0.001092424150556326, 0.9999994039535522, 0.9999990463256836, 0.9999998807907104, 1.9268572941655293e-06, 8.087201422313228e-06, 0.9999805688858032, 5.2025861805304885e-06, 2.68857093033148e-07, 0.0014643847243860364, 0.9999997615814209, 0.0022758175618946552, 4.229765636409866e-06, 0.9999357461929321, 6.8197732616681606e-06, 0.9999998807907104, 0.999995231628418, 0.9999998807907104, 1.0, 0.0007858718163333833, 1.0, 8.844581067535273e-09, 1.0228591236227658e-06, 0.7036199569702148, 0.0029716542921960354, 5.466861694003455e-06, 1.2753409464494325e-05, 2.292068757014931e-06, 0.0009707248536869884, 0.9999864101409912, 1.0, 4.593440507960622e-07, 1.359564635095012e-06, 0.9999986886978149, 0.9614529609680176, 0.9999990463256836, 0.9997559189796448, 0.00018329246086068451, 1.0, 0.9999997615814209, 1.0, 0.9999737739562988, 0.11144793778657913, 0.8940371870994568, 0.0031921083573251963, 1.0, 0.00015858070401009172, 0.9999505281448364, 0.9999996423721313, 0.0005535109085030854, 9.83296376944054e-06, 0.999372661113739, 0.029051650315523148, 3.302563345641829e-05, 1.1889755313632122e-07, 0.0152752585709095, 0.999992847442627, 1.6270179514776828e-07, 0.9999997615814209, 1.0, 1.280535252590198e-05, 1.0, 0.9999990463256836, 1.0, 0.9999974966049194, 1.0, 4.252818897043653e-08, 1.0, 2.8504562578746118e-05, 1.0, 1.0, 0.9999834299087524, 4.607547089108266e-05, 9.689163562143221e-06, 0.9999958276748657, 1.0, 4.550264748104382e-06, 4.93280049340683e-06, 0.0005220939638093114, 1.5345016436185688e-05, 0.00018527766223996878, 0.999998927116394, 1.00665547009271e-08, 1.0, 6.853956438135356e-05, 0.9999991655349731, 1.6868942793735187e-06, 5.830491431879636e-07, 1.9928509118471993e-06, 0.9999656677246094, 1.0, 0.014922126196324825, 0.0005356431356631219, 1.5022555089672096e-05, 0.0005230153910815716, 0.9999980926513672, 0.9968223571777344, 0.0013352824607864022, 0.9999996423721313, 8.08407276053913e-05, 0.05535486713051796, 0.00011064873251598328, 1.0, 0.000542709487490356, 0.0007679444970563054, 2.976726420911291e-07, 0.9999971389770508, 0.9999998807907104, 1.0, 0.9999998807907104, 6.449679403885966e-07, 6.850056024632067e-07, 3.577543566279928e-06, 0.0009392670472152531, 5.5511409300379455e-06, 1.0, 0.9999505281448364, 1.1499083711896674e-06, 1.749654984450899e-05, 1.0, 6.3745906118128914e-06, 0.00021455729438457638, 1.0, 3.908127473550849e-05, 0.9999995231628418, 0.9999996423721313, 0.9999861717224121, 4.7921730583766475e-06, 0.9999992847442627, 0.9999738931655884, 0.9995595812797546, 1.0, 5.629413863061927e-06, 0.9993884563446045, 9.507241571782288e-08, 0.9999511241912842, 0.03695863112807274, 4.837493605691634e-08, 0.9999939203262329, 0.00016107944247778505, 1.0, 0.0012461873702704906, 0.999997615814209, 0.001038985326886177, 9.916833732859232e-06, 0.00011353938316460699, 1.0, 7.981427643244388e-07, 0.999997615814209, 0.9987599849700928, 1.0, 0.9998598098754883, 2.080053945974214e-06, 0.9999895095825195, 2.801858727252693e-07, 0.0005663963966071606, 1.0, 2.2308710434515433e-09, 1.0, 0.9999969005584717, 0.9999991655349731, 2.291952796440455e-06, 8.179763000271123e-08, 0.9974787831306458, 4.574494028020126e-08, 1.0274354281136766e-05, 1.9855508071486838e-05, 5.186994930794775e-11, 1.0, 3.1863646654528566e-06, 0.00031079709879122674, 0.9999966621398926, 0.9999998807907104, 1.0, 8.529010665370151e-05, 0.9999691247940063, 8.858807996148244e-05, 0.9999867677688599, 1.0, 7.363457825704245e-06, 4.750455957491795e-07, 4.174241439613979e-06, 0.9994978904724121, 0.9999585151672363, 1.4238047185699543e-07, 9.243434760719538e-05, 1.0, 1.0, 1.0, 0.9994683861732483, 2.7040070563089103e-05, 1.0, 0.9998112320899963, 2.446254484311794e-06, 1.0976146995744784e-06, 5.408316155808279e-06, 2.0926573540691606e-07, 0.983792781829834, 3.588288421951802e-08, 3.463173925410956e-07, 0.00029344644281081855, 9.850758942775428e-06, 0.999997615814209, 0.00012164194049546495, 0.9999710321426392, 1.0, 3.046676101803314e-05, 1.0, 1.0, 0.999312162399292, 0.9999693632125854, 0.9999688863754272, 5.558095139690522e-08, 1.0, 0.9999991655349731, 1.0188421128987102e-06, 2.3356815859187918e-07, 1.0, 1.0, 1.8399440193661576e-08, 1.0, 1.0, 9.0168890892528e-05, 4.7086541599128395e-06, 9.926541679305956e-06, 0.9999998807907104, 1.0, 4.199900558887748e-06, 0.9999988079071045, 0.21127967536449432, 0.641237735748291, 0.9959529638290405, 0.0017410358414053917, 0.9989870190620422, 0.9991662502288818, 1.0, 0.00939291249960661, 3.269318540333188e-07, 0.9999998807907104, 1.7536908671900164e-06, 9.752094774739817e-05, 1.8085289710256802e-08, 0.9999998807907104, 0.9999990463256836, 0.994869589805603, 0.9999974966049194, 0.9999992847442627, 5.348326885723509e-05, 1.8571538021205924e-05, 3.70468711707872e-07, 5.374672440439099e-08, 1.0, 0.988206684589386, 6.503427698589803e-07, 0.8210970759391785, 0.0006452204543165863, 0.9999998807907104, 1.0, 5.590613727690652e-05, 0.00013832858530804515, 1.5354910942733113e-07, 5.858476583853189e-07, 0.9971910119056702, 1.0, 0.9979863166809082, 1.0, 2.0138383405310378e-08, 1.7384889972049677e-08, 0.9999895095825195, 1.0, 1.0, 9.682974450697657e-06, 1.0, 0.001120070111937821, 0.9985842704772949, 1.0, 1.785753011063207e-05, 2.3757513645250583e-06, 1.0, 5.4974575505184475e-06, 5.749331467086449e-05, 0.0005628638900816441, 0.9999979734420776, 2.1881280787283686e-08, 5.647633471994595e-09, 1.2299617537792074e-06, 5.6695412240515e-06, 0.9999991655349731, 2.6399814032629365e-06, 1.0, 0.9998784065246582, 0.00027224337100051343, 4.149561391386669e-06, 0.9999997615814209, 0.9999997615814209, 1.0, 5.314302143233363e-06, 0.9997788071632385]

#Metricas de validación
Best_acc4_1 = 1.0
acc_aux4_1 = 1.0
Recall4_1 = 1.0
Especificidad4_1 = 1.0
Tiempo4_1 = 844.2523174285889

######## Base5 ########
#MODELO = VGG11
#Datos perdidas
xx_train5_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train5_1 = [0.05817592993010709, 0.043409071485569446, 0.05615353140934014, 0.036055352124902934, 0.04293343589445691, 0.03283385332719779, 0.0308529960612456, 0.032343147906624244, 0.027287478938146873, 0.025675135933690602, 0.0328663863333655, 0.027599720353329624, 0.02312879740823934, 0.02582346960718249, 0.025248363568091098, 0.029643632489957928, 0.02613195095349241, 0.026998332602742276, 0.026356491648856506, 0.023294615543182986, 0.027194872516908764, 0.026889009865713707, 0.026877276286666776, 0.025318309517554295, 0.025813595637863066]
xx_val5_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val5_1 = [0.005647908647855123, 0.22537591722276476, 0.022287473910384707, 0.006240617566638523, 0.011670480006270939, 0.009641111062632667, 0.007402954830063714, 0.005705324643188053, 0.00694273528125551, 0.003535394701692793, 0.003655466437339783, 0.00366938097609414, 0.0037115017573038736, 0.003740763829814063, 0.0036970168352127076, 0.003702553113301595, 0.0037038970324728224, 0.0037029748161633807, 0.0037063287364112007, 0.003709448046154446, 0.0037095270223087735, 0.0037096016936832006, 0.003709666099813249, 0.003709686961438921, 0.0037097377909554377]
epoch_min5_1 = 0
stop_val5_1 = 0.005647908647855123

#Datos accuracy
acc_train5_1 = [0.9765432098765432, 0.9845679012345678, 0.979320987654321, 0.9864197530864197, 0.983641975308642, 0.9864197530864197, 0.9879629629629629, 0.9898148148148148, 0.9895061728395061, 0.9907407407407407, 0.9876543209876543, 0.990432098765432, 0.991358024691358, 0.9895061728395061, 0.9916666666666667, 0.9888888888888888, 0.9901234567901235, 0.9907407407407407, 0.9901234567901235, 0.9907407407407407, 0.9879629629629629, 0.9879629629629629, 0.9901234567901235, 0.9916666666666667, 0.9898148148148148]
acc_val5_1 = [1.0, 0.8916666666666667, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels5_1 = [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
best_prob5_1 = [0.9999151229858398, 0.9999982118606567, 0.993861198425293, 1.7399279386154376e-05, 0.0742710679769516, 0.019758354872465134, 0.00026508799055591226, 0.9630785584449768, 0.0019657688681036234, 0.09216056764125824, 0.0014791552675887942, 0.9999998807907104, 3.2940177334239706e-05, 0.9999902248382568, 0.001589560299180448, 0.9999868869781494, 0.0004203373100608587, 0.9999892711639404, 0.029399853199720383, 0.9999988079071045, 2.449994781272835e-06, 0.00039180490421131253, 0.999976634979248, 0.00022565203835256398, 0.9989930987358093, 1.0, 3.569803448044695e-05, 0.9999974966049194, 3.2708099752198905e-05, 0.0002939173427876085, 1.0076862054120284e-05, 0.00018988789815921336, 0.9965676069259644, 3.3644911923147447e-07, 0.9825385808944702, 0.9999313354492188, 0.9992861151695251, 0.0002461725380271673, 0.9981207251548767, 0.0005715668085031211, 0.0017587592592462897, 0.999946117401123, 0.00018660345813259482, 0.9999754428863525, 9.746909199748188e-07, 0.9999983310699463, 0.9999698400497437, 0.00068295729579404, 7.08406062699396e-08, 4.675654417951591e-05, 0.00017892591131385416, 2.728535832829948e-07, 0.9999654293060303, 0.9999922513961792, 0.999750554561615, 0.9999940395355225, 0.9958652257919312, 0.9992110729217529, 1.4876127885088408e-08, 1.899247945402749e-05, 0.9999500513076782, 3.874994581565261e-05, 0.9998834133148193, 0.9999926090240479, 1.668878030613996e-05, 3.2876343425414234e-07, 0.00020182535808999091, 0.9985585808753967, 1.9142603058242003e-09, 0.002459985902532935, 0.999762237071991, 0.001538218930363655, 1.0, 0.000595143239479512, 0.9997307658195496, 7.653925422346219e-07, 7.543829269707203e-06, 1.0, 0.9999990463256836, 9.646728722145781e-05, 2.749417762970552e-05, 0.00010922888031927869, 0.00010586578719085082, 3.6394380003912374e-05, 0.9998687505722046, 0.0007260427810251713, 0.00014371299766935408, 0.9998283386230469, 0.9997791647911072, 4.340573013905669e-06, 0.7227540612220764, 0.9999997615814209, 0.00010839796595973894, 0.00029583636205643415, 0.9997029900550842, 0.0005942294956184924, 0.9999972581863403, 0.9999465942382812, 0.999998927116394, 8.030779596879256e-09, 0.9999955892562866, 0.9986065030097961, 0.9999390840530396, 0.0011781007051467896, 0.9999719858169556, 0.9998724460601807, 0.9999997615814209, 1.378304830268462e-07, 0.00019366838387213647, 0.9999364614486694, 0.9999958276748657, 0.9991732239723206, 0.9114842414855957, 3.712942486799875e-07, 0.9999555349349976, 1.0, 0.99996018409729, 0.9999995231628418, 9.39723179271823e-08, 0.9994063377380371, 6.636088073719293e-06, 0.005561295431107283, 0.9997749924659729, 0.999968409538269, 0.9999980926513672, 0.9999651908874512, 0.0006053197430446744, 0.9999943971633911, 0.00045166717609390616, 0.9880065321922302, 0.9998995065689087, 0.999937891960144, 0.8419942259788513, 0.0009657913469709456, 0.9999995231628418, 1.0, 0.9993681311607361, 0.9995518326759338, 4.1926173253159504e-06, 0.006922536995261908, 0.008669216185808182, 0.9995438456535339, 0.000270084390649572, 0.9999966621398926, 0.9999982118606567, 0.000821187102701515, 0.001171851996332407, 3.2258962164632976e-05, 4.2028549529504744e-08, 1.3746787772106472e-05, 0.9998459815979004, 0.9999982118606567, 4.303173045627773e-05, 0.0016504412051290274, 0.994690477848053, 0.11929316818714142, 1.0, 0.030604813247919083, 0.011795603670179844, 0.9573959112167358, 0.0005803925450891256, 0.0007188006420619786, 1.1301724043732975e-05, 2.5455702257204393e-07, 0.0047022202052176, 0.0011443381663411856, 0.9999872446060181, 0.9998577833175659, 0.0028182140085846186, 2.016275902860798e-05, 0.020709030330181122, 0.00012556421279441565, 0.9996633529663086, 0.00019507837714627385, 0.9999352693557739, 0.9983844757080078, 0.9995542168617249, 0.9999971389770508, 0.9999287128448486, 0.9999958276748657, 0.9999899864196777, 0.0005334548186510801, 0.999916672706604, 0.0001419208711013198, 7.656803546751689e-08, 0.99937504529953, 0.9999862909317017, 1.5045023246784694e-05, 0.999996542930603, 0.0017466661520302296, 0.0008486745064146817, 0.9999135732650757, 0.9999979734420776, 0.9998334646224976, 0.9395811557769775, 1.0449035471538082e-05, 0.9999998807907104, 1.8656781321624294e-05, 4.41864585809526e-06, 1.0, 9.578566096024588e-05, 4.0838105633156374e-05, 0.9943558573722839, 3.7035515561001375e-05, 0.999974250793457, 0.9857126474380493, 1.0, 0.9999982118606567, 0.9998173117637634, 0.9995884299278259, 0.9999960660934448, 3.056280561963831e-08, 0.999970555305481, 0.9999680519104004, 0.9988259673118591, 0.9999792575836182, 0.0033564362674951553, 0.9999895095825195, 0.8972957730293274, 0.000221238486119546, 0.9999665021896362, 7.100373409230087e-07, 0.9988645315170288, 0.9986798167228699, 0.0009689087746664882, 0.09138922393321991, 8.322523967763118e-07, 3.335886731292703e-07, 0.0032413445878773928, 0.9997565150260925, 0.9991596937179565, 0.9996435642242432, 0.0014757405733689666, 0.999983549118042, 2.508584839233663e-05, 0.9975177049636841, 1.3481844689522404e-06, 0.9999498128890991, 0.01935684122145176, 0.0008017703075893223, 0.9999251365661621, 6.69271500441937e-08, 0.9997057318687439, 7.004278813838027e-06, 1.099446126318071e-05, 9.844211490417365e-07, 4.832319973502308e-06, 0.06048499420285225, 0.9976755976676941, 0.9999992847442627, 6.993005172262201e-06, 0.9999983310699463, 0.9999921321868896, 0.9999983310699463, 1.0, 1.3500886097972398e-06, 1.9141828033752972e-06, 1.0, 1.4341046153276693e-05, 0.00027538646827451885, 6.245477561606094e-05, 3.697211695907754e-06, 9.991726557245784e-08, 8.754723967285827e-05, 2.746687277976889e-05, 0.9999954700469971, 0.002928216941654682, 4.196117151877843e-05, 1.9894139313691994e-06, 0.00021769132581539452, 0.06810262054204941, 1.258258635061793e-05, 6.457594992070881e-08, 0.9999983310699463, 0.9998905658721924, 0.9999592304229736, 0.9956246018409729, 0.9999850988388062, 0.0011329952394589782, 0.9999984502792358, 0.9999994039535522, 1.0, 0.9999997615814209, 0.00024428730830550194, 2.2211586838238873e-05, 0.9999940395355225, 0.9999966621398926, 0.9996750354766846, 0.9923650622367859, 0.0008685894426889718, 0.0028251316398382187, 0.9999502897262573, 0.001030111568979919, 1.5741650827294507e-07, 1.1042801162375326e-07, 1.3759716921413201e-08, 0.0006251145969145, 0.0005303080542944372, 2.5945003017113777e-06, 0.9993329644203186, 0.0013565878616645932, 0.0008111161878332496, 0.02236577682197094, 0.000310542993247509, 0.9998680353164673, 0.9997929930686951, 0.9999724626541138, 0.9999946355819702, 0.00870164018124342, 0.9999983310699463, 0.9997685551643372, 0.008956140838563442, 0.9988088607788086, 0.999588668346405, 0.9999626874923706, 0.9998466968536377, 2.961640348075889e-05, 0.999984860420227, 0.007605599705129862, 0.00033508421620354056, 5.4315460147336125e-05, 8.915583862290077e-07, 0.9999743700027466, 0.006030604243278503, 0.0078666927292943, 0.9998682737350464, 0.9999991655349731, 0.9998990297317505, 0.9974049925804138, 1.0, 0.9997218251228333, 5.723552021663636e-06, 0.9999263286590576, 3.2528822657695855e-07, 1.0, 0.0005591753288172185, 2.759830124432483e-07, 4.145634738961235e-05, 1.0, 0.01112874411046505, 6.303962862830303e-08, 0.0003449146752245724, 2.5091065253946e-05, 0.9985771179199219, 1.6983195791908656e-06, 0.0008832730818539858, 0.9999700784683228, 0.9998517036437988, 0.9999988079071045, 0.9999996423721313, 0.9998314380645752, 0.0015219966880977154, 0.9999998807907104, 0.01010876428335905, 0.9999611377716064, 0.09403801709413528, 0.007135157473385334, 0.8749261498451233, 8.624060399142763e-08, 0.0010953648015856743]

#Metricas de validación
Best_acc5_1 = 1.0
acc_aux5_1 = 1.0
Recall5_1 = 1.0
Especificidad5_1 = 1.0
Tiempo5_1 = 852.8007211685181

######## Base6 ########
#MODELO = VGG11
#Datos perdidas
xx_train6_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train6_1 = [0.04918559335264159, 0.03705816175098772, 0.04315621490463798, 0.03472445869335422, 0.03965546197177451, 0.02456666362690337, 0.02997578153455699, 0.021650808258557024, 0.023625389100224884, 0.023871378904139554, 0.02021329007398935, 0.02607843859328164, 0.02378465626158832, 0.023454674664470883, 0.01941681151036863, 0.021182680470340047, 0.026329233745733895, 0.0209634338816007, 0.023877933465036347, 0.01948673516327952, 0.027157885296109283, 0.029280143985409797, 0.025039528273505928, 0.02308286946312881, 0.02130268347668059]
xx_val6_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val6_1 = [0.012589140401946173, 0.0162213706307941, 0.012484306924872929, 0.011731492231289546, 0.021763484676678976, 0.014365829858514997, 0.013662087089485592, 0.012858998278776805, 0.012643617888291677, 0.013312579029136233, 0.0133057811193996, 0.013297984500726064, 0.013282022376855215, 0.01322589558031824, 0.013186461726824443, 0.013184898263878293, 0.01318250877989663, 0.013181310643752416, 0.01318163408173455, 0.013180352416303423, 0.013180323772960238, 0.013180278821123971, 0.01318023486269845, 0.01318026449945238, 0.013180254234208]
epoch_min6_1 = 0
stop_val6_1 = 0.012589140401946173

#Datos accuracy
acc_train6_1 = [0.979320987654321, 0.9891975308641975, 0.9848765432098765, 0.9891975308641975, 0.9873456790123457, 0.9907407407407407, 0.9885802469135803, 0.9932098765432098, 0.991358024691358, 0.9919753086419753, 0.9925925925925926, 0.9901234567901235, 0.9910493827160494, 0.9910493827160494, 0.9935185185185185, 0.9910493827160494, 0.9888888888888888, 0.9932098765432098, 0.9919753086419753, 0.9907407407407407, 0.9919753086419753, 0.9907407407407407, 0.991358024691358, 0.9916666666666667, 0.9922839506172839]
acc_val6_1 = [0.9888888888888889, 0.9916666666666667, 0.9972222222222222, 0.9944444444444445, 0.9888888888888889, 0.9888888888888889, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667]

#Datos curva ROC
best_labels6_1 = [1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0]
best_prob6_1 = [0.9902273416519165, 1.0, 1.0, 0.9999992847442627, 0.9741166234016418, 2.3429227439919487e-05, 0.9999997615814209, 8.258148227469064e-06, 0.0003937058208975941, 0.0004510493890848011, 2.4245659005828202e-05, 0.9999921321868896, 0.0002863521222025156, 1.0, 1.0, 1.0, 0.9999966621398926, 0.0031437622383236885, 0.9997931122779846, 0.0010842749616131186, 0.0003732100303750485, 0.9949216246604919, 0.0023902691900730133, 3.614720480982214e-05, 1.2285220691410359e-05, 0.9999936819076538, 0.9999873638153076, 0.9999938011169434, 4.983286271453835e-05, 0.9813992381095886, 0.03250394016504288, 0.0008049127063713968, 1.1437899047450628e-05, 5.129578767082421e-06, 0.9999556541442871, 2.7881020287168212e-05, 1.0, 0.0011272862320765853, 0.0004168922605458647, 0.997669517993927, 0.9999998807907104, 1.0, 1.7486806882516248e-06, 1.0, 1.0, 0.4283631443977356, 0.0014288075035437942, 0.9999303817749023, 0.000943413411732763, 0.9999997615814209, 2.4044600195338717e-06, 0.0002270654367748648, 5.079843322164379e-05, 5.354542736313306e-05, 0.00018535630078986287, 4.133846960030496e-05, 0.9999984502792358, 0.9999988079071045, 0.9997910857200623, 0.002640711609274149, 0.00014188098430167884, 0.9969499707221985, 0.0014325923984870315, 1.0, 1.0, 0.9999994039535522, 1.0, 0.00011938982061110437, 1.0, 0.9872097969055176, 2.291386863362277e-06, 0.9999974966049194, 0.999996542930603, 0.9999992847442627, 6.893676527397474e-06, 1.0, 1.5968416846590117e-05, 0.5903658270835876, 1.0, 1.0, 1.1584495950955898e-05, 5.092551145935431e-05, 0.9998518228530884, 0.9999942779541016, 0.9999994039535522, 9.320723620476201e-05, 0.9999946355819702, 1.6018535461626016e-05, 1.8036196706816554e-05, 1.8893797459895723e-07, 0.9999865293502808, 1.4985561165303807e-06, 0.9999983310699463, 1.0, 0.9999991655349731, 6.275949999690056e-05, 3.956233740609605e-06, 4.0037641156231984e-05, 1.0, 1.6088286429294385e-05, 0.0018392963102087379, 0.9999905824661255, 0.9999978542327881, 0.9999982118606567, 5.8037116104969755e-05, 0.9999949932098389, 1.803469302785743e-07, 0.9999905824661255, 6.634733836108353e-06, 0.6433050632476807, 1.0346873295929981e-06, 0.9999922513961792, 0.999852180480957, 1.593163688085042e-05, 0.9999929666519165, 1.6426491129095666e-05, 0.9999998807907104, 0.9999997615814209, 1.6946696632658131e-06, 2.976544783450663e-05, 5.088826924293244e-07, 6.33574109087931e-06, 1.1394555343713364e-07, 3.3900203106895788e-06, 0.9991680383682251, 0.9979640245437622, 0.00012694373435806483, 4.01058241550345e-06, 0.9999992847442627, 0.9999998807907104, 0.002893631113693118, 0.9999322891235352, 1.0, 0.0031647575087845325, 0.9998840093612671, 0.00615523150190711, 1.0, 1.0, 1.0, 0.0008129606139846146, 6.200395432642836e-07, 0.00014301404007710516, 0.00017463589028920978, 1.0, 0.9999997615814209, 0.9999666213989258, 0.9996333122253418, 0.997810423374176, 0.9999998807907104, 0.0002245160867460072, 1.0, 1.0, 0.9958303570747375, 1.3786776435154025e-05, 0.9999955892562866, 4.796946541318903e-06, 0.9999929666519165, 0.9999982118606567, 0.9996151924133301, 0.9999768733978271, 0.9999701976776123, 7.520756753365276e-06, 0.00011333602742524818, 0.0011672248365357518, 0.00010846413351828232, 0.9998360872268677, 0.003144189715385437, 0.9999254941940308, 2.0711624983960064e-06, 1.0, 0.0008032844634726644, 3.0139777663862333e-05, 1.96386990864994e-05, 3.7285803955455776e-06, 0.00018107419600710273, 3.136982923024334e-05, 0.0013673907378688455, 2.4700566427782178e-05, 0.0012295187916606665, 0.0006965110660530627, 0.5422828197479248, 0.9956419467926025, 0.9818972945213318, 0.000902960600797087, 0.00027852682978846133, 1.0, 0.008726237341761589, 1.0, 0.9999969005584717, 0.9999959468841553, 0.0006903226021677256, 0.9999996423721313, 1.1558466894712183e-06, 1.0, 0.9977099895477295, 0.9999991655349731, 1.0, 0.9999690055847168, 0.9999997615814209, 1.0, 3.505402855807915e-05, 1.9273606994829606e-06, 0.9999939203262329, 1.0, 1.6543206129426835e-06, 0.999985933303833, 4.569611519400496e-06, 1.0, 0.9999984502792358, 0.0015465019969269633, 0.9999980926513672, 0.01415286585688591, 2.744519224506803e-05, 0.9999994039535522, 0.9999935626983643, 0.9994605183601379, 1.3771829117104062e-06, 0.9999263286590576, 1.0, 7.539964599345694e-07, 0.9999861717224121, 0.0002553054073359817, 1.0, 0.00015982578042894602, 1.0, 0.9999994039535522, 2.9108300623192918e-06, 2.748078441072721e-05, 7.136733074730728e-06, 3.5139128158334643e-06, 0.999990701675415, 1.2730345588352066e-05, 2.9280361104611075e-06, 0.0010586879216134548, 0.9999971389770508, 0.0018393970094621181, 1.172686779682408e-06, 0.00019213258929084986, 4.987587089999579e-06, 0.9999463558197021, 1.0, 2.6838220946956426e-05, 0.9999998807907104, 0.9999998807907104, 3.267714419052936e-05, 0.9999998807907104, 1.0, 0.9999972581863403, 5.534261049433553e-07, 1.4340184861794114e-06, 0.000510231708176434, 4.682977305492386e-05, 4.512288069236092e-05, 0.9999982118606567, 4.101178274140693e-05, 0.9999904632568359, 1.0, 1.5636403986718506e-05, 0.9999877214431763, 4.7231897042365745e-05, 0.007039215415716171, 1.2338890883256681e-05, 1.0, 0.0004444877558853477, 0.9999734163284302, 7.660286200916744e-07, 0.012838581576943398, 1.0, 5.617376882582903e-05, 0.004483929835259914, 1.0, 2.4817938538035378e-05, 0.4914494454860687, 0.0008214598055928946, 0.00010770073276944458, 0.9999775886535645, 0.9998201727867126, 2.781300281640142e-06, 8.34996099001728e-05, 0.0013909493573009968, 1.4412228210858302e-07, 1.0, 0.0003655047621577978, 0.0017900458769872785, 0.9997749924659729, 1.0, 1.0, 0.9996333122253418, 1.0, 2.094617229886353e-06, 1.0, 1.0, 0.9999996423721313, 0.9999345541000366, 0.9999991655349731, 0.9999997615814209, 0.9999998807907104, 1.0, 1.5484774849028327e-05, 0.9999501705169678, 0.0016936339670792222, 0.9966192245483398, 0.9999996423721313, 6.282333515628125e-07, 5.582877929555252e-05, 0.9999984502792358, 1.0, 7.590534551127348e-06, 0.9999973773956299, 0.999998927116394, 1.0, 6.466650120273698e-06, 0.9999996423721313, 1.0, 0.0003720899694599211, 6.085823315515881e-06, 1.0, 0.9999998807907104, 0.0057254997082054615, 6.040087100700475e-06, 0.0001016812093439512, 0.9989151954650879, 0.999849796295166, 7.664242730243132e-05, 0.00022879050811752677, 1.549951957713347e-05, 4.968036228092387e-05, 0.016861364245414734, 0.013150112703442574, 2.6088360755238682e-05, 0.0008598688873462379, 1.0, 4.385960039599013e-07, 1.0, 0.9999996423721313, 7.5952906627208e-05, 0.000446674763225019, 5.982490620226599e-06, 0.00021021513384766877, 0.9997199177742004, 7.985919978636957e-07, 0.00017539116379339248, 1.0, 1.0, 1.0, 0.0013050602283328772, 2.383115133852698e-05, 0.9999998807907104, 1.0, 0.9999992847442627, 3.647964604169829e-06, 0.00033921090653166175, 6.187613621477794e-07, 0.9999716281890869, 0.00301360827870667, 0.9999998807907104, 0.00027173536363989115, 0.00020625621255021542, 0.9999998807907104, 0.9999998807907104]

#Metricas de validación
Best_acc6_1 = 0.9888888888888889
acc_aux6_1 = 0.9972222222222222
Recall6_1 = 0.9835164835164835
Especificidad6_1 = 0.9943820224719101
Tiempo6_1 = 854.6769406795502

######## Base7 ########
#MODELO = VGG11
#Datos perdidas
xx_train7_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train7_1 = [0.05852958387062873, 0.03384096909452368, 0.03953490048462962, 0.04400728856339867, 0.035346895677072034, 0.0377863601953895, 0.031278013750726794, 0.026471325175629723, 0.02922380072651086, 0.028201628733932237, 0.04032089082915106, 0.023549460233361633, 0.01971004968624056, 0.02196755004517826, 0.020913212792372998, 0.019259911941157448, 0.019651195287336536, 0.03240718580322501, 0.0258543569272683, 0.02165581585816395, 0.03307449226210147, 0.026977912290596668, 0.0243395769117791, 0.022742072097313257, 0.023265505342939755]
xx_val7_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val7_1 = [0.023077238765027788, 0.021317399872673882, 0.030452036857604982, 0.02625253746906916, 0.01859600328736835, 0.02540991289748086, 0.025667249576912986, 0.02466870049635569, 0.02636363059282303, 0.02652507656150394, 0.026453219685289596, 0.0264891740348604, 0.02635027633772956, 0.02637395229604509, 0.02638760887914234, 0.026376289129257203, 0.02637026177512275, 0.026375729507870146, 0.02637224925888909, 0.02636455645163854, 0.026364417208565607, 0.026364492376645406, 0.026364482442537942, 0.026364634765519035, 0.026364663077725305]
epoch_min7_1 = 4
stop_val7_1 = 0.01859600328736835

#Datos accuracy
acc_train7_1 = [0.9765432098765432, 0.9867283950617284, 0.986111111111111, 0.9848765432098765, 0.9885802469135803, 0.9876543209876543, 0.9885802469135803, 0.9898148148148148, 0.9882716049382716, 0.991358024691358, 0.9854938271604938, 0.9910493827160494, 0.9925925925925926, 0.9907407407407407, 0.9932098765432098, 0.9925925925925926, 0.9925925925925926, 0.9910493827160494, 0.9925925925925926, 0.9922839506172839, 0.9891975308641975, 0.990432098765432, 0.9922839506172839, 0.9941358024691358, 0.9925925925925926]
acc_val7_1 = [0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9916666666666667, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112]

#Datos curva ROC
best_labels7_1 = [0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0]
best_prob7_1 = [0.0007313649402931333, 1.0, 0.9999892711639404, 0.9999696016311646, 1.0, 2.0072320694453083e-05, 0.0006660935468971729, 0.00018530628585722297, 0.0023381838109344244, 5.924090510234237e-05, 0.0001312055828748271, 0.9999997615814209, 0.9999537467956543, 0.005887896753847599, 0.999995231628418, 1.0, 0.0024448400363326073, 0.0001819360622903332, 1.0, 1.0, 0.0007526571280322969, 0.0019961304496973753, 5.262190825305879e-05, 0.6229854822158813, 1.0, 0.45692116022109985, 0.9999889135360718, 1.0, 1.0, 0.000692852889187634, 0.001667142496444285, 1.0, 0.9999775886535645, 1.1007258535755682e-06, 0.9999997615814209, 0.0025613724719733, 0.9999914169311523, 0.9999991655349731, 8.402551611652598e-05, 0.0007594107300974429, 0.0003625558747444302, 5.3242874855641276e-05, 0.0007251532515510917, 0.00017174815002363175, 0.9999707937240601, 0.9999998807907104, 0.00048349431017413735, 5.310374035616405e-05, 0.8716818690299988, 0.00027138247969560325, 0.9999878406524658, 0.9999517202377319, 0.9999041557312012, 1.0, 0.999703586101532, 0.9999837875366211, 0.9999998807907104, 0.9999641180038452, 0.004192539490759373, 1.0, 0.01592036336660385, 0.0003770147159229964, 1.2357238119875547e-05, 1.0, 0.013438201509416103, 1.0, 1.0, 0.00036777742207050323, 0.9999643564224243, 0.9998621940612793, 0.9999778270721436, 0.0015603245701640844, 0.9999998807907104, 0.0001413457066519186, 1.0, 0.999925971031189, 0.0801851749420166, 1.8876268086387427e-06, 0.9999203681945801, 1.0, 1.0, 1.0, 0.00013476383173838258, 0.0007744206814095378, 0.999998927116394, 1.0, 0.9999978542327881, 9.726008102006745e-06, 0.9999951124191284, 0.9999995231628418, 1.0, 0.0006694630137644708, 0.6359909772872925, 0.9999997615814209, 2.9817327231285162e-05, 9.595735900802538e-05, 0.9999998807907104, 0.9999997615814209, 0.9999876022338867, 0.9997857213020325, 2.902627556977677e-06, 1.0, 0.9999926090240479, 0.0010208503808826208, 0.9999922513961792, 0.0002739349438343197, 0.9999756813049316, 0.056155767291784286, 1.0, 4.156822615186684e-05, 0.9999998807907104, 0.9999927282333374, 1.0, 0.9997568726539612, 1.3722856238018721e-05, 1.0, 1.8494090909371153e-05, 0.00039145510527305305, 0.00038675410905852914, 3.347794745423016e-06, 0.005463504232466221, 0.00047261605504900217, 0.9999405145645142, 0.32180896401405334, 0.0002102478320011869, 0.9999996423721313, 5.57364292035345e-05, 0.0004724102036561817, 0.9999984502792358, 0.00012170901754871011, 0.9996517896652222, 0.9999988079071045, 0.9999995231628418, 1.0, 9.256303314941761e-07, 9.346570732304826e-05, 8.125823660520837e-05, 0.0012815891532227397, 0.9999998807907104, 0.00655154325067997, 0.001874609268270433, 9.755855217008502e-07, 1.0, 8.871454338077456e-05, 0.9999994039535522, 0.9999990463256836, 0.00044107396388426423, 0.17715655267238617, 0.13496549427509308, 1.0, 0.9999854564666748, 0.00010205033322563395, 0.9999644756317139, 0.0008226367644965649, 1.0, 0.0003354674845468253, 0.9999994039535522, 0.011783533729612827, 1.0, 0.002430480206385255, 1.0, 0.9999996423721313, 3.854609531117603e-06, 0.9999972581863403, 0.9999960660934448, 0.9999551773071289, 0.9999995231628418, 0.9999998807907104, 1.0, 0.007631437852978706, 1.0, 0.002448095940053463, 0.9999997615814209, 0.00041142525151371956, 0.9999344348907471, 0.3898348808288574, 2.5144143364741467e-05, 0.9999909400939941, 1.0, 0.0005509481416083872, 3.742188255273504e-06, 0.0018104970222339034, 0.999908447265625, 0.002915899269282818, 3.9317587834375445e-06, 0.9999997615814209, 0.00022035134315956384, 0.0007461298955604434, 1.0, 4.761694071930833e-05, 0.9999996423721313, 1.0, 0.9999994039535522, 1.753025344441994e-06, 0.9966235160827637, 0.0003618117771111429, 0.9999980926513672, 0.9999998807907104, 0.9999998807907104, 5.97631469645421e-06, 0.00010368361108703539, 0.9999721050262451, 0.9999998807907104, 2.2840495148557238e-05, 0.11710047721862793, 0.0013575170887634158, 0.9994797110557556, 0.0008266930235549808, 1.1014474694093224e-05, 0.9999969005584717, 1.0, 1.0, 0.0014742068015038967, 6.2302624428411946e-06, 5.83435466978699e-05, 0.9999909400939941, 7.389364327536896e-05, 0.9999045133590698, 0.00011673642438836396, 1.0, 0.9999701976776123, 0.9999998807907104, 0.9999996423721313, 3.3204545957232767e-07, 1.3021189261053223e-05, 6.990331939960015e-07, 0.0007212197524495423, 0.007002776954323053, 1.0, 0.0007508912240155041, 0.0009085160563699901, 0.9999996423721313, 0.0001260927238035947, 0.9999983310699463, 0.9999997615814209, 0.0001610140170669183, 7.71202167015872e-07, 6.115821634011809e-06, 0.0003922111645806581, 9.438201959710568e-05, 5.75013218622189e-05, 3.6460344290389912e-06, 0.9999904632568359, 0.00026788399554789066, 0.9999998807907104, 1.0332466899853898e-06, 0.0002717685420066118, 0.007055595982819796, 0.021421821787953377, 0.9999086856842041, 1.813673588912934e-05, 0.999919056892395, 1.0, 1.0, 1.0, 0.0010035717859864235, 0.0002455286157783121, 0.0014403144596144557, 0.9999971389770508, 0.9999487400054932, 0.000934946583583951, 1.0, 0.9999523162841797, 0.0038989875465631485, 0.553511381149292, 0.9999997615814209, 0.9999966621398926, 0.9999912977218628, 4.07877450925298e-06, 0.9999915361404419, 0.999964714050293, 1.0, 0.19478049874305725, 1.0, 1.0, 0.9999990463256836, 0.0012510825181379914, 8.405253174714744e-05, 9.456163752474822e-06, 0.00010023705544881523, 0.002223890507593751, 1.0, 0.0012067240895703435, 8.265604378721036e-07, 0.9999966621398926, 9.19226004043594e-05, 1.0, 0.9990467429161072, 1.0, 0.0014708148082718253, 0.998562753200531, 0.9999998807907104, 0.022857002913951874, 0.999066174030304, 1.0, 0.0013601465616375208, 0.9999834299087524, 3.3669514323264593e-06, 0.9999822378158569, 0.9999709129333496, 0.999691367149353, 0.9999986886978149, 0.9981119632720947, 0.9999985694885254, 0.0008120021084323525, 0.9819477796554565, 0.9996985197067261, 0.0005966169992461801, 0.9998968839645386, 0.9999997615814209, 0.9999997615814209, 0.003914250992238522, 1.0, 0.00022014178102836013, 1.0, 0.010905385948717594, 0.9999990463256836, 0.9999978542327881, 9.892607266692721e-08, 0.00012354503269307315, 1.0, 1.0, 0.4406174123287201, 0.00016415446589235216, 1.0, 6.223191348908585e-07, 0.9999988079071045, 0.00024603644851595163, 0.00024815555661916733, 2.307698423464899e-06, 0.005906844977289438, 0.00591244176030159, 1.0, 8.86116595211206e-06, 0.9999939203262329, 1.0, 3.212906085536815e-05, 0.9998934268951416, 1.5166490811679978e-05, 1.0, 0.00029398908372968435, 2.7754349503084086e-05, 0.0005085555603727698, 1.4388678209797945e-05, 1.0, 4.154682392254472e-05, 0.9999669790267944, 0.0001661322166910395, 0.001800787285901606, 1.0, 0.00043676531640812755, 0.00010735364776337519, 4.098487988812849e-05, 7.892323878877505e-07, 0.4525894224643707, 1.0, 0.004418340977281332, 0.9999997615814209, 0.0022914086002856493, 2.1997455405653454e-05]

#Metricas de validación
Best_acc7_1 = 0.9916666666666667
acc_aux7_1 = 0.9941358024691358
Recall7_1 = 0.9836065573770492
Especificidad7_1 = 1.0
Tiempo7_1 = 894.2863526344299

######## Base8 ########
#MODELO = VGG11
#Datos perdidas
xx_train8_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train8_1 = [0.046421508288677825, 0.034413222444278226, 0.02639211568935418, 0.0436305057671335, 0.029985065408694892, 0.023382328745023703, 0.021578978958689137, 0.020145348146741772, 0.025200657215383317, 0.023099886460436717, 0.02106822653685087, 0.01973838035799103, 0.026550952510701286, 0.02446625299843741, 0.02612892761826515, 0.024484127548373775, 0.019555849168035717, 0.017445055386166513, 0.018739922013547686, 0.016575264332839, 0.02148130476290797, 0.01571363754294537, 0.024136177652779922, 0.01992616848445233, 0.02029860081312097]
xx_val8_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val8_1 = [0.0038984379834598967, 0.005241189897060394, 0.003806668519973755, 0.011790271765655941, 0.013888336386945513, 0.006854506333669027, 0.004389532407124837, 0.0053770090142885845, 0.005566597481568654, 0.004826113084952037, 0.00474945820040173, 0.004644647902912564, 0.004661834571096632, 0.004803670528862211, 0.004930889937612745, 0.004930959807501899, 0.0049328306482897865, 0.00493025713496738, 0.0049314717451731365, 0.004932430552111732, 0.004932356046305763, 0.004932389656702677, 0.004932311342822181, 0.004932421776983473, 0.004932499759727054]
epoch_min8_1 = 0
stop_val8_1 = 0.0038984379834598967

#Datos accuracy
acc_train8_1 = [0.9802469135802468, 0.9867283950617284, 0.9895061728395061, 0.9851851851851852, 0.9895061728395061, 0.9901234567901235, 0.9941358024691358, 0.9938271604938271, 0.9910493827160494, 0.9929012345679012, 0.9925925925925926, 0.9922839506172839, 0.9916666666666667, 0.9910493827160494, 0.990432098765432, 0.990432098765432, 0.9916666666666667, 0.9935185185185185, 0.9950617283950617, 0.9935185185185185, 0.9941358024691358, 0.9941358024691358, 0.9907407407407407, 0.9925925925925926, 0.9932098765432098]
acc_val8_1 = [1.0, 1.0, 1.0, 0.9944444444444445, 0.9944444444444445, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels8_1 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
best_prob8_1 = [1.0, 0.9999986886978149, 0.9999352693557739, 1.0, 0.9999967813491821, 0.6378328204154968, 0.9998015761375427, 1.8248605556436814e-05, 0.002528809243813157, 1.0, 0.0001983672264032066, 0.9997918009757996, 0.0002973605296574533, 3.065515193156898e-05, 8.354265446541831e-06, 0.9999872446060181, 0.9999780654907227, 0.9999487400054932, 1.2771301044267602e-05, 1.0, 0.9999948740005493, 0.0016315425746142864, 8.83105822140351e-05, 1.0, 8.096894453046843e-05, 0.0014929140452295542, 0.9999325275421143, 1.0, 2.5439496766921366e-06, 4.5617903765560186e-07, 1.8065442418446764e-05, 1.0, 0.9999610185623169, 0.9999558925628662, 0.1550901085138321, 0.003566569881513715, 0.999980092048645, 1.4639046639786102e-05, 5.00500354974065e-05, 3.879727955791168e-05, 1.0, 2.1249372366582975e-05, 0.9999957084655762, 2.2065467419452034e-05, 0.0014986000023782253, 9.313718578596308e-07, 1.0, 0.00045533350203186274, 1.0, 0.9997852444648743, 0.999678373336792, 0.00045972137013450265, 0.9984815716743469, 1.0, 0.9998897314071655, 1.0, 1.0, 2.0366020180517808e-05, 0.00019143459212500602, 0.9998284578323364, 0.9989218711853027, 0.00010749203647719696, 0.9997557997703552, 0.9999922513961792, 1.0, 0.9999756813049316, 1.0, 1.846786472015083e-05, 0.00015504015027545393, 2.2661501134280115e-05, 0.020655237138271332, 0.00019194948254153132, 0.9999951124191284, 2.0952060367562808e-05, 9.347007289761677e-05, 0.0010045325616374612, 1.730848816805519e-05, 0.9999886751174927, 0.9999996423721313, 0.9987609386444092, 0.999167799949646, 0.000332446041284129, 0.9999916553497314, 4.967387212673202e-05, 1.0, 6.082017080188962e-06, 7.874852599343285e-06, 0.9999783039093018, 0.9999877214431763, 0.9981499910354614, 0.9999899864196777, 0.0011757340980693698, 0.00016932982543949038, 0.9999998807907104, 7.178220130299451e-06, 0.9978412389755249, 0.9993577599525452, 0.9999651908874512, 0.02586119994521141, 0.9999982118606567, 0.999947190284729, 2.3377899083243392e-07, 4.96783759444952e-05, 0.9998408555984497, 0.9942607879638672, 0.9999984502792358, 1.0, 0.9998964071273804, 0.9998819828033447, 2.7111105737276375e-05, 0.9999428987503052, 2.1858939362573437e-05, 2.1205705706961453e-05, 0.018142499029636383, 0.00029981433181092143, 0.0017311497358605266, 0.0007239403785206378, 0.0033443542197346687, 0.9999982118606567, 0.001812804490327835, 7.144558185245842e-05, 0.9999997615814209, 1.0, 0.0015854532830417156, 0.9951024055480957, 0.9999998807907104, 0.006761359516531229, 0.9998453855514526, 1.4060530702408869e-05, 2.8198768632137217e-05, 0.0004751614760607481, 0.9999998807907104, 0.0009066270431503654, 0.0005629097577184439, 0.00012320527457632124, 8.541227725800127e-05, 0.9999946355819702, 0.9999740123748779, 0.9999992847442627, 0.00021288063726387918, 0.00035923506948165596, 4.427195017342456e-05, 0.0001408425159752369, 1.0, 0.001852572662755847, 1.0, 0.9999998807907104, 0.9999932050704956, 0.9999805688858032, 0.0019083949737250805, 0.9999653100967407, 6.756164111720864e-06, 1.2256574336788617e-05, 1.0, 0.9999294281005859, 4.879548214375973e-05, 0.0005290439003147185, 0.999995231628418, 2.3593082005390897e-05, 1.0, 0.9999784231185913, 0.00045258086174726486, 0.99992835521698, 0.003494641510769725, 0.0005422490648925304, 0.037922587245702744, 8.696740223967936e-06, 1.0, 0.9962453246116638, 1.0, 0.9999994039535522, 1.4531528336192423e-07, 0.9990247488021851, 2.130080247297883e-05, 0.9999735355377197, 9.268633220926858e-06, 8.06438474683091e-05, 0.999969482421875, 0.9999932050704956, 0.9980239868164062, 5.905179932597093e-06, 5.5434342357330024e-05, 6.19315551375621e-06, 1.0, 1.0, 1.4544202713295817e-06, 0.9999959468841553, 0.9999452829360962, 1.0, 0.9999998807907104, 0.9999818801879883, 0.999915361404419, 2.0471383322728798e-05, 0.9999879598617554, 0.0039892494678497314, 3.0167782369971974e-06, 0.002031537238508463, 0.00018530027591623366, 2.0446718735911418e-06, 0.0007670968770980835, 4.123343842366012e-06, 1.0, 0.005394685082137585, 0.9999998807907104, 1.2006182714685565e-07, 2.2016702132532373e-05, 1.0, 0.0030682410579174757, 0.00034475160646252334, 3.149746771669015e-05, 0.9999902248382568, 9.503571345703676e-05, 1.0, 3.2811517485242803e-06, 6.893557838338893e-06, 0.9993987083435059, 1.0, 0.9999973773956299, 0.9999843835830688, 4.8684182729630265e-06, 0.004132136702537537, 0.001860406599007547, 0.0009682812378741801, 0.9999960660934448, 1.0, 0.999196469783783, 0.0004752767563331872, 0.9977495074272156, 7.073310553096235e-05, 0.00011404188262531534, 0.0008911712793633342, 0.9997536540031433, 0.0024022276047617197, 1.0, 0.9999998807907104, 0.9918437600135803, 0.0005170057993382215, 1.342587529507e-05, 1.390990200889064e-05, 0.024445446208119392, 0.9999955892562866, 2.367110391787719e-05, 0.9999998807907104, 1.0, 0.9999794960021973, 0.9955911636352539, 0.9999909400939941, 2.3725662686047144e-05, 0.9997588992118835, 1.0, 0.999314546585083, 0.0001916341861942783, 0.7314041256904602, 3.520357859088108e-05, 0.0005211678799241781, 0.9999947547912598, 0.9371607899665833, 5.242490806267597e-05, 0.9509874582290649, 0.9999698400497437, 2.233946906926576e-05, 2.9714965421590023e-05, 0.9999971389770508, 0.9999974966049194, 0.9999979734420776, 0.9999524354934692, 1.5014916243671905e-06, 0.00012622469512280077, 1.0, 2.8725382435368374e-05, 4.716779585578479e-06, 4.30819891334977e-05, 0.00019500400230754167, 2.9184154755057534e-06, 0.0010406456422060728, 7.740522960375529e-06, 0.00016839854652062058, 0.9993478655815125, 1.2759031051245984e-05, 0.0009468735661357641, 1.0, 1.0, 0.9999953508377075, 0.005203702487051487, 2.7178639356861822e-05, 0.9997566342353821, 0.9999998807907104, 0.9989897608757019, 1.0, 9.815143857849762e-05, 0.0010974538745358586, 1.0, 1.0, 1.0, 1.3864741958968807e-05, 0.9997333884239197, 1.4721928891958669e-05, 0.0002055776712950319, 4.655350494431332e-06, 0.0008080320549197495, 2.8536494937725365e-05, 0.9997438788414001, 0.00019903636712115258, 2.700926870602416e-06, 0.9999804496765137, 0.000150592255522497, 0.00012292858446016908, 0.9994577765464783, 1.0, 1.0, 1.0, 2.882961052819155e-05, 0.9993420243263245, 7.720401481492445e-05, 1.0, 0.9999852180480957, 0.0013170242309570312, 1.0, 0.0034166770055890083, 9.835239325184375e-05, 0.9999920129776001, 0.9979375600814819, 0.00010011917038355023, 2.0434223202414614e-08, 1.0, 0.027091501280665398, 0.0002468741440679878, 0.9865053296089172, 1.0, 0.9999996423721313, 4.463510322239017e-06, 0.9997275471687317, 0.03534848242998123, 0.00025007277145050466, 0.999983549118042, 1.0, 1.8596419977257028e-05, 0.9950426816940308, 0.9997718930244446, 7.979831570992246e-05, 0.00027828552993014455, 1.4211841516953427e-05, 1.0, 0.9999078512191772, 1.46415595736471e-05, 0.00015297066420316696, 1.0, 5.767941146928024e-08, 1.9478162357700057e-05, 0.9998307228088379, 0.9981958270072937, 1.077944671123987e-05, 3.790667733483133e-06, 0.9998745918273926, 0.99991774559021, 0.9997097849845886, 7.4344775384815875e-06, 4.6186505642253906e-05, 0.0003684671828523278, 0.00012049741781083867]

#Metricas de validación
Best_acc8_1 = 1.0
acc_aux8_1 = 1.0
Recall8_1 = 1.0
Especificidad8_1 = 1.0
Tiempo8_1 = 954.8154344558716

######## Base9 ########
#MODELO = VGG11
#Datos perdidas
xx_train9_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train9_1 = [0.05008739565993533, 0.030124667778979115, 0.03905812425561893, 0.031451870721799356, 0.0306359583304988, 0.020245667151462886, 0.024473380738938295, 0.020690904235766257, 0.01941438748696704, 0.019568255286157868, 0.022868822303451136, 0.02352248008052508, 0.018511450906962525, 0.031854685230387585, 0.018151707027429416, 0.015581404224590019, 0.024743240105885047, 0.020266211290418366, 0.01934050246613261, 0.022652703367265655, 0.02368181179519053, 0.02157351824880382, 0.01877833354620286, 0.01838201543799153, 0.017487220659300133]
xx_val9_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val9_1 = [0.009206521759430568, 0.013660121295187209, 0.031009900901052687, 0.01188034498029285, 0.0062439632084634565, 0.006890527655680974, 0.007262146224578222, 0.008157674885458417, 0.009090600245528751, 0.008015692234039307, 0.008077068130175273, 0.00806555598974228, 0.008084084176354938, 0.008038753602239821, 0.008090065999163521, 0.00808830377128389, 0.008086713982952966, 0.008089500665664672, 0.00809018760919571, 0.008088436639971204, 0.008088396737972896, 0.008088432914680905, 0.008088454686933094, 0.008088522652784983, 0.008088571826616923]
epoch_min9_1 = 4
stop_val9_1 = 0.0062439632084634565

#Datos accuracy
acc_train9_1 = [0.9808641975308642, 0.9879629629629629, 0.9851851851851852, 0.9885802469135803, 0.9895061728395061, 0.9922839506172839, 0.990432098765432, 0.9932098765432098, 0.9938271604938271, 0.9922839506172839, 0.9922839506172839, 0.9925925925925926, 0.9941358024691358, 0.9898148148148148, 0.9950617283950617, 0.995679012345679, 0.9922839506172839, 0.9932098765432098, 0.9935185185185185, 0.9916666666666667, 0.9919753086419753, 0.9944444444444445, 0.9929012345679012, 0.9932098765432098, 0.9944444444444445]
acc_val9_1 = [0.9972222222222222, 0.9944444444444445, 0.9888888888888889, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222]

#Datos curva ROC
best_labels9_1 = [0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0]
best_prob9_1 = [7.767345232423395e-05, 1.0, 6.243026291485876e-06, 0.9999866485595703, 0.00038004847010597587, 0.9999006986618042, 0.00021802408446092159, 0.999862551689148, 1.2191554787932546e-07, 1.0, 0.0001502172090113163, 0.9999344348907471, 0.9114512205123901, 1.0, 0.9999977350234985, 1.0, 3.5111997931380756e-06, 3.5560282185542746e-07, 2.4027906420087675e-07, 0.9997802376747131, 0.9999997615814209, 1.0, 5.51402854398475e-06, 7.826753426343203e-05, 0.9999994039535522, 1.0, 0.9999587535858154, 3.6336423363536596e-06, 0.9999985694885254, 0.00016959678032435477, 0.004016919061541557, 0.9632338285446167, 1.0, 0.9999996423721313, 0.9999996423721313, 0.0003305590071249753, 0.9945354461669922, 0.9999994039535522, 9.61578763281068e-08, 1.0, 0.9803982377052307, 0.9999961853027344, 0.9999992847442627, 1.4370248209161218e-06, 1.0, 4.6233608941292914e-07, 8.233157132053748e-05, 0.000868358591105789, 0.9999997615814209, 1.3732287698076107e-06, 9.407673132955097e-06, 0.9999277591705322, 9.530201168672647e-06, 4.1794792196014896e-05, 0.9997859597206116, 1.0, 0.9975979924201965, 0.0011749598197638988, 9.619347110856324e-06, 5.3072701120981947e-05, 0.003246742533519864, 1.0, 0.999924898147583, 9.626447763366741e-07, 0.9999587535858154, 2.225766183983069e-05, 0.9999860525131226, 2.980373756145127e-05, 1.0, 1.0, 0.9988351464271545, 6.076695353840478e-05, 3.0366920327651314e-05, 1.0, 2.2330821593641303e-05, 0.9999990463256836, 4.726541374111548e-05, 1.0, 1.0, 0.009521163068711758, 0.9999998807907104, 0.9994931221008301, 5.2473174037004355e-06, 0.0017169798957183957, 1.2284529475437012e-05, 1.0, 1.0, 0.9999964237213135, 0.0017447228310629725, 2.4224418666562997e-05, 0.0001278606941923499, 0.0011618806747719646, 6.696005584672093e-05, 0.9999842643737793, 2.7924363621423254e-06, 8.97876759609062e-07, 1.4545670978804992e-07, 0.00021217930770944804, 1.0, 2.44432362705993e-06, 0.000244905793806538, 0.0019577015191316605, 0.9999687671661377, 0.9999665021896362, 4.1149603930534795e-05, 0.9999982118606567, 0.00374134280718863, 0.999970555305481, 3.630230594353634e-06, 0.9999301433563232, 0.9999819993972778, 0.0010112186428159475, 1.0, 8.516667003277689e-05, 4.7223202273016796e-06, 0.9995954632759094, 1.0, 1.0, 0.00023094228527043015, 0.9999330043792725, 0.9999996423721313, 0.9999996423721313, 1.0, 2.743036702668178e-06, 1.0629703410813818e-06, 8.754072041483596e-05, 1.0, 0.011540654115378857, 0.0003635041357483715, 0.9999998807907104, 3.594300324039068e-06, 0.9999377727508545, 1.0, 1.3259328852655017e-06, 1.0582790537227993e-06, 0.9999982118606567, 0.9999995231628418, 3.158397157676518e-05, 9.450611287320498e-06, 0.0004690739733632654, 0.0002802190138027072, 0.0008090054616332054, 1.1703807558660628e-06, 1.0, 1.0, 0.9999998807907104, 0.9999951124191284, 0.999971866607666, 0.029845673590898514, 0.9999395608901978, 1.0, 0.9999488592147827, 9.250547918782104e-06, 0.999976634979248, 1.0, 1.0, 0.9999990463256836, 0.999914288520813, 0.9999998807907104, 0.9999996423721313, 0.0010961017105728388, 0.999987006187439, 1.108679953176761e-05, 4.5652154767594766e-07, 6.5071144490502775e-06, 0.9505278468132019, 3.3012220228556544e-05, 0.9994016885757446, 9.630744898458943e-05, 0.00047356533468700945, 3.724081125255907e-06, 3.037207352463156e-05, 0.9999830722808838, 3.0000039259903133e-05, 0.9997212290763855, 5.278261738794754e-08, 0.9999932050704956, 0.9999363422393799, 8.686367800692096e-05, 0.0038061446975916624, 1.3416637330010417e-06, 0.9999678134918213, 0.0006160184857435524, 0.9999969005584717, 1.3165773452783469e-05, 0.0004724622704088688, 0.060288187116384506, 0.9999996423721313, 0.9999971389770508, 0.0026473598554730415, 5.1398474170127884e-05, 0.9999992847442627, 1.8218846662421129e-06, 0.9999994039535522, 5.786541237284837e-07, 0.9999995231628418, 0.9999902248382568, 0.15606771409511566, 0.9999977350234985, 0.9999525547027588, 3.376391759957187e-06, 1.0, 2.712020830131223e-07, 0.9999998807907104, 1.1697353329509497e-05, 1.0, 0.9981768131256104, 0.9999997615814209, 2.0614987079170533e-05, 0.9996528625488281, 1.884026346488099e-06, 6.501527241198346e-05, 0.9991633892059326, 0.9999593496322632, 4.838216773350723e-05, 0.0005711637786589563, 0.0014744749059900641, 0.9999582767486572, 0.0010588605655357242, 0.00024233355361502618, 0.001823892118409276, 5.664238500457941e-08, 0.9999997615814209, 0.00014257399016059935, 5.6515233154641464e-05, 0.9998688697814941, 0.00010492488945601508, 4.177129540039459e-07, 0.9998254179954529, 0.9999997615814209, 3.2545983685849933e-07, 5.447990133689018e-06, 0.9999998807907104, 0.00012089897063560784, 0.9997608065605164, 0.9999998807907104, 1.0, 0.9999991655349731, 0.00022342528973240405, 0.0001359925081487745, 0.9999442100524902, 1.0, 1.0, 1.0, 0.9998418092727661, 0.0004081331717316061, 0.013797815889120102, 0.0012179409386590123, 6.598851882699819e-07, 0.0005950073827989399, 4.200236617180053e-06, 0.03198579326272011, 0.9979973435401917, 0.9980531930923462, 8.406196684518363e-06, 0.7231290936470032, 8.93417454790324e-05, 0.9999998807907104, 0.00013971919543109834, 0.999988317489624, 0.0005073393112979829, 0.0008770603453740478, 1.0, 0.9999688863754272, 1.0, 0.9994246959686279, 8.55751713970676e-05, 0.00016474028234370053, 0.00022412215184886009, 0.001115876017138362, 1.0, 0.9999282360076904, 0.9994737505912781, 3.99273112350329e-08, 0.33485281467437744, 0.9900469183921814, 0.00010959455539705232, 2.6978354071616195e-05, 0.9999983310699463, 1.0, 0.0001224648003699258, 0.0001238033000845462, 0.9999985694885254, 0.00014670941163785756, 1.6896903616725467e-05, 1.9521452486515045e-05, 0.00020655675325542688, 0.9999887943267822, 0.9999995231628418, 0.9999990463256836, 5.855060226167552e-05, 0.0029841293580830097, 3.215677133994177e-05, 1.0, 6.813960595764001e-08, 0.9997648596763611, 0.017059609293937683, 0.9999990463256836, 1.0, 1.306705416936893e-05, 0.9999985694885254, 0.9999998807907104, 2.1485320189640333e-07, 0.0005334291490726173, 0.02380979061126709, 5.407295248005539e-06, 0.9999994039535522, 3.905273479176685e-05, 0.00018248087144456804, 0.0002602742752060294, 1.0, 0.00033320634975098073, 0.9999998807907104, 0.08500538766384125, 0.0034669949673116207, 0.999987006187439, 1.0, 1.0, 9.784108988242224e-05, 0.9999985694885254, 0.9999992847442627, 0.026437873020768166, 0.9999998807907104, 0.988310694694519, 1.0, 0.9999773502349854, 0.9999997615814209, 2.709443106141407e-05, 0.9999991655349731, 0.9999980926513672, 1.0, 0.9999918937683105, 1.6942398133323877e-06, 8.844609692459926e-05, 0.0006542209885083139, 0.9999852180480957, 0.0017189834034070373, 9.272028546547517e-05, 1.0, 0.00021152765839360654, 1.095154402719345e-06, 7.317560084629804e-05, 0.9999985694885254, 0.9999998807907104, 0.002505842363461852, 2.3737458221262386e-08, 2.7830836302200623e-07, 0.003040996380150318, 0.9999920129776001, 1.0, 1.0, 0.026208721101284027, 0.9999986886978149, 4.911297310172813e-06, 0.9999997615814209, 0.9921841025352478, 1.48199899285828e-06, 0.00012344458082225174, 2.717143615882378e-05, 1.0]

#Metricas de validación
Best_acc9_1 = 0.9972222222222222
acc_aux9_1 = 0.9972222222222222
Recall9_1 = 1.0
Especificidad9_1 = 0.994475138121547
Tiempo9_1 = 968.4241545200348

######## Base10 ########
#MODELO = VGG11
#Datos perdidas
xx_train10_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train10_1 = [0.04136746614067643, 0.03839869050332058, 0.03565037853923845, 0.02738686850593414, 0.04814706551439968, 0.03727288613164866, 0.028032615348512743, 0.02062614270382457, 0.025941099980731068, 0.021512421395307707, 0.02738985096414884, 0.02911046278881438, 0.024004964300511796, 0.027616211394836873, 0.0172469597539784, 0.018977551097855157, 0.020909790814290813, 0.022369208499605272, 0.02775783704386817, 0.02844404986611119, 0.02529104542401102, 0.024688724823939946, 0.023033653144483215, 0.025429839466089083, 0.01937122961253296]
xx_val10_1 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val10_1 = [0.043857738292879526, 0.0035139244463708664, 0.0015666562649938795, 0.580500144428677, 0.001662922567791409, 0.0016755034526189168, 0.0013709018627802532, 0.001423169838057624, 0.0011883689297570122, 0.0010875287983152602, 0.0010784486929575601, 0.0010721849070654975, 0.0010550727446873983, 0.0010469983021418254, 0.0010235736767450969, 0.0010232667128245035, 0.0010233319467968412, 0.0010232223404778374, 0.0010238336192237006, 0.0010243839687771267, 0.001024391253789266, 0.0010243998633490668, 0.001024393571747674, 0.001024407148361206, 0.0010243988699383205]
epoch_min10_1 = 12
stop_val10_1 = 0.0010550727446873983

#Datos accuracy
acc_train10_1 = [0.9839506172839506, 0.9858024691358025, 0.9873456790123457, 0.991358024691358, 0.983641975308642, 0.9827160493827161, 0.9895061728395061, 0.9916666666666667, 0.990432098765432, 0.991358024691358, 0.9907407407407407, 0.9885802469135803, 0.9922839506172839, 0.9882716049382716, 0.9938271604938271, 0.9929012345679012, 0.9935185185185185, 0.9929012345679012, 0.9901234567901235, 0.991358024691358, 0.9901234567901235, 0.9907407407407407, 0.9922839506172839, 0.990432098765432, 0.9932098765432098]
acc_val10_1 = [0.9833333333333334, 1.0, 1.0, 0.8361111111111111, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels10_1 = [1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0]
best_prob10_1 = [1.0, 1.0, 0.9999884366989136, 0.9999995231628418, 0.00010894220031332225, 1.0, 0.9999933242797852, 0.9999661445617676, 0.0007956173503771424, 0.9999942779541016, 6.776853751944145e-06, 0.9999982118606567, 1.0, 0.0007578470394946635, 3.613900480559096e-05, 0.9999679327011108, 0.9999996423721313, 7.735387043794617e-06, 1.3105168363836128e-05, 1.0, 0.9996589422225952, 1.3630049124913057e-06, 1.0870526239159517e-06, 0.0012977088335901499, 1.5458597772521898e-05, 0.9999990463256836, 1.0, 0.9999551773071289, 0.9999804496765137, 3.5849912819685414e-05, 0.9997193217277527, 1.023062941385433e-05, 0.00039986131014302373, 0.9999326467514038, 0.00015154614811763167, 0.00040941720362752676, 0.10317498445510864, 0.9770326614379883, 0.9999027252197266, 0.9999960660934448, 0.9999672174453735, 0.9999949932098389, 0.0002131288347300142, 0.0004152090987190604, 1.5142991287575569e-05, 6.168386789795477e-06, 0.9999728202819824, 5.239041911409004e-06, 1.0, 0.9998888969421387, 0.9999333620071411, 0.9999816417694092, 0.011071011424064636, 0.0005673484411090612, 0.9999089241027832, 0.0005328541737981141, 0.9999996423721313, 1.0, 0.01627410762012005, 0.9999587535858154, 4.095205804333091e-05, 4.145147613598965e-06, 5.779973434982821e-06, 0.999975323677063, 0.001755648641847074, 0.9993537068367004, 1.0, 0.00018096026906277984, 1.7606116671231575e-06, 4.625844303518534e-05, 0.9999591112136841, 0.9999959468841553, 0.9999904632568359, 8.39446292957291e-05, 1.0, 0.0006996674928814173, 0.0007939556962810457, 0.9999786615371704, 1.0, 0.000359119672793895, 1.0, 0.004858413245528936, 1.4664841728517786e-05, 3.364749090906116e-06, 0.0004845083749387413, 0.9999970197677612, 1.684732887952123e-05, 3.590103005990386e-05, 8.581595466239378e-06, 1.0, 1.0, 0.9999703168869019, 0.0016890736296772957, 0.9999923706054688, 1.6166244677151553e-05, 0.999710738658905, 0.999990701675415, 2.872367304007639e-06, 0.0004548489232547581, 0.99997878074646, 5.180317657504929e-06, 1.0, 0.0005200563464313745, 0.00535814743489027, 0.9999873638153076, 0.9999979734420776, 0.9999895095825195, 0.00013167157885618508, 8.740884368307889e-05, 0.0014448519796133041, 0.9999439716339111, 1.777314355422277e-05, 0.0008036776562221348, 0.00028297328390181065, 1.0648774150467943e-05, 0.9999953508377075, 0.00011010278831236064, 3.689890945679508e-05, 0.9985613226890564, 6.369225047819782e-06, 0.9983968138694763, 1.0, 1.0, 0.00016543935635127127, 0.0023129419423639774, 0.9999330043792725, 1.1600756977259152e-07, 1.0, 0.004019713029265404, 0.9988627433776855, 1.0, 9.316795330960304e-05, 0.0025931219570338726, 0.9999825954437256, 0.9999932050704956, 1.6149342627613805e-05, 1.3974362445878796e-05, 0.0029191721696406603, 0.001248860964551568, 0.9999254941940308, 0.00012226536637172103, 4.489925049711019e-05, 1.0, 0.0018304417608305812, 0.00024715601466596127, 4.864872153120814e-06, 0.9999990463256836, 0.9999997615814209, 0.9999967813491821, 1.0, 0.9999983310699463, 5.053722998127341e-05, 0.9999973773956299, 8.486002116114832e-06, 1.0, 0.00018616841407492757, 1.0, 5.92191190662561e-06, 0.0012281429953873158, 0.9999923706054688, 0.9999662637710571, 0.0001145279238699004, 0.0001698850392131135, 0.9999696016311646, 2.2760652427677996e-05, 0.9992774128913879, 1.7349586414638907e-05, 0.0009970967657864094, 0.002280463697388768, 0.9999943971633911, 6.945895438548177e-05, 0.004263715352863073, 4.969661677023396e-05, 0.00038760763709433377, 4.50814695795998e-05, 1.014975623547798e-05, 0.005414089187979698, 1.0079947060148697e-05, 0.0021761709358543158, 0.9999948740005493, 0.00010668421600712463, 6.8189729063306e-06, 0.9999926090240479, 0.9999444484710693, 1.0, 0.999992847442627, 0.02985871583223343, 0.9999892711639404, 0.00012671855802182108, 3.053465479752049e-05, 0.9999959468841553, 0.00015353808703366667, 1.11948302219389e-05, 0.005675897933542728, 0.9999982118606567, 0.9999833106994629, 2.5295834348071367e-05, 0.9999945163726807, 0.002185746794566512, 0.9999969005584717, 0.9999922513961792, 0.9998666048049927, 1.660106008216644e-08, 6.694291414532927e-07, 0.9999996423721313, 0.9996967315673828, 0.0007494456949643791, 0.00023739250900689512, 1.0, 0.9999923706054688, 0.9999921321868896, 0.00041927178972400725, 0.9987443685531616, 0.0006453330279327929, 0.9999791383743286, 0.0002571706718299538, 0.0009144467767328024, 9.6575911356922e-07, 6.77285868277977e-07, 1.6635103747830726e-05, 0.9999462366104126, 0.9991207718849182, 1.2889991921838373e-05, 0.9993636012077332, 4.816758973902324e-06, 0.9990934133529663, 2.067643617920112e-05, 0.001387863652780652, 0.9999085664749146, 0.9999983310699463, 3.116065405706081e-09, 0.9999986886978149, 1.0255108463752549e-05, 0.0016112423036247492, 0.0028723652940243483, 1.0, 0.9999810457229614, 1.0, 0.9999998807907104, 0.9999996423721313, 0.9999995231628418, 1.0, 2.5813953470787965e-05, 0.9999927282333374, 1.3059903722023591e-05, 0.9999985694885254, 0.9999977350234985, 7.105304575816263e-07, 0.00023115903604775667, 1.0, 0.9999936819076538, 6.193378794705495e-05, 1.0, 4.778081347467378e-06, 0.0006047367351129651, 0.9998410940170288, 1.3354991779124248e-06, 0.9999500513076782, 0.9999659061431885, 0.9999980926513672, 3.623133352448349e-06, 0.00010867263335967436, 0.031998660415410995, 8.191689994418994e-05, 0.9999865293502808, 1.7392078007105738e-05, 1.714394602458924e-05, 0.9999972581863403, 4.5110795326763764e-05, 8.065372639975976e-06, 0.9999967813491821, 0.9999270439147949, 0.9997772574424744, 0.000162564727361314, 0.9999957084655762, 1.0, 0.0016980496002361178, 0.000407337152864784, 7.625710168213118e-06, 0.9999513626098633, 1.5411158926781354e-07, 0.9999375343322754, 0.9873278737068176, 0.9999984502792358, 0.9999949932098389, 0.00010114208271261305, 0.00018751776951830834, 2.726017783061252e-06, 1.0, 1.0, 0.9999971389770508, 2.6695818888811118e-08, 0.0011648294748738408, 0.999996542930603, 9.105045319302008e-05, 0.999953031539917, 7.72734983911505e-06, 7.238790658448124e-06, 1.0, 0.00011665117926895618, 0.0011935682268813252, 1.0, 0.9999938011169434, 7.079423812683672e-05, 0.9998552799224854, 4.3834912503371015e-05, 4.2337873310316354e-05, 7.838310921215452e-06, 0.9998218417167664, 0.9999881982803345, 0.9999998807907104, 7.372374966507778e-05, 0.9999464750289917, 0.9998729228973389, 1.0, 0.00011025965795852244, 0.9999899864196777, 0.9999740123748779, 0.999994158744812, 1.0, 3.750634641619399e-05, 0.9999963045120239, 0.9998663663864136, 0.999983549118042, 0.9997938275337219, 0.999893307685852, 4.0260582068185613e-07, 1.6621499526081607e-05, 1.7019585357047617e-05, 0.999996542930603, 0.9999433755874634, 5.9732892623287626e-06, 0.0025866772048175335, 0.9600175023078918, 0.0005897647351957858, 1.0, 0.9999779462814331, 2.8375137844705023e-05, 0.9999977350234985, 9.211444194079377e-06, 0.9997885823249817, 1.1138687113998458e-05, 0.00022532287403009832, 6.831385690020397e-06, 4.991423452338495e-07, 0.9999767541885376, 0.0006153522408567369, 0.0002271641860716045, 0.9999967813491821, 0.9999936819076538, 0.9999992847442627, 0.9999847412109375, 0.9999998807907104, 0.9999890327453613, 0.9999912977218628, 0.9999579191207886, 0.0006933942786417902, 0.0006428585620597005, 0.9999985694885254, 9.458844579057768e-05]

#Metricas de validación
Best_acc10_1 = 1.0
acc_aux10_1 = 1.0
Recall10_1 = 1.0
Especificidad10_1 = 1.0
Tiempo10_1 = 868.9031457901001
###################################################################

######## Base1 ########
#MODELO = VGG11_BN
#Datos perdidas
xx_train1_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train1_2 = [0.2184301690554913, 0.10326206018159419, 0.06617469484055484, 0.08087875546864522, 0.08067546729688291, 0.06017408521822941, 0.05086850538665866, 0.05159557675137932, 0.04969877553758798, 0.04344734733487353, 0.04496122122178843, 0.046345526936613486, 0.05207076839826725, 0.054005381924870574, 0.04518667032689224, 0.041237399929467546, 0.048956905351376825, 0.04884645626132871, 0.0451028483517376, 0.048076097814389214, 0.049117270976672935, 0.04579927490817176, 0.04771008623970879, 0.04523799379850611, 0.05188934793810786]
xx_val1_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val1_2 = [0.11787565482987297, 0.05253313167227639, 0.0472585282391972, 0.04639755470885171, 0.0477694743209415, 0.03224909702936808, 0.03184325479798847, 0.03013968285587099, 0.030195060703489516, 0.03122851120101081, 0.03014287319448259, 0.029833174579673345, 0.03161492761638429, 0.032461346685886384, 0.029446892771455978, 0.029949982629881966, 0.03130664279063543, 0.03148862040705151, 0.029135974331034556, 0.028365024510357113, 0.03254071358177397, 0.03131658716334237, 0.030481148262818655, 0.030242286125818887, 0.03125145725078053]
epoch_min1_2 = 5
stop_val1_2 = 0.03224909702936808

#Datos accuracy
acc_train1_2 = [0.9117283950617284, 0.9626543209876544, 0.9728395061728395, 0.9725308641975309, 0.9694444444444444, 0.9765432098765432, 0.9824074074074074, 0.9799382716049383, 0.9817901234567901, 0.9845679012345678, 0.9830246913580246, 0.983641975308642, 0.9780864197530864, 0.9814814814814815, 0.9842592592592593, 0.9867283950617284, 0.9811728395061728, 0.9814814814814815, 0.9833333333333333, 0.9817901234567901, 0.9814814814814815, 0.9842592592592593, 0.9811728395061728, 0.9845679012345678, 0.9814814814814815]
acc_val1_2 = [0.9527777777777778, 0.9777777777777779, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9888888888888889, 0.9861111111111112, 0.9833333333333334, 0.9861111111111112, 0.9861111111111112, 0.9888888888888889, 0.9805555555555556, 0.9805555555555556, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9861111111111112, 0.9888888888888889, 0.9916666666666667, 0.9861111111111112, 0.9861111111111112, 0.9888888888888889, 0.9861111111111112, 0.9861111111111112]

#Datos curva ROC
best_labels1_2 = [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0]
best_prob1_2 = [0.00016671967750880867, 0.9870484471321106, 0.9995927214622498, 0.9999892711639404, 0.9731711745262146, 0.999976634979248, 0.9998915195465088, 0.999976634979248, 0.006618204060941935, 0.00122202024795115, 0.0001281536533497274, 5.437152140075341e-06, 0.9948529601097107, 0.9999961853027344, 0.9999345541000366, 1.4536212802340742e-05, 0.999997615814209, 0.0008705445216037333, 9.390882769366726e-05, 0.00028253544587641954, 0.9184796810150146, 0.9964776635169983, 0.9999980926513672, 0.0002552654768805951, 0.9999947547912598, 4.2887029849225655e-05, 0.9989456534385681, 0.0003079620364587754, 0.9999959468841553, 0.9999372959136963, 1.575555324961897e-05, 0.02777300588786602, 0.00015764385170768946, 0.0028994816821068525, 0.9787893891334534, 0.003809383837506175, 0.0005214692209847271, 0.9836843013763428, 0.9999958276748657, 0.00017875383491627872, 0.00038540136301890016, 0.9985125660896301, 0.9999773502349854, 0.33797311782836914, 0.0002675986324902624, 0.00015465242904610932, 0.0013562157982960343, 0.9998393058776855, 0.0007674145745113492, 0.0003483550972305238, 0.0007503218366764486, 0.9999969005584717, 0.01005377247929573, 0.9998379945755005, 0.9932906627655029, 0.9991769194602966, 0.9999845027923584, 0.00012340093962848186, 0.0021267132833600044, 0.9999434947967529, 0.0007826493820175529, 0.00098305637948215, 0.21535643935203552, 0.9999997615814209, 0.0001823591155698523, 0.9997426867485046, 0.0019858123268932104, 0.9999970197677612, 0.0005786227993667126, 0.9986461997032166, 0.0001140859312727116, 0.999702513217926, 0.00021204299991950393, 0.999981164932251, 0.000675804796628654, 0.011295749805867672, 0.9999834299087524, 2.9305976568139158e-05, 0.999885082244873, 1.0, 0.994121253490448, 2.437302282487508e-05, 0.003004213096573949, 0.00023101606348063797, 8.201179298339412e-05, 0.5116074681282043, 0.9999905824661255, 0.999871015548706, 0.011704884469509125, 0.9979233741760254, 0.0003245323314331472, 0.0019273563520982862, 0.00014629645738750696, 0.00010267925244988874, 4.431811976246536e-05, 0.0335986353456974, 0.030831413343548775, 0.999821126461029, 0.00011603059829212725, 0.0007703137816861272, 0.9998822212219238, 0.00025559618370607495, 0.9930071830749512, 0.999872088432312, 3.865011967718601e-05, 0.0003421882283873856, 0.0008328516851179302, 0.999752938747406, 0.0013121587689965963, 0.003083253977820277, 0.003116082400083542, 0.9999014139175415, 0.9998799562454224, 0.0028827746864408255, 0.0017375759780406952, 0.3314993977546692, 0.999816358089447, 0.9973695278167725, 4.8312049329979345e-05, 0.0007738368585705757, 0.00040134196751751006, 0.0002201747556682676, 0.9999587535858154, 0.00020856325863860548, 0.9999887943267822, 0.45559340715408325, 0.999977707862854, 0.999936580657959, 0.9999964237213135, 0.013672536239027977, 0.0004681707650888711, 0.999997615814209, 0.0007091521983966231, 4.01742581743747e-05, 7.35717621864751e-05, 7.835132419131696e-05, 0.9960761666297913, 0.002947595901787281, 0.42023560404777527, 0.9999548196792603, 0.0009648544946685433, 0.9785135984420776, 0.9999979734420776, 0.0046114628203213215, 0.9999982118606567, 0.9826933145523071, 0.00011542189167812467, 0.9976862668991089, 0.00013802318426314741, 0.0012066063936799765, 8.627153874840587e-06, 0.00039558386197313666, 0.9126123785972595, 1.0, 0.00017724672215990722, 0.9947235584259033, 0.999982476234436, 0.0001736865087877959, 2.7644671718007885e-05, 0.9826390743255615, 0.9999819993972778, 0.00019423759658820927, 0.9999951124191284, 0.00017630835645832121, 9.057106581167318e-06, 0.99122154712677, 0.0001462963264202699, 0.9999741315841675, 0.9999983310699463, 2.5397353965672664e-05, 0.9999963045120239, 5.505390618054662e-06, 0.00013896927703171968, 0.9999806880950928, 0.9999890327453613, 0.0014315014705061913, 0.9999302625656128, 0.0006114786956459284, 0.998309850692749, 0.005709587596356869, 0.0011636574054136872, 0.00011862564861075953, 0.9997130036354065, 7.664645090699196e-05, 8.087926289590541e-06, 2.1805470169056207e-05, 0.0004599457897711545, 0.9998542070388794, 9.247145499102771e-05, 0.9999644756317139, 0.9999933242797852, 0.00012122717453166842, 0.9989601373672485, 0.0003731627366505563, 0.9970417618751526, 0.99998939037323, 1.0, 0.9870619177818298, 0.9999945163726807, 1.0315037798136473e-05, 0.9999954700469971, 0.003258694661781192, 0.9999911785125732, 5.85022680752445e-05, 0.999778687953949, 0.4862341284751892, 0.9999955892562866, 0.0016768943751230836, 0.9999881982803345, 0.08447859436273575, 0.0004104633117094636, 0.9996142387390137, 0.0007394685526378453, 0.9382228255271912, 0.001257924479432404, 0.9996744394302368, 0.9999998807907104, 0.9995906949043274, 0.9946633577346802, 0.0039223055355250835, 0.003837383585050702, 0.00021285445836838335, 0.00517679238691926, 0.9998499155044556, 0.9982349872589111, 0.9999997615814209, 0.0008064093999564648, 0.9995743632316589, 0.9999845027923584, 0.0025729599874466658, 0.9989994168281555, 0.9997219443321228, 0.0002602641179692, 0.9997270703315735, 0.9997397065162659, 0.0013554280158132315, 0.9976662397384644, 0.012226518243551254, 0.9999996423721313, 0.7870796322822571, 0.1707581877708435, 0.9999904632568359, 0.9998486042022705, 0.9956858158111572, 0.00035381567431613803, 1.0, 0.0002979263663291931, 0.9952776432037354, 0.9999895095825195, 1.0, 0.9979730248451233, 0.9999988079071045, 0.9996676445007324, 0.0019983781967312098, 2.6526286092121154e-05, 0.999893307685852, 0.00044073202298022807, 0.0028832517564296722, 0.9659723043441772, 0.998638927936554, 1.9991814042441547e-05, 7.882059435360134e-05, 0.0019253836944699287, 0.9999879598617554, 0.004643609281629324, 0.9998981952667236, 0.999996542930603, 0.9999418258666992, 0.005117854569107294, 0.9999277591705322, 0.999988317489624, 0.9999712705612183, 0.00010155920608667657, 0.999982476234436, 0.9999641180038452, 0.00017605435277801007, 0.0001556477800477296, 0.021578768268227577, 0.002476653316989541, 0.00022366827761288732, 0.9999998807907104, 0.00020002909877803177, 0.9996515512466431, 0.10503684729337692, 0.9999591112136841, 0.9992566704750061, 0.001163062872365117, 9.57321681198664e-05, 0.001546854036860168, 0.0003918385482393205, 0.9999910593032837, 0.9999643564224243, 0.002223774092271924, 0.9999996423721313, 7.779338920954615e-05, 0.0002169832296203822, 0.9989363551139832, 0.9999805688858032, 0.9999442100524902, 0.0007039013435132802, 0.9999854564666748, 0.00034430346568115056, 0.9999563694000244, 0.988686203956604, 0.0001029944178299047, 0.9999998807907104, 0.9996738433837891, 1.2624776900338475e-05, 0.9999924898147583, 0.00039089389611035585, 0.9999089241027832, 0.999976396560669, 0.00013920373748987913, 0.9999920129776001, 0.9813668727874756, 0.0173628069460392, 2.251330079161562e-05, 0.9999432563781738, 0.00011857553909067065, 0.00012902052549179643, 0.999995231628418, 0.005193404387682676, 0.99980229139328, 0.0002711098932195455, 0.0001534859766252339, 0.9906377792358398, 0.0002534621162340045, 0.9997780919075012, 0.9999909400939941, 0.0023015441838651896, 0.00028829267830587924, 5.46767478226684e-05, 0.00011603059829212725, 3.581458440748975e-05, 0.9998481273651123, 0.9999487400054932, 0.004587074741721153, 0.9999191761016846, 3.021751399501227e-05, 0.9999992847442627, 0.9999511241912842, 0.9998025298118591, 0.0002568244526628405, 0.00036821173853240907, 0.9999980926513672, 0.0004946336266584694, 0.9999977350234985, 0.9999998807907104, 0.9956966638565063, 0.0005511046620085835, 0.002289792522788048, 0.0008059389074333012, 0.9968438148498535, 8.636677375761792e-05, 0.9991459846496582, 0.9999897480010986, 0.00026808917755261064, 0.999852180480957, 0.9999884366989136, 0.0006024781032465398]

#Metricas de validación
Best_acc1_2 = 0.9861111111111112
acc_aux1_2 = 0.9916666666666667
Recall1_2 = 0.9943502824858758
Especificidad1_2 = 0.9781420765027322
Tiempo1_2 = 640.9403622150421

######## Base2 ########
#MODELO = VGG11_BN
#Datos perdidas
xx_train2_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train2_2 = [0.09227443287769953, 0.06371768144545732, 0.05635245045026143, 0.051829561140434244, 0.04918988655194824, 0.045002383058085856, 0.03704390956295861, 0.038106519121814654, 0.03297635671901114, 0.03565108062308511, 0.04028750915586213, 0.038800323368222624, 0.03248186079256329, 0.03945791920945968, 0.0333566086527742, 0.035853430573587065, 0.03392833419788031, 0.03864959296252993, 0.03978984984166828, 0.03220633175454022, 0.03127179753817158, 0.03913427972131305, 0.036189100367051585, 0.03725146786482246, 0.03261543726663531]
xx_val2_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val2_2 = [0.05146126465664969, 0.043222594095600975, 0.07279341841737429, 0.11057261327902476, 0.055979737556642956, 0.04460529585679372, 0.04232117533683777, 0.03901186237732569, 0.041893057359589474, 0.041496022211180796, 0.0409373144308726, 0.04228105785118209, 0.041657549805111355, 0.040449392050504684, 0.04168814635939068, 0.04042427548103862, 0.04124196006192101, 0.04075121796793408, 0.041225271920363106, 0.0444998863670561, 0.03683496076199744, 0.04069262039330271, 0.04336626049545076, 0.03864630320005947, 0.03853852541910278]
epoch_min2_2 = 1
stop_val2_2 = 0.043222594095600975

#Datos accuracy
acc_train2_2 = [0.9716049382716049, 0.9802469135802468, 0.9811728395061728, 0.9783950617283951, 0.9814814814814815, 0.9842592592592593, 0.9864197530864197, 0.986111111111111, 0.9888888888888888, 0.9882716049382716, 0.9848765432098765, 0.9888888888888888, 0.9888888888888888, 0.9879629629629629, 0.9867283950617284, 0.9879629629629629, 0.9873456790123457, 0.9864197530864197, 0.986111111111111, 0.9885802469135803, 0.9891975308641975, 0.9854938271604938, 0.987037037037037, 0.986111111111111, 0.9901234567901235]
acc_val2_2 = [0.9722222222222222, 0.9833333333333334, 0.9666666666666667, 0.9666666666666667, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9888888888888889, 0.9833333333333334, 0.9833333333333334, 0.9861111111111112, 0.9861111111111112, 0.9833333333333334, 0.9861111111111112, 0.9833333333333334, 0.9833333333333334, 0.9861111111111112, 0.9861111111111112, 0.9888888888888889, 0.9833333333333334, 0.9888888888888889, 0.9833333333333334, 0.9833333333333334, 0.9861111111111112, 0.9861111111111112]

#Datos curva ROC
best_labels2_2 = [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0]
best_prob2_2 = [0.00010943780944216996, 0.00010852766718016937, 0.00028602060046978295, 0.9998728036880493, 0.9998505115509033, 0.00012532078835647553, 0.0003864833852276206, 0.00022331115906126797, 0.0016602427931502461, 0.9999892711639404, 0.9998127818107605, 0.997357189655304, 0.9991620779037476, 0.00011972387437708676, 0.0028867300134152174, 1.8604190699988976e-05, 9.278954166802578e-06, 0.014679240062832832, 0.0012198872864246368, 0.9995018243789673, 0.006285025272518396, 0.9990060925483704, 0.9990094900131226, 0.34735482931137085, 0.997395396232605, 8.261792390840128e-05, 0.9997690320014954, 0.9992173910140991, 0.9986751675605774, 0.0014528814936056733, 0.0002559659187681973, 8.160401921486482e-05, 0.00021317860228009522, 0.9998379945755005, 0.9999383687973022, 0.9686347246170044, 0.0002933490613941103, 0.0005547368200495839, 0.00015156710287556052, 0.0002592146338429302, 7.008809006947558e-06, 0.0030219117179512978, 7.706315955147147e-05, 0.004714163485914469, 0.0010915118036791682, 0.9999427795410156, 0.8766904473304749, 0.0020231823436915874, 0.004133281297981739, 0.0002054211072390899, 0.0011896720388904214, 0.9999856948852539, 0.999957799911499, 0.9980016350746155, 0.0007597506046295166, 0.99969482421875, 0.005724772810935974, 1.4387812370841857e-05, 0.00023199453426059335, 0.9999600648880005, 0.9940617680549622, 0.007853621616959572, 0.999665379524231, 0.8503351211547852, 0.9998956918716431, 0.9993247985839844, 7.726664989604615e-06, 0.9950951337814331, 0.002802569419145584, 0.00019200218957848847, 0.9980796575546265, 0.9995081424713135, 0.9997022747993469, 4.4097589125158265e-05, 0.5642222166061401, 0.9891071915626526, 0.9999798536300659, 0.9970435500144958, 0.9992994070053101, 0.00023659532598685473, 0.998898983001709, 0.9938516616821289, 0.0010200833203271031, 0.9999340772628784, 0.0015499155269935727, 0.00020129223412368447, 1.5056562006066088e-05, 0.00013090792344883084, 2.7232657885178924e-05, 0.9977548718452454, 0.99969482421875, 0.999254047870636, 0.000873137207236141, 0.00013494412996806204, 0.9995735287666321, 0.0010851758997887373, 0.0007011530105955899, 0.9986562728881836, 0.12281680852174759, 0.0009329061140306294, 0.9987233281135559, 0.0020750982221215963, 0.9998239874839783, 0.9995844960212708, 8.246123798016924e-06, 3.540125908330083e-05, 0.9989522695541382, 0.006062950007617474, 0.9992202520370483, 0.9991342425346375, 0.9804896712303162, 0.9935593605041504, 0.966002345085144, 0.9996654987335205, 0.8882349133491516, 0.00022630917374044657, 0.0002685117651708424, 0.000318449194310233, 0.00044962068204768, 0.0003142158966511488, 0.9998865127563477, 0.9980207681655884, 0.004263879265636206, 0.010861510410904884, 0.9253771901130676, 1.5621379134245217e-05, 0.9095419645309448, 0.9962707757949829, 0.9826279282569885, 0.9969280362129211, 6.498901825580106e-07, 0.9960852861404419, 0.998612642288208, 8.420141057285946e-06, 0.9998334646224976, 2.005737587751355e-05, 0.0001654310035519302, 2.7731623049476184e-05, 0.9992743134498596, 0.9988541603088379, 0.9998602867126465, 0.9990793466567993, 0.00044486482511274517, 0.00012160878395661712, 0.9999672174453735, 0.9971404075622559, 1.966533318409347e-06, 6.31191287538968e-05, 0.9999536275863647, 0.9971610307693481, 0.9999992847442627, 0.00030684570083394647, 6.037351795384893e-06, 0.0005576522671617568, 0.0003267094143666327, 2.2113747036200948e-05, 2.4182430934160948e-05, 0.999980092048645, 0.9995297193527222, 1.4178971468936652e-05, 1.544814949738793e-05, 0.9999945163726807, 0.0005455224891193211, 0.9835431575775146, 2.1106483472976834e-05, 0.0007744893664494157, 0.9904524087905884, 0.0005528231849893928, 4.2865853174589574e-05, 0.9999045133590698, 0.0004040417552459985, 0.0002640544844325632, 0.8909890055656433, 0.9999921321868896, 2.7646201488096267e-05, 0.0028331545181572437, 0.00112466374412179, 0.0001939036010298878, 8.94567565410398e-05, 0.00020210804359521717, 0.0004045562818646431, 1.034888373396825e-05, 0.9998453855514526, 0.9998618364334106, 7.217336678877473e-05, 0.0002243998897029087, 0.0036249924451112747, 0.00018047503544948995, 8.014004379219841e-06, 0.9999426603317261, 0.9999954700469971, 8.539787086192518e-05, 0.9996758699417114, 0.9999850988388062, 0.9998427629470825, 0.999983549118042, 0.0027207108214497566, 3.5512908652890474e-05, 9.817577847570647e-06, 0.13441097736358643, 0.15928122401237488, 0.9993440508842468, 0.02874048240482807, 0.0014245903585106134, 0.9989099502563477, 0.00010790488886414096, 0.9999997615814209, 0.9982843995094299, 0.0005930373445153236, 6.428061169572175e-05, 0.8427284359931946, 0.99906986951828, 0.9987867474555969, 0.013367216102778912, 0.9999740123748779, 0.00017275067511945963, 0.9999394416809082, 0.9978979825973511, 3.768974920603796e-06, 0.00017863456741906703, 0.8584729433059692, 0.9987075328826904, 0.0007758541032671928, 0.000911787967197597, 0.003338538808748126, 0.9999947547912598, 0.7521312236785889, 1.946880183822941e-05, 0.9994520545005798, 0.9997027516365051, 0.9988551139831543, 0.0005115430685691535, 0.009506303817033768, 1.1670387721096631e-05, 0.9997196793556213, 0.9999607801437378, 0.0012188699329271913, 0.9994439482688904, 0.993314802646637, 3.367024692124687e-05, 0.9960792660713196, 0.0015028305351734161, 1.0844229109352455e-05, 1.5857580137890182e-06, 0.7777650356292725, 0.9996410608291626, 0.9747272729873657, 0.9889094233512878, 0.9990880489349365, 0.0004923679516650736, 0.99997878074646, 5.51929697394371e-05, 0.001472978969104588, 0.9993916749954224, 0.9997562766075134, 0.0002492869389243424, 0.005480251740664244, 0.9953609108924866, 0.0004926155670545995, 0.9988466501235962, 0.9998471736907959, 9.932119610311929e-06, 0.9994896650314331, 0.7571494579315186, 0.0005308709223754704, 0.005038183182477951, 0.17738258838653564, 9.119322203332558e-05, 0.9991329312324524, 0.9997170567512512, 0.9996165037155151, 0.9992031455039978, 0.0006743405247107148, 0.00015364968567155302, 0.7565951347351074, 0.9943321943283081, 0.9999876022338867, 8.444225386483595e-06, 0.9995653033256531, 0.9979683756828308, 0.0002089412446366623, 0.0028802426531910896, 0.9996664524078369, 8.17481050034985e-05, 0.9972825050354004, 0.9999786615371704, 0.0015587074449285865, 0.9999755620956421, 0.9999600648880005, 9.796422818908468e-05, 0.9999908208847046, 0.9998619556427002, 0.0004850596305914223, 0.9999805688858032, 0.9928399324417114, 0.9990247488021851, 2.8775970349670388e-05, 0.9996383190155029, 0.5424748063087463, 0.022512366995215416, 0.9997342228889465, 0.991008460521698, 0.0004992820904590189, 0.002556601772084832, 0.9998364448547363, 0.9997729659080505, 2.852335455827415e-05, 0.30309414863586426, 0.9998804330825806, 0.9999788999557495, 0.0002273912978125736, 0.0005133072845637798, 0.9987245202064514, 0.38043493032455444, 0.0015041534788906574, 0.9996949434280396, 4.7881676437100396e-05, 0.9991440773010254, 0.9999879598617554, 0.0028043892234563828, 1.0798860785143916e-05, 0.00013477397442329675, 5.718100624108047e-07, 0.9979013204574585, 0.9950803518295288, 0.9996194839477539, 0.0018769974121823907, 0.9882844686508179, 0.00014791883586440235, 4.780446829499851e-07, 0.9986782670021057, 0.9999667406082153, 7.338159048231319e-05, 0.0023851320147514343, 1.2774517017533071e-05, 0.0004983411636203527, 0.9998461008071899, 0.999969482421875, 0.9999288320541382, 0.9999959468841553, 0.9997691512107849, 6.42657105345279e-05, 0.9819830656051636, 0.998891294002533, 0.00016333538223989308, 0.002120654098689556, 0.9999433755874634, 0.999964714050293, 0.9992252588272095, 0.00038865694659762084, 0.00010460919293109328, 0.9999911785125732, 0.9999932050704956, 9.289366971643176e-06, 0.00035763555206358433, 5.2408111514523625e-05, 0.9987331032752991, 4.172374974587001e-05, 0.9999840259552002, 0.9999978542327881]

#Metricas de validación
Best_acc2_2 = 0.9833333333333334
acc_aux2_2 = 0.9901234567901235
Recall2_2 = 0.9887640449438202
Especificidad2_2 = 0.978021978021978
Tiempo2_2 = 648.0858445167542

######## Base3 ########
#MODELO = VGG11_BN
#Datos perdidas
xx_train3_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train3_2 = [0.079405405216011, 0.0543495817316903, 0.05568786187120426, 0.05053692036020903, 0.046955237160494295, 0.04135258880294399, 0.039602733182686346, 0.0369957114075437, 0.03746759235674952, 0.031230363562519167, 0.0331238838257613, 0.03508211427632673, 0.03913924148605193, 0.03368257998868271, 0.04079261596004168, 0.038247677987372435, 0.029816913623132824, 0.031210981879705266, 0.03732011827789707, 0.03532071874281507, 0.03606560519999928, 0.038891758852534826, 0.0299976274923042, 0.03213555122047295, 0.041606511920690536]
xx_val3_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val3_2 = [0.01694533112976286, 0.014153257177935707, 0.07478667249282202, 0.02461437318060133, 0.02642513861258825, 0.022803934415181477, 0.025667347841792636, 0.023624034225940706, 0.026733433703581493, 0.02751165794001685, 0.02368347015645769, 0.0214331136809455, 0.025021031830045913, 0.023268671996063656, 0.022274087369441985, 0.02673581921392017, 0.023134544160630967, 0.02276981274286906, 0.018393514057000477, 0.02699113173617257, 0.01927276270257102, 0.02321663647890091, 0.015960964229371814, 0.03291060063574049, 0.028569682935873666]
epoch_min3_2 = 1
stop_val3_2 = 0.014153257177935707

#Datos accuracy
acc_train3_2 = [0.975, 0.9814814814814815, 0.979320987654321, 0.9802469135802468, 0.9817901234567901, 0.983641975308642, 0.9854938271604938, 0.9876543209876543, 0.987037037037037, 0.990432098765432, 0.9876543209876543, 0.986111111111111, 0.9848765432098765, 0.9888888888888888, 0.9858024691358025, 0.9851851851851852, 0.9885802469135803, 0.9895061728395061, 0.9879629629629629, 0.9885802469135803, 0.987037037037037, 0.9845679012345678, 0.9907407407407407, 0.987037037037037, 0.9842592592592593]
acc_val3_2 = [0.9944444444444445, 0.9944444444444445, 0.9722222222222222, 0.9916666666666667, 0.9861111111111112, 0.9888888888888889, 0.9861111111111112, 0.9861111111111112, 0.9833333333333334, 0.9833333333333334, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9833333333333334, 0.9888888888888889, 0.9833333333333334, 0.9861111111111112, 0.9861111111111112, 0.9916666666666667, 0.9833333333333334, 0.9888888888888889, 0.9861111111111112, 0.9972222222222222, 0.9805555555555556, 0.9861111111111112]

#Datos curva ROC
best_labels3_2 = [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]
best_prob3_2 = [0.9994619488716125, 3.8003506688255584e-06, 1.26320746858255e-05, 2.8692691557807848e-05, 0.5207088589668274, 5.256883741822094e-05, 0.9999629259109497, 0.0004464504308998585, 0.00017308603855781257, 0.9993382096290588, 0.0001255428942386061, 0.9997811913490295, 0.009731163270771503, 0.999795138835907, 0.9999101161956787, 6.362990006891778e-06, 9.735889034345746e-05, 0.9999232292175293, 5.2836920076515526e-05, 0.9999111890792847, 0.00012237793998792768, 4.602246008289512e-06, 2.5548711164447013e-06, 0.9997294545173645, 0.999825656414032, 3.4828775824280456e-05, 4.095510666957125e-05, 0.9999736547470093, 5.6059698181343265e-06, 0.9992751479148865, 0.000479776703286916, 0.9998553991317749, 9.017198317451403e-05, 1.4892920262354892e-05, 0.9997934699058533, 0.0007390895625576377, 0.9956958293914795, 0.9999725818634033, 9.113323903875425e-05, 0.017446408048272133, 0.9999880790710449, 0.9895874261856079, 0.00018913960957434028, 0.05269259959459305, 0.9999823570251465, 0.00031724190921522677, 0.9429093599319458, 0.9999990463256836, 8.867867290973663e-05, 0.9996588230133057, 0.0015141995390877128, 0.0009502749890089035, 0.0002067886816803366, 7.860244295443408e-06, 0.9999998807907104, 0.9974126219749451, 0.9995967745780945, 0.9990027546882629, 3.6221244954504073e-06, 0.999901533126831, 0.9999760389328003, 0.00045190853415988386, 3.887209004460601e-06, 0.9998751878738403, 0.965501606464386, 0.0005476590595208108, 0.001970803365111351, 0.9994636178016663, 0.9921781420707703, 0.0011402195086702704, 0.09792349487543106, 0.9999741315841675, 7.071667340596832e-08, 0.9999527931213379, 2.7991422030027024e-05, 0.9999357461929321, 0.0038413756992667913, 0.001045775949023664, 0.9999654293060303, 0.9999576807022095, 9.760246939549688e-07, 0.9998689889907837, 0.9999959468841553, 0.9999891519546509, 0.0005163648165762424, 0.9883092641830444, 0.0028337985277175903, 0.0005834947223775089, 7.600797107443213e-05, 0.999972939491272, 0.9996606111526489, 1.9404074919293635e-05, 0.9993731379508972, 0.9999954700469971, 0.0003081270260736346, 1.7860296566141187e-06, 2.6323654310544953e-05, 0.00018934349645860493, 0.0007102509262040257, 0.0016226370353251696, 0.9984734654426575, 0.005723521113395691, 1.590525926076225e-06, 0.9999579191207886, 0.00013244678848423064, 9.33803676161915e-05, 0.999994158744812, 9.524719644105062e-05, 2.3761982447467744e-05, 0.9776020646095276, 1.332271767751081e-05, 0.990068256855011, 0.00014094141079112887, 0.0001355666754534468, 0.001137111452408135, 0.999963641166687, 2.96131256618537e-05, 0.9999887943267822, 0.9999829530715942, 0.000345006090356037, 0.9958100318908691, 0.9999133348464966, 0.0007095952751114964, 0.9998156428337097, 0.9992066025733948, 0.023101139813661575, 0.9997050166130066, 0.0006461817538365722, 0.9983105659484863, 0.9999982118606567, 0.003154679434373975, 0.9921017289161682, 0.9998569488525391, 0.9990670084953308, 0.9994031190872192, 0.9999140501022339, 0.9992814660072327, 0.9992290735244751, 0.9997474551200867, 5.11888247274328e-05, 0.0038335227873176336, 0.0002637465368025005, 0.996681272983551, 0.9999920129776001, 0.9997943043708801, 0.0023967272136360407, 0.0003669577417895198, 0.00018337040091864765, 0.9961109757423401, 0.9999954700469971, 0.9995822310447693, 2.1381318219937384e-05, 0.0003023931058123708, 0.9999984502792358, 0.9997134804725647, 0.9999896287918091, 0.999855637550354, 0.999985933303833, 0.01381154078990221, 0.9999749660491943, 0.13330695033073425, 0.9997686743736267, 0.9995230436325073, 0.9991711378097534, 0.00016723043518140912, 0.22473162412643433, 0.9998513460159302, 0.011068119667470455, 0.001144741429015994, 0.9996957778930664, 0.9999912977218628, 2.1229623598628677e-05, 7.928374543553218e-05, 0.003328199265524745, 0.00010130985174328089, 0.9999712705612183, 0.9999873638153076, 0.9999176263809204, 0.9993312358856201, 8.714034265722148e-06, 3.2119894513016334e-06, 8.695019641891122e-05, 0.9999735355377197, 5.803025123896077e-05, 0.9762416481971741, 1.6565132682444528e-05, 0.0007456167368218303, 1.8962253307108767e-05, 0.00012018346023978665, 0.9996036887168884, 0.9999994039535522, 0.9964871406555176, 0.9520670175552368, 1.8142765156881069e-06, 0.9990770816802979, 0.027460314333438873, 0.0005752301076427102, 7.169840614551504e-07, 0.9999524354934692, 0.9999462366104126, 0.999909520149231, 0.007383351214230061, 0.9993162155151367, 0.9999568462371826, 1.6367538364647771e-06, 0.9999847412109375, 0.9999823570251465, 0.07724633067846298, 0.9488680958747864, 0.00010825520439539105, 0.9997887015342712, 6.400437268894166e-05, 0.8723253011703491, 0.9999778270721436, 6.448919521062635e-06, 0.00033846794394776225, 0.0001961665548151359, 6.067429694667226e-06, 0.9909430146217346, 0.9999957084655762, 0.9990183115005493, 0.0016624347772449255, 0.029145818203687668, 2.6075554160343017e-06, 0.9999388456344604, 0.9999736547470093, 0.9990556836128235, 0.9999990463256836, 0.00023258855799213052, 0.0026814700104296207, 0.9999690055847168, 0.0012464700266718864, 2.6116242224816233e-05, 0.9999995231628418, 8.144053572323173e-05, 0.9999685287475586, 0.9998751878738403, 0.9996864795684814, 6.020199725753628e-05, 0.999992847442627, 3.449219002504833e-05, 7.961703522596508e-05, 0.0048298886977136135, 9.672797750681639e-05, 0.9997560381889343, 0.00018172764976043254, 0.9999854564666748, 5.702313865185715e-05, 0.999962568283081, 0.006718214135617018, 0.00042057083919644356, 0.00046805647434666753, 2.3429785869666375e-05, 0.9986585378646851, 0.00026416959008201957, 0.9999967813491821, 0.9971572160720825, 0.9979821443557739, 0.0002938646939583123, 1.4687207112729084e-05, 9.199967462336645e-05, 0.0001438458712073043, 6.170233973534778e-06, 0.021559979766607285, 0.9999793767929077, 0.9998140931129456, 0.00017933548951987177, 0.00010515275062061846, 0.0037291066255420446, 0.9999892711639404, 0.9932752847671509, 0.9947997331619263, 0.9984170198440552, 6.395824311766773e-05, 2.1806634322274476e-05, 7.879863915150054e-06, 0.9999792575836182, 0.9998008608818054, 6.977788871154189e-05, 0.9988454580307007, 2.291531018272508e-06, 0.9997053742408752, 9.697122732177377e-05, 0.9978899359703064, 0.9987303614616394, 0.000227720636758022, 0.9983665347099304, 0.0008934902143664658, 0.039348304271698, 0.9999704360961914, 0.9999376535415649, 0.0002215851127402857, 0.9999501705169678, 0.9999035596847534, 0.9999371767044067, 6.607469458685955e-06, 0.9786596894264221, 0.00013821125321555883, 0.0008418854558840394, 0.003433928592130542, 1.4734203432453796e-05, 0.9687472581863403, 0.9991004467010498, 0.0004824772768188268, 0.9992793202400208, 0.9995251893997192, 0.9997764229774475, 0.9996984004974365, 0.9998630285263062, 0.11412514001131058, 0.5798370242118835, 0.9999804496765137, 0.9998109936714172, 9.164776565739885e-05, 0.999984860420227, 0.9999710321426392, 0.00012068370415363461, 0.9997432827949524, 0.9999918937683105, 0.00010145747364731506, 0.9999761581420898, 0.0001422464702045545, 0.9999986886978149, 0.0010018516331911087, 0.00021681489306502044, 1.0789585758175235e-05, 0.8677994608879089, 0.9998031258583069, 0.9997987151145935, 0.8735582828521729, 0.9998924732208252, 6.998669960012194e-06, 0.9999978542327881, 0.00012574956053867936, 0.0005659773596562445, 2.260406472487375e-05, 0.00019433171837590635, 0.9989068508148193, 0.9991934895515442, 7.340306183323264e-06, 0.0005316310562193394, 0.9999219179153442, 0.999737560749054, 6.277954526012763e-05, 0.9997263550758362, 0.0001748083159327507, 0.0007909893756732345, 7.798905244271737e-06, 3.23232015944086e-05, 0.999755322933197, 0.9999955892562866, 0.9999816417694092, 0.9973273277282715, 3.517055665724911e-05, 0.9999997615814209, 2.89438776235329e-05, 0.0006628198898397386, 0.9994112253189087, 0.999872088432312, 0.0001498061028541997]

#Metricas de validación
Best_acc3_2 = 0.9944444444444445
acc_aux3_2 = 0.9972222222222222
Recall3_2 = 0.989010989010989
Especificidad3_2 = 1.0
Tiempo3_2 = 652.620997428894

######## Base4 ########
#MODELO = VGG11_BN
#Datos perdidas
xx_train4_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train4_2 = [0.07297137321697342, 0.05971058735876907, 0.041223294563867426, 0.04129328572271783, 0.050234419015454654, 0.03840210919762835, 0.038278194829637624, 0.03612133016188939, 0.03906098033910916, 0.035489303094369394, 0.027512242727809482, 0.03316689008547936, 0.032657373466609435, 0.03929531184243567, 0.034720151291953195, 0.02916350911980794, 0.029286477024908417, 0.02634103830765795, 0.03349302974931988, 0.034039990741897515, 0.031149137526014704, 0.03321242285547433, 0.033068782008356515, 0.032651418999389364, 0.03208904502752386]
xx_val4_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val4_2 = [0.01167013429933124, 0.013140358693069882, 0.024539637565612792, 0.02693075570795271, 0.011476840575536091, 0.00756527947054969, 0.00721656365527047, 0.006780354347493914, 0.007878707928789986, 0.007130288415484958, 0.006583439558744431, 0.00638066753745079, 0.005959410137600369, 0.006042699019114177, 0.005886791149775187, 0.006671045223871867, 0.006223342236545351, 0.005634193039602704, 0.006946203609307607, 0.006237090958489312, 0.006914736827214559, 0.00660334759288364, 0.0076108454830116695, 0.0061539381742477415, 0.00961143241988288]
epoch_min4_2 = 12
stop_val4_2 = 0.005959410137600369

#Datos accuracy
acc_train4_2 = [0.9768518518518519, 0.9774691358024691, 0.9824074074074074, 0.9830246913580246, 0.9808641975308642, 0.986111111111111, 0.986111111111111, 0.9864197530864197, 0.9879629629629629, 0.9842592592592593, 0.991358024691358, 0.9885802469135803, 0.9879629629629629, 0.9854938271604938, 0.9879629629629629, 0.9907407407407407, 0.9885802469135803, 0.9907407407407407, 0.9879629629629629, 0.9885802469135803, 0.9882716049382716, 0.9888888888888888, 0.9879629629629629, 0.9873456790123457, 0.987037037037037]
acc_val4_2 = [0.9944444444444445, 0.9972222222222222, 0.9888888888888889, 0.9916666666666667, 0.9944444444444445, 1.0, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9972222222222222]

#Datos curva ROC
best_labels4_2 = [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0]
best_prob4_2 = [0.000310083240037784, 0.9999991655349731, 4.2064613808179274e-05, 1.2871269973402377e-05, 0.00018328266742173582, 0.9999994039535522, 1.356005850539077e-05, 0.9998095631599426, 0.9999593496322632, 0.0008739399490877986, 0.0001642432325752452, 0.0004239166446495801, 0.9991881251335144, 0.0008610383956693113, 6.712936738040298e-05, 7.0058486016932875e-06, 6.3634633988840505e-06, 3.854716487694532e-05, 1.6770465663284995e-05, 0.9999855756759644, 9.764005517354235e-05, 0.9999868869781494, 3.692481914185919e-05, 1.0333500313208788e-06, 5.287391832098365e-05, 0.9998849630355835, 0.9999693632125854, 0.9995299577713013, 0.9999967813491821, 1.0, 0.999783456325531, 0.9999959468841553, 0.862799346446991, 0.9999995231628418, 0.0001930349098984152, 0.999994158744812, 0.9999992847442627, 0.8674312233924866, 0.9999986886978149, 0.00014588635531254113, 0.004378539510071278, 0.995419979095459, 0.999853253364563, 0.0004048352420795709, 0.9999995231628418, 0.996208667755127, 0.9984149932861328, 0.999995231628418, 0.9999998807907104, 0.9999746084213257, 0.9999986886978149, 0.0009240069193765521, 9.275370757677592e-06, 0.00011740376066882163, 0.9999998807907104, 0.9999988079071045, 2.301377435287577e-06, 4.732982415589504e-05, 0.9999990463256836, 0.025068026036024094, 0.9996483325958252, 0.9999911785125732, 0.9999992847442627, 0.999936580657959, 0.0001439537008991465, 0.9999861717224121, 0.00021403054415713996, 5.395959669840522e-05, 0.00011517913662828505, 0.9999997615814209, 0.9999994039535522, 0.99993896484375, 1.8771628447211697e-06, 1.6737583791837096e-05, 0.9999992847442627, 0.9995338916778564, 0.9994269609451294, 0.00016046348900999874, 9.315522220276762e-07, 0.0001965811097761616, 0.003853653557598591, 2.2592965251533315e-05, 0.00010979005310218781, 0.9999990463256836, 7.901573553681374e-06, 0.0004118484794162214, 0.040961943566799164, 5.459021485876292e-05, 0.9997850060462952, 5.517678800970316e-06, 0.9999983310699463, 2.216467692051083e-05, 0.005786758381873369, 0.9999990463256836, 0.0005950763006694615, 0.00019756931578740478, 0.9999939203262329, 0.999778687953949, 1.4935447325115092e-05, 0.9992521405220032, 0.9999953508377075, 6.251307695492869e-06, 0.9999948740005493, 5.9972797316731885e-05, 0.9988447427749634, 0.00012199891352793202, 4.303238893044181e-05, 0.999984622001648, 0.9937939643859863, 7.024252681731014e-06, 0.9999964237213135, 0.9998884201049805, 0.0013603370171040297, 0.9999688863754272, 1.0, 0.999966025352478, 0.9999994039535522, 0.006713544949889183, 0.9999669790267944, 0.999788224697113, 0.9982171654701233, 2.6838091798708774e-05, 0.9999998807907104, 0.9999948740005493, 1.2310507372603752e-05, 0.9727038741111755, 4.63694341306109e-05, 0.9982569813728333, 0.9999985694885254, 3.464075746251183e-07, 3.9545811887364835e-06, 0.00010809201194206253, 7.303212623810396e-05, 2.8761305657099e-06, 0.0013450415572151542, 0.9610337615013123, 0.0002572128432802856, 0.999995231628418, 0.9999990463256836, 0.9999613761901855, 0.00010514163295738399, 0.00043195337639190257, 6.254517938941717e-05, 0.0008792491862550378, 0.0023851615842431784, 0.00026632979279384017, 0.0010014204308390617, 0.9999730587005615, 0.999996542930603, 0.9999134540557861, 5.006646097172052e-05, 0.00021876978280488402, 7.997001375770196e-05, 3.708993062900845e-06, 0.0008818855276331306, 0.001125312177464366, 0.9999970197677612, 9.374026558361948e-05, 1.9010618416359648e-05, 0.9999406337738037, 0.9999715089797974, 0.9999988079071045, 0.0004925578832626343, 1.0935190175587195e-06, 0.7725875973701477, 0.9992885589599609, 0.9725818634033203, 1.0, 0.9999984502792358, 0.9999997615814209, 0.9998935461044312, 7.548744633822935e-06, 3.84426084565348e-06, 0.0024733804166316986, 5.644353950629011e-05, 0.9999963045120239, 0.004324975423514843, 0.0001806993968784809, 0.9999117851257324, 9.833673539105803e-05, 6.197255515871802e-06, 0.9999995231628418, 0.058245714753866196, 0.9996942281723022, 0.999993085861206, 8.45816612127237e-05, 0.9999998807907104, 6.551984552061185e-05, 0.9997994303703308, 1.772106998032541e-06, 0.9999997615814209, 0.00011455402272986248, 0.001870668726041913, 7.839757745387033e-05, 0.9999998807907104, 2.649751513672527e-05, 0.0005939087714068592, 3.967428710893728e-05, 0.999991774559021, 0.9999536275863647, 4.903974331682548e-05, 0.9999926090240479, 9.305957064498216e-06, 3.0558774597011507e-05, 3.1293841402657563e-06, 0.9998451471328735, 0.9999412298202515, 0.004210031125694513, 0.6837795972824097, 9.250471339328215e-05, 3.669622117286053e-07, 2.9098116556269815e-06, 0.9999998807907104, 2.972831680381205e-05, 0.9999891519546509, 0.9999953508377075, 0.9999992847442627, 2.27617820200976e-05, 0.9999994039535522, 9.312509064329788e-06, 0.9999674558639526, 7.352322427323088e-05, 0.0004413505084812641, 0.00011902947881026193, 0.9997711777687073, 0.9999967813491821, 0.018032945692539215, 0.8766646981239319, 0.9999932050704956, 0.9999439716339111, 0.005379528738558292, 1.449393778329977e-07, 1.4710535367612465e-07, 0.9998817443847656, 0.9999998807907104, 0.08087347447872162, 0.002470551524311304, 0.9999971389770508, 0.9999574422836304, 1.4296215340436902e-06, 0.9605207443237305, 1.0, 0.9998112320899963, 0.9999994039535522, 0.9999948740005493, 0.3032195270061493, 0.9999998807907104, 0.9999628067016602, 5.093668369227089e-05, 0.999895453453064, 3.6997887946199626e-05, 1.0224094694422092e-05, 0.00023301206238102168, 0.9999960660934448, 0.00017154848319478333, 0.00017796507745515555, 0.9999995231628418, 0.9999443292617798, 1.452235574106453e-05, 0.0029245102778077126, 0.00030057967524044216, 0.9999945163726807, 1.0, 0.9999116659164429, 2.9608212571474724e-05, 0.009378244169056416, 0.9999916553497314, 2.904153006966226e-06, 0.9999985694885254, 0.999977707862854, 0.9999991655349731, 8.195414920919575e-06, 0.9993452429771423, 1.0360527085140347e-05, 2.284528818563558e-05, 1.0282863513566554e-05, 0.9999963045120239, 8.613174600213824e-07, 3.3899104892043397e-06, 0.9999902248382568, 0.9999988079071045, 5.796681944048032e-05, 0.9999593496322632, 2.3983400865290605e-07, 0.0009291109163314104, 0.9999997615814209, 0.9998800754547119, 1.038252548823948e-06, 0.9998902082443237, 0.9999985694885254, 0.9999812841415405, 0.9999927282333374, 2.9194816306699067e-05, 9.035980474436656e-05, 9.752931509865448e-05, 0.000499409856274724, 0.0009563404018990695, 0.9999988079071045, 2.5930360152415233e-06, 0.9946357607841492, 3.391897553228773e-05, 0.9999998807907104, 0.001891158171929419, 0.00031189987203106284, 0.001323029980994761, 0.9999303817749023, 0.9999922513961792, 0.00011330890993122011, 1.1524177352839615e-05, 0.0026318179443478584, 0.00035358298919163644, 0.0004372856055852026, 0.0016273390501737595, 0.9999418258666992, 0.999990701675415, 0.0023930352181196213, 0.9999973773956299, 0.9999814033508301, 0.9999997615814209, 0.9974660873413086, 0.8346147537231445, 7.618590984748153e-07, 0.9999949932098389, 0.9999974966049194, 0.00010297026165062562, 0.00036091500078327954, 3.4901189792435616e-05, 0.9999935626983643, 0.9999970197677612, 0.9999963045120239, 0.9999686479568481, 0.9999977350234985, 0.999795138835907, 0.9958219528198242, 0.0027670878916978836, 0.00020391595899127424, 1.0, 0.9999511241912842, 0.002532732207328081, 0.00016206649888772517, 3.074710548389703e-07, 0.0002839482913259417, 6.467652565333992e-05, 0.9999991655349731, 0.9999703168869019, 0.9942060708999634, 2.349518945266027e-05, 0.9998619556427002, 0.0002487719466444105, 0.9999983310699463, 0.999969482421875, 0.9686019420623779, 5.523435174836777e-05, 0.00047948709107004106, 0.9999880790710449, 0.9999998807907104, 0.000403831887524575, 0.9999954700469971, 0.00012511851673480123, 0.9999889135360718]

#Metricas de validación
Best_acc4_2 = 1.0
acc_aux4_2 = 1.0
Recall4_2 = 1.0
Especificidad4_2 = 1.0
Tiempo4_2 = 653.6454572677612

######## Base5 ########
#MODELO = VGG11_BN
#Datos perdidas
xx_train5_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train5_2 = [0.06189490643180447, 0.049157645534953956, 0.04125401385036516, 0.039507603277394804, 0.032093811964179264, 0.03274454646456389, 0.028352327571239, 0.03003746290449743, 0.027981544782718023, 0.028104562247976844, 0.02812723407038936, 0.02658806399062828, 0.027494727148685925, 0.03118054770209171, 0.024018481327795688, 0.023097526999535385, 0.02418379055129157, 0.02908211401951166, 0.02221690829337379, 0.025269837346341875, 0.02531811739375562, 0.033909862755257404, 0.030728884786367417, 0.02416550318400065, 0.024159428468089043]
xx_val5_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val5_2 = [0.024848411977291106, 0.004775021473566691, 0.004851618409156799, 0.003741711874802907, 0.010006507900026109, 0.0014890909194946289, 0.004587761229938931, 0.0027413812776406607, 0.001668917801645067, 0.0035080871648258635, 0.0028979119327333237, 0.002052938938140869, 0.001241422692934672, 0.002052759130795797, 0.0016111152039633857, 0.0017072197463777331, 0.0031796084509955512, 0.002463811884323756, 0.003014949957529704, 0.0015326794650819566, 0.0026448526316218907, 0.002554835379123688, 0.0017832885185877481, 0.005230830775366889, 0.002956631282965342]
epoch_min5_2 = 5
stop_val5_2 = 0.0014890909194946289

#Datos accuracy
acc_train5_2 = [0.9802469135802468, 0.9833333333333333, 0.9833333333333333, 0.986111111111111, 0.9879629629629629, 0.9864197530864197, 0.990432098765432, 0.9901234567901235, 0.9873456790123457, 0.9888888888888888, 0.990432098765432, 0.9898148148148148, 0.9907407407407407, 0.9879629629629629, 0.9901234567901235, 0.991358024691358, 0.9901234567901235, 0.9873456790123457, 0.9925925925925926, 0.9888888888888888, 0.9910493827160494, 0.9876543209876543, 0.9885802469135803, 0.9919753086419753, 0.991358024691358]
acc_val5_2 = [0.9916666666666667, 1.0, 0.9972222222222222, 1.0, 0.9972222222222222, 1.0, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9972222222222222, 1.0]

#Datos curva ROC
best_labels5_2 = [0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0]
best_prob5_2 = [5.56704580958467e-05, 0.9980283379554749, 0.999984622001648, 1.5689255405959557e-06, 0.9999961853027344, 0.00019997093477286398, 0.9998681545257568, 0.9999997615814209, 5.89124731220636e-08, 0.9999839067459106, 0.006228448823094368, 0.9999988079071045, 0.9999914169311523, 2.684712399059208e-06, 0.000189315527677536, 0.999962568283081, 8.211649401346222e-05, 0.9999867677688599, 0.9999982118606567, 1.7425380065105855e-05, 8.129915659083053e-05, 0.9987481832504272, 8.097666068351828e-06, 1.6645641153445467e-05, 0.9999996423721313, 5.613578490937243e-09, 4.3396272531026625e-07, 1.067360777540216e-08, 3.774175274884328e-05, 0.9999982118606567, 0.9996670484542847, 0.9993034601211548, 6.746544300995083e-08, 1.313372990807693e-06, 1.1640059938144987e-06, 0.9999891519546509, 1.364984200336039e-05, 3.6234058597983676e-07, 0.9999905824661255, 0.9999988079071045, 5.081237191006949e-07, 0.0026184830348938704, 0.9998936653137207, 0.9997839331626892, 2.2542348233400844e-05, 0.9999980926513672, 0.9999982118606567, 4.5880136894993484e-05, 5.90591162108467e-06, 0.9938663840293884, 1.0, 0.9242105484008789, 7.176859639912436e-07, 9.627323379390873e-06, 0.9999951124191284, 0.999988317489624, 0.005675558932125568, 0.9999856948852539, 0.9999576807022095, 0.0005525654996745288, 8.141275429807138e-06, 1.9455903384368867e-05, 0.00011887907749041915, 0.9999980926513672, 0.9999991655349731, 0.9999964237213135, 0.0017347463872283697, 9.304645232077746e-07, 0.9998002648353577, 1.3363498396756768e-07, 0.9999738931655884, 1.0, 1.8525565792515408e-07, 1.691004217718728e-05, 0.9999984502792358, 0.9999700784683228, 0.9983125925064087, 0.9999717473983765, 0.9999967813491821, 1.242470570872456e-08, 0.9999680519104004, 0.9999829530715942, 0.00015783790149725974, 3.831534201026443e-08, 0.9999995231628418, 0.9999812841415405, 2.0077877707080916e-06, 0.0019307680195197463, 0.9999821186065674, 1.0851854312932119e-05, 0.9998471736907959, 3.6289941363065736e-07, 5.159216016181745e-06, 7.067883984746004e-07, 1.0, 0.9988763928413391, 0.9999192953109741, 0.9999996423721313, 4.0092279959935695e-05, 1.6366789168387186e-06, 4.762620301335119e-05, 0.9985892176628113, 0.9999873638153076, 9.741874151814045e-08, 8.775772641911317e-08, 7.540673686889932e-05, 2.6473268007976003e-05, 1.7565447478773422e-06, 0.9999852180480957, 0.9999849796295166, 0.9999792575836182, 3.313139586680336e-06, 0.00032299585291184485, 1.0, 0.9999979734420776, 0.9999945163726807, 3.797950557782315e-05, 0.002435815753415227, 0.9999293088912964, 0.9999988079071045, 0.9999804496765137, 0.9999967813491821, 1.3932186675447156e-06, 0.9999920129776001, 0.999950647354126, 8.895410428522155e-05, 3.5252310226496775e-06, 0.9999938011169434, 0.9999953508377075, 0.9995943903923035, 0.9995276927947998, 3.655657565104775e-06, 0.9997158646583557, 0.9999040365219116, 0.9999507665634155, 2.691398037768522e-07, 0.9998964071273804, 0.9999991655349731, 0.9974429607391357, 0.9999104738235474, 7.978916300999117e-07, 0.9999994039535522, 0.9999998807907104, 6.33240688330261e-06, 3.415482206037268e-05, 5.870831500942586e-07, 0.9919420480728149, 6.631090400333051e-06, 0.999126136302948, 7.042295692372136e-06, 0.9996968507766724, 0.0001900402276078239, 0.9999531507492065, 0.0001540184603072703, 0.9730501770973206, 3.229629874113016e-05, 1.7324966393061914e-05, 0.9999983310699463, 3.216581899323501e-05, 0.001995678525418043, 0.000561935652513057, 1.2495357850639266e-06, 0.9999996423721313, 0.9999998807907104, 0.9991821646690369, 1.990860255318694e-06, 3.7236333128021215e-07, 0.9985160231590271, 5.1366772822802886e-05, 0.00019053989672102034, 6.413933988369536e-06, 0.999993085861206, 0.9999994039535522, 3.261582648406147e-08, 0.9999226331710815, 0.9999920129776001, 2.2187704473708436e-07, 0.9999904632568359, 4.977580942977511e-07, 3.561482344593969e-06, 2.1627509340760298e-05, 1.131015505961841e-05, 4.7648507461417466e-05, 0.00026576215168461204, 0.9999383687973022, 1.0, 1.2550935934996232e-07, 5.560825115935586e-07, 0.9999964237213135, 0.00016433863493148237, 0.999995231628418, 5.678999173142074e-07, 0.9999388456344604, 0.9999932050704956, 2.8986085453652777e-05, 4.625010490144632e-07, 0.9999901056289673, 7.571929927507881e-08, 0.9999992847442627, 8.539905707038997e-07, 0.9983408451080322, 0.9997841715812683, 0.9999983310699463, 0.9998024106025696, 1.6960349967121147e-05, 0.9871535897254944, 0.9999988079071045, 3.411034565914406e-09, 2.8438800654839724e-05, 2.2918959075468592e-05, 0.9999574422836304, 1.0049693628388923e-06, 0.999962329864502, 7.681906936340965e-06, 7.323535101022571e-05, 3.110859267962951e-07, 0.9999247789382935, 0.9998747110366821, 0.9999990463256836, 4.2468201399969985e-08, 1.0, 0.9996746778488159, 0.0005806858534924686, 0.999822199344635, 1.092107822842081e-06, 1.825870015181863e-07, 0.9999996423721313, 0.0004422521451488137, 7.4198865149810445e-06, 6.320990360109136e-05, 2.5364881821587915e-06, 1.0, 0.00020179321290925145, 1.106683882312609e-07, 9.57732554525137e-05, 0.9916832447052002, 0.0011586424661800265, 0.99998939037323, 0.0018790343310683966, 4.608746621670434e-06, 7.849202120269183e-06, 8.316300409205724e-06, 0.9999995231628418, 7.707956683589146e-06, 2.384315121162217e-05, 0.9999867677688599, 1.8105249182553962e-05, 1.3634647984872572e-05, 0.9999997615814209, 0.00015048528439365327, 5.010356858292653e-07, 0.9999992847442627, 3.5386733543418813e-07, 7.451807846337033e-07, 1.0255941873538177e-08, 0.9999963045120239, 0.9999810457229614, 0.9998607635498047, 0.9999741315841675, 0.9999998807907104, 3.989381980318285e-07, 0.9993540644645691, 0.9999939203262329, 0.0006295665516518056, 0.9999984502792358, 6.113566541898763e-07, 0.8160408735275269, 0.9999865293502808, 0.9999967813491821, 0.9991307854652405, 2.7329940621712012e-06, 6.835413387307199e-06, 6.842469701950904e-08, 0.9998936653137207, 1.4847860256850254e-06, 2.0381673948577372e-06, 0.9999502897262573, 0.9999992847442627, 1.0, 0.00015748071018606424, 0.9999663829803467, 1.6869698811206035e-06, 1.086318306420253e-07, 0.9999970197677612, 5.189774674363434e-05, 0.9612752795219421, 0.9994524121284485, 0.9968288540840149, 4.926247584080556e-06, 0.0001264773600269109, 0.9999990463256836, 0.9999815225601196, 0.00010245995508739725, 0.9999997615814209, 2.650717033247929e-05, 0.9999921321868896, 4.411710960994242e-06, 0.9999997615814209, 0.0007572019821964204, 0.0005396695923991501, 0.999717652797699, 0.0006500289309769869, 0.9999542236328125, 0.9989736080169678, 1.9001663531525992e-05, 0.9999998807907104, 1.0, 1.9758930491775573e-08, 0.9999971389770508, 0.9956207871437073, 0.9999390840530396, 0.9994961023330688, 0.9989652633666992, 8.067976887105033e-05, 0.9999942779541016, 0.014755860902369022, 7.342074468397186e-07, 0.9999996423721313, 0.9999986886978149, 3.5586272133514285e-06, 1.754431622202901e-08, 0.00023960827093105763, 0.0001364468625979498, 0.9982103109359741, 0.000177176741999574, 5.221899118623696e-05, 1.0, 2.2187184640642954e-06, 0.000319650920573622, 0.002931086113676429, 0.9999985694885254, 0.9999963045120239, 4.5839708917583266e-08, 0.9981877207756042, 0.9999984502792358, 0.9999808073043823, 0.9999480247497559, 0.9999988079071045, 1.8435715674058883e-06, 0.9999023675918579, 0.9426858425140381, 0.999245285987854, 1.3535769483041804e-07, 9.461270565225277e-06, 0.9999393224716187, 0.9999995231628418, 2.1794721760670654e-06, 8.313667422044091e-06, 2.142242192348931e-05, 0.999988317489624, 1.0779893955259467e-06, 2.1305679183569737e-05, 3.892472159350291e-05, 1.0036306008487372e-07, 0.9999994039535522, 0.9999309778213501, 8.819502545520663e-05, 0.00021912026568315923, 0.9997853636741638, 7.41059125175525e-08]

#Metricas de validación
Best_acc5_2 = 1.0
acc_aux5_2 = 1.0
Recall5_2 = 1.0
Especificidad5_2 = 1.0
Tiempo5_2 = 662.8431072235107

######## Base6 ########
#MODELO = VGG11_BN
#Datos perdidas
xx_train6_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train6_2 = [0.05536259626164849, 0.039788607121617706, 0.03360710309611426, 0.037803512112593946, 0.0315754704453327, 0.031783555070927116, 0.024713592498022834, 0.019500502290917033, 0.024849967814889955, 0.023654628342684406, 0.02660735026921755, 0.018151468158136178, 0.02903838868678352, 0.025211331570222054, 0.027874165028333665, 0.03141904140788096, 0.020134388029943277, 0.025434802132256237, 0.029538528703980976, 0.02688151838970773, 0.027141049007574716, 0.022652121872445682, 0.026646403011716443, 0.02163862543709484, 0.021121249154761986]
xx_val6_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val6_2 = [1.0285554303063287, 0.004067019869883855, 0.008024439215660095, 0.0071182133422957525, 0.02139061540365219, 0.0038318360845247903, 0.005365423775381512, 0.003992791473865509, 0.0038029366069369844, 0.0030761036607954238, 0.0032017232643233406, 0.003580988778008355, 0.0033677660756640964, 0.0030534116758240593, 0.0036039758059713575, 0.0035315689113405014, 0.0033903055720859105, 0.0026155031389660304, 0.004138038390212589, 0.004743914637300704, 0.0034238388140996295, 0.002653785463836458, 0.003556363450156318, 0.002545694344573551, 0.0027964202066262565]
epoch_min6_2 = 1
stop_val6_2 = 0.004067019869883855

#Datos accuracy
acc_train6_2 = [0.9817901234567901, 0.9854938271604938, 0.9867283950617284, 0.9873456790123457, 0.9882716049382716, 0.9895061728395061, 0.9929012345679012, 0.994753086419753, 0.9916666666666667, 0.9916666666666667, 0.9910493827160494, 0.9916666666666667, 0.9876543209876543, 0.9916666666666667, 0.990432098765432, 0.9895061728395061, 0.9932098765432098, 0.9929012345679012, 0.9895061728395061, 0.9925925925925926, 0.9885802469135803, 0.9922839506172839, 0.9916666666666667, 0.9916666666666667, 0.9919753086419753]
acc_val6_2 = [0.5805555555555556, 1.0, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 1.0, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels6_2 = [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0]
best_prob6_2 = [0.999998927116394, 0.0002202064060838893, 0.0007121642702259123, 0.0001120784945669584, 4.084865940967575e-05, 5.894168862141669e-05, 4.113697286811657e-06, 0.0015666285762563348, 0.0035445811226963997, 0.9984184503555298, 0.9999476671218872, 0.002185253193601966, 0.0005981329013593495, 0.9999910593032837, 0.9985775947570801, 0.9993126392364502, 8.710219844942912e-06, 0.001323207630775869, 1.0558964277151972e-05, 0.9999997615814209, 0.9999982118606567, 0.00013440065959002823, 0.0001995111961150542, 2.6072342734551057e-05, 1.4873970940243453e-05, 0.00011198831634828821, 0.9995062351226807, 0.9988584518432617, 2.7383699489291757e-05, 0.00020717232837341726, 1.937016349984333e-06, 0.0005738971522077918, 0.9979097247123718, 0.0015866878675296903, 0.9975196719169617, 0.04340916872024536, 0.0005316558526828885, 0.0005648448714055121, 0.9999997615814209, 0.999994158744812, 0.999737560749054, 0.9999147653579712, 5.925005825702101e-05, 0.00042958505218848586, 6.003539601806551e-05, 0.9999599456787109, 0.19685867428779602, 1.0, 0.9999995231628418, 0.9999808073043823, 0.00038567293086089194, 0.0004923076485283673, 0.999990701675415, 1.5933071608742466e-06, 0.9999997615814209, 0.999998927116394, 6.535479769809172e-05, 2.831655729096383e-05, 0.9999884366989136, 1.255415645573521e-05, 4.464277026272612e-06, 0.999991774559021, 0.9999908208847046, 6.3952497839636635e-06, 0.9999802112579346, 0.017972907051444054, 0.9998557567596436, 0.0015238733030855656, 0.9999996423721313, 0.9999990463256836, 0.9999997615814209, 0.9992771744728088, 0.9995906949043274, 0.9999998807907104, 0.0004349118971731514, 0.9998936653137207, 9.966636753233615e-06, 9.627907274989411e-05, 5.422112371888943e-05, 0.0005873789195902646, 8.74352699611336e-05, 1.1917780284420587e-05, 0.9998339414596558, 0.9988590478897095, 0.9998173117637634, 0.9999905824661255, 0.0021418954711407423, 0.9999920129776001, 0.000550052965991199, 8.08937766123563e-05, 0.9999997615814209, 0.9995641112327576, 1.2339385648374446e-05, 6.919167208252475e-05, 0.9999791383743286, 0.0005330204730853438, 0.9999997615814209, 0.007599606178700924, 0.0002739574119914323, 0.9999861717224121, 2.2453781639342196e-05, 0.9999746084213257, 3.638576572484453e-06, 0.00013932911679148674, 0.9990532994270325, 0.9999997615814209, 0.9999645948410034, 0.027345385402441025, 0.9999967813491821, 3.7174104363657534e-06, 0.0003530565300025046, 2.7571586542762816e-05, 0.011558584868907928, 0.9999986886978149, 0.9999500513076782, 8.161528057826217e-07, 0.0014189245412126184, 0.9999990463256836, 0.06920968741178513, 0.9999881982803345, 2.267820718770963e-06, 0.0024447841569781303, 0.00021449162159115076, 0.9999891519546509, 6.867317551950691e-06, 0.9998190999031067, 0.001996905542910099, 6.231734732864425e-05, 0.9999979734420776, 0.014492147602140903, 0.00019490807608235627, 0.9999996423721313, 1.5160894690779969e-05, 7.246671884786338e-06, 7.12882392690517e-05, 0.0004859500622842461, 0.9999995231628418, 0.9999750852584839, 0.002295000245794654, 0.9999042749404907, 0.0008845056290738285, 0.999855637550354, 0.9979454874992371, 0.9999853372573853, 7.12802866473794e-05, 0.00012201170466141775, 0.055324405431747437, 3.7017016438767314e-05, 0.0006109605892561376, 0.004194997251033783, 0.00289228861220181, 0.22336386144161224, 0.9999815225601196, 0.9997907280921936, 0.9999985694885254, 0.9999998807907104, 0.9999947547912598, 0.9997920393943787, 0.0574372299015522, 0.9999986886978149, 0.001479610218666494, 0.9986407160758972, 0.9999268054962158, 0.9990407824516296, 0.9977633953094482, 0.9999982118606567, 0.9999459981918335, 0.9999663829803467, 0.9999829530715942, 0.999920129776001, 0.9999061822891235, 0.00018730330339167267, 0.0052230688743293285, 2.7967064397671493e-07, 0.9997904896736145, 0.9999916553497314, 0.9999364614486694, 0.9999891519546509, 0.9999417066574097, 3.8740894524380565e-05, 9.976312139770016e-05, 0.9999914169311523, 0.9948817491531372, 0.9999967813491821, 0.9999957084655762, 6.560027031810023e-06, 0.9986459612846375, 0.00013174026389606297, 0.10495112836360931, 0.9999340772628784, 0.9999191761016846, 6.684376421617344e-05, 7.690024972362153e-07, 0.9999352693557739, 4.8932670324575156e-05, 0.9999536275863647, 1.0, 3.0256555874075275e-06, 4.6344455768121406e-05, 0.0016632328042760491, 0.9999954700469971, 0.0035382190253585577, 0.0026358950417488813, 4.4166652514832094e-05, 2.2121096208138624e-06, 3.362121424288489e-05, 0.999997615814209, 0.9995444416999817, 0.00019174751651007682, 0.9999895095825195, 0.9999949932098389, 0.9286966919898987, 0.9999887943267822, 0.03385109826922417, 0.9999462366104126, 5.933973807259463e-05, 0.999990701675415, 0.13021472096443176, 0.999002993106842, 0.00036926649045199156, 0.9985321760177612, 0.9979264736175537, 0.002459077164530754, 0.9726702570915222, 0.0024126467760652304, 0.001014334731735289, 0.9999918937683105, 0.999987006187439, 9.907153980748262e-06, 0.0003578091273084283, 0.9999985694885254, 8.048760719248094e-06, 0.0022543855011463165, 0.9999550580978394, 6.475271220551804e-06, 0.020957065746188164, 0.00014103511057328433, 0.9999915361404419, 0.9996585845947266, 3.3551557407918153e-06, 0.001327692880295217, 0.0005099275149405003, 0.9999301433563232, 0.9999957084655762, 0.999882698059082, 0.0003784841683227569, 0.9991427659988403, 0.9999926090240479, 6.156221388664562e-06, 0.9998507499694824, 0.9999973773956299, 0.9998507499694824, 2.0090568796149455e-05, 0.000172032683622092, 3.216686309315264e-05, 0.0001487559056840837, 0.9999998807907104, 0.00014266319340094924, 0.00018967890355270356, 0.9999799728393555, 0.9999512434005737, 0.9999796152114868, 0.9999181032180786, 0.0007277381955645978, 0.9999984502792358, 0.999982476234436, 0.00019930300186388195, 0.00046476026182062924, 0.00023476214846596122, 5.3053114243084565e-05, 1.7339111764158588e-06, 0.9999977350234985, 0.00012327791773714125, 0.0017994160298258066, 0.9936057329177856, 0.013389768078923225, 0.9999904632568359, 0.0006235064938664436, 0.9999961853027344, 1.8395412553218193e-05, 0.9999473094940186, 0.9999761581420898, 4.281539804651402e-05, 0.9996752738952637, 0.0007787143695168197, 0.0009777565719559789, 0.9999983310699463, 8.256531145889312e-05, 0.9998026490211487, 0.9989513158798218, 0.9999736547470093, 2.8615659175557084e-05, 0.9999396800994873, 0.9999836683273315, 6.263911018322688e-06, 0.999981164932251, 2.9158059987821616e-05, 0.09013307094573975, 0.9999998807907104, 0.9999616146087646, 0.9999992847442627, 0.9999765157699585, 0.00028940552147105336, 1.0, 1.5059171118991799e-06, 0.9999852180480957, 0.00021737252245657146, 0.00016531493747606874, 4.6128225221764296e-05, 0.9993844032287598, 0.9997946619987488, 0.015210526064038277, 0.0005365635151974857, 0.9999998807907104, 0.9999806880950928, 0.000499468413181603, 6.115499854786322e-05, 2.9913991966168396e-05, 0.9999954700469971, 5.065028290118789e-06, 8.841641101753339e-05, 3.8841404602862895e-05, 0.9999750852584839, 0.9999878406524658, 0.9991024732589722, 0.9999961853027344, 0.9998677968978882, 6.36793129160651e-06, 0.9999830722808838, 0.9999957084655762, 1.0, 0.9999986886978149, 8.31879151519388e-05, 0.9999972581863403, 0.9999960660934448, 5.818048975925194e-06, 7.590432505821809e-05, 0.9997884631156921, 0.9999920129776001, 0.9999927282333374, 1.156877442554105e-05, 0.001854651141911745, 0.9999750852584839, 0.9999992847442627, 0.9990695118904114, 0.0006865897448733449, 0.0011371293803676963, 1.0, 0.005074352025985718, 0.9997304081916809, 0.9999727010726929, 0.9999959468841553, 5.043261353421258e-06, 4.736333266919246e-06, 0.9999713897705078, 0.9999958276748657, 1.215983593283454e-05, 0.9999886751174927, 0.9985854625701904, 0.013453912921249866]

#Metricas de validación
Best_acc6_2 = 1.0
acc_aux6_2 = 1.0
Recall6_2 = 1.0
Especificidad6_2 = 1.0
Tiempo6_2 = 676.5709435939789

######## Base7 ########
#MODELO = VGG11_BN
#Datos perdidas
xx_train7_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train7_2 = [0.045510095872996766, 0.026787020082091106, 0.04253243155501507, 0.03404216783834092, 0.033984646753028584, 0.031502207469793014, 0.03032969515080805, 0.024362780825223453, 0.01690456775235541, 0.0282168269801287, 0.026998895204361575, 0.025386902639343414, 0.031038531228715992, 0.025487424745971776, 0.025661218810228655, 0.023164200948344336, 0.02097664698958397, 0.021762223762494547, 0.018915043973628386, 0.027148964605581614, 0.022004899989675593, 0.025921046457908773, 0.023297190095907375, 0.028888279420358164, 0.027445103651211587]
xx_val7_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val7_2 = [0.017876672744750976, 0.005031372358401617, 0.019686136063602235, 0.01683648352821668, 0.023467224919133715, 0.010399974385897319, 0.00784640320473247, 0.008920124338732825, 0.00717013246483273, 0.011683300799793668, 0.011444649017519421, 0.009801785813437568, 0.010570330835051007, 0.007609774337874519, 0.006905406216780345, 0.007764679359065162, 0.007497564620441861, 0.010738804605272082, 0.009447480407026079, 0.0075315506094031865, 0.007243690143028895, 0.009099746329916848, 0.009000021633174685, 0.008265248437722524, 0.007434643308321635]
epoch_min7_2 = 1
stop_val7_2 = 0.005031372358401617

#Datos accuracy
acc_train7_2 = [0.9851851851851852, 0.991358024691358, 0.983641975308642, 0.9867283950617284, 0.9879629629629629, 0.9888888888888888, 0.9910493827160494, 0.9910493827160494, 0.9944444444444445, 0.9910493827160494, 0.9910493827160494, 0.991358024691358, 0.9885802469135803, 0.9910493827160494, 0.990432098765432, 0.9932098765432098, 0.9941358024691358, 0.9929012345679012, 0.9932098765432098, 0.9895061728395061, 0.9925925925925926, 0.990432098765432, 0.9901234567901235, 0.9888888888888888, 0.991358024691358]
acc_val7_2 = [0.9916666666666667, 1.0, 0.9916666666666667, 0.9944444444444445, 0.9916666666666667, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9944444444444445, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222, 0.9944444444444445, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222]

#Datos curva ROC
best_labels7_2 = [0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
best_prob7_2 = [2.939316073025111e-05, 0.9731760025024414, 0.999924898147583, 0.9995835423469543, 9.687894362286897e-07, 0.9997250437736511, 0.9998989105224609, 0.00019362958846613765, 0.9999991655349731, 0.9999842643737793, 0.998950183391571, 0.9999431371688843, 1.6767618944868445e-05, 0.011782472021877766, 0.6556793451309204, 0.994536817073822, 0.998524010181427, 0.9999995231628418, 0.9999560117721558, 5.416376006905921e-05, 2.5370241928612813e-05, 0.9998887777328491, 1.489764372308855e-06, 6.011651566950604e-05, 1.4384575479198247e-05, 6.044622296030866e-06, 0.9972549080848694, 0.9991589784622192, 0.9999864101409912, 0.00021579954773187637, 0.9996029734611511, 0.9999994039535522, 0.9986481070518494, 0.9999997615814209, 0.9999997615814209, 0.000788527715485543, 0.0007297452539205551, 0.0003168773546349257, 0.9999916553497314, 6.116619624663144e-05, 3.1907660741126165e-05, 0.9996073842048645, 0.00690452428534627, 0.012424182146787643, 0.9992414712905884, 0.9989896416664124, 3.0039257126190932e-06, 7.934712630230933e-05, 3.8883257502675406e-07, 0.9979968667030334, 2.058182644759654e-06, 0.9993801116943359, 0.9999998807907104, 0.00019739815616048872, 0.9999397993087769, 0.9999923706054688, 0.999893069267273, 0.19810859858989716, 0.9999988079071045, 0.9997110962867737, 0.9999972581863403, 1.0089265742863063e-05, 0.9999871253967285, 3.229589856346138e-05, 0.9999730587005615, 0.9999631643295288, 0.9999130964279175, 4.286985858925618e-05, 0.9999033212661743, 5.409845584836148e-07, 0.000124204860185273, 0.00011482350237201899, 0.9999985694885254, 8.178628377208952e-06, 0.99998939037323, 0.00030764739494770765, 6.1886923504062e-05, 0.9996825456619263, 0.9999215602874756, 0.0006609912379644811, 0.0007365302299149334, 0.9990631937980652, 0.9999982118606567, 5.241373628450674e-07, 0.9999784231185913, 2.895461420848733e-06, 0.9999985694885254, 0.9999552965164185, 1.2571937077154871e-05, 0.03153858333826065, 0.9974772334098816, 4.848960452363826e-05, 0.999599039554596, 0.0002212422841694206, 0.00017036958888638765, 0.9999915361404419, 0.0018444492015987635, 0.9957577586174011, 0.0011403063545003533, 0.9999994039535522, 9.356084774481133e-05, 6.10835684256017e-07, 3.7745319332316285e-06, 1.9138515199301764e-05, 0.999985933303833, 0.9999995231628418, 5.595057700702455e-06, 1.3572531315730885e-05, 0.9978137016296387, 0.0011888269800692797, 0.005076644942164421, 3.461137384874746e-05, 0.003930018749088049, 0.00010684344306355342, 0.0001716391125228256, 1.5525454728049226e-05, 0.9992844462394714, 9.933708497555926e-05, 1.3321548976819031e-05, 0.999950647354126, 0.0003224595566280186, 0.9997732043266296, 0.0009381438139826059, 3.5804641811409965e-05, 1.6567520333410357e-06, 0.9999001026153564, 8.079060535237659e-06, 0.9993864297866821, 0.9999972581863403, 0.9730082750320435, 0.0001296289701713249, 0.9999947547912598, 5.590103228314547e-06, 0.04485383629798889, 0.9998347759246826, 0.9998502731323242, 0.9986775517463684, 0.9999675750732422, 0.9999293088912964, 3.6954796087229624e-05, 1.0, 0.9999990463256836, 0.9999998807907104, 4.286233524908312e-05, 2.454020250297617e-05, 0.9999691247940063, 4.288204218028113e-05, 0.9973944425582886, 7.138733963074628e-06, 5.9408364904811606e-05, 3.1515530736214714e-06, 0.9999969005584717, 3.083137926296331e-05, 0.9998844861984253, 0.00013577897334471345, 0.9999933242797852, 1.976685268800793e-07, 0.9999876022338867, 0.9997606873512268, 0.9999399185180664, 0.9999964237213135, 3.759483206522418e-06, 0.9999837875366211, 0.999992847442627, 0.9996956586837769, 0.9970707893371582, 0.9999985694885254, 0.9999973773956299, 0.999998927116394, 0.9997884631156921, 1.5528430594713427e-06, 0.0003188307455275208, 4.4516866182675585e-05, 1.2767392945534084e-05, 9.063837205758318e-05, 0.9999293088912964, 0.0004534855834208429, 0.0003523629857227206, 0.00012151257396908477, 0.00014864049444440752, 0.9998651742935181, 0.9986131191253662, 0.999996542930603, 0.0014219818403944373, 0.9999988079071045, 0.033601004630327225, 0.9998553991317749, 0.9999994039535522, 9.615796443540603e-05, 0.9996147155761719, 2.6893608264799695e-06, 5.0471342547098175e-05, 0.0004686283937189728, 0.9999576807022095, 0.999220609664917, 0.9999881982803345, 0.0003473780816420913, 3.6522706068353727e-06, 2.5176681447192095e-05, 0.005161943845450878, 0.45932435989379883, 7.589208689751104e-05, 5.43841379112564e-05, 0.9999953508377075, 0.0006474475376307964, 0.9999715089797974, 0.000521451816894114, 1.4173116369420313e-06, 0.9999971389770508, 0.9998010993003845, 1.6666756437189179e-06, 1.0, 0.000773300533182919, 1.6147382098097296e-07, 0.9999693632125854, 3.001823927206715e-07, 0.0006957593723200262, 0.9999991655349731, 0.999976634979248, 0.9973172545433044, 3.729743912117556e-05, 0.00021635458688251674, 8.232615800807253e-05, 3.5329423553775996e-05, 4.4917065679328516e-05, 0.9999444484710693, 0.9996662139892578, 0.00018365179130341858, 0.9991995692253113, 2.9334019927773625e-05, 0.0007478312472812831, 0.9998441934585571, 0.00030844539287500083, 0.999993085861206, 2.6410616555949673e-06, 0.9995073080062866, 6.434389797504991e-05, 0.9991089701652527, 0.008360093459486961, 4.418540356709855e-06, 0.9998433589935303, 1.2417273865139578e-05, 1.0, 2.738916919042822e-06, 0.9998922348022461, 0.9872758984565735, 0.9999966621398926, 0.9999901056289673, 0.9999279975891113, 0.0007945414981804788, 0.0007775308913551271, 0.9988952875137329, 0.0021357140503823757, 0.9997428059577942, 0.9999929666519165, 0.9999964237213135, 0.9998656511306763, 0.9991457462310791, 0.9999902248382568, 0.0010732311056926847, 0.0004941918305121362, 4.798296868102625e-05, 1.0, 0.00010824963101185858, 0.999997615814209, 0.9999994039535522, 0.9992530941963196, 0.9999195337295532, 0.9999624490737915, 0.0010959404753521085, 0.006314489990472794, 5.131135185365565e-05, 0.9997639060020447, 0.9999266862869263, 0.00021819294488523155, 1.0897381798713468e-05, 0.9990774393081665, 0.9999560117721558, 0.999998927116394, 0.9999847412109375, 0.06648340076208115, 4.093235111213289e-05, 0.00031494320137426257, 0.0005755879683420062, 0.000972371082752943, 1.2691717756752041e-06, 1.1237616490689106e-05, 0.9999943971633911, 0.002293453784659505, 0.999995231628418, 0.9989045858383179, 0.00011409311991883442, 0.9999985694885254, 4.3699677917174995e-05, 0.0005036350339651108, 0.999976396560669, 1.010403821055661e-06, 0.9999955892562866, 0.9983599781990051, 0.999996542930603, 9.737393702380359e-05, 0.9999877214431763, 0.00013862621563021094, 5.060136754764244e-05, 0.9996539354324341, 2.3414252154907444e-06, 0.9999977350234985, 0.999998927116394, 2.6388024707557634e-05, 0.9999994039535522, 2.265676357637858e-06, 8.283217312055058e-07, 0.9900786876678467, 3.934806591132656e-05, 5.41315930604469e-05, 2.0023016986669973e-06, 0.0010823059128597379, 0.9995645880699158, 0.9999949932098389, 0.9999943971633911, 0.0002417377254460007, 0.9999996423721313, 2.568790478107985e-05, 0.9985307455062866, 0.00013335426046978682, 5.1038012315984815e-05, 0.9999934434890747, 1.3876137927582022e-06, 1.202721591653244e-06, 0.9999982118606567, 0.00025418231962248683, 0.9999269247055054, 0.00012608527322299778, 0.00022725907911080867, 0.9999998807907104, 0.9995982050895691, 2.5037927116500214e-05, 0.9999980926513672, 0.9999979734420776, 1.1531861673574895e-05, 0.00011591293150559068, 5.9776943999168e-06, 0.9999991655349731, 0.0009384165168739855, 0.9997005462646484, 0.99692302942276, 1.5660416465834714e-05, 0.9993084669113159, 0.9999788999557495, 0.9999984502792358, 9.67429400589026e-07, 0.9961934089660645, 0.0006710634916089475, 0.9811680316925049, 0.9990276098251343, 0.001178447506390512, 0.00048743575462140143, 0.9999995231628418, 2.4063312594080344e-05, 0.10330982506275177]

#Metricas de validación
Best_acc7_2 = 1.0
acc_aux7_2 = 1.0
Recall7_2 = 1.0
Especificidad7_2 = 1.0
Tiempo7_2 = 645.1120157241821

######## Base8 ########
#MODELO = VGG11_BN
#Datos perdidas
xx_train8_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train8_2 = [0.048216659134184874, 0.02956438458260195, 0.034303336480149514, 0.026083860086438097, 0.03402310683403486, 0.028794302414228888, 0.033364433178931105, 0.03206121941775451, 0.02727393224651431, 0.02151096808137717, 0.023585062630382583, 0.02997830002396195, 0.03127761905392011, 0.03017044142808443, 0.02450445277823342, 0.026526471218209207, 0.02261469935377439, 0.02571097218695982, 0.021779364283070153, 0.025714263669502586, 0.029293951245001806, 0.024459258641725705, 0.025758992181515988, 0.023866541748061593, 0.028225661942988266]
xx_val8_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val8_2 = [0.0013539940118789673, 0.003039491011036767, 0.0015836909413337707, 0.0009117818540996975, 0.004177306095759074, 0.0019243215521176657, 0.0017416634493403966, 0.0013379746013217502, 0.002058199710316128, 0.0024190773566563923, 0.0015967475043402778, 0.0013910166091389126, 0.0020084929135110643, 0.0017632098661528694, 0.0015456784102651808, 0.001227728029092153, 0.001635988884501987, 0.001551262703206804, 0.0013606232073571946, 0.0021716604630152383, 0.0013780317372745939, 0.0020822119381692676, 0.0017032212681240505, 0.0020828563306066724, 0.0015634861257341173]
epoch_min8_2 = 3
stop_val8_2 = 0.0009117818540996975

#Datos accuracy
acc_train8_2 = [0.9811728395061728, 0.9901234567901235, 0.9867283950617284, 0.9898148148148148, 0.9867283950617284, 0.9895061728395061, 0.9879629629629629, 0.9898148148148148, 0.9895061728395061, 0.9901234567901235, 0.9910493827160494, 0.9885802469135803, 0.9882716049382716, 0.9895061728395061, 0.9901234567901235, 0.991358024691358, 0.9919753086419753, 0.9916666666666667, 0.9932098765432098, 0.9898148148148148, 0.9901234567901235, 0.9907407407407407, 0.9907407407407407, 0.9891975308641975, 0.990432098765432]
acc_val8_2 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels8_2 = [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0]
best_prob8_2 = [0.9999891519546509, 0.9999516010284424, 4.9711645260686055e-05, 0.00026492224424146116, 1.4144748092803638e-05, 0.0002951790811493993, 0.0003128889366053045, 0.0002232247352367267, 0.9999963045120239, 0.9998339414596558, 5.6654720538062975e-05, 2.1887716172841465e-07, 0.0002974855888169259, 0.9998716115951538, 9.143425268121064e-05, 0.9999985694885254, 0.9999961853027344, 0.9999771118164062, 9.975090506486595e-06, 2.183608194172848e-05, 7.453058401551971e-07, 0.9999850988388062, 0.9989933371543884, 0.9999715089797974, 0.999980092048645, 0.00022888217063155025, 5.002083707950078e-05, 1.3857821841156692e-06, 0.0004208784084767103, 0.999962568283081, 1.735072692099493e-05, 0.9999957084655762, 3.1612958991900086e-05, 5.319818683346966e-06, 2.978606426040642e-05, 1.470840402362228e-06, 0.9997394680976868, 0.00022871504188515246, 0.9999972581863403, 0.9999908208847046, 1.7290618416154757e-05, 0.9999951124191284, 0.0031820463482290506, 0.0008115000673569739, 0.9999502897262573, 2.420675264147576e-05, 1.5674718270020094e-06, 2.6655452529666945e-06, 2.8396557354426477e-06, 6.74800539854914e-05, 0.00040402018930763006, 5.5139567848527804e-05, 5.865555431228131e-05, 0.9999802112579346, 1.0763586033135653e-05, 0.0014548426261171699, 2.7659963961923495e-05, 1.99611122297938e-06, 1.4604075659008231e-05, 0.00012714436161331832, 1.4273846318246797e-06, 0.9999686479568481, 0.9996691942214966, 7.29495077393949e-05, 0.999988317489624, 0.9998989105224609, 0.9999998807907104, 0.9999990463256836, 0.9999986886978149, 0.9999916553497314, 0.9999998807907104, 2.1724272301071323e-05, 0.999996542930603, 0.0007391934632323682, 0.00019560809596441686, 7.878007636463735e-06, 0.0008741698693484068, 0.9999983310699463, 0.9999955892562866, 0.9772685170173645, 4.3522450141608715e-06, 0.9998977184295654, 1.0, 0.999998927116394, 0.0003397212421987206, 0.999985933303833, 0.9999692440032959, 8.698979314658573e-08, 3.6664634535554796e-05, 0.9999929666519165, 0.9999656677246094, 0.9999912977218628, 2.4008371838135645e-05, 0.9999996423721313, 0.9999967813491821, 0.9999990463256836, 1.606681507837493e-06, 6.863860562589252e-06, 0.9999849796295166, 0.9995898604393005, 0.9998111128807068, 0.9999717473983765, 0.9999934434890747, 1.6455986042274162e-05, 0.999927282333374, 0.9999971389770508, 0.00023126798623707145, 0.9996789693832397, 0.9992493987083435, 5.4427969189418945e-06, 1.0317942269466585e-06, 5.401721136877313e-05, 0.00701519288122654, 0.9999891519546509, 0.0004425155057106167, 0.9999997615814209, 0.9999464750289917, 0.9831634163856506, 5.178552964935079e-05, 0.9999996423721313, 5.186989437788725e-05, 0.9999111890792847, 0.0014165288303047419, 0.9999998807907104, 0.0002430226159049198, 0.9990863800048828, 0.9999953508377075, 0.00026295188581570983, 0.999998927116394, 0.9999997615814209, 4.5282243377187115e-07, 5.8247256674803793e-05, 2.8393518732627854e-05, 3.5697350540431216e-05, 0.9999014139175415, 0.00020468072034418583, 0.9992966651916504, 6.644662062171847e-05, 0.9999850988388062, 0.9996384382247925, 0.9999895095825195, 1.0, 6.16765973973088e-05, 3.108305827481672e-05, 0.00016029342077672482, 0.9999990463256836, 4.540808731690049e-05, 0.002350785071030259, 0.9998937845230103, 1.0, 0.9999898672103882, 0.00016661480185575783, 0.9999769926071167, 6.16601319052279e-05, 0.0003118335152976215, 0.9999996423721313, 0.98897784948349, 0.000121469704026822, 1.0, 1.0, 0.00017072017362806946, 4.820780668524094e-05, 0.9999998807907104, 0.08007551729679108, 0.9999953508377075, 1.6074391169240698e-05, 0.9999985694885254, 0.9999978542327881, 0.00015098087897058576, 0.9999996423721313, 0.9999986886978149, 3.0745804906473495e-06, 0.9999998807907104, 0.9999990463256836, 0.0012160473270341754, 8.218237780965865e-06, 0.9999393224716187, 0.9999943971633911, 0.9999545812606812, 0.9999964237213135, 0.0002247615484520793, 2.9796236049151048e-05, 0.00044935676851309836, 4.516791523201391e-05, 0.00015607608656864613, 0.999993085861206, 1.8668225720830378e-06, 1.0, 0.00012825243175029755, 6.250283490771835e-07, 7.984930562088266e-06, 0.9999903440475464, 0.0021746226120740175, 0.9999995231628418, 0.9998313188552856, 0.9999982118606567, 5.886899452889338e-05, 0.003399680368602276, 8.428794535575435e-05, 0.9999998807907104, 0.9997170567512512, 0.9999982118606567, 0.999998927116394, 0.9996788501739502, 9.579535799275618e-06, 5.103745934320614e-06, 0.9999934434890747, 3.980723340646364e-05, 0.00655296491459012, 5.709544711862691e-05, 7.795432793500368e-06, 3.419685526750982e-06, 0.9999810457229614, 0.0006582857458852232, 0.0010811681859195232, 0.0005457326187752187, 0.9999815225601196, 0.999985933303833, 0.9999996423721313, 0.999990701675415, 0.9999980926513672, 0.0016217289958149195, 1.6221360965573695e-07, 0.9999990463256836, 0.9999986886978149, 3.167542672599666e-05, 3.8890739233465865e-05, 0.9999969005584717, 0.9999982118606567, 4.724513473775005e-06, 0.9999758005142212, 7.105750228220131e-06, 0.9999996423721313, 0.0004488006525207311, 0.9999637603759766, 0.9999986886978149, 0.999996542930603, 6.06835510552628e-06, 0.9999734163284302, 0.9999994039535522, 2.8829274469899246e-06, 0.0008534708176739514, 1.2520440577645786e-05, 0.9999983310699463, 0.9999978542327881, 0.9997865557670593, 0.9996719360351562, 0.9999735355377197, 0.00033088718191720545, 0.00010739092977019027, 0.01658613048493862, 0.999940037727356, 0.9999951124191284, 0.0007990721496753395, 0.00023149386106524616, 0.00045202940236777067, 0.9999980926513672, 0.9985489249229431, 0.9999923706054688, 0.9999120235443115, 4.870963721259614e-07, 0.9999998807907104, 6.335425860015675e-05, 1.0, 0.01216802652925253, 0.9999840259552002, 0.9930517673492432, 0.00034705715370364487, 0.9999945163726807, 0.003157899482175708, 0.9999990463256836, 0.9999963045120239, 0.9999947547912598, 0.00011584088497329503, 0.011044934391975403, 0.004291324410587549, 0.9999185800552368, 3.312486660433933e-05, 6.013121037540259e-06, 0.9999945163726807, 0.009784458205103874, 0.9999600648880005, 6.970798131078482e-05, 0.010398231446743011, 0.999987006187439, 1.0, 0.0004791243118233979, 0.9999946355819702, 0.9999998807907104, 0.9999902248382568, 0.00733179971575737, 0.99395751953125, 0.9999997615814209, 0.999994158744812, 0.999992847442627, 1.0, 0.0003335371729917824, 9.896475239656866e-05, 3.495915370876901e-05, 0.00010971731535391882, 4.251467908034101e-05, 0.9999946355819702, 1.2106078429496847e-05, 0.00035009265411645174, 0.999652624130249, 1.274521400773665e-05, 5.624810000881553e-06, 0.999998927116394, 0.9998152852058411, 1.0, 6.338102684821934e-05, 2.844065818408126e-07, 0.9999997615814209, 0.9999978542327881, 4.942584928357974e-05, 0.9999772310256958, 1.6169113223440945e-05, 0.9999958276748657, 8.745720151637215e-06, 1.2007068335151416e-06, 0.9993559718132019, 0.9999915361404419, 0.9999514818191528, 1.5224659364321269e-05, 0.9999649524688721, 3.516498691169545e-05, 0.9999995231628418, 0.0001705220201984048, 0.999988317489624, 0.9999963045120239, 2.3433372007275466e-06, 0.999998927116394, 0.9999545812606812, 0.00044128092122264206, 0.9998024106025696, 5.3404681239044294e-05, 1.3852423762727994e-05, 0.00024914415553212166, 6.1920923144498374e-06, 3.060945891775191e-05, 0.9969999194145203, 0.00037185027031227946, 0.000684007303789258, 0.9999994039535522, 0.9999988079071045, 0.9999979734420776, 3.088002267759293e-05, 1.1733676075209587e-07, 1.3307150084074237e-06, 3.0315363801491912e-06, 0.9999974966049194, 0.0001957282511284575, 1.2359124411887024e-05, 0.9999017715454102, 0.9999880790710449, 0.03812136501073837, 1.0, 0.9999642372131348, 4.496669862419367e-05, 1.7587958609510679e-06]

#Metricas de validación
Best_acc8_2 = 1.0
acc_aux8_2 = 1.0
Recall8_2 = 1.0
Especificidad8_2 = 1.0
Tiempo8_2 = 708.6789722442627

######## Base9 ########
#MODELO = VGG11_BN
#Datos perdidas
xx_train9_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train9_2 = [0.040854564051569245, 0.03397639201563082, 0.025052405203934067, 0.032907977019563134, 0.02818229583862387, 0.05037120829025905, 0.031194280235119807, 0.029353002192061623, 0.03227953817925335, 0.022804773405746177, 0.017568776333405648, 0.032604820124896954, 0.026432027750545078, 0.0207617988097079, 0.025745281328757603, 0.023677087685576192, 0.028767047693699966, 0.021389863364122533, 0.022385467690082244, 0.027252775219119625, 0.024150443803748967, 0.02972936784779584, 0.01790717859511022, 0.027924492633269158, 0.02588195104473903]
xx_val9_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val9_2 = [0.010368292033672333, 0.004651658568117354, 0.011431421008374956, 0.007608974642223782, 0.01607162058353424, 0.00876055517130428, 0.008068791362974378, 0.00945262246661716, 0.008374994662072923, 0.007592236581775877, 0.007800215482711792, 0.00800811962948905, 0.0075440969732072614, 0.007988590664333767, 0.007182552417119344, 0.008853334022892847, 0.00771902021434572, 0.007252718591027789, 0.007609030604362488, 0.008555263446436989, 0.00791609055466122, 0.007393739951981439, 0.009844546847873264, 0.008023787372642094, 0.007533143295182122]
epoch_min9_2 = 1
stop_val9_2 = 0.004651658568117354

#Datos accuracy
acc_train9_2 = [0.9827160493827161, 0.9873456790123457, 0.9916666666666667, 0.9879629629629629, 0.9891975308641975, 0.9811728395061728, 0.9885802469135803, 0.9901234567901235, 0.9876543209876543, 0.9910493827160494, 0.994753086419753, 0.9885802469135803, 0.991358024691358, 0.9941358024691358, 0.9901234567901235, 0.991358024691358, 0.9910493827160494, 0.9935185185185185, 0.9929012345679012, 0.9901234567901235, 0.991358024691358, 0.9888888888888888, 0.9941358024691358, 0.9901234567901235, 0.9916666666666667]
acc_val9_2 = [0.9916666666666667, 1.0, 0.9944444444444445, 0.9972222222222222, 0.9944444444444445, 0.9944444444444445, 0.9972222222222222, 0.9944444444444445, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9944444444444445, 0.9944444444444445, 1.0, 0.9972222222222222, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222]

#Datos curva ROC
best_labels9_2 = [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0]
best_prob9_2 = [0.0078287897631526, 0.999998927116394, 0.9999997615814209, 0.0021458144765347242, 0.0001304640172747895, 0.0002502168354112655, 0.9999996423721313, 3.078252848354168e-05, 0.9998780488967896, 0.9803353548049927, 0.9999117851257324, 0.9999783039093018, 1.1114165317849256e-05, 0.9999943971633911, 0.00041014456655830145, 0.9999986886978149, 2.5683224521344528e-05, 0.9999604225158691, 9.179540938930586e-05, 7.919292693259194e-05, 0.0015167132951319218, 0.9998900890350342, 1.3551992196880747e-05, 0.9987642765045166, 0.9998939037322998, 0.9999469518661499, 0.0002562755544204265, 9.62997364695184e-05, 0.0030648603569716215, 0.001724198809824884, 0.0005435518105514348, 0.014795885421335697, 5.7655390264699236e-05, 2.024509967668564e-06, 0.9982770681381226, 0.9628841280937195, 0.001321749179624021, 9.378622053191066e-05, 0.9997884631156921, 0.9998966455459595, 0.9999946355819702, 0.9998445510864258, 0.017604751512408257, 0.999413013458252, 0.9999784231185913, 0.9999685287475586, 0.9999923706054688, 6.808996886320529e-07, 3.721750545082614e-05, 3.479870656519779e-06, 0.9997344613075256, 0.00025816811830736697, 0.9999918937683105, 7.74659711169079e-05, 7.890835695434362e-05, 0.755748450756073, 0.00010953970195259899, 6.64157050778158e-05, 8.515009540133178e-05, 0.9998979568481445, 0.9999746084213257, 0.001122916117310524, 3.2981370168272406e-06, 0.0004483457305468619, 2.6795205485541373e-05, 0.001868490595370531, 0.9999926090240479, 0.00022609459119848907, 2.6445623007020913e-06, 0.0024306250270456076, 0.9999330043792725, 0.00031755477539263666, 0.9999475479125977, 0.004758522845804691, 0.9999892711639404, 4.264372182660736e-05, 0.9999970197677612, 0.00030530410003848374, 0.9999829530715942, 0.0009134596912190318, 0.9999984502792358, 0.999941349029541, 0.00017342239152640104, 0.9999978542327881, 0.9999996423721313, 0.9998371601104736, 5.913925269851461e-05, 1.0420727676319075e-06, 0.9999402761459351, 0.9999973773956299, 0.00018453964730724692, 0.9999501705169678, 0.9998779296875, 0.9999803304672241, 0.038091015070676804, 0.9999995231628418, 0.0005469487514346838, 0.9999803304672241, 0.004078916274011135, 0.9999653100967407, 0.0003630511346273124, 0.9999988079071045, 0.9999328851699829, 0.9999949932098389, 0.00025285626179538667, 5.061474439571612e-06, 0.999976396560669, 0.0003336471854709089, 0.0004871958226431161, 0.9999997615814209, 0.009170428849756718, 7.492693384847371e-06, 1.2276238976483e-06, 0.9998856782913208, 0.9989921450614929, 9.473838872509077e-06, 0.9998695850372314, 0.9997617602348328, 0.9999836683273315, 0.9998841285705566, 0.9999988079071045, 0.9999978542327881, 1.1852008356072474e-05, 1.693210106168408e-05, 0.9499142169952393, 0.9995014667510986, 0.9999395608901978, 0.9999614953994751, 0.9999712705612183, 0.00335309817455709, 0.0003564989019650966, 0.0024960855953395367, 1.3944168131274637e-05, 0.00016303584561683238, 1.0064934031106532e-05, 0.0017316607991233468, 0.9999055862426758, 2.8973342978133587e-06, 0.011875353753566742, 1.785533459042199e-05, 0.9999967813491821, 0.9999936819076538, 1.201250051963143e-05, 2.7292478989693336e-05, 0.9999973773956299, 0.9999972581863403, 0.00020496390061452985, 0.9999979734420776, 2.9433555027935654e-05, 0.9999905824661255, 0.9999940395355225, 0.9999707937240601, 0.999987006187439, 0.001883357996121049, 0.9999985694885254, 0.9999372959136963, 0.9993840456008911, 0.0011534633813425899, 0.001650882768444717, 1.1617187738011125e-05, 0.9992616772651672, 0.9999969005584717, 1.0, 0.9999996423721313, 5.18368324264884e-06, 5.885261344928949e-08, 6.470292282756418e-05, 0.9999914169311523, 0.018669474869966507, 1.1530552910699043e-05, 1.0, 4.038556289742701e-05, 0.005100766662508249, 0.00023771390260662884, 0.9999940395355225, 0.0018567267106845975, 7.899229967733845e-05, 2.900917252190993e-06, 0.00039894753717817366, 4.344778403719829e-08, 6.756543007213622e-05, 0.9999028444290161, 0.999652624130249, 0.9994656443595886, 0.9999990463256836, 0.9977310299873352, 8.391110100092192e-07, 0.004195937421172857, 0.9999686479568481, 0.9999972581863403, 0.999993085861206, 0.9963994026184082, 0.9999879598617554, 1.5375178918475285e-05, 0.00014015940541867167, 0.0011540293926373124, 7.11876928107813e-05, 0.9999892711639404, 0.9996694326400757, 0.999984622001648, 0.9987775683403015, 0.9983781576156616, 0.9958264231681824, 9.481982851866633e-05, 0.0013253554934635758, 0.9999998807907104, 4.985204668628285e-06, 0.09854014962911606, 0.9999731779098511, 0.00015484572213608772, 0.00019824203627649695, 0.999993085861206, 0.9999995231628418, 0.9999998807907104, 0.9999996423721313, 3.2499687222298235e-05, 0.999976396560669, 0.9999662637710571, 0.0014290728140622377, 0.9999932050704956, 0.9999861717224121, 0.9999998807907104, 0.9996887445449829, 0.9999183416366577, 4.345414708950557e-06, 3.663632378447801e-05, 1.9840178993035806e-06, 0.9999382495880127, 1.6735093311126548e-08, 1.2940415217599366e-05, 0.999954104423523, 0.00010335805563954636, 0.782731831073761, 0.9999828338623047, 1.0121068044099957e-05, 0.00020000641234219074, 3.37280253006611e-05, 0.999519944190979, 1.8740127416094765e-05, 0.00032816038583405316, 0.00011282006744295359, 0.0010923565132543445, 0.9999985694885254, 2.536685497034341e-05, 0.9999977350234985, 1.4491143701889087e-05, 0.0011717895977199078, 0.9999992847442627, 7.878362521296367e-05, 0.9999982118606567, 2.437534749333281e-05, 0.9999996423721313, 0.9999591112136841, 0.00020668026991188526, 5.453250651044073e-06, 0.9998481273651123, 0.00025406116037629545, 0.005119382869452238, 0.9281240701675415, 0.9975034594535828, 0.9999876022338867, 0.999977707862854, 0.9996597766876221, 0.99998939037323, 0.007741165813058615, 8.938615064835176e-05, 0.0011153393425047398, 0.00250646541826427, 0.9999998807907104, 0.9999836683273315, 7.169285254349234e-06, 9.828182555793319e-06, 0.9999192953109741, 3.9467079204769107e-07, 0.000456561945611611, 0.9998598098754883, 0.00011605348845478147, 0.003784299362450838, 0.0004936602199450135, 0.9806929230690002, 0.9999762773513794, 3.322341217426583e-05, 0.9999958276748657, 0.0004550657467916608, 0.0015074033290147781, 0.0004797634610440582, 0.0005235288990661502, 4.621778498403728e-05, 0.9999983310699463, 0.9999979734420776, 0.9999676942825317, 0.9999232292175293, 0.9997120499610901, 0.9999935626983643, 0.999993085861206, 0.9999847412109375, 0.006751490291208029, 0.0019678224343806505, 4.289570370019646e-06, 0.999915361404419, 0.999984860420227, 7.728851778665558e-05, 1.6648435121169314e-05, 0.9999716281890869, 8.628486830275506e-05, 0.9999681711196899, 0.9999986886978149, 2.3410020730807446e-05, 0.9999996423721313, 1.5130547581065912e-05, 1.7872353055281565e-05, 2.3453421249541861e-07, 0.9999951124191284, 0.9999995231628418, 0.9999988079071045, 0.00036360271042212844, 0.00019381708989385515, 0.00010704627493396401, 0.3605249524116516, 0.9999961853027344, 0.0002250027609989047, 0.9999868869781494, 0.000817343418020755, 8.257040917669656e-07, 2.9719614758505486e-05, 0.00046111198025755584, 0.9999964237213135, 0.9999523162841797, 0.9999812841415405, 0.9999982118606567, 2.088941437250469e-05, 1.1168617675139103e-05, 0.0005411299061961472, 0.9999866485595703, 0.999987006187439, 1.9516686734277755e-05, 0.9999995231628418, 0.9999833106994629, 0.9999780654907227, 0.000207088582101278, 8.144584171532188e-06, 3.1238952942658216e-05, 0.9713954925537109, 0.9997814297676086, 3.148275209241547e-05, 0.9993582367897034, 1.9227038137614727e-05, 0.9999569654464722, 1.276092234547832e-06, 0.9999903440475464, 0.9998708963394165, 0.999998927116394, 0.00016730825882405043, 0.9999864101409912, 0.9999136924743652, 1.274832015951688e-06, 0.9999765157699585, 0.9999740123748779, 0.8915547132492065, 5.5234198953257874e-05]

#Metricas de validación
Best_acc9_2 = 1.0
acc_aux9_2 = 1.0
Recall9_2 = 1.0
Especificidad9_2 = 1.0
Tiempo9_2 = 710.7266974449158

######## Base10 ########
#MODELO = VGG11_BN
#Datos perdidas
xx_train10_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train10_2 = [0.05201648392795045, 0.029415505830152535, 0.030064805725842348, 0.030411819239825378, 0.031345740429781103, 0.01983500086230996, 0.02612792419982545, 0.023558380684734863, 0.023112426431458673, 0.019083800626757703, 0.028639102423632587, 0.028621363593840304, 0.021560275977776374, 0.01803579266976427, 0.02387358283187136, 0.01719359573390749, 0.019519916397553905, 0.0214933255204448, 0.023898996431150555, 0.01826768691341082, 0.0215862870676282, 0.022736314555377137, 0.020492155095677318, 0.011611975021200415, 0.02581373887297548]
xx_val10_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val10_2 = [0.0022738878925641377, 0.004147000445259942, 0.00045547783374786375, 0.0021883454587724475, 0.00121676590707567, 0.0008000112242168851, 0.0006893701023525662, 0.0006015429894129436, 0.0005152043369081286, 0.0005342466963662041, 0.0007817122671339247, 0.0006180935435824924, 0.000806292560365465, 0.0006314482953813341, 0.0007548418309953477, 0.0007412483294804891, 0.0010393818219502768, 0.0006859676705466376, 0.00084943821032842, 0.0006560083892610338, 0.0005408022138807508, 0.001059796412785848, 0.0006669799486796062, 0.000796486602889167, 0.001243458522690667]
epoch_min10_2 = 2
stop_val10_2 = 0.00045547783374786375

#Datos accuracy
acc_train10_2 = [0.9787037037037036, 0.9885802469135803, 0.9885802469135803, 0.9879629629629629, 0.9888888888888888, 0.9935185185185185, 0.9901234567901235, 0.9907407407407407, 0.9907407407407407, 0.9941358024691358, 0.9901234567901235, 0.9891975308641975, 0.9932098765432098, 0.9932098765432098, 0.9895061728395061, 0.9932098765432098, 0.9944444444444445, 0.9916666666666667, 0.991358024691358, 0.994753086419753, 0.9916666666666667, 0.9929012345679012, 0.9938271604938271, 0.9972222222222222, 0.9901234567901235]
acc_val10_2 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels10_2 = [0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
best_prob10_2 = [4.1397568566026166e-05, 0.9999955892562866, 0.0005400092923082411, 0.9999926090240479, 0.9999213218688965, 8.795495887170546e-06, 8.14502016055485e-07, 0.9999964237213135, 0.9997569918632507, 0.00019838177831843495, 6.833301540609682e-06, 0.9999790191650391, 1.539856839372078e-06, 0.9999973773956299, 0.9999986886978149, 6.591265264432877e-05, 0.9992738366127014, 0.9999979734420776, 0.999909520149231, 1.1328795324061502e-07, 0.0024210503324866295, 0.9999771118164062, 4.0443401303491555e-06, 0.9999853372573853, 5.7329425544594415e-06, 0.9999990463256836, 0.9998490810394287, 2.9513883418985642e-05, 0.00014586270845029503, 7.723658654867904e-07, 0.9999867677688599, 0.9999758005142212, 0.9999951124191284, 2.3299574536395085e-07, 0.9999773502349854, 4.021931545139523e-06, 3.0884898478689138e-06, 0.999992847442627, 3.8962471649028885e-07, 1.387486008752603e-05, 0.9999698400497437, 0.9999936819076538, 5.08688208356034e-05, 3.5566085898608435e-06, 4.916587840853026e-06, 0.999924898147583, 1.4521026059810538e-05, 0.9999799728393555, 4.892394645139575e-05, 0.9999972581863403, 0.9999054670333862, 0.9999911785125732, 0.9999285936355591, 0.9981459379196167, 0.999992847442627, 1.91306335182162e-05, 0.9999891519546509, 2.1403267965069972e-05, 0.9999978542327881, 0.999987006187439, 8.297109843624639e-08, 7.403930794680491e-05, 0.9999791383743286, 1.5853567674639635e-05, 4.86703493152163e-06, 0.9999939203262329, 5.275282433103712e-08, 0.00013388773368205875, 1.543547296023462e-05, 5.283999416860752e-05, 7.212724995042663e-08, 0.9999977350234985, 0.9999649524688721, 0.9999904632568359, 0.9999933242797852, 1.0615242899802979e-05, 9.60239049163647e-05, 0.9999934434890747, 9.982365008909255e-05, 2.3806165700079873e-05, 0.0038196034729480743, 0.9999990463256836, 0.9999452829360962, 0.9999179840087891, 0.00017191757797263563, 0.9998244643211365, 0.9999966621398926, 0.9999644756317139, 1.8884764358517714e-05, 0.0013833795674145222, 0.0011943111894652247, 5.2719649829668924e-05, 0.9999996423721313, 0.9999876022338867, 6.891020802868297e-06, 0.9995532631874084, 0.999984622001648, 1.2208104180899682e-06, 2.863644795070286e-06, 0.9999964237213135, 0.0038655223324894905, 0.9999685287475586, 2.1637224563164636e-05, 0.9999692440032959, 1.0, 0.9986633062362671, 0.9999949932098389, 0.00018247999832965434, 0.9999864101409912, 0.9557032585144043, 0.9999992847442627, 1.2884423085779417e-05, 0.9999895095825195, 1.4626896245317766e-06, 4.040771455038339e-05, 0.9998032450675964, 0.9999951124191284, 3.341555566294119e-05, 1.7517385231258231e-06, 0.9998261332511902, 0.9999785423278809, 0.9999563694000244, 0.9999964237213135, 0.9999547004699707, 0.9998714923858643, 0.002273834776133299, 0.9999697208404541, 0.00024472418590448797, 0.9978225231170654, 0.9999997615814209, 0.0010547396959736943, 0.9998769760131836, 9.994841093430296e-05, 0.0006380206323228776, 0.999998927116394, 9.121370567299891e-06, 0.9999963045120239, 0.9999955892562866, 1.5600595361320302e-05, 0.9999915361404419, 1.0, 3.4767181205097586e-05, 0.9999943971633911, 0.004762584809213877, 8.04977971711196e-05, 0.9998732805252075, 0.9999984502792358, 2.0083342633370194e-07, 0.9999932050704956, 0.9998037219047546, 1.93634036804724e-06, 0.9999945163726807, 0.999763548374176, 2.0492028852459043e-05, 0.9998757839202881, 6.475169357145205e-05, 0.9999450445175171, 9.015031537273899e-05, 3.371631578374945e-07, 1.1817032827821095e-05, 0.9999086856842041, 0.999996542930603, 2.591394877526909e-05, 5.461520777316764e-05, 0.9999932050704956, 0.9995662569999695, 1.8918043451776612e-06, 2.3460976080968976e-05, 1.0675403245841153e-05, 0.00021431008644867688, 0.00015505051123909652, 1.70866242115153e-05, 0.99868243932724, 2.1434297536870872e-07, 1.5863533917581663e-05, 1.004355181066785e-05, 3.666071643237956e-05, 9.414269698027056e-06, 4.965091648045927e-06, 1.3621404093555611e-07, 0.9998905658721924, 8.031570359889884e-06, 1.183473887067521e-05, 0.9998823404312134, 4.839634584641317e-06, 0.9999029636383057, 0.9999642372131348, 0.9914880990982056, 1.0, 3.359365291544236e-05, 3.898167960869614e-06, 5.525725100596901e-06, 9.719635727378773e-07, 0.0018549406668171287, 0.9994913339614868, 0.9999420642852783, 5.717943167837802e-06, 0.9999974966049194, 5.439727601697086e-07, 1.0587202723399969e-06, 0.9997377991676331, 0.9991546869277954, 0.9974230527877808, 0.00012637874169740826, 0.9969654679298401, 0.9999997615814209, 0.9996763467788696, 3.990392360719852e-05, 7.416440894303378e-06, 0.9999470710754395, 0.999984860420227, 0.9999922513961792, 4.15749273088295e-05, 0.9979966282844543, 3.949641541112214e-05, 2.510360354790464e-05, 0.9999985694885254, 0.9999392032623291, 0.9999998807907104, 1.1798158993769903e-05, 1.508366153757379e-06, 1.8217826436739415e-05, 0.00029819406336173415, 6.363806051012943e-07, 0.9999217987060547, 0.9998074173927307, 0.9999997615814209, 2.3877650164649822e-05, 0.9991251826286316, 0.9999849796295166, 0.0038814020808786154, 2.788386700558476e-05, 0.9999972581863403, 7.991177699295804e-05, 1.0952671800623648e-06, 3.5272014429210685e-06, 0.0006680812803097069, 1.4280994946602732e-05, 0.9998853206634521, 0.9999864101409912, 0.9997233748435974, 0.00017010013107210398, 1.748334034346044e-05, 0.0008858969085849822, 6.453404694184428e-06, 0.9998973608016968, 0.0001298553979722783, 7.501506615881226e-07, 0.00011120882845716551, 0.0002203658368671313, 4.2948999180225655e-05, 7.375510176643729e-05, 4.472050568438135e-05, 0.0029880066867917776, 0.9997256398200989, 0.9999862909317017, 0.9999997615814209, 1.689032046670036e-06, 0.9999619722366333, 0.0005707977688871324, 0.9999990463256836, 0.00013934877642896026, 1.638229878153652e-05, 0.9999971389770508, 0.9999988079071045, 0.9999370574951172, 0.9998631477355957, 0.00015822876594029367, 0.000176908855792135, 1.0186672625422943e-06, 7.718409506196622e-06, 0.9999575614929199, 5.492473974300083e-06, 7.850256338315376e-07, 4.9516103899804875e-05, 1.558449184813071e-05, 0.03686121478676796, 0.9999998807907104, 0.9999517202377319, 0.99992835521698, 0.9999346733093262, 0.9999988079071045, 0.9999988079071045, 0.9999991655349731, 0.9999550580978394, 1.7468208170612343e-05, 0.9999597072601318, 0.9999819993972778, 9.649867251937394e-07, 0.9999988079071045, 7.226529851322994e-05, 0.9999991655349731, 2.919125336120487e-06, 0.9997816681861877, 1.664399133005645e-05, 2.8501191991381347e-05, 8.054453815020679e-08, 0.9999961853027344, 0.00016070602578110993, 0.9998225569725037, 0.9999990463256836, 5.127245458425023e-06, 5.4880692914593965e-05, 0.9999997615814209, 0.9998835325241089, 3.955460852012038e-05, 6.579427281394601e-05, 5.565729225054383e-06, 0.9983086585998535, 0.9999735355377197, 2.5953220756491646e-05, 0.9999383687973022, 6.19708998783608e-06, 0.9999922513961792, 0.0006383375730365515, 4.2699393816292286e-05, 1.5240158063534182e-05, 0.99991774559021, 0.0002552964142523706, 7.034739246591926e-05, 0.9999899864196777, 0.9999499320983887, 0.9989424347877502, 0.0013141093077138066, 0.9998658895492554, 0.9999910593032837, 8.422663086093962e-06, 0.9999682903289795, 0.9999997615814209, 1.1656088645395357e-06, 0.9996787309646606, 0.9999548196792603, 0.9999972581863403, 0.00015490212535951287, 1.7740751445671776e-06, 1.0, 0.9999544620513916, 0.999853253364563, 0.9998111128807068, 0.9999877214431763, 3.6380082747200504e-05, 0.0006095006829127669, 8.630503543827217e-06, 5.809316689919797e-07, 0.9999580383300781, 4.3424755858723074e-05, 0.9999972581863403, 1.828037784434855e-05, 0.9998350143432617, 0.9999982118606567, 0.0012225490063428879, 0.9999980926513672, 1.0, 0.999915361404419, 0.9999773502349854, 4.622021094746742e-07, 0.0003087646036874503, 0.00016127093113027513, 0.9999582767486572, 2.0829895220231265e-05]

#Metricas de validación
Best_acc10_2 = 1.0
acc_aux10_2 = 1.0
Recall10_2 = 1.0
Especificidad10_2 = 1.0
Tiempo10_2 = 721.7887086868286

###################################################################

######## Base1 ########
#MODELO = VGG13
#Datos perdidas
xx_train1_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train1_3 = [0.1883650662906376, 0.10726975684548602, 0.06795765984757447, 0.07340994066110364, 0.06314222240521584, 0.04403438725405269, 0.037636765708893904, 0.03535379962420758, 0.03514325590229329, 0.03913016221773477, 0.03717004582285881, 0.03718759330150522, 0.03798824969081231, 0.03938276909383727, 0.0346845424009694, 0.0378894880597974, 0.04046648549444881, 0.04483691247892969, 0.03522920429154679, 0.03470012582378623, 0.03760183208886488, 0.03548051944853347, 0.038211334110410125, 0.04119789514828611, 0.04242942167652978]
xx_val1_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val1_3 = [0.19824944370322758, 0.048795848091443376, 0.07874103370640013, 0.04409844179948171, 0.0509664687845442, 0.03651909844742881, 0.03431270139084922, 0.03214588314294815, 0.03333136555221346, 0.0321089704003599, 0.03236435966359245, 0.03229566944970025, 0.03210100490185949, 0.031990724305311836, 0.031701270739237467, 0.031694987416267396, 0.03171230504910151, 0.03171459486087163, 0.03171315449807379, 0.03171077519655228, 0.031710574196444616, 0.03171043255262905, 0.03171041690640979, 0.03171049033602079, 0.03171055648061964]
epoch_min1_3 = 7
stop_val1_3 = 0.03214588314294815

#Datos accuracy
acc_train1_3 = [0.9253086419753086, 0.9608024691358025, 0.9753086419753086, 0.9734567901234568, 0.9768518518518519, 0.983641975308642, 0.987037037037037, 0.9867283950617284, 0.9854938271604938, 0.9833333333333333, 0.9858024691358025, 0.986111111111111, 0.9867283950617284, 0.9867283950617284, 0.9876543209876543, 0.986111111111111, 0.9839506172839506, 0.9839506172839506, 0.9867283950617284, 0.9867283950617284, 0.9851851851851852, 0.9873456790123457, 0.9854938271604938, 0.9833333333333333, 0.9854938271604938]
acc_val1_3 = [0.9222222222222223, 0.9805555555555556, 0.9638888888888889, 0.9861111111111112, 0.9805555555555556, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334]

#Datos curva ROC
best_labels1_3 = [0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0]
best_prob1_3 = [1.0692103387555107e-05, 0.0007865932420827448, 0.9999977350234985, 0.9999997615814209, 0.00016017028247006238, 0.9999046325683594, 0.9999501705169678, 4.6761095291003585e-05, 0.0003044025506824255, 0.9999996423721313, 0.9955897331237793, 0.0012525978963822126, 0.00010422843479318544, 0.9837214350700378, 0.9999332427978516, 2.974348376483249e-07, 1.0, 0.001791544258594513, 0.011272257193922997, 0.009019355289638042, 0.9999538660049438, 1.0, 1.0, 0.0013327630003914237, 1.2477651125664124e-06, 1.0, 0.0024438099935650826, 4.281768269720487e-05, 6.788784230593592e-05, 1.0, 0.9999665021896362, 0.9999998807907104, 0.9999969005584717, 0.00024228643451351672, 0.0016422589542344213, 0.9982420206069946, 0.9999997615814209, 1.7059620631698635e-06, 0.0006773245986551046, 0.001535141607746482, 0.1633099615573883, 0.00021664935047738254, 0.9999992847442627, 0.9997908473014832, 0.0005514577496796846, 0.002995680086314678, 0.0025954635348170996, 0.9999850988388062, 0.9999721050262451, 0.9972119927406311, 1.3335836229089182e-05, 0.3510052561759949, 0.9997116923332214, 1.0, 7.340390766330529e-06, 0.0015795588260516524, 0.9998452663421631, 0.00024331099120900035, 0.999755322933197, 0.9979515671730042, 0.9996311664581299, 8.700011676410213e-05, 0.2031005471944809, 1.0, 0.9999996423721313, 0.011100342497229576, 0.9999966621398926, 0.00015266188711393625, 0.9996802806854248, 0.04418889805674553, 0.00162614812143147, 1.2843276635976508e-05, 0.9999817609786987, 0.9999488592147827, 0.01049052644520998, 0.03149555251002312, 0.00024570824461989105, 0.001939564011991024, 0.5162085294723511, 0.9999998807907104, 0.37899649143218994, 1.0, 0.9999977350234985, 0.0006761018303222954, 3.910976374754682e-05, 2.7635736842057668e-05, 0.9999401569366455, 0.9999994039535522, 0.0010734805837273598, 0.0001224932784680277, 0.0006335088983178139, 1.0, 0.00026452698512002826, 1.6486490494571626e-05, 0.9942031502723694, 5.8021023505716585e-06, 0.9999996423721313, 0.9999754428863525, 0.001982714282348752, 0.00014212280802894384, 1.0, 1.0, 0.0002995298127643764, 8.794002496870235e-06, 0.9989921450614929, 0.9909250736236572, 0.9771211743354797, 4.6820125135127455e-05, 0.0007447382668033242, 4.581975736073218e-05, 0.0007570328307338059, 0.9999969005584717, 8.118707341964182e-07, 0.00011633703979896381, 1.0887991265917663e-05, 5.006550418329425e-05, 0.0002831875754054636, 0.25248730182647705, 0.8378728032112122, 0.4107871949672699, 0.9980043768882751, 5.776418402092531e-05, 0.001117293955758214, 0.9817894101142883, 0.003141466062515974, 0.001555324997752905, 0.0002175282425014302, 0.0024952127132564783, 0.00043033057590946555, 4.583784902933985e-05, 0.9999915361404419, 0.9999998807907104, 0.9999428987503052, 8.806877303868532e-06, 0.9913496971130371, 0.9998564720153809, 1.0, 0.9999996423721313, 2.839327498804778e-05, 0.9999991655349731, 6.180481250339653e-06, 1.1411432154773138e-07, 0.9999998807907104, 0.999990701675415, 0.0002722249482758343, 0.9997461438179016, 2.6607553991198074e-06, 0.9995430707931519, 0.9993366599082947, 0.9999253749847412, 0.00017278724408242851, 0.000182214513188228, 0.00010760167060652748, 0.9999998807907104, 0.012677758000791073, 0.00030113954562693834, 0.005281536839902401, 3.068460864597e-05, 0.9809468388557434, 1.0, 0.00020689675875473768, 0.11355289816856384, 0.002912549301981926, 1.0, 0.9798747897148132, 0.0728677287697792, 6.245703843887895e-05, 0.029589731246232986, 6.614870926568983e-06, 9.286808904107602e-07, 0.99988853931427, 0.025694403797388077, 0.9999744892120361, 0.001601299736648798, 0.0009181582136079669, 0.9999728202819824, 0.9999898672103882, 0.99272221326828, 3.5956050851382315e-05, 0.0003011257795151323, 0.9999998807907104, 2.2622965900609415e-07, 1.3585105989477597e-05, 0.0013093088055029511, 0.0001695493992883712, 0.999473512172699, 0.00025438208831474185, 0.9960886240005493, 9.615907038096339e-05, 9.220861829817295e-05, 0.0007138422806747258, 0.9998922348022461, 0.9989662170410156, 2.501864219084382e-05, 5.262542254058644e-05, 0.9999970197677612, 0.0013826077338308096, 1.0, 0.9635932445526123, 0.9990684390068054, 0.00020230143854860216, 0.9999686479568481, 2.3535196305601858e-05, 0.9947456121444702, 1.0, 0.9984298348426819, 0.9999990463256836, 0.0017067440785467625, 0.9974240064620972, 0.006271669641137123, 8.170431829057634e-05, 2.876037569876644e-06, 0.0025286551099270582, 2.6544594220467843e-06, 1.0, 1.0, 0.00016334660176653415, 0.002731774467974901, 0.9999704360961914, 1.5276873455150053e-05, 0.9994210004806519, 0.9999994039535522, 0.9999998807907104, 0.0018023966113105416, 1.0, 7.133039616746828e-05, 0.0017014772165566683, 0.9999498128890991, 0.9999724626541138, 0.9999943971633911, 2.7559859518078156e-05, 0.9962182641029358, 8.50895230541937e-06, 0.0012766161235049367, 0.9999353885650635, 0.9999995231628418, 0.0003740558458957821, 0.9999831914901733, 0.9999998807907104, 0.00023996812524273992, 0.0030751912854611874, 0.8892290592193604, 0.9999971389770508, 6.427766948036151e-06, 0.9998363256454468, 0.9999998807907104, 0.7218912839889526, 0.9990342855453491, 0.0019392012618482113, 2.467530248395633e-05, 0.9977144002914429, 0.9999998807907104, 8.264254574896768e-06, 0.999998927116394, 0.9999850988388062, 4.753599569085054e-05, 0.9999899864196777, 0.00017369164561387151, 0.9999761581420898, 0.0005587386549450457, 0.9999643564224243, 0.00045182392932474613, 0.024331744760274887, 0.9997487664222717, 0.9999607801437378, 0.0001634703075978905, 0.9999773502349854, 0.9999876022338867, 1.0, 0.0004010715347249061, 0.00029279134469106793, 7.981146154634189e-06, 1.0, 0.9999997615814209, 0.9999986886978149, 0.00025656743673607707, 0.000102160447568167, 1.0611406651150901e-05, 0.9959737658500671, 0.9997941851615906, 1.0, 1.0, 0.999354898929596, 0.21578824520111084, 0.9999997615814209, 0.999996542930603, 0.00020006629347335547, 0.9999997615814209, 1.0, 0.9999412298202515, 0.9998371601104736, 1.0, 0.0001226709719048813, 1.0, 0.00039312581066042185, 0.06717249751091003, 1.0, 1.0, 0.9999798536300659, 0.9999817609786987, 0.9999938011169434, 0.9999997615814209, 1.0, 0.00021882298460695893, 0.00048565829638391733, 0.0023441268131136894, 0.002264448907226324, 1.0, 0.001653927261941135, 1.0, 0.0003054339613299817, 0.9999992847442627, 0.9999953508377075, 0.00529963755980134, 1.0, 0.04935617744922638, 0.9999281167984009, 1.0, 0.0013520141365006566, 0.0008016438805498183, 1.4619680769101251e-05, 0.0007534500909969211, 0.130085289478302, 0.9999949932098389, 0.9961762428283691, 0.0010182822588831186, 0.9999966621398926, 1.0, 0.9999998807907104, 1.0, 0.00010196697257924825, 0.9353703260421753, 1.0, 1.62298533723515e-06, 0.999991774559021, 0.0014975628582760692, 0.9999986886978149, 0.9999992847442627, 0.0010192722547799349, 0.41693273186683655, 0.00010861069313250482, 1.0, 0.00015484278264921159, 0.9999990463256836, 0.0005246488726697862, 7.895721500972286e-05, 0.7977742552757263, 0.997858464717865, 5.006550418329425e-05, 1.6480753401992843e-06, 0.013173387385904789, 0.00014867876598145813, 0.0006285467534326017, 2.6795076337293722e-05, 0.9999995231628418, 1.0, 0.9999990463256836, 0.0001531799789518118, 0.00010466506500961259, 0.9869424104690552]

#Metricas de validación
Best_acc1_3 = 0.9833333333333334
acc_aux1_3 = 0.9876543209876543
Recall1_3 = 0.9943181818181818
Especificidad1_3 = 0.9728260869565217
Tiempo1_3 = 1273.3918566703796

######## Base2 ########
#MODELO = VGG13
#Datos perdidas
xx_train2_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train2_3 = [0.08136339299840692, 0.06784101785333069, 0.05956001035225244, 0.07246964739428627, 0.04600128247230141, 0.04006650917875914, 0.03871492924697605, 0.03366157450241807, 0.030063684845780148, 0.023953713669821068, 0.029630386498239306, 0.022105004987966866, 0.033162380000691355, 0.026199992847295454, 0.029315198985514816, 0.03254817702152111, 0.030753468142615425, 0.031040657964753516, 0.028637429575125377, 0.03253928118465859, 0.03258029480039337, 0.03144511925401511, 0.03120195468266805, 0.030477485575793703, 0.0250767242025446]
xx_val2_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val2_3 = [0.08039356023073196, 0.055818073120382096, 0.021760163373417324, 0.09135618557532628, 0.0591088506910536, 0.046266812831163406, 0.04015241050057941, 0.03802974373102188, 0.031802240345213145, 0.034572612245877585, 0.034259998384449215, 0.03405762745274438, 0.034042816443575756, 0.03381960954931047, 0.03379687600665622, 0.03378408799568812, 0.03378915786743164, 0.03377749423185984, 0.033763924903339806, 0.03375223990943697, 0.03375164485639996, 0.033751253286997476, 0.03375114285283619, 0.03375105890962812, 0.033750871154997086]
epoch_min2_3 = 2
stop_val2_3 = 0.021760163373417324

#Datos accuracy
acc_train2_3 = [0.9679012345679012, 0.9768518518518519, 0.979320987654321, 0.9728395061728395, 0.9805555555555555, 0.9879629629629629, 0.9864197530864197, 0.9873456790123457, 0.9898148148148148, 0.9932098765432098, 0.9888888888888888, 0.9922839506172839, 0.9879629629629629, 0.9901234567901235, 0.9882716049382716, 0.9879629629629629, 0.9888888888888888, 0.9885802469135803, 0.9882716049382716, 0.9888888888888888, 0.9879629629629629, 0.9885802469135803, 0.9895061728395061, 0.9879629629629629, 0.991358024691358]
acc_val2_3 = [0.9750000000000001, 0.9777777777777779, 0.9861111111111112, 0.9722222222222222, 0.9777777777777779, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9861111111111112, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334]

#Datos curva ROC
best_labels2_3 = [0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
best_prob2_3 = [0.00027025514282286167, 0.5823794603347778, 7.870912668295205e-05, 0.9999967813491821, 1.0, 1.0, 0.9795518517494202, 0.9999998807907104, 0.06792406737804413, 0.9025421738624573, 0.00023814835003577173, 0.9999957084655762, 0.00023508159210905433, 0.000348379515344277, 0.9999808073043823, 0.9923927783966064, 0.0008485265425406396, 9.423103620065376e-05, 0.9983285069465637, 0.00023740676988381892, 0.00047803675988689065, 0.03214462101459503, 1.0, 0.9988524913787842, 0.9999449253082275, 8.207171777030453e-05, 1.339814843959175e-06, 1.0, 0.0005949759506620467, 1.0, 0.9999998807907104, 0.9999834299087524, 0.0010140486992895603, 0.01194752473384142, 0.9999817609786987, 1.5862688087509014e-05, 1.0, 0.00041623192373663187, 0.9999960660934448, 0.0029925398994237185, 0.9935298562049866, 1.0, 0.009809127077460289, 0.006749175488948822, 0.0007351652602665126, 0.9999905824661255, 0.9998207688331604, 0.9999762773513794, 0.7903527617454529, 0.9999088048934937, 0.9999244213104248, 1.0, 1.0, 0.003971566446125507, 0.00043947368976660073, 0.9999933242797852, 0.001435165060684085, 0.999985933303833, 0.9999574422836304, 0.0012328041484579444, 0.00010366680362494662, 0.0004733717069029808, 8.490585059917066e-06, 0.0001334046246483922, 0.00018810151959769428, 1.0, 0.006610355339944363, 4.340687155490741e-05, 0.9999655485153198, 0.001171750482171774, 0.008447703905403614, 0.0009035023394972086, 0.0009082290926016867, 0.9933668971061707, 0.3861991763114929, 1.0582443792372942e-05, 0.00017096323426812887, 0.0009965146891772747, 0.0005341335781849921, 0.9999998807907104, 1.0, 1.0, 0.9999997615814209, 0.999997615814209, 1.8234617527923547e-05, 0.9973905682563782, 0.9911726117134094, 0.004158379510045052, 3.6373076000018045e-05, 1.0, 1.0, 0.009776510298252106, 0.9999600648880005, 0.999127209186554, 0.9992582201957703, 1.0, 0.0014661992900073528, 0.03924849256873131, 0.9999727010726929, 1.0, 0.0021438084077090025, 0.9997867941856384, 0.011582273989915848, 0.0010167999425902963, 0.9999949932098389, 0.9999998807907104, 0.0007832906558178365, 0.01267439778894186, 0.00013644047430716455, 6.994429713813588e-05, 0.9999988079071045, 2.6547240850050002e-05, 0.00026877198251895607, 0.0012818932300433517, 1.0, 0.999998927116394, 0.025569714605808258, 0.0005000634118914604, 0.00028019765159115195, 1.8014692614087835e-05, 1.5126912330742925e-05, 0.9999616146087646, 9.747705917106941e-05, 0.9998660087585449, 0.00303096161223948, 1.0, 0.01088206097483635, 0.9995325803756714, 0.9996126294136047, 0.9998533725738525, 0.9987640380859375, 0.00047702580923214555, 2.8737536013068166e-06, 0.9999972581863403, 0.0008458681404590607, 0.9999998807907104, 0.9999921321868896, 0.0011989572085440159, 0.0004189136379864067, 0.998868465423584, 0.9999979734420776, 3.967708835261874e-05, 4.643293141270988e-05, 1.0, 1.0, 0.4704549014568329, 0.0038335358258336782, 0.9997697472572327, 0.00034490710822865367, 0.9989033937454224, 1.616795634618029e-05, 1.0, 0.0001415113074472174, 0.0017709510866552591, 0.9979998469352722, 0.00030926428735256195, 1.908023659780156e-05, 5.3931722504785284e-05, 0.06542611867189407, 4.571422323351726e-05, 1.0, 0.8353422284126282, 0.0042458404786884785, 0.0003140269545838237, 1.0, 0.998199462890625, 0.9999521970748901, 0.8710501790046692, 1.0, 0.00048684023204259574, 0.9999881982803345, 0.9999998807907104, 0.9999997615814209, 0.9997585415840149, 2.446197504468728e-05, 0.02614918351173401, 0.03784555569291115, 0.9996888637542725, 0.00010375008423579857, 0.9973964691162109, 0.9999996423721313, 0.00019323565356899053, 0.0003634722379501909, 0.9998881816864014, 0.0017072935588657856, 0.00045275173033587635, 0.9998687505722046, 0.0322391577064991, 0.00022867077495902777, 7.549236033810303e-05, 0.0001041286886902526, 0.9999799728393555, 0.9992314577102661, 0.999763548374176, 1.0, 1.0, 0.3464473485946655, 0.0011378618655726314, 0.9999679327011108, 1.0, 0.004348539747297764, 0.0843101441860199, 0.9981074333190918, 0.00026313491980545223, 1.0, 0.004155410919338465, 0.0012190951965749264, 0.9985793828964233, 0.0060539571568369865, 1.0, 0.9999936819076538, 1.0, 0.9997866749763489, 0.9995526671409607, 0.0008989421185106039, 0.0006254384643398225, 7.55497530917637e-05, 1.0, 0.9999997615814209, 0.9985669255256653, 0.9959160685539246, 0.0018922598101198673, 0.999692440032959, 0.0011381757212802768, 0.999984622001648, 0.0008697836892679334, 0.9969329833984375, 0.0003594073641579598, 0.007288897875696421, 0.00025417067809030414, 0.9999852180480957, 0.00017896275676321238, 1.0, 1.0, 0.9981484413146973, 0.9998947381973267, 0.9885919690132141, 2.4823431886034086e-05, 0.0019915332086384296, 0.0013076047180220485, 0.00046274400665424764, 0.9999594688415527, 1.0, 0.0007138622459024191, 0.0016214025672525167, 0.9998230338096619, 0.005889812018722296, 0.0008795801550149918, 4.4918178900843486e-05, 1.0, 0.00187692791223526, 5.5963799240998924e-05, 0.0016588743310421705, 1.0, 0.0015771000180393457, 0.9999957084655762, 0.9999672174453735, 1.3929455235484056e-05, 0.9998216032981873, 3.386181788300746e-06, 0.9903008937835693, 5.6995402701431885e-05, 0.9999717473983765, 0.9999786615371704, 0.9999308586120605, 0.9995810389518738, 0.9999932050704956, 0.9994162321090698, 1.0, 0.9999736547470093, 0.0025923005305230618, 0.0005350252031348646, 0.999977707862854, 0.00010335983097320423, 0.00012185952073195949, 0.00016031743143685162, 0.017130151391029358, 0.9999922513961792, 0.9995900988578796, 0.0024467932526022196, 5.7776363973971456e-05, 0.9999905824661255, 0.9999992847442627, 0.9989476799964905, 0.9999322891235352, 1.0, 0.00018893413653131574, 0.9944868683815002, 2.469086211931426e-05, 5.589035936282016e-05, 0.9992436170578003, 0.9901401400566101, 0.999894380569458, 7.959316462802235e-06, 0.9999994039535522, 0.03688636049628258, 0.9999996423721313, 0.00513858487829566, 0.0001518578501418233, 0.9999996423721313, 0.016715234145522118, 0.9602334499359131, 0.9999998807907104, 0.7198708653450012, 0.006628233473747969, 0.9999088048934937, 0.00011488187010399997, 0.00040079886093735695, 0.9999932050704956, 3.0315935873659328e-05, 0.0004563772235997021, 0.5429555773735046, 0.0001368709054077044, 4.7329103836091235e-05, 0.024660583585500717, 1.0, 0.00036901241401210427, 0.00019598785729613155, 1.0, 0.0012277370551601052, 3.3114254620159045e-05, 0.003266944782808423, 0.002524743555113673, 2.5085896595555823e-06, 0.9893247485160828, 0.99993896484375, 0.9999929666519165, 0.9999864101409912, 0.9999998807907104, 1.0, 0.9999973773956299, 0.00020185632456559688, 0.999765932559967, 0.0014623298775404692, 0.9999998807907104, 0.3588758111000061, 0.9642573595046997, 1.0, 0.9999985694885254, 0.9989357590675354, 0.9991174340248108, 0.9999995231628418, 0.014947591349482536, 4.8070774937514216e-05, 0.9996652603149414, 1.0, 1.4743015526619274e-05, 0.0017663107719272375, 0.999963641166687, 1.0, 8.513677130395081e-06, 0.9998788833618164, 0.007667942903935909, 0.006680385209619999, 1.0, 6.548480450874195e-05, 0.00044753740075975657, 0.00039867835585027933, 0.0011518241371959448, 0.0021837097592651844]

#Metricas de validación
Best_acc2_3 = 0.9861111111111112
acc_aux2_3 = 0.9932098765432098
Recall2_3 = 0.9834254143646409
Especificidad2_3 = 0.9888268156424581
Tiempo2_3 = 1230.772979259491

######## Base3 ########
#MODELO = VGG13
#Datos perdidas
xx_train3_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train3_3 = [0.06392896589305666, 0.05860861797851545, 0.05961954717834791, 0.04144854673448904, 0.05265885388225685, 0.040728137853704854, 0.03620854468993199, 0.030665495255847035, 0.03541599421037568, 0.025220930006033107, 0.021389446269582817, 0.03093185599571393, 0.023762806872894733, 0.028611108108803077, 0.027799637376526257, 0.026508500436205923, 0.027982898258868558, 0.022264312511240995, 0.035505575337159784, 0.02543728866694886, 0.026498376035395965, 0.03146188902082266, 0.02971098415645552, 0.029856364447393535, 0.03051416528997598]
xx_val3_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val3_3 = [0.010240453481674194, 0.012036655180984073, 0.016211356553766463, 0.01725979165898429, 0.022094062467416128, 0.012518270313739777, 0.011931892070505355, 0.011047057973013984, 0.012676562700006697, 0.013941697941886055, 0.013795814166466394, 0.013793864018387265, 0.013532906108432345, 0.013309511873457167, 0.013044621381494734, 0.013036062982347276, 0.013032934731907315, 0.01301947616868549, 0.013008991132179896, 0.013007028566466437, 0.013006977405813006, 0.013007364836004046, 0.013007387518882751, 0.01300747576687071, 0.013007566001680163]
epoch_min3_3 = 0
stop_val3_3 = 0.010240453481674194

#Datos accuracy
acc_train3_3 = [0.9728395061728395, 0.9780864197530864, 0.9780864197530864, 0.9854938271604938, 0.9783950617283951, 0.9858024691358025, 0.9845679012345678, 0.9891975308641975, 0.986111111111111, 0.990432098765432, 0.9929012345679012, 0.9879629629629629, 0.9922839506172839, 0.990432098765432, 0.9898148148148148, 0.9901234567901235, 0.990432098765432, 0.9919753086419753, 0.9864197530864197, 0.9891975308641975, 0.9901234567901235, 0.9867283950617284, 0.9907407407407407, 0.9898148148148148, 0.9888888888888888]
acc_val3_3 = [0.9972222222222222, 0.9944444444444445, 0.9944444444444445, 0.9916666666666667, 0.9916666666666667, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445]

#Datos curva ROC
best_labels3_3 = [0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
best_prob3_3 = [0.0039461636915802956, 0.9998273253440857, 0.9999995231628418, 0.9999997615814209, 0.0001315975096076727, 0.0007013761787675321, 8.498104580212384e-05, 1.0, 0.9999969005584717, 0.9999982118606567, 0.004980048630386591, 1.2850247912865598e-05, 1.0, 1.0, 0.9999997615814209, 1.0, 0.9999958276748657, 9.151798440143466e-05, 1.0, 0.9999619722366333, 0.9999961853027344, 0.0007864112267270684, 0.00020512263290584087, 0.9843364953994751, 0.00013404281344264746, 0.0019178660586476326, 0.9999972581863403, 0.9999949932098389, 2.1379195459303446e-05, 1.0, 0.9999997615814209, 0.004154075402766466, 0.001640350092202425, 0.9999998807907104, 0.0026496322825551033, 0.9999803304672241, 0.9999926090240479, 0.999756395816803, 0.999991774559021, 0.012504322454333305, 0.9999691247940063, 1.6580243027419783e-05, 0.9999244213104248, 0.9999788999557495, 0.9999998807907104, 0.9999970197677612, 0.007050279527902603, 0.9986030459403992, 0.0010965837864205241, 0.9999994039535522, 0.0002898395759984851, 0.0003684736730065197, 0.0002196549321524799, 0.9999991655349731, 3.216597542632371e-05, 0.0001449263363610953, 0.006165082100778818, 0.9793965816497803, 0.931360125541687, 0.9999638795852661, 0.9999868869781494, 0.014164266176521778, 0.9980525970458984, 7.33538981876336e-05, 0.0003658657951746136, 0.00973621103912592, 1.7663785911281593e-05, 1.0, 0.00030832955962978303, 0.00032987145823426545, 0.9999996423721313, 0.013683349825441837, 0.9999933242797852, 0.0005886177532374859, 0.9918364882469177, 0.9998983144760132, 0.9999910593032837, 0.0003338234964758158, 0.9671704769134521, 0.9999862909317017, 0.00441273907199502, 0.9999837875366211, 1.0361447948525893e-06, 0.0002718641480896622, 1.0, 0.9997296929359436, 1.2853385669586714e-05, 0.03866862133145332, 0.9999896287918091, 8.347256880369969e-06, 0.00023924440029077232, 0.0004699242999777198, 0.002307616174221039, 0.7061439156532288, 0.9999933242797852, 3.241033118683845e-05, 0.0015897919656708837, 0.9999954700469971, 1.0, 7.507110422011465e-05, 1.0, 0.9999922513961792, 0.00038721729652024806, 2.1084890988731786e-07, 0.005934201646596193, 5.374452666728757e-05, 0.00020460071391426027, 1.0, 0.9999696016311646, 2.507862518541515e-05, 0.9999991655349731, 1.0, 0.9999995231628418, 0.002628589514642954, 0.00010734669194789603, 2.5495950467302464e-05, 0.9999438524246216, 5.430850706034107e-06, 0.0020839183125644922, 3.7155015888856724e-05, 0.0010991143062710762, 8.314587466884404e-05, 1.2869736565335188e-05, 0.9999921321868896, 1.0, 0.8295862078666687, 3.631802883319324e-06, 0.9998399019241333, 5.51063249076833e-06, 0.9999998807907104, 0.0003476414713077247, 9.488597925155773e-07, 0.00041191341006197035, 6.29465575912036e-05, 0.999970555305481, 0.9999911785125732, 9.662803495302796e-05, 0.9999544620513916, 0.9999653100967407, 0.004816598258912563, 0.0007578318472951651, 0.00016937083273660392, 7.382935109490063e-06, 5.268396853352897e-05, 0.0008188463980332017, 1.0, 2.0184728782624006e-05, 0.0016595612978562713, 0.0006720718229189515, 0.9994174242019653, 0.9951576590538025, 0.0019585364498198032, 1.0, 0.9664500951766968, 1.420525586581789e-05, 0.9999998807907104, 0.0006091080722399056, 1.0, 0.0023149102926254272, 0.9999996423721313, 3.279902375652455e-05, 4.730870841740398e-07, 0.999997615814209, 1.0, 0.02537773735821247, 0.999811589717865, 0.9916022419929504, 0.9999983310699463, 0.9999961853027344, 0.00014927568554412574, 0.998076319694519, 1.0, 1.7190305925396387e-06, 0.003165461355820298, 0.0002598644350655377, 0.9999998807907104, 0.0051124002784490585, 0.00044257668196223676, 7.655792433070019e-05, 1.4300455859483918e-06, 0.03884933516383171, 0.0012516319984570146, 0.08015676587820053, 3.61068632628303e-05, 0.999997615814209, 0.0009493528050370514, 0.9999974966049194, 0.9988704323768616, 0.9992179870605469, 0.9999992847442627, 9.494684491073713e-06, 0.9999997615814209, 1.7915392163558863e-05, 0.003365139476954937, 3.497052239254117e-05, 1.0, 3.4735708140942734e-06, 0.9997619986534119, 0.9999637603759766, 4.4293818064033985e-05, 1.0, 0.9998992681503296, 0.0024409659672528505, 0.9999045133590698, 0.9999568462371826, 5.8953151892637834e-05, 0.9670826196670532, 0.9999938011169434, 5.280387631501071e-05, 0.9999942779541016, 1.0, 1.0, 0.9976795315742493, 1.0, 0.0001679364067967981, 0.0009911215165629983, 0.14665968716144562, 0.995010495185852, 0.961383581161499, 1.8729697615071927e-07, 0.006545177660882473, 9.203577064909041e-06, 0.0034405405167490244, 1.0, 1.1884422747243661e-05, 0.9999927282333374, 0.9999998807907104, 1.0, 0.9999982118606567, 0.9998345375061035, 0.000367167143849656, 1.0, 1.0, 0.00022630572493653744, 0.0009985379874706268, 1.0, 0.00016573601169511676, 0.9999876022338867, 1.0, 0.00020170955394860357, 0.9999669790267944, 2.7644030069495784e-06, 0.6879001259803772, 0.9999991655349731, 0.008354106917977333, 0.9997102618217468, 0.9999785423278809, 6.320978718576953e-05, 0.9999781847000122, 0.999998927116394, 0.00042717528413049877, 0.9524818658828735, 0.00020094438514206558, 6.943849257368129e-06, 0.9997599720954895, 0.0005535238306038082, 0.9999778270721436, 0.9999998807907104, 0.9999997615814209, 0.0031110646668821573, 0.00019964079547207803, 3.432245648582466e-05, 9.96563903754577e-06, 0.9999943971633911, 0.009805306792259216, 6.871818669651475e-08, 0.0015730058075860143, 0.9992662072181702, 0.026662681251764297, 0.9999998807907104, 0.9999783039093018, 5.75118383494555e-06, 1.1586399750740384e-06, 0.003122714813798666, 0.9963772892951965, 1.0, 0.003324332181364298, 0.9954448938369751, 0.9997256398200989, 1.9187747966498137e-05, 0.4373992681503296, 0.9140996932983398, 0.9999972581863403, 0.9999998807907104, 0.9999996423721313, 0.9999829530715942, 0.00530123570933938, 2.7817573936772533e-05, 0.00035550250322557986, 2.3979013349162415e-05, 6.605815769944456e-07, 0.9992212057113647, 0.994293749332428, 2.4692481019883417e-06, 0.0006087976507842541, 0.9999998807907104, 0.9999959468841553, 0.0006937849102541804, 6.256087362999097e-05, 0.0001533939503133297, 0.9964924454689026, 0.9707164168357849, 1.0, 0.9998062252998352, 1.0, 0.0006670334259979427, 0.00022527776309289038, 0.9999997615814209, 0.9999954700469971, 6.37166067463113e-06, 0.9979592561721802, 1.0, 0.9999701976776123, 0.0005066668963991106, 7.924823876237497e-06, 1.0, 1.0, 0.9999998807907104, 6.2293415794556495e-06, 0.9999864101409912, 0.00018230834393762052, 0.0005514257354661822, 0.9999873638153076, 1.0, 0.00018238730262964964, 0.0005769407725892961, 1.0, 0.9998449087142944, 0.00032862904481589794, 0.9999269247055054, 0.00040236610220745206, 1.0, 0.25029245018959045, 0.9999949932098389, 0.00014505977742373943, 1.9729270661628107e-06, 0.9999983310699463, 0.00019616525969468057, 1.3408528047875734e-06, 0.03420494124293327, 0.0060510458424687386, 0.9980722665786743, 7.665781822652207e-07, 1.0505009413463995e-05, 0.004129860084503889, 1.0, 0.9999821186065674, 0.9979827404022217, 0.000753976171836257, 0.999946117401123, 0.0001954615581780672, 0.9999734163284302, 1.6193018382182345e-05, 0.9999951124191284, 0.9999998807907104, 0.9996452331542969, 0.9990395903587341, 0.0001447586837457493, 1.4071932127990294e-05, 0.021542062982916832]

#Metricas de validación
Best_acc3_3 = 0.9972222222222222
acc_aux3_3 = 0.9972222222222222
Recall3_3 = 0.994475138121547
Especificidad3_3 = 1.0
Tiempo3_3 = 1233.865479707718

######## Base4 ########
#MODELO = VGG13
#Datos perdidas
xx_train4_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train4_3 = [0.0667707494839474, 0.0488523415301317, 0.04623561291782944, 0.04713629475898213, 0.04165549258942957, 0.034707479381266934, 0.03079895884902389, 0.030601494014263153, 0.030854441087555, 0.03169472487988295, 0.02728383815214958, 0.030834028676704123, 0.023928296906344685, 0.02875398138055095, 0.030214704168431552, 0.02616679799961455, 0.02639706997591772, 0.0251212045090434, 0.026318407849765118, 0.023534146972276546, 0.022235651626998996, 0.022426730431156394, 0.023851705857633073, 0.026983679251538384, 0.026585012885891362]
xx_val4_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val4_3 = [0.014309870700041452, 0.04039894938468933, 0.028942701054943933, 0.032665750715467665, 0.023908360881937873, 0.014394627925422456, 0.014785001013014052, 0.013502066830794017, 0.014427090519004399, 0.015793160100777946, 0.015796828601095412, 0.015622089968787299, 0.015552963316440582, 0.015560629301600987, 0.015362449983755747, 0.015361431737740834, 0.015350964665412904, 0.015348137749565973, 0.015348189738061693, 0.01534219053056505, 0.015342122978634305, 0.015342427790164948, 0.015342432426081763, 0.015342231591542561, 0.015342206921842363]
epoch_min4_3 = 0
stop_val4_3 = 0.014309870700041452

#Datos accuracy
acc_train4_3 = [0.9762345679012345, 0.9787037037037036, 0.9842592592592593, 0.9842592592592593, 0.9830246913580246, 0.9876543209876543, 0.987037037037037, 0.9895061728395061, 0.9888888888888888, 0.9901234567901235, 0.9919753086419753, 0.9879629629629629, 0.9901234567901235, 0.9895061728395061, 0.9873456790123457, 0.9910493827160494, 0.9882716049382716, 0.9901234567901235, 0.990432098765432, 0.9907407407407407, 0.9916666666666667, 0.9916666666666667, 0.9922839506172839, 0.9895061728395061, 0.9901234567901235]
acc_val4_3 = [0.9916666666666667, 0.9861111111111112, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667]

#Datos curva ROC
best_labels4_3 = [1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
best_prob4_3 = [0.9997871518135071, 0.9999988079071045, 0.0001513273746240884, 0.9999905824661255, 0.0038774178829044104, 4.666002496378496e-05, 0.999945878982544, 0.0004280873981770128, 0.999964714050293, 0.9999979734420776, 0.9997031092643738, 0.0015291905729100108, 0.9999995231628418, 0.9996918439865112, 0.9999998807907104, 0.0007885126979090273, 0.9998939037322998, 0.9999853372573853, 0.00690035056322813, 4.256773081579013e-06, 0.9999977350234985, 1.0, 0.9999979734420776, 0.00011303975770715624, 0.00013842927000951022, 0.9999972581863403, 0.9999996423721313, 0.9999995231628418, 1.0, 7.285920582944527e-05, 0.9999613761901855, 0.9999994039535522, 3.4989120649697725e-06, 2.3517830413766205e-05, 0.030125722289085388, 0.9999831914901733, 0.9999334812164307, 0.9999996423721313, 0.9999823570251465, 0.000605379173066467, 0.9999624490737915, 0.0019478969043120742, 1.0, 0.00032651296351104975, 4.4260174036026e-05, 0.009668116457760334, 0.9999947547912598, 0.00016575561312492937, 0.9999856948852539, 0.999901294708252, 0.9451457262039185, 0.9954402446746826, 0.0012820392148569226, 8.549214544473216e-05, 0.9992994070053101, 0.9997439980506897, 6.340528216242092e-06, 0.000132067667436786, 0.9999839067459106, 0.9954572916030884, 0.0009187248069792986, 0.999941349029541, 0.9999589920043945, 0.9999672174453735, 0.9999927282333374, 0.9999418258666992, 0.9994718432426453, 0.9766085147857666, 1.7892358300741762e-05, 0.9999998807907104, 0.9999984502792358, 0.0033931774087250233, 0.00019224142306484282, 5.894337664358318e-05, 0.9999381303787231, 0.9999958276748657, 0.9999427795410156, 1.0, 0.9999921321868896, 0.9999557733535767, 0.9979938268661499, 5.447709554573521e-06, 1.0, 0.9999476671218872, 0.011134019121527672, 0.9999946355819702, 0.06894728541374207, 0.9999901056289673, 0.9999996423721313, 0.9884577393531799, 0.0004043735098093748, 0.9999997615814209, 0.9999951124191284, 0.9999935626983643, 0.9992995262145996, 0.00012309277371969074, 3.196485704393126e-05, 0.00039926788304001093, 0.015647858381271362, 0.9999971389770508, 0.9992265701293945, 0.9999997615814209, 0.9999657869338989, 0.003146496834233403, 0.01723971590399742, 6.521764134959085e-06, 0.9999817609786987, 0.009206168353557587, 3.8533307815669104e-05, 3.1685638532508165e-05, 1.5922469174256548e-06, 2.732500979618635e-05, 0.001023447490297258, 0.057524267584085464, 0.9998409748077393, 3.946078322769608e-06, 0.00010714093514252454, 0.0006959304446354508, 0.999913215637207, 0.9996780157089233, 1.0, 0.9998854398727417, 0.0003399029956199229, 0.00043964566430076957, 3.6280121094023343e-06, 0.0007169194286689162, 4.550179801299237e-05, 0.0004974446492269635, 0.004470423329621553, 0.000412376452004537, 0.999976396560669, 0.9999959468841553, 0.0003475748235359788, 0.9748728275299072, 4.581066968967207e-05, 0.00012440055434126407, 0.00011333051224937662, 0.9999898672103882, 0.9999833106994629, 0.9999825954437256, 0.0005027376464568079, 0.0008257498848251998, 0.025358933955430984, 5.869316009921022e-05, 0.9998879432678223, 0.0010925172828137875, 0.00011351210559951141, 0.9989280104637146, 0.9999990463256836, 0.00028327692416496575, 0.0065020602196455, 0.0029763055499643087, 0.9891586303710938, 0.9999833106994629, 0.04492693394422531, 0.9989450573921204, 5.552777656703256e-05, 0.012631633318960667, 2.5754621674423106e-05, 0.00027585210045799613, 0.9999972581863403, 0.9998949766159058, 0.9999990463256836, 0.0005500917905010283, 0.9999332427978516, 0.9965440630912781, 0.0005299750482663512, 6.276871135924011e-05, 0.0016109385760501027, 1.0, 0.0008946679881773889, 5.676368891727179e-05, 1.2591685845109168e-05, 0.9992177486419678, 0.0125831114128232, 0.9999971389770508, 0.0007729306817054749, 0.999005138874054, 0.9846485257148743, 7.200630352599546e-05, 0.0001326540659647435, 5.830905865877867e-05, 0.590820848941803, 0.9999668598175049, 2.9290580641827546e-05, 0.9999990463256836, 1.077642446034588e-05, 0.0004513585881795734, 0.021938910707831383, 5.051891639595851e-05, 0.9999983310699463, 0.9678143262863159, 0.999983549118042, 0.00023463796242140234, 0.0036662158090621233, 0.9999991655349731, 7.359842857113108e-05, 0.0056533426977694035, 0.00020272449182812124, 0.00026797724422067404, 0.5415941476821899, 1.0, 0.002114811446517706, 0.9999951124191284, 0.9999949932098389, 1.0, 0.00026527862064540386, 0.9744759798049927, 0.9999545812606812, 0.9998509883880615, 0.0006672349409200251, 9.523411426926032e-05, 0.9999940395355225, 0.998329222202301, 5.111847713124007e-05, 1.0, 0.9999929666519165, 0.9999805688858032, 0.9999957084655762, 0.9911265969276428, 9.160468835034408e-06, 1.3417580703389831e-05, 0.02405264601111412, 0.9965558052062988, 0.9935274720191956, 0.9997233748435974, 0.9999752044677734, 0.64276522397995, 0.0001336536806775257, 0.0005616971175186336, 0.0006729540764354169, 2.2173053366714157e-06, 0.999991774559021, 0.0007993535837158561, 0.0009404392912983894, 4.142519628658192e-06, 0.0008656781865283847, 0.9996287822723389, 0.0025180745869874954, 1.0, 3.356495653861202e-05, 4.822376013180474e-06, 0.9991868138313293, 0.0007142317481338978, 0.0036174103152006865, 0.9999668598175049, 0.9941615462303162, 0.9999767541885376, 0.000135078516905196, 0.0005691120750270784, 2.7812870939669665e-06, 6.172047869767994e-05, 0.9999978542327881, 0.9999911785125732, 0.999998927116394, 1.9174869521521032e-05, 0.999920129776001, 0.00359680550172925, 0.00191916071344167, 9.796983795240521e-05, 0.9999974966049194, 0.9972773194313049, 0.9999997615814209, 0.006436669733375311, 0.0004674300434999168, 0.9949083924293518, 0.9998835325241089, 0.00015814686776138842, 1.0, 0.9999978542327881, 0.9999998807907104, 2.274559301440604e-05, 4.665699816541746e-05, 5.542423878068803e-06, 0.9999300241470337, 4.750318839796819e-05, 0.00028634583577513695, 0.9999979734420776, 0.000141788725159131, 0.9999874830245972, 0.9999608993530273, 0.9999997615814209, 9.834286174736917e-06, 0.9999687671661377, 0.9999921321868896, 1.069180689228233e-05, 0.3873583674430847, 0.0012312551261857152, 0.9999746084213257, 0.9979456067085266, 0.9999568462371826, 0.9999994039535522, 0.00014340515190269798, 1.0, 0.004046017304062843, 0.08371560275554657, 0.0009084282792173326, 6.149082878437184e-07, 0.9967706203460693, 0.9999996423721313, 0.001812448026612401, 0.9999998807907104, 0.9546025395393372, 0.0045336391776800156, 0.999940037727356, 0.0001520178047940135, 0.00071917031891644, 0.00739897508174181, 0.0022493780124932528, 0.03620355576276779, 0.0001089199649868533, 0.00010647464659996331, 0.0009484319598414004, 0.00015971931861713529, 0.002619019476696849, 0.0013683268334716558, 0.9999998807907104, 0.9964488744735718, 0.0015431327046826482, 0.9969677329063416, 1.5963765918058925e-06, 1.0, 0.00025780085707083344, 0.9999967813491821, 0.0009478272404521704, 1.0, 0.06877033412456512, 0.999980092048645, 0.999970555305481, 7.718333654338494e-05, 0.003968185745179653, 0.4389491677284241, 0.9999996423721313, 0.9999992847442627, 0.9999978542327881, 0.9999998807907104, 0.9999991655349731, 0.9999997615814209, 0.29411447048187256, 0.9999312162399292, 0.9816848635673523, 0.9808921217918396, 0.9999983310699463, 0.008580024354159832, 0.00013732739898841828, 0.0006748702726326883, 0.0015887467889115214, 0.0009222670923918486, 0.00036258590989746153, 0.0023352582938969135, 0.9999862909317017, 0.01386570930480957, 5.3359883168013766e-05, 1.0, 1.0, 4.565693234326318e-05, 6.362488784361631e-05, 0.9998434782028198, 0.00185195985250175, 0.0002708846004679799]

#Metricas de validación
Best_acc4_3 = 0.9916666666666667
acc_aux4_3 = 0.9922839506172839
Recall4_3 = 0.994413407821229
Especificidad4_3 = 0.988950276243094
Tiempo4_3 = 1234.0375764369965

######## Base5 ########
#MODELO = VGG13
#Datos perdidas
xx_train5_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train5_3 = [0.05962997089013641, 0.051056758682301015, 0.06507716116345959, 0.049312732966593754, 0.047713918772376615, 0.10718719985014126, 0.04538406106056991, 0.037420418205452555, 0.035761357678307425, 0.03374902259236501, 0.0320772849575237, 0.03612143218885233, 0.035061450164627146, 0.033102552381194666, 0.03840052738417814, 0.03475753769830421, 0.03442316601673762, 0.03175762179825041, 0.03597851925426059, 0.030285895312273943, 0.03728422500469066, 0.02845562595276185, 0.031687342513490604, 0.035491327499901804, 0.025260037422916035]
xx_val5_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val5_3 = [0.020820810894171395, 1.157847151491377, 0.20670913259188334, 0.012461902697881062, 0.21976925747262108, 0.02671195169289907, 0.024350242482291326, 0.018020876579814486, 0.012231024023559358, 0.012275021109316085, 0.012188137239880032, 0.011821818517314063, 0.01147563076681561, 0.011166655520598093, 0.010915525257587433, 0.01091189401017295, 0.010907084991534552, 0.010911962803867128, 0.010905626830127505, 0.01090011265542772, 0.010900009175141652, 0.010899787810113694, 0.010899632506900363, 0.010899543596638573, 0.010899406754308276]
epoch_min5_3 = 3
stop_val5_3 = 0.012461902697881062

#Datos accuracy
acc_train5_3 = [0.9762345679012345, 0.9811728395061728, 0.9768518518518519, 0.9839506172839506, 0.9824074074074074, 0.9660493827160493, 0.9827160493827161, 0.9858024691358025, 0.9873456790123457, 0.9864197530864197, 0.9867283950617284, 0.987037037037037, 0.9867283950617284, 0.9882716049382716, 0.987037037037037, 0.9864197530864197, 0.9854938271604938, 0.9891975308641975, 0.9851851851851852, 0.990432098765432, 0.9864197530864197, 0.9888888888888888, 0.9876543209876543, 0.9882716049382716, 0.9910493827160494]
acc_val5_3 = [0.9888888888888889, 0.8416666666666667, 0.9333333333333333, 0.9972222222222222, 0.9361111111111111, 0.9888888888888889, 0.9916666666666667, 0.9916666666666667, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222]

#Datos curva ROC
best_labels5_3 = [0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]
best_prob5_3 = [5.8701607486000285e-05, 0.9999985694885254, 0.002582259476184845, 1.0, 2.512693390599452e-05, 0.9998501539230347, 0.9999995231628418, 3.4355230127403047e-06, 0.00011372825974831358, 0.0069094011560082436, 0.999903678894043, 0.9999926090240479, 0.9998828172683716, 1.0651582442733343e-06, 0.9999991655349731, 0.9999991655349731, 1.7289350580540486e-05, 0.9999496936798096, 0.00684009026736021, 0.9998966455459595, 3.2618910950077407e-07, 6.682600997010013e-06, 0.9999995231628418, 0.9999728202819824, 6.855597894173115e-05, 0.00022990706202108413, 0.0028511604759842157, 1.0833238093255204e-06, 0.9999730587005615, 1.1617597010626923e-05, 0.9999583959579468, 0.0002131924411514774, 0.00023660210717935115, 0.01064586453139782, 0.007387587800621986, 0.0005608211504295468, 2.871033530027489e-06, 0.002046426758170128, 0.0006734728813171387, 0.00011669969535432756, 0.013859742321074009, 0.9998856782913208, 0.999862790107727, 6.568594835698605e-05, 0.999994158744812, 0.9999804496765137, 5.9654953474819195e-06, 0.9999994039535522, 0.00020532771304715425, 0.9360881447792053, 7.554853073088452e-05, 0.0029473071917891502, 9.810939900489757e-07, 6.658249731117394e-06, 0.9999990463256836, 9.935451089404523e-05, 0.0071124117821455, 0.9999687671661377, 4.020711912744446e-06, 6.114930783951422e-08, 1.0, 0.0019618659280240536, 0.9999957084655762, 0.9999983310699463, 0.0005371877341531217, 3.670864998639445e-06, 0.25524842739105225, 0.0008988385088741779, 0.053041547536849976, 0.9999967813491821, 2.689158463908825e-05, 4.538981374935247e-05, 2.4222432330134325e-05, 9.797121265364694e-07, 0.9947668313980103, 0.9999998807907104, 0.9999115467071533, 0.9999995231628418, 1.0, 0.9999974966049194, 0.9992031455039978, 0.0001188712558359839, 2.031742587860208e-05, 0.999955415725708, 2.712656532821711e-05, 0.00023797173344064504, 0.999957799911499, 0.9999849796295166, 0.9998949766159058, 1.0, 0.001115395687520504, 0.019176872447133064, 0.9999996423721313, 0.00024267622211482376, 0.9999998807907104, 6.743611720594345e-06, 0.9999990463256836, 2.5136764634225983e-06, 2.0300052710808814e-05, 0.9999998807907104, 0.999998927116394, 0.9851101636886597, 0.0009966285433620214, 7.685212040087208e-06, 0.999643087387085, 0.9999737739562988, 0.9705877900123596, 4.82404129797942e-06, 0.0016190145397558808, 0.000237464060774073, 0.06066714972257614, 0.004866761155426502, 9.7123091109097e-05, 0.9999955892562866, 0.9999970197677612, 0.00028127143741585314, 0.9997966885566711, 0.000691920577082783, 7.145853260226431e-07, 5.319037882145494e-06, 0.7625242471694946, 0.9999815225601196, 0.004175832495093346, 0.9998996257781982, 0.9999972581863403, 0.0023735191207379103, 0.9998230338096619, 0.9999969005584717, 0.9999852180480957, 0.9999978542327881, 2.9935737984487787e-05, 0.007417800836265087, 0.0010677288519218564, 0.00017022539395838976, 0.9999957084655762, 1.0, 1.0423907497170148e-06, 0.0004404609790071845, 0.999967098236084, 0.9999985694885254, 0.9999998807907104, 0.9999790191650391, 0.9999780654907227, 0.9999996423721313, 6.0157231928315014e-05, 0.9999977350234985, 0.9991734623908997, 6.695852789562196e-05, 0.9999988079071045, 0.0312972366809845, 0.9996546506881714, 0.9999980926513672, 0.000501081405673176, 0.0004765365447383374, 0.999998927116394, 1.0, 3.212866431567818e-05, 0.9936065673828125, 1.0, 0.0024470738135278225, 0.999891996383667, 0.00012025291653117165, 0.2571060359477997, 0.0015144660137593746, 0.9999986886978149, 0.00011802517110481858, 2.088738438033033e-05, 0.9982796907424927, 0.9919841885566711, 5.194403638597578e-05, 0.0062217116355896, 1.0, 0.9999995231628418, 3.1267325539374724e-05, 0.0006621734355576336, 0.997468113899231, 4.0081922634271905e-05, 0.9999457597732544, 1.0, 0.9999899864196777, 0.9999947547912598, 0.9999723434448242, 0.9999988079071045, 0.9999305009841919, 0.00016911425336729735, 0.9999994039535522, 0.9999994039535522, 2.3798619963599776e-07, 1.0, 3.399907882339903e-06, 1.0, 0.9999139308929443, 0.9988387227058411, 1.0, 0.9999995231628418, 0.9999986886978149, 1.5288154827430844e-05, 0.9995310306549072, 2.4588249289081432e-05, 0.0016082319198176265, 0.9999487400054932, 0.9999973773956299, 0.9999977350234985, 0.00523875979706645, 7.576453299407149e-06, 1.425565733370604e-05, 1.6299560456900508e-06, 0.000284819194348529, 0.9977414608001709, 0.9999849796295166, 3.2842231121321674e-06, 0.9989427924156189, 1.0, 3.5241581031186797e-07, 1.0, 1.5297193840524415e-06, 0.9999439716339111, 8.487990044159233e-07, 1.940026095326175e-06, 0.9999997615814209, 0.9999990463256836, 0.06667368113994598, 0.0008384150569327176, 1.0, 0.9999650716781616, 6.129512621555477e-05, 0.0008753088186495006, 0.9999940395355225, 3.524837097756972e-07, 0.9984402060508728, 0.9999966621398926, 0.9999985694885254, 2.971111098304391e-05, 0.9999980926513672, 0.9999991655349731, 5.99693667027168e-05, 9.716543900140096e-06, 0.9999024868011475, 6.356760422931984e-06, 0.9999924898147583, 0.23805437982082367, 0.00037692434852942824, 0.9999979734420776, 6.311202923825476e-06, 0.008106345310807228, 0.00019230814359616488, 9.413120096724015e-06, 0.0001782225153874606, 1.2420317943906412e-05, 0.9999626874923706, 0.0002669715031515807, 0.00010609903984004632, 0.9998600482940674, 0.9999948740005493, 0.9999895095825195, 1.0, 1.7824455426307395e-05, 3.6432855267776176e-05, 0.0018460486317053437, 5.305832746671513e-05, 1.0, 0.14784620702266693, 2.062555211068684e-07, 0.9999988079071045, 1.0, 0.9999998807907104, 1.0, 1.0439113395932509e-07, 0.999995231628418, 0.9999988079071045, 1.0, 1.287909981328994e-06, 1.9898821847164072e-05, 1.0, 0.0001420976041117683, 0.9999995231628418, 0.999997615814209, 2.2491927666123956e-05, 0.0006606611423194408, 0.9999985694885254, 0.9999958276748657, 0.9999868869781494, 9.15652999537997e-05, 1.0, 0.9999167919158936, 2.8308300898061134e-05, 0.005013482645153999, 0.9999998807907104, 0.0005542680155485868, 0.9999464750289917, 1.6854212390171597e-06, 0.0008817490306682885, 0.9999992847442627, 0.9999996423721313, 1.0, 0.9999933242797852, 1.0901393579842988e-05, 1.0, 1.0, 0.9999574422836304, 0.0033160496968775988, 0.00067235465394333, 0.0003504861961118877, 2.1987056243233383e-05, 8.905437312023423e-07, 0.999901294708252, 6.203144494065782e-06, 0.9999994039535522, 0.00019713859364856035, 0.9999998807907104, 1.0, 0.004162833094596863, 0.41839051246643066, 2.3710734240012243e-05, 1.0, 0.9999992847442627, 0.9993504881858826, 0.9998511075973511, 1.0, 0.9999988079071045, 0.4745737612247467, 0.9999836683273315, 1.0, 0.0016183078987523913, 6.282818389991007e-07, 0.9999954700469971, 0.007193079683929682, 4.802646435564384e-05, 0.0027982606552541256, 0.010148430243134499, 0.000883155211340636, 0.9998914003372192, 0.002888968214392662, 1.0, 0.9998773336410522, 7.958610126479471e-07, 1.0, 0.00424985820427537, 0.9999996423721313, 1.9250239802204305e-06, 0.9999817609786987, 0.9998911619186401, 0.9999997615814209, 0.9999997615814209, 9.109270649787504e-06, 0.9999852180480957, 0.9999866485595703, 0.0010222153505310416, 1.0, 0.9993492960929871, 4.837072629015893e-05, 0.0041218893602490425, 6.693723548778507e-07, 0.005686813034117222, 1.0, 6.858749657112639e-07, 0.7701942920684814, 3.887853836204158e-06, 0.9999997615814209, 0.006299154367297888]

#Metricas de validación
Best_acc5_3 = 0.9972222222222222
acc_aux5_3 = 0.9972222222222222
Recall5_3 = 0.994475138121547
Especificidad5_3 = 1.0
Tiempo5_3 = 1228.077320098877

######## Base6 ########
#MODELO = VGG13
#Datos perdidas
xx_train6_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train6_3 = [0.049749852146631406, 0.03704980429675844, 0.04340788392741003, 0.03773190151026219, 0.04685731831892037, 0.07721940558633686, 0.041942449962651285, 0.02938120436889154, 0.02313372375052652, 0.028268271132751748, 0.02988591327527423, 0.023998399180026702, 0.027785048339469934, 0.03314114864225741, 0.02899800071745743, 0.02571400665206674, 0.030826731513679765, 0.027479205308137116, 0.023843058005527214, 0.03188762508424712, 0.02510049519715486, 0.028822462315912598, 0.02862164462790077, 0.024127071948699008, 0.027069702919250652]
xx_val6_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val6_3 = [0.0205120041138596, 0.015507715609338548, 0.022602668156226476, 0.5041905838582251, 0.2550393621126811, 0.027929091619120702, 0.022392098936769696, 0.01825664540131887, 0.01711720989810096, 0.019517862465646532, 0.019453097383181253, 0.019416363537311555, 0.019307253261407215, 0.019096154305669997, 0.018792813850773707, 0.018787119454807707, 0.01878580351670583, 0.01878334151373969, 0.01877883772055308, 0.01877310441599952, 0.0187730363673634, 0.01877300292253494, 0.01877306087149514, 0.018773033883836533, 0.018773071633444893]
epoch_min6_3 = 1
stop_val6_3 = 0.015507715609338548

#Datos accuracy
acc_train6_3 = [0.9820987654320987, 0.9858024691358025, 0.9830246913580246, 0.986111111111111, 0.9827160493827161, 0.9746913580246913, 0.9867283950617284, 0.990432098765432, 0.9916666666666667, 0.9910493827160494, 0.9898148148148148, 0.9922839506172839, 0.9898148148148148, 0.9867283950617284, 0.9898148148148148, 0.9901234567901235, 0.9898148148148148, 0.9916666666666667, 0.9916666666666667, 0.9882716049382716, 0.9919753086419753, 0.9907407407407407, 0.9907407407407407, 0.9907407407407407, 0.9898148148148148]
acc_val6_3 = [0.9944444444444445, 0.9944444444444445, 0.9916666666666667, 0.9388888888888889, 0.8777777777777778, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445]

#Datos curva ROC
best_labels6_3 = [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
best_prob6_3 = [3.89782286447371e-07, 0.9999998807907104, 7.483770900762465e-07, 0.999718964099884, 1.0301541806256864e-05, 2.0526416847133078e-05, 0.0003028411592822522, 1.4959344696308108e-07, 0.9999927282333374, 5.999196872608081e-08, 0.9999864101409912, 0.999997615814209, 0.9999971389770508, 0.29765379428863525, 0.9999997615814209, 4.3549422912292357e-07, 0.05172191187739372, 0.9632298350334167, 9.460368346481118e-06, 3.638160023911041e-06, 1.0, 0.00011442618415458128, 1.2649144991883077e-05, 1.9256097161246544e-08, 0.0002586011542007327, 1.0, 1.3859702718832523e-08, 0.9999887943267822, 0.0004400752659421414, 6.083357584429905e-05, 2.606085763545707e-05, 1.0194063548851773e-07, 0.9999774694442749, 1.0, 0.9999980926513672, 3.2850750812940532e-06, 1.0, 0.001607982674613595, 1.0, 1.0, 1.0, 0.9999992847442627, 1.0, 2.51310741106181e-08, 1.0, 0.0007133189938031137, 4.541683301795274e-05, 0.9998428821563721, 0.9993798732757568, 0.9999990463256836, 0.9999998807907104, 1.0, 1.0, 3.461860615061596e-06, 0.9999874830245972, 1.0, 1.2245844516201032e-07, 1.0, 1.76844503130269e-06, 7.096985063981265e-05, 2.955374789337384e-08, 1.199411258312466e-06, 1.0, 0.0022083530202507973, 0.00026776528102345765, 0.9997979998588562, 0.9999936819076538, 1.0, 0.9999988079071045, 1.0, 0.999840259552002, 1.0027004115897853e-08, 1.0, 0.00035007327096536756, 1.0, 1.0, 4.7302302164098364e-07, 0.9999957084655762, 0.00020077737281098962, 4.735375824793664e-08, 2.8511841776435176e-09, 6.538601127203947e-08, 0.9998088479042053, 0.0001325472112512216, 1.947659939105506e-06, 1.7695826670660608e-07, 1.0, 0.9999998807907104, 0.0005277710733935237, 1.5250886065132363e-07, 0.8741223812103271, 0.9965086579322815, 0.002213519997894764, 1.852927562140394e-05, 0.9999954700469971, 5.1951868954347447e-05, 0.9999998807907104, 1.0, 0.9999998807907104, 2.0053927585195197e-07, 3.0334562325151637e-05, 2.2592945242649876e-05, 0.0004775507841259241, 8.32279838505201e-05, 0.9999997615814209, 0.9999998807907104, 3.388048980923486e-06, 1.0, 1.3942305643332986e-10, 1.0, 0.9999985694885254, 2.991271452756905e-09, 1.0, 2.9295584624833282e-08, 1.0947911505354568e-05, 0.9999992847442627, 0.9999997615814209, 0.9999984502792358, 0.0005674389540217817, 1.0, 0.9999998807907104, 1.0, 1.0, 0.9999982118606567, 0.9999237060546875, 0.01466641016304493, 0.999988317489624, 0.9999997615814209, 0.00013419383321888745, 1.0, 0.9996235370635986, 6.360244242387125e-07, 0.9999994039535522, 1.0, 0.999920129776001, 3.452278107829443e-08, 0.999997615814209, 7.515906759181235e-07, 1.0608866432448849e-05, 4.333556353230961e-05, 0.9999986886978149, 2.9082495984766865e-06, 0.00013724244490731508, 0.9999996423721313, 3.142681180179352e-06, 1.977317879209295e-06, 0.506385862827301, 1.6687905372236855e-05, 1.0, 8.465468999929726e-05, 1.0, 0.9999988079071045, 1.0, 0.9999998807907104, 0.9999880790710449, 0.0002815792104229331, 0.999998927116394, 1.7844378135123407e-06, 0.9999995231628418, 0.20454101264476776, 0.00010271587234456092, 2.04026946448721e-06, 0.00016726521425880492, 1.0, 0.9996920824050903, 1.8832661226042546e-05, 1.0, 0.9997346997261047, 2.243287777758951e-09, 0.9988023042678833, 2.856057790268096e-07, 0.9999995231628418, 0.9998377561569214, 1.0, 1.0, 1.0, 8.44255046104081e-05, 0.0011875892523676157, 0.9999998807907104, 0.0006219968781806529, 0.9984232187271118, 1.0, 2.7104042601422407e-06, 1.0, 1.2471822401494137e-06, 0.12339619547128677, 7.173978247010382e-07, 1.1736170563381165e-05, 2.1687799289793475e-06, 0.9966712594032288, 1.0, 0.9999994039535522, 2.6457407376057063e-09, 5.207676440477371e-05, 1.8162145352107473e-05, 0.9998987913131714, 2.485137656549341e-06, 0.002325704786926508, 9.389013939653523e-07, 1.0, 0.9999998807907104, 6.314050551736727e-06, 0.9999995231628418, 0.0010632044868543744, 1.0, 3.5172115531167947e-06, 0.9999964237213135, 4.099207581020892e-05, 1.439272523384716e-07, 2.1909779661655193e-06, 1.3440619284210698e-07, 0.999819815158844, 1.0, 1.7770058548194356e-05, 2.0142071122108973e-08, 0.9997105002403259, 7.735317922197282e-05, 0.00018134871788788587, 1.4238456742532435e-06, 1.0, 4.427136082085781e-05, 0.9998388290405273, 0.9999997615814209, 1.0, 1.0, 1.0, 0.9999995231628418, 1.7552482134419733e-08, 0.9654140472412109, 1.3732115178299864e-07, 0.9998806715011597, 6.43206066186508e-09, 0.9999988079071045, 1.0, 0.9999997615814209, 0.05927162617444992, 1.0, 6.893173122080043e-05, 1.0, 0.9999998807907104, 1.0, 0.004300130996853113, 1.2886421245639212e-06, 0.000984975486062467, 2.5074825771298492e-06, 0.9998950958251953, 0.9999796152114868, 1.0, 6.399692938430235e-05, 1.0, 1.0, 0.9999802112579346, 0.9999983310699463, 0.9999996423721313, 1.0, 1.3747904858973925e-07, 0.04499258100986481, 0.02623984031379223, 0.9999998807907104, 3.865497788524408e-08, 1.0, 0.9999903440475464, 5.4526324674952775e-05, 1.5298302002975106e-07, 0.9999998807907104, 1.0, 8.61313019413501e-05, 3.934613232559059e-06, 1.7665032601144048e-06, 4.118615834158845e-06, 0.9999996423721313, 7.531038903607623e-08, 0.9999949932098389, 1.0, 9.674040484242141e-06, 5.383123152569169e-06, 0.999998927116394, 2.1466496036737226e-05, 0.9999518394470215, 5.10658610437531e-05, 1.356686766484927e-06, 0.9999998807907104, 0.00015247233386617154, 0.9999420642852783, 0.0017820504726842046, 3.471791671927349e-07, 0.9999997615814209, 0.4896262586116791, 0.00023424724349752069, 0.998063862323761, 0.9999994039535522, 1.4904975387253216e-06, 1.873598165502699e-08, 8.351187119615133e-08, 0.9999997615814209, 0.0001647267781663686, 0.9999997615814209, 4.912622898700647e-06, 0.9995296001434326, 1.0, 0.9999991655349731, 0.9999998807907104, 3.07358568534255e-05, 0.9999511241912842, 0.001611680956557393, 0.975884199142456, 0.9999679327011108, 0.9999233484268188, 5.919708200963214e-05, 1.5925658569670986e-09, 0.9967126846313477, 0.9989738464355469, 3.558986645657569e-05, 0.00015869250637479126, 1.08991036995576e-06, 1.425383652531309e-05, 1.0, 0.0029605338349938393, 4.944687680108473e-05, 1.018253624351928e-05, 2.2733815967512783e-06, 0.0013528938870877028, 1.8821680214387015e-07, 4.697253189078765e-06, 0.9999995231628418, 0.9999964237213135, 0.19724790751934052, 0.00016909620899241418, 0.000959047582000494, 0.9999396800994873, 1.7462255641476077e-08, 8.998854355013464e-06, 4.4897046080905056e-08, 7.329276286327513e-06, 3.7517866076086648e-06, 2.251939986308571e-05, 8.77260845300043e-06, 0.9995296001434326, 5.253328367871291e-07, 1.0, 0.9999998807907104, 1.0, 0.9999990463256836, 0.9999822378158569, 1.0, 4.138661097385921e-06, 1.0, 1.0, 1.0, 2.02993870601631e-08, 9.706459422886837e-06, 0.0002984858292620629, 1.0177219422757844e-07, 4.919261868963076e-07, 7.101279607013566e-06, 0.0010404716012999415, 0.0011631585657596588, 0.9999921321868896, 0.9999536275863647, 7.077388204379531e-07]

#Metricas de validación
Best_acc6_3 = 0.9944444444444445
acc_aux6_3 = 0.9944444444444445
Recall6_3 = 1.0
Especificidad6_3 = 0.989010989010989
Tiempo6_3 = 1220.4142580032349

######## Base7 ########
#MODELO = VGG13
#Datos perdidas
xx_train7_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train7_3 = [0.05058337250425492, 0.037477837544348504, 0.04531279113373639, 0.034058631303501716, 0.03283877493238743, 0.04004840112210792, 0.01999598483980438, 0.02940249335434702, 0.03413714703034471, 0.02386843693661101, 0.023397605700625313, 0.02439482762581772, 0.020285641236437692, 0.024539605095798587, 0.025401227792472014, 0.024998083121982623, 0.032344143984862316, 0.02514353120400582, 0.02022415805194113, 0.021147997033448867, 0.02269862778024909, 0.02274936266519405, 0.023639621484426804, 0.027459194189236487, 0.024654248816731537]
xx_val7_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val7_3 = [0.02406864646408293, 0.015435220632288192, 0.02815478609667884, 0.011022748880916172, 0.010668961372640397, 0.013275724649429322, 0.018290151158968607, 0.022118482655949064, 0.023089875280857087, 0.024425618847211204, 0.024350346542066997, 0.024480079942279392, 0.024438284503089058, 0.02447345968749788, 0.02458871023522483, 0.024589499831199645, 0.02459091974629296, 0.024591404365168677, 0.02459177987443076, 0.024589864081806606, 0.024589848518371583, 0.024589841564496357, 0.024589664571815068, 0.02458957764837477, 0.02458950512939029]
epoch_min7_3 = 3
stop_val7_3 = 0.011022748880916172

#Datos accuracy
acc_train7_3 = [0.9771604938271605, 0.9854938271604938, 0.9833333333333333, 0.9858024691358025, 0.9879629629629629, 0.9858024691358025, 0.9919753086419753, 0.9901234567901235, 0.9895061728395061, 0.9916666666666667, 0.9932098765432098, 0.9907407407407407, 0.991358024691358, 0.991358024691358, 0.9932098765432098, 0.9922839506172839, 0.9885802469135803, 0.990432098765432, 0.9925925925925926, 0.9916666666666667, 0.990432098765432, 0.9925925925925926, 0.9916666666666667, 0.990432098765432, 0.9898148148148148]
acc_val7_3 = [0.9861111111111112, 0.9916666666666667, 0.9861111111111112, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112]

#Datos curva ROC
best_labels7_3 = [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]
best_prob7_3 = [0.0001076421031029895, 5.0059433931437525e-08, 0.0003103683702647686, 1.7952024791156873e-05, 0.9999536275863647, 1.374091738171046e-07, 6.606562965316698e-05, 0.00012238819908816367, 1.0, 5.927153324591927e-05, 1.0, 6.753008165105712e-06, 5.364429966903117e-07, 0.9998192191123962, 1.0, 0.9999996423721313, 1.1360900771251181e-06, 0.9999996423721313, 0.000313029158860445, 1.0, 1.009053630696144e-05, 0.9979609251022339, 1.0, 3.567693511286052e-06, 2.6342570436099777e-07, 0.001123651280067861, 1.0, 0.9999927282333374, 1.0, 0.00020802505605388433, 3.720272616192233e-06, 3.65927153325174e-05, 0.9999995231628418, 0.9999934434890747, 0.999994158744812, 0.9999998807907104, 2.7528878376870125e-07, 0.17599421739578247, 1.0, 0.9998842477798462, 2.3363040781987365e-06, 0.9998940229415894, 1.0, 7.133291364880279e-05, 2.4498615402990254e-06, 1.0, 0.9999995231628418, 8.99823207873851e-05, 1.0, 1.7376727612372633e-07, 0.00039635502616874874, 1.0, 2.6166948373429477e-05, 2.5295937575720018e-06, 5.794775574941013e-07, 9.544685930507057e-09, 0.6264910697937012, 0.0001307521015405655, 9.59298267844133e-06, 2.605969662283769e-09, 0.00037206654087640345, 9.667556696513202e-06, 7.545288127630556e-08, 1.0953967830573674e-06, 0.999987006187439, 1.538083961349912e-05, 1.0, 1.9076260286965407e-06, 0.00010343850590288639, 1.0027252983491053e-06, 8.711881349654504e-08, 1.0, 5.7680572354001924e-06, 1.2234805808475357e-06, 0.9999991655349731, 0.9999922513961792, 8.542822342860745e-07, 1.0, 0.00038909149589017034, 0.999998927116394, 0.9999538660049438, 0.9999765157699585, 2.981649004141218e-06, 0.9998601675033569, 1.0, 1.9950090063503012e-05, 2.2225274733500555e-05, 0.9996248483657837, 0.9999979734420776, 0.9999995231628418, 1.0, 4.682011422119103e-06, 0.9999812841415405, 1.0, 0.9999924898147583, 1.0, 1.0, 0.999997615814209, 0.0001735590340103954, 1.0, 0.9999474287033081, 7.639193063369021e-05, 0.9998592138290405, 0.9999796152114868, 0.7996318340301514, 0.9999834299087524, 3.394368119913338e-09, 1.4612376617151313e-05, 0.9997492432594299, 0.9999998807907104, 0.9999998807907104, 5.288626425681286e-07, 0.9999936819076538, 0.9999701976776123, 1.0, 4.1275132389273494e-05, 1.0, 7.988883226062171e-06, 0.9999909400939941, 0.9999997615814209, 2.2141919544083066e-05, 3.47331251759897e-06, 4.2459985706955194e-05, 1.6110820411086024e-07, 3.1219437914842274e-06, 1.0, 0.9999884366989136, 3.5871094041795004e-06, 3.951177404815098e-06, 0.9999945163726807, 3.921520885796781e-08, 9.882253237947225e-08, 0.00011908238957403228, 0.3366842269897461, 8.268941087408166e-08, 5.874287580809323e-06, 1.7964390281122178e-05, 1.2381113265291788e-05, 5.497911115526222e-05, 0.9999998807907104, 0.9999997615814209, 0.9999998807907104, 0.9999911785125732, 0.004334966652095318, 0.9999994039535522, 1.7728005730077712e-07, 2.7945546321461734e-07, 2.9297511900949758e-06, 0.9999995231628418, 0.999966025352478, 5.734523256251123e-06, 1.0, 1.0, 0.9999998807907104, 1.1073865607613698e-05, 0.9997988343238831, 0.9999291896820068, 0.9999499320983887, 2.39563833019929e-05, 0.00019572638848330826, 0.9999998807907104, 5.100346456288207e-08, 0.9999980926513672, 1.0, 3.7103251088410616e-05, 8.843150862958282e-05, 7.84776930231601e-05, 1.0, 0.9999804496765137, 1.286164547309454e-06, 6.389254991745474e-08, 0.9997984766960144, 1.0, 0.9998127818107605, 1.0, 0.0003893909160979092, 0.9999995231628418, 0.0007147247088141739, 1.0, 1.0, 0.9999998807907104, 1.0, 0.9999338388442993, 1.0, 1.5632171198376454e-05, 0.9999997615814209, 0.9999731779098511, 0.0009227180853486061, 0.0016656359657645226, 0.9999995231628418, 1.1467237754914095e-06, 1.696216500590708e-08, 2.9085826099617407e-05, 1.0, 0.7305958271026611, 1.0, 1.5606876331730746e-05, 0.9999961853027344, 0.00174060242716223, 0.0004894143785350025, 9.031281820170989e-07, 1.0, 0.9999998807907104, 4.408535460243002e-06, 1.0, 0.00020333309657871723, 1.0, 0.0012833847431465983, 0.43356990814208984, 0.9999998807907104, 0.9999464750289917, 2.5284187358920462e-05, 5.807560796711186e-07, 0.9999997615814209, 1.4736478988197632e-05, 1.264639593046013e-07, 6.1385171647998504e-06, 8.179635187843814e-06, 6.461339944507927e-05, 0.9999063014984131, 0.999998927116394, 1.0, 6.742672553627926e-07, 0.000244982133153826, 5.533573857974261e-05, 0.9999926090240479, 0.9999991655349731, 0.0010107912821695209, 0.04101318493485451, 1.0, 0.00044731906382367015, 0.9999884366989136, 2.049445356533397e-05, 0.034270599484443665, 1.3515083310267073e-06, 1.0, 8.412172610405833e-05, 1.8031522941441835e-08, 0.0012485707411542535, 0.999904990196228, 1.0, 0.0012537511065602303, 2.0205970940878615e-05, 1.0, 1.2109416275052354e-05, 0.999951958656311, 0.009366175159811974, 0.9999998807907104, 2.3779659841238754e-06, 1.0, 0.9999951124191284, 0.999998927116394, 0.9915129542350769, 1.3821940228808671e-05, 1.3819034165862831e-06, 0.9999452829360962, 2.1032444692536956e-06, 0.9999438524246216, 3.0566643545171246e-05, 0.00021793159248773009, 0.00024113827385008335, 0.0004695759271271527, 1.0, 0.0007707535405643284, 0.9999799728393555, 1.0, 0.0007035438320599496, 0.0002226940996479243, 1.0, 1.0, 0.9988505840301514, 5.383112693380099e-06, 9.342007274426578e-07, 0.9998040795326233, 0.9999330043792725, 1.0927935363724828e-05, 0.9999827146530151, 9.210320058627985e-06, 6.032073997630505e-06, 4.127864940528525e-06, 0.0016951244324445724, 8.193308167392388e-05, 0.9999351501464844, 0.9990515112876892, 3.991344419773668e-05, 1.0, 0.9999997615814209, 8.219351002480835e-06, 1.0, 0.9999666213989258, 5.907436570851132e-05, 0.9999995231628418, 0.0003599083283916116, 0.0025177954230457544, 0.999954342842102, 1.0, 4.591415199683979e-05, 1.0, 0.9999998807907104, 2.1491094230441377e-05, 1.0, 1.0, 1.2463850907806773e-05, 8.383119711652398e-05, 1.0, 0.9999998807907104, 0.00010202648991253227, 0.9999468326568604, 0.9999998807907104, 0.9999947547912598, 5.484742814587662e-06, 0.9999842643737793, 0.9999998807907104, 1.712678113108268e-06, 0.9999992847442627, 3.7130721466382965e-05, 9.823681921261596e-07, 1.0, 1.0, 0.9999957084655762, 1.0, 0.9998039603233337, 0.9343234300613403, 0.999982476234436, 0.001190349692478776, 1.0, 1.0, 0.9999018907546997, 0.9999998807907104, 0.00019892005366273224, 0.9999979734420776, 0.9999972581863403, 0.9999932050704956, 0.9991201758384705, 0.9998904466629028, 0.9999988079071045, 1.6363896904181274e-08, 2.1214514163148124e-06, 0.0001805239007808268, 3.3777920180000365e-05, 0.9999897480010986, 1.0, 6.638911600020947e-06, 0.00014510583423543721, 1.0596964784781449e-05, 0.9999573230743408, 4.8004521886468865e-06, 0.999954104423523, 8.959138540376443e-06, 4.816387081518769e-05, 0.9997990727424622, 1.0, 1.7578669258000446e-06, 6.55780459055677e-05, 0.00020729108655359596, 0.0005991533980704844, 1.7326345913559749e-10, 0.9305952787399292, 9.268191206501797e-06, 0.9999961853027344]

#Metricas de validación
Best_acc7_3 = 0.9944444444444445
acc_aux7_3 = 0.9944444444444445
Recall7_3 = 0.989010989010989
Especificidad7_3 = 1.0
Tiempo7_3 = 1227.4396147727966

######## Base8 ########
#MODELO = VGG13
#Datos perdidas
xx_train8_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train8_3 = [0.0454035152071788, 0.042261775969355195, 0.038439848209604804, 0.036998791882285366, 0.029061025197123305, 0.01940445735314746, 0.020442359466795568, 0.026016957386408322, 0.01941638274325265, 0.02235226203446035, 0.02102093767420745, 0.019754065968740134, 0.02240367822441054, 0.01729646216572067, 0.014020187343344277, 0.021340558733101245, 0.021807282749149535, 0.01977146534640112, 0.019547253469994037, 0.026163602850319427, 0.019630325548810723, 0.021270001086371917, 0.018061863014727463, 0.020172367760060747, 0.03038450855715775]
xx_val8_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val8_3 = [0.02823376258214315, 0.04695668518543243, 0.0075262073013517596, 0.004296794533729553, 0.005889804164568583, 0.008741453621122572, 0.011089426610204909, 0.01069236761993832, 0.009940945936573877, 0.008412204517258538, 0.00842456469933192, 0.008524116045898861, 0.008597187532318962, 0.008588418033387927, 0.008530295723014407, 0.008533322811126709, 0.008537347780333625, 0.008541485832797156, 0.008539882136715783, 0.008544990420341491, 0.00854514671696557, 0.008545019560390048, 0.008545001182291243, 0.008545066250695123, 0.008545171552234226]
epoch_min8_3 = 3
stop_val8_3 = 0.004296794533729553

#Datos accuracy
acc_train8_3 = [0.979320987654321, 0.9848765432098765, 0.9848765432098765, 0.9854938271604938, 0.9922839506172839, 0.9919753086419753, 0.9938271604938271, 0.990432098765432, 0.9932098765432098, 0.9916666666666667, 0.9929012345679012, 0.9929012345679012, 0.9910493827160494, 0.9941358024691358, 0.994753086419753, 0.9916666666666667, 0.9922839506172839, 0.9929012345679012, 0.9938271604938271, 0.9907407407407407, 0.9922839506172839, 0.9910493827160494, 0.9935185185185185, 0.9919753086419753, 0.9885802469135803]
acc_val8_3 = [0.9888888888888889, 0.9833333333333334, 0.9944444444444445, 1.0, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222]

#Datos curva ROC
best_labels8_3 = [1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0]
best_prob8_3 = [0.9999693632125854, 0.008686687797307968, 0.9999994039535522, 0.9999980926513672, 0.9998200535774231, 5.1351438742130995e-05, 2.976963742185035e-06, 0.9999997615814209, 0.9987000226974487, 5.765015202996437e-07, 0.0008600568980909884, 0.9999805688858032, 0.9999998807907104, 0.9999992847442627, 0.0027607595548033714, 0.9999998807907104, 0.9999697208404541, 4.613455359958607e-07, 0.00010431693226564676, 0.9999994039535522, 0.9999896287918091, 2.3321056232816773e-06, 7.896804163465276e-06, 0.9051637649536133, 1.0, 0.00022524598171003163, 1.0, 1.7754902046362986e-06, 0.9982669353485107, 0.9999910593032837, 0.9999995231628418, 0.9999995231628418, 2.9625183742609806e-05, 0.9999985694885254, 0.00035422371001914144, 0.9999994039535522, 0.9988688826560974, 1.9017234080820344e-05, 2.8484057565947296e-06, 0.009974120184779167, 3.337445946272055e-07, 5.2853520173812285e-06, 0.9999966621398926, 8.931821184887667e-07, 3.8065893477323698e-06, 0.0011413275497034192, 1.749483089952264e-05, 1.0, 0.9999977350234985, 2.3430618512065848e-07, 0.9999899864196777, 0.9999980926513672, 0.00029981491388753057, 0.9999990463256836, 5.502747080754489e-05, 0.0001190415132441558, 0.9999998807907104, 4.355303099146113e-05, 0.9999984502792358, 1.0, 0.9999940395355225, 0.9999914169311523, 1.0200272299698554e-05, 5.807328307128046e-06, 1.0, 1.3840701740264194e-07, 0.0004532531020231545, 0.00020567355386447161, 9.464619506616145e-05, 0.9999996423721313, 0.9999552965164185, 0.0011453676270321012, 3.19367645715829e-05, 8.441841055173427e-05, 0.9999830722808838, 0.9728322625160217, 0.9962964653968811, 0.9998766183853149, 0.9996389150619507, 0.9999985694885254, 1.2914661056129262e-05, 8.096020742698329e-09, 1.0, 0.9999905824661255, 0.9999977350234985, 0.9999200105667114, 2.6232717118546134e-06, 0.9999516010284424, 0.9999998807907104, 0.002448412822559476, 1.0199221833318006e-05, 0.0001872958237072453, 0.9999909400939941, 0.0005607776110991836, 1.4608690435125027e-06, 5.1617054850794375e-05, 0.0016424511559307575, 1.8557236103333707e-07, 0.9999966621398926, 0.9999868869781494, 0.9999946355819702, 9.660667501520948e-07, 3.10050199914258e-05, 0.9999964237213135, 8.552508916181978e-06, 0.00022451330733019859, 1.0, 0.006504940800368786, 0.0033978100400418043, 0.00020693069382105023, 1.5429615814355202e-05, 0.0008766526007093489, 1.4298980488547386e-07, 1.0, 0.9998767375946045, 0.9999717473983765, 4.589825766743161e-05, 0.999508261680603, 0.9999997615814209, 0.9999992847442627, 0.9999901056289673, 0.9999513626098633, 0.9999997615814209, 0.00019340486323926598, 0.00015338121738750488, 1.2040049114148133e-05, 0.9999635219573975, 0.004694288596510887, 0.9999988079071045, 5.323999516804179e-07, 0.9828358292579651, 0.9999873638153076, 0.9999881982803345, 0.9999974966049194, 8.134313247865066e-05, 0.0003188458795193583, 0.00041486721602268517, 0.00020781530474778265, 1.0, 0.9993165731430054, 0.9991089701652527, 3.392234066268429e-05, 0.9999535083770752, 0.9999637603759766, 0.9999998807907104, 0.0044477819465100765, 0.9945566654205322, 0.00018855980306398124, 0.00014106323942542076, 0.9999754428863525, 2.4946477424236946e-05, 1.1100882147729862e-05, 0.00013955208123661578, 0.0016613613115623593, 0.00010033058060798794, 4.933077798341401e-05, 0.0012611448764801025, 0.9995771050453186, 0.00010103767999680713, 1.0, 0.9999988079071045, 0.0005696170846931636, 0.35893893241882324, 4.8534907364228275e-06, 0.0002560486609581858, 1.0, 0.9999994039535522, 0.9999667406082153, 1.9755139874177985e-05, 2.112609251980757e-07, 1.3829620684191468e-06, 8.085173249128275e-06, 3.7420355511130765e-05, 1.0, 0.9999579191207886, 0.9999985694885254, 1.0, 2.635161035868805e-05, 0.9999179840087891, 0.9999991655349731, 9.87531166174449e-05, 0.9999922513961792, 4.997901115189052e-08, 3.598782711833337e-07, 0.9999535083770752, 0.9999966621398926, 1.3271410352899693e-06, 3.314774585305713e-05, 0.0004210086481180042, 0.1104101613163948, 0.013917780481278896, 0.9999431371688843, 0.9999996423721313, 0.9999998807907104, 2.6087393507623347e-06, 0.9999998807907104, 0.9999990463256836, 0.9999945163726807, 0.003640212817117572, 3.4906861401395872e-06, 1.8543486248745467e-06, 0.001983253750950098, 0.8031022548675537, 0.9999985694885254, 0.9999746084213257, 0.00012122543557779863, 1.7283790612054872e-06, 1.554306800244376e-05, 0.0028065114747732878, 1.4686581835121615e-06, 0.9999212026596069, 0.9999755620956421, 2.303658597213598e-09, 0.9999992847442627, 1.0, 8.942390195443295e-06, 0.00012347767187748104, 7.413132774303222e-09, 8.542628648910977e-08, 0.0023221648298203945, 0.9998534917831421, 2.598184494218003e-07, 0.9999914169311523, 1.2639560509342118e-06, 0.9988904595375061, 5.533336297958158e-05, 0.9999215602874756, 9.01924886420602e-06, 0.9998881816864014, 4.740359599963995e-06, 0.000529034819919616, 5.089373644295847e-06, 0.03980257362127304, 1.0, 4.320097286836244e-05, 0.00034300883999094367, 3.918727998097893e-06, 7.730841753073037e-05, 0.9999929666519165, 0.00011772853031288832, 0.00013897563621867448, 1.0, 0.999995231628418, 1.8399132386548445e-05, 0.0005738419713452458, 4.431230049561918e-09, 1.0, 0.9999988079071045, 0.0002831081801559776, 0.9996987581253052, 0.00021329897572286427, 5.3171635954640806e-05, 1.9303817680338398e-05, 0.9999771118164062, 9.626567043596879e-05, 0.0004523706447798759, 0.9998337030410767, 0.9999969005584717, 0.9999712705612183, 0.9999998807907104, 0.999997615814209, 0.0006098254816606641, 0.9999854564666748, 1.0, 0.9999921321868896, 0.9991776347160339, 0.999843955039978, 0.9999998807907104, 0.9999938011169434, 1.0, 0.9992263317108154, 0.0003211568691767752, 0.9999881982803345, 1.0, 0.9999980926513672, 0.9999415874481201, 4.528886620391859e-06, 0.0003468352369964123, 0.999993085861206, 0.9999911785125732, 1.677422369539272e-05, 0.977674126625061, 9.165904657493229e-07, 0.00021354356431402266, 0.00036849951720796525, 7.187426945165498e-06, 0.9999697208404541, 0.9928668737411499, 6.6790164510166505e-09, 0.000544058158993721, 0.9999867677688599, 0.9999997615814209, 0.9923654198646545, 0.9999998807907104, 0.9999967813491821, 0.9999858140945435, 0.0007442934438586235, 2.010010859976319e-07, 4.579269443638623e-07, 0.002210939070209861, 0.9999725818634033, 0.9998903274536133, 0.9999995231628418, 0.9996116757392883, 0.0003207474946975708, 0.9999771118164062, 0.003022542456164956, 0.999958872795105, 0.9999828338623047, 0.1205432116985321, 0.9999955892562866, 0.9999822378158569, 0.9999997615814209, 0.9999927282333374, 2.5472523702774197e-05, 0.9990031123161316, 0.0065130628645420074, 4.111630914849229e-06, 1.0, 6.0377577028702945e-05, 0.9999948740005493, 1.0, 1.3605241292680148e-05, 0.9999864101409912, 7.256693152157823e-06, 0.23449820280075073, 0.0025805507320910692, 3.6874826037092134e-06, 1.1649015505099669e-05, 0.9999784231185913, 2.24430593220859e-07, 1.0, 0.9999966621398926, 0.999998927116394, 2.8361887416394893e-06, 0.9999167919158936, 0.9999996423721313, 4.4354437704896554e-05, 1.5537600120296702e-05, 2.248973942187149e-05, 3.753637429326773e-05, 0.999993085861206, 0.9999480247497559, 0.012035361491143703, 1.6542463754376513e-06, 0.9999927282333374, 0.9999980926513672, 3.0757819331483915e-05, 0.9999604225158691, 1.7461596144130453e-05, 0.002498181536793709, 1.0, 0.00011666920181596652, 8.876818174030632e-05, 0.9999969005584717, 0.0011392532614991069, 0.9999719858169556, 0.9973627924919128, 1.0, 0.0005327995168045163]

#Metricas de validación
Best_acc8_3 = 1.0
acc_aux8_3 = 1.0
Recall8_3 = 1.0
Especificidad8_3 = 1.0
Tiempo8_3 = 1220.7943723201752

######## Base9 ########
#MODELO = VGG13
#Datos perdidas
xx_train9_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train9_3 = [0.04767653490474195, 0.030562428927715914, 0.03562864726524294, 0.030391490339864918, 0.03667874677497664, 0.024296408781298885, 0.024488954485198597, 0.019688794587129427, 0.019808683378828896, 0.017423896427139825, 0.020631812154142944, 0.0230030988101606, 0.020108765851200364, 0.018856478399700588, 0.01835062136804616, 0.016436926238698725, 0.01940207439992163, 0.015660602102677026, 0.02319535838417065, 0.017056441656601282, 0.019042759545055438, 0.01604955525677881, 0.017667652160665136, 0.016962202776361394, 0.01870717974724593]
xx_val9_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val9_3 = [0.043440120915571846, 0.022552241716119976, 0.03319018085797628, 0.014008066554864248, 0.009209519624710083, 0.00903776122464074, 0.008905217548211415, 0.008822835485140483, 0.00850935760471556, 0.008889221317238278, 0.00882703959941864, 0.008807880348629421, 0.00880110479063458, 0.008796541558371649, 0.008813513815402985, 0.008812676618496577, 0.008814208540651533, 0.008814312434858746, 0.008812956677542793, 0.008812068237198723, 0.008812077012326983, 0.008812055985132853, 0.00881210532453325, 0.008812083469496832, 0.008812099281284545]
epoch_min9_3 = 4
stop_val9_3 = 0.009209519624710083

#Datos accuracy
acc_train9_3 = [0.9851851851851852, 0.9888888888888888, 0.987037037037037, 0.9876543209876543, 0.9842592592592593, 0.9901234567901235, 0.9925925925925926, 0.9932098765432098, 0.9938271604938271, 0.9925925925925926, 0.9919753086419753, 0.9929012345679012, 0.9929012345679012, 0.9944444444444445, 0.9935185185185185, 0.9950617283950617, 0.9929012345679012, 0.995679012345679, 0.9932098765432098, 0.9935185185185185, 0.9932098765432098, 0.994753086419753, 0.9932098765432098, 0.9938271604938271, 0.9925925925925926]
acc_val9_3 = [0.9861111111111112, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222]

#Datos curva ROC
best_labels9_3 = [0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0]
best_prob9_3 = [0.0002539720735512674, 0.004631269723176956, 0.9999998807907104, 0.999997615814209, 1.0, 0.9999812841415405, 0.026448983699083328, 0.9999996423721313, 0.010431974194943905, 8.401246304856613e-05, 0.003399372100830078, 0.0002482135023456067, 0.0006973887793719769, 0.00017156942340079695, 0.014894933439791203, 5.3973138847140945e-08, 7.788284710841253e-05, 0.9999998807907104, 0.9999998807907104, 0.012045446783304214, 0.9999953508377075, 0.0011908147716894746, 0.056863363832235336, 0.0004549861769191921, 0.9999691247940063, 0.9999922513961792, 1.6504141967743635e-05, 0.005685006733983755, 0.019048787653446198, 0.0009710934828035533, 0.9970319271087646, 0.008123494684696198, 0.999843955039978, 0.9999551773071289, 1.0, 1.0, 0.23025567829608917, 0.0002486159501131624, 0.0032193216029554605, 1.0, 0.9990321397781372, 0.0009843121515586972, 0.02189471572637558, 0.9758017063140869, 0.040046460926532745, 0.9999998807907104, 0.052947305142879486, 0.005353506654500961, 2.8070577172911726e-05, 0.9989890456199646, 0.0010719813872128725, 0.0035563597921282053, 7.537812780356035e-05, 0.017577245831489563, 0.0013302488951012492, 0.9980308413505554, 0.014140036888420582, 0.9999758005142212, 0.8178972601890564, 0.9999979734420776, 1.0, 0.0005792729207314551, 0.0019652731716632843, 0.9999977350234985, 0.9999996423721313, 0.0003879947180394083, 1.0, 0.0002639681624714285, 0.99996018409729, 0.9999773502349854, 0.9912235736846924, 0.002263723174110055, 0.002880113897845149, 0.9999927282333374, 0.016235869377851486, 0.9999979734420776, 1.4488463421002962e-05, 1.8925220501841977e-05, 1.0, 0.002055376535281539, 0.9988430738449097, 0.013795844279229641, 0.0018503149040043354, 0.00023922249965835363, 0.014291145838797092, 0.9999958276748657, 0.009036583825945854, 0.9999992847442627, 0.9999946355819702, 0.004524076357483864, 0.9994964599609375, 0.9965313076972961, 0.0010046676034107804, 0.9999887943267822, 0.9997946619987488, 1.0, 0.9999986886978149, 0.00498536741361022, 0.006912333890795708, 0.0007421675836667418, 1.6814537957543507e-05, 0.03556615114212036, 0.9999988079071045, 0.9996532201766968, 0.9999874830245972, 1.0, 0.999997615814209, 0.00076828378951177, 0.0019454779103398323, 0.970982015132904, 0.9999903440475464, 0.9697562456130981, 0.0012304432457312942, 3.21137486025691e-05, 0.00035531289177015424, 0.9999927282333374, 1.0, 0.9999052286148071, 0.9999997615814209, 0.0010672570206224918, 1.0, 3.2646951808601443e-07, 0.9999768733978271, 0.9999974966049194, 1.0, 0.00012157214223407209, 0.9876992702484131, 1.7934829884325154e-05, 6.939419836271554e-05, 0.9999997615814209, 0.0009896008996292949, 0.04572337865829468, 0.9999995231628418, 0.03194339945912361, 1.0, 0.9974583983421326, 1.0, 0.9999998807907104, 0.9999693632125854, 0.00027672931901179254, 0.9999850988388062, 0.9998617172241211, 0.0005921136471442878, 0.9998219609260559, 0.003837279975414276, 6.02247346250806e-05, 0.9999998807907104, 1.0, 0.999984622001648, 0.0015306415734812617, 0.006179347168654203, 0.9999839067459106, 0.0005270492401905358, 7.910861313575879e-05, 3.0318626158987172e-05, 0.06750313192605972, 0.0001316061825491488, 0.9999858140945435, 0.00021208846010267735, 1.0, 0.9999998807907104, 0.9999760389328003, 0.004297918640077114, 9.056138878804632e-06, 0.015255007892847061, 0.9997633099555969, 0.0010246621677652001, 3.155234026053222e-06, 1.4147121873975266e-05, 0.005183055531233549, 0.9771012663841248, 0.00012677824997808784, 0.9999960660934448, 0.000732833577785641, 6.574885628651828e-05, 0.9999972581863403, 0.043655529618263245, 0.9999994039535522, 0.9999855756759644, 0.9999940395355225, 0.004445631522685289, 0.9995146989822388, 0.9999933242797852, 9.249844879377633e-05, 0.9999998807907104, 0.0007403471972793341, 0.999969482421875, 0.9999935626983643, 0.9999998807907104, 0.999710738658905, 6.497406138805673e-05, 0.007872894406318665, 0.0016290558269247413, 1.0, 9.980047934732283e-07, 1.5968904335750267e-05, 3.9521455619251356e-06, 1.0, 0.0005717452149838209, 0.999863862991333, 1.5043588064145297e-05, 0.0022478101309388876, 1.0, 0.9999802112579346, 0.9999877214431763, 0.999975323677063, 9.555442375130951e-07, 0.00020898488583043218, 8.5399420640897e-05, 0.9998506307601929, 0.9993094205856323, 0.9999996423721313, 0.999996542930603, 0.9999998807907104, 1.0, 0.00041497169877402484, 0.006429765373468399, 0.0018091704696416855, 0.9999874830245972, 0.9999674558639526, 0.9999992847442627, 0.9996123909950256, 1.0, 0.9999992847442627, 0.001660638488829136, 0.9999679327011108, 4.8064859583973885e-05, 0.9999980926513672, 0.0013741092989221215, 0.999985933303833, 0.0008020587265491486, 0.9992087483406067, 6.262856913963333e-05, 0.01986543834209442, 0.9999958276748657, 0.00027204485377296805, 1.0, 0.9999991655349731, 0.022362658753991127, 0.002174645196646452, 0.9999984502792358, 0.9999997615814209, 9.036864980771497e-07, 0.999983549118042, 0.9999992847442627, 0.00755343260243535, 0.9999971389770508, 0.0047772713005542755, 0.008655602112412453, 0.9992591738700867, 0.9999994039535522, 0.9234415888786316, 0.9999994039535522, 3.2204150102188578e-06, 3.71418718714267e-05, 0.9999469518661499, 0.006020064931362867, 0.0001400980690959841, 0.988100528717041, 0.008461350575089455, 4.924189852317795e-05, 0.0007077845511958003, 1.0, 0.9999936819076538, 0.04396423324942589, 1.0, 0.999922513961792, 0.0015605866210535169, 0.9997500777244568, 7.262377039296553e-05, 0.0006583412759937346, 8.73285171110183e-06, 1.0, 8.93850447027944e-05, 0.00016840689931996167, 0.06869533658027649, 4.438676842255518e-05, 0.9995958209037781, 0.007717788685113192, 0.9999812841415405, 0.99998939037323, 0.9999891519546509, 1.0, 0.0007428539684042335, 0.00013254025543574244, 0.00019425722712185234, 1.6532714653294533e-05, 0.9999990463256836, 1.0, 0.0003372082719579339, 0.0014307827223092318, 7.3951332524302416e-06, 0.01555583719164133, 0.9992730021476746, 0.9999834299087524, 0.9974547028541565, 0.9999892711639404, 0.00026188601623289287, 0.002018935978412628, 0.07918081432580948, 0.012159179896116257, 1.0, 0.0035244242753833532, 0.9420716166496277, 0.0017594267847016454, 0.996553897857666, 0.000744188146200031, 0.9999958276748657, 1.0, 0.9999635219573975, 0.9999946355819702, 0.004136406816542149, 0.999981164932251, 0.9998064637184143, 0.04824190214276314, 0.0020022487733513117, 0.012174121104180813, 0.9999755620956421, 0.9997578263282776, 1.0, 0.0039020925760269165, 0.002634515520185232, 0.9999974966049194, 8.587977617935394e-07, 1.9784401956712827e-05, 0.004381142556667328, 2.178306431233068e-06, 0.012366237118840218, 0.020215194672346115, 0.9188224077224731, 0.9999864101409912, 4.7515800360997673e-07, 0.00028802096494473517, 0.9999998807907104, 0.9999936819076538, 1.480718907487244e-07, 0.9999943971633911, 0.003565778722986579, 0.9999926090240479, 0.9999492168426514, 0.9999978542327881, 0.00022553178132511675, 0.013094228692352772, 3.57296084985137e-06, 0.9991632699966431, 0.9999995231628418, 0.9999759197235107, 4.180196629022248e-05, 0.9898595213890076, 0.0017963424324989319, 0.008611028082668781, 0.9999996423721313, 0.9546889066696167, 0.9999996423721313, 0.999991774559021, 0.999997615814209, 1.0, 0.00011813708988483995, 4.3929271669185255e-06, 0.9999839067459106]

#Metricas de validación
Best_acc9_3 = 0.9972222222222222
acc_aux9_3 = 0.9972222222222222
Recall9_3 = 1.0
Especificidad9_3 = 0.994475138121547
Tiempo9_3 = 1231.9908821582794

######## Base10 ########
#MODELO = VGG13
#Datos perdidas
xx_train10_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train10_3 = [0.038735101776726455, 0.028075175042505618, 0.027410043527682622, 0.032623904281192356, 0.029830760784723142, 0.017824239716117764, 0.022387720561689802, 0.02033689940969149, 0.025036551812548696, 0.02275343914458781, 0.016497950660593716, 0.019912885184641238, 0.021628550200918573, 0.02506045080077501, 0.019303627348976372, 0.014936121856724775, 0.01941909426248368, 0.01722547231632986, 0.022183406711728484, 0.016909447018011118, 0.016468835143763343, 0.020317802247073916, 0.0231890258413774, 0.020629499907846804, 0.019292464015292533]
xx_val10_3 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val10_3 = [0.0036604038543171353, 0.0009231542547543844, 0.0022684997154606714, 0.0032375310858090717, 0.0008019796676105923, 0.0008100569248199463, 0.0009286233120494419, 0.0008713086446126302, 0.0009205786718262566, 0.0006496373150083754, 0.0006471322642432319, 0.000640246934360928, 0.000646631916364034, 0.0006574518150753445, 0.0006522029638290405, 0.0006520380576451619, 0.0006521797842449612, 0.0006524407201343112, 0.0006528056330151028, 0.000652745697233412, 0.000652741723590427, 0.0006527509954240587, 0.0006527629163530137, 0.0006527685456805759, 0.0006527784797880384]
epoch_min10_3 = 9
stop_val10_3 = 0.0006496373150083754

#Datos accuracy
acc_train10_3 = [0.9851851851851852, 0.9888888888888888, 0.9885802469135803, 0.986111111111111, 0.9885802469135803, 0.9935185185185185, 0.991358024691358, 0.9916666666666667, 0.9910493827160494, 0.9925925925925926, 0.9938271604938271, 0.9929012345679012, 0.9907407407407407, 0.9907407407407407, 0.9935185185185185, 0.9953703703703703, 0.9929012345679012, 0.9938271604938271, 0.9938271604938271, 0.9938271604938271, 0.9929012345679012, 0.9919753086419753, 0.9916666666666667, 0.9910493827160494, 0.9925925925925926]
acc_val10_3 = [1.0, 1.0, 1.0, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels10_3 = [1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0]
best_prob10_3 = [0.993124783039093, 1.2940139413331053e-06, 0.0001457496837247163, 0.9999972581863403, 8.861173228069674e-06, 0.9999138116836548, 0.9999769926071167, 1.0, 1.0, 1.0, 6.453248033722048e-07, 1.4089175659481867e-10, 0.9999985694885254, 5.047400577495864e-07, 0.9999988079071045, 2.417235691609676e-07, 2.607639726193156e-05, 0.9999982118606567, 1.2706003872153815e-05, 0.9999991655349731, 1.0, 0.00031944437068887055, 2.768049114365567e-07, 0.9999998807907104, 0.9999998807907104, 0.9999970197677612, 1.0, 9.072839930013288e-06, 3.173291361235897e-06, 1.0, 1.0, 0.9999996423721313, 0.9999998807907104, 0.0006280966917984188, 1.0, 1.0, 7.92237333371304e-05, 0.9999973773956299, 0.9999054670333862, 0.0025276150554418564, 1.0, 1.0, 5.1955607887066435e-06, 8.177576091839e-06, 0.9999936819076538, 0.9999996423721313, 4.9125344503409e-07, 0.9999971389770508, 0.999991774559021, 0.9999936819076538, 0.9999958276748657, 0.9999964237213135, 2.098073764500441e-06, 0.02310057356953621, 0.0006486114580184221, 1.0, 1.0, 1.826690640882589e-05, 0.9999946355819702, 0.0056527769193053246, 0.9995459914207458, 0.9999988079071045, 0.9998904466629028, 0.9999998807907104, 0.9999998807907104, 1.0, 4.4429260270817394e-09, 0.9997245669364929, 0.9999986886978149, 1.5730151062598452e-05, 3.135484075755812e-05, 0.9999990463256836, 0.0009586000232957304, 0.970795750617981, 0.00014129570627119392, 0.9999996423721313, 0.9999233484268188, 1.1571957614364692e-08, 1.0, 0.9999998807907104, 1.0, 0.9999998807907104, 1.8365613868809305e-05, 0.9999994039535522, 0.9999788999557495, 0.999992847442627, 0.9999994039535522, 0.001393578597344458, 6.655690231127664e-05, 1.0, 0.9999556541442871, 1.0, 3.152785666316049e-06, 0.006703574676066637, 8.809237442619633e-06, 0.9999954700469971, 0.9999790191650391, 0.9999477863311768, 1.0, 1.1799242116694586e-07, 5.826272513331787e-07, 1.229144345415989e-05, 3.028166929652798e-06, 3.423012913117418e-07, 0.00015137095761019737, 4.961389640811831e-05, 0.00013013998977839947, 0.04097350314259529, 1.0, 0.9999957084655762, 5.39096959073504e-07, 2.6766523660626262e-05, 0.9999929666519165, 8.811260272523214e-07, 1.0, 1.5928513903418207e-06, 0.0003049948427360505, 1.832222733355593e-05, 1.767772300809156e-05, 0.9999988079071045, 3.3186843211296946e-05, 2.1225928321655374e-06, 4.2036248260046705e-07, 0.9999909400939941, 0.9999880790710449, 0.9999997615814209, 9.374821274832357e-06, 1.0, 1.4664468608316383e-06, 0.000301569962175563, 3.833673326880671e-05, 5.723562935600057e-06, 0.00045148577191866934, 0.999994158744812, 1.0, 1.1226154583710013e-06, 0.9999994039535522, 1.3855620636604726e-05, 0.9999986886978149, 0.9999977350234985, 1.0, 3.0449743917415617e-06, 0.9999980926513672, 0.9905800223350525, 2.1042991647846065e-05, 0.9999983310699463, 1.0, 4.3311057518202745e-10, 0.0012629402335733175, 3.6081614780414384e-06, 4.266592441126704e-06, 0.0017582997679710388, 2.45158498728415e-05, 8.638394888293988e-09, 0.9999998807907104, 0.9999978542327881, 0.9999288320541382, 7.449552413163474e-06, 0.9999978542327881, 1.1974250355706317e-06, 0.00020088347082491964, 0.9999991655349731, 0.9999998807907104, 0.9999938011169434, 1.099348423849733e-06, 1.8703527757679694e-06, 0.9999326467514038, 4.860522651028987e-08, 0.00012165122461738065, 1.0, 0.0010832620318979025, 1.0, 0.00015816256927791983, 1.0819965012842658e-07, 1.0, 7.744181684810059e-11, 9.264958293897507e-07, 2.0141580534982495e-05, 1.0, 6.105858041394185e-10, 1.0, 6.129632765805582e-07, 0.9999980926513672, 2.1862149424123345e-06, 0.9899729490280151, 0.9999942779541016, 0.9999943971633911, 1.0, 0.9999959468841553, 4.430818444234319e-05, 0.0026120797265321016, 1.7891220522869844e-06, 1.0, 0.9999432563781738, 3.1896724976832047e-06, 6.794301589252427e-05, 0.9714141488075256, 0.9999998807907104, 4.6154227106853796e-07, 3.992836718680337e-05, 1.0, 4.449080370250158e-05, 1.911469553306233e-05, 0.0001437476894352585, 6.40067009953782e-05, 0.9999983310699463, 7.2788793659128714e-06, 1.0, 0.0016052145510911942, 0.9999986886978149, 2.9826130685250973e-06, 4.545392471300147e-07, 0.999992847442627, 0.0009970137616619468, 0.005150977987796068, 6.330924118280734e-10, 1.2289929074427164e-09, 1.0, 9.120980166699155e-07, 3.015134808492803e-09, 0.999995231628418, 0.9999990463256836, 1.0, 0.9999998807907104, 0.0002630330855026841, 0.0003265042614657432, 0.9999986886978149, 3.8653179217362776e-05, 1.0, 3.221721271984279e-05, 2.147986742784269e-05, 0.9999967813491821, 6.135410899332783e-07, 1.0, 0.9999663829803467, 0.9999998807907104, 1.0, 9.46283180383034e-05, 1.2911003977933433e-05, 3.915878608040657e-07, 1.0, 0.9999680519104004, 9.20387446967652e-06, 0.9999996423721313, 0.9999960660934448, 3.570776607375592e-05, 5.792615098698661e-08, 2.9291559258126654e-05, 1.0, 6.970419781282544e-05, 0.9999998807907104, 0.9999997615814209, 3.00500905723311e-05, 0.9999915361404419, 2.089045210595941e-06, 1.0, 6.485614534312845e-08, 2.247263175547687e-08, 4.837284723180346e-05, 1.0, 0.9999961853027344, 8.47352475830121e-06, 0.9999992847442627, 0.0008970354101620615, 1.0, 2.3967308152350597e-05, 0.9999997615814209, 1.8719798390520737e-05, 8.77088496054057e-06, 0.9999997615814209, 0.9999998807907104, 1.0, 1.0, 1.0, 6.123262664914364e-06, 1.5775582141941413e-05, 8.098824764601886e-06, 4.4571873303311804e-08, 0.999997615814209, 0.9999996423721313, 0.0040248725563287735, 1.5472051018150523e-06, 3.5382763599045575e-05, 0.9999991655349731, 0.9999997615814209, 0.9999986886978149, 0.0008812281885184348, 4.866779818257783e-06, 0.9999978542327881, 7.831159791749087e-07, 1.0, 0.00043613501475192606, 0.9999997615814209, 5.261523710942129e-06, 4.1453963603999e-06, 2.887198206735775e-05, 1.0, 1.3692874745174777e-06, 1.0, 3.056845162063837e-05, 5.0477308832341805e-05, 0.00014513266796711832, 0.9998815059661865, 1.0, 0.9999996423721313, 1.0, 1.0, 1.0371569260314573e-05, 1.0, 0.001711330027319491, 0.9999982118606567, 1.1460755331427208e-06, 4.798342615686124e-06, 0.0016485677333548665, 1.0842561515289617e-08, 4.5622519451171684e-07, 2.419104475848144e-06, 7.243398431455716e-05, 3.4722287978183886e-07, 1.204936415888369e-05, 5.522527317225467e-06, 0.9999967813491821, 0.9999970197677612, 0.0010204794816672802, 9.223042434314266e-05, 0.9999996423721313, 1.0, 1.0602752809063531e-06, 1.3880100595997646e-05, 0.9999991655349731, 0.9999959468841553, 1.0, 4.76200190391296e-09, 0.9999963045120239, 1.0, 0.9999703168869019, 0.006822416093200445, 0.0022892598062753677, 0.999891996383667, 0.9999897480010986, 0.9999992847442627, 3.973206548835151e-05, 0.9999998807907104, 3.972388367401436e-05, 6.6373024310451e-05, 0.9998660087585449, 5.536496246349998e-06, 9.76350520431879e-07, 0.9998977184295654, 0.0016237135278061032, 6.241253913685796e-07, 0.9999998807907104, 1.0, 5.31456244061701e-05, 1.0, 0.9999794960021973, 0.9999959468841553, 0.9999951124191284, 0.02128368616104126, 2.520834641472902e-05]

#Metricas de validación
Best_acc10_3 = 1.0
acc_aux10_3 = 1.0
Recall10_3 = 1.0
Especificidad10_3 = 1.0
Tiempo10_3 = 1242.694509267807
###################################################################

######## Base1 ########
#MODELO = VGG13_BN
#Datos perdidas
xx_train1_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train1_4 = [0.22297283352157216, 0.10304563052860308, 0.08782891923630678, 0.07079340385066138, 0.05424575288722545, 0.050388051081954696, 0.04635136568987811, 0.046066551977469596, 0.05294588962454855, 0.05157523215001012, 0.04928559600203126, 0.04641258952058392, 0.03894557371551608, 0.045209086677174506, 0.044349880737287024, 0.04351279720847989, 0.040594253514283966, 0.036822289136456854, 0.03489050651773994, 0.04803712099422643, 0.04159523861275779, 0.05166297100576354, 0.04893587744898266, 0.05067199347195802, 0.05010966142570531]
xx_val1_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val1_4 = [0.07858550912804074, 0.07011904451582167, 0.056758716371324326, 0.062177591025829315, 0.05179516358508004, 0.04823666512966156, 0.050022027889887494, 0.047099417116906905, 0.047115933232837255, 0.04824235886335373, 0.04228448801570468, 0.0413107262717353, 0.046919063644276726, 0.04246830277972751, 0.05072198195589913, 0.045449399285846286, 0.047041414429744084, 0.0547281950712204, 0.04842821823226081, 0.04846596154901717, 0.046814779937267306, 0.043036932829353544, 0.04202679014868206, 0.05357462598217858, 0.04953521009948519]
epoch_min1_4 = 10
stop_val1_4 = 0.04228448801570468

#Datos accuracy
acc_train1_4 = [0.9123456790123456, 0.9617283950617284, 0.9672839506172839, 0.9756172839506173, 0.9774691358024691, 0.9811728395061728, 0.9814814814814815, 0.9827160493827161, 0.9805555555555555, 0.9790123456790123, 0.9805555555555555, 0.9833333333333333, 0.986111111111111, 0.9827160493827161, 0.9839506172839506, 0.9827160493827161, 0.9858024691358025, 0.987037037037037, 0.9882716049382716, 0.9830246913580246, 0.9851851851851852, 0.9820987654320987, 0.9811728395061728, 0.9802469135802468, 0.9817901234567901]
acc_val1_4 = [0.9750000000000001, 0.9666666666666667, 0.9805555555555556, 0.9750000000000001, 0.9750000000000001, 0.9777777777777779, 0.9750000000000001, 0.9805555555555556, 0.9750000000000001, 0.9777777777777779, 0.9805555555555556, 0.9861111111111112, 0.9777777777777779, 0.9833333333333334, 0.9750000000000001, 0.9805555555555556, 0.9777777777777779, 0.9777777777777779, 0.9750000000000001, 0.9777777777777779, 0.9805555555555556, 0.9833333333333334, 0.9777777777777779, 0.9750000000000001, 0.9777777777777779]

#Datos curva ROC
best_labels1_4 = [1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0]
best_prob1_4 = [0.9999934434890747, 5.724054062739015e-06, 5.012335896026343e-05, 1.0, 0.0003217581834178418, 0.9920337796211243, 1.4777531760046259e-05, 0.9999781847000122, 0.9992697834968567, 0.0016534964088350534, 0.5140389800071716, 0.0004282627487555146, 9.727908036438748e-05, 0.9354854226112366, 7.551044109277427e-05, 0.9987364411354065, 2.504822077753488e-05, 0.9999635219573975, 1.4034929336048663e-05, 1.7282623957726173e-05, 0.999656081199646, 0.007055528461933136, 2.5429503693885636e-06, 0.9999451637268066, 0.999964714050293, 3.261574238422327e-05, 0.0009330750326626003, 0.9999951124191284, 0.9999977350234985, 0.00011176481348229572, 0.9998859167098999, 0.9997373223304749, 0.9997475743293762, 9.78159823716851e-06, 3.7631420127581805e-05, 0.0001571282191434875, 0.0003735460340976715, 0.00019388270447961986, 8.350105053978041e-05, 0.9998160004615784, 0.0021175858564674854, 6.01895953877829e-05, 0.9998565912246704, 0.9999785423278809, 0.9999723434448242, 0.9992688298225403, 0.000905029010027647, 0.9982196688652039, 0.08113518357276917, 0.0014604540774598718, 4.102641105419025e-05, 0.9956521391868591, 0.00931961927562952, 0.9999960660934448, 0.9999505281448364, 0.9999836683273315, 0.9999986886978149, 0.9986860156059265, 0.9999998807907104, 1.2879570022050757e-05, 0.9999676942825317, 0.9997466206550598, 0.00013579087681137025, 0.9999977350234985, 0.9999749660491943, 0.0007614644709974527, 1.5898125639068894e-05, 0.9999409914016724, 5.253812105365796e-06, 0.0002393543691141531, 0.000286274851532653, 0.9999836683273315, 0.9964524507522583, 0.9999990463256836, 0.0032889610156416893, 3.411230864003301e-05, 0.9999544620513916, 0.001712431781925261, 0.9999901056289673, 2.4142522306647152e-05, 0.999998927116394, 0.9999643564224243, 0.9999920129776001, 0.9999972581863403, 0.9999455213546753, 5.999784843879752e-05, 7.510520845244173e-06, 4.321309006627416e-06, 0.9999001026153564, 0.999998927116394, 0.9957758784294128, 0.00014452503819484264, 0.9999998807907104, 0.00011385152174625546, 0.9999904632568359, 1.6604642951278947e-05, 0.0038615947123616934, 0.9999643564224243, 6.874680548207834e-05, 0.9999916553497314, 0.9998660087585449, 0.0063804080709815025, 0.9999980926513672, 0.0026770485565066338, 0.9999922513961792, 0.04948343709111214, 0.004591881763190031, 0.04871954023838043, 0.9973649382591248, 0.9998223185539246, 0.0002275910519529134, 0.0004930085269734263, 4.296659597002872e-07, 4.6921404646127485e-06, 0.0006331229815259576, 0.9993981122970581, 0.40185683965682983, 5.025660357205197e-06, 0.9888311624526978, 0.9999713897705078, 0.9999480247497559, 0.9999436140060425, 3.377076791366562e-05, 0.0026477843057364225, 0.0015725595876574516, 1.2984025488549378e-05, 0.9987521171569824, 0.9998779296875, 0.9996910095214844, 0.9996687173843384, 0.0009195962338708341, 0.00014879718946758658, 0.0001303668977925554, 0.9999998807907104, 0.0006441433797590435, 0.0006294957711361349, 0.999998927116394, 5.045180660090409e-05, 0.999983549118042, 0.9974709749221802, 5.950721970293671e-06, 5.3931722504785284e-05, 0.9957226514816284, 2.6773210265673697e-05, 0.999953031539917, 1.2327614058449399e-06, 0.00013051927089691162, 1.0, 2.1481711883097887e-05, 0.03844247758388519, 0.00015195287414826453, 0.9999165534973145, 0.9999309778213501, 0.0004977462813258171, 0.0002855949860531837, 0.999834418296814, 0.9971605539321899, 1.3476596905093174e-05, 0.9999812841415405, 0.0024086262565106153, 0.9997124075889587, 0.9999724626541138, 0.00016990526637528092, 7.427114724123385e-06, 0.9999984502792358, 0.9999032020568848, 8.44664918986382e-06, 0.015003131702542305, 0.8804026246070862, 0.0010930299758911133, 0.0001652158098295331, 0.9999803304672241, 0.0008898476953618228, 0.9988558292388916, 0.0001284755562664941, 0.29935508966445923, 0.9998049139976501, 0.999440610408783, 3.7549474654952064e-05, 0.9999608993530273, 3.302102641100646e-06, 0.9999890327453613, 1.3753108760283794e-05, 0.9999979734420776, 0.9998677968978882, 0.0007128092111088336, 0.999972939491272, 0.04231354594230652, 0.9999996423721313, 0.9995594620704651, 0.00020246370695531368, 0.9999595880508423, 0.9996015429496765, 0.06819523870944977, 0.9999676942825317, 0.0005614236579276621, 0.9974101185798645, 0.9999830722808838, 0.40383705496788025, 0.0019472027197480202, 0.0003309923049528152, 0.999953031539917, 2.622550709929783e-05, 0.9951245188713074, 0.9999911785125732, 5.7830715377349406e-05, 0.9967063069343567, 1.8627448298502713e-05, 4.776222226610116e-07, 0.9996281862258911, 5.524834705283865e-06, 0.9999938011169434, 0.9997195601463318, 0.9999722242355347, 6.78450987834367e-06, 4.318556602811441e-05, 5.367262929212302e-06, 0.9943034648895264, 6.898526044096798e-05, 8.89106813701801e-05, 0.9749098420143127, 0.00026309804525226355, 0.00017640773148741573, 1.2984025488549378e-05, 1.4737125638930593e-05, 0.9999852180480957, 6.566646334249526e-05, 0.9999957084655762, 1.6197727745748125e-05, 0.0008358011837117374, 0.9947291016578674, 1.0, 2.22482276512892e-06, 3.7970090488670394e-05, 1.0, 0.9999368190765381, 4.7518788051093e-06, 8.37888819660293e-06, 7.434075814671814e-05, 0.0002745292440522462, 0.0001695530954748392, 0.9999943971633911, 4.3340205593267456e-06, 0.005611318629235029, 0.02382451854646206, 6.116870645200834e-05, 6.404876330634579e-05, 0.9933358430862427, 0.9999680519104004, 0.01698697917163372, 0.9999567270278931, 0.9997949004173279, 0.9997718930244446, 0.00016034524014685303, 9.243927343050018e-05, 0.9998694658279419, 3.0188937216735212e-06, 0.0001076250773621723, 0.9999977350234985, 0.026911579072475433, 2.721580858633388e-05, 0.9997519850730896, 3.6883966458844952e-06, 0.9958184361457825, 0.9950525164604187, 0.9999617338180542, 0.99985671043396, 0.9999790191650391, 1.0843182280950714e-06, 0.9999760389328003, 0.00012974166020285338, 3.564098369679414e-05, 0.9994794726371765, 0.9999669790267944, 0.9997000694274902, 1.0, 0.8907665014266968, 0.9978085160255432, 1.175326087832218e-05, 0.00015534470730926841, 0.9987097978591919, 0.9986403584480286, 3.358583489898592e-05, 0.0016873282147571445, 7.313401147257537e-05, 0.9999998807907104, 0.9989545345306396, 0.9999723434448242, 1.7763621144695207e-05, 0.0010838505113497376, 0.9940702319145203, 0.004469419363886118, 0.002412649104371667, 0.9999573230743408, 0.9998694658279419, 0.0010348951909691095, 8.205237463698722e-06, 1.0, 0.00035948792356066406, 0.23476898670196533, 0.0007483131485059857, 9.547831723466516e-06, 0.9999833106994629, 0.9995610117912292, 0.9999741315841675, 0.9998205304145813, 0.998889148235321, 0.00015456913388334215, 0.00012782913108821958, 0.0014421874657273293, 0.00012523213808890432, 0.9999984502792358, 0.9999825954437256, 0.0018793931230902672, 0.0002192663523601368, 5.938514004810713e-05, 0.999804675579071, 0.9991846680641174, 0.9999665021896362, 6.50183719699271e-05, 0.9999727010726929, 1.027584403345827e-05, 7.342731259996071e-05, 0.0011123112635686994, 0.999976396560669, 2.256893458252307e-05, 0.9998455047607422, 0.002129986183717847, 0.9997583031654358, 0.003955286927521229, 0.0003504001651890576, 0.938719630241394, 0.9999880790710449, 0.9997662901878357, 0.9999227523803711, 0.9999922513961792, 0.0049537718296051025, 0.0001476572797400877, 0.999894380569458, 0.35246801376342773, 0.999953031539917, 0.28219345211982727, 0.9986299276351929, 0.9999266862869263, 2.938632133009378e-05, 0.9999932050704956, 0.0011527390452101827, 0.9999103546142578, 2.2152362362248823e-06, 0.0003183007356710732, 0.06997670978307724, 0.429325133562088, 1.0, 0.8954707384109497, 3.0020815756870434e-05, 0.9951686859130859, 0.0005070004262961447, 0.00046335134538821876, 0.9998657703399658, 0.9999420642852783]

#Metricas de validación
Best_acc1_4 = 0.9805555555555556
acc_aux1_4 = 0.9882716049382716
Recall1_4 = 0.9942857142857143
Especificidad1_4 = 0.9675675675675676
Tiempo1_4 = 1269.5174231529236

######## Base2 ########
#MODELO = VGG13_BN
#Datos perdidas
xx_train2_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train2_4 = [0.09053777340385649, 0.053949871824847326, 0.0595867598682274, 0.05564275113520799, 0.04242196240358882, 0.03455523544071633, 0.031579131338699364, 0.031162614320163373, 0.029128466192402957, 0.028691936689026563, 0.033283556933388296, 0.029165182097090615, 0.03749456110375899, 0.0313435482206168, 0.029300326430871163, 0.03334384853089297, 0.03450431525707245, 0.032574413339664904, 0.031417241545371066, 0.03191272300886519, 0.037691256486707264, 0.03201409054943073, 0.04282529716506416, 0.028804984339225438, 0.03171046761634909]
xx_val2_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val2_4 = [0.05099189943737454, 0.06441164828009076, 0.05511150194538964, 0.04894887407620748, 0.048507215413782334, 0.04918060104052226, 0.046021070745256215, 0.04780626280440225, 0.0458482982383834, 0.04752628687355254, 0.04734036475419998, 0.0478223500152429, 0.044976053966416255, 0.050279739830229014, 0.04876040005021625, 0.04801101767354542, 0.04436162610848745, 0.045208607117335, 0.047393144336011675, 0.04864231927527322, 0.051270614398850334, 0.048616729511155025, 0.04795037888818317, 0.049154949022663964, 0.04759426514307658]
epoch_min2_4 = 0
stop_val2_4 = 0.05099189943737454

#Datos accuracy
acc_train2_4 = [0.9688271604938271, 0.9799382716049383, 0.9762345679012345, 0.9796296296296296, 0.986111111111111, 0.9879629629629629, 0.9895061728395061, 0.9898148148148148, 0.9907407407407407, 0.9898148148148148, 0.9885802469135803, 0.9907407407407407, 0.9876543209876543, 0.9879629629629629, 0.991358024691358, 0.9895061728395061, 0.9885802469135803, 0.987037037037037, 0.9895061728395061, 0.9876543209876543, 0.9873456790123457, 0.9885802469135803, 0.9839506172839506, 0.9901234567901235, 0.9898148148148148]
acc_val2_4 = [0.9777777777777779, 0.9750000000000001, 0.9805555555555556, 0.9805555555555556, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112]

#Datos curva ROC
best_labels2_4 = [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
best_prob2_4 = [0.8079723119735718, 0.06413000822067261, 0.999213695526123, 0.0009064776240848005, 0.00028218523948453367, 0.0017313507851213217, 0.9981683492660522, 0.00033728720154613256, 0.00037562259240075946, 0.9998759031295776, 0.9999958276748657, 0.9985826015472412, 0.9999892711639404, 0.9960203766822815, 0.0023402071092277765, 0.9506601691246033, 0.9999995231628418, 0.9993504881858826, 0.0005612834938801825, 0.0007790890522301197, 0.9991588592529297, 0.9983842372894287, 0.00029917334904894233, 0.9999696016311646, 0.0026569911278784275, 0.0013096155598759651, 0.0018446276662871242, 0.0005978556582704186, 0.002812738297507167, 0.005321755539625883, 0.9999973773956299, 0.9997168183326721, 0.9999971389770508, 0.0006942079053260386, 0.007240172941237688, 0.00029978490783832967, 0.0008942398126237094, 0.9998456239700317, 0.9999719858169556, 0.0008256295695900917, 0.999402642250061, 0.9987788796424866, 0.9998373985290527, 0.999794065952301, 0.004511908162385225, 0.004117458127439022, 0.0002688145323190838, 0.9999821186065674, 0.9998413324356079, 0.8598063588142395, 0.02476442977786064, 0.9998231530189514, 4.774926492245868e-05, 0.9999998807907104, 0.00032767956145107746, 0.00015826437447685748, 0.012529348023235798, 0.9999935626983643, 0.000525803305208683, 0.008230258710682392, 0.9744837880134583, 0.9990814924240112, 0.08988961577415466, 0.9999927282333374, 0.9829628467559814, 0.017508449032902718, 0.00841375719755888, 0.013652725145220757, 0.9927313923835754, 0.00024959875736385584, 0.00713778892531991, 0.0016161035746335983, 0.9999310970306396, 0.0029155085794627666, 0.9982869029045105, 0.9999858140945435, 0.0011607169872149825, 0.9964067339897156, 0.9994551539421082, 8.62455417518504e-05, 0.9998780488967896, 9.092543041333556e-05, 0.006082131993025541, 0.0029361797496676445, 0.9990471005439758, 0.9995738863945007, 0.03524161875247955, 0.003226639237254858, 0.9995655417442322, 0.9784526228904724, 0.0021891260985285044, 0.99982750415802, 0.9999212026596069, 0.9997268319129944, 0.0025936767924576998, 0.003080361755564809, 0.9992584586143494, 0.0002048330061370507, 0.9998517036437988, 4.4730357330990955e-05, 0.0004996079369448125, 0.9997983574867249, 0.9997542500495911, 0.9993739724159241, 0.9995892643928528, 0.0010221184929832816, 0.4454749524593353, 0.0031385021284222603, 0.7307271361351013, 0.9999098777770996, 0.021370692178606987, 0.9992679953575134, 0.012166417203843594, 0.003654484637081623, 0.008450720459222794, 0.5688121318817139, 0.9997461438179016, 0.9997225403785706, 0.9997944235801697, 0.006813175976276398, 0.9998823404312134, 0.9899802803993225, 0.032191209495067596, 0.9960023760795593, 0.9999970197677612, 2.7352853066986427e-05, 0.0004660952545236796, 0.9997970461845398, 0.999549925327301, 0.0010519871721044183, 0.0032524894922971725, 0.9999767541885376, 0.9978961944580078, 0.0012099434388801455, 0.9999988079071045, 0.0002499624097254127, 0.0003950289392378181, 0.9993125200271606, 0.9980373978614807, 0.99993896484375, 0.9999995231628418, 0.0009265635744668543, 0.0016449373215436935, 0.004864044487476349, 0.0005931334453634918, 0.9995601773262024, 0.9252317547798157, 0.0022168608848005533, 0.9997323155403137, 0.0012149958638474345, 0.0014533664798364043, 0.003829657332971692, 0.0010253193322569132, 0.988713800907135, 0.0012638390762731433, 0.0014408385613933206, 0.9998366832733154, 0.9991841912269592, 0.0012029093923047185, 0.9768311977386475, 0.9994596838951111, 0.9997894167900085, 0.00039568229112774134, 0.0007698587724007666, 0.9999126195907593, 0.0007748714415356517, 0.9996523857116699, 0.9996446371078491, 0.00012651184806600213, 0.0064498926512897015, 0.006905927322804928, 0.011835641227662563, 0.005198848899453878, 0.004856601357460022, 0.038539450615644455, 0.9959824085235596, 0.9999984502792358, 0.93485426902771, 0.9981117248535156, 0.9996917247772217, 0.9999721050262451, 0.9998816251754761, 0.9994902610778809, 0.999542236328125, 0.00026761164190247655, 0.00034162544761784375, 0.0006784926517866552, 0.9996196031570435, 0.0003280956298112869, 0.9995378255844116, 0.9999284744262695, 0.00010498693882254884, 0.9999988079071045, 0.9998263716697693, 0.997431218624115, 0.9998599290847778, 0.0032663424499332905, 0.9961089491844177, 0.9934239387512207, 0.9422194361686707, 0.9958958625793457, 0.008137914352118969, 0.999987006187439, 0.9993926286697388, 0.9995056390762329, 0.9952144622802734, 0.00021425410523079336, 5.169409269001335e-05, 0.9999641180038452, 0.002604302018880844, 0.9997450709342957, 0.9999839067459106, 0.000759237795136869, 0.0018883097218349576, 0.999969482421875, 0.032062772661447525, 0.00012631487334147096, 0.999845027923584, 0.9999632835388184, 0.9999912977218628, 0.999975323677063, 0.9998524188995361, 0.9998975992202759, 0.9999996423721313, 0.0016885477816686034, 0.7682049870491028, 0.9990357160568237, 0.00013838808808941394, 0.00021833924984093755, 0.013340930454432964, 0.9905559420585632, 0.9999555349349976, 0.00017479430243838578, 0.996455729007721, 0.002285954775288701, 0.0006275488412939012, 0.9987884163856506, 9.562833292875439e-05, 0.9999806880950928, 0.9736330509185791, 0.9996645450592041, 0.002684972947463393, 0.0003573100839275867, 0.000738204806111753, 0.00033278949558734894, 0.8346664309501648, 0.9998878240585327, 0.00043018785072490573, 0.0008191171218641102, 0.9999926090240479, 0.000692694797180593, 0.0012701645027846098, 0.9947434663772583, 0.0044431183487176895, 0.9999992847442627, 0.9998074173927307, 0.021364310756325722, 0.996933102607727, 0.9998890161514282, 0.9985870122909546, 0.0015882288571447134, 0.9996490478515625, 0.008074738085269928, 0.9999628067016602, 0.00028404121985659003, 0.00163681257981807, 0.9950036406517029, 0.9991486072540283, 0.00026670892839320004, 0.9996401071548462, 0.00022209489543456584, 0.011959295719861984, 0.9999589920043945, 0.0015858368715271354, 0.000675101880915463, 0.999607503414154, 0.0009078264120034873, 0.03611946851015091, 0.9998600482940674, 0.00028444608324207366, 0.0015769341262057424, 0.9998999834060669, 0.9999949932098389, 0.0044251736253499985, 8.171857189154252e-05, 0.0005940670380368829, 4.8964782763505355e-05, 0.00036728932172991335, 0.9998843669891357, 0.0012635309249162674, 0.9996696710586548, 0.027432627975940704, 0.9998050332069397, 0.0006073768017813563, 0.9952204823493958, 0.004233585204929113, 6.684516120003536e-05, 0.0003914808330591768, 0.000668855500407517, 0.9908483028411865, 0.9999997615814209, 0.9998251795768738, 0.011660377494990826, 0.9991480112075806, 0.3552292585372925, 0.026399504393339157, 2.6973619242198765e-05, 0.001420348882675171, 0.9995717406272888, 0.999643087387085, 0.9999946355819702, 0.5789071917533875, 0.9990471005439758, 0.00015169633843470365, 0.011884666979312897, 0.0002891666954383254, 0.09265173971652985, 0.998690664768219, 0.9992426633834839, 0.001668524811975658, 0.9999996423721313, 0.9988667964935303, 0.003906995989382267, 0.0002442924305796623, 0.004661114886403084, 0.9999736547470093, 0.9999074935913086, 0.9988201260566711, 0.9999226331710815, 0.0018812768394127488, 0.9988922476768494, 0.9997702240943909, 0.9993667006492615, 0.002412442583590746, 0.00046078901505097747, 0.9993284940719604, 0.9999945163726807, 0.0022648219019174576, 0.9999475479125977, 0.998976469039917, 0.9999966621398926, 0.9999806880950928, 0.007905506528913975, 0.9998972415924072, 0.999679684638977, 0.001289683161303401, 0.034486353397369385, 0.9998974800109863, 0.0005360239301808178, 0.0002566511684563011, 0.006965262815356255, 0.014123867265880108, 0.01414534356445074, 0.9993736147880554, 0.000365280982805416, 0.0013774266699329019, 0.007385836914181709, 0.9997381567955017, 0.9999960660934448, 0.006248888093978167]

#Metricas de validación
Best_acc2_4 = 0.9777777777777779
acc_aux2_4 = 0.991358024691358
Recall2_4 = 0.967391304347826
Especificidad2_4 = 0.9886363636363636
Tiempo2_4 = 1288.7372119426727

######## Base3 ########
#MODELO = VGG13_BN
#Datos perdidas
xx_train3_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train3_4 = [0.08105986138552795, 0.05397611288008867, 0.0559086134036382, 0.055548142025500166, 0.04830883656754906, 0.040941727621319854, 0.042064476519087214, 0.030004588799712097, 0.03212897531044336, 0.0486827943979958, 0.038050847629338136, 0.04102773315008776, 0.03774105781390343, 0.03532326580197723, 0.032892649675960894, 0.028425317735951625, 0.031917447403625204, 0.032369678182366454, 0.0314829540565426, 0.030901600779206666, 0.042105280212414116, 0.026655989784149476, 0.03484332913785805, 0.033850584188361225, 0.030987104423988013]
xx_val3_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val3_4 = [0.04413776513602999, 0.04285836981402503, 0.010934773915343815, 0.08516718496878942, 0.011567538976669312, 0.00903178271320131, 0.008964833782778845, 0.009719648957252502, 0.008583883941173553, 0.00917715910408232, 0.007654689583513472, 0.008894270824061499, 0.008332576685481602, 0.007893213795291053, 0.008578164213233524, 0.008603574997848935, 0.008586229715082381, 0.008589645640717613, 0.008077438755167855, 0.00809136794673072, 0.008311050799157884, 0.008299630052513546, 0.008171418805917104, 0.008686296145121257, 0.008116597102748023]
epoch_min3_4 = 10
stop_val3_4 = 0.007654689583513472

#Datos accuracy
acc_train3_4 = [0.9694444444444444, 0.9790123456790123, 0.9820987654320987, 0.9780864197530864, 0.9802469135802468, 0.9864197530864197, 0.9848765432098765, 0.9895061728395061, 0.9867283950617284, 0.9830246913580246, 0.9848765432098765, 0.986111111111111, 0.9848765432098765, 0.986111111111111, 0.9891975308641975, 0.990432098765432, 0.9885802469135803, 0.9876543209876543, 0.9888888888888888, 0.9885802469135803, 0.9848765432098765, 0.9910493827160494, 0.9879629629629629, 0.9873456790123457, 0.9891975308641975]
acc_val3_4 = [0.9805555555555556, 0.9888888888888889, 0.9916666666666667, 0.9666666666666667, 0.9944444444444445, 1.0, 1.0, 0.9972222222222222, 1.0, 0.9972222222222222, 1.0, 0.9972222222222222, 1.0, 1.0, 1.0, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9972222222222222, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels3_4 = [1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
best_prob3_4 = [0.9999797344207764, 0.9999825954437256, 5.2837422117590904e-05, 0.9998397827148438, 0.0036961876321583986, 0.00017076234507840127, 0.0002925954759120941, 2.8254169592401013e-05, 0.9994707703590393, 0.9997465014457703, 0.9999866485595703, 0.00044199678814038634, 0.0021038693375885487, 1.0, 0.0005483234999701381, 0.9998537302017212, 0.9999761581420898, 0.00017144595040008426, 0.9999680519104004, 0.9999958276748657, 0.05802418664097786, 0.9999995231628418, 0.9999938011169434, 0.6288213133811951, 0.09606631100177765, 6.413432856788859e-06, 4.56160069006728e-06, 2.8186137569718994e-05, 0.0002061429840978235, 0.9999954700469971, 0.9999918937683105, 0.9999619722366333, 0.00016982767556328326, 0.0004490058636292815, 0.0002520194393582642, 6.049571652511077e-07, 1.2863546317021246e-06, 0.9998786449432373, 0.999994158744812, 0.999982476234436, 0.9999920129776001, 1.0, 4.3778185499832034e-05, 0.9970672726631165, 0.999996542930603, 0.7851641178131104, 6.178067269502208e-05, 0.00040260495734401047, 0.9999979734420776, 0.008981234394013882, 0.9999860525131226, 1.9946839984186227e-06, 0.0003812944341916591, 0.006933172699064016, 0.9942988157272339, 1.7192180166603066e-05, 3.831037815871241e-07, 0.9999344348907471, 1.0, 6.546270742546767e-05, 3.992086567450315e-05, 0.9991293549537659, 0.006368803791701794, 0.9999948740005493, 0.9996944665908813, 8.015211278689094e-06, 0.003822298487648368, 0.999969482421875, 0.9999990463256836, 3.550514975358965e-07, 0.9999995231628418, 0.9971006512641907, 0.9999943971633911, 3.0427859201154206e-06, 0.9999572038650513, 4.101276772416895e-06, 0.0011222707107663155, 9.771288023330271e-05, 0.9954338669776917, 2.298387698829174e-05, 0.9999946355819702, 0.0008641679887659848, 0.9999979734420776, 0.997149646282196, 0.00020399723143782467, 0.9990583062171936, 7.855181320337579e-05, 0.9993906021118164, 1.3736513210460544e-05, 0.003059993265196681, 0.999665379524231, 1.751314448483754e-05, 1.8822036054189084e-06, 2.612077651065192e-06, 1.1190346640432836e-06, 0.9999911785125732, 9.330333341495134e-06, 0.9999983310699463, 1.5859228597037145e-06, 0.9999942779541016, 5.376975241233595e-05, 3.153503348585218e-05, 0.9999969005584717, 0.9999829530715942, 2.8920974727952853e-05, 0.9999934434890747, 0.9999151229858398, 0.9999998807907104, 0.9998290538787842, 0.9999226331710815, 0.9999969005584717, 0.9997830986976624, 0.9996961355209351, 2.9660081963811535e-06, 0.1675763577222824, 0.9999935626983643, 1.0, 0.009829326532781124, 0.00011703132622642443, 0.99997878074646, 0.9999881982803345, 0.0008072989294305444, 0.9998917579650879, 1.730119220155757e-05, 0.9999401569366455, 2.9263443138916045e-05, 0.00016289552149828523, 0.9997531771659851, 0.9997944235801697, 2.3169970518210903e-05, 0.9915562868118286, 1.0, 0.9999768733978271, 0.9989560842514038, 0.9998601675033569, 8.065260772127658e-05, 0.0017802647780627012, 1.3158892215869855e-06, 3.5422464861767367e-05, 4.272126898285933e-05, 4.4072601212974405e-07, 2.6594017981551588e-05, 0.9999990463256836, 0.9999817609786987, 0.9997316002845764, 0.9999901056289673, 2.0235138435964473e-06, 9.18851037567947e-06, 0.9990609288215637, 0.9992581009864807, 1.0, 0.0013173316838219762, 0.0018269965657964349, 0.9999878406524658, 0.9998425245285034, 6.3119432525127195e-06, 0.001815279247239232, 0.9990147352218628, 0.9999916553497314, 0.9998975992202759, 0.9999994039535522, 0.9996414184570312, 6.51853497402044e-07, 0.9999864101409912, 4.0630980947753415e-05, 1.35647414936102e-05, 0.9999661445617676, 0.00047181439003907144, 0.9999886751174927, 0.0038455475587397814, 0.9999963045120239, 0.0001155198915512301, 0.00018507638014853, 1.7679680240689777e-05, 4.067959889653139e-05, 0.0009174266597256064, 0.2130216658115387, 1.6727581169106998e-05, 0.9999836683273315, 0.9999650716781616, 0.9998335838317871, 0.9998168349266052, 0.9999635219573975, 0.002714554313570261, 1.2527678336482495e-05, 0.9996902942657471, 0.9998817443847656, 8.777578477747738e-05, 8.893319318303838e-06, 7.983713294379413e-05, 0.00012351418263278902, 0.00042259207111783326, 7.578744407510385e-05, 0.9999430179595947, 0.9999959468841553, 0.00028657796792685986, 0.009675486944615841, 0.999994158744812, 3.1694493372924626e-05, 4.061850177095039e-06, 0.9999996423721313, 0.9999855756759644, 0.9999986886978149, 0.00031912687700241804, 0.00019065402739215642, 0.9996445178985596, 0.9994072914123535, 4.836962489207508e-06, 0.9999994039535522, 0.9999998807907104, 0.0009832876967266202, 0.9999932050704956, 0.0006149206310510635, 0.00015435338718816638, 0.9999761581420898, 0.999998927116394, 0.9932831525802612, 0.999997615814209, 0.9989493489265442, 0.9999392032623291, 0.999996542930603, 1.71989686350571e-05, 1.0, 2.765578983598971e-06, 0.9999953508377075, 0.00010466966341482475, 9.868518873190624e-07, 9.109392522077542e-06, 2.648812824190827e-06, 0.00011428649304434657, 1.5910836737020873e-05, 7.166557043092325e-05, 0.00995603296905756, 1.0, 1.4147783986118156e-05, 0.9999985694885254, 1.0, 0.9999985694885254, 0.3996860086917877, 0.9974995255470276, 0.9993809461593628, 1.2626561556317029e-06, 0.9951062202453613, 3.4409822546876967e-06, 1.3147780919098295e-05, 0.9999774694442749, 0.9999995231628418, 0.9999994039535522, 0.9945564270019531, 0.9999871253967285, 0.9997344613075256, 0.9991011619567871, 9.714085535961203e-07, 6.430004759749863e-06, 0.0007374471751973033, 1.2489658729464281e-07, 3.731191100087017e-06, 0.09995529800653458, 0.10821453481912613, 3.774021251956583e-06, 1.0, 0.0002031050535151735, 0.1269219070672989, 0.9999724626541138, 0.9999414682388306, 9.967888036044315e-05, 0.00015286145207937807, 1.3927993677498307e-05, 0.9999998807907104, 0.9997624754905701, 0.9999785423278809, 3.529393143253401e-05, 0.999995231628418, 0.0010230941697955132, 0.9999716281890869, 0.9999690055847168, 0.00026977117522619665, 0.00030120188603177667, 0.0006247009732760489, 0.9966752529144287, 0.9999994039535522, 0.9555569291114807, 0.9999606609344482, 1.0259540204060613e-06, 0.00011585093307076022, 0.003050951287150383, 7.70759506849572e-05, 0.9999979734420776, 0.9999997615814209, 0.9900107979774475, 0.9999997615814209, 0.9999538660049438, 0.003914995118975639, 0.9999997615814209, 0.999975323677063, 0.9999746084213257, 0.9999995231628418, 6.985684740357101e-05, 0.0016440972685813904, 0.9999995231628418, 0.007610552944242954, 0.9999563694000244, 0.0012222176883369684, 0.0029847980476915836, 4.313683803047752e-06, 9.603196667740121e-05, 1.043240104081633e-06, 3.4570537081890507e-06, 0.0001860915799625218, 0.999847412109375, 0.9995686411857605, 0.00013330492947716266, 3.951430699089542e-05, 0.999934196472168, 0.999426007270813, 0.003360402537509799, 0.05749423801898956, 0.9999998807907104, 0.9918962121009827, 1.7119301674028975e-06, 0.9999656677246094, 5.079465336166322e-05, 1.612120740901446e-06, 3.1840770589042222e-06, 3.825214207608951e-06, 0.0579318031668663, 0.00014331890270113945, 0.9936935305595398, 0.0011202963069081306, 1.699035692581674e-06, 1.2071825494786026e-06, 9.775340714668346e-08, 0.9999980926513672, 0.0005102000432088971, 0.9909075498580933, 0.9999860525131226, 0.9999839067459106, 0.0030245778616517782, 0.0003530186659190804, 0.9999996423721313, 0.0013508558040484786, 5.715352813240315e-07, 0.9999924898147583, 0.9999754428863525, 0.9999991655349731, 1.0, 0.9747783541679382, 0.8358927965164185, 0.9997580647468567, 0.0004394083225633949, 0.00012752962356898934, 0.9983830451965332, 0.9999980926513672, 0.9999171495437622, 0.9999971389770508, 0.9994077682495117, 2.808224962791428e-06, 0.00013112703163642436, 1.8506336346035823e-05, 4.09606218454428e-06]

#Metricas de validación
Best_acc3_4 = 1.0
acc_aux3_4 = 1.0
Recall3_4 = 1.0
Especificidad3_4 = 1.0
Tiempo3_4 = 1281.5566222667694

######## Base4 ########
#MODELO = VGG13_BN
#Datos perdidas
xx_train4_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train4_4 = [0.06611286054054896, 0.03718475105217946, 0.03176395668659681, 0.035679579139859585, 0.033499983486569955, 0.049977500672325675, 0.03832576423883438, 0.03154267050601818, 0.029782057921459645, 0.02783429745906665, 0.02501434154900504, 0.02557175401368259, 0.0353683459997913, 0.02687647215746067, 0.022951157207106367, 0.032210449009765815, 0.030788210503481052, 0.030114352997438407, 0.02865100493769587, 0.028305946906775604, 0.030783664333967516, 0.029934015759715327, 0.02971987921146699, 0.027881798506886872, 0.026230872597223447]
xx_val4_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val4_4 = [0.006274769206841787, 0.006847617692417569, 0.005458566712008582, 0.009541537198755477, 0.0063079289264149135, 0.0046699964337878755, 0.004823059505886502, 0.004057514336374071, 0.003701779411898719, 0.0041736220320065815, 0.0033875601159201727, 0.0053659074008464815, 0.003552725248866611, 0.003428970774014791, 0.0039008702668878767, 0.0031641791264216106, 0.004003899130556318, 0.004437711007065243, 0.003183439042833116, 0.004492042793167962, 0.0037782359454366895, 0.0031120445993211535, 0.004850468370649549, 0.003161414795451694, 0.004307995405462053]
epoch_min4_4 = 10
stop_val4_4 = 0.0033875601159201727

#Datos accuracy
acc_train4_4 = [0.9802469135802468, 0.9867283950617284, 0.9895061728395061, 0.9873456790123457, 0.9885802469135803, 0.983641975308642, 0.9848765432098765, 0.9885802469135803, 0.9895061728395061, 0.990432098765432, 0.9916666666666667, 0.9916666666666667, 0.9848765432098765, 0.991358024691358, 0.9907407407407407, 0.9879629629629629, 0.9888888888888888, 0.9879629629629629, 0.9891975308641975, 0.9888888888888888, 0.9885802469135803, 0.9879629629629629, 0.9888888888888888, 0.9882716049382716, 0.9907407407407407]
acc_val4_4 = [1.0, 0.9972222222222222, 1.0, 0.9944444444444445, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels4_4 = [0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
best_prob4_4 = [9.615926501282956e-06, 0.9999955892562866, 0.9999988079071045, 0.9999988079071045, 5.309447033141623e-07, 1.796412220755883e-06, 0.9999990463256836, 1.144521047535818e-05, 2.2798117527145223e-07, 1.0806208194935607e-07, 1.0, 2.2719207208865555e-06, 3.92756010114681e-05, 0.9999966621398926, 0.9999902248382568, 4.862546848016791e-05, 5.735228114644997e-08, 0.999998927116394, 0.9999970197677612, 0.9820515513420105, 0.00018767516303341836, 0.9999972581863403, 0.9974557757377625, 1.0, 0.9994733929634094, 0.9999996423721313, 0.0001135659113060683, 0.9999938011169434, 0.0003983949718531221, 0.9999948740005493, 0.9999852180480957, 0.9999476671218872, 1.1181653007952264e-06, 1.0720928003138397e-05, 6.323787965811789e-05, 0.9993065595626831, 0.9999982118606567, 0.00020638566638808697, 0.9999862909317017, 9.822429092309903e-06, 0.9999953508377075, 0.9997230172157288, 0.9999359846115112, 7.076726546983991e-07, 0.9999454021453857, 0.0002464682038407773, 6.906942417117534e-06, 0.9999575614929199, 0.0014543486759066582, 0.9999998807907104, 0.9999997615814209, 0.003700532717630267, 0.0004047638503834605, 9.304587001679465e-05, 0.9993108510971069, 0.00012349640019237995, 0.9995496869087219, 3.203321102773771e-05, 0.9999995231628418, 0.9999871253967285, 1.0667304195521865e-05, 0.00211276370100677, 0.9999998807907104, 0.9999997615814209, 0.9999920129776001, 0.9999983310699463, 0.9997918009757996, 0.9999614953994751, 0.9999994039535522, 0.0014668465591967106, 4.288511263439432e-05, 5.998755295877345e-05, 5.881551601305546e-07, 0.00015925410843919963, 3.7153722587390803e-07, 0.9999997615814209, 0.9464136958122253, 1.533575527901121e-06, 0.9172562956809998, 0.9999648332595825, 2.7007388325728243e-06, 0.9999998807907104, 3.5451801522867754e-05, 2.6117884772247635e-05, 0.9999940395355225, 1.2211225111968815e-05, 0.00040449961670674384, 0.9999995231628418, 0.9999997615814209, 0.7367554306983948, 0.9999954700469971, 0.9999957084655762, 0.9999860525131226, 0.9999997615814209, 0.0007973245810717344, 0.00013797541032545269, 0.9999949932098389, 0.9999746084213257, 0.9995075464248657, 8.71100346557796e-05, 0.999997615814209, 0.9999926090240479, 0.9999840259552002, 0.0020986422896385193, 0.999954104423523, 1.5193892011211574e-07, 0.9999617338180542, 0.9999990463256836, 0.9999996423721313, 0.9999967813491821, 0.9993054866790771, 0.9999992847442627, 0.0002303986402694136, 0.9998713731765747, 0.9999493360519409, 0.9998304843902588, 1.7367874534102157e-06, 0.9998210072517395, 9.485050895818858e-07, 0.9999994039535522, 3.5573113564169034e-05, 2.221934119006619e-05, 4.082704890606692e-06, 0.02099895477294922, 0.999996542930603, 0.9999994039535522, 6.88106501911534e-06, 1.3800615306536201e-05, 0.9760604500770569, 2.4494003810104914e-05, 0.9999995231628418, 0.003518808865919709, 4.763548986375099e-07, 0.9999302625656128, 0.9999810457229614, 8.020785458029422e-07, 0.9999992847442627, 6.338881939882413e-05, 0.9999988079071045, 0.9975518584251404, 0.6899815201759338, 4.780660560754768e-07, 1.0, 9.08435779933825e-08, 0.999825656414032, 7.788745278958231e-06, 4.102858213173022e-07, 6.597056199098006e-05, 0.9999966621398926, 0.9962950348854065, 0.9999995231628418, 0.00011389754945412278, 0.999985933303833, 0.9969306588172913, 2.7726360713131726e-05, 1.0748113936642767e-06, 1.4921253566058112e-08, 0.9999711513519287, 1.2735804375552107e-06, 2.1047428163001314e-05, 0.0006752315675839782, 0.9999887943267822, 0.9998883008956909, 0.9999995231628418, 0.9999977350234985, 9.688812860986218e-05, 0.0004026633105240762, 3.289287269581109e-05, 0.9999997615814209, 0.9999388456344604, 2.8032372938469052e-06, 0.00019701418932527304, 0.9999996423721313, 6.404765827028314e-06, 0.9999995231628418, 0.9999854564666748, 6.191604029481823e-07, 0.9999996423721313, 2.476620011293562e-07, 9.569309440848883e-06, 0.9999902248382568, 0.9999994039535522, 0.0141648193821311, 0.00025934388395398855, 0.9999992847442627, 1.7451056919526309e-06, 0.9999289512634277, 4.100232035852969e-05, 0.0001120849046856165, 2.433967893011868e-06, 4.4394931819624617e-07, 0.9999605417251587, 6.658997153863311e-05, 0.0001769020891515538, 6.384270818671212e-05, 0.923716127872467, 8.949208131525666e-05, 1.1930470463994425e-05, 0.9999532699584961, 5.557243639486842e-05, 0.9999997615814209, 0.999998927116394, 0.9999985694885254, 0.9999786615371704, 4.168850864516571e-06, 0.9999057054519653, 1.1800702850450762e-05, 0.9999995231628418, 0.9999980926513672, 0.999997615814209, 0.9725223183631897, 0.0765073224902153, 0.9999743700027466, 0.9999614953994751, 0.9999967813491821, 2.7229463739786297e-05, 0.0003685421834234148, 5.308972959028324e-06, 0.00011973881191806868, 0.9999992847442627, 4.197615339762706e-07, 0.9999929666519165, 0.9999716281890869, 0.9995597004890442, 0.9999997615814209, 0.9999970197677612, 0.9999959468841553, 5.35432072013009e-08, 4.7309836759268364e-07, 0.9998412132263184, 0.999789297580719, 0.9999990463256836, 0.9999961853027344, 6.361351552186534e-06, 5.434324634734367e-07, 9.349940228275955e-05, 6.104757517277903e-07, 3.0004041491338285e-06, 0.9999721050262451, 0.0396001935005188, 0.9999959468841553, 1.0, 3.224087777198292e-05, 0.9990277290344238, 0.00013746706827078015, 1.8179708831667085e-06, 0.00024556188145652413, 0.9999996423721313, 1.7590675270184875e-05, 5.4273405112326145e-06, 5.0631868361961097e-05, 7.458149866579333e-07, 9.468103962717578e-05, 0.9999895095825195, 0.9999535083770752, 0.9999321699142456, 2.559807944635395e-05, 0.9999991655349731, 0.9999521970748901, 2.3417611373588443e-05, 0.0013451273553073406, 2.968800072267186e-05, 1.9091115177616302e-07, 0.0027480428107082844, 2.406397834420204e-05, 0.9785200953483582, 0.9999995231628418, 3.9453607314499095e-05, 0.9999996423721313, 2.710244473291823e-07, 0.9999995231628418, 0.9999083280563354, 0.9999990463256836, 4.4103128260530866e-08, 0.0006428683991543949, 0.9999997615814209, 0.9998502731323242, 9.032970069711155e-07, 0.9999545812606812, 2.119073542417027e-06, 0.0006118534947745502, 3.0060697099543177e-05, 0.9999957084655762, 0.00019403212354518473, 0.9999998807907104, 0.999602735042572, 0.999984860420227, 2.8063614081474952e-06, 2.8982656658627093e-05, 1.0, 1.6751635484979488e-05, 0.00262514129281044, 0.9999966621398926, 0.00011338997137499973, 7.971257218741812e-07, 2.280962917211582e-06, 3.28133637594874e-06, 0.0006850218051113188, 0.9999920129776001, 0.00027637361199595034, 8.595551292955861e-08, 0.9999879598617554, 0.9999986886978149, 0.0005519234109669924, 0.011410772800445557, 1.1781880715489024e-07, 0.9999246597290039, 0.9999998807907104, 0.9999998807907104, 2.2537183212989476e-06, 0.9997977614402771, 0.00023884371330495924, 2.0724801288452e-05, 0.9999998807907104, 2.1449324776767753e-05, 0.0002699319738894701, 0.9999988079071045, 0.9996699094772339, 3.2523596473765792e-06, 0.0008936738013289869, 0.999997615814209, 0.0033760606311261654, 1.1236801356062642e-06, 0.9999961853027344, 1.1580761565710418e-05, 0.00020644845790229738, 0.9999575614929199, 0.9999580383300781, 0.000984524260275066, 7.8513139669667e-06, 0.9997239708900452, 1.2786163097189274e-06, 0.9999301433563232, 0.00012360607797745615, 0.999993085861206, 3.8923581087146886e-06, 2.144645577573101e-06, 0.9998730421066284, 0.9999749660491943, 7.875971277826466e-06, 0.9999994039535522, 0.9999972581863403, 5.923528192397498e-07, 9.879263416223694e-07, 0.9998466968536377, 4.928803537040949e-05, 0.0019939022604376078, 8.2634056752795e-07, 0.9999867677688599, 0.00019414797134231776, 5.610495009022998e-06, 0.9999929666519165, 0.9999973773956299, 2.013984783388878e-07, 0.00011311123671475798, 2.097981450788211e-05, 1.8394921426079236e-05, 0.9998175501823425, 0.00019554859318304807, 2.5412046511519293e-07]

#Metricas de validación
Best_acc4_4 = 1.0
acc_aux4_4 = 1.0
Recall4_4 = 1.0
Especificidad4_4 = 1.0
Tiempo4_4 = 1286.5230903625488

######## Base5 ########
#MODELO = VGG13_BN
#Datos perdidas
xx_train5_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train5_4 = [0.05368472711171633, 0.033590879484459206, 0.030955217594717754, 0.02843947324119968, 0.041340955631968414, 0.028433059174337506, 0.023751156639169765, 0.028758224265442955, 0.02620852631919178, 0.024912337830037247, 0.025480027064497087, 0.024427652892507153, 0.029243738719342667, 0.023674714979198243, 0.022612835687619667, 0.024556351646229074, 0.02741906173803188, 0.025078284666862016, 0.02312943293356601, 0.0264890831929666, 0.02312406650111999, 0.02222992855089682, 0.029386001503393974, 0.027229081885314282, 0.023796939491121858]
xx_val5_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val5_4 = [0.005082942545413971, 0.0019718875487645467, 0.002169468171066708, 0.0014804886447058784, 0.002115182081858317, 0.001219091812769572, 0.0011993000904719036, 0.0010522266228993734, 0.0009120904737048679, 0.001415744423866272, 0.0014124178224139744, 0.0008072455724080404, 0.0009821239444944593, 0.0010122289260228476, 0.0008546597427792019, 0.001124296916855706, 0.001230240199300978, 0.0008860137727525499, 0.0007760239972008599, 0.0010547558466593423, 0.0009729855590396457, 0.001009543736775716, 0.0011380513509114583, 0.0009885066085391575, 0.001455263959036933]
epoch_min5_4 = 11
stop_val5_4 = 0.0008072455724080404

#Datos accuracy
acc_train5_4 = [0.9796296296296296, 0.9873456790123457, 0.9882716049382716, 0.9895061728395061, 0.9839506172839506, 0.9895061728395061, 0.9907407407407407, 0.9882716049382716, 0.9910493827160494, 0.990432098765432, 0.9919753086419753, 0.9907407407407407, 0.9876543209876543, 0.9919753086419753, 0.9922839506172839, 0.9907407407407407, 0.9898148148148148, 0.9910493827160494, 0.9919753086419753, 0.9888888888888888, 0.9910493827160494, 0.9919753086419753, 0.9882716049382716, 0.9888888888888888, 0.990432098765432]
acc_val5_4 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels5_4 = [1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0]
best_prob5_4 = [0.9999783039093018, 0.999739944934845, 1.4660030501545407e-05, 0.9998301267623901, 9.130468970397487e-05, 0.999997615814209, 2.368382894246679e-07, 0.9999996423721313, 0.9999960660934448, 2.3095491997082718e-05, 5.4316901696438435e-06, 0.9999967813491821, 0.9999827146530151, 0.9999933242797852, 9.791369848244358e-06, 0.0004137854266446084, 0.9999649524688721, 0.9999885559082031, 0.9999152421951294, 0.999957799911499, 1.3654164831677917e-05, 0.9998714923858643, 0.9995651841163635, 2.9199970867921365e-06, 0.9997827410697937, 0.9999980926513672, 0.002793141407892108, 0.9999855756759644, 6.645637040492147e-05, 0.0001698798587312922, 3.111861360594048e-06, 1.1946498261750094e-06, 2.1903449578530854e-06, 0.00010104896500706673, 7.286528216354782e-07, 0.9999866485595703, 2.902389496739488e-06, 4.5071747081237845e-06, 0.9999974966049194, 0.9981324076652527, 0.0006889336509630084, 0.0011139153502881527, 4.2869917393772994e-08, 0.9996626377105713, 0.9999592304229736, 0.9999983310699463, 0.00013555129407905042, 0.9999994039535522, 0.998689591884613, 0.9999892711639404, 0.000216947024455294, 0.9996330738067627, 8.811697398414253e-07, 0.9999974966049194, 5.369261089072097e-07, 1.0, 0.9973918199539185, 0.9999970197677612, 0.00018861336866393685, 0.9999830722808838, 2.9694114346057177e-05, 0.009056051261723042, 0.9999960660934448, 0.9999648332595825, 7.4865788519673515e-06, 0.9999997615814209, 0.999920129776001, 0.9999992847442627, 0.9999986886978149, 0.0003376351378392428, 0.9999990463256836, 0.00014287202793639153, 0.999997615814209, 0.9999929666519165, 0.9999990463256836, 2.174914470742806e-06, 0.9999988079071045, 0.9999631643295288, 0.9999511241912842, 4.384882686281344e-06, 0.9999226331710815, 0.9999997615814209, 0.9999884366989136, 5.791946387034841e-05, 0.0004387227527331561, 0.0005491647752933204, 0.9998363256454468, 0.00027852231869474053, 0.9999934434890747, 0.9989921450614929, 1.175494344352046e-05, 0.0007170805474743247, 0.019361667335033417, 0.9784137010574341, 0.9999462366104126, 1.0, 0.00010372298129368573, 0.9996793270111084, 0.9999849796295166, 2.251153546239948e-06, 0.00027250716811977327, 0.9999486207962036, 0.9999450445175171, 0.9998447895050049, 0.0003714376362040639, 0.9827739000320435, 2.098842287523439e-06, 8.665791142448143e-07, 0.9999896287918091, 0.00012082797184120864, 0.9999998807907104, 0.9999957084655762, 0.0045916358940303326, 0.0011360974749550223, 0.9999847412109375, 8.634397090645507e-06, 5.281509629639913e-07, 1.4338090181809093e-07, 0.9994382262229919, 1.0, 6.2327940213435795e-06, 0.0012857321416959167, 1.0, 0.9999971389770508, 0.002146192593500018, 9.612945461867639e-08, 2.0072615569688423e-08, 1.0, 1.0, 0.006357838399708271, 0.9999991655349731, 0.9999998807907104, 0.00040463151526637375, 3.268824002589099e-05, 0.9999923706054688, 0.999993085861206, 0.00011315041047055274, 1.0, 0.000665852683596313, 0.9999994039535522, 0.0003158699255436659, 0.9999997615814209, 0.9999966621398926, 8.152006216732843e-07, 1.0, 1.4941459085093811e-05, 0.014453272335231304, 0.9999954700469971, 0.0003714931081049144, 0.0005739805637858808, 0.9999426603317261, 0.9993476271629333, 0.00030977270216681063, 0.9999680519104004, 0.9999455213546753, 1.7256879800697789e-06, 0.00028464983915910125, 0.9999995231628418, 0.9999978542327881, 0.9999728202819824, 0.9990703463554382, 0.9999998807907104, 0.9997840523719788, 0.9998995065689087, 1.0, 0.999970555305481, 0.9999841451644897, 0.9997548460960388, 6.711579044349492e-05, 1.6154035620274954e-07, 0.9999641180038452, 2.936394594144076e-06, 0.0005755783640779555, 0.999997615814209, 0.9999988079071045, 0.0001709754578769207, 4.5418179070111364e-05, 0.00092295155627653, 0.0004815117863472551, 2.973826553898107e-07, 0.9999560117721558, 1.5820512544451049e-06, 0.9996644258499146, 1.5046593944134656e-06, 0.00018946159980259836, 0.0005235802964307368, 0.9999167919158936, 0.00021907681366428733, 8.382624218938872e-05, 8.763778168940917e-06, 7.624536806360993e-07, 3.631581785157323e-05, 1.8719641730058356e-06, 0.00012466846965253353, 0.999984860420227, 0.9999991655349731, 1.265689547835791e-06, 8.571556122660695e-07, 3.210867021152808e-07, 0.9994416832923889, 0.0029567608144134283, 7.4193058026139624e-06, 1.0, 0.9999995231628418, 2.7871850761584938e-05, 9.337910000795091e-07, 0.03297124058008194, 0.9999995231628418, 0.99998939037323, 4.78283800475765e-05, 0.9999991655349731, 0.0001649502810323611, 9.006699883684632e-07, 0.9997838139533997, 0.9998013377189636, 4.9930417134191885e-08, 1.6956146282609552e-05, 4.826346048503183e-05, 0.004961411934345961, 0.0001874941517598927, 0.999998927116394, 2.550541285017971e-05, 0.9998817443847656, 9.389450497110374e-06, 1.0684203516575508e-05, 0.9999992847442627, 0.00020632505766116083, 0.9999995231628418, 0.9999880790710449, 0.9999936819076538, 1.8187495243182639e-06, 0.9895468354225159, 0.9999904632568359, 3.4110760793737427e-07, 1.0059054744715468e-07, 9.052176756085828e-05, 0.9999994039535522, 0.9999988079071045, 0.9997327923774719, 0.9999465942382812, 1.2733617040794343e-06, 0.9994157552719116, 0.9999783039093018, 0.9999997615814209, 0.9999998807907104, 0.9801614880561829, 1.0, 0.0009838874684646726, 0.9999854564666748, 0.0044027226977050304, 3.965287760365754e-06, 3.8816042433609255e-06, 6.696202490275027e-06, 1.0, 0.9999943971633911, 0.9999840259552002, 2.3437239633494755e-06, 0.0017935401992872357, 4.676406206272077e-06, 2.8564536478370428e-05, 0.00012543627235572785, 0.9999895095825195, 0.9999997615814209, 0.00011320025805616751, 0.003019866766408086, 2.769131413060677e-07, 0.9999854564666748, 1.7339694750262424e-05, 0.01033813040703535, 0.9982763528823853, 2.318737415407668e-06, 7.627656941622263e-07, 5.632738520944258e-06, 9.830756607698277e-05, 1.0, 1.0, 0.9999935626983643, 4.130869456275832e-06, 0.0001849052932811901, 0.00021931048831902444, 1.0, 3.168409250520199e-07, 2.0113764549023472e-05, 0.9999991655349731, 2.301987296959851e-05, 9.997215238399804e-05, 0.9998995065689087, 6.803787255194038e-05, 0.9999731779098511, 1.6620328096905723e-05, 0.9999326467514038, 0.9999337196350098, 3.8598365790676326e-05, 2.761904625003808e-06, 0.9999995231628418, 9.924486221279949e-05, 0.9999963045120239, 4.674604383581027e-07, 0.9999794960021973, 4.104701645246678e-07, 3.0415552828344516e-05, 0.9999970197677612, 1.0, 9.913363464875147e-05, 5.552150810217427e-07, 0.9999973773956299, 0.9999960660934448, 8.390525181312114e-05, 6.899808067828417e-05, 3.738318628165871e-05, 0.9999953508377075, 2.412144795016502e-06, 0.999998927116394, 2.5218443624908105e-05, 3.817717697529588e-06, 0.9999866485595703, 0.9978744983673096, 0.9998995065689087, 0.9999942779541016, 0.9998377561569214, 3.8234215026022866e-05, 1.5232341866067145e-05, 0.9999998807907104, 0.00032958193332888186, 1.0, 1.0, 1.3097919691062998e-06, 0.0005561885773204267, 0.9999890327453613, 0.9999866485595703, 0.9999992847442627, 1.4023044059285894e-06, 0.9999953508377075, 1.2656109902309254e-06, 0.00015457194240298122, 3.1113901059143245e-05, 0.9999995231628418, 6.656241566815879e-07, 0.9998550415039062, 8.221931580010278e-07, 0.030580539256334305, 0.9998468160629272, 1.894122760859318e-07, 0.00012400677951518446, 2.957597757813346e-07, 0.001709187519736588, 0.00028450004174374044, 0.9999911785125732, 0.999996542930603, 0.001220456324517727, 5.9386356952018104e-06, 0.9999947547912598, 5.8430769058759324e-06, 0.9999786615371704, 0.9997263550758362, 0.9999996423721313, 0.9999942779541016, 0.9752330183982849, 1.1028642802557442e-05, 0.9999594688415527]

#Metricas de validación
Best_acc5_4 = 1.0
acc_aux5_4 = 1.0
Recall5_4 = 1.0
Especificidad5_4 = 1.0
Tiempo5_4 = 1289.5601437091827

######## Base6 ########
#MODELO = VGG13_BN
#Datos perdidas
xx_train6_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train6_4 = [0.043667309004583474, 0.026437120378753284, 0.0263511224201432, 0.03115837307255945, 0.025881513907217685, 0.03956145153001503, 0.026305163559722313, 0.025273463451935922, 0.0227918007031635, 0.025912229505218105, 0.024014986913513253, 0.0222093783043049, 0.0222225691801236, 0.023572254190106452, 0.02265275515339993, 0.025152132834917233, 0.032835531142758737, 0.019262425113975266, 0.020619244606774532, 0.01730111159476233, 0.01822308725045051, 0.023410636168203235, 0.026134807956807407, 0.025383042718893217, 0.02561294921202424]
xx_val6_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val6_4 = [0.010518156240383785, 0.002584963457451926, 0.00541146629386478, 0.004544492562611898, 0.008706817693180508, 0.0032396258579360116, 0.0030060514807701113, 0.0029947901765505474, 0.0035944332679112754, 0.004855139884683821, 0.005274955100483364, 0.005591309732860989, 0.006004248062769572, 0.005164016948805915, 0.0045091766450140215, 0.00492446372906367, 0.0039599960048993426, 0.004018305904335446, 0.0050516888499259945, 0.0037655242615275914, 0.006526629461182488, 0.0036256399419572617, 0.004504840075969696, 0.004526872932910919, 0.00418475286828147]
epoch_min6_4 = 1
stop_val6_4 = 0.002584963457451926

#Datos accuracy
acc_train6_4 = [0.9851851851851852, 0.9907407407407407, 0.991358024691358, 0.990432098765432, 0.9916666666666667, 0.9867283950617284, 0.9901234567901235, 0.9919753086419753, 0.991358024691358, 0.9898148148148148, 0.9922839506172839, 0.9919753086419753, 0.9929012345679012, 0.9919753086419753, 0.9907407407407407, 0.9929012345679012, 0.9876543209876543, 0.9938271604938271, 0.9925925925925926, 0.9944444444444445, 0.9925925925925926, 0.9922839506172839, 0.9901234567901235, 0.9919753086419753, 0.991358024691358]
acc_val6_4 = [0.9944444444444445, 1.0, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 1.0, 1.0, 1.0, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222]

#Datos curva ROC
best_labels6_4 = [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0]
best_prob6_4 = [1.0, 0.00011881855607498437, 5.768221171820187e-07, 0.0585528239607811, 8.660142157168593e-06, 2.07658922590781e-06, 3.0808178053121082e-06, 0.9999992847442627, 0.9999997615814209, 2.9124848879291676e-06, 3.640961949713528e-05, 0.9999302625656128, 2.2962276489124633e-05, 5.4496591474162415e-05, 4.142037596466253e-06, 7.085606625878427e-07, 0.9999997615814209, 0.9999992847442627, 0.9999998807907104, 0.9999980926513672, 0.9999998807907104, 0.9998049139976501, 0.9999918937683105, 0.9999974966049194, 7.2641432780073956e-06, 0.00021707906853407621, 0.9999991655349731, 0.996285080909729, 2.91842661681585e-06, 3.11748735839501e-05, 0.20021316409111023, 2.170505467802286e-06, 0.00018704828107729554, 0.00042745500104501843, 0.9999998807907104, 0.0002571081568021327, 0.9999899864196777, 0.9999873638153076, 5.679341484210454e-05, 0.999998927116394, 0.9998369216918945, 1.1507202088978374e-06, 0.0011120388517156243, 0.9999986886978149, 1.9756176698138006e-05, 0.9999998807907104, 0.9999998807907104, 0.9999991655349731, 0.9999985694885254, 0.9998667240142822, 1.0, 5.5805146985221654e-05, 0.9999996423721313, 7.961230039654765e-06, 0.9996203184127808, 0.9999970197677612, 0.9983153343200684, 0.999977707862854, 0.00015695182082708925, 1.0, 0.9996390342712402, 4.554579845716944e-06, 2.596413833089173e-05, 2.6471761884749867e-06, 0.9999872446060181, 0.9999476671218872, 8.144900493789464e-05, 3.600901436584536e-06, 0.999997615814209, 0.9999964237213135, 0.00016778706049080938, 2.887804112106096e-05, 0.00022362479649018496, 8.712867725080287e-07, 0.9995579123497009, 0.9999994039535522, 0.00019148550927639008, 0.9999957084655762, 0.9999774694442749, 0.00018443727458361536, 0.9999964237213135, 0.00011256700963713229, 1.7392592610576685e-07, 4.222594361635856e-05, 0.00046681801904924214, 0.9999990463256836, 9.425713869859464e-06, 0.9999963045120239, 1.0369899428042118e-06, 0.9999861717224121, 0.9993687272071838, 0.9999504089355469, 0.9998883008956909, 0.9999804496765137, 0.9999879598617554, 0.00046322704292833805, 4.813311079487903e-06, 0.9997275471687317, 2.643800769419613e-07, 0.9999998807907104, 1.0283588380843867e-06, 1.0, 0.00035914022009819746, 1.1321530109853484e-05, 1.0, 0.9998477697372437, 0.999957799911499, 7.419076268888602e-07, 0.9998749494552612, 0.0001250878704013303, 0.0001717646955512464, 5.827725544804707e-05, 0.9999997615814209, 0.9999995231628418, 0.9999997615814209, 0.9999986886978149, 4.3873674258065876e-06, 0.9999403953552246, 7.090645522112027e-05, 4.539959627436474e-05, 1.743362554407213e-05, 0.9999516010284424, 0.0034412089735269547, 1.8191727804151014e-06, 0.9999997615814209, 0.9999561309814453, 0.009021719917654991, 7.786375499563292e-05, 0.9999707937240601, 1.026815425575478e-05, 0.999700665473938, 0.9999998807907104, 5.694526805655187e-08, 0.0007415543077513576, 0.00028007474611513317, 0.9999456405639648, 4.43543067376595e-05, 0.0007882024510763586, 9.929144289344549e-05, 0.00021452027431223541, 0.00036613133852370083, 0.9998513460159302, 0.00013398172450251877, 1.0, 0.9999829530715942, 0.9999611377716064, 0.9999421834945679, 0.9999995231628418, 0.9994314312934875, 1.1353047739248723e-05, 0.000336599099682644, 0.008006560616195202, 0.0005384862888604403, 0.015757810324430466, 0.9999686479568481, 0.00013625339488498867, 0.9999918937683105, 0.9999997615814209, 0.999700665473938, 2.4426344680250622e-06, 2.0241901665940532e-07, 5.262246304482687e-06, 4.450412598089315e-05, 1.3498711268766783e-06, 2.643933294166345e-05, 8.2141004895675e-06, 0.9999992847442627, 0.9996324777603149, 1.450735140906545e-07, 0.99996018409729, 0.9999858140945435, 1.572610199218616e-05, 9.507833055977244e-06, 0.9999996423721313, 1.04747969089658e-05, 6.940697494428605e-05, 0.9999982118606567, 0.999993085861206, 1.0, 9.749951459525619e-06, 0.9999910593032837, 1.038417963172833e-06, 1.0, 3.0297903776954627e-06, 0.9999982118606567, 0.9999991655349731, 0.9999971389770508, 0.9999692440032959, 7.4199783739459235e-06, 0.9999978542327881, 9.346445585833862e-05, 0.9999982118606567, 0.999954342842102, 9.053057146957144e-05, 0.9963706731796265, 0.0004600312968250364, 0.9999886751174927, 0.9999994039535522, 2.559788481448777e-05, 5.138808046467602e-05, 3.851952715194784e-05, 0.9999985694885254, 0.9999911785125732, 0.00026081682881340384, 0.9999985694885254, 0.9999936819076538, 5.094338484923355e-05, 0.9997879862785339, 1.8593155459711852e-08, 0.00011951740452786908, 0.9995686411857605, 0.9999972581863403, 0.9999072551727295, 0.00017172259686049074, 0.9999959468841553, 1.7348263270378084e-07, 1.0, 0.007209313567727804, 0.9999964237213135, 6.406562170013785e-05, 0.999991774559021, 1.4668856238131411e-05, 1.5011485629656818e-05, 7.840419129934162e-06, 0.9999663829803467, 1.0, 3.3337000786559656e-05, 0.9999595880508423, 1.0974306860589422e-05, 0.9999991655349731, 2.1945643311482854e-06, 3.9650949474889785e-06, 0.9999998807907104, 1.813505292602713e-07, 0.0014036173233762383, 0.9999920129776001, 3.347919346197159e-06, 0.9998939037322998, 2.3314307327382267e-05, 4.164785423199646e-06, 0.9999980926513672, 0.9999996423721313, 0.007312854286283255, 0.9999982118606567, 6.260801796997839e-07, 0.999903678894043, 9.202488399751019e-06, 1.855776281445287e-05, 3.8098216919024708e-06, 9.80003005679464e-06, 0.9999967813491821, 0.9981063604354858, 1.0, 0.012895659543573856, 0.9991121888160706, 0.9999990463256836, 0.00014308362733572721, 0.9999988079071045, 3.7403693568194285e-05, 2.5572064714651788e-06, 0.00016628135927021503, 0.9991676807403564, 0.9999421834945679, 0.9999998807907104, 0.9999765157699585, 1.4501474652206525e-06, 0.9999992847442627, 0.00011946408631047234, 3.393923179828562e-05, 1.0, 5.52375468032551e-06, 9.004725143313408e-07, 0.9999985694885254, 0.9999967813491821, 2.7945598048972897e-06, 0.9999508857727051, 1.45610201798263e-05, 1.8564542187959887e-06, 0.9999998807907104, 1.0, 0.05029670521616936, 2.865811211449909e-06, 0.9999954700469971, 0.00010045081580756232, 0.9999966621398926, 2.1896482849115273e-07, 8.840730515657924e-06, 0.9999892711639404, 1.8308044673176482e-05, 0.9999915361404419, 0.00011873066978296265, 0.9999185800552368, 0.9999969005584717, 4.642218755179783e-06, 0.9998043179512024, 5.1726527772189e-07, 7.555552292615175e-05, 0.9999856948852539, 4.237388111505425e-06, 0.9999997615814209, 0.9999507665634155, 0.9999939203262329, 7.836756594770122e-06, 0.9902138113975525, 0.9999855756759644, 0.3243620693683624, 1.0, 4.074354365002364e-06, 0.9999996423721313, 0.9999998807907104, 0.9998824596405029, 1.1124864613520913e-05, 0.0001177438025479205, 0.00021872618526685983, 1.1737497516151052e-06, 1.0, 0.9999208450317383, 0.999995231628418, 0.0002528413024265319, 0.9999819993972778, 3.2006919354898855e-05, 1.6299405842801207e-06, 0.999948263168335, 6.621684747187828e-07, 5.531763963517733e-05, 0.0006142833735793829, 6.366847173921997e-07, 0.9999148845672607, 1.0, 0.9999984502792358, 1.5006842204456916e-06, 0.008599548600614071, 4.120199264434632e-06, 0.9999802112579346, 0.9999282360076904, 0.9999995231628418, 0.9999947547912598, 6.9072325459274e-06, 0.9999990463256836, 0.9999974966049194, 3.596405576899997e-06, 0.9999986886978149, 0.9999891519546509, 4.774812259711325e-05, 0.9999959468841553, 0.9999521970748901, 2.8417553039616905e-06, 5.08351547523489e-07, 3.4802160371327773e-06, 4.588945739669725e-05, 0.003458680585026741, 4.885368252871558e-05, 0.9994346499443054, 0.9223847985267639, 2.8163996830699034e-05, 0.9999985694885254, 0.9999998807907104, 5.161238004802726e-05, 0.9998748302459717, 0.0002783410018309951]

#Metricas de validación
Best_acc6_4 = 1.0
acc_aux6_4 = 1.0
Recall6_4 = 1.0
Especificidad6_4 = 1.0
Tiempo6_4 = 1286.1060304641724

######## Base7 ########
#MODELO = VGG13_BN
#Datos perdidas
xx_train7_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train7_4 = [0.04781743505118806, 0.025300733101220777, 0.03004875084684219, 0.01960477336873243, 0.020121415069809665, 0.016948559126000344, 0.022895709259642495, 0.023237342056300906, 0.02314845304246302, 0.016187415971064274, 0.01575591403209133, 0.023795693772442547, 0.020637098258292234, 0.017800244679789485, 0.01804742619027326, 0.019203901916374395, 0.016554117911023857, 0.02152744660406937, 0.01687083425529209, 0.022243998052529346, 0.023017797582311393, 0.03189826855137024, 0.01871769075960289, 0.02045956846372581, 0.023056958321435953]
xx_val7_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val7_4 = [0.015700843764675988, 0.010886763698524898, 0.012437503039836883, 0.016705200903945498, 0.01609599995944235, 0.00971439133087794, 0.010489030679066976, 0.011149481270048353, 0.010231402350796594, 0.010635176632139419, 0.012704627215862274, 0.01264078716437022, 0.011979760395156013, 0.014891665511661106, 0.0085033623708619, 0.00936801102426317, 0.017123141884803773, 0.013709275424480439, 0.010460800098048315, 0.011942477193143633, 0.011175195376078287, 0.013548487342066234, 0.011699434452586704, 0.012817189759678311, 0.011202055878109402]
epoch_min7_4 = 5
stop_val7_4 = 0.00971439133087794

#Datos accuracy
acc_train7_4 = [0.9805555555555555, 0.991358024691358, 0.9879629629629629, 0.9944444444444445, 0.9919753086419753, 0.9941358024691358, 0.9919753086419753, 0.9932098765432098, 0.9916666666666667, 0.9953703703703703, 0.9941358024691358, 0.9922839506172839, 0.9935185185185185, 0.9944444444444445, 0.994753086419753, 0.994753086419753, 0.9944444444444445, 0.9919753086419753, 0.9941358024691358, 0.9932098765432098, 0.9925925925925926, 0.9888888888888888, 0.9935185185185185, 0.9932098765432098, 0.991358024691358]
acc_val7_4 = [0.9916666666666667, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445]

#Datos curva ROC
best_labels7_4 = [1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
best_prob7_4 = [0.9999983310699463, 0.9999980926513672, 1.0, 0.9998775720596313, 8.278078166767955e-05, 0.9999973773956299, 3.104188363067806e-05, 3.5278736731925164e-07, 7.877760799601674e-05, 1.5913360584818292e-06, 2.472875712555833e-05, 1.1613265371579473e-07, 2.9089458166708937e-06, 0.9999902248382568, 0.9999995231628418, 4.327773694967618e-08, 0.9999978542327881, 0.9999986886978149, 2.3115897420211695e-05, 6.378094985848293e-05, 7.47426602174528e-05, 9.320812387159094e-05, 0.0001190029361168854, 0.9999850988388062, 7.255343462020392e-06, 0.9998339414596558, 2.7045227398048155e-05, 8.732508263165073e-07, 0.9999862909317017, 0.999993085861206, 3.0824635359749664e-06, 0.9984524250030518, 0.999982476234436, 0.0007857273449189961, 3.523082625633833e-07, 2.0253874026820995e-05, 0.9999836683273315, 0.9999759197235107, 1.0, 0.9999704360961914, 1.4532401110045612e-05, 0.9999878406524658, 1.0, 0.9999821186065674, 0.9999996423721313, 0.9999998807907104, 6.721724639646709e-05, 0.9999920129776001, 0.9999984502792358, 0.9999996423721313, 5.32346803083783e-06, 0.999993085861206, 0.00025269819889217615, 1.5481880836887285e-05, 2.3464316711852007e-07, 0.9999951124191284, 0.9999573230743408, 0.9999978542327881, 0.9999904632568359, 0.9997312426567078, 3.3635847103141714e-06, 0.9999898672103882, 3.8140475226100534e-05, 0.9999947547912598, 2.2071280909585766e-06, 0.9999814033508301, 4.883219389739679e-06, 0.9996473789215088, 0.9999494552612305, 0.9999998807907104, 4.8292745304934215e-06, 0.0015217309119179845, 5.980442438158207e-06, 0.9996334314346313, 2.999185255703196e-08, 0.9999696016311646, 2.1887346520088613e-05, 2.466214937157929e-05, 5.491138381330529e-06, 1.2506230859798961e-06, 0.9999974966049194, 8.890860044630244e-06, 0.9999997615814209, 0.0099007748067379, 1.0034322258434258e-05, 0.9999998807907104, 0.9999843835830688, 0.9972856044769287, 3.492626274237409e-05, 0.00019331672228872776, 6.440246215788648e-05, 0.9999951124191284, 6.258193752728403e-05, 4.7348879888886586e-05, 0.9999998807907104, 0.9998910427093506, 0.999886155128479, 6.710414891131222e-05, 0.9999992847442627, 0.999771773815155, 0.9999983310699463, 2.057322944892803e-06, 0.953474223613739, 7.189362349890871e-08, 2.7053558824263746e-06, 0.9999991655349731, 0.9999940395355225, 4.213996263047193e-08, 0.003252760274335742, 3.6834685488429386e-06, 0.9999982118606567, 0.9999996423721313, 1.1380504474800546e-05, 1.0, 0.9999709129333496, 0.003030562773346901, 0.9999685287475586, 0.9999998807907104, 2.173343273170758e-05, 1.7713888382786536e-06, 6.607365889976791e-07, 0.9999812841415405, 0.9999966621398926, 0.9930621981620789, 0.9999994039535522, 1.0, 6.063815476409218e-07, 0.9997679591178894, 1.493725676482427e-08, 0.9998642206192017, 0.9999977350234985, 4.862836249230895e-07, 0.9999309778213501, 0.9999997615814209, 1.9887913367711008e-05, 8.57096256368095e-06, 0.9999997615814209, 1.3985541045258287e-06, 0.12598814070224762, 5.282751317281509e-06, 0.9999996423721313, 0.0002788172278087586, 4.2408851186337415e-06, 0.0003734395722858608, 1.3079977634333773e-07, 0.9999992847442627, 0.999993085861206, 0.9999994039535522, 0.5807006359100342, 4.436382425865304e-07, 0.9999963045120239, 0.9998794794082642, 2.482782974766451e-06, 0.9999904632568359, 0.9999996423721313, 1.1580871614569332e-05, 0.9998428821563721, 0.9999973773956299, 2.6808471375261433e-05, 1.0, 1.5945643099257722e-06, 0.9997541308403015, 0.16662493348121643, 0.9999909400939941, 2.0901742914247734e-07, 1.269434193318375e-07, 0.9999994039535522, 1.3447531273413915e-05, 0.9998236298561096, 3.4231343306601048e-06, 0.9999948740005493, 0.02669961377978325, 0.9999834299087524, 0.9999610185623169, 0.9999998807907104, 0.9999866485595703, 2.9305501811904833e-05, 1.869644989938024e-07, 0.0031025020871311426, 0.999995231628418, 9.039467840921134e-06, 0.999988317489624, 0.999946117401123, 0.9999806880950928, 3.5470728221298486e-07, 1.0, 0.9999955892562866, 0.9999983310699463, 0.00010116137855220586, 0.9999839067459106, 0.9999935626983643, 1.5734281078039203e-06, 1.1183455171703827e-05, 0.9999988079071045, 1.397903474753548e-06, 0.0001266185281565413, 6.241391048433798e-08, 1.8865488300434663e-06, 1.9928962501580827e-05, 5.13687809871044e-05, 0.9999972581863403, 0.0011552238138392568, 0.9999998807907104, 0.00014877803914714605, 0.022324269637465477, 1.9107285424979636e-06, 0.9999998807907104, 2.6967443318426376e-06, 9.566282415107707e-07, 9.645441605243832e-05, 4.706692379841115e-06, 3.335289875394665e-05, 0.9999990463256836, 7.158401786000468e-06, 0.9999961853027344, 8.089677976386156e-06, 0.9999798536300659, 2.257971800645464e-06, 0.9999809265136719, 0.9999732971191406, 0.0001330448576482013, 0.9999803304672241, 2.491440227458952e-07, 0.9931071400642395, 0.9999992847442627, 0.9999948740005493, 1.3482625945471227e-05, 0.9999963045120239, 7.292556603033518e-08, 0.9999938011169434, 9.72864290815778e-06, 0.00016006706573534757, 0.9990363121032715, 0.9999817609786987, 0.2153903841972351, 2.38727722035037e-07, 3.0692373087504166e-08, 0.9999961853027344, 0.9999995231628418, 0.9999529123306274, 0.9999998807907104, 0.9999979734420776, 0.9999998807907104, 1.585483732924331e-05, 0.00013385237252805382, 0.000126932849525474, 6.011430286889663e-08, 0.999990701675415, 4.935576725983992e-06, 0.9999998807907104, 0.0005499822436831892, 7.532564723078394e-06, 0.9999979734420776, 1.70862658706028e-05, 3.92268105997573e-07, 0.9999648332595825, 0.999991774559021, 3.662647577584721e-05, 5.742139705944282e-07, 0.9999990463256836, 4.991928221897979e-07, 3.0205964662854967e-07, 3.302543746031006e-06, 1.322350762222868e-08, 3.6568467294273432e-06, 3.299981835880317e-05, 8.874697954297517e-08, 0.00018742478277999908, 1.1758148502849508e-05, 0.9999790191650391, 0.9999918937683105, 1.0, 0.9993583559989929, 0.9999783039093018, 1.0, 1.0, 0.00034947291715070605, 6.09411245022784e-07, 0.9999966621398926, 0.00032116638612933457, 1.0, 3.29737740685232e-05, 0.00010657256643753499, 2.1203388769208686e-06, 0.0007488346309401095, 2.8539115959347328e-08, 3.775849961584754e-07, 5.286503323986835e-07, 0.9999579191207886, 0.8163605332374573, 8.296546002384275e-05, 6.1583355090988334e-06, 1.8066028133034706e-05, 0.9998960494995117, 0.9999997615814209, 0.9999972581863403, 0.9999738931655884, 0.9999955892562866, 0.9999970197677612, 0.9999997615814209, 0.9998660087585449, 0.9998928308486938, 0.9999995231628418, 0.9999995231628418, 0.8065018653869629, 2.502549068594817e-05, 0.9999990463256836, 0.9999986886978149, 0.9998792409896851, 8.060198410930752e-07, 0.999976634979248, 1.0689013834053185e-05, 0.00037463041371665895, 1.1873616131197196e-05, 6.521714385598898e-05, 8.452523616142571e-06, 1.8155037651013117e-06, 1.0, 1.0, 0.9999974966049194, 4.05368581368748e-07, 0.9999967813491821, 7.180075044743717e-05, 2.6183968770965294e-08, 0.9999500513076782, 0.9999995231628418, 0.004516060929745436, 0.9999995231628418, 0.9999809265136719, 0.00018812809139490128, 6.19413549429737e-06, 0.9999884366989136, 0.9999995231628418, 1.8186609622716787e-06, 3.2833700970513746e-05, 9.318570164396078e-07, 0.9999996423721313, 1.596635343048547e-06, 0.9999986886978149, 0.999994158744812, 0.9999995231628418, 7.539451326010749e-05, 0.9999996423721313, 0.9999957084655762, 0.9999998807907104, 0.9999803304672241, 6.802448751841439e-06, 0.9999994039535522, 0.00018229307897854596, 0.9986234903335571, 9.18616060516797e-05, 1.0, 4.9787864497830014e-08, 0.9999998807907104, 0.00045410505845211446, 0.0004788869118783623, 9.451028017792851e-05, 0.9999992847442627, 9.152444704341178e-07, 1.2016713526463718e-06]

#Metricas de validación
Best_acc7_4 = 0.9944444444444445
acc_aux7_4 = 0.9953703703703703
Recall7_4 = 0.989010989010989
Especificidad7_4 = 1.0
Tiempo7_4 = 1284.717532634735

######## Base8 ########
#MODELO = VGG13_BN
#Datos perdidas
xx_train8_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train8_4 = [0.041756003727147606, 0.035934582913731354, 0.028317663745011813, 0.03200699362305947, 0.023687874084269558, 0.025697966545452307, 0.02865413382281492, 0.018427473306655884, 0.02506984610248495, 0.022202827413508923, 0.0164575369545707, 0.016001703488973923, 0.017135305629100327, 0.018534986426432927, 0.017090910912295917, 0.015346420169980437, 0.02198108363850617, 0.016154657000744786, 0.02588804613476918, 0.020056604456018517, 0.019683424420194863, 0.01448699653516581, 0.01955014584057125, 0.016987110885940952, 0.024514697593303374]
xx_val8_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val8_4 = [0.0033892878227763704, 0.004824523048268424, 0.0005673696597417196, 0.001132703489727444, 0.002390131685468886, 0.0006801154878404406, 0.0011820064650641547, 0.0005431254704793294, 0.0008346878819995456, 0.0006261795759201049, 0.0004540287786059909, 0.0004257049825456407, 0.0005736238426632351, 0.0003837373521592882, 0.0011065486404630872, 0.0004909253782696194, 0.0009640889035330879, 0.0006923175520367092, 0.000514890087975396, 0.0006367176771163941, 0.0005609204371770222, 0.00043842262691921657, 0.0008826504151026408, 0.000671984420882331, 0.0004998991886774699]
epoch_min8_4 = 2
stop_val8_4 = 0.0005673696597417196

#Datos accuracy
acc_train8_4 = [0.9827160493827161, 0.9864197530864197, 0.990432098765432, 0.9882716049382716, 0.9916666666666667, 0.9922839506172839, 0.9898148148148148, 0.9932098765432098, 0.9919753086419753, 0.9935185185185185, 0.9932098765432098, 0.9941358024691358, 0.994753086419753, 0.9941358024691358, 0.9935185185185185, 0.9932098765432098, 0.9922839506172839, 0.9959876543209877, 0.990432098765432, 0.9944444444444445, 0.9929012345679012, 0.994753086419753, 0.9944444444444445, 0.9950617283950617, 0.9922839506172839]
acc_val8_4 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels8_4 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0]
best_prob8_4 = [1.5477216948056594e-06, 1.8859838746720925e-05, 1.0717758414102718e-05, 2.770661740214564e-05, 3.9639453461859375e-05, 0.005060351453721523, 1.795758907974232e-05, 0.9998929500579834, 0.999962329864502, 2.8562096758832922e-06, 0.9999945163726807, 2.8586689950316213e-05, 1.4116342754277866e-05, 4.592547611537157e-06, 0.999947190284729, 0.9999555349349976, 0.9999997615814209, 1.1371099390089512e-05, 3.422196925839671e-07, 0.9999682903289795, 1.695126229606103e-05, 1.366088599752402e-05, 0.9999878406524658, 4.870563770964509e-06, 2.6495958991290536e-06, 2.5817893401836045e-05, 0.999833345413208, 3.06045294564683e-05, 0.0036891144700348377, 0.00012867037730757147, 0.9999899864196777, 5.675838110619225e-06, 0.9995799660682678, 0.9999942779541016, 0.9999854564666748, 1.1542193533387035e-05, 9.526366739009973e-06, 0.9999998807907104, 0.0031422311440110207, 9.506744390819222e-06, 0.00012895952386315912, 6.330104952212423e-05, 0.9999936819076538, 0.9993646740913391, 0.999985933303833, 0.9999958276748657, 0.9999101161956787, 0.9999707937240601, 0.9998095631599426, 0.9999998807907104, 9.79047308646841e-06, 5.5805998272262514e-05, 0.9999878406524658, 0.00013033497089054435, 3.51373637386132e-05, 0.9999955892562866, 2.510803597033373e-06, 2.0703268091892824e-05, 2.5244862627005205e-05, 0.00023877313651610166, 0.0001315786939812824, 1.1931768312933855e-05, 2.164556235584314e-06, 2.931936614913866e-05, 0.999984860420227, 0.9991274476051331, 0.9999899864196777, 0.9999727010726929, 0.000795721251051873, 0.9999829530715942, 2.2161228230288543e-07, 1.0, 0.9999898672103882, 0.011206590570509434, 0.0006653274758718908, 0.00036364569677971303, 1.8423448636895046e-05, 1.4784326594963204e-05, 4.117316166230012e-06, 0.9999994039535522, 0.9999911785125732, 0.9999841451644897, 0.9999595880508423, 0.9998977184295654, 0.9999161958694458, 0.999993085861206, 1.4256690519687254e-05, 0.9998490810394287, 3.394790837774053e-05, 0.00011068227468058467, 1.060279828379862e-05, 5.1432798500172794e-05, 7.55238943384029e-05, 4.0587756302556954e-06, 0.996006429195404, 0.9998593330383301, 0.9999889135360718, 2.3795177185093053e-05, 0.9998188614845276, 3.398911576368846e-05, 0.9999955892562866, 1.1163358522026101e-06, 9.316490263699961e-07, 0.9999557733535767, 1.5446339602931403e-06, 0.9998427629470825, 0.9995470643043518, 0.9990811347961426, 0.9999991655349731, 0.9918681979179382, 0.0005127190961502492, 0.999998927116394, 2.2350124709191732e-05, 0.00034406816121190786, 0.9999467134475708, 0.9999324083328247, 5.751545813836856e-06, 0.9999998807907104, 1.1549045666470192e-06, 0.004993730690330267, 5.153761594556272e-05, 0.9999195337295532, 0.9998598098754883, 5.5019995670591015e-06, 0.00017657468561083078, 1.0, 0.9995003938674927, 2.9847655241610482e-05, 0.9999980926513672, 0.9999955892562866, 0.9999672174453735, 3.7787707697134465e-05, 0.9996844530105591, 1.0, 0.9999507665634155, 7.4437502917135134e-06, 0.9999904632568359, 3.939201633329503e-06, 0.9934012293815613, 6.562646012753248e-05, 0.9999573230743408, 0.9999996423721313, 0.999858021736145, 0.9999704360961914, 0.00015301996609196067, 0.9851594567298889, 0.9970735311508179, 0.9997701048851013, 0.9988192915916443, 0.9999994039535522, 0.999763548374176, 7.589767392346403e-06, 4.820054164156318e-05, 0.9995800852775574, 0.9996082186698914, 3.300128582850448e-06, 0.999517560005188, 0.999995231628418, 2.1764749362773728e-06, 2.045405153694446e-06, 0.9756725430488586, 0.9999754428863525, 0.00018376143998466432, 8.685584361955989e-06, 9.107845471589826e-06, 0.00019370106747373939, 1.646695818635635e-05, 0.9999998807907104, 0.9999997615814209, 2.0709961972897872e-05, 1.0, 0.9905713796615601, 1.203332249133382e-05, 0.9991740584373474, 0.9999977350234985, 0.00012363177665974945, 8.45084578031674e-05, 0.9999686479568481, 0.9986761212348938, 7.77907757765206e-07, 0.9997977614402771, 0.9999406337738037, 3.817207925749244e-06, 1.0, 0.9992233514785767, 2.6225658075418323e-05, 0.9999997615814209, 2.544944663895876e-06, 0.0001314551627729088, 1.0575809028523508e-06, 4.5121290895622224e-05, 0.010376917198300362, 1.002801809590892e-06, 1.8462898879079148e-05, 0.9995470643043518, 1.7618629499338567e-05, 1.0634941645548679e-05, 1.1835935765702743e-05, 4.053739758091979e-05, 0.9998726844787598, 0.00012220921053085476, 0.9995918869972229, 0.00019933645671699196, 0.00012565535143949091, 0.0002463093842379749, 0.00017177255358546972, 0.999957799911499, 0.00019783304014708847, 0.9999901056289673, 0.9998553991317749, 8.345087394445727e-07, 0.9973828196525574, 0.999976634979248, 2.864537918867427e-06, 0.999996542930603, 0.999881386756897, 1.0, 0.9999985694885254, 0.9996261596679688, 0.9999998807907104, 0.9999990463256836, 3.493476106086746e-05, 0.0008874217746779323, 0.9941348433494568, 0.9996770620346069, 0.999996542930603, 0.9999656677246094, 2.992780355270952e-05, 7.140430534491315e-05, 0.9993280172348022, 4.6565692173317075e-05, 4.176921720500104e-05, 1.949431265302337e-07, 9.007498192659114e-06, 0.9877166152000427, 0.9997748732566833, 0.9999624490737915, 0.0001225101004820317, 0.9999381303787231, 0.9998904466629028, 0.9998304843902588, 0.0015385148581117392, 1.0, 0.9999053478240967, 0.9999991655349731, 0.9999973773956299, 5.898470590182114e-06, 0.9999971389770508, 5.204379704082385e-05, 0.9999570846557617, 0.00022104111849330366, 5.994215371174505e-06, 0.0008081802516244352, 3.741179170901887e-05, 0.0001280263822991401, 0.9999983310699463, 1.0, 0.9999971389770508, 0.999992847442627, 8.968182555690873e-06, 8.252519364759792e-06, 0.0001430898846592754, 3.315501817269251e-05, 0.9976135492324829, 5.3715948524768464e-06, 1.6670535842422396e-05, 3.276415100117447e-06, 0.9999667406082153, 0.9999252557754517, 0.9993023872375488, 0.9997815489768982, 5.400085683504585e-06, 0.999405026435852, 3.2574967917753384e-05, 0.9999995231628418, 0.9998942613601685, 6.907680472068023e-06, 0.9999997615814209, 8.96916608326137e-05, 0.00035755219869315624, 2.691328882065136e-05, 0.00013184596900828183, 0.9998433589935303, 2.087744724121876e-05, 0.9999998807907104, 0.0056613413617014885, 0.0002661292674019933, 0.9999974966049194, 1.1559597624000162e-05, 2.6186118702753447e-05, 0.9986938834190369, 5.8598634495865554e-05, 0.0001743447210174054, 5.417497277449002e-07, 0.9998551607131958, 2.8815566111006774e-05, 6.242055860639084e-06, 0.0023184216115623713, 0.9999971389770508, 0.9998632669448853, 0.9997753500938416, 0.999976634979248, 0.00020527585002128035, 0.9964529275894165, 0.00573680829256773, 0.00011781445209635422, 3.2464224204886705e-05, 0.0008908511372283101, 0.9999998807907104, 0.9996833801269531, 2.648289046192076e-05, 0.0025384544860571623, 1.2587207493197639e-05, 0.9994825124740601, 0.9999996423721313, 1.3732982324654586e-06, 0.9999961853027344, 0.9999359846115112, 1.0, 0.9999946355819702, 0.9999998807907104, 0.9999973773956299, 1.0, 0.998216450214386, 5.515040174941532e-05, 2.5572462618583813e-05, 7.166694558691233e-06, 0.9998754262924194, 0.9999748468399048, 0.9999995231628418, 0.9998761415481567, 0.9999426603317261, 0.00020470510935410857, 1.5158046153374016e-05, 1.3066873805200885e-07, 9.335709364677314e-06, 1.0094609024235979e-06, 1.0, 2.2284084479906596e-05, 0.9999510049819946, 7.224325236165896e-05, 0.9999483823776245, 0.00023522909032180905, 0.9984484910964966, 0.9999967813491821, 1.0, 0.9999682903289795, 7.565360533590137e-07, 9.234148456016555e-05, 0.9990339279174805, 0.9911634922027588, 0.002464227844029665, 1.0727289009082597e-05, 0.9999946355819702, 1.2510811302490765e-06, 0.9999717473983765, 0.999997615814209, 5.803546628158074e-06, 4.338795770308934e-05, 0.9999998807907104]

#Metricas de validación
Best_acc8_4 = 1.0
acc_aux8_4 = 1.0
Recall8_4 = 1.0
Especificidad8_4 = 1.0
Tiempo8_4 = 1287.7447452545166

######## Base9 ########
#MODELO = VGG13_BN
#Datos perdidas
xx_train9_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train9_4 = [0.03962298493694376, 0.0278362314734194, 0.028225548638973707, 0.018082654034649884, 0.022031828126421682, 0.019153391459473857, 0.01426197855192938, 0.02063907398669808, 0.013456419846157969, 0.01917943462361524, 0.019368521239102623, 0.01566543713395978, 0.019741429222954646, 0.009235990856900628, 0.017750488893117432, 0.01757438701611978, 0.019065033745618513, 0.014541687402460311, 0.013808767267583328, 0.018145891932057744, 0.014603587258377192, 0.014842853051276854, 0.014715238016696624, 0.017183646430939805, 0.018436336554126975]
xx_val9_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val9_4 = [0.0067155679066975916, 0.005867800282107459, 0.0067890415589014685, 0.006233898964193132, 0.007366974817381965, 0.006813072495990329, 0.005555639995468987, 0.005715539223617978, 0.006044332517517938, 0.0062641494803958465, 0.006989217301209768, 0.007660337951448229, 0.006814308298958673, 0.008236515190866258, 0.006222393777635362, 0.008555663956536187, 0.00629393607378006, 0.006933462288644578, 0.00544330808851454, 0.008078172140651279, 0.0076821256015035845, 0.007276289992862278, 0.0074579735596974695, 0.0067091905408435395, 0.00572025179862976]
epoch_min9_4 = 1
stop_val9_4 = 0.005867800282107459

#Datos accuracy
acc_train9_4 = [0.9842592592592593, 0.9901234567901235, 0.991358024691358, 0.9932098765432098, 0.9916666666666667, 0.9922839506172839, 0.9938271604938271, 0.9941358024691358, 0.9941358024691358, 0.9941358024691358, 0.9925925925925926, 0.994753086419753, 0.9935185185185185, 0.9972222222222222, 0.9935185185185185, 0.9941358024691358, 0.9932098765432098, 0.994753086419753, 0.995679012345679, 0.9935185185185185, 0.995679012345679, 0.9932098765432098, 0.994753086419753, 0.9950617283950617, 0.9944444444444445]
acc_val9_4 = [0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222]

#Datos curva ROC
best_labels9_4 = [0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]
best_prob9_4 = [7.232310963445343e-06, 0.9990554451942444, 1.590943975315895e-05, 0.9999988079071045, 0.0007721065194346011, 0.9999614953994751, 0.9999696016311646, 0.001753785996697843, 0.9999722242355347, 0.9998815059661865, 1.0, 0.00014519774413201958, 0.9999518394470215, 0.00019621070532593876, 1.483212258790445e-06, 2.1315975118341157e-06, 3.4771837817970663e-06, 1.3787303032586351e-05, 0.0004308411735109985, 0.0001593937340658158, 0.9999239444732666, 4.383962732390501e-06, 0.9999953508377075, 0.9999991655349731, 0.999947190284729, 7.717304106336087e-05, 0.9986061453819275, 0.9999891519546509, 0.9999990463256836, 9.005519677884877e-05, 0.9994648098945618, 0.9999762773513794, 3.93161790270824e-05, 0.999997615814209, 0.9999916553497314, 0.9986153841018677, 0.9967160224914551, 0.9999597072601318, 4.211981831758749e-06, 5.312761459208559e-06, 9.038554708240554e-06, 0.9999940395355225, 2.8658441806328483e-06, 2.3816406610421836e-05, 4.566960342344828e-05, 0.9999270439147949, 7.860337063902989e-05, 0.9999600648880005, 0.0006382256979122758, 3.6194873246131465e-05, 8.824455107969698e-06, 1.1288829227851238e-05, 1.694349521130789e-06, 0.9998694658279419, 0.9975409507751465, 1.454044422644074e-06, 8.782868826529011e-05, 1.0068709599408976e-07, 4.327155329519883e-05, 0.9999953508377075, 1.946198835867108e-06, 0.9999974966049194, 0.9999884366989136, 0.9999997615814209, 0.9982373714447021, 4.9807517825684045e-06, 1.5332070688600652e-06, 1.301430643252388e-07, 1.9098249595117522e-06, 0.9999896287918091, 8.486030128551647e-05, 0.9999791383743286, 0.9999890327453613, 3.848611959256232e-05, 0.9999419450759888, 0.00046198730706237257, 0.9999997615814209, 1.62706299988713e-07, 1.6991221855278127e-05, 0.9999916553497314, 2.9124488719389774e-05, 0.9999550580978394, 1.9930253984057344e-05, 3.042273135633877e-07, 0.000208675250178203, 4.0346452578887693e-07, 0.9999953508377075, 7.177205407060683e-05, 0.9999815225601196, 3.14356896069512e-08, 0.9999793767929077, 0.9999996423721313, 1.2186003004899248e-05, 0.9958445429801941, 4.7731005906825885e-05, 6.1053206081851386e-06, 9.973473424906842e-06, 0.0009953189874067903, 5.826669803354889e-05, 3.440995669734548e-06, 0.00045088212937116623, 0.9999886751174927, 0.99998939037323, 1.0944289897452109e-05, 0.9999829530715942, 0.999998927116394, 0.9999923706054688, 0.99967360496521, 7.316753908526152e-06, 0.9993250370025635, 1.3899638361181132e-05, 2.0320758267189376e-05, 1.5050569572849781e-06, 0.9998015761375427, 0.9997431635856628, 0.9999955892562866, 0.9999998807907104, 2.3565467927255668e-05, 1.626543166821648e-07, 0.9985262155532837, 0.9999997615814209, 1.8600468365548295e-07, 4.5720935304416344e-05, 0.9999490976333618, 0.868897557258606, 0.9999998807907104, 0.0003722558030858636, 0.9999998807907104, 0.9999784231185913, 0.99993896484375, 0.9999750852584839, 0.9995949864387512, 7.672718311368953e-06, 8.645618436275981e-06, 0.03170691430568695, 0.9999755620956421, 4.263108621671563e-07, 0.0005947939353063703, 0.9999237060546875, 0.9999774694442749, 5.26834719494218e-06, 0.0003881195734720677, 1.0, 0.999996542930603, 0.9999786615371704, 0.9999135732650757, 0.9995026588439941, 8.207046630559489e-05, 0.0010516218608245254, 0.9999685287475586, 0.9999984502792358, 9.380345034060156e-08, 0.9999998807907104, 0.00018413187353871763, 0.9999700784683228, 0.9999984502792358, 3.815925447270274e-05, 0.9998955726623535, 1.1599551186236567e-07, 0.9999916553497314, 0.9999998807907104, 0.9999916553497314, 1.1424947842897382e-05, 0.7828006744384766, 0.9999971389770508, 5.064134711574297e-06, 1.4388075442184345e-06, 0.9999856948852539, 5.152597623236943e-06, 0.9915058016777039, 0.9999420642852783, 0.9933992624282837, 1.0580394700809848e-05, 2.0866738850600086e-05, 6.138536718935939e-07, 5.657824431182235e-07, 4.404551873449236e-05, 0.9999097585678101, 0.9989253878593445, 1.0232873137283605e-05, 4.4203901779837906e-05, 0.9999990463256836, 2.2737003746442497e-05, 1.6135149053297937e-05, 9.860421414487064e-06, 0.9917268753051758, 0.0004977533244527876, 0.9999985694885254, 0.9992647767066956, 0.9999923706054688, 7.699786124248931e-07, 0.9999963045120239, 9.598811629984993e-06, 0.9999862909317017, 0.9994977712631226, 0.9966136813163757, 5.350487981559127e-07, 0.9999641180038452, 0.9999946355819702, 0.0002702783385757357, 0.0003790648188441992, 0.9999994039535522, 0.9999243021011353, 0.9999440908432007, 1.0150791922569624e-06, 1.940882839335245e-06, 9.988982441200278e-08, 7.365104011114454e-07, 3.74472037947271e-05, 0.0007887267856858671, 0.9999985694885254, 1.5193616036413005e-06, 0.9999998807907104, 5.74543309994624e-06, 1.0, 1.2783316378772724e-05, 1.0, 0.9999984502792358, 2.983117701660376e-05, 0.9999423027038574, 0.9999736547470093, 0.9999861717224121, 0.0006704507977701724, 0.9999997615814209, 6.964128829167748e-07, 0.0006274376064538956, 0.9998431205749512, 0.9999929666519165, 0.008434979245066643, 0.9986849427223206, 0.9999998807907104, 0.9999040365219116, 0.00019705308659467846, 0.9984207153320312, 0.9999997615814209, 0.00042219183524139225, 0.9999970197677612, 0.9999877214431763, 0.9999997615814209, 8.646424021208077e-07, 1.5298306607292034e-05, 0.9999997615814209, 0.0005605451297014952, 0.9999997615814209, 0.9999866485595703, 0.0006544352509081364, 0.0001366727810818702, 8.508627615810838e-06, 7.892257417552173e-05, 0.9971663355827332, 2.201075949415099e-05, 1.1269371498201508e-05, 0.9999964237213135, 2.0892166503472254e-06, 0.9999947547912598, 0.9999071359634399, 6.6088316089008e-05, 4.976751370122656e-05, 0.9999914169311523, 1.6308868922010333e-08, 5.846852945978753e-05, 6.4062683122756425e-06, 0.9999973773956299, 4.4928463466931134e-05, 0.9999998807907104, 1.033362696034601e-05, 0.9999974966049194, 0.9999972581863403, 0.33978384733200073, 1.8587272279546596e-05, 1.1096700518464786e-06, 8.930126114137238e-07, 2.1144326183275552e-06, 0.9999892711639404, 0.9999998807907104, 2.7642010536510497e-05, 0.005742027424275875, 2.2727595023752656e-06, 0.00015714620531070977, 1.1938673196709715e-06, 7.169389277805749e-07, 0.00011401035590097308, 0.9997667670249939, 1.9658054952742532e-05, 1.0, 7.220582119771279e-06, 0.9999635219573975, 7.896120024497577e-08, 0.9998314380645752, 0.9999688863754272, 0.9999619722366333, 1.925377182487864e-05, 8.206874895222427e-07, 7.397683657472953e-05, 1.0, 0.9999580383300781, 1.2489981315866316e-07, 0.9999997615814209, 0.9999144077301025, 0.9999738931655884, 0.9999978542327881, 0.9999947547912598, 3.2512928100913996e-06, 0.9999585151672363, 7.35517551220255e-06, 0.9999176263809204, 1.2770911780535243e-05, 0.00013877991295885295, 0.9999997615814209, 1.0, 9.105588105740026e-06, 0.9999966621398926, 0.6169467568397522, 0.9999762773513794, 1.9391865862417035e-05, 0.999990701675415, 0.9999837875366211, 0.9999996423721313, 1.8310381619812688e-06, 0.9985705614089966, 0.999983549118042, 0.9999995231628418, 5.367602966543927e-07, 3.988008302258095e-06, 6.166863386170007e-06, 4.583546626690804e-07, 1.0468119171491708e-06, 0.00010791941167553887, 4.064683878368669e-07, 1.8673797967494465e-05, 0.99982088804245, 0.999924898147583, 0.9992730021476746, 1.5170855931501137e-06, 0.999873161315918, 1.6832538676680997e-05, 0.9999707937240601, 0.9997383952140808, 1.0797450158861466e-05, 7.509702300012577e-07, 0.9633542895317078, 3.9282935176743194e-05, 0.9999994039535522, 0.9999923706054688, 4.811998223885894e-06, 0.00017977668903768063, 0.9994974136352539, 0.9999935626983643, 0.9999727010726929, 9.888258500723168e-05, 0.9999971389770508, 4.0654430932818286e-08, 2.227786808361998e-06, 0.9978365302085876, 1.0913187225014553e-06, 0.9999997615814209, 1.3992111234983895e-05, 3.7936874832666945e-06, 0.9998936653137207, 2.2878753952682018e-05]

#Metricas de validación
Best_acc9_4 = 0.9972222222222222
acc_aux9_4 = 0.9972222222222222
Recall9_4 = 1.0
Especificidad9_4 = 0.994475138121547
Tiempo9_4 = 1275.2853462696075

######## Base10 ########
#MODELO = VGG13_BN
#Datos perdidas
xx_train10_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train10_4 = [0.038528424584203295, 0.031058951246517676, 0.02592296439372463, 0.026235267216408693, 0.01925401155963356, 0.029335355832252972, 0.025250724187971633, 0.02010778174908073, 0.017826986055315276, 0.025217925803160963, 0.01790623789952125, 0.01626821767400812, 0.025151865736201956, 0.01666444801254037, 0.02596659218823468, 0.020784690295472558, 0.01788014343675272, 0.02683601454820162, 0.019771753150371857, 0.02105628788287257, 0.015207298900242205, 0.01901797299031858, 0.018415325788068183, 0.01747181064184801, 0.022855631602399142]
xx_val10_4 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val10_4 = [0.0019607428047392103, 0.0007956420381863912, 0.00031123260656992595, 0.0008453336026933458, 0.0003021197186575996, 0.00035556091202629936, 0.0003719084792666965, 0.0004661646154191759, 0.0005135859052340189, 0.0005498942401674059, 0.0004876943098174201, 0.00043353868855370415, 0.0005613476037979126, 0.0005464399854342143, 0.0005488726827833387, 0.00041333321068021985, 0.000696919858455658, 0.0008732429809040494, 0.0005882478422588772, 0.0010034212635623085, 0.001090829653872384, 0.0006428644061088562, 0.00046279728412628173, 0.00041820042663150364, 0.0006224471661779615]
epoch_min10_4 = 2
stop_val10_4 = 0.00031123260656992595

#Datos accuracy
acc_train10_4 = [0.9864197530864197, 0.9879629629629629, 0.990432098765432, 0.9916666666666667, 0.9932098765432098, 0.990432098765432, 0.9907407407407407, 0.9922839506172839, 0.9935185185185185, 0.9910493827160494, 0.9922839506172839, 0.9941358024691358, 0.9922839506172839, 0.9941358024691358, 0.9907407407407407, 0.9907407407407407, 0.9944444444444445, 0.9901234567901235, 0.9919753086419753, 0.9925925925925926, 0.9929012345679012, 0.9929012345679012, 0.9938271604938271, 0.995679012345679, 0.9932098765432098]
acc_val10_4 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels10_4 = [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]
best_prob10_4 = [6.191420106915757e-05, 0.9999991655349731, 0.9999717473983765, 0.9999440908432007, 0.9999663829803467, 0.9997623562812805, 6.478634168161079e-05, 1.3088437356145732e-07, 1.4480191566690337e-05, 2.0627612684620544e-06, 0.9999908208847046, 2.956021489808336e-07, 1.0, 3.0020760277693626e-07, 1.0, 4.395500127429841e-06, 0.9999997615814209, 0.9982526898384094, 0.000586067559197545, 0.999992847442627, 0.9999998807907104, 2.818441680574324e-05, 0.9999582767486572, 1.4125836969469674e-05, 4.13147432709593e-07, 0.9999285936355591, 0.9999986886978149, 8.099250408122316e-07, 2.3949485694174655e-05, 0.9999984502792358, 0.9999812841415405, 0.9999977350234985, 0.9999891519546509, 0.9999985694885254, 2.0626961486414075e-05, 0.9999912977218628, 0.9999908208847046, 1.2377265193208586e-05, 0.999998927116394, 2.7231853891862556e-05, 0.9999673366546631, 2.474127313689678e-06, 0.9997325539588928, 2.490723090886604e-05, 0.9999791383743286, 0.9999651908874512, 7.509938586736098e-05, 1.4125972484180238e-05, 0.9999939203262329, 2.446561302349437e-05, 0.9982324242591858, 1.349413923890097e-05, 1.031087890623894e-06, 0.00010728672350523993, 3.800749027504935e-06, 1.0290868885931559e-05, 2.272203971642739e-07, 1.0, 1.0701371593313524e-06, 0.03293691575527191, 0.9999814033508301, 2.929905122073251e-06, 0.9999995231628418, 8.332402785526938e-07, 0.9941354990005493, 0.9999884366989136, 6.855401579741738e-07, 3.5558387025957927e-06, 2.175391728087561e-06, 0.9998040795326233, 0.9996151924133301, 1.1112247193523217e-05, 0.9976715445518494, 7.992520068000886e-08, 3.0101375614322023e-07, 0.9999289512634277, 0.9999270439147949, 0.9999604225158691, 0.9995726943016052, 0.9999816417694092, 0.9999914169311523, 0.9989297986030579, 3.812389071100597e-09, 0.9992353916168213, 5.545560270547867e-05, 0.999964714050293, 0.9999710321426392, 2.4377166596423194e-07, 2.8349177227937616e-06, 0.0016767748165875673, 0.999993085861206, 1.1235237025175593e-06, 0.9999994039535522, 3.456872218521312e-05, 0.9999868869781494, 0.9999977350234985, 0.9999607801437378, 1.9863577108480968e-05, 0.9999301433563232, 0.0003896515700034797, 0.999968409538269, 0.9999973773956299, 0.9999974966049194, 1.7725698853610083e-05, 0.9999995231628418, 1.5945201994327363e-06, 0.9999438524246216, 5.8543555496726185e-06, 6.673415668956295e-07, 4.987491593055893e-06, 0.999996542930603, 0.9999068975448608, 4.902541149931494e-06, 1.2783951206074562e-05, 0.9999887943267822, 2.1652045688824728e-05, 1.0087891268995008e-06, 0.00030455575324594975, 1.0450131412653718e-05, 0.9999576807022095, 0.9999837875366211, 0.9999819993972778, 3.0996820896689314e-06, 0.999987006187439, 8.292004167742562e-06, 0.9999479055404663, 0.9995372295379639, 0.9999829530715942, 1.0151313745154766e-06, 0.9999994039535522, 0.00035929158912040293, 0.9999936819076538, 8.355200975529442e-07, 6.804766599088907e-05, 5.174886155145941e-06, 2.799999265334918e-06, 1.343784333585063e-06, 4.959962893735792e-07, 2.406464147952647e-07, 0.9999194145202637, 0.9999573230743408, 0.9999948740005493, 0.9999710321426392, 3.114742139587179e-05, 0.9999983310699463, 4.5829110604245216e-05, 0.9999779462814331, 0.9999837875366211, 0.999882698059082, 0.9999991655349731, 0.9999998807907104, 6.85014219925506e-06, 4.1327098188048694e-06, 7.945613589299683e-08, 2.2298980439927618e-08, 0.9999983310699463, 0.00016886628873180598, 0.9998272061347961, 0.9999537467956543, 0.9999969005584717, 0.9999856948852539, 0.9984567165374756, 0.9998687505722046, 1.1340343917254359e-06, 0.0010504723759368062, 0.9999983310699463, 0.0008063939749263227, 1.201289819618978e-06, 2.361707174713956e-06, 2.5027950414369116e-06, 0.9999902248382568, 0.9999810457229614, 0.9999712705612183, 2.4026489882089663e-06, 0.9999600648880005, 0.9999998807907104, 0.00011982848809566349, 0.9997256398200989, 9.450484867556952e-06, 5.000642704544589e-05, 0.0009081742027774453, 1.2079311773049994e-06, 0.9999994039535522, 0.9999878406524658, 1.0, 0.00014301705232355744, 0.9999991655349731, 0.9709132313728333, 1.7731482557792333e-06, 2.4700093490537256e-05, 2.3967493234522408e-06, 1.638798607928038e-07, 2.578358817117987e-06, 5.978789704386145e-05, 0.9999650716781616, 0.9999445676803589, 1.087552391254576e-06, 9.419624689144257e-07, 1.4688486089653452e-06, 0.9999250173568726, 6.75947603667737e-07, 0.9999034404754639, 0.9999964237213135, 0.00015147261728998274, 0.0005177719285711646, 0.9999899864196777, 9.460079127165955e-06, 0.9999955892562866, 0.9999994039535522, 0.9999876022338867, 0.9999990463256836, 0.9999951124191284, 6.717636279063299e-05, 1.777538500391529e-06, 3.094820158366929e-06, 9.929570978783886e-07, 0.9999909400939941, 2.1567717340076342e-05, 0.9999868869781494, 0.999980092048645, 1.0466307998058255e-07, 0.9991990923881531, 2.166256152236201e-08, 0.999997615814209, 0.9991135001182556, 3.820183849256864e-07, 0.9999986886978149, 0.9998759031295776, 0.9999951124191284, 0.9999967813491821, 0.999996542930603, 0.9999866485595703, 0.9999585151672363, 6.734326598234475e-05, 0.9999791383743286, 2.4955947083071806e-05, 0.9999653100967407, 0.9998201727867126, 0.9999885559082031, 1.0119030775967985e-05, 6.3064680944080465e-06, 0.9999843835830688, 0.9999998807907104, 0.0003416285035200417, 1.7155132809421048e-05, 2.0195722754579037e-05, 0.9994688630104065, 0.9999219179153442, 0.9999755620956421, 2.7868868528457824e-06, 2.472258984198561e-07, 0.9996083378791809, 1.0533771273912862e-05, 3.7922148976576864e-08, 0.9999985694885254, 0.9999953508377075, 0.9999264478683472, 0.9999977350234985, 0.0007142195827327669, 2.8760963687091134e-05, 1.039816106640501e-05, 0.9985452890396118, 0.9999825954437256, 1.1577647455851547e-05, 0.9999985694885254, 0.9999970197677612, 0.9999998807907104, 1.0, 0.9999810457229614, 7.2900779741758015e-06, 0.001635085791349411, 2.0855181901424658e-06, 0.9998997449874878, 0.9998310804367065, 0.9999990463256836, 0.0008699457976035774, 6.401464958116776e-08, 1.7964329117603484e-06, 1.8912121959147044e-05, 8.174606591637712e-07, 9.78449634203571e-07, 4.523364793840301e-07, 0.9999144077301025, 0.9999496936798096, 1.1222505236219149e-05, 0.9999995231628418, 1.1756391415929102e-07, 0.9999901056289673, 8.272559739452845e-07, 7.156415904319147e-06, 0.9999946355819702, 2.046790541498922e-06, 2.76891041721683e-05, 2.1541814021475147e-06, 0.0002573065576143563, 0.9999268054962158, 4.970483132638037e-06, 0.9998134970664978, 0.9999661445617676, 9.561707514649243e-08, 5.29221892975329e-07, 2.4486387701472268e-05, 0.9999995231628418, 1.012113671094994e-06, 3.7310132938728202e-06, 0.9999980926513672, 3.722494057001313e-06, 1.540183075121604e-05, 0.9993219375610352, 1.627044730412308e-05, 4.074075832249946e-07, 5.576785406447016e-05, 0.9999998807907104, 1.3867630741515313e-06, 0.9999619722366333, 1.616579720575828e-05, 6.689053861919092e-06, 1.0, 0.0003409327764529735, 0.9998688697814941, 0.9999656677246094, 6.361109171848511e-06, 5.1590241127996705e-06, 0.9999998807907104, 1.2436736369636492e-06, 2.731584345383453e-06, 7.07843792042695e-05, 6.047591796232155e-06, 0.999974250793457, 4.7555511173413834e-07, 0.999998927116394, 0.9999700784683228, 0.999984622001648, 1.1135331078548916e-05, 0.9999270439147949, 2.8241140626050765e-07, 1.102417354559293e-05, 0.9999896287918091, 1.4327418284665328e-05, 1.1332947906339541e-05, 1.0, 6.013941629134933e-07, 5.369116024667164e-06, 0.9999829530715942, 0.9999977350234985, 2.6653774511942174e-06, 0.9999990463256836, 1.1541224012034945e-05, 0.9959688186645508, 0.999993085861206, 0.998430073261261, 0.9999997615814209, 4.115518095204607e-06, 8.330461241712328e-06, 2.3392394155052898e-07, 0.9960891008377075, 0.9999994039535522, 4.8596353735774755e-05, 5.3663614380639046e-05, 5.452457685350964e-07]

#Metricas de validación
Best_acc10_4 = 1.0
acc_aux10_4 = 1.0
Recall10_4 = 1.0
Especificidad10_4 = 1.0
Tiempo10_4 = 1291.4401032924652
###################################################################

###################################################################

######## Base1 ########
#MODELO = VGG16
#Datos perdidas
xx_train1_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train1_5 = [0.19098466127007097, 0.10651685658980299, 0.08150277590310132, 0.07820301661152898, 0.0735599567308838, 0.04783124059070776, 0.04910925505337892, 0.04349551683774701, 0.046075919638445345, 0.04252938057905362, 0.04159026810048539, 0.03971687390664477, 0.037393145511547723, 0.04296990441687313, 0.03985646582312054, 0.03620602407573182, 0.04040748671065142, 0.03979997411370277, 0.04636887517976172, 0.041649820306050925, 0.03967369846355768, 0.04651472723042523, 0.040818047909824934, 0.04674369302796729, 0.04076767318226673]
xx_val1_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val1_5 = [0.07579575495587455, 0.05510998864968618, 0.21176544692781235, 0.17563543220361075, 0.04260362204578188, 0.04034269948800405, 0.0391715025736226, 0.03762331340048048, 0.036436136811971664, 0.03671435564756394, 0.03672167840931151, 0.036676213807529875, 0.036668814056449464, 0.0366330870323711, 0.036623084710703954, 0.036621902055210535, 0.0366196741660436, 0.036617221600479546, 0.03661537484990226, 0.036610972550180226, 0.03661090003119575, 0.03661083016130659, 0.03661072494255172, 0.03661072295573023, 0.03661066401335928]
epoch_min1_5 = 7
stop_val1_5 = 0.03762331340048048

#Datos accuracy
acc_train1_5 = [0.9194444444444444, 0.9595679012345679, 0.9694444444444444, 0.9725308641975309, 0.9719135802469135, 0.9802469135802468, 0.9796296296296296, 0.9833333333333333, 0.9839506172839506, 0.9845679012345678, 0.9842592592592593, 0.9851851851851852, 0.9873456790123457, 0.9851851851851852, 0.9858024691358025, 0.987037037037037, 0.9839506172839506, 0.9851851851851852, 0.9820987654320987, 0.9839506172839506, 0.9864197530864197, 0.983641975308642, 0.9867283950617284, 0.9824074074074074, 0.9848765432098765]
acc_val1_5 = [0.9750000000000001, 0.9805555555555556, 0.9361111111111111, 0.913888888888889, 0.9833333333333334, 0.9833333333333334, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556]

#Datos curva ROC
best_labels1_5 = [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
best_prob1_5 = [0.9999955892562866, 0.999955415725708, 1.0, 0.00015170819824561477, 1.0, 0.9999998807907104, 0.9999140501022339, 0.9999940395355225, 0.000396795105189085, 1.0, 0.0007037959876470268, 0.005943579133599997, 4.4997286750003695e-05, 1.0, 1.0, 0.9999996423721313, 0.0022817477583885193, 0.008753237314522266, 0.01274245884269476, 0.999991774559021, 0.9999995231628418, 0.0018509490182623267, 1.0, 1.0, 1.0434975592943374e-05, 0.9893823862075806, 4.4174140612085466e-07, 0.9999997615814209, 0.0005106181488372386, 0.9994897842407227, 0.003663158742710948, 0.00015228184929583222, 1.0, 0.9874336123466492, 0.9999997615814209, 1.0, 0.9995205402374268, 0.0006377966492436826, 0.0017239116132259369, 0.0006749230669811368, 0.06696479767560959, 0.9999866485595703, 0.9997829794883728, 0.9999982118606567, 0.9745773077011108, 0.002526211319491267, 0.00010199360986007378, 0.9999998807907104, 0.14883510768413544, 0.9949597120285034, 1.0746887824097939e-07, 0.0005086668534204364, 0.9998464584350586, 0.9936333894729614, 1.5290338239992707e-07, 0.9978467226028442, 0.0012126016663387418, 0.0023624214809387922, 0.0016213366761803627, 0.0002206557837780565, 0.0005353415035642684, 0.9985896944999695, 0.9999992847442627, 0.0029216003604233265, 0.9962375164031982, 5.805095861433074e-05, 0.9999983310699463, 0.2512437701225281, 3.3047817851183936e-05, 0.0006849741330370307, 0.0002811283047776669, 1.0, 0.005575680639594793, 0.9991186261177063, 0.7689655423164368, 0.9996364116668701, 0.10762505233287811, 0.001071195350959897, 0.9999337196350098, 1.0, 8.564472227590159e-05, 1.0, 0.00027873218641616404, 2.225577372882981e-05, 6.574120925506577e-05, 0.9999998807907104, 0.002827960764989257, 0.11311925947666168, 0.0011452902108430862, 0.9975647926330566, 0.1099494993686676, 3.9056885725585744e-08, 0.9999996423721313, 0.0007659962866455317, 0.9996917247772217, 0.9878877997398376, 0.013126272708177567, 0.003268249100074172, 0.001650527585297823, 0.002590577583760023, 0.0003279780503362417, 0.9994038343429565, 0.9834038615226746, 0.00045648511149920523, 0.9999768733978271, 0.9998144507408142, 0.9999998807907104, 0.9989616870880127, 0.11394466459751129, 0.9999998807907104, 0.011085073463618755, 0.9997277855873108, 0.0013506576651707292, 1.2199888260511216e-05, 2.0285073333070613e-05, 1.0, 0.000837208004668355, 0.00021427168394438922, 8.356366015505046e-05, 0.999245285987854, 0.0021974570117890835, 0.0069295563735067844, 0.9999998807907104, 0.9998446702957153, 0.00026995490770787, 0.012699214741587639, 0.9961208701133728, 0.9995139837265015, 0.9989473223686218, 1.0, 0.013484740629792213, 0.00019160770170856267, 0.0020123866852372885, 0.9999998807907104, 0.9998806715011597, 0.0007084200042299926, 0.00025475636357441545, 0.9952239394187927, 5.750612217525486e-07, 1.0, 0.0011372690787538886, 0.001303897937759757, 0.9999363422393799, 0.9996141195297241, 0.9994542002677917, 0.9994668364524841, 1.0, 1.0, 1.0, 0.9740444421768188, 3.24039974657353e-05, 0.0008838290232233703, 0.00011374799214536324, 1.1870977687067352e-05, 0.9953195452690125, 0.9999892711639404, 0.000402794627007097, 0.019823666661977768, 1.0, 2.032986003541737e-06, 0.00014885210839565843, 0.00012100357707822695, 0.99970942735672, 0.9999997615814209, 0.0008688895613886416, 0.033999815583229065, 0.5481944680213928, 0.000551754841580987, 0.000141450873343274, 0.002475541550666094, 0.9999732971191406, 0.9946879148483276, 0.9999260902404785, 0.9999998807907104, 0.0004975520423613489, 0.0006726511637680233, 0.0021258611232042313, 0.0002299965126439929, 7.609529711771756e-05, 0.032211802899837494, 0.9993645548820496, 1.0, 0.9999986886978149, 0.005809931084513664, 2.693811893550446e-07, 2.7963085813098587e-05, 0.9965720176696777, 0.9999997615814209, 0.9995381832122803, 1.0, 0.0008700353209860623, 1.0, 2.036835030594375e-05, 0.0003962449263781309, 1.297995368076954e-05, 0.35086655616760254, 0.0005892584449611604, 1.0, 0.9999998807907104, 0.9997596144676208, 1.789814450603444e-05, 9.541746112518013e-05, 3.2380862649006303e-06, 1.4098126484896056e-05, 0.9995112419128418, 0.9999996423721313, 0.9999996423721313, 0.998467743396759, 0.026587286964058876, 0.000192979714483954, 0.03141210600733757, 0.9982814788818359, 0.9945679903030396, 6.1231323343236e-05, 0.9999997615814209, 1.0, 0.0002159609430236742, 0.00043672931496985257, 0.0015042602317407727, 0.9996864795684814, 4.5041915086585504e-07, 0.008109137415885925, 0.9999314546585083, 0.0002547048788983375, 0.9985752105712891, 1.0, 0.9989323019981384, 0.9999940395355225, 0.0005212955293245614, 1.0, 0.9999597072601318, 1.0, 0.9999998807907104, 7.452275167452171e-05, 0.005425682291388512, 0.002015483332797885, 0.9999998807907104, 0.0008307488169521093, 1.0, 0.0026225880719721317, 0.002324730157852173, 0.024439774453639984, 0.0001900851639220491, 0.37253740429878235, 0.9797240495681763, 0.9993557333946228, 3.2462921808473766e-06, 0.9998700618743896, 1.0, 0.7727376222610474, 0.002430725609883666, 0.004691564943641424, 0.2796608805656433, 0.8892227411270142, 0.9999651908874512, 0.9598879814147949, 0.0008894980419427156, 0.0006818881374783814, 0.838961660861969, 1.0, 0.9994346499443054, 0.00023287435760721564, 0.9992120265960693, 0.9898223876953125, 1.0, 0.9999997615814209, 0.9995947480201721, 0.004393677692860365, 0.755809485912323, 0.0002066644956357777, 0.991734504699707, 7.887998799560592e-05, 1.89974616660038e-05, 1.0, 0.9996951818466187, 0.9998133778572083, 0.9999997615814209, 0.007504782173782587, 8.287212949653622e-06, 3.709720112965442e-05, 0.0008465019636787474, 0.9999994039535522, 0.996841311454773, 0.0017280293395742774, 0.00023649338982068002, 7.045333040878177e-05, 0.9999997615814209, 0.9999997615814209, 0.11740468442440033, 0.9972424507141113, 0.9975851774215698, 0.9999997615814209, 0.0005668235826306045, 0.9999983310699463, 0.9999817609786987, 0.000438643095549196, 0.9999994039535522, 1.0, 1.0, 0.9998409748077393, 0.0004413641872815788, 0.01002658810466528, 0.9999880790710449, 0.9998262524604797, 0.9556300044059753, 0.0031520880293101072, 0.9999998807907104, 0.3334844410419464, 0.0008569085039198399, 0.007287304382771254, 1.0, 0.9999986886978149, 0.5408343076705933, 0.9914759397506714, 0.9999443292617798, 0.9999456405639648, 5.328371480572969e-05, 0.00021667664987035096, 0.00019624698325060308, 0.9999992847442627, 0.020681943744421005, 6.33953732176451e-06, 0.9717905521392822, 0.9998557567596436, 0.512992799282074, 0.9998119473457336, 1.0, 1.1786092727561481e-05, 0.998883068561554, 0.005380511283874512, 0.9998818635940552, 0.0004133491893298924, 0.001643658964894712, 0.9999717473983765, 7.211988304334227e-07, 0.9999994039535522, 0.44443196058273315, 0.9999998807907104, 0.004461309406906366, 0.9999954700469971, 2.1561136236414313e-05, 0.0005266673397272825, 0.9999994039535522, 8.397527381021064e-06, 0.4326295852661133, 0.0002066644956357777, 0.00020636522094719112, 0.002413470996543765, 0.9958264231681824, 0.002117651281878352, 0.0025929193943738937, 0.9993190765380859, 0.9999768733978271, 0.03135709464550018, 0.9999880790710449, 0.00039671227568760514, 0.011076746508479118, 4.5774682803312317e-05, 0.0010019600158557296, 0.00022359195281751454]

#Metricas de validación
Best_acc1_5 = 0.9805555555555556
acc_aux1_5 = 0.9873456790123457
Recall1_5 = 0.9832402234636871
Especificidad1_5 = 0.9779005524861878
Tiempo1_5 = 1502.982961177826

######## Base2 ########
#MODELO = VGG16
#Datos perdidas
xx_train2_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train2_5 = [0.09759128340232519, 0.05798519744917199, 0.05365923431552486, 0.04645965941894201, 0.049240474080965844, 0.029381106655906746, 0.026521494736274083, 0.031985869654166846, 0.030889083316296705, 0.023297002332078084, 0.0229134133476534, 0.025549635254306556, 0.03267311516367359, 0.026412837602842, 0.027063850984897143, 0.02543829513920678, 0.03343358500688164, 0.024825921921450413, 0.024147124533300045, 0.03002640379615772, 0.02846867376823484, 0.02528748705431267, 0.028616827615985163, 0.027886602015774926, 0.029219462805324132]
xx_val2_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val2_5 = [0.06277770598729451, 0.12052056690057118, 0.07176440821753607, 0.09223909444279141, 0.055304667353630065, 0.052653306391504076, 0.053263703485329945, 0.05464695427152846, 0.05217246777481503, 0.05483037763171726, 0.054582508073912725, 0.05446369548638662, 0.05435416003068288, 0.05421694682704078, 0.05405512948830923, 0.05404121908876631, 0.054050327340761824, 0.054049274656507704, 0.05404194957680172, 0.0540246413813697, 0.054024301634894474, 0.05402411255571577, 0.0540240036116706, 0.05402377976311578, 0.05402326418293847]
epoch_min2_5 = 4
stop_val2_5 = 0.055304667353630065

#Datos accuracy
acc_train2_5 = [0.9629629629629629, 0.9777777777777777, 0.9808641975308642, 0.9845679012345678, 0.9854938271604938, 0.9888888888888888, 0.9898148148148148, 0.9885802469135803, 0.9895061728395061, 0.9919753086419753, 0.9898148148148148, 0.9907407407407407, 0.9888888888888888, 0.9901234567901235, 0.9895061728395061, 0.990432098765432, 0.9888888888888888, 0.9901234567901235, 0.9901234567901235, 0.9888888888888888, 0.9891975308641975, 0.9919753086419753, 0.9876543209876543, 0.9910493827160494, 0.9888888888888888]
acc_val2_5 = [0.9722222222222222, 0.9527777777777778, 0.9722222222222222, 0.9750000000000001, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556]

#Datos curva ROC
best_labels2_5 = [1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0]
best_prob2_5 = [1.0, 0.0001684592425590381, 8.181494195014238e-05, 1.0, 0.00012140531180193648, 0.9999998807907104, 0.9999988079071045, 6.714977644151077e-06, 0.9999979734420776, 8.428769797319546e-05, 1.0, 1.4596793334931135e-05, 1.0, 0.0016907444223761559, 0.00011724713840521872, 1.5082745449035428e-05, 1.0, 0.9999992847442627, 0.9999957084655762, 2.141659933840856e-05, 0.9999998807907104, 0.999890923500061, 0.9999853372573853, 0.999977707862854, 8.809245628071949e-06, 0.8083353042602539, 8.892356709111482e-05, 0.00021449859195854515, 0.999907374382019, 0.00014337013999465853, 0.9999994039535522, 1.0, 0.9985936284065247, 3.1800609576748684e-05, 0.0010121713858097792, 0.008766722865402699, 1.0, 4.01126999349799e-05, 1.468530172132887e-05, 0.9999418258666992, 6.914666755619692e-06, 0.9999767541885376, 4.865828486799728e-07, 1.0, 9.660008799983189e-06, 0.9999778270721436, 0.9999397993087769, 1.0, 3.6832607293035835e-05, 1.0, 0.9999407529830933, 0.9999933242797852, 1.769563596099033e-06, 0.9999983310699463, 0.9999741315841675, 0.9999945163726807, 0.999889612197876, 0.006051951553672552, 1.0, 0.002360311569646001, 0.003463529050350189, 0.9999994039535522, 0.0016727979527786374, 0.9999728202819824, 0.9999836683273315, 1.0, 0.9774423837661743, 0.00016709401097614318, 2.099074117722921e-05, 0.002157713519409299, 0.999997615814209, 8.570325007895008e-06, 4.833789716940373e-05, 0.010604835115373135, 0.9999998807907104, 6.36626427876763e-05, 1.0, 0.0033198154997080564, 0.0030917872209101915, 9.941613825503737e-07, 1.0, 0.9999998807907104, 0.9999978542327881, 0.9999898672103882, 5.5116812291089445e-05, 0.9999998807907104, 0.0011365088867023587, 0.9999953508377075, 0.9999998807907104, 3.437388295424171e-05, 5.109986159368418e-05, 2.6307265216019005e-05, 0.002452370710670948, 0.9999997615814209, 0.9999995231628418, 3.12837291858159e-05, 5.571294605033472e-05, 0.9999984502792358, 0.008623105473816395, 0.9999998807907104, 1.0369116353103891e-05, 1.0, 6.096552169765346e-06, 7.881638157414272e-05, 0.999998927116394, 0.00016594698536209762, 1.0, 1.0, 0.9999995231628418, 0.999975323677063, 0.999955415725708, 0.9989583492279053, 0.00025626164278946817, 0.9999836683273315, 0.00017049895541276783, 6.406244938261807e-05, 4.7357276343973354e-05, 0.00010113052121596411, 0.9999924898147583, 0.0004201273841317743, 0.9999990463256836, 1.1826368790934794e-05, 0.8032941222190857, 0.00040292480844073, 0.9999947547912598, 0.9998825788497925, 0.0005596013506874442, 0.0001149776580859907, 0.9999947547912598, 0.9999860525131226, 0.9999969005584717, 1.0, 1.2885861906397622e-05, 0.00023429549764841795, 1.0, 0.0003965632931794971, 0.0011633094400167465, 0.9999910593032837, 1.0, 0.0006315295468084514, 0.9999932050704956, 0.9999998807907104, 2.128675077983644e-05, 0.0001493519957875833, 0.9999808073043823, 0.9965870380401611, 1.0, 0.9999610185623169, 1.0, 0.8127495050430298, 1.0, 0.9999998807907104, 0.9999990463256836, 0.0015772884944453835, 1.0, 0.00040968324174173176, 0.00017035560449585319, 0.6813562512397766, 0.9999339580535889, 1.0, 5.1658171287272125e-05, 1.0, 0.999964714050293, 0.0001828197855502367, 1.9688919564941898e-05, 0.9999933242797852, 0.9999803304672241, 0.9999964237213135, 0.9998739957809448, 1.0, 1.11632937205286e-06, 0.0003583907091524452, 1.0, 1.0, 1.0, 3.131670746370219e-05, 0.00012078996951458976, 0.016744522377848625, 1.1232880297029624e-06, 0.0010300384601578116, 1.0, 1.0798490620800294e-05, 7.73979991208762e-06, 0.0529976449906826, 7.107212150003761e-05, 0.999998927116394, 0.00037733186036348343, 0.0009868578054010868, 3.5285029298393056e-07, 5.813027746626176e-05, 4.7845074732322246e-05, 1.0, 0.00038408333784900606, 0.9505357146263123, 0.9999707937240601, 6.79187678542803e-06, 0.9996614456176758, 1.0, 4.7003144572954625e-05, 0.9999990463256836, 0.9999798536300659, 0.00015732836618553847, 0.0006200418574735522, 8.26367613626644e-05, 0.013226199895143509, 0.9999997615814209, 0.0014396879123523831, 4.863135836785659e-05, 1.0, 0.999998927116394, 0.9999909400939941, 1.0348222531320062e-05, 0.0004459715273696929, 0.0001457291073165834, 7.375208497251151e-06, 0.0007125776610337198, 1.0, 0.6611177921295166, 1.0, 0.9999997615814209, 0.0004052101285196841, 0.00010641455446602777, 0.9999994039535522, 1.0, 0.0001067674602381885, 0.9999998807907104, 0.9999455213546753, 0.0002082119754049927, 1.0, 0.9999997615814209, 0.00041575561044737697, 4.3803825974464417e-05, 0.0012850495986640453, 0.9999772310256958, 3.9536731492262334e-05, 0.1361561119556427, 0.0026211466174572706, 3.9615388232050464e-05, 0.999995231628418, 1.0, 0.0010567482095211744, 0.9999904632568359, 0.0005233856500126421, 1.0, 0.002091856673359871, 0.00044236707617528737, 1.0, 1.0, 5.5060383601812646e-05, 0.00011467064177850261, 0.0008583410526625812, 0.9999998807907104, 0.9950717091560364, 1.0, 0.9999775886535645, 0.00012277769565116614, 1.8799642930389382e-05, 9.564840001985431e-05, 0.00010319595457985997, 0.9999964237213135, 0.9995040893554688, 0.9955553412437439, 8.295874067698605e-07, 0.9999954700469971, 1.0, 2.536241709094611e-06, 0.999992847442627, 1.0, 0.00021288877178449184, 1.0, 0.9991594552993774, 1.0, 0.00014437202480621636, 0.9999285936355591, 0.0015487057389691472, 1.0, 2.2990496290731244e-05, 5.169251744518988e-05, 0.00026444022660143673, 7.08174702594988e-05, 0.9986405968666077, 1.0, 0.004331981297582388, 0.010776435025036335, 0.9999246597290039, 0.0013065309030935168, 0.9999412298202515, 0.9999208450317383, 1.0, 1.0, 8.289189281640574e-06, 5.815123950014822e-05, 0.9999904632568359, 6.344312714645639e-05, 0.9999779462814331, 0.9999997615814209, 1.6009350929380162e-06, 0.9999988079071045, 1.45360882015666e-05, 0.9999998807907104, 0.0005130515201017261, 0.9999992847442627, 9.067588689504191e-05, 1.0, 5.319179763318971e-06, 0.9999562501907349, 0.00022023120254743844, 1.0, 1.0, 0.0016013020649552345, 1.0, 0.004094899166375399, 2.301666791026946e-05, 1.00480920082191e-05, 7.12399123585783e-05, 8.520723326910229e-07, 0.0001483340747654438, 0.0008574940729886293, 0.9999922513961792, 8.522588359483052e-06, 0.003698530839756131, 7.027403626125306e-05, 1.0, 0.0008096710662357509, 0.9999997615814209, 0.9999942779541016, 0.975904643535614, 1.0, 0.0003497216966934502, 0.0005208064103499055, 1.0, 1.0, 0.012900365516543388, 0.0008876092033460736, 0.9999992847442627, 0.7180174589157104, 9.931503882398829e-06, 0.9999966621398926, 0.005141764879226685, 0.00018303781689610332, 0.054460275918245316, 0.005536852870136499, 0.9999947547912598, 0.9999874830245972, 0.9999997615814209, 0.00018986398936249316, 0.9999997615814209, 1.575092574057635e-05, 0.002519196830689907, 2.6780281814353657e-07, 1.0, 0.000932755065150559, 0.000127457533380948, 1.0, 8.102501305984333e-05, 0.9999988079071045, 0.9999960660934448, 2.4738894353504293e-05, 0.004308221861720085, 1.0]

#Metricas de validación
Best_acc2_5 = 0.9805555555555556
acc_aux2_5 = 0.9919753086419753
Recall2_5 = 0.9726775956284153
Especificidad2_5 = 0.9887005649717514
Tiempo2_5 = 1506.0015907287598

######## Base3 ########
#MODELO = VGG16
#Datos perdidas
xx_train3_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train3_5 = [0.04909824800712091, 0.06999720873104201, 0.04590384525097447, 0.03817268656911673, 0.03686820015679171, 0.036636787285039454, 0.0321507594835611, 0.028874643091802245, 0.026681347844409353, 0.029467139181531505, 0.024285147605854786, 0.0233409847834228, 0.027772642294933765, 0.025768901021392256, 0.025929244773255453, 0.024112252054023155, 0.026712755426580522, 0.027039572053853378, 0.025337597120691228, 0.027208591362944356, 0.025132786942484938, 0.02788725984685215, 0.034566697662259324, 0.03171120170089933, 0.03191575671051755]
xx_val3_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val3_5 = [0.014321113791730669, 0.025613285932275985, 0.011352813409434424, 0.025896111792988247, 0.010914393928315905, 0.01019182006518046, 0.01011554648478826, 0.009451682865619659, 0.009556082056628333, 0.00932904084523519, 0.009282754527197944, 0.009313381877210406, 0.009332336650954353, 0.009346904357274373, 0.009408906433317396, 0.009408415026134915, 0.009409839908281961, 0.00941075219048394, 0.009410701195398967, 0.00940961589415868, 0.009409619371096294, 0.009409727487299178, 0.0094096882475747, 0.009409732454352908, 0.009409816728697883]
epoch_min3_5 = 5
stop_val3_5 = 0.01019182006518046

#Datos accuracy
acc_train3_5 = [0.9817901234567901, 0.9756172839506173, 0.9845679012345678, 0.9864197530864197, 0.9851851851851852, 0.9858024691358025, 0.9879629629629629, 0.9891975308641975, 0.9885802469135803, 0.9882716049382716, 0.9907407407407407, 0.9925925925925926, 0.9885802469135803, 0.9895061728395061, 0.9901234567901235, 0.9910493827160494, 0.9916666666666667, 0.9901234567901235, 0.9898148148148148, 0.9910493827160494, 0.9910493827160494, 0.9888888888888888, 0.9879629629629629, 0.9885802469135803, 0.9867283950617284]
acc_val3_5 = [0.9916666666666667, 0.9916666666666667, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222]

#Datos curva ROC
best_labels3_5 = [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
best_prob3_5 = [1.0, 0.743843674659729, 0.992432713508606, 2.1141495381016284e-05, 3.835285440345615e-07, 0.0009010293870232999, 0.9999607801437378, 0.999995231628418, 0.030882246792316437, 0.0012876719702035189, 3.490022572805174e-05, 0.0001905017561512068, 0.9999161958694458, 0.010218329727649689, 0.9999979734420776, 0.9999699592590332, 0.9999994039535522, 1.0, 7.085691322572529e-05, 0.08593909442424774, 0.013374658301472664, 0.9990111589431763, 0.0005974830710329115, 5.246166983852163e-05, 0.9999985694885254, 0.9999998807907104, 9.223717825079802e-07, 5.1077404350508004e-05, 0.00014942747657187283, 0.9999998807907104, 3.196891339030117e-05, 0.5180596113204956, 0.00025549897691234946, 0.9865135550498962, 9.257260558115377e-08, 4.298800558899529e-05, 1.0, 0.9999985694885254, 0.9999758005142212, 0.008689410984516144, 7.493449811590835e-05, 0.028361497446894646, 0.9999973773956299, 0.9999704360961914, 0.000130416767206043, 9.910320659400895e-05, 0.0017680705059319735, 0.001748572918586433, 0.00016165875422302634, 0.9997445940971375, 0.0014625848270952702, 0.999982476234436, 0.999871015548706, 0.007605282124131918, 0.9381897449493408, 0.9999998807907104, 7.870252738939598e-05, 1.9547609554138035e-05, 0.0008122815052047372, 0.9999957084655762, 0.011575262062251568, 6.004960596328601e-06, 1.0, 0.0015453353989869356, 0.0001423634384991601, 0.9999032020568848, 1.0, 0.9999998807907104, 0.9999996423721313, 0.003234659321606159, 0.0001020507188513875, 0.0006227318081073463, 1.7897433224334236e-07, 0.006321464665234089, 0.9999992847442627, 0.9999998807907104, 0.006027976982295513, 0.9999805688858032, 1.0, 1.0, 0.9999934434890747, 0.9994713664054871, 0.9999996423721313, 0.0005574790411628783, 0.9999986886978149, 0.999985933303833, 0.9999984502792358, 2.139823891411652e-06, 3.744491550605744e-05, 0.008970698341727257, 0.9997392296791077, 0.9998531341552734, 1.2323275768721942e-05, 0.0038961919490247965, 0.00010329282667953521, 2.7238297661824618e-06, 0.8817072510719299, 1.0, 0.9999971389770508, 1.23864836609755e-07, 0.9999988079071045, 1.0, 0.04804607480764389, 3.2201485737459734e-05, 0.9999971389770508, 0.9999997615814209, 0.9098700881004333, 0.9998304843902588, 0.00016538398631382734, 2.9618155167554505e-05, 1.0, 7.193509190983605e-06, 1.0, 0.001146193826571107, 0.9999219179153442, 1.4526207451126538e-05, 0.000353393261320889, 0.00020030603627674282, 0.9997748732566833, 0.004364865832030773, 0.0009357990347780287, 0.0010851718252524734, 0.9585144519805908, 3.6508259654510766e-05, 1.0, 7.933517190394923e-05, 7.924997589725535e-06, 2.0578920612024376e-06, 0.005248639732599258, 0.9999984502792358, 0.0003963157651014626, 0.9999948740005493, 0.9999921321868896, 0.0002399781660642475, 0.9998424053192139, 0.9999947547912598, 0.9999511241912842, 0.9995356798171997, 0.9999995231628418, 1.0, 0.9999983310699463, 6.071087682357756e-06, 0.006199426483362913, 1.0, 0.00010181704419665039, 2.091185888275504e-05, 1.0, 0.9999510049819946, 0.9999988079071045, 0.0002728779218159616, 1.0, 1.0, 0.9999791383743286, 1.2721625353151467e-06, 0.9999008178710938, 0.9998902082443237, 4.671697024605237e-05, 1.0, 0.9999934434890747, 1.0, 0.9999998807907104, 1.0, 0.05012461915612221, 1.0, 0.0002084642619593069, 1.0, 1.0, 0.9995585083961487, 0.9999135732650757, 1.9222172795707593e-06, 5.638279162667459e-06, 1.1065440958191175e-05, 0.00019501254428178072, 1.0, 0.00018472342344466597, 0.0010003356728702784, 3.0824616260360926e-05, 0.9985963702201843, 4.816681212105323e-06, 0.999983549118042, 8.801234798738733e-06, 1.0, 6.51638038107194e-05, 0.002289144555106759, 1.0, 1.7180083887069486e-05, 1.4814825590292457e-05, 0.9999998807907104, 0.010707175359129906, 1.0, 0.00012270091974642128, 2.5157096388284117e-05, 0.9999939203262329, 1.0, 4.9745405704015866e-05, 2.8120787192165153e-06, 0.9999994039535522, 0.0021126032806932926, 1.9302528016851284e-05, 0.0002775526372715831, 0.0047144657000899315, 2.6742336558527313e-05, 0.999987006187439, 0.9999955892562866, 0.00010214087524218485, 1.0, 1.0, 8.19662818685174e-05, 0.9959120750427246, 0.00043124836520291865, 0.9999678134918213, 0.9999998807907104, 5.532912382477662e-06, 0.00067559425951913, 0.07981836050748825, 2.036914111158694e-06, 8.448885751022317e-07, 0.9999096393585205, 0.990935742855072, 1.0, 0.024341348558664322, 7.754231774015352e-05, 0.0012689931318163872, 1.0, 1.0, 0.999976634979248, 1.0, 0.003814881667494774, 0.9999959468841553, 0.04499656707048416, 0.0001963158865692094, 0.004257020074874163, 1.034004435496172e-05, 0.9999996423721313, 0.9999998807907104, 1.0, 0.9999977350234985, 0.10408882051706314, 0.9999995231628418, 6.706268322886899e-05, 1.1716266271832865e-05, 0.00549370376393199, 3.4014060474873986e-06, 3.556387753178569e-07, 0.9999984502792358, 0.9920040965080261, 0.00198803236708045, 0.034022241830825806, 5.063590720055799e-07, 0.002185023156926036, 0.9999995231628418, 3.536532312864438e-05, 0.9999911785125732, 1.0, 0.0003868300700560212, 4.630973364783131e-07, 1.0, 0.9999943971633911, 1.2694050383288413e-05, 6.334327736112755e-06, 0.06889107078313828, 0.9999997615814209, 0.0014829081483185291, 0.11456602066755295, 0.9999998807907104, 0.9999760389328003, 1.0, 1.0, 0.003057326888665557, 0.0025139516219496727, 0.9998401403427124, 3.900295553194155e-07, 1.0, 1.0, 1.1778605539802811e-06, 0.9999983310699463, 0.9995957016944885, 0.0005484154680743814, 5.9039288316853344e-05, 0.999998927116394, 0.9999998807907104, 0.9999978542327881, 0.0011562862200662494, 1.0, 2.4626473532407545e-05, 1.0, 1.0, 0.9999997615814209, 0.0020142269786447287, 0.9987372756004333, 0.0004734856483992189, 0.9999964237213135, 0.9999405145645142, 3.368469697306864e-05, 0.9999997615814209, 0.9999998807907104, 0.9999929666519165, 0.9999896287918091, 1.040132701746188e-06, 0.0013353613903746009, 0.9224388599395752, 0.9999998807907104, 6.633077282458544e-05, 0.9999842643737793, 0.0089561827480793, 7.446591189363971e-05, 1.0, 1.0, 1.0, 1.3229207070253324e-05, 0.9999994039535522, 0.9999992847442627, 0.9999991655349731, 0.00010338250285713002, 0.9999998807907104, 1.0, 0.9999997615814209, 0.0014583657030016184, 0.0004068540292792022, 0.00031549614504911005, 0.9999998807907104, 6.412577204173431e-05, 0.004595765378326178, 1.0, 1.0, 0.003156434278935194, 0.9999998807907104, 1.0, 1.0218215720669832e-05, 0.9999973773956299, 0.9999978542327881, 1.0, 1.947797500179149e-06, 0.999998927116394, 1.7951697373064235e-05, 0.9796817302703857, 0.9999431371688843, 0.99977046251297, 0.00011345918028382584, 0.0013837438309565187, 0.00020415506151039153, 0.9999942779541016, 0.01968211866915226, 0.0002139026328222826, 1.0, 1.0, 1.0, 8.668817827128805e-06, 0.9999998807907104, 1.0, 0.9972890615463257, 0.10780663043260574, 0.00022357319539878517, 0.0007287022308446467, 0.0008437045034952462, 0.003420648630708456, 0.020543331280350685, 9.962660260498524e-05, 0.9982032775878906, 1.170599603028677e-06]

#Metricas de validación
Best_acc3_5 = 0.9944444444444445
acc_aux3_5 = 0.9972222222222222
Recall3_5 = 0.989010989010989
Especificidad3_5 = 1.0
Tiempo3_5 = 1499.074206352234

######## Base4 ########
#MODELO = VGG16
#Datos perdidas
xx_train4_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train4_5 = [0.056114723708158655, 0.03574851919655447, 0.02948092414457121, 0.04058925969365202, 0.031258004894595084, 0.07474076046067991, 0.022856556835733816, 0.027151000564113074, 0.024848195478136156, 0.02379803583945757, 0.02373645739790834, 0.022326586469088073, 0.019094116562678492, 0.026018279643706333, 0.022713175756327898, 0.021767817078917114, 0.024083930805877403, 0.024787950883676975, 0.02796919658596133, 0.02427288215653396, 0.023607945902112088, 0.02502981611975917, 0.02650749992441248, 0.023538803888691798, 0.027768448933407113]
xx_val4_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val4_5 = [0.003325889011224111, 0.0034604253868261974, 0.004018402265177833, 0.0029220195280181037, 0.030480939480993483, 0.004173498186800215, 0.003748447861936357, 0.0036488081018129985, 0.0036931479970614117, 0.0032738402485847475, 0.0032635092735290526, 0.003247096637884776, 0.0032454152901967366, 0.0032420987884203595, 0.00322884072860082, 0.003228693703810374, 0.0032285744945208233, 0.003227904604540931, 0.0032274858819113838, 0.0032267110215293037, 0.003226720458931393, 0.0032267293996281095, 0.0032267315520180598, 0.003226728406217363, 0.0032267443007893033]
epoch_min4_5 = 3
stop_val4_5 = 0.0029220195280181037

#Datos accuracy
acc_train4_5 = [0.9780864197530864, 0.987037037037037, 0.9882716049382716, 0.9854938271604938, 0.987037037037037, 0.9783950617283951, 0.9910493827160494, 0.9888888888888888, 0.9910493827160494, 0.9901234567901235, 0.9910493827160494, 0.9910493827160494, 0.9929012345679012, 0.9907407407407407, 0.9916666666666667, 0.9922839506172839, 0.9901234567901235, 0.9891975308641975, 0.9888888888888888, 0.9898148148148148, 0.9910493827160494, 0.9901234567901235, 0.990432098765432, 0.9907407407407407, 0.9910493827160494]
acc_val4_5 = [1.0, 1.0, 1.0, 1.0, 0.9888888888888889, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels4_5 = [0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]
best_prob4_5 = [7.69278807322138e-11, 1.7477216118777505e-08, 0.9999932050704956, 0.999996542930603, 0.9999969005584717, 1.408520517998113e-07, 0.00034098539617843926, 3.150856855427264e-07, 1.3899099826630845e-07, 2.352561523366603e-08, 0.9999972581863403, 1.0, 1.0, 4.419912897901668e-07, 0.0005964208394289017, 1.0, 1.0, 1.0, 0.001231761765666306, 0.9999998807907104, 1.0, 5.2406687700568e-07, 1.0, 0.9999994039535522, 0.9999877214431763, 0.9999997615814209, 0.9999994039535522, 0.0015422990545630455, 1.0, 1.0, 2.8946693419129588e-05, 1.0, 1.0, 7.069048297125846e-05, 1.8713965346250916e-06, 2.3867413601053045e-10, 4.260016055468441e-07, 1.9305382181755704e-07, 0.000288954732241109, 0.9999980926513672, 1.0, 2.4871928872016724e-07, 4.6429615707666017e-08, 5.742349003412528e-06, 7.273441724464647e-07, 1.0, 1.6966587281785905e-06, 1.0, 0.9999997615814209, 1.0, 0.9999998807907104, 6.224542630661745e-06, 0.9999998807907104, 1.5943560356390662e-06, 0.6300525665283203, 2.606712428132596e-07, 0.9999997615814209, 1.0, 0.9999550580978394, 0.9999984502792358, 0.00014812490553595126, 5.8294126858982054e-08, 0.9999969005584717, 0.9999998807907104, 2.1135920791337526e-10, 1.0, 1.0, 0.9999998807907104, 0.9999994039535522, 1.0, 0.999993085861206, 0.9998582601547241, 0.998516857624054, 8.792657268941184e-09, 0.00033661132329143584, 0.9999991655349731, 9.04296975932084e-05, 1.0, 2.2228377929423004e-06, 1.0, 0.0013163777766749263, 1.1921422737870557e-09, 1.0, 0.9999758005142212, 3.977145388489589e-05, 8.3904669736512e-06, 1.0, 7.132746304705506e-07, 0.9999992847442627, 7.621301847393624e-07, 0.9530715346336365, 1.0, 1.0364812652596811e-07, 5.256199742120771e-08, 2.3007544314168626e-06, 0.9968284964561462, 5.288823740556836e-05, 1.0, 1.0, 5.494252377502562e-07, 7.196727437985828e-06, 0.9999995231628418, 6.148191289412352e-08, 4.647129389923066e-05, 0.9999997615814209, 1.0, 1.0, 1.2846731806348544e-05, 0.00019160532974638045, 1.5488569260924123e-05, 0.9992144107818604, 0.9999957084655762, 3.790153641602956e-05, 2.6474920105101774e-06, 3.117449693945673e-07, 0.00012920521839987487, 6.889772663498661e-08, 0.9999797344207764, 0.9999610185623169, 1.0, 1.0, 8.123653242364526e-05, 1.0, 1.0, 3.939524503948633e-06, 1.0, 0.9999997615814209, 1.1504422303687534e-07, 3.1089939511730336e-07, 0.0036505262833088636, 1.0, 0.9999979734420776, 1.728545385049074e-06, 0.9999938011169434, 1.3331029549590312e-05, 0.006662821397185326, 5.0833396016969346e-06, 6.97000723448582e-05, 5.784720542578725e-07, 0.0001705831673461944, 0.9999990463256836, 7.667500199204369e-07, 1.0, 0.9999996423721313, 0.9525671005249023, 1.0, 0.9999966621398926, 6.507697889901465e-06, 0.0002219035231973976, 3.455912974459352e-06, 1.337749466756577e-08, 1.0, 0.9999998807907104, 4.26051119575277e-05, 1.0, 6.454363028751686e-05, 0.0003606762911658734, 6.174280429149803e-07, 0.9999994039535522, 4.461289336177288e-06, 0.9999983310699463, 1.0, 0.9999998807907104, 0.9999967813491821, 2.748118959061685e-06, 1.0, 9.386931054677916e-08, 0.9992794394493103, 1.4613384280437458e-07, 0.9999997615814209, 0.9999998807907104, 2.0656647393479943e-05, 8.707559686627064e-07, 3.7640895698132226e-06, 0.9999998807907104, 4.310277006425167e-07, 7.09699816070497e-05, 0.9999997615814209, 1.0, 1.0, 2.410332058389031e-07, 0.00013503267837222666, 3.820317928671102e-08, 1.0, 4.653326413972536e-06, 1.0, 0.0005092997453175485, 0.0002813782193697989, 0.9999996423721313, 1.6588990092714084e-06, 2.0389690291722218e-07, 1.0, 1.0, 1.0, 0.9999996423721313, 1.7835167454904877e-05, 1.0, 0.9999849796295166, 1.4419861145142931e-06, 1.0, 0.9999996423721313, 4.3989471123495605e-06, 6.644002326083864e-08, 0.9979704022407532, 1.0, 0.9999988079071045, 0.9999911785125732, 9.834868563984855e-08, 0.999984860420227, 0.0014394725440070033, 0.0020797422621399164, 0.9999973773956299, 3.436450788285583e-05, 0.0089358976110816, 1.0, 0.9999997615814209, 0.0008020049426704645, 1.0, 4.752522272610804e-06, 1.0, 1.0, 0.9999998807907104, 7.023946818662807e-05, 3.5876569199899677e-06, 0.9999803304672241, 0.9999996423721313, 0.00011896798241650686, 0.00021589301468338817, 0.00010925095557468012, 2.033527744060848e-05, 2.9333349638172024e-10, 2.942294941021828e-06, 1.7624154913242762e-10, 1.0, 6.857461585241253e-07, 0.9999960660934448, 1.4638172842751374e-06, 1.2360720447190943e-08, 1.0, 0.9999995231628418, 1.3440607515846636e-09, 1.4184242047576845e-07, 9.767453548192861e-07, 4.935896868119016e-06, 4.805032105537066e-08, 0.00013585330452769995, 9.132915874943137e-05, 1.0, 0.9999985694885254, 2.0306364604039118e-05, 0.9999926090240479, 1.3021973721549784e-08, 0.9999998807907104, 0.8981403112411499, 1.0, 5.4140935390023515e-05, 0.9999998807907104, 3.5740882594836876e-05, 0.9999947547912598, 2.4267458229587646e-06, 6.216006568138255e-06, 1.0, 5.386041834753996e-07, 1.43499505611544e-07, 1.3086062011780086e-08, 1.0, 0.9999997615814209, 4.2258118782001475e-08, 2.905662910279716e-08, 0.0003723542904481292, 0.9727908372879028, 1.0, 3.2025411655922653e-06, 0.8039239048957825, 4.1737563151400536e-05, 0.9277801513671875, 1.0, 0.00034973921719938517, 1.550726665300317e-05, 0.9999997615814209, 1.0, 1.7206791369517305e-07, 0.002249170560389757, 0.000362010468961671, 0.996845006942749, 0.9999997615814209, 0.9999998807907104, 0.9999997615814209, 3.976743755629286e-06, 0.999997615814209, 2.4807897602840967e-07, 6.12182731174471e-08, 1.0, 1.0, 2.156210499038025e-08, 1.0, 1.9340388845989764e-08, 0.9995231628417969, 3.6686881230707513e-06, 0.9999997615814209, 8.929438877203211e-08, 0.9999663829803467, 3.012808065250283e-06, 1.0, 0.9999996423721313, 0.9999996423721313, 6.205682190341122e-09, 0.9999902248382568, 0.00011249403178226203, 1.0, 2.716278140724171e-05, 0.0001554030750412494, 1.0, 7.988853212737013e-06, 8.88287559064338e-07, 8.728411557967775e-07, 8.36548430527273e-09, 6.845700409030542e-05, 0.9999895095825195, 0.9860695600509644, 0.9996461868286133, 0.00010969649883918464, 1.0, 0.9999996423721313, 7.3058910032841595e-09, 0.9999688863754272, 3.124808634069609e-09, 0.9999582767486572, 0.9999997615814209, 2.7362148102838546e-06, 1.0, 7.17975353836664e-06, 1.22033429761359e-06, 0.9999840259552002, 1.0, 1.3368664895097027e-06, 1.0, 2.091004716930911e-06, 0.9995562434196472, 6.358907285175519e-06, 2.6784391593537293e-06, 0.9994848966598511, 3.2141681458597304e-06, 1.0, 1.0, 4.642661679099547e-06, 1.0, 1.1988019650743809e-05, 4.008954874734627e-07, 0.9999992847442627, 1.1677005886667757e-06, 0.00014508662570733577, 1.1763655493268743e-05, 1.0, 0.999997615814209, 4.4821319988841424e-08, 2.56119875530203e-07, 7.740147339063697e-06, 0.9999998807907104, 1.0]

#Metricas de validación
Best_acc4_5 = 1.0
acc_aux4_5 = 1.0
Recall4_5 = 1.0
Especificidad4_5 = 1.0
Tiempo4_5 = 1469.2802340984344

######## Base5 ########
#MODELO = VGG16
#Datos perdidas
xx_train5_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train5_5 = [0.04728921025439545, 0.03787109416760044, 0.03846880982135549, 0.03177252535558777, 0.02846126909609194, 0.022326241222060755, 0.027170133949429902, 0.02145541223846836, 0.01991496084280956, 0.02116953933864464, 0.019403765111425778, 0.0208861464979472, 0.01620959272539174, 0.021125089469147318, 0.018850875429717112, 0.025604909042149415, 0.023529651623081276, 0.02331136424600342, 0.01797742779240196, 0.0214203199946586, 0.017162059247493745, 0.022226899761108705, 0.019340409881170886, 0.016455805439639974, 0.024155521622778457]
xx_val5_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val5_5 = [0.004550535976886749, 0.0026555765006277296, 0.001981528600056966, 0.0018432370490498013, 0.0014764891730414498, 0.0008337926533487108, 0.0007854059338569641, 0.000723346405559116, 0.000678967105017768, 0.0006288998656802708, 0.0006272219949298434, 0.0006274408764309354, 0.0006239808268017239, 0.0006205270687739055, 0.0006186693906784057, 0.0006189730432298448, 0.000619345572259691, 0.0006194263696670532, 0.0006193488836288453, 0.00061937239434984, 0.0006193608045578003, 0.0006193641159269545, 0.0006193631225162083, 0.0006193634536531237, 0.0006193651093377008]
epoch_min5_5 = 9
stop_val5_5 = 0.0006288998656802708

#Datos accuracy
acc_train5_5 = [0.9833333333333333, 0.9845679012345678, 0.9824074074074074, 0.9879629629629629, 0.9895061728395061, 0.9910493827160494, 0.991358024691358, 0.991358024691358, 0.991358024691358, 0.9898148148148148, 0.9922839506172839, 0.9916666666666667, 0.9935185185185185, 0.9922839506172839, 0.9922839506172839, 0.9888888888888888, 0.9901234567901235, 0.9919753086419753, 0.9932098765432098, 0.9916666666666667, 0.9944444444444445, 0.9919753086419753, 0.9910493827160494, 0.9935185185185185, 0.9891975308641975]
acc_val5_5 = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels5_5 = [0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0]
best_prob5_5 = [2.7504230004637975e-08, 0.9990987777709961, 0.0062225074507296085, 0.9999997615814209, 1.0, 0.999998927116394, 6.782016134820879e-05, 1.0, 5.971016889816383e-06, 1.0219258911092766e-05, 0.9999920129776001, 6.669507168766131e-09, 0.9999996423721313, 0.00011753055878216401, 1.0, 1.2085477465006988e-05, 2.493180363671854e-05, 3.035635631754019e-10, 9.393833124704543e-07, 3.046193887712434e-05, 8.819549748295685e-07, 3.2290552098857006e-07, 1.0, 0.9999958276748657, 0.9999849796295166, 0.0011063225101679564, 3.77091380343586e-09, 3.671883064271242e-08, 3.862579944780009e-07, 0.9999890327453613, 1.0, 1.0, 0.9999982118606567, 6.88434681705985e-07, 1.3856651094101835e-05, 8.337256440427154e-05, 0.9999998807907104, 0.9999998807907104, 0.0006527527002617717, 0.9999996423721313, 1.0, 5.8701766647573095e-06, 0.00011836779594887048, 8.777388260661212e-10, 1.0, 0.9999998807907104, 0.9999961853027344, 8.494517373947019e-07, 9.046099876286462e-06, 7.342203844018513e-06, 0.9999994039535522, 0.9999995231628418, 1.0, 3.906478013959713e-06, 0.0007324610487557948, 0.9999939203262329, 0.0002034294739132747, 0.0007373421103693545, 0.9999978542327881, 0.0009260424412786961, 6.478390332631534e-06, 1.0, 1.0, 0.9999949932098389, 1.0, 1.1670198318824987e-07, 3.947145721383549e-09, 7.654086289221596e-07, 0.00019361244630999863, 0.9992974996566772, 1.990786842043235e-07, 0.9999992847442627, 0.0008169684442691505, 1.0, 5.081752533442341e-05, 0.9999996423721313, 0.9999998807907104, 0.9999352693557739, 1.0, 0.9999997615814209, 0.0007248423062264919, 0.9999996423721313, 0.99991774559021, 1.0, 0.9999998807907104, 0.9995878338813782, 0.9997805953025818, 3.663758207039791e-06, 6.306426075752825e-05, 0.0010468234540894628, 4.780432846018812e-06, 1.1675586364390256e-07, 1.0, 1.0458955330250319e-05, 1.0, 3.0635860515104696e-09, 0.00037826955667696893, 1.0, 1.0, 1.0474790315129212e-06, 0.9999986886978149, 0.0005015588249079883, 3.555737202987075e-05, 0.9999932050704956, 1.170247472259689e-07, 1.0, 0.9999996423721313, 1.0, 0.9995967745780945, 4.0770444797999517e-08, 0.9999994039535522, 0.016852086409926414, 2.648727104315185e-07, 0.9999998807907104, 0.9999896287918091, 2.2846356841910165e-06, 0.9999998807907104, 0.9997932314872742, 1.0, 4.903570811620739e-07, 1.0, 7.334473934861307e-07, 0.999992847442627, 1.0, 9.047422366847968e-08, 4.634526248992188e-06, 0.9999998807907104, 0.0002059327671304345, 2.2378987551974205e-09, 0.9999961853027344, 0.9999996423721313, 0.0001527568238088861, 2.2932682952614414e-07, 4.4869480575471243e-07, 1.1281223177839195e-10, 0.9999619722366333, 1.0, 5.05892483460002e-09, 0.00029402330983430147, 1.5414524341395008e-06, 0.9999300241470337, 0.0012967376969754696, 1.0466886379845164e-07, 0.9968787431716919, 0.003564950544387102, 8.840662485454232e-05, 2.1305033293117503e-09, 0.9999982118606567, 0.9999663829803467, 1.0, 2.0620964278350584e-06, 0.9999065399169922, 0.0006009943899698555, 0.9999998807907104, 1.1078682291554287e-05, 0.0001263405429199338, 0.9999929666519165, 0.9999995231628418, 0.9999998807907104, 1.358841359433427e-06, 1.0, 2.352055616938742e-06, 0.9998005032539368, 0.9961054921150208, 1.4653405742137693e-05, 1.0, 1.3356508361539454e-06, 0.0013981821248307824, 0.9999994039535522, 0.9999996423721313, 0.9999897480010986, 1.220493868459016e-05, 0.9999997615814209, 0.9999964237213135, 0.9999978542327881, 0.9999998807907104, 3.857389074823914e-08, 7.96237145550549e-05, 0.9999988079071045, 3.479849692666903e-05, 0.912223756313324, 1.0, 0.9999998807907104, 5.092186370347918e-07, 0.9999909400939941, 0.9999992847442627, 7.998214277904481e-05, 7.473145302583362e-08, 1.0, 1.0, 0.9999998807907104, 1.0, 2.0423591195140034e-05, 1.0, 7.204379670611161e-08, 2.788186520774616e-06, 0.00030143657932057977, 3.896702764905058e-05, 0.0015394246438518167, 0.9999998807907104, 1.0, 4.093597817700356e-05, 4.676922742419265e-07, 9.674612556409556e-06, 0.9999997615814209, 0.999968409538269, 4.697393160313368e-05, 0.00037537614116445184, 1.0, 0.0009763925918377936, 2.4451908302580705e-06, 1.355235781375086e-06, 0.9999998807907104, 2.2129151844296757e-09, 0.9999970197677612, 0.9999998807907104, 3.186246999575815e-08, 0.9999980926513672, 5.014361704525072e-06, 2.1735837435699068e-05, 1.0, 0.00011914849164895713, 5.876130671822466e-06, 1.0, 0.9981240630149841, 1.0, 0.999985933303833, 1.0, 0.010300587862730026, 0.9999998807907104, 2.62302110058954e-05, 1.9514395432906895e-08, 1.0, 2.1443836430989904e-06, 1.0, 8.01006899564527e-06, 0.9956533908843994, 1.0, 0.9999837875366211, 1.0, 1.0, 3.112426156803849e-07, 0.9999991655349731, 0.999966025352478, 0.9999996423721313, 0.01487115677446127, 4.598504528985359e-05, 0.00010939294588752091, 1.0, 8.514827953831627e-08, 0.9999538660049438, 3.6998757391160098e-09, 0.9999994039535522, 0.9999929666519165, 1.0, 0.9999585151672363, 0.9999613761901855, 0.9999985694885254, 1.0, 0.999991774559021, 1.0, 6.841987669758964e-06, 1.0, 1.1843193092886395e-08, 2.362338136663311e-06, 2.8882943297503516e-05, 0.9999998807907104, 8.629859604702972e-10, 0.9999874830245972, 0.9999998807907104, 0.9999982118606567, 1.3816095361107728e-06, 0.9990850687026978, 7.558149377473455e-07, 1.4521084494845127e-06, 3.2032162380346563e-06, 0.9999585151672363, 4.145742309447087e-07, 3.362452389410464e-06, 1.257999038273283e-08, 1.0, 0.001317088259384036, 0.9999979734420776, 1.4279319202614715e-06, 0.00020410542492754757, 0.9999998807907104, 0.9979594945907593, 0.9999998807907104, 6.329239710112233e-08, 5.34031734389373e-08, 0.9999998807907104, 8.291650601677247e-07, 0.9971906542778015, 1.584721864844596e-08, 0.9999997615814209, 1.0, 0.9997292160987854, 8.601353407300394e-09, 0.004007555544376373, 3.3526877274425715e-08, 6.775022484362125e-05, 3.900435331161134e-05, 1.0, 0.9999998807907104, 0.9999874830245972, 4.70294480692246e-07, 1.0, 1.3639642304497102e-07, 5.627723112411331e-06, 0.999980092048645, 1.1662343979423895e-07, 5.914005782869936e-07, 4.043578805124071e-09, 1.7258938896702603e-05, 3.1790353460792176e-08, 0.00011759243352571502, 1.5608249137599728e-09, 0.999976396560669, 0.9999998807907104, 1.0, 1.0, 1.3478810387823614e-06, 0.00036863729474134743, 0.9999998807907104, 8.937361417338252e-05, 0.0001610176987014711, 0.9999998807907104, 0.0020517741795629263, 1.0, 1.0, 5.641136522172019e-05, 1.506538137618918e-05, 5.319491901900619e-05, 0.9996751546859741, 8.125110616674647e-05, 2.991187955103669e-07, 0.012450315058231354, 3.764118082472123e-05, 0.0008535665110684931, 0.9999992847442627, 4.993642050976632e-07, 0.9999898672103882, 1.8641752831172198e-05, 1.7585230693839549e-07, 2.7740522767771836e-08, 5.530586349777877e-06, 0.9999998807907104, 2.1672045477316715e-05, 0.9999998807907104, 1.0, 2.6881377834797604e-07, 3.05428693536669e-06, 0.9999985694885254, 0.9999997615814209, 1.0, 8.190063454094343e-06, 5.352285825210856e-06, 0.9999998807907104, 0.9999996423721313, 0.980389416217804]

#Metricas de validación
Best_acc5_5 = 1.0
acc_aux5_5 = 1.0
Recall5_5 = 1.0
Especificidad5_5 = 1.0
Tiempo5_5 = 1493.4170570373535

######## Base6 ########
#MODELO = VGG16
#Datos perdidas
xx_train6_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train6_5 = [0.0458618234337112, 0.03337969108496183, 0.02378485186232461, 0.028615034022081046, 0.03687987159247751, 0.1255561410644908, 0.04393425477139744, 0.033726990075758946, 0.03450941910714279, 0.023466792776260847, 0.026569373270979635, 0.02569112107157707, 0.02662062641279197, 0.027436273177096872, 0.030133683188461963, 0.027887225601776148, 0.025653591761250556, 0.029298095340714043, 0.0281503082609471, 0.02855333380493117, 0.020518099939749564, 0.03208556505449024, 0.027148644423779147, 0.025028764622079, 0.025754116116850464]
xx_val6_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val6_5 = [0.011226797766155667, 0.009685014850563473, 0.0075641909407244785, 0.009403150445885128, 0.9121960070398119, 0.03274188919199838, 0.021029745870166355, 0.018045634693569606, 0.016995301842689513, 0.013966661194960277, 0.014058013591501448, 0.014065412183602652, 0.014197176694869995, 0.014280215981933806, 0.014425647258758546, 0.014435617294576432, 0.014438465568754407, 0.01443975783056683, 0.014438346690601773, 0.014442117346657648, 0.01444208597143491, 0.014442133158445359, 0.014442063205771976, 0.014442026946279737, 0.01444220956828859]
epoch_min6_5 = 2
stop_val6_5 = 0.0075641909407244785

#Datos accuracy
acc_train6_5 = [0.9802469135802468, 0.9898148148148148, 0.990432098765432, 0.9879629629629629, 0.9885802469135803, 0.9512345679012345, 0.9876543209876543, 0.9898148148148148, 0.9895061728395061, 0.9916666666666667, 0.9922839506172839, 0.9907407407407407, 0.9919753086419753, 0.9916666666666667, 0.9898148148148148, 0.9901234567901235, 0.9922839506172839, 0.9910493827160494, 0.9910493827160494, 0.9891975308641975, 0.9938271604938271, 0.9879629629629629, 0.9901234567901235, 0.9929012345679012, 0.9910493827160494]
acc_val6_5 = [0.9944444444444445, 1.0, 0.9972222222222222, 0.9944444444444445, 0.6111111111111112, 0.9888888888888889, 0.9888888888888889, 0.9916666666666667, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445]

#Datos curva ROC
best_labels6_5 = [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]
best_prob6_5 = [0.000169483944773674, 1.0, 3.625609679147601e-05, 0.9999990463256836, 4.614050340023823e-06, 3.876510163536295e-05, 1.0, 1.0, 1.0872994380406453e-06, 1.0, 1.0, 2.7452679205453023e-05, 0.9999990463256836, 0.9999970197677612, 4.6927031149834875e-08, 2.69639741645733e-07, 0.00472207460552454, 0.9999982118606567, 1.0, 1.0, 1.0, 1.0, 0.9999995231628418, 0.0004950198926962912, 1.0, 1.0, 1.0, 0.9999996423721313, 0.9999982118606567, 0.00013064553786534816, 0.9999994039535522, 1.0, 3.881089469359722e-06, 5.132038793931315e-09, 0.9891848564147949, 5.437641084427014e-05, 3.4322016517762677e-07, 0.9999997615814209, 4.127534339204431e-06, 1.680963102046462e-08, 1.0, 0.0003070221864618361, 1.0, 1.0, 1.0, 1.0, 0.00038113549817353487, 0.0009257954079657793, 0.9999936819076538, 1.0, 1.0, 1.2444633057384635e-07, 0.9999971389770508, 1.0, 0.0004146590654272586, 1.0, 0.0005281203193590045, 0.9999998807907104, 1.0, 1.0251422281726263e-05, 1.0, 0.0009764922433532774, 0.22291480004787445, 1.0, 4.0066304052288615e-08, 0.999569833278656, 1.2593654901138507e-05, 7.579919838462956e-07, 0.9999812841415405, 0.31408536434173584, 1.0, 1.0, 0.999984622001648, 1.793710544006899e-05, 0.9999998807907104, 1.0, 3.9527989770249405e-07, 1.0, 1.0, 4.5506141759688035e-05, 0.9999998807907104, 1.4710702089359984e-05, 0.9999998807907104, 0.9999932050704956, 1.158005488832714e-05, 2.238180201175055e-07, 3.240337553833683e-09, 0.0014599498827010393, 4.515159616858e-06, 0.0005470439791679382, 1.0, 1.1524000910867471e-05, 1.4074075807002373e-06, 1.0, 1.0, 2.2099145553511335e-06, 0.9999978542327881, 0.4455476701259613, 1.6534419046365656e-05, 0.9999998807907104, 0.05255690962076187, 0.0021838105749338865, 1.0, 1.0, 5.3851439588470384e-05, 1.0, 7.635161836105908e-08, 3.3547039492987096e-05, 0.000517114473041147, 0.0018157700542360544, 0.0009825535817071795, 1.0, 1.0, 1.3998550230098772e-07, 1.0, 1.0, 0.9999966621398926, 5.301973828863993e-07, 0.9999995231628418, 3.17270023515448e-05, 0.999527096748352, 3.6787987482966855e-05, 1.8320636741009366e-07, 0.00039781813393346965, 0.9999983310699463, 0.9999998807907104, 0.998593270778656, 1.0, 0.9999997615814209, 2.1557893603585399e-07, 0.9999990463256836, 7.393425039481372e-05, 2.359004611207638e-05, 2.3033317120280117e-06, 1.2590257028932683e-05, 1.0, 3.94818243876216e-06, 0.9999992847442627, 1.9204637879965958e-08, 2.8138174457126297e-07, 0.008339681662619114, 0.00014161391300149262, 2.6497273211134598e-06, 0.0061764041893184185, 1.0, 1.0, 0.002655888209119439, 1.0, 1.7776746119579911e-07, 0.9999994039535522, 8.620553160199051e-09, 2.675454879863537e-07, 5.821939794259379e-06, 0.9999998807907104, 3.7200206861598417e-06, 1.058035468304297e-05, 2.2239688405534253e-05, 1.403589249093784e-05, 0.9999995231628418, 1.0, 6.5118777037298514e-09, 3.5432660183687403e-07, 5.827428140037227e-06, 0.9999992847442627, 1.0, 0.0003788088506553322, 0.367013156414032, 0.9999954700469971, 5.036638981437136e-07, 3.8425014281529e-06, 1.6717134712962434e-05, 0.9999998807907104, 0.9999992847442627, 4.859199543716386e-05, 0.9999997615814209, 1.0, 0.0009799127001315355, 1.0, 1.0, 3.6570594375007204e-07, 1.0, 2.610678166092839e-06, 1.0368802350058104e-06, 1.0, 0.9999983310699463, 0.0018751375610008836, 5.435980710899457e-06, 0.9999995231628418, 2.40604435930436e-07, 1.0, 2.6958700800605584e-06, 3.6471050179898157e-07, 0.0005859435768797994, 1.0, 1.833921281502171e-08, 1.0, 6.074847260606475e-05, 1.0, 1.0, 5.015716305933893e-05, 1.0, 1.0, 0.9999998807907104, 7.366548288700869e-06, 1.0, 0.9999991655349731, 1.0, 7.38402595743537e-05, 2.113343361997977e-05, 9.555195674693095e-07, 5.825414336868562e-05, 0.0005719235050491989, 5.572148964461121e-08, 1.0, 9.863460093129106e-08, 0.00023492652690038085, 1.0, 0.00021044595632702112, 1.1202777372432138e-08, 8.624060399142763e-08, 1.0, 1.0, 1.0, 0.007018239237368107, 6.484439296627897e-08, 0.0006303651025518775, 0.00039085885509848595, 1.0, 1.0, 0.9999998807907104, 0.0009497457649558783, 4.405694198794663e-05, 1.0, 2.304707095390768e-06, 1.5899171557975933e-05, 1.0, 6.385257438523695e-05, 1.6841985370774637e-06, 0.9999939203262329, 0.9999980926513672, 1.0, 1.0, 0.9996167421340942, 1.0, 1.0, 3.3736249349658465e-08, 8.351518587801365e-09, 0.9999994039535522, 6.774124631192535e-05, 1.0, 5.844890620210208e-05, 1.0, 1.0, 4.2215961002511904e-05, 1.3405574463831726e-06, 3.7377198168542236e-05, 9.368799283038243e-07, 1.0, 6.918472990946611e-06, 0.9973614811897278, 5.523735671886243e-05, 0.9996703863143921, 1.0, 2.865973272037081e-07, 1.0, 1.0, 8.99668739293702e-05, 0.999991774559021, 1.0, 1.0, 1.0, 1.0, 4.2413805090291135e-07, 1.0, 1.2177860497786241e-08, 1.0568670631982968e-06, 5.328874249244109e-05, 0.9999996423721313, 1.0, 0.003759894287213683, 1.0, 2.3278880689758807e-06, 7.318829739233479e-05, 0.9999887943267822, 1.6774388313933741e-06, 0.006120422855019569, 1.467213905925746e-08, 0.998455286026001, 1.0, 0.02579011209309101, 1.0, 0.5098573565483093, 5.368583515519276e-06, 0.9999792575836182, 1.0, 1.0, 1.0, 9.456130669605045e-07, 1.0, 3.862889207084663e-05, 4.991012701793807e-06, 1.0, 5.383667030400829e-06, 0.0001265724131371826, 0.0014629728393629193, 1.0, 1.0, 8.226639636177424e-08, 1.0, 3.592011353248381e-06, 0.9999810457229614, 0.9999985694885254, 0.006598486099392176, 0.9999988079071045, 1.0, 1.0, 1.0, 1.0, 9.767487790668383e-05, 4.473603257793002e-05, 0.008137861266732216, 2.6874513423535973e-05, 5.946995429439994e-07, 1.0, 0.9999935626983643, 0.1171647310256958, 0.9996949434280396, 9.38749872148037e-06, 1.0, 8.72317195899086e-06, 0.0017033751355484128, 1.0, 0.00010343456960981712, 1.0, 1.0, 2.773785809040419e-06, 0.9996857643127441, 1.0, 1.0, 1.0, 1.4144974329610704e-06, 0.00023855807376094162, 1.2310064079201766e-08, 0.9999998807907104, 1.0, 0.999660849571228, 0.9999980926513672, 0.9998892545700073, 0.020345930010080338, 0.009946795180439949, 1.0, 0.00019213258929084986, 1.0, 1.5882320440141484e-05, 0.0005262931808829308, 5.18953626738039e-08, 2.662986616996932e-06, 0.9999998807907104, 1.6732812582631595e-05, 1.0]

#Metricas de validación
Best_acc6_5 = 0.9972222222222222
acc_aux6_5 = 1.0
Recall6_5 = 0.994475138121547
Especificidad6_5 = 1.0
Tiempo6_5 = 1506.6822793483734

######## Base7 ########
#MODELO = VGG16
#Datos perdidas
xx_train7_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train7_5 = [0.030742326498399548, 0.027692288850192672, 0.03139001981895647, 0.023899473203921023, 0.03147717140339039, 0.028992040952046712, 0.018646675789797746, 0.018779119518068103, 0.017362913424954003, 0.0235281957336414, 0.020179841152311844, 0.023484399923939765, 0.01738580756901223, 0.01772769111909984, 0.01836178034176061, 0.014241641290761806, 0.021083162476619084, 0.00926035996756436, 0.02225886736019158, 0.02005326865631857, 0.018157755316775524, 0.017597338833190777, 0.017277881650277126, 0.016412138212242244, 0.021232312108263558]
xx_val7_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val7_5 = [0.035472504297892254, 0.02967601948314243, 0.017388411197397445, 0.023303058495124183, 0.02616819077067905, 0.02489141689406501, 0.025064946545494928, 0.024076797895961337, 0.025584079656336044, 0.02681522203816308, 0.026772930721441903, 0.02674682786067327, 0.026810416082541148, 0.026852962209118737, 0.026805887950791252, 0.02679987301429113, 0.026795995897716945, 0.026791984173986646, 0.026787919799486797, 0.02678558263513777, 0.026785390906863743, 0.02678535117043389, 0.02678519735733668, 0.02678519437710444, 0.02678509751955668]
epoch_min7_5 = 2
stop_val7_5 = 0.017388411197397445

#Datos accuracy
acc_train7_5 = [0.987037037037037, 0.9895061728395061, 0.9910493827160494, 0.9929012345679012, 0.9864197530864197, 0.9891975308641975, 0.9925925925925926, 0.9919753086419753, 0.9935185185185185, 0.9925925925925926, 0.9935185185185185, 0.9922839506172839, 0.9938271604938271, 0.9929012345679012, 0.9919753086419753, 0.9950617283950617, 0.9925925925925926, 0.9966049382716049, 0.9929012345679012, 0.9916666666666667, 0.9935185185185185, 0.9935185185185185, 0.9950617283950617, 0.9932098765432098, 0.9922839506172839]
acc_val7_5 = [0.9861111111111112, 0.9861111111111112, 0.9916666666666667, 0.9916666666666667, 0.9861111111111112, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889]

#Datos curva ROC
best_labels7_5 = [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
best_prob7_5 = [0.9999125003814697, 1.8982120309374295e-05, 9.064502819455811e-07, 6.006259354762733e-05, 8.500776857545134e-06, 0.00024221459170803428, 1.0, 2.169205981772393e-05, 1.1573673130982343e-09, 6.744859319951502e-07, 0.0006786815356463194, 0.9999996423721313, 0.9999980926513672, 9.811794370762073e-06, 0.00011160231952089816, 2.599523213575594e-06, 0.00020629043865483254, 0.9999791383743286, 0.0010027497773990035, 6.360874976962805e-05, 1.266492404283781e-06, 5.879483069293201e-05, 7.612688477820484e-06, 0.0002622460888233036, 1.1851149793074e-05, 0.4395236670970917, 0.9999971389770508, 1.0, 1.0, 1.0, 1.0, 0.002049713395535946, 0.9999998807907104, 9.917155693983659e-06, 0.9999932050704956, 0.001504126819781959, 1.0, 1.6615731510682963e-05, 1.0, 0.9999825954437256, 1.8604740716909873e-06, 3.168767307215603e-06, 0.999998927116394, 1.0, 0.9999892711639404, 2.6041730052384082e-06, 0.9064377546310425, 3.525110878399573e-05, 1.0, 1.0, 0.0002909231698140502, 1.0, 1.691930719971424e-06, 2.548720112827141e-05, 1.0, 0.0004489255661610514, 5.27881711605005e-05, 3.5244553146185353e-05, 0.0008087633759714663, 0.9999737739562988, 0.000158147478941828, 0.9999912977218628, 1.0, 0.9999980926513672, 1.9109464119537733e-05, 1.0, 0.9999994039535522, 0.999987006187439, 8.61161170178093e-05, 5.680944013874978e-05, 1.0, 0.9999890327453613, 0.9999979734420776, 0.9999960660934448, 8.40191660245182e-06, 0.00030317623168230057, 0.999990701675415, 0.9999998807907104, 2.1784499040222727e-05, 2.8539161576190963e-05, 0.9999992847442627, 1.0, 0.0034988236147910357, 0.9999626874923706, 1.0, 4.304018511902541e-05, 0.9999964237213135, 0.0002137896663043648, 0.9999878406524658, 1.0, 0.9999963045120239, 1.0, 0.9999986886978149, 0.00022544471721630543, 1.0, 0.9999943971633911, 0.3863110840320587, 0.00018018734408542514, 0.0018452350050210953, 2.3079162474459736e-06, 1.0, 8.123632142087445e-06, 0.0004464653320610523, 1.0, 2.052664967777673e-05, 0.00014088941679801792, 0.9999991655349731, 0.9999997615814209, 1.5739304217277095e-05, 3.218760684831068e-05, 1.0, 1.0, 0.7681107521057129, 0.9999992847442627, 1.0, 0.9999978542327881, 1.0, 0.0001428224495612085, 1.0071252290799748e-05, 0.00014873064355924726, 0.0001109337608795613, 1.0, 3.749422467080876e-05, 1.0, 0.9999978542327881, 0.9999991655349731, 0.999997615814209, 1.0, 1.0, 1.0, 6.873391157569131e-06, 3.1935530842019944e-06, 1.0, 0.0017226155614480376, 1.0, 0.006675609387457371, 1.0, 1.6935298845055513e-05, 0.0006605276721529663, 1.2256492482265458e-05, 0.0006412033690139651, 9.43439663387835e-05, 9.440203285748794e-08, 7.155699859140441e-05, 1.0789294719870668e-06, 3.965064479416469e-06, 1.0, 1.8283375538885593e-05, 0.0004390963586047292, 2.398708147666184e-07, 1.0, 0.9999995231628418, 5.850693924003281e-06, 2.8497686798800714e-05, 0.9999979734420776, 1.0, 7.710499403401627e-07, 1.0, 1.0, 1.0, 0.9999938011169434, 6.470637890743092e-05, 0.9999984502792358, 2.196668356191367e-05, 0.001705727307125926, 1.0, 0.0017757421592250466, 1.0, 0.00014926742005627602, 4.1601228417675884e-07, 0.9999911785125732, 1.0, 3.151288638036931e-06, 0.00012114847777411342, 1.0, 1.1323311809974257e-05, 0.9999998807907104, 1.0, 0.9999969005584717, 1.0, 0.999983549118042, 0.9953463673591614, 0.0006213741144165397, 0.0006960458704270422, 0.0001134970661951229, 1.0, 0.0002728071704041213, 0.0014606465119868517, 0.9866010546684265, 1.6285688616335392e-05, 0.0007902010111138225, 6.585363280464662e-06, 0.9999920129776001, 0.0004320482548791915, 0.0009271552553400397, 3.7331628846004605e-06, 1.0, 7.042255674605258e-06, 1.0, 1.0, 1.0, 0.9999978542327881, 1.0, 1.0, 0.9999986886978149, 4.4680858991341665e-05, 2.7312432848702883e-06, 0.9241382479667664, 0.0002691613044589758, 1.0, 2.51461096922867e-05, 0.9999568462371826, 1.858273390098475e-05, 0.00018178450409322977, 3.9596278611497837e-07, 2.5666160581749864e-05, 4.872139243161655e-07, 4.6920737077016383e-05, 0.0004340103187132627, 6.803197175031528e-05, 0.00016083019727375358, 1.0, 0.9999979734420776, 0.9945297837257385, 9.435931133339182e-06, 1.0, 0.9999995231628418, 1.0, 0.9999979734420776, 0.9999970197677612, 4.624825305654667e-05, 0.9999990463256836, 1.0, 0.9999916553497314, 0.9999998807907104, 0.0055101774632930756, 1.0, 0.9999909400939941, 0.0005592585075646639, 0.0005073869833722711, 0.0002912392665166408, 1.1911622095794883e-05, 0.00014784367522224784, 1.0, 1.4117056707618758e-05, 0.9999983310699463, 0.0013311190996319056, 0.999936580657959, 0.0009007203625515103, 7.350808300543576e-05, 0.999944806098938, 0.999984622001648, 0.0001302142336498946, 0.9999983310699463, 5.768827395513654e-05, 0.9999978542327881, 0.9999995231628418, 9.587540989741683e-05, 0.0010005345102399588, 0.9999902248382568, 1.0, 2.9095814170432277e-05, 1.0, 0.9999996423721313, 0.9998574256896973, 1.0, 1.0, 3.8439313357230276e-05, 1.0, 0.0006551940459758043, 6.373656651703641e-05, 0.9999964237213135, 0.0001731606462271884, 0.9999991655349731, 0.9999914169311523, 0.0003546100342646241, 0.9999998807907104, 1.0, 0.9999998807907104, 0.00021542806643992662, 0.0003623121010605246, 7.248317160701845e-06, 0.0011395758483558893, 0.9999840259552002, 0.9999154806137085, 9.549027890898287e-05, 0.9999982118606567, 2.393672502876143e-06, 0.9999932050704956, 1.0, 4.411260306369513e-05, 0.0005969298654235899, 1.0, 8.129218622343615e-05, 0.9999998807907104, 8.762274774198886e-06, 0.9999912977218628, 3.5187144931114744e-07, 1.8947324861073866e-05, 1.0, 1.0, 0.9999911785125732, 0.9999973773956299, 1.0, 1.0, 1.2204053746245336e-05, 1.0, 2.257267988170497e-05, 2.8206812203279696e-05, 2.5637316866777837e-05, 2.5859926608973183e-06, 1.4759649502593675e-06, 0.0054238964803516865, 0.0006664586835540831, 6.91338354954496e-05, 0.9999998807907104, 1.0, 1.0, 5.299635631672572e-06, 0.9999754428863525, 1.0, 0.9999980926513672, 1.0, 1.0, 0.9999933242797852, 3.919335358659737e-05, 0.001415970386005938, 1.0, 0.9999501705169678, 0.00018461125728208572, 0.0005148024647496641, 6.75525443512015e-05, 0.0015122757758945227, 1.0, 1.0, 1.0, 0.9999696016311646, 0.08835728466510773, 4.9850834329845384e-05, 0.9999938011169434, 2.303223664057441e-05, 0.9998620748519897, 0.0007345126941800117, 1.0, 1.0, 0.00046447705244645476, 1.0, 1.0, 1.0, 1.0, 0.0025864632334560156, 0.9999995231628418, 1.0, 1.6318943380611017e-05, 0.6651335954666138, 6.579259093086876e-07, 0.0063111670315265656, 8.233447442762554e-05, 0.00012408838665578514, 1.0]

#Metricas de validación
Best_acc7_5 = 0.9916666666666667
acc_aux7_5 = 0.9966049382716049
Recall7_5 = 0.9836065573770492
Especificidad7_5 = 1.0
Tiempo7_5 = 1494.2719311714172

######## Base8 ########
#MODELO = VGG16
#Datos perdidas
xx_train8_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train8_5 = [0.042355549501048195, 0.03063620913911749, 0.030770463698808058, 0.022036410794581896, 0.03340522180552836, 0.029940669615695505, 0.0187456151769485, 0.02251535086536113, 0.025100653535789913, 0.022796365480732033, 0.017472211611859592, 0.018855498363206415, 0.017555516608703282, 0.01875051886395172, 0.020905009132844432, 0.019398410694190748, 0.015663851383659576, 0.017959747049543593, 0.01537417119115959, 0.01580729925154168, 0.018425881770657904, 0.018580824467870925, 0.01506472529820454, 0.018165958222047782, 0.0191006293543327]
xx_val8_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val8_5 = [0.008982144792874654, 0.10416042059659958, 0.019387255277898578, 0.005739768511719174, 0.012871826357311673, 0.009751482639047834, 0.00800974683629142, 0.007565055953131782, 0.0060517109102673, 0.008788175384203593, 0.008688571718004015, 0.008554331296020084, 0.008432920277118682, 0.008359425597720675, 0.008203006618552737, 0.008202835255199008, 0.008199212782912784, 0.0081971052620146, 0.008194837636417813, 0.008187295993169148, 0.008187401791413624, 0.00818744980626636, 0.008187393347422281, 0.008187440368864271, 0.008187347484959497]
epoch_min8_5 = 3
stop_val8_5 = 0.005739768511719174

#Datos accuracy
acc_train8_5 = [0.979320987654321, 0.9895061728395061, 0.9898148148148148, 0.9907407407407407, 0.9876543209876543, 0.9830246913580246, 0.990432098765432, 0.9935185185185185, 0.9922839506172839, 0.9929012345679012, 0.9941358024691358, 0.9922839506172839, 0.9932098765432098, 0.9929012345679012, 0.9925925925925926, 0.9929012345679012, 0.995679012345679, 0.9929012345679012, 0.9962962962962962, 0.9941358024691358, 0.9929012345679012, 0.994753086419753, 0.994753086419753, 0.9941358024691358, 0.9929012345679012]
acc_val8_5 = [0.9972222222222222, 0.9722222222222222, 0.9944444444444445, 0.9972222222222222, 0.9888888888888889, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222]

#Datos curva ROC
best_labels8_5 = [0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]
best_prob8_5 = [0.0024274487514048815, 1.0, 0.8864351511001587, 1.0, 1.0, 0.0008104422013275325, 0.0018152013653889298, 2.316212572850418e-07, 0.0015009986236691475, 0.00010058701445814222, 0.9999977350234985, 1.0, 1.0, 4.409048051456921e-05, 6.109798414399847e-05, 7.72967905504629e-06, 1.0, 5.982125730952248e-05, 1.0, 1.0, 1.0, 1.0, 0.0004360264865681529, 0.0024136353749781847, 1.0, 0.0005164216854609549, 0.0001629536272957921, 4.301673754980584e-07, 0.00027300676447339356, 1.0, 0.00023895394406281412, 1.0236085472570267e-05, 0.0010999049991369247, 1.0, 1.0, 1.0, 4.0670682210475206e-05, 1.0, 1.0, 0.0006013106903992593, 8.926771442929748e-06, 0.9782443642616272, 0.9999926090240479, 0.0003061195893678814, 0.9999018907546997, 0.0031568126287311316, 1.042818166752113e-05, 0.9999998807907104, 1.0, 1.0, 0.9999997615814209, 1.0, 3.25791324939928e-06, 0.00022924516815692186, 1.0, 0.9999986886978149, 1.0308805030945223e-05, 0.9999997615814209, 1.6115771359181963e-05, 1.0, 1.0, 1.0, 0.9999613761901855, 1.0, 0.003770998679101467, 1.0, 6.414115318875702e-07, 0.0006553048733621836, 1.0, 0.003267959924414754, 1.3062171092315111e-05, 0.0008722947677597404, 1.0, 1.0, 0.0009589429828338325, 7.210545300040394e-05, 6.926441528776195e-06, 0.00015649618580937386, 3.131004996248521e-05, 0.00047468882985413074, 2.140582182619255e-07, 1.0, 1.0, 0.0002977250551339239, 6.350637704599649e-05, 1.0, 7.571952664875425e-06, 1.0, 1.0, 0.01292420644313097, 1.0, 1.0, 1.0231117812509183e-05, 0.0012372755445539951, 1.0, 9.22788240131922e-05, 0.00017320293409284204, 1.0, 0.9999998807907104, 7.238882972160354e-05, 1.0, 0.00010400443716207519, 0.00015492695092689246, 0.9999998807907104, 1.0, 1.2687539310718421e-05, 1.0, 2.965446765301749e-05, 1.0, 0.00021548209770116955, 5.045695434091613e-05, 1.0, 9.377307287650183e-05, 1.0, 0.0012879499699920416, 0.9999357461929321, 0.0009107677615247667, 1.0, 7.622892735525966e-05, 0.0008850362501107156, 1.0, 5.8473091485211626e-05, 3.387288234080188e-05, 1.0, 0.9999604225158691, 3.2803967769723386e-05, 0.0012205997481942177, 1.0, 8.076674566837028e-05, 0.0005439749220386147, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0003049430961254984, 1.4904064755683066e-06, 1.0, 0.00202834140509367, 6.352928176056594e-05, 0.00017944238788913935, 0.00020164954185020179, 1.0, 0.0006207053083926439, 1.0, 0.9999998807907104, 0.0003810437337961048, 1.0, 1.0, 0.0005414647748693824, 3.0164133931975812e-05, 0.7328583002090454, 1.0, 1.0, 1.0, 3.6822948459303007e-05, 0.1865735799074173, 1.858078576333355e-05, 5.0654474762268364e-05, 0.9999719858169556, 0.00013547996059060097, 1.0, 1.0, 0.00030119469738565385, 0.0002639012527652085, 1.0, 1.0, 1.0, 0.0006053338875062764, 1.0, 1.0, 5.0194055802421644e-05, 1.0, 1.0, 0.0005039829411543906, 1.0, 9.023972779687028e-06, 0.00012227505794726312, 0.9999415874481201, 4.374821946839802e-05, 1.0, 1.0, 0.0006603057845495641, 0.9999973773956299, 1.0, 4.19102179876063e-05, 3.203967708031996e-06, 1.0, 1.0, 0.9797505140304565, 0.9975152015686035, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999990463256836, 8.237791917053983e-06, 0.0032768563833087683, 1.0, 0.9999996423721313, 6.619969553867122e-06, 1.0, 1.0, 1.0, 1.0, 0.0011880196398124099, 1.0381019819760695e-05, 1.0, 0.9999978542327881, 1.0, 1.0, 2.067316381726414e-05, 0.004362421575933695, 1.0, 0.9998575448989868, 1.0, 1.0, 0.0016713435761630535, 0.9999998807907104, 1.0, 0.0008231907850131392, 3.240628575440496e-05, 0.9999995231628418, 5.658716690959409e-05, 1.061791590473149e-05, 0.9999992847442627, 1.0, 8.22830145352782e-07, 3.483655018499121e-05, 1.0, 3.400649075047113e-05, 1.4968441064411309e-05, 5.710797950086999e-07, 1.0, 5.4800359066575766e-05, 1.0, 0.999998927116394, 1.0, 1.7742762793204747e-05, 5.901142685615923e-06, 1.0, 1.0, 0.0008451414760202169, 1.0, 0.0006584718357771635, 0.0029293294064700603, 1.0, 0.0007571104215458035, 6.512988329632208e-05, 0.12172902375459671, 7.883585203671828e-05, 0.0012170833069831133, 0.013650297187268734, 3.558083960797376e-07, 1.0, 5.904920726607088e-06, 1.0, 1.0, 0.0004565645649563521, 1.0, 0.0017387409461662173, 0.0002750716230366379, 0.0005293106660246849, 9.048845095094293e-05, 1.0, 0.0010083079105243087, 1.0, 6.533080158988014e-05, 0.0003960095054935664, 1.0, 4.5740675886918325e-06, 7.920364441815764e-05, 0.004018773790448904, 1.866026832431089e-05, 1.0, 1.0, 1.0, 1.0, 0.0001108540382119827, 3.067971920245327e-05, 1.0, 6.557149390573613e-06, 1.0, 0.0002458190720062703, 0.00015262070519383997, 1.0, 1.0, 1.0, 0.0006054091500118375, 1.0, 1.0, 1.0, 1.0, 0.9999997615814209, 5.327151666278951e-05, 0.9999886751174927, 0.9326236844062805, 6.071934421925107e-07, 0.9941102862358093, 0.9999533891677856, 1.0, 0.00017168594058603048, 1.0, 0.0244895052164793, 0.9999966621398926, 5.2493949624476954e-05, 1.0, 8.700991020305082e-05, 0.0015690089203417301, 0.9999998807907104, 1.0, 2.1975070012558717e-06, 1.0, 1.0, 1.0, 1.0, 5.474784643411112e-07, 8.654188422951847e-05, 1.0, 6.596297339456214e-07, 1.0, 1.0, 0.9999998807907104, 0.9999988079071045, 0.9999886751174927, 1.0, 0.00029968630406074226, 1.0, 0.0004543288960121572, 0.00038625593879260123, 0.008806845173239708, 8.401614468311891e-05, 0.0003405999159440398, 0.0001243923616129905, 2.9301229460543254e-06, 6.793395732529461e-05, 0.0007533912430517375, 0.023076873272657394, 1.0, 0.9999994039535522, 2.174690962419845e-05, 0.0009000555146485567, 0.0003059208975173533, 1.0, 0.0036793341860175133, 1.0, 5.7426626881351694e-05, 0.0021237542387098074, 6.29933419986628e-05, 0.0005894446512684226, 3.844312595902011e-05, 9.280237281927839e-05, 0.0005889648455195129, 0.0011107170721516013, 0.000173361215274781, 0.9999996423721313, 0.0006197683396749198, 1.0]

#Metricas de validación
Best_acc8_5 = 0.9972222222222222
acc_aux8_5 = 0.9972222222222222
Recall8_5 = 0.994475138121547
Especificidad8_5 = 1.0
Tiempo8_5 = 1477.147088766098

######## Base9 ########
#MODELO = VGG16
#Datos perdidas
xx_train9_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train9_5 = [0.03158495059167897, 0.01899655043710897, 0.17553135145593574, 0.041961691114637586, 0.04484293426812431, 0.03232337084633333, 0.023582694127603814, 0.022304849795721195, 0.02664223418191627, 0.026386779004041058, 0.029713992176600444, 0.028988354910670974, 0.02509268880074407, 0.023818062954110864, 0.02232424646248052, 0.02237140417650894, 0.02875000935645751, 0.024356724119480744, 0.02205190324672946, 0.023195728337691155, 0.020065011608379858, 0.024561187036243486, 0.022466572098158024, 0.022983504224706578, 0.023025069440956468]
xx_val9_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val9_5 = [0.00747736824883355, 0.023635876841015285, 0.0383062943816185, 0.025184664295779334, 0.015020198457770877, 0.015124028258853488, 0.014852516518698799, 0.01288762092590332, 0.013969839778211382, 0.013549783246384726, 0.013515685167577531, 0.013439192871252696, 0.013445399204889933, 0.013395239661137263, 0.013334209389156765, 0.013334987560908, 0.013341686874628066, 0.013345687919192844, 0.013348902761936187, 0.013349782096015082, 0.013349793271885978, 0.013349905692868763, 0.013349889053238763, 0.013349885907438066, 0.013349875724977917]
epoch_min9_5 = 0
stop_val9_5 = 0.00747736824883355

#Datos accuracy
acc_train9_5 = [0.9873456790123457, 0.9929012345679012, 0.9462962962962963, 0.9848765432098765, 0.983641975308642, 0.9882716049382716, 0.9916666666666667, 0.9916666666666667, 0.9919753086419753, 0.9901234567901235, 0.9895061728395061, 0.990432098765432, 0.991358024691358, 0.9916666666666667, 0.9929012345679012, 0.9925925925925926, 0.9910493827160494, 0.9922839506172839, 0.9929012345679012, 0.9922839506172839, 0.994753086419753, 0.9916666666666667, 0.9941358024691358, 0.9922839506172839, 0.9925925925925926]
acc_val9_5 = [0.9972222222222222, 0.9944444444444445, 0.9833333333333334, 0.9916666666666667, 0.9888888888888889, 0.9916666666666667, 0.9944444444444445, 0.9944444444444445, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667]

#Datos curva ROC
best_labels9_5 = [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0]
best_prob9_5 = [3.2247269245999632e-06, 1.0, 0.9781769514083862, 0.9999997615814209, 0.9974027276039124, 0.9999784231185913, 0.00863795168697834, 0.9999954700469971, 0.9999897480010986, 3.5049499729211675e-06, 1.0, 1.1215624908800237e-06, 3.150002157781273e-05, 1.0, 6.118528290244285e-06, 0.000633019139058888, 1.0, 0.9988874793052673, 2.7190722903469577e-05, 0.999993085861206, 2.4983854018501006e-05, 1.0, 6.878537533339113e-05, 0.00018903649470303208, 1.0, 6.849141209386289e-05, 0.9999998807907104, 0.004292008932679892, 6.51761656627059e-05, 1.0, 1.0, 2.995172872033436e-05, 1.0, 1.1679673662001733e-05, 0.9999998807907104, 1.0, 0.004449587315320969, 0.9999994039535522, 0.9999998807907104, 0.00021406749146990478, 1.0, 1.0, 0.999995231628418, 0.00036704380181618035, 0.9999997615814209, 6.41816295683384e-05, 1.0719925739977043e-05, 0.9996139407157898, 0.00111412035766989, 2.496466004231479e-05, 1.0, 1.0, 0.8746670484542847, 1.0, 2.0804085579584353e-05, 4.632250693248352e-06, 0.00033030350459739566, 0.9999996423721313, 6.963856139918789e-05, 2.1271969217195874e-06, 2.390425493103976e-07, 0.004177337512373924, 1.0, 1.0, 0.0007727628108114004, 9.28132503759116e-05, 0.00010360080341342837, 1.3754027349932585e-05, 1.0, 0.00010558993380982429, 1.0061143029815867e-06, 0.9999994039535522, 0.999998927116394, 3.9962800656212494e-05, 0.0001398936874466017, 2.6585410068946658e-06, 0.9999963045120239, 0.0029525496065616608, 8.142873411998153e-05, 0.0034711703192442656, 1.0, 0.0004605924477800727, 0.0001852616114774719, 1.0, 0.9999839067459106, 0.000881536805536598, 5.195337507757358e-06, 1.0, 0.02925662323832512, 7.080505838530371e-06, 9.853415576799307e-07, 0.00014018638466950506, 0.0003159277548547834, 1.0, 1.0, 0.9556759595870972, 3.441999751885305e-06, 2.5894878490362316e-05, 1.2615701052709483e-05, 1.0, 1.0, 0.9999986886978149, 1.0, 2.5285318770329468e-05, 0.0006093090050853789, 2.4865170189514174e-07, 2.7190848683744662e-08, 1.0, 0.9999957084655762, 7.308497060876107e-06, 0.0006084280903451145, 8.749115659156814e-05, 1.0, 0.000706865219399333, 0.9999967813491821, 0.9999949932098389, 6.028055565820978e-08, 0.0003488459624350071, 1.0, 0.00013765634503215551, 0.9888947010040283, 0.0004894701996818185, 0.0005715498700737953, 0.9999768733978271, 1.0, 0.9966149926185608, 1.0, 0.00010319358989363536, 0.0006792671047151089, 0.0006981597980484366, 1.7344933667118312e-06, 9.384278882862418e-07, 1.0, 6.297188974713208e-06, 5.740204142057337e-05, 0.9999997615814209, 1.0, 1.0, 0.00020623458840418607, 0.9999997615814209, 0.00012390580377541482, 0.9947875738143921, 0.9999997615814209, 1.0, 1.0, 3.4325091746723047e-07, 0.999994158744812, 1.5031407656351803e-06, 0.00035293569089844823, 0.00011362701479811221, 0.0052167936228215694, 1.0, 3.673653825586598e-09, 1.9864297428284772e-05, 1.0, 1.0, 0.00030987223726697266, 0.9999996423721313, 0.00014881660172250122, 0.00020495998614933342, 0.9999998807907104, 3.403431037440896e-06, 0.001887685153633356, 8.563197866351402e-07, 0.999983549118042, 0.0003382956492714584, 0.999089241027832, 1.1954646197409602e-06, 1.0, 1.0, 1.0, 1.0, 0.00011742906644940376, 0.03632785752415657, 6.9147043646466955e-09, 7.207320606994472e-08, 1.0, 0.00016605191922280937, 1.0, 0.9999985694885254, 0.9905716180801392, 8.19728484202642e-07, 1.7528899434182676e-06, 0.0039090062491595745, 0.00029631907818838954, 1.0, 1.0, 0.008866936899721622, 0.9999996423721313, 1.0, 3.291609027655795e-05, 1.0, 5.442257133836392e-06, 1.0, 0.0002026611182373017, 6.535767170134932e-06, 1.0, 0.9999997615814209, 0.9982966780662537, 0.0009387016762048006, 2.3364020762528526e-06, 7.068605100357672e-06, 0.999962329864502, 4.347700814832933e-05, 0.999775230884552, 0.0022977949120104313, 1.3851181392965373e-05, 8.539689588360488e-05, 1.8001853732130257e-06, 0.002187300007790327, 1.0, 7.121363159967586e-05, 0.9999761581420898, 8.082792919594795e-05, 1.0, 0.999998927116394, 0.0004737158596981317, 2.3061406864144374e-06, 7.264226496772608e-06, 0.9999995231628418, 3.2518976240680786e-06, 6.788344762753695e-05, 0.9999995231628418, 0.00024607375962659717, 0.0022847303189337254, 1.0, 0.9999306201934814, 1.0, 8.113573858281597e-05, 1.0, 0.9999704360961914, 0.03395021706819534, 3.038763315998949e-05, 5.052792403148487e-05, 0.9999961853027344, 1.0, 0.9999998807907104, 0.0018837956013157964, 0.9999998807907104, 6.5482327045174316e-06, 0.7362848520278931, 0.930041491985321, 1.0, 0.1650441437959671, 0.9998316764831543, 1.0970592484227382e-05, 0.9999998807907104, 0.9999998807907104, 0.999671220779419, 1.0, 0.9999833106994629, 1.0, 1.0, 5.077108653495088e-06, 0.0013240390690043569, 0.9999977350234985, 3.956278760597343e-06, 1.0, 0.9999998807907104, 0.9999978542327881, 1.0, 1.0, 0.00012839020928367972, 1.0, 0.9999998807907104, 0.0010568116558715701, 0.00038942284299992025, 3.9721368416678615e-09, 0.0009126293589361012, 1.0, 0.9995338916778564, 3.8967733416939154e-05, 4.7224191803252324e-05, 1.0, 1.0, 0.9999986886978149, 5.134939442541508e-07, 1.0, 1.0, 8.760678610997275e-06, 1.0, 1.0, 0.9960892200469971, 0.9999974966049194, 1.0, 0.9999998807907104, 1.0, 0.0013076593168079853, 1.0, 0.0002046694135060534, 6.338911771308631e-05, 0.9999974966049194, 0.9999991655349731, 1.0, 0.00020465067063923925, 8.804979188425932e-06, 0.9999985694885254, 0.00022445105423685163, 0.9999663829803467, 1.0, 6.397758261300623e-05, 1.0, 1.0, 0.00013429124373942614, 1.0, 1.0, 4.6157751967257354e-06, 0.008737249299883842, 0.999992847442627, 0.999403715133667, 1.0, 2.8112981453887187e-05, 0.0011174083920195699, 1.0, 1.3319792913080164e-07, 1.0, 0.9999158382415771, 1.0, 0.024972569197416306, 0.0004158016527071595, 8.652959593291598e-08, 0.00015308214642573148, 9.706282435217872e-05, 1.0, 2.3316608803725103e-06, 0.0003870617365464568, 0.9999942779541016, 0.0005548775079660118, 0.999962568283081, 1.0146717613679357e-05, 3.066468343604356e-05, 0.012361444532871246, 0.035982079803943634, 0.9999939203262329, 4.0946343915493344e-07, 0.9999998807907104, 0.00020065985154360533, 7.470987839042209e-06, 6.855251513115945e-07, 7.78705179982353e-06, 1.0, 4.595198333845474e-06, 0.000109716689621564, 0.9999967813491821, 0.00010498063056729734, 1.0, 0.9999730587005615, 1.0, 0.0001712771481834352, 0.9999768733978271, 0.9999997615814209, 1.0514330824662466e-05, 1.0, 1.4727817188031622e-06, 1.0, 0.9999998807907104, 1.0, 1.679379965935368e-05, 0.0006871425430290401, 1.0]

#Metricas de validación
Best_acc9_5 = 0.9972222222222222
acc_aux9_5 = 0.9972222222222222
Recall9_5 = 1.0
Especificidad9_5 = 0.994475138121547
Tiempo9_5 = 1512.1851978302002

######## Base10 ########
#MODELO = VGG16
#Datos perdidas
xx_train10_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train10_5 = [0.02953861947228879, 0.0318858917756581, 0.027284592813179815, 0.052038883500629005, 0.025047422651155494, 0.02072344899729446, 0.018777382640558997, 0.021300408648855892, 0.015348763487957141, 0.019347059220811466, 0.025883976894396324, 0.02080196362034774, 0.019724323444160415, 0.01572502084352352, 0.018964829075115697, 0.016440613788587077, 0.015431372453401117, 0.02001368621433223, 0.017269452734861846, 0.01858657807295705, 0.01701070096021817, 0.010575979754880623, 0.018494369034413937, 0.0152878036653554, 0.01630677221181952]
xx_val10_5 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val10_5 = [0.000996646285057068, 0.00041314827071295846, 0.9420888185501098, 0.0013405361109309726, 0.0003051247861650255, 0.0004075033797158135, 0.0006517089075512356, 0.0006285326348410712, 0.0005303733878665501, 0.0005489117569393582, 0.0005535526408089532, 0.0005532966719733344, 0.0005543490250905354, 0.0005484604173236423, 0.0005456623103883532, 0.0005455242262946235, 0.0005452659395005967, 0.0005455136299133301, 0.0005446374416351318, 0.0005444463756349351, 0.0005444463756349351, 0.0005444390906227959, 0.0005444308121999105, 0.0005444308121999105, 0.0005444258451461792]
epoch_min10_5 = 4
stop_val10_5 = 0.0003051247861650255

#Datos accuracy
acc_train10_5 = [0.9873456790123457, 0.9895061728395061, 0.9901234567901235, 0.987037037037037, 0.991358024691358, 0.9929012345679012, 0.9925925925925926, 0.9916666666666667, 0.9953703703703703, 0.9929012345679012, 0.9895061728395061, 0.9935185185185185, 0.9932098765432098, 0.9959876543209877, 0.9929012345679012, 0.9941358024691358, 0.9941358024691358, 0.994753086419753, 0.9938271604938271, 0.9950617283950617, 0.994753086419753, 0.9975308641975309, 0.9916666666666667, 0.9935185185185185, 0.994753086419753]
acc_val10_5 = [1.0, 1.0, 0.9222222222222223, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels10_5 = [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0]
best_prob10_5 = [0.9996137022972107, 0.0005817222991026938, 0.0015071406960487366, 1.2197677278891206e-05, 0.9999991655349731, 2.5473630103078904e-06, 0.9999977350234985, 0.00011857406934723258, 1.1255227036599535e-05, 1.4457259567279834e-06, 1.0, 4.793248444912024e-05, 3.7516790030167613e-07, 0.9999819993972778, 1.0, 0.9999992847442627, 1.8087404896505177e-05, 0.0002565160975791514, 1.0, 5.4686079238308594e-05, 0.9999998807907104, 0.00018728295981418341, 2.6646668629837222e-05, 0.9999978542327881, 1.0, 1.0, 0.0013457220047712326, 5.973267320769082e-07, 3.511190880089998e-05, 1.0, 0.9999966621398926, 1.0, 4.5905391743872315e-05, 1.0, 0.9999994039535522, 0.9999980926513672, 0.9999986886978149, 0.00020239557488821447, 1.425034270141623e-06, 0.9999948740005493, 0.9999997615814209, 1.0, 8.796263136900961e-05, 0.9999997615814209, 0.9999991655349731, 1.0, 0.9999998807907104, 0.0009248093119822443, 0.9995230436325073, 2.0663276245613815e-07, 0.00013524311361834407, 0.9999896287918091, 8.924601388571318e-06, 0.9996317625045776, 9.341936220153002e-07, 1.0, 0.9999613761901855, 7.610641205246793e-06, 0.9999793767929077, 0.9999991655349731, 0.9999933242797852, 0.0006420068675652146, 2.123601234416128e-06, 0.00010337876301491633, 0.9999366998672485, 0.9999833106994629, 3.33863027890402e-07, 2.3951928596943617e-05, 0.9999995231628418, 0.9996747970581055, 1.0, 0.9999985694885254, 1.6640498870401643e-05, 3.320123369121575e-06, 0.9999970197677612, 0.0007702300790697336, 1.0, 6.019372813170776e-05, 0.9999995231628418, 0.00023595937818754464, 4.588271985994652e-05, 0.9999992847442627, 1.0, 0.999998927116394, 0.000377947639208287, 0.0022894570138305426, 7.171034667408094e-05, 0.9999997615814209, 0.9999984502792358, 0.9998447895050049, 1.3335990161067457e-06, 0.9999986886978149, 5.895787398912944e-05, 1.0, 0.9999940395355225, 1.0, 0.0023644340690225363, 0.0003267147403676063, 0.9999920129776001, 3.0762817004870158e-06, 0.0006998602184467018, 0.9999984502792358, 2.5876832296489738e-05, 0.9999995231628418, 6.462818419095129e-05, 8.609347901256115e-07, 0.9999991655349731, 1.0, 5.127099939272739e-05, 0.9999991655349731, 5.101847364130663e-06, 0.0156251173466444, 1.0, 0.00019670711481012404, 3.305557038402185e-05, 0.9999170303344727, 2.0129336917307228e-06, 2.3136217350838706e-06, 3.5871187265001936e-07, 0.9999220371246338, 0.9999630451202393, 0.0005460442625917494, 2.554425236667157e-06, 0.9999997615814209, 0.999665379524231, 5.432500529423123e-07, 1.196799303215812e-06, 1.0, 3.633296364569105e-05, 0.9999995231628418, 5.720485205529258e-06, 1.7430449588573538e-05, 0.00016063063230831176, 2.812429920595605e-05, 2.4998869776027277e-05, 0.0002546638424973935, 0.9999897480010986, 1.0, 2.9979157716297777e-06, 1.0, 1.0, 0.999998927116394, 0.9999991655349731, 0.0006529431557282805, 1.0, 1.0, 1.0, 4.511345252922183e-07, 3.0775307095609605e-05, 0.0008427771972492337, 5.128879638505168e-05, 5.6630629842402413e-05, 3.763565473491326e-06, 0.9999997615814209, 0.9999992847442627, 1.0, 5.245896318228915e-06, 4.295219332561828e-05, 0.9984121322631836, 0.9999971389770508, 7.081707735778764e-06, 0.00011655577691271901, 1.1599098797887564e-05, 2.3776576654199744e-06, 4.0502236515749246e-05, 0.00013831316027790308, 1.0, 5.888140890419891e-07, 0.9999982118606567, 6.311522156465799e-06, 0.9999209642410278, 0.9999998807907104, 0.007151592988520861, 0.9999958276748657, 0.0003144511429127306, 0.9999914169311523, 0.001470533199608326, 7.057934908516472e-06, 1.0, 0.9962553977966309, 4.1330822568852454e-05, 1.0, 8.904057904146612e-05, 0.9999991655349731, 3.1179897632682696e-05, 0.9999850988388062, 5.210238214203855e-06, 0.004051489289849997, 1.0, 1.0, 0.9999998807907104, 1.0, 1.0, 2.3270860765478574e-05, 0.9999817609786987, 0.9999992847442627, 0.9957222938537598, 1.0, 1.0, 0.0027276258915662766, 0.999427855014801, 0.9999912977218628, 3.123147962469375e-07, 0.9999996423721313, 0.9999998807907104, 0.9999971389770508, 1.6369959610074147e-07, 1.0, 0.9998204112052917, 1.0, 0.009023357182741165, 0.0006539177265949547, 1.0, 4.9349084292771295e-05, 8.727048225409817e-06, 0.9999991655349731, 2.3872844394645654e-06, 0.999988317489624, 0.9999823570251465, 6.732597768177584e-08, 1.0, 0.0007724498282186687, 1.0, 1.5974570487742312e-05, 0.9998413324356079, 1.0, 0.00015964060730766505, 0.9999986886978149, 2.8779866624972783e-05, 3.1454659620067105e-05, 4.30523368777358e-06, 9.211669384967536e-05, 1.3551854181059753e-06, 0.0001832059642765671, 1.357743855123772e-07, 0.000374583643861115, 0.00014256284339353442, 0.003220916260033846, 0.9999996423721313, 1.9666716980282217e-05, 1.6446241488665692e-06, 0.9999773502349854, 6.459713404183276e-07, 0.9999949932098389, 1.0, 0.9992413520812988, 0.0006198232294991612, 2.645969743753085e-06, 8.975882224149245e-07, 0.9999920129776001, 4.5112047430961866e-09, 2.220661008323077e-05, 0.9999998807907104, 0.0010794237023219466, 0.00040498480666428804, 1.0, 0.000324608146911487, 1.0, 0.00040922287735156715, 0.999995231628418, 0.9999998807907104, 0.9999957084655762, 0.9999997615814209, 0.9999498128890991, 0.9999984502792358, 2.2744441139366245e-06, 0.9999998807907104, 3.3822976547526196e-05, 4.010035809187684e-06, 4.7771291065146215e-06, 0.999997615814209, 0.0003309705061838031, 0.9999998807907104, 6.191596912685782e-05, 5.031818659517739e-07, 1.582793083798606e-05, 0.9999994039535522, 0.9999984502792358, 0.9999988079071045, 5.857679934706539e-05, 0.9895145893096924, 0.0006176066235639155, 3.3208048989763483e-05, 9.077204055074617e-08, 9.19110561881098e-07, 1.3540468899009284e-05, 0.9999932050704956, 1.9441416952759027e-05, 0.9999927282333374, 0.9999994039535522, 1.0, 0.9999624490737915, 0.0005370863946154714, 1.4848200180495041e-06, 0.9999998807907104, 0.0007956537301652133, 3.8011792639736086e-05, 5.9994988987455145e-05, 0.9999983310699463, 4.772557076648809e-06, 9.576534694133443e-07, 0.0006867389311082661, 0.9999992847442627, 2.1025625756010413e-06, 0.9999994039535522, 7.677768735447899e-06, 0.9999911785125732, 0.9999985694885254, 1.4955979565911548e-07, 5.665482967742719e-05, 6.404936357284896e-06, 0.9999995231628418, 1.0, 1.8983050722454209e-06, 1.0, 0.9999544620513916, 3.897356509696692e-05, 1.0, 0.9999784231185913, 0.999997615814209, 0.00016728720220271498, 1.0, 0.9999864101409912, 0.9994611144065857, 1.0, 0.0070257894694805145, 1.0, 0.00010036541061708704, 0.9999929666519165, 3.8669845707772765e-06, 0.9999889135360718, 2.842893445631489e-05, 2.1129440938238986e-05, 0.9999980926513672, 1.0, 0.00020515317737590522, 0.9999964237213135, 0.9999951124191284, 0.00010418848250992596, 4.5590149966301396e-05, 0.9949955940246582, 1.9392753529245965e-05, 0.0011437756475061178, 2.0328587197582237e-05, 0.9999841451644897, 1.0, 1.0, 0.9999996423721313, 4.878773779637413e-06, 0.9999768733978271, 0.0003791596391238272, 1.0, 6.615861639147624e-05, 5.6858167226891965e-05, 1.673766746534966e-06, 1.0, 0.9999980926513672, 3.3829248877736973e-06, 2.138725176337175e-05, 0.9999991655349731]

#Metricas de validación
Best_acc10_5 = 1.0
acc_aux10_5 = 1.0
Recall10_5 = 1.0
Especificidad10_5 = 1.0
Tiempo10_5 = 1511.5584514141083
###################################################################

######## Base1 ########
#MODELO = VGG16_BN
#Datos perdidas
xx_train1_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train1_6 = [0.22328280054124786, 0.09663783057972238, 0.07726140910828555, 0.06675558399271082, 0.06222702368725965, 0.049089050256175756, 0.05274832880607358, 0.05507819519182782, 0.04982166044689991, 0.04890409161647161, 0.048172943110083355, 0.04820678694932549, 0.044411889160120926, 0.043874588783508466, 0.0460646236292374, 0.04416135288315055, 0.045284808997386766, 0.05189539329321296, 0.04828891179259912, 0.04165605661677726, 0.039807937174667546, 0.042373877892523636, 0.04823108485635416, 0.04630232044024232, 0.046731501045050444]
xx_val1_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val1_6 = [0.07305911944972145, 0.06880606843365564, 0.07462631240487098, 0.06253433939483431, 0.04702816853920619, 0.050173427992396886, 0.04697648104694155, 0.044869090947839946, 0.04087354938189189, 0.04214409954018063, 0.04013463854789734, 0.04200088034073512, 0.042243247893121505, 0.03835589438676834, 0.041272849672370486, 0.04226904312769572, 0.044935373961925505, 0.04013270967536502, 0.04193731745084127, 0.04459837509526147, 0.04134498900837368, 0.04373873538441128, 0.043003948198424444, 0.0417876531680425, 0.04235941610402531]
epoch_min1_6 = 8
stop_val1_6 = 0.04087354938189189

#Datos accuracy
acc_train1_6 = [0.9040123456790123, 0.9645061728395061, 0.9700617283950617, 0.9765432098765432, 0.9768518518518519, 0.9814814814814815, 0.9796296296296296, 0.9783950617283951, 0.9802469135802468, 0.9805555555555555, 0.9814814814814815, 0.9817901234567901, 0.9842592592592593, 0.9839506172839506, 0.9811728395061728, 0.983641975308642, 0.9851851851851852, 0.9808641975308642, 0.9805555555555555, 0.9848765432098765, 0.9851851851851852, 0.9842592592592593, 0.9808641975308642, 0.9820987654320987, 0.9824074074074074]
acc_val1_6 = [0.9722222222222222, 0.9777777777777779, 0.9833333333333334, 0.9694444444444444, 0.9861111111111112, 0.9833333333333334, 0.9888888888888889, 0.9833333333333334, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9888888888888889, 0.9861111111111112, 0.9888888888888889, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112]

#Datos curva ROC
best_labels1_6 = [0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0]
best_prob1_6 = [1.416397026332561e-05, 4.884306326857768e-05, 1.0, 0.9999624490737915, 5.708615390176419e-06, 0.9998338222503662, 0.9999663829803467, 0.06845847517251968, 2.473945642122999e-06, 0.002314390614628792, 0.9998849630355835, 0.9998315572738647, 0.9778689742088318, 5.226114080869593e-05, 0.0005461393157020211, 1.0, 0.9997758269309998, 1.1268565685895737e-05, 0.9998151659965515, 0.9999984502792358, 0.013872737996280193, 0.9999940395355225, 0.9999992847442627, 0.00041095310007221997, 0.00042538694106042385, 0.06759855151176453, 0.0015787857118993998, 0.9999997615814209, 4.505048127612099e-05, 3.919391383533366e-05, 0.9992430210113525, 0.0009956335416063666, 0.999691367149353, 7.798562000971287e-05, 0.9999997615814209, 2.160980147891678e-05, 0.9997419714927673, 0.0015580120962113142, 0.9999566078186035, 0.224894717335701, 0.9999997615814209, 7.142513368307846e-06, 0.9999997615814209, 0.9998249411582947, 0.9705156087875366, 0.0020540154073387384, 0.9999299049377441, 0.9989091157913208, 0.9996590614318848, 0.000263748544966802, 0.004046793561428785, 0.03424883261322975, 0.00245053693652153, 2.653881165315397e-05, 2.7389134629629552e-05, 1.5097348295967095e-06, 0.999613344669342, 0.9999831914901733, 0.00015588586393278092, 0.9999996423721313, 0.999964714050293, 0.9999794960021973, 0.9999998807907104, 0.9998433589935303, 0.9998574256896973, 0.000649951514787972, 0.9999872446060181, 0.00011537578393472359, 0.9996572732925415, 0.9998550415039062, 0.9999600648880005, 0.0009396631503477693, 0.9999984502792358, 0.00013210873294156045, 0.9982764720916748, 6.460698932642117e-05, 0.004832623526453972, 0.9999368190765381, 0.9978251457214355, 0.043405476957559586, 0.011180730536580086, 0.9973203539848328, 0.000530691584572196, 0.9994025230407715, 1.0, 0.00022979597270023078, 0.9773228168487549, 0.9968738555908203, 0.0014435022603720427, 0.9994643330574036, 0.9999963045120239, 0.4645227789878845, 0.01331714354455471, 3.292312248959206e-05, 1.0, 0.9999884366989136, 1.647754652367439e-05, 0.9999163150787354, 3.6762179661309347e-05, 0.0006232431042008102, 2.7781095923273824e-05, 0.00013915926683694124, 3.341969568282366e-05, 0.9931204319000244, 0.9963395595550537, 0.9997435212135315, 0.00030730970320291817, 0.00012434032396413386, 0.9999830722808838, 0.0002884287678170949, 0.9999637603759766, 0.00038931629387661815, 4.60682576886029e-06, 0.9998741149902344, 0.9999425411224365, 0.0008808516431599855, 0.9999467134475708, 0.9999998807907104, 0.9999485015869141, 0.0005954723455943167, 0.00022979597270023078, 0.998144268989563, 0.00018241304496768862, 0.9999991655349731, 0.002599055878818035, 6.846608448540792e-05, 9.812654025154188e-05, 0.9985142350196838, 0.9959646463394165, 6.999885044933762e-06, 0.9998936653137207, 1.3701043826586101e-05, 0.9999299049377441, 0.9999101161956787, 0.0010976246558129787, 3.136264786007814e-05, 3.4799293644027784e-05, 0.9995735287666321, 0.9999810457229614, 0.00016874958237167448, 0.0001487703702878207, 3.0890067137079313e-05, 0.00032881301012821496, 3.983667284046533e-06, 0.0003840875579044223, 0.9999856948852539, 0.0016043595969676971, 0.9999493360519409, 0.9996542930603027, 1.7918877347256057e-05, 0.9999874830245972, 0.9999971389770508, 0.00040618586353957653, 0.9999918937683105, 0.9998406171798706, 0.9999845027923584, 0.9867968559265137, 3.115103936579544e-06, 5.84206500207074e-05, 7.009480759734288e-05, 0.9999101161956787, 0.9999902248382568, 0.999457061290741, 0.0001187057641800493, 0.9999822378158569, 0.0001398124877596274, 4.637926394934766e-06, 0.998862624168396, 0.9994387030601501, 0.997118353843689, 3.161814311170019e-05, 0.99997878074646, 0.9999250173568726, 0.00013976528134662658, 0.0020614834502339363, 0.9998086094856262, 0.9999986886978149, 0.9999421834945679, 1.0325657058274373e-05, 0.9876642823219299, 0.0038605753798037767, 3.369799742358737e-05, 0.0022701057605445385, 0.004601499065756798, 0.9566282629966736, 0.9999889135360718, 0.1535732001066208, 0.9999417066574097, 0.00027311115991324186, 0.006030249409377575, 0.9988757967948914, 0.0003865965409204364, 0.00011547958274604753, 2.2985173018241767e-06, 4.857069143326953e-05, 1.0, 1.0, 0.9997784495353699, 0.0023176120594143867, 0.969711184501648, 4.962004459230229e-05, 0.9964872598648071, 0.9999210834503174, 0.9999371767044067, 0.9999994039535522, 0.0008162728627212346, 0.9999997615814209, 4.113712566322647e-05, 0.9999841451644897, 0.9999754428863525, 0.002430704655125737, 0.00022197926591616124, 0.00035366154043003917, 1.4587289115297608e-05, 9.636122740630526e-06, 0.9999997615814209, 0.00031112172291614115, 7.027618266874924e-05, 0.9999802112579346, 0.999984860420227, 0.9999978542327881, 0.9999996423721313, 0.9999986886978149, 0.999573290348053, 0.765569269657135, 0.0005278236931189895, 0.0003554431605152786, 0.002335131634026766, 6.511888932436705e-05, 0.9999430179595947, 0.00010346535418648273, 0.9999880790710449, 3.5725548514164984e-05, 0.0006204231758601964, 0.9999918937683105, 0.9661507606506348, 0.9925959706306458, 0.9999580383300781, 0.9999969005584717, 3.057591675315052e-05, 0.00022407021606341004, 0.046507637947797775, 0.9999932050704956, 2.3849792341934517e-05, 0.9999974966049194, 0.9999587535858154, 1.059690384863643e-05, 0.9998756647109985, 1.8634109437698498e-06, 3.6968415315641323e-06, 0.9998185038566589, 0.0031466030050069094, 0.9999129772186279, 0.9998867511749268, 0.0015193079598248005, 0.1778964102268219, 4.490717037697323e-05, 2.4993385522975586e-05, 0.0007221326814033091, 0.00025440219906158745, 0.9998533725738525, 3.9033679058775306e-05, 0.003047533333301544, 0.9999939203262329, 0.0027881634887307882, 7.827051740605384e-05, 0.0020321831107139587, 0.9999552965164185, 0.9999994039535522, 5.03918954564142e-06, 0.9999562501907349, 0.9999970197677612, 0.9998026490211487, 0.00013458482862915844, 0.9959762692451477, 0.004827945958822966, 0.14526766538619995, 0.9997302889823914, 0.00040668706060387194, 0.9999948740005493, 0.7767869830131531, 0.0011304273502901196, 0.9999558925628662, 6.618228508159518e-05, 0.8475737571716309, 0.00011651733075268567, 9.523938206257299e-05, 0.9998999834060669, 0.9999815225601196, 0.026429833844304085, 7.312931302294601e-06, 0.9999781847000122, 0.9999548196792603, 0.001633734442293644, 0.0015282378299161792, 0.9997219443321228, 0.0018154553836211562, 0.0003973873390350491, 0.9999953508377075, 0.9999986886978149, 0.9951320886611938, 0.9999725818634033, 0.0008558569243177772, 0.00011521505075506866, 0.9998791217803955, 2.79115884040948e-05, 0.9999988079071045, 0.0008716218289919198, 0.9998359680175781, 0.9999973773956299, 0.9999645948410034, 0.0004216400266159326, 0.00012314468040131032, 0.999954342842102, 8.68956558406353e-05, 0.7295151948928833, 0.9998563528060913, 6.346794543787837e-05, 0.00038090156158432364, 3.727532384800725e-05, 0.9393948316574097, 0.0012674253666773438, 0.9994468092918396, 2.0702438632724807e-05, 0.7997877597808838, 0.001417192630469799, 0.9982174038887024, 0.9999994039535522, 0.9965399503707886, 0.0005199026199989021, 2.2197906218934804e-05, 0.9998427629470825, 0.9998971223831177, 7.905891834525391e-05, 0.9997116923332214, 0.9999784231185913, 0.9379373788833618, 0.9999508857727051, 0.00016291867359541357, 0.0003593440051190555, 0.0009680128423497081, 0.00011576126416912302, 4.246841126587242e-05, 0.00018048568745143712, 0.9996403455734253, 0.2481822371482849, 0.9999406337738037, 0.00019143038662150502, 0.00010558762005530298, 0.9992725253105164, 0.9966654181480408, 0.0002806904085446149, 0.0008169318316504359, 0.999897837638855, 0.0036272008437663317, 0.9995005130767822, 0.9999887943267822, 0.004509416408836842, 4.8353940655943006e-05, 0.9998227953910828]

#Metricas de validación
Best_acc1_6 = 0.9888888888888889
acc_aux1_6 = 0.9888888888888889
Recall1_6 = 0.9888888888888889
Especificidad1_6 = 0.9888888888888889
Tiempo1_6 = 1528.0537061691284

######## Base2 ########
#MODELO = VGG16_BN
#Datos perdidas
xx_train2_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train2_6 = [0.08169865470241618, 0.05317253753726865, 0.06051266158436552, 0.049184519512417875, 0.041654344657321034, 0.04010943036388468, 0.029852929058266274, 0.03522271087140213, 0.031499469252648175, 0.03296808844179283, 0.03142809719767099, 0.029344336771302753, 0.02872807847128974, 0.03392400918183503, 0.03230123069183326, 0.03277028509312206, 0.03602668034073747, 0.027500630133313896, 0.026273982493597785, 0.03357628659333712, 0.0350994992311354, 0.03320566081890353, 0.026329946407565363, 0.03313440313493764, 0.03186302684523441]
xx_val2_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val2_6 = [0.10175004187557432, 0.1974435206916597, 0.026397143801053364, 0.02962845497661167, 0.032269979102744, 0.02904888242483139, 0.028092222412427266, 0.03079111443625556, 0.028575275341669717, 0.02895680765310923, 0.026463030278682707, 0.02396114452017678, 0.029297299683094025, 0.026839348011546665, 0.025297465920448303, 0.02721745330426428, 0.025563087148798836, 0.027238933079772525, 0.031448675029807624, 0.02789150526126226, 0.025104305810398526, 0.026139553719096712, 0.023300081491470337, 0.02681335541937086, 0.026112261994017495]
epoch_min2_6 = 2
stop_val2_6 = 0.026397143801053364

#Datos accuracy
acc_train2_6 = [0.9746913580246913, 0.9799382716049383, 0.9796296296296296, 0.9830246913580246, 0.9876543209876543, 0.9864197530864197, 0.9882716049382716, 0.9879629629629629, 0.9888888888888888, 0.9867283950617284, 0.987037037037037, 0.9916666666666667, 0.9898148148148148, 0.9888888888888888, 0.9891975308641975, 0.987037037037037, 0.9851851851851852, 0.9901234567901235, 0.9916666666666667, 0.9858024691358025, 0.9867283950617284, 0.9879629629629629, 0.990432098765432, 0.9885802469135803, 0.9879629629629629]
acc_val2_6 = [0.9666666666666667, 0.9277777777777778, 0.9888888888888889, 0.9861111111111112, 0.9833333333333334, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9833333333333334, 0.9861111111111112, 0.9888888888888889, 0.9861111111111112, 0.9861111111111112, 0.9888888888888889, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9805555555555556, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9833333333333334, 0.9861111111111112, 0.9861111111111112]

#Datos curva ROC
best_labels2_6 = [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
best_prob2_6 = [0.0002770067367237061, 0.999032735824585, 0.9999918937683105, 0.11375246942043304, 0.9999985694885254, 0.9996016621589661, 0.9956965446472168, 0.9998594522476196, 0.0005868892767466605, 0.9999856948852539, 0.9996592998504639, 0.0019658340606838465, 0.9998332262039185, 0.9998049139976501, 0.9997135996818542, 0.9999980926513672, 0.000409050437156111, 0.00023757797316648066, 0.001645707176066935, 0.0025098551996052265, 0.9980338215827942, 0.967172384262085, 4.863362846663222e-05, 0.00012243105447851121, 0.0007815161952748895, 0.0009428611374460161, 3.6776553315576166e-05, 0.0009583845385350287, 1.084257473848993e-05, 0.9999500513076782, 0.009076399728655815, 0.0006881602457724512, 0.9995818734169006, 8.626240742160007e-06, 0.00044745937339030206, 0.9998247027397156, 0.9998786449432373, 0.00040945305954664946, 0.0008317286265082657, 0.9995954632759094, 9.647345723351464e-05, 0.002605522284284234, 0.000847191084176302, 0.0019890705589205027, 0.9999946355819702, 0.9998791217803955, 0.0004043635562993586, 7.711551916145254e-06, 0.00562446191906929, 3.541506885085255e-05, 0.9999991655349731, 0.9996335506439209, 0.9999874830245972, 0.00045490654883906245, 0.9998332262039185, 3.5647139156935737e-05, 0.9998502731323242, 0.0004807021177839488, 0.00039638872840441763, 0.9999715089797974, 0.0010037400061264634, 0.0050431168638169765, 0.9999958276748657, 0.999846339225769, 0.9999262094497681, 0.9998941421508789, 0.9999359846115112, 6.286503776209429e-05, 0.9783907532691956, 0.03975005820393562, 0.9994600415229797, 0.00016575482732150704, 0.9999990463256836, 0.014532327651977539, 0.9998549222946167, 0.9994186162948608, 0.013693314976990223, 0.9994168281555176, 0.0002828295109793544, 0.9998700618743896, 0.9998568296432495, 0.00019578536739572883, 0.9998594522476196, 0.01791193149983883, 0.9998093247413635, 6.706051499349996e-05, 5.303217767504975e-05, 0.0006653756718151271, 0.00023778510512784123, 9.49050263443496e-06, 0.9999490976333618, 0.0005929820472374558, 0.999997615814209, 0.0009430593345314264, 0.005761726759374142, 9.772071643965319e-05, 0.0005474823992699385, 0.00021710098371841013, 0.000526893010828644, 0.9587413668632507, 0.9999408721923828, 9.215122554451227e-05, 0.9999996423721313, 0.9998834133148193, 8.518103277310729e-05, 0.0037981339264661074, 9.776304068509489e-05, 0.0004716502735391259, 0.001331964274868369, 2.4688813937245868e-05, 0.9999947547912598, 0.9986483454704285, 0.0002664994972292334, 0.9999994039535522, 0.002444243524223566, 0.9998465776443481, 1.0611193829390686e-05, 0.013476121239364147, 0.999983549118042, 8.409318252233788e-05, 0.998907208442688, 0.9999979734420776, 0.0002093025395879522, 5.306414459482767e-05, 0.00010867201490327716, 0.9997316002845764, 0.9971762895584106, 0.3444916009902954, 1.5531362805631943e-05, 1.2643909940379672e-05, 0.01481722667813301, 0.0006320392712950706, 0.9913715124130249, 0.9999871253967285, 0.9998925924301147, 0.999795138835907, 0.9999986886978149, 0.9998145699501038, 0.9997792840003967, 0.03129492700099945, 0.9999306201934814, 0.9998138546943665, 0.0008013246697373688, 0.00042540780850686133, 0.9998677968978882, 0.001539476215839386, 0.9999961853027344, 0.9999305009841919, 0.11445514857769012, 0.9999973773956299, 0.9999997615814209, 0.00767130684107542, 0.9996919631958008, 0.9999239444732666, 0.03844105452299118, 1.9235749277868308e-05, 0.000668057007715106, 0.9986898303031921, 3.6683311918750405e-05, 0.9990131855010986, 3.9100665162550285e-05, 1.565050297358539e-05, 0.013413588516414165, 0.0003534340357873589, 0.9999901056289673, 0.9999479055404663, 0.0004674073134083301, 0.0025633361656218767, 0.9923976063728333, 0.9998984336853027, 0.9991989731788635, 0.000704255246091634, 0.0007772622630000114, 0.0006300191744230688, 0.00014870711311232299, 0.9999635219573975, 0.9999843835830688, 0.2550657391548157, 0.9986960291862488, 7.955457840580493e-05, 0.9999731779098511, 2.4010249035200104e-05, 0.0014748724643141031, 0.9893217086791992, 0.0005252148257568479, 0.022015005350112915, 0.0011859822552651167, 0.9999995231628418, 0.0006690943264402449, 0.00012037384294671938, 0.03432997688651085, 0.9990677237510681, 0.00038068098365329206, 0.0017822616500779986, 0.00020132293866481632, 0.9999794960021973, 0.9996871948242188, 0.00016839597083162516, 0.0011210772208869457, 0.999229907989502, 0.9989315867424011, 0.00018522060418035835, 7.915275637060404e-05, 0.0008565825992263854, 0.00015156752488110214, 0.9640632271766663, 0.0006348362658172846, 0.002470671432092786, 0.9999980926513672, 0.9999376535415649, 0.00015299647930078208, 0.0008621212327852845, 0.9999666213989258, 0.9999784231185913, 0.9999961853027344, 0.9999810457229614, 0.004706935957074165, 4.029604679089971e-05, 0.9973502159118652, 0.9999982118606567, 0.9996695518493652, 0.0009210404241457582, 0.9994903802871704, 0.9987552165985107, 0.9999943971633911, 0.9999970197677612, 0.9999864101409912, 0.9999983310699463, 0.9998840093612671, 4.651230847230181e-05, 2.6916472052107565e-05, 0.9995390176773071, 0.0009779257234185934, 0.5539081692695618, 0.9999998807907104, 0.9999880790710449, 0.004780793562531471, 6.556122389156371e-05, 0.9984161853790283, 0.0008256715955212712, 0.9995793700218201, 0.9997970461845398, 0.996845543384552, 0.002007482573390007, 0.998960018157959, 0.6979619264602661, 0.999752938747406, 0.9999974966049194, 0.9998741149902344, 0.0065788025967776775, 0.03784485533833504, 0.9991958737373352, 0.994712233543396, 0.9985774755477905, 0.004085870925337076, 2.2776286641601473e-05, 0.0008354380843229592, 7.475935126421973e-05, 0.9999415874481201, 2.5236966394004412e-05, 0.999290943145752, 0.999866247177124, 0.9999908208847046, 0.9999005794525146, 6.310601747827604e-05, 0.9929983615875244, 0.9999548196792603, 0.003647837322205305, 0.9974902868270874, 0.9991115927696228, 0.9952974915504456, 0.004772186744958162, 0.00041763053741306067, 0.9999324083328247, 0.9999929666519165, 8.355265890713781e-05, 0.9993956089019775, 0.04221143573522568, 0.9997614026069641, 0.0040990510024130344, 9.014472016133368e-05, 0.9991942048072815, 0.00013862858759239316, 0.9987924098968506, 0.999929666519165, 0.00038764701457694173, 1.8085782357957214e-05, 0.9978277087211609, 0.8219470381736755, 0.9988558292388916, 0.9998263716697693, 0.0003931357350666076, 0.0003365030570421368, 0.9999988079071045, 0.0071136243641376495, 0.9997074007987976, 0.999841570854187, 0.9995982050895691, 0.9999591112136841, 0.9999814033508301, 9.817576210480183e-05, 0.9999978542327881, 0.00019462077762000263, 0.005983142647892237, 0.010698369704186916, 0.9999996423721313, 0.001748204231262207, 0.00037043599877506495, 0.9999942779541016, 2.6181330667895963e-06, 0.9997902512550354, 4.355049895821139e-05, 0.9998185038566589, 0.9987474679946899, 0.9999959468841553, 0.997139573097229, 2.831826168403495e-05, 0.9993699193000793, 6.184892845340073e-05, 0.0012670699506998062, 0.9964731335639954, 0.9974121451377869, 0.9999150037765503, 0.9999194145202637, 0.0011851995950564742, 0.000130755957798101, 5.745516682509333e-05, 0.0005294752190820873, 0.9971622824668884, 0.9998815059661865, 0.9998766183853149, 6.004042734275572e-05, 0.993678867816925, 0.0007817518780939281, 0.866412341594696, 0.9999980926513672, 0.003368723439052701, 5.734667502110824e-05, 0.9999402761459351, 0.842195451259613, 0.9999417066574097, 0.9790294170379639, 0.9998189806938171, 0.999870777130127, 0.9999973773956299, 2.8647578801610507e-05, 3.333801942062564e-05, 0.001442733220756054, 0.9995678067207336, 0.9999986886978149, 0.9999122619628906, 0.005207171197980642, 0.0004101789672859013, 0.9994181394577026, 6.415499956347048e-05, 0.000675804156344384, 0.00016774405958130956, 0.9998669624328613, 0.00010029739496530965, 0.0025396610144525766]

#Metricas de validación
Best_acc2_6 = 0.9888888888888889
acc_aux2_6 = 0.9916666666666667
Recall2_6 = 0.9835164835164835
Especificidad2_6 = 0.9943820224719101
Tiempo2_6 = 1557.760235786438

######## Base3 ########
#MODELO = VGG16_BN
#Datos perdidas
xx_train3_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train3_6 = [0.06544143307356187, 0.04360607233312395, 0.056755744491094426, 0.03694158878039431, 0.0359661025305589, 0.0426611326082989, 0.03055348892086818, 0.03206726395421558, 0.024144377761784894, 0.02920565492944953, 0.029609131261154456, 0.02789231339170609, 0.026634032674777656, 0.030945826790950915, 0.03475981767714759, 0.027677467437805952, 0.01875291191685347, 0.028202225497843306, 0.03250565063438298, 0.027302454927085357, 0.031861195382144714, 0.025871690951379728, 0.02569857638007329, 0.02831417992711067, 0.029746172567944467]
xx_val3_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val3_6 = [0.008300454666217169, 0.008311647176742553, 0.007889611687925126, 0.010604970157146454, 0.0146393112010426, 0.011096817089451684, 0.012320564024978213, 0.012999943561024137, 0.016065051820543078, 0.009988576670487722, 0.013013977971341874, 0.010857691533035702, 0.012460941738552517, 0.012036416762404972, 0.010936787393358018, 0.011688042845990922, 0.011314694583415986, 0.014263051417138841, 0.012610834174686008, 0.015095321337382, 0.011100864410400391, 0.011710066596666973, 0.011514232224888272, 0.012786233590708839, 0.01274310482872857]
epoch_min3_6 = 0
stop_val3_6 = 0.008300454666217169

#Datos accuracy
acc_train3_6 = [0.9765432098765432, 0.9842592592592593, 0.9759259259259259, 0.986111111111111, 0.9867283950617284, 0.9811728395061728, 0.987037037037037, 0.9888888888888888, 0.9907407407407407, 0.9888888888888888, 0.9885802469135803, 0.9901234567901235, 0.9907407407407407, 0.9882716049382716, 0.9864197530864197, 0.9885802469135803, 0.9929012345679012, 0.9895061728395061, 0.9882716049382716, 0.9901234567901235, 0.9873456790123457, 0.990432098765432, 0.9916666666666667, 0.9895061728395061, 0.9891975308641975]
acc_val3_6 = [0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9916666666666667, 0.9916666666666667, 0.9944444444444445, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9972222222222222, 0.9916666666666667, 0.9972222222222222, 0.9944444444444445, 0.9916666666666667, 0.9944444444444445, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9972222222222222, 0.9944444444444445, 0.9944444444444445, 0.9916666666666667, 0.9916666666666667]

#Datos curva ROC
best_labels3_6 = [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
best_prob3_6 = [0.9998847246170044, 0.0014749354450032115, 0.00011861953680636361, 0.002338993828743696, 0.999998927116394, 0.0005615467671304941, 0.003049361053854227, 0.9997568726539612, 0.9999969005584717, 2.222582588728983e-05, 0.0005687696393579245, 0.002562759444117546, 0.0002576043189037591, 7.067755632306216e-06, 0.9918397665023804, 0.9999406337738037, 0.9960069060325623, 1.1525034096848685e-05, 0.9997870326042175, 0.9998047947883606, 3.381708893357427e-06, 0.010708502493798733, 0.9998509883880615, 1.773563963070046e-05, 0.00034600673825480044, 0.9981235861778259, 0.0002873769262805581, 3.1291067443817155e-06, 0.9999711513519287, 0.9999420642852783, 0.9996869564056396, 0.9999885559082031, 0.0002397113130427897, 1.5595092918374576e-05, 0.0005831837188452482, 0.9999051094055176, 0.9999436140060425, 0.9983671307563782, 0.00017880942323245108, 1.3022952771279961e-05, 0.0009744986891746521, 0.9980629086494446, 0.9912882447242737, 1.2866577208114904e-06, 7.181560795288533e-05, 5.763400986324996e-05, 0.0008453662740066648, 0.9995518326759338, 0.9993871450424194, 0.9999771118164062, 2.3549380784970708e-05, 0.002838823478668928, 3.0452472401520936e-06, 0.9995946288108826, 0.0009890894871205091, 0.0002892223419621587, 0.9969791173934937, 0.9991589784622192, 0.9999924898147583, 0.0008654917473904788, 0.0009686945704743266, 0.9999575614929199, 0.9996633529663086, 0.0003523352788761258, 0.0023676615674048662, 0.0010508751729503274, 0.9979962110519409, 5.211194002185948e-05, 0.9986351132392883, 0.002432196633890271, 3.283777186879888e-05, 0.00549711287021637, 1.9097713447990827e-05, 7.88467423262773e-06, 0.9984042048454285, 0.9998080134391785, 0.15191572904586792, 0.9999700784683228, 0.0013463783543556929, 5.925982986809686e-05, 0.9993681311607361, 0.9996402263641357, 0.0002892071788664907, 0.9998118281364441, 0.9974386692047119, 1.0431863302073907e-06, 0.007305540610104799, 0.9995695948600769, 0.9961269497871399, 0.9949049949645996, 1.2038623253829428e-06, 0.9986087679862976, 6.8966910475865e-05, 0.9909862279891968, 0.9989983439445496, 0.0002254853316117078, 2.871779361157678e-05, 0.999987006187439, 8.529767910658848e-06, 0.0003432445228099823, 0.9971170425415039, 0.00012082463945262134, 0.0002852885518223047, 0.999748170375824, 0.0003970709803979844, 0.9714294672012329, 2.035056422755588e-05, 7.48276652302593e-05, 0.0025254858192056417, 4.4665437599178404e-05, 0.9999731779098511, 0.9968470931053162, 0.9809361696243286, 0.999489426612854, 0.9999724626541138, 0.9999951124191284, 0.001248989487066865, 0.990971565246582, 0.10103882849216461, 1.1511962838994805e-05, 0.9999474287033081, 8.575293031753972e-05, 0.9996408224105835, 0.9999067783355713, 0.9999828338623047, 0.30085113644599915, 0.0003925084602087736, 0.9994713664054871, 9.823000254982617e-06, 0.9992917776107788, 0.3369726836681366, 0.0002872136828955263, 6.690240297757555e-06, 0.9998714923858643, 2.1327003196347505e-05, 0.999991774559021, 1.7009243720167433e-06, 0.0007394696003757417, 5.285399993226747e-07, 0.9952906370162964, 0.0011262655025348067, 0.999840497970581, 0.00014102368731983006, 0.997352123260498, 0.999920129776001, 0.999309778213501, 0.9999978542327881, 0.9968611001968384, 0.00012756600335706025, 0.00016038591274991632, 0.9996669292449951, 0.0001313546672463417, 0.00012571144907269627, 0.00028788833878934383, 0.9920560717582703, 0.9994603991508484, 1.4600167332901037e-06, 0.9998262524604797, 0.00029213944799266756, 6.05218083364889e-05, 0.9953254461288452, 0.0015846474561840296, 0.9995459914207458, 0.9999363422393799, 0.99994957447052, 0.9997619986534119, 0.003990424331277609, 0.0009063333855010569, 0.9926812052726746, 0.01125349197536707, 0.00929106306284666, 0.9999227523803711, 0.00013233695062808692, 0.9999254941940308, 0.9995344877243042, 1.1476338841021061e-05, 0.9999881982803345, 4.6698105506948195e-06, 0.9999755620956421, 0.010910572484135628, 0.999778687953949, 0.9998774528503418, 0.9999728202819824, 0.0007216487429104745, 6.746692520209763e-07, 1.2641208741115406e-05, 3.589301559259184e-05, 0.9983274340629578, 2.968228181998711e-05, 0.00026370177511125803, 0.9990696310997009, 0.0001028095357469283, 0.9998860359191895, 0.004344157408922911, 3.013730383827351e-05, 0.0008828674326650798, 0.996657133102417, 6.215484972926788e-06, 0.00017182085139211267, 2.1317671780707315e-05, 0.9993454813957214, 0.0008481470285914838, 0.9998929500579834, 0.00023457377392333, 0.9991852641105652, 0.001494088675826788, 0.0003301942488178611, 2.99579569400521e-05, 0.9996591806411743, 0.9936417937278748, 0.9979438185691833, 0.9960812926292419, 0.0002033249766100198, 6.772143478883663e-06, 1.4118026228970848e-05, 0.9955165982246399, 4.3869072214874905e-06, 1.4774501323699951e-05, 8.403231731790584e-06, 0.9999948740005493, 0.9999748468399048, 0.003082336625084281, 0.9948115348815918, 0.9999774694442749, 0.0001632055500522256, 0.999995231628418, 0.9987931251525879, 0.0014826941769570112, 0.9991434812545776, 4.286985586077208e-06, 1.41422660817625e-05, 0.9999891519546509, 2.0266452338546515e-05, 0.9999887943267822, 0.9998722076416016, 0.9998874664306641, 0.0004286948824301362, 0.9998854398727417, 0.0004644161381293088, 0.9999823570251465, 0.9988658428192139, 0.0008317432948388159, 0.9995669722557068, 0.999882698059082, 0.014983553439378738, 3.364520671311766e-05, 7.694390546930663e-07, 0.9999470710754395, 0.0009043238824233413, 0.9999171495437622, 0.9999467134475708, 2.4176206352422014e-05, 0.99793541431427, 3.4898541798611404e-06, 3.548109089024365e-05, 0.9994875192642212, 0.9995251893997192, 0.9999419450759888, 0.9999047517776489, 1.68288461281918e-05, 0.9999245405197144, 6.570298864971846e-05, 0.9968825578689575, 3.7467201764229685e-05, 0.0011893014889210463, 0.9999252557754517, 0.9993667006492615, 0.9999938011169434, 0.9999619722366333, 1.6867939848452806e-05, 0.0001956125779543072, 6.588493124581873e-05, 0.9997777342796326, 0.9924542307853699, 5.6271219364134595e-06, 0.9165269136428833, 0.00015439046546816826, 0.9544017314910889, 0.0005517210811376572, 0.04537108540534973, 0.001603919779881835, 0.0020685831550508738, 0.00040145445382222533, 0.9996902942657471, 0.0038814591243863106, 0.9988020658493042, 0.9995081424713135, 0.00010909199772868305, 0.0062274509109556675, 0.9999364614486694, 0.993535041809082, 0.9998854398727417, 0.007126444485038519, 0.9998226761817932, 0.9911001324653625, 0.9999641180038452, 0.976549506187439, 1.2480645636969712e-05, 5.591466469923034e-05, 0.9999910593032837, 0.9999328851699829, 0.0003615469322539866, 1.6787369077064795e-06, 0.0022689371835440397, 0.0005751840653829277, 8.446972060482949e-06, 0.996673583984375, 0.9998922348022461, 0.00026329606771469116, 3.2978132367134094e-06, 0.9987882971763611, 0.9958574175834656, 1.930447979248129e-05, 0.9999096393585205, 2.6522757252678275e-06, 0.9999836683273315, 0.9988777041435242, 0.9999959468841553, 0.9998255372047424, 1.844755161073408e-06, 0.00025451407418586314, 0.6499114036560059, 0.0005238163867034018, 0.0011746236123144627, 0.9928798079490662, 0.999713122844696, 0.007542388513684273, 0.9999972581863403, 2.3887854695203714e-05, 0.9999700784683228, 0.0010040432680398226, 0.0002012108889175579, 0.9998676776885986, 0.9999960660934448, 0.9999798536300659, 6.19996280875057e-05, 0.011867297813296318, 0.999902606010437, 1.6302828953485005e-05, 0.9987348914146423, 0.9993619322776794, 0.00022950320271775126, 0.00013655827206093818, 0.999259889125824, 0.9999974966049194, 8.436835196334869e-05, 0.9743868112564087, 0.9973751306533813, 2.5210558305843733e-05, 0.9999895095825195, 0.0015489408979192376, 0.9995418787002563, 0.9997310042381287, 0.9995393753051758, 0.9999611377716064, 0.9997187256813049, 0.9999884366989136, 0.9985148310661316, 0.9984403252601624, 0.0004978914512321353]

#Metricas de validación
Best_acc3_6 = 0.9972222222222222
acc_aux3_6 = 0.9972222222222222
Recall3_6 = 1.0
Especificidad3_6 = 0.994475138121547
Tiempo3_6 = 1556.3426506519318

######## Base4 ########
#MODELO = VGG16_BN
#Datos perdidas
xx_train4_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train4_6 = [0.06496321785229224, 0.04828319137479052, 0.042547007530559726, 0.03868097565424295, 0.03976223172596943, 0.05097762283719616, 0.04161826182662705, 0.028872456263612817, 0.02626512481106652, 0.03247195255057311, 0.03474162228313493, 0.02961957449162448, 0.03198030245156936, 0.037557112195609525, 0.02558499244444164, 0.04189458166559537, 0.03542970242323699, 0.02778142801957366, 0.03290209213708654, 0.0246574577174069, 0.030647070153995797, 0.023748768350960296, 0.03254717922321072, 0.03482044772417457, 0.03245681979589992]
xx_val4_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val4_6 = [0.012221437361505296, 0.0076747947269015845, 0.05035061322980457, 0.025446447564495933, 0.020585764447848, 0.020216614339086746, 0.015264717075559828, 0.011092194418112437, 0.011854647431108686, 0.015987888475259147, 0.010859860479831696, 0.00947405132982466, 0.010240496363904741, 0.00888001521428426, 0.008701001273261176, 0.008557288514243231, 0.01094167646434572, 0.010458753837479486, 0.010068000356356303, 0.011020292672846052, 0.009081806739171345, 0.008994081285264757, 0.009859029452006023, 0.0090062172876464, 0.013121622138553196]
epoch_min4_6 = 1
stop_val4_6 = 0.0076747947269015845

#Datos accuracy
acc_train4_6 = [0.975, 0.9833333333333333, 0.9851851851851852, 0.9830246913580246, 0.9851851851851852, 0.9799382716049383, 0.9858024691358025, 0.9879629629629629, 0.9898148148148148, 0.9879629629629629, 0.9873456790123457, 0.9907407407407407, 0.987037037037037, 0.987037037037037, 0.9901234567901235, 0.986111111111111, 0.9867283950617284, 0.9891975308641975, 0.9879629629629629, 0.9919753086419753, 0.9882716049382716, 0.990432098765432, 0.9885802469135803, 0.9895061728395061, 0.9888888888888888]
acc_val4_6 = [0.9944444444444445, 0.9972222222222222, 0.9777777777777779, 0.9888888888888889, 0.9944444444444445, 0.9888888888888889, 0.9888888888888889, 0.9916666666666667, 0.9944444444444445, 0.9888888888888889, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9944444444444445, 0.9972222222222222, 0.9944444444444445, 0.9944444444444445, 0.9972222222222222, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9888888888888889]

#Datos curva ROC
best_labels4_6 = [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0]
best_prob4_6 = [0.999854326248169, 0.9998971223831177, 0.00014675194688607007, 0.00036720570642501116, 0.9998549222946167, 0.001983138732612133, 0.9994843006134033, 2.218868030468002e-05, 1.1936579085158883e-06, 0.9999916553497314, 0.9999886751174927, 8.643895125715062e-05, 0.9999959468841553, 0.9999191761016846, 0.002199712209403515, 0.00022411443933378905, 0.0006475256523117423, 0.9999710321426392, 0.9998959302902222, 0.00011192020610906184, 0.999351441860199, 4.619051651388872e-06, 0.9999921321868896, 0.9999645948410034, 0.9999992847442627, 2.8528686016215943e-05, 0.9999537467956543, 0.43459683656692505, 0.9999158382415771, 1.861956116044894e-05, 4.858844476984814e-06, 1.351240098301787e-05, 1.833226087910589e-05, 0.9999693632125854, 0.0003006963524967432, 0.99994957447052, 0.0009094926645047963, 0.9996479749679565, 0.001859198440797627, 0.9995569586753845, 0.9999933242797852, 0.9999399185180664, 7.650695806660224e-06, 0.9998899698257446, 0.9999017715454102, 2.557814514148049e-05, 0.9998118281364441, 0.00016497370961587876, 0.9999208450317383, 0.9972502589225769, 0.0003037542919628322, 0.00012854971282649785, 0.9999971389770508, 0.9991211295127869, 2.0944547941326164e-05, 0.003594493493437767, 0.00023178893025033176, 0.00033314849133603275, 0.9989064931869507, 0.9996476173400879, 5.337006223271601e-05, 1.0396216566732619e-05, 0.0002090410707751289, 0.00014222909521777183, 0.9999927282333374, 0.9999964237213135, 0.9996738433837891, 0.00030710353166796267, 2.518138717277907e-05, 0.9883660674095154, 0.00015403579163830727, 6.507593207061291e-05, 0.9999932050704956, 9.393821528647095e-06, 0.9999805688858032, 0.999079704284668, 0.9997946619987488, 0.0034423116594552994, 0.9999966621398926, 0.9993221759796143, 0.9999157190322876, 0.9999256134033203, 0.9997610449790955, 0.9986760020256042, 3.083773117396049e-05, 0.9999234676361084, 1.883048753370531e-05, 0.9997541308403015, 1.255139159184182e-05, 0.9999727010726929, 0.9999028444290161, 0.9999979734420776, 2.2510983399115503e-05, 0.9997406601905823, 0.9999651908874512, 0.9999675750732422, 5.643724580295384e-05, 0.9987459182739258, 0.763451337814331, 2.9572713174275123e-05, 5.295369919622317e-05, 4.0615908801555634e-05, 1.3889679451040138e-07, 1.2957902072230354e-05, 0.9999308586120605, 0.872509777545929, 5.825031257700175e-05, 4.926176643493818e-06, 0.9998201727867126, 0.9934762120246887, 0.9999285936355591, 0.9943568110466003, 0.9996727705001831, 0.9999260902404785, 0.9999794960021973, 0.9999053478240967, 0.0025644495617598295, 0.9999990463256836, 0.00019884594075847417, 0.9999852180480957, 0.001973612466827035, 0.9999867677688599, 0.9998683929443359, 6.304147063929122e-06, 0.9996263980865479, 0.9999927282333374, 0.9993399977684021, 1.4803837984800339e-05, 0.0007176336948759854, 0.0016941113863140345, 0.999961256980896, 6.740082608303055e-05, 0.000160288080223836, 0.0001829397660912946, 5.110001097818895e-07, 0.002541871275752783, 0.9998372793197632, 7.2383695624012034e-06, 4.16894254158251e-05, 0.999968409538269, 0.9999933242797852, 7.240081686177291e-06, 0.999955415725708, 0.008516600355505943, 7.352545708272373e-06, 0.9914557337760925, 0.0001731735246721655, 0.00012923181930091232, 0.00035827551619149745, 0.9997664093971252, 0.00014281699259299785, 0.9999316930770874, 2.983814920298755e-05, 3.575942537281662e-05, 0.9999938011169434, 0.9882766604423523, 0.9996324777603149, 0.9996329545974731, 0.0004824119678232819, 0.9999151229858398, 0.9829704761505127, 1.2799680462194374e-06, 1.860459815361537e-05, 1.0496180493646534e-06, 0.9974632263183594, 0.9983136653900146, 0.9992471933364868, 0.0001314234541496262, 6.860567373223603e-05, 0.9999599456787109, 0.9998608827590942, 0.9999573230743408, 2.756289916305832e-07, 0.999930739402771, 0.9999910593032837, 8.030038770812098e-06, 0.9999966621398926, 1.3389780178840738e-05, 8.945343142841011e-05, 0.00041027143015526235, 0.00016865918587427586, 0.9996637105941772, 0.00036160348099656403, 9.870876965578645e-05, 0.9995691180229187, 2.3382965537166456e-06, 0.010521866381168365, 0.0010539551731199026, 0.9999905824661255, 0.00010941683285636827, 0.00010575338819762692, 0.9999961853027344, 7.997779903234914e-06, 0.999523401260376, 0.00025816293782554567, 0.9999151229858398, 0.9999163150787354, 1.3076902177999727e-05, 0.9999406337738037, 0.9778404831886292, 0.00032269666553474963, 0.9994237422943115, 0.9998520612716675, 3.865520739054773e-06, 0.9999675750732422, 0.00014898301742505282, 0.010577169246971607, 0.8139901161193848, 0.9999470710754395, 0.00010343061876483262, 1.7301202888120315e-06, 0.017813798040151596, 3.781082114073797e-06, 0.00018041342264041305, 0.9997633099555969, 0.9999948740005493, 4.177965456619859e-05, 0.9997828602790833, 0.00042975335964001715, 0.5927324295043945, 2.1151417968212627e-05, 0.9792767763137817, 0.0058226389810442924, 0.9998706579208374, 0.035373128950595856, 0.999940037727356, 0.00028917138115502894, 0.0002559293352533132, 0.9761097431182861, 0.9999886751174927, 4.1680959839140996e-05, 0.9990942478179932, 0.9999938011169434, 0.9997606873512268, 0.00014008271682541817, 7.286313348231488e-07, 0.0001879446645034477, 9.11667575564934e-06, 0.9997074007987976, 0.9999960660934448, 1.4328195902635343e-05, 0.9985085129737854, 7.125898719095858e-06, 0.0015811071498319507, 0.0003622003714554012, 3.5492153983796015e-05, 0.00042655106517486274, 0.9999020099639893, 0.9999818801879883, 0.9999228715896606, 0.9999861717224121, 2.233077793789562e-05, 0.999972939491272, 3.714568265422713e-06, 0.9999599456787109, 0.9999368190765381, 0.00024743497488088906, 2.6184521630057134e-05, 5.3445033699972555e-05, 0.00015559345774818212, 5.202633474254981e-05, 3.7641002563759685e-06, 1.4177917364577297e-05, 0.999756395816803, 0.9479463696479797, 0.006088592577725649, 0.8426640033721924, 5.327170129021397e-06, 0.0003287951403763145, 4.335680898748251e-07, 1.8054118982036016e-06, 0.9999891519546509, 0.00011351632565492764, 0.9997360110282898, 0.9974023699760437, 0.9983515739440918, 0.8503211736679077, 0.9999877214431763, 5.828626308357343e-05, 0.9999852180480957, 0.005282335914671421, 2.5446920517424587e-06, 0.9999876022338867, 3.411595389479771e-05, 9.292452318732103e-07, 0.9998459815979004, 5.550653895625146e-06, 0.0008281741756945848, 0.0024213027209043503, 0.999880313873291, 0.9999935626983643, 0.9998748302459717, 0.9829298257827759, 0.0002708432439249009, 2.9782768251607195e-05, 2.9874126994400285e-06, 0.00014004037075210363, 1.0684550034056883e-05, 9.598243195796385e-06, 0.0004134427581448108, 8.336019732269051e-07, 0.9999997615814209, 0.9969684481620789, 0.0005751802236773074, 0.9984272718429565, 0.00012058109132340178, 0.0015419129049405456, 1.6386248944399995e-06, 1.0, 0.9998948574066162, 9.113502528634854e-06, 0.9999061822891235, 0.9844063520431519, 0.9999758005142212, 0.0008486825972795486, 6.7082573877996765e-06, 0.9999775886535645, 0.00014057659427635372, 0.00010799475421663374, 0.0007602706318721175, 0.9999942779541016, 0.9996882677078247, 0.0006268405122682452, 0.9999951124191284, 0.9999816417694092, 0.0011957723181694746, 0.9999958276748657, 2.6663615244615357e-06, 0.00033820216776803136, 1.0686404493753798e-05, 0.9916419982910156, 0.9999854564666748, 0.9999977350234985, 0.000910476257558912, 0.0019792146049439907, 2.0312931155785918e-05, 0.9999927282333374, 0.00726389791816473, 0.9999712705612183, 0.9999892711639404, 0.9999368190765381, 0.9664841890335083, 0.9998993873596191, 0.9999371767044067, 0.9999619722366333, 1.4244719750422519e-05, 0.9997331500053406, 0.999992847442627, 0.00016104841779451817, 1.5974335383361904e-06, 5.6272958318004385e-05, 0.9999850988388062, 0.00014102569548413157, 1.4909617675584741e-05, 0.9999759197235107, 0.9987369179725647, 1.7240479337488068e-06, 0.9996373653411865, 0.000183952521183528, 0.999681830406189]

#Metricas de validación
Best_acc4_6 = 0.9972222222222222
acc_aux4_6 = 0.9972222222222222
Recall4_6 = 1.0
Especificidad4_6 = 0.994475138121547
Tiempo4_6 = 1566.8071029186249

######## Base5 ########
#MODELO = VGG16_BN
#Datos perdidas
xx_train5_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train5_6 = [0.06073325594634186, 0.033831846290900386, 0.03882100620755443, 0.034493322376115826, 0.04455810205803977, 0.036786265588468973, 0.027607719443830444, 0.02737532798890714, 0.02820726152923372, 0.023090301012551342, 0.02515034767580621, 0.03204267368088534, 0.023671951595647837, 0.025587900314066145, 0.02646226877415622, 0.026984592922675757, 0.025337562995192447, 0.0220668762922287, 0.028661377470434447, 0.025047548777527278, 0.023737465148722683, 0.026109934451035512, 0.025474568484006106, 0.0315667781288977, 0.028046119820188592]
xx_val5_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val5_6 = [0.005676728155877855, 0.041537938018639885, 0.011322150296635098, 0.04643575731250975, 0.00405776451031367, 0.00395514749818378, 0.0057902713616689045, 0.003531264265378316, 0.004176446464326647, 0.003116249044736226, 0.004626126090685526, 0.005783439841535357, 0.0065409749746322635, 0.0046696989072693715, 0.008874316016832988, 0.005665715701050228, 0.007867273026042515, 0.005534880028830634, 0.005080395274692112, 0.005526816182666355, 0.00939088695579105, 0.00620503822962443, 0.007753422856330872, 0.003774060805638631, 0.009922200275792017]
epoch_min5_6 = 9
stop_val5_6 = 0.003116249044736226

#Datos accuracy
acc_train5_6 = [0.9780864197530864, 0.9879629629629629, 0.9867283950617284, 0.9867283950617284, 0.9845679012345678, 0.9879629629629629, 0.9922839506172839, 0.9907407407407407, 0.9898148148148148, 0.9922839506172839, 0.991358024691358, 0.9898148148148148, 0.990432098765432, 0.990432098765432, 0.9898148148148148, 0.991358024691358, 0.9919753086419753, 0.991358024691358, 0.991358024691358, 0.9888888888888888, 0.9919753086419753, 0.9898148148148148, 0.9916666666666667, 0.9882716049382716, 0.991358024691358]
acc_val5_6 = [0.9972222222222222, 0.9805555555555556, 0.9944444444444445, 0.9888888888888889, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 1.0, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222]

#Datos curva ROC
best_labels5_6 = [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
best_prob5_6 = [0.9999942779541016, 0.00022499225451610982, 0.0004182799893897027, 5.452286018226005e-07, 0.00047360139433294535, 1.2034206520183943e-05, 0.9999998807907104, 0.01797659508883953, 4.744905254483456e-06, 0.9982512593269348, 0.060777757316827774, 0.9999681711196899, 0.9999996423721313, 6.218919565981196e-07, 0.9999996423721313, 0.9999983310699463, 0.9999998807907104, 0.9999955892562866, 0.9999955892562866, 1.0749286047939677e-05, 0.0008622383465990424, 1.1825873258430875e-07, 0.9998953342437744, 0.9999992847442627, 1.3519965591513028e-07, 1.0, 1.0, 0.9999940395355225, 4.90195416205097e-05, 0.9995835423469543, 6.646054657721834e-07, 0.9862167835235596, 0.9993641972541809, 0.999997615814209, 0.9999687671661377, 0.9999988079071045, 3.853885573334992e-06, 0.9999759197235107, 0.9998811483383179, 6.666598892479669e-06, 4.391050879348768e-06, 0.9999992847442627, 0.9999984502792358, 0.999997615814209, 0.9999953508377075, 0.8397340178489685, 0.038905367255210876, 5.384486939874478e-05, 0.9999948740005493, 4.461998059923644e-07, 3.3466250215496984e-07, 0.9999964237213135, 1.0329780764095631e-07, 3.681192174553871e-05, 4.1786737710935995e-05, 1.1162910595885478e-06, 0.00014189776265993714, 1.0, 0.0003817618126049638, 0.9999970197677612, 0.9999990463256836, 1.1642163855185572e-07, 0.9999508857727051, 7.485472451662645e-05, 0.9999990463256836, 4.932452065986581e-05, 0.9998675584793091, 0.9999910593032837, 9.413514817424584e-06, 0.9999879598617554, 0.9999649524688721, 0.9999973773956299, 0.9999979734420776, 0.9999871253967285, 0.9997404217720032, 0.9999998807907104, 0.9999986886978149, 2.850806777132675e-05, 0.9999735355377197, 0.9999316930770874, 3.5236920666648075e-05, 0.9987397789955139, 0.999923586845398, 0.9999923706054688, 0.9999814033508301, 0.9999988079071045, 0.9999750852584839, 2.9655711841769516e-05, 0.9999998807907104, 2.8297476092120633e-05, 0.0010364989284425974, 0.9999955892562866, 0.9950101375579834, 0.006371049676090479, 0.00048472959315404296, 2.721985765674617e-05, 2.381920012339833e-06, 0.00010602994734654203, 0.0008143251179717481, 0.9999912977218628, 0.00030572732794098556, 0.9999819993972778, 2.5614619758584922e-08, 0.002971179550513625, 0.9995262622833252, 2.115170673278044e-06, 0.9999605417251587, 0.999976396560669, 0.9999998807907104, 0.9999912977218628, 5.70612792216707e-06, 0.0008424994302913547, 3.6849617117695743e-06, 1.6043881259975024e-05, 9.278982027183247e-09, 0.9998865127563477, 0.0002692333946470171, 0.9999818801879883, 0.00018674424791242927, 0.00013357172429095954, 0.9999833106994629, 0.015473618172109127, 0.9999948740005493, 1.4734277442585153e-07, 0.000301378226140514, 9.172531281365082e-05, 0.0004543743852991611, 0.9999996423721313, 2.443776338623138e-06, 0.0015422095311805606, 0.9999890327453613, 3.449939072197594e-07, 1.619101021788083e-05, 0.9997406601905823, 0.999994158744812, 0.9925976991653442, 2.0917743313475512e-05, 9.811064046516549e-06, 9.489561489317566e-05, 0.9999756813049316, 2.843820402631536e-05, 0.9999980926513672, 1.1866142735073026e-08, 1.9323028027429245e-05, 0.9999998807907104, 0.00020672479877248406, 0.9999737739562988, 0.00020464774570427835, 0.00011076527880504727, 0.999955415725708, 0.00017834252503234893, 0.001473995391279459, 0.9999979734420776, 0.9999938011169434, 4.797577730641933e-07, 1.672297003096901e-05, 0.0001622173876967281, 0.9999921321868896, 0.9997941851615906, 9.899239557853434e-06, 0.9999836683273315, 7.241595449158922e-05, 6.089725047786487e-06, 0.9999988079071045, 0.9997428059577942, 3.7727366475337476e-07, 0.0013696750393137336, 7.557262620139227e-07, 0.9999995231628418, 0.9999984502792358, 0.00015392713248729706, 0.0005541851278394461, 1.962857140824781e-06, 0.006128445267677307, 0.9999971389770508, 0.9999489784240723, 0.9999891519546509, 5.153985398465011e-07, 1.782360595825594e-05, 0.9991815686225891, 0.9999995231628418, 0.9999922513961792, 1.3577969184552785e-05, 0.9999986886978149, 0.9999257326126099, 0.00020433754252735525, 0.00013275654055178165, 5.9675098782463465e-06, 0.9997718930244446, 2.297009814355988e-06, 4.664923380914843e-06, 0.00036132504465058446, 1.4715948964294512e-05, 9.016290277941152e-06, 4.1786040583247086e-07, 0.9999898672103882, 1.8051019878839725e-06, 0.9999783039093018, 1.877791191873257e-06, 0.9999867677688599, 1.8936061962904205e-07, 0.9999297857284546, 7.941717399262416e-07, 0.013222765177488327, 0.9999710321426392, 0.999971866607666, 0.9999947547912598, 0.9999998807907104, 3.080848910030909e-05, 7.945005222609325e-07, 0.0013912766007706523, 0.0010306475451216102, 0.9999326467514038, 0.9999860525131226, 0.9999970197677612, 0.9999986886978149, 7.653947591279575e-07, 0.9999995231628418, 0.9994133710861206, 0.0001380510802846402, 4.404554232451119e-08, 4.816309342459135e-07, 0.9999988079071045, 0.9994475245475769, 0.9886455535888672, 1.0, 1.0, 0.9999935626983643, 8.530458580935374e-05, 0.9999997615814209, 0.9999885559082031, 0.9999819993972778, 0.9999582767486572, 0.999382734298706, 0.9999998807907104, 1.561691306051216e-06, 0.9999992847442627, 3.684736498144048e-07, 0.9999963045120239, 8.411193994106725e-05, 6.219938313734019e-06, 0.00027449813205748796, 6.922181796653604e-07, 0.999998927116394, 0.9999531507492065, 4.337169957580045e-05, 2.0721643068100093e-06, 0.00029119872488081455, 0.001786378095857799, 0.00025818852009251714, 0.9999990463256836, 0.9999992847442627, 0.9998127818107605, 0.00034346809843555093, 0.0001256155810551718, 5.916465397604043e-07, 0.9995085000991821, 0.9999994039535522, 0.9999996423721313, 5.602435066975886e-07, 3.1083767680684105e-05, 0.9992000460624695, 0.9999973773956299, 0.9999959468841553, 0.9999991655349731, 0.9999754428863525, 0.9999979734420776, 0.0009122046758420765, 0.9999988079071045, 0.99998939037323, 0.00015751522732898593, 4.599235126079293e-08, 0.0001184645589091815, 0.9999734163284302, 0.9999985694885254, 1.7076034055207856e-05, 0.00017585571913514286, 0.9887650012969971, 1.038160988287018e-07, 8.222870383178815e-05, 0.9999933242797852, 1.153973130385566e-06, 0.9999992847442627, 0.9999995231628418, 0.9999995231628418, 0.9999908208847046, 0.9999291896820068, 0.9999456405639648, 0.9999891519546509, 0.000880147737916559, 0.9996210336685181, 0.0013525948161259294, 8.052757766563445e-05, 0.998573899269104, 0.0004077297344338149, 1.2168060493422672e-05, 0.9994356036186218, 0.9999984502792358, 0.9999982118606567, 5.227294423093554e-06, 0.999998927116394, 7.401593757094815e-05, 0.00024072593078017235, 0.9999608993530273, 0.00010386145004304126, 0.9999669790267944, 1.1120240515083424e-06, 6.880419823573902e-05, 1.1912613473441525e-07, 1.5195219305041974e-08, 0.9996256828308105, 1.5705267287557945e-05, 0.9999995231628418, 0.0001663106813794002, 0.0005816940101794899, 0.9995483756065369, 0.9999996423721313, 0.0009210237767547369, 0.002644500695168972, 0.9999884366989136, 0.0005746687529608607, 0.9999994039535522, 6.252960815800179e-07, 0.9999958276748657, 0.9999405145645142, 0.999894380569458, 0.9999816417694092, 9.302130638388917e-05, 0.9999974966049194, 9.778230378287844e-06, 0.0032195940148085356, 0.9999926090240479, 0.00010046076931757852, 4.799239832209423e-05, 0.999983549118042, 2.6541711122263223e-06, 0.9999998807907104, 0.004335707053542137, 5.948073180661595e-07, 0.5110574960708618, 0.9999426603317261, 1.974107988189644e-07, 0.0001641272392589599, 0.9999974966049194, 0.004343664273619652, 0.9999997615814209, 0.9999971389770508, 1.1249567677396044e-07, 0.9999997615814209, 1.930569669639226e-05, 0.9999911785125732, 8.574295497965068e-05, 6.7765546418741e-07, 0.9999997615814209, 4.689107299782336e-06, 0.00044933363096788526, 8.683749911142513e-05, 0.001124534523114562, 7.031010085256639e-08, 0.0017990856431424618]

#Metricas de validación
Best_acc5_6 = 1.0
acc_aux5_6 = 1.0
Recall5_6 = 1.0
Especificidad5_6 = 1.0
Tiempo5_6 = 1545.2980654239655

######## Base6 ########
#MODELO = VGG16_BN
#Datos perdidas
xx_train6_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train6_6 = [0.22381642554645184, 0.08963192383080353, 0.08193219253494416, 0.0707828942824293, 0.06621615279603887, 0.062412969354126185, 0.05479052957193351, 0.04387014311404876, 0.04853438972323029, 0.050486737130968656, 0.051639814932405215, 0.05120466547431769, 0.04509344851529157, 0.03995472216679726, 0.044530887966170725, 0.049564406018198275, 0.04908528905591847, 0.039913016888830394, 0.04609240752495365, 0.04604798242633725, 0.04468295079322509, 0.046215438769187454, 0.048654336325916245, 0.04654431971318928, 0.04889564271326418]
xx_val6_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val6_6 = [0.06593709290027619, 0.05677471541696125, 0.044773079951604206, 0.07255103968911701, 0.05517884029282464, 0.03595330400599374, 0.033146173755327864, 0.028800049506955677, 0.028635215759277344, 0.024405122962262896, 0.028290578722953798, 0.023903713954819574, 0.023851956095960407, 0.02354199629690912, 0.025351780984136794, 0.02497580846150716, 0.023160111076302, 0.024247767362329694, 0.024904262357287935, 0.02635319232940674, 0.022831670774353875, 0.02577007512251536, 0.023703693515724607, 0.026090835365984173, 0.027031094829241435]
epoch_min6_6 = 9
stop_val6_6 = 0.024405122962262896

#Datos accuracy
acc_train6_6 = [0.9070987654320988, 0.9679012345679012, 0.9712962962962963, 0.9731481481481481, 0.9756172839506173, 0.9787037037037036, 0.9796296296296296, 0.9851851851851852, 0.983641975308642, 0.9814814814814815, 0.9796296296296296, 0.9833333333333333, 0.9824074074074074, 0.9848765432098765, 0.9827160493827161, 0.9824074074074074, 0.9814814814814815, 0.9848765432098765, 0.9827160493827161, 0.9833333333333333, 0.9820987654320987, 0.9814814814814815, 0.9811728395061728, 0.9845679012345678, 0.9824074074074074]
acc_val6_6 = [0.9694444444444444, 0.9777777777777779, 0.9805555555555556, 0.9694444444444444, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9916666666666667, 0.9861111111111112, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9888888888888889, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9861111111111112, 0.9916666666666667, 0.9888888888888889, 0.9888888888888889, 0.9861111111111112, 0.9916666666666667]

#Datos curva ROC
best_labels6_6 = [0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]
best_prob6_6 = [0.0007563276449218392, 0.00030785371200181544, 3.001445747941034e-06, 0.9999923706054688, 0.9999954700469971, 0.9999996423721313, 0.9999996423721313, 0.9999998807907104, 0.15217874944210052, 0.9999134540557861, 0.9997038245201111, 0.0013771840604022145, 0.9997877478599548, 3.0020387953300087e-07, 0.9996411800384521, 0.9999892711639404, 0.9999996423721313, 0.03476839140057564, 6.207416299730539e-05, 0.9999960660934448, 0.03429495170712471, 0.00013382032921072096, 0.999619722366333, 0.9999760389328003, 0.9999951124191284, 3.0794009944656864e-05, 0.005634890403598547, 0.0027212039567530155, 0.0001594552886672318, 0.999997615814209, 0.022195449098944664, 0.9999833106994629, 0.9999669790267944, 9.563688763591927e-06, 0.9999767541885376, 8.869136217981577e-05, 0.9998144507408142, 0.9955611228942871, 0.9898635745048523, 0.9999977350234985, 5.915370593356784e-07, 0.9999676942825317, 0.0031382576562464237, 0.9820980429649353, 0.9988229870796204, 1.717169084258785e-06, 2.853255864465609e-06, 0.9999971389770508, 1.5806373994564638e-05, 0.9997087121009827, 5.511034760274924e-05, 0.0015666590770706534, 0.0012936266139149666, 3.0797211366007105e-05, 0.9999910593032837, 0.0007718325359746814, 8.04834344307892e-05, 9.743282862473279e-05, 0.9999427795410156, 0.00023887105635367334, 0.00041939070797525346, 0.23430213332176208, 0.9999961853027344, 0.9999951124191284, 3.069654485443607e-05, 1.324435379501665e-05, 0.0002800608635880053, 0.0007931866566650569, 0.9726547002792358, 0.0016726481262594461, 7.26132429917925e-06, 1.304134821111802e-05, 0.9999616146087646, 0.9999974966049194, 0.9999105930328369, 0.00063577841501683, 0.003549272892996669, 0.999966025352478, 1.2402763786667492e-05, 0.00015706110571045429, 0.000108455860754475, 0.00011350226122885942, 0.0003877469862345606, 0.9999469518661499, 0.0002460413670632988, 0.0009901653975248337, 4.573032867938309e-07, 0.9992523789405823, 0.999998927116394, 0.9999929666519165, 0.001994509482756257, 0.00040742839337326586, 0.9999908208847046, 0.0008729117689654231, 0.9999911785125732, 0.9999510049819946, 0.9998143315315247, 0.9991583824157715, 3.828182889265008e-05, 0.9999539852142334, 6.761120312148705e-05, 7.33770320948679e-06, 0.9998531341552734, 0.9999966621398926, 0.001608299440704286, 7.528793958044844e-06, 0.9999997615814209, 0.9999852180480957, 0.07260221242904663, 0.0004623163549695164, 0.9995161294937134, 0.9999958276748657, 0.9999589920043945, 0.9996381998062134, 0.9992097616195679, 0.9996211528778076, 1.0, 0.001900677802041173, 6.06762187089771e-05, 0.9999969005584717, 0.831007719039917, 0.9999979734420776, 3.821008431259543e-05, 0.9999864101409912, 3.8334765122272074e-05, 1.0245118573948275e-05, 0.9999755620956421, 0.9382771849632263, 0.0006820872658863664, 0.999993085861206, 0.999457061290741, 0.9992355108261108, 0.996635377407074, 0.0006630609859712422, 0.00015745397831778973, 0.9999960660934448, 5.126590622239746e-06, 0.0021140531171113253, 0.9999998807907104, 0.0002725043159443885, 0.00027252850122749805, 0.9999744892120361, 9.785629117686767e-06, 0.17995326220989227, 1.5739724403829314e-05, 0.9999949932098389, 6.307706826191861e-06, 1.020187823996821e-06, 0.9996229410171509, 0.9993070363998413, 0.9999982118606567, 6.633702923863893e-06, 0.9999908208847046, 0.9986832737922668, 0.0019054037984460592, 0.9999456405639648, 0.9998822212219238, 9.01598104974255e-06, 6.966919130491078e-08, 7.995484338607639e-06, 0.00028777611441910267, 0.9999908208847046, 0.00026444628019817173, 0.9993365406990051, 0.9998743534088135, 0.0013447609962895513, 0.9999980926513672, 0.7352471947669983, 0.0014880259986966848, 0.0011023139813914895, 0.00023506501747760922, 3.100141111644916e-05, 0.0006288516451604664, 0.0026339502073824406, 0.9997214674949646, 0.9999572038650513, 7.5875091170019004e-06, 4.494448148761876e-06, 1.5967024182828027e-06, 1.57126833073562e-05, 9.154828148894012e-05, 0.9999653100967407, 0.005229399539530277, 0.00043860002188012004, 0.9999995231628418, 0.9999319314956665, 0.9999955892562866, 0.0002852605248335749, 7.583284605061635e-05, 0.6948287487030029, 2.6965237339027226e-05, 3.7564157537417486e-05, 0.9998970031738281, 0.9999631643295288, 0.9999822378158569, 1.3112967280903831e-06, 0.9999986886978149, 0.9999362230300903, 1.688649930997599e-08, 0.9999434947967529, 0.00012758400407619774, 0.00048497042735107243, 0.9999876022338867, 0.019996510818600655, 4.739794349006843e-06, 0.9998810291290283, 0.9999881982803345, 0.018082721158862114, 4.188921991499228e-07, 0.9998718500137329, 0.9998233914375305, 1.3770743656493778e-08, 3.4448833048372762e-06, 1.225477490152116e-06, 2.391023372183554e-05, 0.007869209162890911, 4.13533342680239e-07, 0.00014204721082933247, 0.9999915361404419, 0.9999459981918335, 0.4596801996231079, 4.048832124681212e-06, 5.758839961345075e-06, 0.9999473094940186, 0.999931812286377, 0.9999706745147705, 0.0005154958344064653, 0.00010444194049341604, 0.99801105260849, 4.397487828100566e-06, 1.955875995918177e-05, 0.9999722242355347, 0.9999958276748657, 0.9999986886978149, 1.6054117395469802e-06, 0.9997501969337463, 0.9999362230300903, 0.9999759197235107, 0.9993898868560791, 0.9997686743736267, 0.0006779271061532199, 0.00021892898075748235, 0.0023123249411582947, 3.8000064250809373e-07, 1.0550902516115457e-05, 0.0010354191763326526, 0.9999700784683228, 1.3341993508220185e-05, 6.538353045471013e-05, 0.9999995231628418, 0.999951958656311, 0.9703420400619507, 0.9995182752609253, 1.0, 0.9965139031410217, 0.006316010840237141, 1.4765612831979524e-05, 0.787481963634491, 0.999998927116394, 0.00011891694157384336, 0.0001139786108979024, 0.024644462391734123, 0.9999724626541138, 9.231795047526248e-06, 2.9574770451290533e-05, 3.275410733749595e-07, 0.9990605711936951, 0.0003969574172515422, 0.9999898672103882, 0.999982476234436, 0.9996892213821411, 0.9999901056289673, 0.9999829530715942, 0.9999921321868896, 0.9999911785125732, 0.9998772144317627, 0.9999988079071045, 0.00013984981342218816, 0.9712140560150146, 0.4136510491371155, 0.00017254210251849145, 1.0, 2.9859897040296346e-05, 0.9999932050704956, 0.9972656965255737, 0.00014664395712316036, 0.0005775134777650237, 0.9999943971633911, 0.9999916553497314, 0.9999990463256836, 0.9999998807907104, 0.9998691082000732, 0.9999383687973022, 0.999992847442627, 0.9999972581863403, 0.00013823759218212217, 2.9391355838015443e-07, 0.9999997615814209, 0.9997523427009583, 0.0018254659371450543, 7.269001071108505e-05, 0.9726547002792358, 0.9999910593032837, 0.9999921321868896, 0.999966025352478, 0.9999986886978149, 2.7930357191507937e-06, 0.0029972041957080364, 1.566792889207136e-05, 0.9999998807907104, 0.9995527863502502, 0.00016284397861454636, 0.001847060164436698, 0.00012196414172649384, 0.9993948936462402, 0.8153023719787598, 0.00019105292449239641, 0.0008535567321814597, 0.9999933242797852, 0.9999880790710449, 0.016632774844765663, 0.9999953508377075, 1.947545024449937e-05, 0.9985707998275757, 0.9990193843841553, 0.9994460940361023, 0.9999129772186279, 5.190853698877618e-05, 0.00042397750075906515, 0.007695290260016918, 2.040425442828564e-06, 0.002126646228134632, 0.999930739402771, 0.9999905824661255, 0.9999275207519531, 0.9999977350234985, 0.0011012437753379345, 0.9992648959159851, 0.13550467789173126, 0.9999922513961792, 0.9999241828918457, 0.0009665585239417851, 0.9909351468086243, 0.00011717068264260888, 0.00451893825083971, 0.9999431371688843, 0.0020440698135644197, 0.003934983164072037, 4.858020020037657e-06, 0.9999990463256836, 0.9986610412597656, 0.9999914169311523, 2.3659235012019053e-05, 0.014905186370015144, 0.9999639987945557, 0.00016632559709250927, 0.9996789693832397, 0.0008909013122320175, 0.999988317489624, 0.00011464834096841514]

#Metricas de validación
Best_acc6_6 = 0.9916666666666667
acc_aux6_6 = 0.9916666666666667
Recall6_6 = 0.988950276243094
Especificidad6_6 = 0.994413407821229
Tiempo6_6 = 1653

######## Base7 ########
#MODELO = VGG16_BN
#Datos perdidas
xx_train7_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train7_6 = [0.08688358123655672, 0.05604275400623863, 0.046895490687938385, 0.049337131787597396, 0.04338603319577229, 0.058442615266935324, 0.03512272213896116, 0.03218671010784161, 0.03120369576377633, 0.034474682127252035, 0.032473151128233214, 0.03272840766075217, 0.031727696430903896, 0.03944551189556534, 0.03599606897727943, 0.03377923310538869, 0.02979414055560842, 0.04628720096783873, 0.03364954242183838, 0.03659777210818397, 0.024261627944163335, 0.03367447190814548, 0.037041937127525425, 0.0274211135359458, 0.031597089804249044]
xx_val7_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val7_6 = [0.03461179021331999, 0.024641555547714234, 0.059756244719028476, 0.01865708629290263, 0.038940962155659994, 0.03612021638287438, 0.03582900298966302, 0.03224154810110728, 0.028336958090464274, 0.027884161637889014, 0.02702118025885688, 0.02802652915318807, 0.028511136439111497, 0.02812003559536404, 0.027968539463149176, 0.026423330770598517, 0.02967220726940367, 0.028545169863435957, 0.02705688923597336, 0.027509405546718175, 0.027339735958311292, 0.02635152323378457, 0.028570518228742812, 0.030043048328823513, 0.02725639823410246]
epoch_min7_6 = 3
stop_val7_6 = 0.01865708629290263

#Datos accuracy
acc_train7_6 = [0.9709876543209877, 0.9774691358024691, 0.9824074074074074, 0.9820987654320987, 0.9851851851851852, 0.9762345679012345, 0.9882716049382716, 0.9873456790123457, 0.9895061728395061, 0.9873456790123457, 0.9885802469135803, 0.9891975308641975, 0.9898148148148148, 0.987037037037037, 0.9867283950617284, 0.9873456790123457, 0.9885802469135803, 0.983641975308642, 0.9882716049382716, 0.9879629629629629, 0.9922839506172839, 0.9879629629629629, 0.9858024691358025, 0.9901234567901235, 0.9888888888888888]
acc_val7_6 = [0.9833333333333334, 0.9861111111111112, 0.9750000000000001, 0.9916666666666667, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9861111111111112, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9861111111111112, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9861111111111112]

#Datos curva ROC
best_labels7_6 = [0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0]
best_prob7_6 = [1.2762121514242608e-05, 0.9992425441741943, 0.9969808459281921, 0.0009162945789285004, 0.9997102618217468, 0.9999998807907104, 0.9996614456176758, 6.736309643429195e-08, 0.9889132380485535, 0.00013116243644617498, 0.8949677348136902, 0.002699988428503275, 0.9976522326469421, 0.0005567847401835024, 8.536701352568343e-05, 0.9999995231628418, 0.007102404721081257, 0.9999903440475464, 9.001784928841516e-05, 0.00020383548690006137, 0.9999994039535522, 5.0792135880328715e-05, 0.9999988079071045, 0.9999977350234985, 6.477566057583317e-05, 0.9999690055847168, 0.9996978044509888, 5.34675393737416e-07, 0.9990917444229126, 0.999975323677063, 0.9999997615814209, 0.00036790716694667935, 1.5285573681467213e-05, 0.9997674822807312, 0.9999849796295166, 5.211525149206864e-06, 0.9999487400054932, 4.157142484473297e-06, 0.007130473852157593, 0.00022080309281591326, 0.9995457530021667, 8.708785753697157e-05, 0.9999662637710571, 0.990154504776001, 0.9997836947441101, 0.9999935626983643, 0.999690055847168, 0.9999366998672485, 1.008261097013019e-05, 0.999977707862854, 0.9995704293251038, 3.3932343512788066e-07, 2.5531804226375243e-07, 0.00015178779722191393, 0.9998738765716553, 0.002657360164448619, 0.0008995951502583921, 0.9999703168869019, 0.9999967813491821, 0.9999879598617554, 4.26955557486508e-06, 0.9937130808830261, 0.9998972415924072, 0.9995419979095459, 0.9934840798377991, 1.2270947991055436e-05, 0.9999998807907104, 1.8049566278932616e-05, 0.002072305418550968, 0.9999972581863403, 0.0007945558172650635, 7.536756311310455e-05, 3.564646249287762e-05, 0.9999436140060425, 0.00010529955761739984, 0.0008645780617371202, 2.3963239073054865e-05, 0.9999988079071045, 0.9998542070388794, 4.712220834335312e-05, 0.9999972581863403, 0.9999991655349731, 0.99953293800354, 0.9949670433998108, 2.0703011614386924e-05, 5.369761311158072e-06, 0.002510449616238475, 0.9993053674697876, 0.0003397009859327227, 0.00018786959117278457, 0.9999983310699463, 2.8216147711646045e-06, 3.6950193589291302e-06, 0.9999923706054688, 0.00037499750033020973, 0.00027169520035386086, 0.9983100891113281, 0.8147169351577759, 0.9999574422836304, 0.9999990463256836, 0.04712182283401489, 9.473295904172119e-06, 0.00033589566010050476, 0.999910831451416, 0.00010937937622657046, 2.2572594389203005e-05, 0.0004630875773727894, 6.85613340465352e-05, 0.9998451471328735, 0.9999899864196777, 0.9999891519546509, 0.99997878074646, 0.9999430179595947, 0.002116844290867448, 0.9999579191207886, 7.422628982567403e-07, 3.166156602674164e-05, 0.0009862093720585108, 1.0, 0.000287808506982401, 5.380158540901903e-07, 0.997154951095581, 0.9999982118606567, 2.7107924324809574e-05, 0.9998075366020203, 0.9999998807907104, 2.9936394639662467e-05, 0.9999618530273438, 0.00030533087556250393, 0.9999113082885742, 0.0004641316190827638, 0.0011653478723019361, 0.00013465131632983685, 0.0007910123676992953, 5.518940042748e-07, 0.9999821186065674, 7.676125619582308e-07, 0.9985528588294983, 0.9998199343681335, 0.9957934617996216, 0.9999997615814209, 0.9997591376304626, 0.4710499942302704, 0.99950110912323, 0.9976340532302856, 0.9999969005584717, 1.051735944201937e-05, 0.0001351669052382931, 0.9995717406272888, 0.9999271631240845, 0.9998772144317627, 0.9999908208847046, 0.0001854933361755684, 0.9999959468841553, 0.0041535901837050915, 0.9999997615814209, 2.244631218673021e-07, 0.0019441822078078985, 0.9999983310699463, 9.017232514452189e-05, 4.807920390703657e-07, 5.4291180276777595e-05, 0.9999790191650391, 0.00034862422035075724, 3.872335219057277e-05, 6.267194112297148e-05, 1.776159115252085e-05, 0.9999911785125732, 0.0023871613666415215, 0.0008192745153792202, 0.9999775886535645, 0.9999526739120483, 4.338708458817564e-05, 0.0007109967991709709, 2.318175211257767e-05, 0.9999986886978149, 0.0002479904505889863, 0.9992885589599609, 0.00047957804054021835, 0.9992460012435913, 0.0002685460785869509, 0.9999744892120361, 1.2222724308230681e-06, 0.00022611484746448696, 0.9999980926513672, 0.9999879598617554, 0.9999998807907104, 0.9999997615814209, 1.0, 0.9998682737350464, 0.9999990463256836, 0.999985933303833, 0.9996603727340698, 0.001408861600793898, 0.00011403024109313264, 0.9998955726623535, 0.9999970197677612, 0.9988540410995483, 1.061110879163607e-06, 0.0012405689340084791, 0.0005670748651027679, 4.014997830381617e-05, 0.9999911785125732, 0.99470055103302, 1.0, 0.9999916553497314, 0.9996247291564941, 0.9999983310699463, 0.9999978542327881, 0.9999872446060181, 2.0284842321416363e-05, 0.00013220754044596106, 0.9996911287307739, 0.00010796324204420671, 0.0015309505397453904, 0.0002025789872277528, 2.8268023015698418e-05, 0.00021513189130928367, 0.000469271675683558, 0.9998778104782104, 0.9998406171798706, 0.9999933242797852, 0.9994163513183594, 0.000146139194839634, 0.9973032474517822, 0.9999135732650757, 0.00013589269656222314, 0.00039390436722896993, 0.9999984502792358, 0.9999998807907104, 0.9998636245727539, 0.9996436834335327, 0.2558249533176422, 0.9994935989379883, 0.9363927245140076, 3.850733628496528e-05, 1.4172698683978524e-05, 0.9999608993530273, 0.030776236206293106, 0.9999945163726807, 0.9999886751174927, 0.0002847906725946814, 3.639646456576884e-05, 4.3857694436155725e-06, 0.000368353386875242, 0.9999724626541138, 0.9999998807907104, 0.9999938011169434, 0.9995243549346924, 3.1369170756079257e-05, 0.0018022344447672367, 0.9999109506607056, 2.851982571883127e-06, 0.009831647388637066, 5.391187369241379e-05, 0.988416850566864, 0.00014072719204705209, 4.944442480336875e-05, 0.9999256134033203, 0.0002462000120431185, 0.997012734413147, 0.00023418493219651282, 3.677662789414171e-06, 0.00014866699348203838, 0.9996523857116699, 0.00015204127703327686, 0.9999994039535522, 0.9996308088302612, 0.9999793767929077, 5.608101128018461e-05, 1.982177491299808e-05, 0.00015510433877352625, 0.9998553991317749, 0.0006920184823684394, 0.987006664276123, 3.864728114422178e-06, 0.9999933242797852, 0.9999464750289917, 0.00012119919847464189, 2.0963652787031606e-05, 0.9999959468841553, 1.4865638320316066e-07, 0.005359138827770948, 0.9993355870246887, 0.9998891353607178, 0.0027330403681844473, 0.9999994039535522, 0.9997673630714417, 0.997962236404419, 0.37734681367874146, 1.7886881323647685e-05, 0.0004431293054949492, 5.086838064016774e-05, 0.9991942048072815, 0.00031657845829613507, 0.9999932050704956, 7.692065992159769e-05, 0.00031625080737285316, 3.5695102269528434e-05, 0.9998276233673096, 9.965439858206082e-06, 0.999985933303833, 0.0006698804791085422, 1.5405870726681314e-05, 0.9999972581863403, 2.1443514924612828e-05, 0.9993476271629333, 1.454955418012105e-05, 2.8374927296681562e-06, 0.001230683526955545, 0.9999991655349731, 1.4429475413635373e-05, 0.0002334581658942625, 0.001071738894097507, 0.17129454016685486, 0.9999843835830688, 0.9999656677246094, 1.979036460397765e-05, 0.00029981147963553667, 2.0341909475973807e-05, 0.00045948175829835236, 0.002446959726512432, 0.9998083710670471, 0.9995349645614624, 0.008860914036631584, 0.2724996507167816, 0.0003467781934887171, 0.999691367149353, 0.011440655216574669, 0.999729573726654, 0.999523401260376, 0.9998288154602051, 0.9998687505722046, 0.9999996423721313, 1.047271962306695e-05, 1.0, 8.30562552067704e-09, 0.0008034976199269295, 0.9993785619735718, 1.6808093278086744e-05, 0.0002541886060498655, 0.9999929666519165, 0.9999655485153198, 0.650068998336792, 0.9996869564056396, 0.9995989203453064, 0.00012732099276036024, 2.186169012929895e-06, 0.9929627776145935, 0.9999964237213135, 5.8416026149643585e-05, 0.0004766126221511513, 3.772653599298792e-06, 0.00013583256804849952, 0.9999998807907104, 0.999747097492218, 0.9999250173568726, 0.999993085861206, 4.226655073580332e-05, 0.9997970461845398]

#Metricas de validación
Best_acc7_6 = 0.9916666666666667
acc_aux7_6 = 0.9922839506172839
Recall7_6 = 0.988950276243094
Especificidad7_6 = 0.994413407821229
Tiempo7_6 = 1652

######## Base8 ########
#MODELO = VGG16_BN
#Datos perdidas
xx_train8_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train8_6 = [0.06623616584657151, 0.03615440325236615, 0.05244534714722339, 0.04093369550910997, 0.03866416950836594, 0.03114901703816873, 0.03384927364043248, 0.039057043875441136, 0.02783463532909935, 0.030986256446735357, 0.02662358906717948, 0.03137686407124555, 0.023673335554791085, 0.028236471521265714, 0.023914917300880693, 0.029436516642202565, 0.028817207338633362, 0.022974542436408407, 0.029103995427305317, 0.028312741615522056, 0.030139554724281217, 0.024639482934165884, 0.029085148457023834, 0.028100117755892835, 0.03396533741443245]
xx_val8_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val8_6 = [0.004522798955440522, 0.007152068614959717, 0.005122701989279853, 0.006359403828779856, 0.006257017122374641, 0.004962451590432061, 0.007109613882170783, 0.00473172300391727, 0.0049482911825180055, 0.006007338232464261, 0.0077482604318194915, 0.00584704793161816, 0.006934303376409743, 0.007067652377817366, 0.008136930730607774, 0.00488108860121833, 0.007085196011596255, 0.006211179825994704, 0.006021168828010559, 0.007966385947333442, 0.006858883466985491, 0.008075626442829768, 0.006434282494915856, 0.004531661338276333, 0.004084883133570353]
epoch_min8_6 = 0
stop_val8_6 = 0.004522798955440522


#Datos accuracy
acc_train8_6 = [0.9768518518518519, 0.9876543209876543, 0.979320987654321, 0.9851851851851852, 0.9885802469135803, 0.9879629629629629, 0.9885802469135803, 0.987037037037037, 0.9879629629629629, 0.9895061728395061, 0.990432098765432, 0.9891975308641975, 0.9929012345679012, 0.9891975308641975, 0.9891975308641975, 0.9888888888888888, 0.9888888888888888, 0.9935185185185185, 0.9888888888888888, 0.990432098765432, 0.9907407407407407, 0.9907407407407407, 0.9901234567901235, 0.991358024691358, 0.9885802469135803]
acc_val8_6 = [1.0, 0.9972222222222222, 0.9972222222222222, 1.0, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222]

#Datos curva ROC
best_labels8_6 = [1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
best_prob8_6 = [0.9995530247688293, 8.577729022363201e-05, 6.847966142231598e-05, 0.9920389652252197, 1.0, 0.00034270616015419364, 0.00018249166896566749, 0.00041234149830415845, 0.059500955045223236, 0.9996129870414734, 0.00020766972738783807, 0.0050974260084331036, 0.9999867677688599, 0.9999624490737915, 7.631308835698292e-05, 0.9998748302459717, 0.9846994280815125, 8.680522296344861e-05, 0.0006381620769388974, 6.759998359484598e-05, 4.864383299718611e-05, 0.9994825124740601, 0.9984474778175354, 0.0012224569218233228, 0.0017496736254543066, 0.9990574717521667, 0.999991774559021, 0.00013836368452757597, 0.9999176263809204, 0.0037547715473920107, 5.2187726396368816e-05, 0.0005079349502921104, 0.018232565373182297, 0.9933145642280579, 0.9299445152282715, 0.0003294088819529861, 5.379970752983354e-05, 0.984838604927063, 0.0003331777115818113, 0.9998586177825928, 0.9998394250869751, 0.9998657703399658, 0.999779999256134, 0.9999874830245972, 0.9998427629470825, 0.002849358133971691, 8.090226037893444e-05, 0.0014538330724462867, 0.9999157190322876, 3.0486467949231155e-05, 0.001321209012530744, 0.00037263502599671483, 1.60827021318255e-05, 0.999937891960144, 0.024181503802537918, 0.9999412298202515, 0.9999920129776001, 0.9998897314071655, 0.9999182224273682, 0.9999829530715942, 0.9999699592590332, 0.0005713941063731909, 0.0001279481512028724, 4.8874091589823365e-05, 0.0012783458223566413, 0.9999850988388062, 0.0004446869425009936, 0.9999439716339111, 0.00020692931138910353, 0.9985186457633972, 0.0057855513878166676, 0.002386630279943347, 0.9999774694442749, 1.633750230212172e-06, 6.357394886435941e-05, 4.277446714695543e-05, 0.0030432238709181547, 0.9999885559082031, 0.0034687104634940624, 0.9999995231628418, 4.082977102370933e-05, 0.9999936819076538, 0.007285951636731625, 0.9998249411582947, 0.9945189356803894, 0.9992570281028748, 0.0003753354831133038, 0.9998239874839783, 0.0024886627215892076, 0.000160316500114277, 0.99989914894104, 0.0014745380030944943, 0.9999713897705078, 0.973647952079773, 0.005304206162691116, 0.9999492168426514, 0.9998179078102112, 0.0017953900387510657, 0.00035319742164574564, 0.999977707862854, 0.9999995231628418, 0.006537146400660276, 0.9999973773956299, 0.0011554625816643238, 0.00038920907536521554, 0.999886155128479, 9.923882316797972e-06, 4.9826401664176956e-05, 0.9999706745147705, 0.9993394017219543, 0.9998185038566589, 0.9996022582054138, 0.000314009579597041, 0.9990445971488953, 0.9998050332069397, 2.8061931516276672e-05, 8.025819261092693e-05, 0.0005725258379243314, 0.9998821020126343, 0.00024416204541921616, 0.9999274015426636, 0.9999794960021973, 0.0007728806231170893, 0.9998666048049927, 0.999805748462677, 0.9999878406524658, 0.9999862909317017, 3.230101356166415e-05, 0.00041247589979320765, 0.9996386766433716, 0.9936282634735107, 0.9999784231185913, 0.999951958656311, 0.9997982382774353, 0.002098673954606056, 0.9999966621398926, 0.9999276399612427, 0.00010486627434147522, 0.9999505281448364, 0.0017152194632217288, 0.00014158581325318664, 0.9998515844345093, 0.9998530149459839, 0.9681673049926758, 0.99950110912323, 0.000245057133724913, 0.000565079040825367, 0.0001775572745827958, 0.9999847412109375, 0.9992320537567139, 0.003369165351614356, 0.9995529055595398, 0.0009420076967217028, 0.002182838972657919, 0.9999517202377319, 0.9985260367393494, 0.9999861717224121, 0.1576801836490631, 0.9999135732650757, 0.9998449087142944, 0.9998421669006348, 0.00010485816892469302, 0.9999561309814453, 0.9999436140060425, 0.009045423939824104, 0.9999051094055176, 0.9998565912246704, 0.9999679327011108, 0.0013787130592390895, 0.9997498393058777, 0.9998425245285034, 0.9985533356666565, 0.9992285966873169, 0.9999877214431763, 0.00010037689207820222, 2.09335885301698e-05, 0.9998412132263184, 6.54653376841452e-06, 0.00042667757952585816, 0.016451630741357803, 0.9988162517547607, 0.00018766906578093767, 0.9998849630355835, 0.9998992681503296, 0.011176355183124542, 0.9995813965797424, 3.170265699736774e-05, 0.9999682903289795, 0.9998571872711182, 0.9999852180480957, 0.0017943937564268708, 0.00011338909826008603, 0.0009627407416701317, 0.003851142479106784, 0.9947789907455444, 0.005994962062686682, 0.999502420425415, 0.45116928219795227, 0.0005238603334873915, 0.0005085240700282156, 0.0005678878515027463, 0.0003977737214881927, 0.9998843669891357, 0.00023509658058173954, 0.999977707862854, 0.9995551705360413, 0.9712897539138794, 0.0005639292648993433, 0.978805661201477, 0.99996018409729, 0.0001525498228147626, 0.0006488185608759522, 5.9223788412055e-05, 8.920827531255782e-05, 5.1378821808611974e-05, 0.9998791217803955, 0.9999992847442627, 0.00028102242504246533, 3.935387940146029e-05, 0.999872088432312, 0.0001512812013970688, 0.930757999420166, 0.000293979566777125, 0.9973801970481873, 0.9999756813049316, 0.9999642372131348, 0.00044093444012105465, 0.0005756606115028262, 0.9999668598175049, 0.00025188561994582415, 0.000394320348277688, 0.0013776622945442796, 0.9994072914123535, 7.80519112595357e-05, 0.9999958276748657, 0.9999523162841797, 0.00015874954988248646, 2.8735628802678548e-05, 0.9999978542327881, 0.0005293223075568676, 0.0003298994561191648, 0.9999860525131226, 0.00027616790612228215, 3.3578151487745345e-05, 4.351612733444199e-05, 3.068411024287343e-05, 0.0017740264302119613, 0.0007310956716537476, 0.0002638730511534959, 0.9997509121894836, 7.63108255341649e-05, 0.9999412298202515, 0.9999935626983643, 0.9993991851806641, 0.9999969005584717, 0.007877303287386894, 0.0031944424845278263, 0.9988162517547607, 0.0007237458485178649, 3.550843393895775e-05, 4.524931136984378e-05, 3.115463914582506e-05, 0.9997912049293518, 0.00010363518231315538, 7.291994552360848e-05, 0.9423356652259827, 0.9983007311820984, 0.00014238106086850166, 0.9998254179954529, 0.9999130964279175, 0.9998891353607178, 6.020222281222232e-05, 4.761205673275981e-06, 0.999929666519165, 0.00017928917077369988, 0.9999758005142212, 0.9998421669006348, 0.0042165848426520824, 0.00034411242813803256, 1.0309404387953691e-06, 0.9999258518218994, 0.9999077320098877, 0.0007844996871426702, 3.809547706623562e-05, 0.0014198336284607649, 0.9999241828918457, 0.9999895095825195, 0.9996490478515625, 0.9999946355819702, 0.9999852180480957, 0.00016247484018094838, 0.9998656511306763, 0.9946356415748596, 0.00040054848068393767, 0.0002742493525147438, 0.0010086758993566036, 0.00016534299356862903, 0.9999794960021973, 0.9982726573944092, 7.559446544291859e-07, 4.4033004087395966e-05, 0.9997350573539734, 0.9857187867164612, 0.00011317371536279097, 3.8654583477182314e-05, 5.789925126009621e-05, 0.999984860420227, 0.0004965605330653489, 0.0004910614807158709, 0.00011343257210683078, 0.9996993541717529, 0.0010080246720463037, 0.9952282905578613, 0.0034175582695752382, 0.0006987668457441032, 0.999937891960144, 0.9999589920043945, 0.9984462857246399, 2.747902544797398e-05, 0.015538577921688557, 1.0405303328298032e-05, 0.9996881484985352, 0.0012168764369562268, 0.9995649456977844, 0.9897851347923279, 0.00040072755655273795, 0.9980636239051819, 1.0255147572024725e-05, 0.9984074234962463, 0.9995959401130676, 0.9998282194137573, 0.005827748216688633, 0.0005274318973533809, 0.00020585012680385262, 0.9985829591751099, 0.999993085861206, 0.9684746861457825, 0.9999517202377319, 0.0005726764211431146, 0.9997677206993103, 0.0001794271811377257, 0.9999133348464966, 0.9999333620071411, 0.999828577041626, 0.9996194839477539, 0.9996769428253174, 0.9999997615814209, 0.9999208450317383, 0.0005787955014966428, 0.9997984766960144, 0.0006944239721633494, 0.9999978542327881, 0.0023765256628394127, 0.001199187245219946, 0.9997153878211975, 0.0007064362289384007, 2.253140883112792e-05, 2.134522674168693e-06, 0.9429470896720886, 0.0046158842742443085]

#Metricas de validación
Best_acc8_6 = 1.0
acc_aux8_6 = 1.0
Recall8_6 = 1.0
Especificidad8_6 = 1.0
Tiempo8_6 = 1645

######## Base9 ########
#MODELO = VGG16_BN
#Datos perdidas
xx_train9_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train9_6 = [0.2105948100855321, 0.0891910504779698, 0.0870594608562964, 0.055752134083965676, 0.06179369978698683, 0.04749220544357359, 0.049905179900887574, 0.03460176250448933, 0.03493466500514819, 0.041341339355265655, 0.04468887035309532, 0.036979668688626934, 0.03823491138992486, 0.039124653920715236, 0.04219242494415354, 0.04570755866574652, 0.04147239958430514, 0.03886561754309101, 0.04203362799721, 0.049250841407496256, 0.05207019970740801, 0.046734784571108995, 0.04887440379387067, 0.03911021700427856, 0.042906149035250696]
xx_val9_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val9_6 = [0.06275285846657223, 0.06670134315888086, 0.05284188489119212, 0.054629842274718815, 0.03450506279865901, 0.035311112801233925, 0.03444986194372177, 0.0371413787206014, 0.034386937154663934, 0.03906238807572259, 0.03721180276738273, 0.033558100793096755, 0.03587933149602678, 0.03574953567650583, 0.036377776165803274, 0.045485614736874895, 0.0385193235344357, 0.03488429097665681, 0.03603955970870124, 0.04084691148665216, 0.033682452473375535, 0.035663316481643254, 0.034893614302078886, 0.03596977550122473, 0.031689026869005626]
epoch_min9_6 = 4
stop_val9_6 = 0.03450506279865901

#Datos accuracy
acc_train9_6 = [0.9209876543209876, 0.9688271604938271, 0.9691358024691358, 0.9783950617283951, 0.9771604938271605, 0.9805555555555555, 0.9824074074074074, 0.9873456790123457, 0.9879629629629629, 0.986111111111111, 0.9842592592592593, 0.9879629629629629, 0.986111111111111, 0.9845679012345678, 0.9848765432098765, 0.9833333333333333, 0.9833333333333333, 0.987037037037037, 0.9858024691358025, 0.9830246913580246, 0.9811728395061728, 0.9820987654320987, 0.9830246913580246, 0.9867283950617284, 0.9867283950617284]
acc_val9_6 = [0.9777777777777779, 0.9833333333333334, 0.9777777777777779, 0.9833333333333334, 0.9916666666666667, 0.9833333333333334, 0.9916666666666667, 0.9833333333333334, 0.9861111111111112, 0.9888888888888889, 0.9916666666666667, 0.9916666666666667, 0.9861111111111112, 0.9861111111111112, 0.9833333333333334, 0.9805555555555556, 0.9833333333333334, 0.9833333333333334, 0.9888888888888889, 0.9833333333333334, 0.9916666666666667, 0.9861111111111112, 0.9888888888888889, 0.9888888888888889, 0.9916666666666667]

#Datos curva ROC
best_labels9_6 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0]
best_prob9_6 = [6.459195719799027e-05, 0.00024494805256836116, 0.2791435122489929, 0.0042793722823262215, 0.0004818788729608059, 0.00013606654829345644, 0.9985290765762329, 0.9999610185623169, 3.965178166254191e-06, 0.9498260021209717, 0.9998052716255188, 0.9999895095825195, 0.0007017439929768443, 0.9998317956924438, 0.9997424483299255, 0.0003037600836250931, 0.0008499361574649811, 0.9999512434005737, 0.9994219541549683, 0.0007495521567761898, 0.05804308503866196, 0.1447748988866806, 0.008881045505404472, 0.9999806880950928, 0.9999990463256836, 0.9964717626571655, 0.0005807544803246856, 0.9999352693557739, 0.9990531802177429, 0.9998924732208252, 2.8112581276218407e-06, 0.9999991655349731, 0.0006696053897030652, 0.9999852180480957, 0.0003573191352188587, 0.9999935626983643, 0.9999772310256958, 0.9978330731391907, 0.9999048709869385, 0.9996180534362793, 0.022868135944008827, 0.9999949932098389, 0.9997890591621399, 0.9999758005142212, 0.9999891519546509, 0.0011680918978527188, 0.9999376535415649, 0.00010379401646787301, 0.9908939599990845, 0.9817355275154114, 0.06742367893457413, 0.9999974966049194, 0.999937891960144, 0.9999986886978149, 0.0003888422797899693, 0.00737748621031642, 0.0004562428221106529, 0.9993115663528442, 0.00028110764105804265, 0.007735399063676596, 0.0007836220902390778, 0.00542463781312108, 0.0002628668735269457, 0.9998987913131714, 0.6733466982841492, 0.9975355863571167, 0.0015685815596953034, 0.005380853079259396, 0.001458903425373137, 2.5580851797712967e-05, 0.0009070547530427575, 0.9995585083961487, 0.41279828548431396, 0.008894385769963264, 0.99866783618927, 0.9997662901878357, 5.0707250920822844e-05, 2.4574697818025015e-05, 0.00019344875181559473, 9.71298577496782e-05, 0.9975304007530212, 0.0011255000717937946, 0.9995476603507996, 0.999913215637207, 0.00024383887648582458, 0.0012251314474269748, 0.9999911785125732, 2.0481009414652362e-05, 0.9999991655349731, 0.9999932050704956, 1.8005828678724356e-05, 0.9999986886978149, 0.9998804330825806, 2.8011691028950736e-05, 0.9999927282333374, 0.99977046251297, 0.0005001198733225465, 0.13461291790008545, 0.9993708729743958, 0.00016306835459545255, 0.9999998807907104, 0.0005384221440181136, 0.045767150819301605, 6.649503916378308e-07, 0.9993067979812622, 1.3355002920434345e-05, 0.019919751212000847, 0.9999992847442627, 3.202355946996249e-05, 0.0005744720692746341, 0.009080089628696442, 2.9615384846692905e-05, 0.9999929666519165, 0.9999504089355469, 0.00015756812354084104, 0.003131684148684144, 0.9970861077308655, 0.9999828338623047, 0.023353945463895798, 0.0014492624904960394, 0.9999898672103882, 0.9999889135360718, 0.9988821148872375, 0.9999232292175293, 0.9999436140060425, 0.004433422349393368, 0.007455454207956791, 0.6815058588981628, 0.9994199275970459, 0.0026116129010915756, 0.000728941464331001, 0.999995231628418, 0.9999071359634399, 0.9514260292053223, 0.9999299049377441, 0.007534353993833065, 0.9986385703086853, 1.3991035530125373e-07, 0.000260977481957525, 0.0006466949707828462, 7.53305503167212e-05, 0.9999740123748779, 0.00013278325786814094, 0.9998319149017334, 0.9999979734420776, 0.0002735588641371578, 0.0005580372526310384, 0.0016086333198472857, 0.0033465372398495674, 0.0008198930299840868, 0.023144220933318138, 0.999984622001648, 0.9998294115066528, 0.9999361038208008, 1.6159987353603356e-05, 0.22420188784599304, 0.9996198415756226, 0.0016074704471975565, 0.9996161460876465, 0.9902855753898621, 0.004956560209393501, 0.9989976286888123, 5.130831777933054e-05, 0.05025565251708031, 0.9999972581863403, 0.9999743700027466, 0.9995373487472534, 0.006342454347759485, 0.999997615814209, 0.0003489687223918736, 0.9999755620956421, 0.0031589637510478497, 0.9999982118606567, 0.9999110698699951, 0.9998286962509155, 0.9816662669181824, 0.0003937313158530742, 0.9999972581863403, 0.9969736337661743, 0.9999949932098389, 0.9999792575836182, 0.011500393971800804, 0.0001592398330103606, 0.00014516949886456132, 0.9999996423721313, 0.9993928670883179, 0.0004146051360294223, 0.9999570846557617, 0.9999490976333618, 0.9999074935913086, 0.010454988107085228, 0.005503143183887005, 0.9999998807907104, 0.0661819726228714, 0.0002936294476967305, 0.00021298378123901784, 2.189026963606011e-05, 0.9999991655349731, 0.0007219678373076022, 0.0002327132096979767, 0.0008355702739208937, 0.0001390138058923185, 2.022544140345417e-05, 0.9999485015869141, 0.02824385277926922, 0.9999891519546509, 0.0006963384803384542, 0.8553200364112854, 0.9999998807907104, 0.9995974898338318, 0.9998188614845276, 2.077548015222419e-05, 0.2918456792831421, 0.9992535710334778, 0.9981681108474731, 0.0009209022391587496, 5.589121064986102e-05, 0.9991149306297302, 0.8289663195610046, 0.9997745156288147, 0.9999991655349731, 5.736921229981817e-05, 0.0007713245577178895, 0.0018483246676623821, 0.9999797344207764, 0.43110576272010803, 5.477361810335424e-06, 0.9999068975448608, 0.9997497200965881, 0.9998564720153809, 0.0023667779751122, 0.9999986886978149, 9.861879334494006e-06, 0.999963641166687, 0.0006382931605912745, 8.552646613679826e-05, 0.00010936884791590273, 0.3562774360179901, 1.9784363757935353e-05, 0.9980310797691345, 0.9998151659965515, 0.00017161015421152115, 0.9997730851173401, 0.9981070756912231, 0.0004005263326689601, 0.9998341798782349, 0.0031766663305461407, 0.000126582570374012, 0.0004488751292228699, 0.9999386072158813, 8.576142818128574e-07, 0.00030316784977912903, 0.9999663829803467, 0.003164059715345502, 0.985813558101654, 5.347831756807864e-05, 0.9986503720283508, 0.004321742337197065, 0.9999973773956299, 0.0002237472217530012, 0.9904689192771912, 0.9999699592590332, 0.9999998807907104, 4.7756362619111314e-05, 0.0014747760724276304, 0.006279658060520887, 0.0005706171505153179, 0.00031140452483668923, 0.001332957181148231, 0.07729336619377136, 0.03786313533782959, 0.9997093081474304, 0.00012532819528132677, 0.25993025302886963, 0.9708299040794373, 6.770784966647625e-05, 0.9999980926513672, 0.9991992115974426, 0.9699099063873291, 0.014361321926116943, 0.9999805688858032, 0.004979063291102648, 1.0, 0.00019690583576448262, 0.9999529123306274, 0.9989866614341736, 0.9997296929359436, 0.00046906634815968573, 0.00026425247779116035, 0.002968818647786975, 0.828998863697052, 0.2877524197101593, 0.999860405921936, 0.9999996423721313, 0.9999027252197266, 0.00012771326873917133, 0.9998829364776611, 0.004108686000108719, 0.0002520614943932742, 0.017755400389432907, 0.00756927439942956, 0.0002919019607361406, 0.9999986886978149, 0.10382454842329025, 4.157290459261276e-05, 0.9999940395355225, 0.9999233484268188, 0.9999328851699829, 0.3467787504196167, 0.006954117212444544, 0.9999659061431885, 5.846306521561928e-05, 0.0005690272664651275, 0.00026295636780560017, 0.9984275102615356, 0.9780125021934509, 0.9997726082801819, 0.0007985233096405864, 0.0009485561749897897, 0.9999980926513672, 6.192540604388341e-05, 0.00040868346695788205, 0.3415336310863495, 0.9792283773422241, 0.14004771411418915, 8.14602681202814e-05, 0.9987892508506775, 0.9994246959686279, 0.07585146278142929, 0.9987130165100098, 0.0008591059595346451, 0.9999874830245972, 0.9999936819076538, 0.00019717054965440184, 1.921002422022866e-06, 0.999911904335022, 0.999454915523529, 0.9999819993972778, 0.9999862909317017, 0.0014840650837868452, 0.9967702627182007, 0.999941349029541, 5.635223487843177e-07, 0.999626874923706, 0.9950751662254333, 0.9999982118606567, 0.0004755012923851609, 0.9995810389518738, 0.12931489944458008, 0.0003318263916298747, 0.9999889135360718, 0.9998683929443359, 0.9928788542747498, 0.9999994039535522, 0.997497022151947, 0.9992756247520447, 0.003898024093359709, 1.4222797290130984e-05, 0.32503557205200195, 0.6454290747642517]

#Metricas de validación
Best_acc9_6 = 0.9916666666666667
acc_aux9_6 = 0.9916666666666667
Recall9_6 = 1.0
Especificidad9_6 = 0.9836065573770492
Tiempo9_6 = 1554.77179813385

######## Base10 ########
#MODELO = VGG16_BN
#Datos perdidas
xx_train10_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train10_6 = [0.08266511088536109, 0.058339640820099986, 0.051548437654604144, 0.053127181097681137, 0.05602280882957541, 0.10864148367150331, 0.048067950392946784, 0.044617052367072044, 0.03906720886811798, 0.04595861147031372, 0.03555834047772266, 0.03513453495171335, 0.03642222868439592, 0.03371209121412701, 0.042252812598958425, 0.040965772033841524, 0.04099171660196634, 0.03511572854570401, 0.03768161905032617, 0.03741486055063613, 0.03726327066988121, 0.036877116487349995, 0.03994095369621559, 0.03824603864808142, 0.04479717856939928]
xx_val10_6 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val10_6 = [0.012233530316087935, 0.0731334075331688, 0.09313240514861212, 0.13231292731232114, 0.08016062610679203, 0.01694180385933982, 0.00823254237572352, 0.007468588733010822, 0.00524311794175042, 0.005742854045497047, 0.00501795725689994, 0.0050719375411669414, 0.004723460806740655, 0.005096845163239373, 0.004552620649337769, 0.005951925781038072, 0.005189412501123217, 0.006001710726155175, 0.004582719504833221, 0.004307301508055793, 0.004322770320706897, 0.005183591610855526, 0.0065890742672814265, 0.005914693905247582, 0.0046549733314249254]
epoch_min10_6 = 0
stop_val10_6 = 0.012233530316087935

#Datos accuracy
acc_train10_6 = [0.9743827160493826, 0.9787037037037036, 0.9833333333333333, 0.9799382716049383, 0.979320987654321, 0.9635802469135802, 0.9787037037037036, 0.983641975308642, 0.986111111111111, 0.9824074074074074, 0.9876543209876543, 0.9891975308641975, 0.9879629629629629, 0.9873456790123457, 0.9858024691358025, 0.9864197530864197, 0.9864197530864197, 0.9873456790123457, 0.9873456790123457, 0.9885802469135803, 0.9864197530864197, 0.9876543209876543, 0.9845679012345678, 0.9888888888888888, 0.9848765432098765]
acc_val10_6 = [0.9972222222222222, 0.9722222222222222, 0.9722222222222222, 0.9472222222222223, 0.9722222222222222, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9972222222222222, 1.0]

#Datos curva ROC
best_labels10_6 = [1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0]
best_prob10_6 = [0.9999867677688599, 0.004116615746170282, 0.9993603825569153, 0.00023558823158964515, 0.9982749223709106, 0.9978541731834412, 0.9998632669448853, 0.9987249970436096, 0.0033922779839485884, 0.9998252987861633, 0.007800499442964792, 0.9999450445175171, 0.0022519780322909355, 0.9995139837265015, 0.005157667212188244, 0.9994726777076721, 0.0031383694149553776, 0.9993840456008911, 0.02523876540362835, 0.9997549653053284, 0.9968950748443604, 0.002493331441655755, 7.3695344326552e-05, 0.9907673001289368, 0.9933700561523438, 0.010183856822550297, 0.0020061475224792957, 0.9999035596847534, 0.9988141059875488, 0.0017966595478355885, 0.9998849630355835, 0.9948722720146179, 0.0024436903186142445, 0.9999375343322754, 0.9926139116287231, 0.00016345269978046417, 0.9971058964729309, 0.9990249872207642, 0.9981091022491455, 0.9927213788032532, 0.0063688126392662525, 0.9971193075180054, 0.9964749217033386, 0.9993556141853333, 0.993945300579071, 0.9970056414604187, 0.00044262371375225484, 0.9998791217803955, 0.9999735355377197, 0.9999810457229614, 0.9996389150619507, 0.9946548938751221, 0.0027587779331952333, 0.000219513603951782, 0.9998189806938171, 0.9966289401054382, 0.0026189875788986683, 0.010293415747582912, 0.00140286551322788, 0.0011871124152094126, 0.9952345490455627, 0.00017626867338549346, 0.999419093132019, 0.00020730492542497814, 0.9998956918716431, 0.9958565831184387, 0.0004010910342913121, 0.0012003998272120953, 0.9999117851257324, 0.0005765845417045057, 0.9996676445007324, 0.9993017911911011, 2.0544825019896962e-05, 0.9978846907615662, 0.00022657970839645714, 0.0033339080400764942, 0.001058410620316863, 0.9927471280097961, 0.9984697699546814, 0.005516855977475643, 0.9829283952713013, 0.00010165842104470357, 0.00021982192993164062, 0.001158330705948174, 0.999516487121582, 0.0003149828116875142, 0.9757120013237, 0.9987710118293762, 0.003180393949151039, 0.9963009357452393, 0.16491827368736267, 0.9957303404808044, 0.004740639589726925, 0.9912344217300415, 0.0010392433032393456, 0.9984326958656311, 0.015911273658275604, 0.0024129818193614483, 0.0011035342467948794, 0.9972609281539917, 0.002478536684066057, 0.996378481388092, 0.9997120499610901, 8.615044498583302e-05, 0.9999454021453857, 0.9999725818634033, 0.9972286820411682, 0.9999152421951294, 0.0006343125132843852, 0.9645245671272278, 0.032219234853982925, 0.0018374492647126317, 0.9948082566261292, 0.00152447703294456, 0.998386025428772, 0.9996404647827148, 0.9999479055404663, 0.9985503554344177, 0.9998388290405273, 0.9974743723869324, 0.9991703033447266, 0.9999641180038452, 0.9919230341911316, 0.0606984868645668, 0.05881677195429802, 0.0027618661988526583, 0.965039074420929, 0.007477321196347475, 0.9999912977218628, 0.9876508116722107, 0.9999643564224243, 0.0037985965609550476, 0.0008362775552086532, 0.01035955362021923, 0.0014606069307774305, 0.9993182420730591, 0.0007665554294362664, 0.9999603033065796, 0.9997559189796448, 0.00035036797635257244, 0.9946491122245789, 0.0007760688895359635, 0.00019920762861147523, 0.9986076951026917, 0.00024916123948059976, 0.0038583457935601473, 0.002630028873682022, 0.9922439455986023, 0.9995915293693542, 0.056932516396045685, 0.17499931156635284, 0.001935467473231256, 0.999805748462677, 0.0019093613373115659, 0.9999231100082397, 0.0037602121010422707, 0.9999839067459106, 0.0013954777969047427, 0.994469165802002, 0.9715896844863892, 0.9986510872840881, 0.9999812841415405, 0.00036014101351611316, 0.004626493901014328, 0.004225917626172304, 0.0005149141070432961, 0.007621913217008114, 0.9987603425979614, 0.06006879732012749, 0.9999758005142212, 0.001692186458967626, 0.001055647386237979, 0.0011934855720028281, 0.00037605431862175465, 0.00024085289624053985, 0.9998674392700195, 0.9977976083755493, 0.0029121849220246077, 0.00012174871517345309, 0.00047589308815076947, 0.001402782159857452, 0.9999233484268188, 0.8900653719902039, 0.05160292610526085, 0.012399738654494286, 0.00031328294426202774, 0.002185715828090906, 0.9980483055114746, 0.994327962398529, 0.008182954043149948, 0.07465796917676926, 0.9987611770629883, 0.9980303645133972, 0.9965159893035889, 0.9999661445617676, 0.002974876668304205, 0.9978383183479309, 0.9999381303787231, 0.9970824122428894, 0.004388590808957815, 0.9994491934776306, 0.995509147644043, 0.9998103976249695, 0.9992220401763916, 8.606636401964352e-05, 0.0060143619775772095, 0.01031236257404089, 0.0003681164525914937, 0.998641312122345, 0.0025253319181501865, 0.9991766810417175, 0.9999071359634399, 0.005321601405739784, 0.9994736313819885, 0.9968417882919312, 0.0022078845649957657, 0.003091424237936735, 0.0005810589645989239, 0.9994321465492249, 0.0044025154784321785, 0.0020976499654352665, 0.9922601580619812, 0.9983974099159241, 0.01013061311095953, 0.0013931506546214223, 0.9995796084403992, 0.9887879490852356, 0.0028669785242527723, 0.010310289449989796, 0.015819333493709564, 0.010345098562538624, 0.9987862706184387, 0.9994408488273621, 0.9999897480010986, 0.6693107485771179, 0.9999648332595825, 0.0008730148547329009, 0.0012481380254030228, 0.006213614717125893, 0.0009645563550293446, 0.05453706532716751, 0.9999243021011353, 0.0004898403421975672, 0.9992116689682007, 0.9816158413887024, 0.003384136827662587, 0.9952699542045593, 0.9978335499763489, 0.0031803164165467024, 0.003845666302368045, 0.9972870349884033, 0.012132735922932625, 0.0001910205028252676, 0.9998571872711182, 0.9155653119087219, 0.0012256999034434557, 0.0003953622072003782, 0.01014022808521986, 0.9979530572891235, 0.0011126056779175997, 0.0025921997148543596, 0.8571265339851379, 0.00037202361272647977, 0.9951885938644409, 0.9994753003120422, 0.019736232236027718, 0.0006215062458068132, 0.9974712133407593, 0.0004173835623078048, 0.9985589385032654, 0.9941190481185913, 0.18343393504619598, 0.0012775303330272436, 0.9985557198524475, 0.999972939491272, 0.9963303208351135, 0.00018241148791275918, 0.9999929666519165, 0.00019607643480412662, 0.0007718376582488418, 0.9999121427536011, 0.9443033337593079, 0.040171682834625244, 0.011460145935416222, 0.0009012732771225274, 0.9933100938796997, 0.04631137475371361, 0.9967888593673706, 0.0003077480650972575, 0.9999605417251587, 0.9999606609344482, 0.1277780383825302, 0.042495861649513245, 6.26684122835286e-05, 0.9998704195022583, 0.0015980652533471584, 0.44718989729881287, 0.9987479448318481, 0.0001056621695170179, 0.9996391534805298, 0.0026571026537567377, 0.0022807361092418432, 0.9991031885147095, 0.0018164218636229634, 0.00047062450903467834, 0.00026774968137033284, 0.0018783360719680786, 0.9952763319015503, 0.9975008368492126, 0.005202001892030239, 9.207638504449278e-05, 0.9932286143302917, 0.0004887902177870274, 0.0006807554163970053, 0.9972010850906372, 0.1624843180179596, 0.004694309085607529, 0.005418284796178341, 0.0007188191520981491, 0.0004893054720014334, 4.3510564864845946e-05, 0.041536495089530945, 0.999954342842102, 0.002516073640435934, 0.9967561364173889, 0.0006542409537360072, 0.00015439178969245404, 0.00017795726307667792, 0.001779831130988896, 0.9990195035934448, 0.9999054670333862, 0.896117627620697, 0.05874907225370407, 0.9978715181350708, 0.0003788940666709095, 5.621503078145906e-05, 0.9976935982704163, 0.9999464750289917, 0.011546938680112362, 0.9994303584098816, 0.00046750670298933983, 0.9989954829216003, 0.030864663422107697, 0.007565285079181194, 0.9994454979896545, 0.9937323927879333, 0.0038691554218530655, 0.9999972581863403, 0.988381028175354, 0.9999685287475586, 0.008575753308832645, 0.9898393154144287, 0.00023540073016192764, 0.9952865839004517, 0.0032536895014345646, 0.9998225569725037, 0.003285839455202222, 0.9983116388320923, 0.9573714137077332, 0.9999195337295532]

#Metricas de validación
Best_acc10_6 = 0.9972222222222222
acc_aux10_6 = 1.0
Recall10_6 = 1.0
Especificidad10_6 = 0.994475138121547
Tiempo10_6 = 1558.5681464672089

###################################################################

######## Base1 ########
#MODELO = VGG19
#Datos perdidas
xx_train1_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train1_7 = [0.18933299866355496, 0.11901767700910568, 0.08493801432627218, 0.07443701998319155, 0.07520934247676238, 0.04335735912124316, 0.042906262357661755, 0.04073754386217506, 0.04422596054220641, 0.039521123542461865, 0.03635716550512078, 0.033982646805636674, 0.03860397070278356, 0.03881610929965973, 0.04059727232397339, 0.04584647415228832, 0.03829103036795133, 0.03913102562964699, 0.04662935318954197, 0.04271636187662313, 0.04107786559580285, 0.039217942189655186, 0.035176709634654316, 0.0343794492199833, 0.036484291468873435]
xx_val1_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val1_7 = [0.19329894615544213, 0.059213211801317, 0.0861106351017952, 0.25287611385186515, 0.046604287955496045, 0.03994672861364153, 0.03966801630126105, 0.038997540871302284, 0.03961392028464211, 0.04120873361825943, 0.04101439904835489, 0.04087758834163348, 0.04081722026069959, 0.040735675891240435, 0.04056120042999586, 0.040554426030980216, 0.040543561346001095, 0.0405344855454233, 0.04052442991071277, 0.04051747934685813, 0.04051732056670719, 0.04051716857486301, 0.04051720185412301, 0.04051707171731525, 0.04051712097393142]
epoch_min1_7 = 5
stop_val1_7 = 0.03994672861364153

#Datos accuracy
acc_train1_7 = [0.9265432098765432, 0.9558641975308642, 0.9691358024691358, 0.9691358024691358, 0.9743827160493826, 0.9858024691358025, 0.9824074074074074, 0.987037037037037, 0.9830246913580246, 0.9842592592592593, 0.9845679012345678, 0.9879629629629629, 0.9833333333333333, 0.9854938271604938, 0.9845679012345678, 0.9854938271604938, 0.983641975308642, 0.986111111111111, 0.9848765432098765, 0.9820987654320987, 0.983641975308642, 0.9858024691358025, 0.9873456790123457, 0.986111111111111, 0.9885802469135803]
acc_val1_7 = [0.9277777777777778, 0.9777777777777779, 0.9750000000000001, 0.8972222222222223, 0.9805555555555556, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112]

#Datos curva ROC
best_labels1_7 = [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0]
best_prob1_7 = [0.9988779425621033, 0.0034647404681891203, 0.004023903515189886, 0.36948102712631226, 0.007093660533428192, 0.9999668598175049, 0.9998043179512024, 0.0010424843057990074, 0.0009057109709829092, 0.00015535256534349173, 0.0006847502663731575, 0.9999879598617554, 0.9999949932098389, 0.9999998807907104, 0.9997053742408752, 0.9999899864196777, 0.0006086456123739481, 1.0, 0.0005090575432404876, 0.010236981324851513, 0.9993282556533813, 0.013390335254371166, 0.0017983202124014497, 0.9999998807907104, 0.9998923540115356, 0.9999040365219116, 1.8048876881948672e-05, 0.9999909400939941, 0.003027835162356496, 0.0057914345525205135, 0.10430512577295303, 0.014155421406030655, 1.0, 0.9999872446060181, 0.00363742932677269, 1.0, 0.03642323613166809, 3.2760450267232955e-05, 0.9999058246612549, 0.0014415254117920995, 4.4449920096667483e-05, 0.004341248422861099, 0.9998770952224731, 0.00843889731913805, 0.9999997615814209, 0.0058633009903132915, 0.025839217007160187, 0.003718817373737693, 0.9999998807907104, 0.026166053488850594, 0.00664549320936203, 0.00020807840337511152, 0.01669982634484768, 0.2444619983434677, 0.9945566654205322, 1.0, 0.9999973773956299, 0.00015933095710352063, 0.08115661144256592, 0.008188707754015923, 0.6081412434577942, 0.9996907711029053, 0.9996185302734375, 2.5142153390333988e-05, 0.9996154308319092, 0.9999774694442749, 1.0, 0.016598796471953392, 0.0020746970549225807, 0.9995213747024536, 0.02114393562078476, 0.00012918055290356278, 0.9999488592147827, 1.0, 0.02083868905901909, 0.0005257329321466386, 0.030626798048615456, 0.0019045014632865787, 0.9996616840362549, 0.9999889135360718, 0.0028469825629144907, 1.0, 0.9998810291290283, 0.04138436168432236, 0.9999990463256836, 0.9999809265136719, 0.9822559356689453, 0.0005759888445027173, 0.9998114705085754, 1.0, 1.0, 0.012380187399685383, 0.001490306225605309, 0.6632392406463623, 0.020869974046945572, 0.9971977472305298, 0.0015815331134945154, 0.00032324536005035043, 1.0, 0.0008621750166639686, 0.00023407823755405843, 0.00044317450374364853, 1.0, 0.9999998807907104, 0.0003355084336362779, 0.000860833388287574, 0.015156594105064869, 0.06243492290377617, 0.9998706579208374, 0.9930129647254944, 0.004065078217536211, 0.00015535256534349173, 2.9112104584783083e-06, 1.0, 0.031603675335645676, 1.0, 0.005274682305753231, 0.5611345767974854, 0.0010481667704880238, 0.9999345541000366, 0.9999995231628418, 0.01230065431445837, 0.9999986886978149, 1.0, 1.0, 0.007152192760258913, 0.9982348680496216, 0.0005358004127629101, 0.9279245734214783, 1.0, 0.9999927282333374, 0.9999592304229736, 0.00024640103220008314, 0.0008025190327316523, 0.005715429782867432, 1.0, 0.9998077750205994, 0.014458727091550827, 0.00030518477433361113, 0.9757070541381836, 0.0006311899051070213, 0.9999997615814209, 0.9990037083625793, 0.0006999192992225289, 0.9883270859718323, 0.004408081062138081, 0.000849020725581795, 0.9999997615814209, 1.0, 0.999661922454834, 0.9999929666519165, 0.9999998807907104, 0.0066102612763643265, 0.9982190728187561, 0.007919720374047756, 0.014862651005387306, 0.0005596045521087945, 0.0011879269732162356, 0.005089737009257078, 0.0005562180303968489, 0.001525373081676662, 0.9998975992202759, 1.0, 0.9994949102401733, 0.01132424920797348, 0.9985326528549194, 0.9918062686920166, 1.0, 0.0017706191865727305, 0.9998063445091248, 0.002447910839691758, 1.0, 0.9996833801269531, 0.0017036345088854432, 0.0005740394117310643, 0.001112142694182694, 0.004993184003978968, 4.9422313168179244e-05, 0.0006752221961505711, 0.0024302550591528416, 0.9999761581420898, 0.995895266532898, 0.0026797351893037558, 0.003654533065855503, 1.0, 4.908680421067402e-05, 0.9996738433837891, 0.9986584186553955, 0.0013358969008550048, 0.9999996423721313, 0.9999997615814209, 0.9999998807907104, 0.9996695518493652, 0.910439133644104, 0.9963524341583252, 0.0002735985326580703, 0.12265046685934067, 0.9910147786140442, 0.9999986886978149, 0.005783422850072384, 0.9942317605018616, 0.7204815149307251, 0.14954441785812378, 2.1645129891112447e-05, 0.0015226128743961453, 0.0004073061281815171, 0.9999998807907104, 0.005563735496252775, 0.0009071420063264668, 0.001353187602944672, 0.9957385063171387, 0.000687988183926791, 0.0055308956652879715, 0.005568900145590305, 0.9988285899162292, 1.0, 1.848770807555411e-05, 0.0005988876218907535, 0.9999216794967651, 0.999920129776001, 0.9999628067016602, 0.007488277740776539, 2.5681412807898596e-05, 0.36279556155204773, 0.030216122046113014, 0.017941003665328026, 0.007255451753735542, 0.9999990463256836, 0.3096385598182678, 0.9630894064903259, 0.004210398532450199, 0.0019166376441717148, 0.9985892176628113, 0.9999998807907104, 0.9999887943267822, 0.9998854398727417, 0.9999942779541016, 0.9978748559951782, 0.00016063997463788837, 0.9999998807907104, 1.0, 1.0, 9.339324606116861e-06, 8.670074748806655e-06, 0.9999995231628418, 0.9999995231628418, 0.0055932155810296535, 1.0, 0.0005210963427089155, 1.070468351827003e-05, 0.9999955892562866, 0.0011605726322159171, 0.9999419450759888, 0.9998931884765625, 0.005943990312516689, 0.9998553991317749, 0.0006538553861901164, 1.0, 0.369119793176651, 0.00011906853615073487, 0.9999998807907104, 0.9989463686943054, 0.994390070438385, 0.0049246689304709435, 0.0025153271853923798, 0.9993897676467896, 0.0542132742702961, 0.999945878982544, 0.00021786346042063087, 9.979394963011146e-05, 0.008000655099749565, 0.9999895095825195, 0.0001916744076879695, 0.9999995231628418, 0.9999963045120239, 0.0012460905127227306, 0.008749215863645077, 0.9975804090499878, 5.263319326331839e-05, 0.04585667699575424, 0.0005739365587942302, 0.9904705286026001, 1.0, 0.15845546126365662, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9998160004615784, 0.9983477592468262, 0.001210167072713375, 0.00040333811193704605, 0.9999996423721313, 0.9998043179512024, 0.0008051608456298709, 0.00044108700240030885, 0.999981164932251, 0.00032983117853291333, 0.0008721979684196413, 0.0002354261087020859, 0.9999995231628418, 1.0, 0.9999991655349731, 1.0, 0.0007193948258645833, 0.9983068704605103, 0.0005106016760692, 0.9999994039535522, 0.07079187035560608, 0.0031282519921660423, 0.9999643564224243, 0.0004951372393406928, 7.393636769847944e-05, 0.0003624249075073749, 0.9999556541442871, 0.1497621387243271, 1.0, 0.9993190765380859, 1.0, 0.009340677410364151, 0.9995255470275879, 0.9982337951660156, 1.0, 0.0037730843760073185, 1.0, 0.0017223481554538012, 0.9999998807907104, 0.9999982118606567, 0.004266927018761635, 0.0026218832936137915, 0.9997069239616394, 0.9999796152114868, 0.0035242706071585417, 0.003303552744910121, 0.9997223019599915, 0.0025242369156330824, 0.0020062411203980446, 0.0015206929529085755, 0.01658373512327671, 0.9999994039535522, 0.004968591965734959, 0.999909520149231, 0.9999032020568848, 1.0, 0.9999998807907104, 0.9990195035934448, 0.0032513828482478857, 0.006693427916616201, 0.04330569878220558, 1.0, 0.9999924898147583, 0.0001687082403805107, 0.9999980926513672, 0.0045893327333033085, 0.9999998807907104, 0.9996408224105835, 0.9998558759689331, 0.0014961418928578496]

#Metricas de validación
Best_acc1_7 = 0.9861111111111112
acc_aux1_7 = 0.9885802469135803
Recall1_7 = 0.9888268156424581
Especificidad1_7 = 0.9834254143646409
Tiempo1_7 = 1621.2717196941376

######## Base2 ########
#MODELO = VGG19
#Datos perdidas
xx_train2_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train2_7 = [0.07876802872360489, 0.05048471540212631, 0.059114049310669485, 0.05580264569064717, 0.04252325458291136, 0.03249788884027505, 0.027023619652530292, 0.025623204982207146, 0.028861535745270457, 0.028808426861593753, 0.029760314643750955, 0.03113660013050209, 0.03465689278127235, 0.03025496471443294, 0.028317746004940553, 0.027085895560405873, 0.025796973507529422, 0.030942405208393378, 0.02378983288819407, 0.02928722393181589, 0.029268378543633, 0.03200958278260113, 0.02390438778164946, 0.029404390143391527, 0.025194974128662805]
xx_val2_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val2_7 = [0.04387645771106084, 0.08248695482810338, 0.2416262235906389, 0.0339499036471049, 0.04020359284347958, 0.03479551308684879, 0.03922203506032626, 0.041052379707495375, 0.04376215007570055, 0.03740198744667901, 0.03748570366038217, 0.03753954056236479, 0.037460135089026556, 0.037666064666377175, 0.0376027579108874, 0.03760064807203081, 0.03760419934988022, 0.03760813143518236, 0.03760217593775855, 0.037591742475827535, 0.0375915366742346, 0.037591554555628036, 0.03759143319394853, 0.0375915495885743, 0.03759148981836107]
epoch_min2_7 = 3
stop_val2_7 = 0.0339499036471049

#Datos accuracy
acc_train2_7 = [0.9737654320987654, 0.9820987654320987, 0.979320987654321, 0.9790123456790123, 0.9851851851851852, 0.9873456790123457, 0.990432098765432, 0.9898148148148148, 0.9888888888888888, 0.9891975308641975, 0.9882716049382716, 0.9879629629629629, 0.9888888888888888, 0.9888888888888888, 0.9888888888888888, 0.9901234567901235, 0.9916666666666667, 0.9888888888888888, 0.990432098765432, 0.9879629629629629, 0.9882716049382716, 0.9873456790123457, 0.991358024691358, 0.9885802469135803, 0.9901234567901235]
acc_val2_7 = [0.9750000000000001, 0.9777777777777779, 0.9222222222222223, 0.9833333333333334, 0.9805555555555556, 0.9833333333333334, 0.9777777777777779, 0.9777777777777779, 0.9777777777777779, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556]

#Datos curva ROC
best_labels2_7 = [1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
best_prob2_7 = [0.9999983310699463, 0.0002797227934934199, 0.0009032238158397377, 1.0, 0.000653665978461504, 0.9999035596847534, 0.0003957267908845097, 0.503131091594696, 0.00038554114871658385, 0.9999995231628418, 1.0, 0.9999991655349731, 0.9988262057304382, 0.9999796152114868, 0.03573064133524895, 0.9999970197677612, 1.0, 0.9997398257255554, 0.006275118328630924, 0.00015970697859302163, 1.0, 0.0009077084250748158, 0.9999890327453613, 1.0, 0.9999960660934448, 0.9984388947486877, 0.9999668598175049, 0.00026177120162174106, 0.001379698165692389, 0.9996076226234436, 0.9999969005584717, 0.9999998807907104, 0.0013230890035629272, 0.0009413128136657178, 0.00047797115985304117, 1.0, 0.0013983565149828792, 0.999985933303833, 3.230242873542011e-05, 1.5009938579169102e-05, 1.4583533811673988e-05, 0.9998999834060669, 0.9999634027481079, 0.9659805297851562, 0.9999983310699463, 1.0287884833815042e-05, 0.000808589335065335, 0.9999990463256836, 0.0032373694702982903, 0.9994117021560669, 0.9999996423721313, 8.802840602584183e-05, 1.0, 1.0, 0.9999803304672241, 0.00011811118019977584, 0.0008803550153970718, 0.00012825339217670262, 1.0, 3.321371696074493e-05, 1.0, 0.9998782873153687, 0.00030663629877381027, 0.0002072814095299691, 0.9999994039535522, 0.9999673366546631, 0.0003909277729690075, 0.9998801946640015, 0.9999998807907104, 0.003509488422423601, 0.9982141256332397, 0.002115601673722267, 0.0009119909373112023, 0.00014548377657774836, 8.403592801187187e-05, 1.0, 0.00031720197875984013, 0.0005082812858745456, 0.005096506793051958, 0.0022124480456113815, 0.994941234588623, 0.0006062822067178786, 2.969280785691808e-07, 1.0, 0.00016846825019456446, 0.003070952370762825, 0.0036125462502241135, 0.9999998807907104, 0.0003467737406026572, 0.00027314035105518997, 0.0010221913689747453, 0.9999984502792358, 0.00018355829524807632, 0.0004108655557502061, 0.9999899864196777, 0.9998669624328613, 0.0005620655720122159, 5.317255613590532e-07, 9.657276677899063e-05, 0.9999991655349731, 0.9999949932098389, 0.003750450676307082, 0.004869084805250168, 1.0, 1.2047220820932125e-07, 0.9998695850372314, 0.9986401200294495, 0.0021440377458930016, 3.2800027838675305e-05, 1.0, 1.0, 0.9963094592094421, 0.0004802049952559173, 1.0, 0.999994158744812, 0.999995231628418, 1.0, 0.9999998807907104, 0.0014612774830311537, 0.0005373478052206337, 0.001598466420546174, 0.9983885288238525, 5.554615563596599e-05, 0.9997323155403137, 0.006590152624994516, 0.00018634673324413598, 0.999993085861206, 0.9999995231628418, 1.0, 0.0024834140203893185, 0.0018710490548983216, 0.9999988079071045, 0.9977951049804688, 0.9999998807907104, 0.9974921941757202, 0.9987500905990601, 0.0008622462046332657, 0.023369796574115753, 0.0032746505457907915, 0.00033512894879095256, 1.0, 0.9999997615814209, 0.9999912977218628, 8.325442468049005e-05, 0.9999927282333374, 0.9991223216056824, 0.0001340442249784246, 1.0, 0.9999995231628418, 0.0007628410821780562, 0.0018123774789273739, 0.9999305009841919, 0.00047982632531784475, 0.000633219548035413, 0.9999998807907104, 0.8779919147491455, 0.0026314137503504753, 0.0010288252960890532, 0.000566689355764538, 1.0, 1.0, 0.9999805688858032, 0.9999793767929077, 3.6658970202552155e-05, 0.001176463789306581, 0.9999998807907104, 0.9999735355377197, 1.0, 0.9999945163726807, 9.45024294196628e-05, 0.0001330921659246087, 0.0003146877570543438, 0.0029914178885519505, 0.9991401433944702, 0.9999996423721313, 0.9997547268867493, 0.999998927116394, 0.9999994039535522, 0.009023011662065983, 1.0, 5.46667433809489e-06, 0.0008603192400187254, 0.938723087310791, 0.0014462823746725917, 0.0010185498977079988, 0.8625196218490601, 0.004747767932713032, 0.9999998807907104, 0.9999700784683228, 0.00013672297063749284, 1.0, 0.9998447895050049, 1.0, 0.0007612535846419632, 0.00020859330834355205, 0.999998927116394, 0.9999486207962036, 1.0, 0.001782091916538775, 0.9974843263626099, 0.9999091625213623, 4.649869151762687e-05, 0.9927374720573425, 0.9997614026069641, 0.002026821719482541, 0.0007892641006037593, 0.9121984839439392, 0.999936580657959, 0.9998360872268677, 0.9999902248382568, 0.9579473733901978, 0.0005982201546430588, 6.027134531905176e-06, 0.00102249626070261, 0.00023722396872472018, 0.0001969413278857246, 0.00020513340132310987, 0.0004930583527311683, 0.0061042411252856255, 0.0020870983134955168, 7.544471736764535e-05, 0.0001767850772012025, 0.00040914423880167305, 0.00014663152978755534, 1.0, 0.9958080053329468, 0.0002026226429734379, 1.0, 1.0, 0.0001377347216475755, 0.9998722076416016, 1.0, 0.0003094695566687733, 0.9999864101409912, 2.3344253349932842e-05, 0.00015091468230821192, 0.0005082967691123486, 0.0002141042350558564, 0.9999998807907104, 0.9988775849342346, 7.399702008115128e-05, 0.9999822378158569, 0.00017632669187150896, 0.000404908147174865, 0.0006708150613121688, 0.9999152421951294, 0.00010563849355094135, 0.0010713511146605015, 0.00040915829595178366, 1.0, 0.9999961853027344, 0.035681504756212234, 0.9998738765716553, 0.9997922778129578, 0.0004370039387140423, 0.9999986886978149, 0.001294577494263649, 0.9995861649513245, 3.22333762596827e-05, 0.0007933979504741728, 0.0003297604853287339, 0.9999998807907104, 0.0005191946984268725, 0.0005271431873552501, 0.0005370789440348744, 0.0006441839504987001, 0.9999816417694092, 1.0, 0.00023652854724787176, 2.409748594800476e-05, 1.0, 0.001590762403793633, 0.014905844815075397, 0.005952285137027502, 0.9997819066047668, 0.00793495960533619, 0.9999845027923584, 0.999624490737915, 0.9999997615814209, 4.3921951146330684e-05, 0.8282930254936218, 0.9999998807907104, 0.0005271442350931466, 0.00012958968000020832, 4.288752279535402e-06, 6.175969610922039e-05, 0.9999980926513672, 0.6897151470184326, 0.9998338222503662, 0.0011869021691381931, 1.0, 1.0, 0.9998180270195007, 0.00014692793774884194, 0.9999997615814209, 0.0016645208233967423, 0.0023059893865138292, 0.9708085060119629, 0.9999397993087769, 1.9987735868198797e-05, 0.0006251645972952247, 0.9999741315841675, 0.9999185800552368, 0.0015772999031469226, 0.9890940189361572, 0.7370019555091858, 3.122361158602871e-05, 0.00022916165471542627, 0.00012013200466753915, 0.9999618530273438, 0.003090702462941408, 0.014819481410086155, 0.0031966117676347494, 1.0, 0.0017391432775184512, 0.9996077418327332, 0.04487650841474533, 1.0, 0.9921143054962158, 0.0029398591723293066, 0.99997878074646, 0.9999549388885498, 0.9999991655349731, 1.0, 0.9999430179595947, 0.9998835325241089, 0.0010348147479817271, 1.0, 0.9998075366020203, 0.004814805928617716, 0.9997932314872742, 1.0, 0.00040185460238717496, 1.0, 0.9998007416725159, 0.9999277591705322, 0.9999390840530396, 1.1046388863178436e-05, 0.9996104836463928, 0.9998935461044312, 0.00035302876494824886, 3.701927198562771e-05, 0.9999637603759766, 0.9999213218688965, 0.0016280419658869505, 1.0, 0.0007574513438157737, 0.007430696859955788, 0.002337416633963585, 0.46541935205459595, 0.0005732356803491712, 0.00010064488014904782, 4.333184187998995e-05, 0.000793228973634541, 1.0, 0.9972134232521057, 0.34969887137413025, 0.9999988079071045, 1.0, 4.793494099430973e-06]

#Metricas de validación
Best_acc2_7 = 0.9833333333333334
acc_aux2_7 = 0.9916666666666667
Recall2_7 = 0.978021978021978
Especificidad2_7 = 0.9887640449438202
Tiempo2_7 = 1618.443826675415

######## Base3 ########
#MODELO = VGG19
#Datos perdidas
xx_train3_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train3_7 = [0.06372812609245748, 0.04677475226330168, 0.03869026919573913, 0.051757313347893, 0.045154451955983665, 0.03254480131062461, 0.027576133875567236, 0.027642700287662907, 0.0243468929036164, 0.02522390889532772, 0.021315917400298295, 0.022308749218045928, 0.02736231647891763, 0.0279634486745905, 0.023303243481082682, 0.03319816697894791, 0.033900850974483254, 0.02423781422369274, 0.02915209616407936, 0.027591531659349983, 0.028072052715737143, 0.024207762150484837, 0.02423276608740842, 0.03296546819217411, 0.025597318436628505]
xx_val3_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val3_7 = [0.015084537035889096, 0.03382655034462611, 0.02348154220316145, 0.014038751025994618, 0.014980972641044192, 0.017537719590796363, 0.018373533089955648, 0.01788373076253467, 0.021214346587657928, 0.01761166552702586, 0.017657879988352457, 0.017703503039148118, 0.01765524132384194, 0.017685917682117885, 0.017690324121051364, 0.01768904709153705, 0.01769692964024014, 0.01769639270173179, 0.01769936548338996, 0.017694959706730313, 0.017694947951369815, 0.017695089015695782, 0.017695182893011304, 0.01769527312782076, 0.017695211205217574]
epoch_min3_7 = 0
stop_val3_7 = 0.015084537035889096

#Datos accuracy
acc_train3_7 = [0.9759259259259259, 0.9839506172839506, 0.9851851851851852, 0.9802469135802468, 0.9808641975308642, 0.987037037037037, 0.9898148148148148, 0.9901234567901235, 0.9907407407407407, 0.9895061728395061, 0.9919753086419753, 0.9907407407407407, 0.9901234567901235, 0.9888888888888888, 0.9901234567901235, 0.9901234567901235, 0.987037037037037, 0.9888888888888888, 0.9885802469135803, 0.9895061728395061, 0.9901234567901235, 0.9907407407407407, 0.9910493827160494, 0.9873456790123457, 0.9888888888888888]
acc_val3_7 = [0.9944444444444445, 0.9916666666666667, 0.9861111111111112, 0.9944444444444445, 0.9888888888888889, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9944444444444445]

#Datos curva ROC
best_labels3_7 = [0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0]
best_prob3_7 = [0.0002039628307102248, 0.9999998807907104, 0.0002844840637408197, 0.9999338388442993, 0.9998972415924072, 0.004092037677764893, 0.9999974966049194, 0.011537086218595505, 0.0003169843112118542, 0.9999916553497314, 0.0003991506528109312, 0.9998181462287903, 0.0017840450163930655, 0.9999403953552246, 0.00019955741299781948, 0.9999428987503052, 0.9998477697372437, 0.031479548662900925, 7.631563494214788e-05, 0.9999843835830688, 0.00022801481827627867, 0.00010596193897072226, 0.0010367735521867871, 0.0003015029651578516, 0.999997615814209, 0.9997262358665466, 0.0037847382482141256, 0.006584311369806528, 1.0, 0.02458183281123638, 0.004367726854979992, 0.9999854564666748, 0.026682522147893906, 5.2096285799052566e-05, 0.9999998807907104, 0.0015005520544946194, 0.0013031592825427651, 0.0027458379045128822, 0.00038790926919318736, 0.009982018731534481, 0.9964287877082825, 0.00031022753682918847, 0.9987377524375916, 0.0013828467344865203, 0.0001289029751205817, 0.9999972581863403, 0.0026406378019601107, 0.999985933303833, 0.9740923047065735, 0.001755279372446239, 0.08489974588155746, 3.515492790029384e-05, 0.0002098376426147297, 0.9995030164718628, 0.999931812286377, 0.002834687940776348, 2.2420616005547345e-05, 0.02306085266172886, 0.9999845027923584, 0.00013009905524086207, 1.0, 0.9999985694885254, 1.0, 1.0, 0.004181163851171732, 0.9998955726623535, 0.0007995344931259751, 0.9999990463256836, 0.9996788501739502, 1.0, 0.0006764897843822837, 0.9999909400939941, 1.7449525557822199e-06, 0.0010714184027165174, 1.0, 0.0010007979581132531, 0.0032229635398834944, 0.0011004646075889468, 0.9816270470619202, 0.0008014353807084262, 0.0017955191433429718, 7.339307194342837e-05, 1.0, 0.9999337196350098, 0.9984550476074219, 0.9998924732208252, 1.0, 1.0, 0.0014728085370734334, 0.00329982559196651, 0.00011346827523084357, 0.9999874830245972, 0.9997949004173279, 0.004559241235256195, 0.999936580657959, 0.9999997615814209, 0.9994468092918396, 4.863961294176988e-05, 0.00015190827252808958, 1.0, 1.0, 0.9999998807907104, 1.0, 0.016939479857683182, 0.0015879204729571939, 2.1879479390918277e-05, 0.9999125003814697, 0.9999970197677612, 0.9998010993003845, 1.0, 0.0070282453671097755, 0.00553557462990284, 0.6916962265968323, 1.0, 0.9848345518112183, 0.9885128736495972, 0.00011845224798889831, 0.9999396800994873, 4.076448931300547e-06, 0.0013666246086359024, 2.068240792141296e-05, 8.985918975668028e-05, 1.0, 0.0026743013877421618, 0.9962872266769409, 0.004734905902296305, 0.9999998807907104, 0.9885411262512207, 0.9999992847442627, 0.017847299575805664, 4.594244092004374e-05, 0.0021214433945715427, 0.9999747276306152, 0.0061877272091805935, 0.00010289763304172084, 0.0003959875903092325, 0.3980793058872223, 0.00029292594990693033, 7.65805525588803e-05, 0.9999822378158569, 0.9626299738883972, 1.0, 3.1608258723281324e-05, 0.9717536568641663, 1.0, 0.0002584473695605993, 0.9999997615814209, 0.9999350309371948, 0.00046021820162422955, 0.0018644060473889112, 0.9999984502792358, 1.0, 0.9999126195907593, 0.999196469783783, 0.08425014466047287, 0.9999988079071045, 0.9670431613922119, 5.678063098457642e-05, 0.3961913585662842, 0.002457569818943739, 0.9999707937240601, 0.9998407363891602, 4.921448271488771e-05, 1.3922597190685337e-06, 0.9999978542327881, 0.000492257415316999, 9.488810974289663e-06, 0.14442068338394165, 8.569635247113183e-05, 0.002588026225566864, 4.1482056985842064e-05, 0.9920401573181152, 5.889213571208529e-05, 0.9997177720069885, 0.9980022311210632, 0.9996753931045532, 0.9999998807907104, 0.999980092048645, 0.9968158602714539, 0.00268954923376441, 1.0, 0.00017564404697623104, 0.9999998807907104, 0.9997931122779846, 1.0, 0.0003917724243365228, 7.859242759877816e-05, 8.8843225967139e-05, 0.9999998807907104, 1.0, 0.9999674558639526, 0.9999783039093018, 0.002155714202672243, 0.9998940229415894, 0.9999978542327881, 0.5567067861557007, 1.0, 0.9999946355819702, 0.0001260239805560559, 0.0002897271479014307, 0.7254469990730286, 0.9997929930686951, 0.9999998807907104, 0.9999803304672241, 0.0003620726056396961, 0.9999963045120239, 1.0, 0.0033999874722212553, 0.0005172983510419726, 0.9999639987945557, 1.0, 7.29059029254131e-05, 0.0025488007813692093, 0.00030630617402493954, 8.81004671100527e-05, 0.01504539791494608, 0.9992849230766296, 0.9999985694885254, 0.008807137608528137, 0.9999947547912598, 0.03474630042910576, 0.00013991890591569245, 0.00013423492782749236, 0.002648891182616353, 0.0001176888108602725, 0.00010509851563256234, 0.00021811641636304557, 2.927365858340636e-05, 0.9999996423721313, 0.9999998807907104, 1.0, 0.00046957278391346335, 0.9125345349311829, 0.007169716060161591, 0.6964843273162842, 0.9969130754470825, 0.001762591884471476, 1.0, 0.0009962905896827579, 0.9968723654747009, 0.9999399185180664, 0.0006621320499107242, 0.9999747276306152, 0.001126192044466734, 0.0017017622012645006, 0.0006538520683534443, 0.001297695329412818, 0.007181643508374691, 1.0, 0.9999991655349731, 0.000292441516648978, 1.0, 0.9999971389770508, 0.0046001304872334, 0.00042473984649404883, 0.9959308505058289, 0.9996688365936279, 4.547707430901937e-05, 1.0, 0.0007932863663882017, 0.9999972581863403, 0.000608451256994158, 1.0, 0.9999529123306274, 0.020326141268014908, 0.0315573550760746, 0.0010199366370216012, 0.9999996423721313, 0.9998419284820557, 0.9520869851112366, 0.006063058972358704, 0.9999865293502808, 0.08305441588163376, 1.1524242836458143e-05, 0.0006397835095413029, 1.0, 0.9999330043792725, 0.000323954300256446, 5.5293214245466515e-05, 0.9997376799583435, 0.00016503459482919425, 0.0007418747991323471, 0.9999998807907104, 5.557597887673182e-06, 0.9995610117912292, 0.9999996423721313, 0.0007009098189882934, 0.0005437046638689935, 0.0006362276035360992, 2.4333125111297704e-05, 0.9999997615814209, 0.0410592220723629, 0.9994797110557556, 0.9999912977218628, 0.9889076352119446, 1.4691843716718722e-05, 0.9997957348823547, 0.0010546473786234856, 0.00013340983423404396, 0.00042926857713609934, 0.0006608444382436574, 0.9948964715003967, 0.9991858601570129, 4.47327911388129e-05, 1.0, 0.9973540306091309, 7.270969217643142e-05, 0.00017348969413433224, 0.999997615814209, 1.0, 0.0018367893062531948, 2.5384884793311357e-05, 0.0006434362730942667, 0.9990007281303406, 0.999988317489624, 0.999455988407135, 1.0, 0.9994949102401733, 0.006338982377201319, 0.9999858140945435, 0.0006337507511489093, 0.9999741315841675, 0.9985272884368896, 0.9999970197677612, 0.9999958276748657, 0.9942405223846436, 0.00015873834490776062, 0.013010352849960327, 0.0028970285784453154, 0.0019813012331724167, 0.0015368309104815125, 0.01770859956741333, 0.0013560452498495579, 0.9999971389770508, 3.662109884317033e-05, 0.9999697208404541, 0.9999992847442627, 0.998921275138855, 0.9999926090240479, 0.9999994039535522, 4.729905413114466e-05, 8.05157014838187e-06, 0.9981124401092529, 0.0030809754971414804, 0.004856389481574297, 3.573481444618665e-05, 0.9924261569976807, 0.9430944323539734, 0.9967355132102966, 0.00013162150571588427, 0.004896245431154966, 1.0, 0.9004119038581848, 0.00011335234739817679, 0.9999969005584717, 0.9999995231628418, 0.9996058344841003, 0.9999998807907104, 0.00042141659650951624, 0.9999942779541016]

#Metricas de validación
Best_acc3_7 = 0.9944444444444445
acc_aux3_7 = 0.9944444444444445
Recall3_7 = 0.9944444444444445
Especificidad3_7 = 0.9944444444444445
Tiempo3_7 = 1614.587068080902

######## Base4 ########
#MODELO = VGG19
#Datos perdidas
xx_train4_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train4_7 = [0.06433212792063937, 0.033458448854493505, 0.05717660758966281, 0.054561068853478376, 0.12890876925285952, 0.12356342279616697, 0.08388251645697488, 0.07755411942431956, 0.07384449467982775, 0.06968449646676028, 0.06197133057831246, 0.07043574938067683, 0.06623749791839977, 0.06581407471755404, 0.06532016642115734, 0.06302832589473253, 0.06836228078162229, 0.059990991357667944, 0.06099626475278242, 0.0636001307287334, 0.070241080261307, 0.062205178768914425, 0.06622620538612943, 0.05749648058855975, 0.060289870487687025]
xx_val4_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val4_7 = [0.022881262997786204, 0.015405370957321592, 0.041626089811325075, 0.017298750744925605, 0.2103898295097881, 0.0535602202018102, 0.046753003034326765, 0.04670537329382367, 0.04504874911573198, 0.040927619735399885, 0.04085348728630278, 0.040592919952339594, 0.040325690309206644, 0.04027324534124798, 0.03992396742105484, 0.03992384241686927, 0.039920303059948814, 0.03990589694844352, 0.03988980385992262, 0.039874467419253455, 0.03987397154172261, 0.03987372169891993, 0.039873498347070484, 0.039872988396220735, 0.039872495498922134]
epoch_min4_7 = 1
stop_val4_7 = 0.015405370957321592

#Datos accuracy
acc_train4_7 = [0.9740740740740741, 0.9879629629629629, 0.979320987654321, 0.9817901234567901, 0.9558641975308642, 0.95, 0.9694444444444444, 0.9709876543209877, 0.9734567901234568, 0.9746913580246913, 0.9762345679012345, 0.975, 0.9753086419753086, 0.9737654320987654, 0.9731481481481481, 0.9737654320987654, 0.9728395061728395, 0.9768518518518519, 0.975, 0.9771604938271605, 0.9712962962962963, 0.9753086419753086, 0.9746913580246913, 0.979320987654321, 0.9783950617283951]
acc_val4_7 = [0.9861111111111112, 0.9861111111111112, 0.9888888888888889, 0.9944444444444445, 0.8944444444444445, 0.9861111111111112, 0.9888888888888889, 0.9861111111111112, 0.9861111111111112, 0.9888888888888889, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112]

#Datos curva ROC
best_labels4_7 = [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
best_prob4_7 = [0.4838102161884308, 2.595455589471385e-05, 0.36707550287246704, 1.0, 3.6948149499949068e-06, 0.00013346949708648026, 1.0, 4.471344709600089e-06, 0.01679917238652706, 1.361182171422115e-07, 1.0, 6.102300744714739e-07, 0.9999994039535522, 0.0006127420347183943, 0.00012605966185219586, 1.0, 0.9999958276748657, 3.444386322826176e-07, 1.0, 1.0, 0.999657392501831, 0.9630832076072693, 1.3118698007019702e-05, 1.613683053847126e-08, 6.51652908345568e-06, 1.5258058150280362e-09, 1.0, 3.6860154040141424e-08, 1.0, 7.787652833712855e-08, 4.17364162785816e-07, 3.6084946941628004e-07, 0.00014772039139643312, 1.0, 1.95389038282201e-08, 0.00015946151688694954, 1.0, 0.9997400641441345, 1.0, 6.576744149278113e-10, 1.0, 0.9999995231628418, 3.643753387905235e-08, 0.9999911785125732, 0.9999914169311523, 1.0, 7.359768972037273e-08, 4.1973496990976855e-05, 1.3330977433057e-09, 3.268207819928648e-06, 1.3428094689516001e-06, 0.9999014139175415, 1.0336300420021871e-06, 0.00035082484828308225, 0.01669529266655445, 1.0, 6.843976620984904e-07, 1.0, 0.9997456669807434, 1.0, 0.9999953508377075, 1.1972780157520901e-05, 1.0, 9.417486346308124e-09, 0.9667224287986755, 0.9989138841629028, 1.682091976817901e-08, 2.2277772870893386e-07, 1.0, 5.396785581979202e-06, 1.0, 0.0022927371319383383, 1.0, 0.9997685551643372, 1.0, 1.0, 3.031695172683868e-10, 1.3771217055591478e-08, 0.00011823648674180731, 0.0031120707280933857, 1.0, 0.0008097512763924897, 1.0, 1.0, 0.9999997615814209, 0.00017230452795047313, 0.00019655056530609727, 5.399187585908294e-08, 1.5182695278781466e-06, 1.0, 1.0, 1.0, 6.205434146977495e-06, 0.9999998807907104, 0.9999998807907104, 1.4935225181034184e-06, 0.002136319875717163, 1.1261863619438373e-06, 0.9990490078926086, 1.0, 3.190149477916293e-09, 1.0, 7.546628239651909e-06, 5.181732944947726e-07, 0.0001835866569308564, 0.9999164342880249, 1.0, 1.0, 1.5820918974895903e-07, 0.0002050315379165113, 1.0, 1.2304486517678015e-05, 0.9999998807907104, 1.0, 1.0, 6.948605005163699e-06, 1.0, 5.476946398630389e-07, 1.130401960836025e-05, 1.0, 0.9999998807907104, 0.9918618202209473, 0.9897335171699524, 1.0, 1.0, 1.0, 1.0, 4.0084402463946844e-09, 7.220399129437283e-05, 0.9999880790710449, 1.0, 0.9999983310699463, 4.118689685128629e-05, 2.3188347313407576e-06, 0.28464624285697937, 0.9999995231628418, 1.0, 2.0034893168485723e-05, 0.9999768733978271, 1.0, 1.0, 3.168234616168775e-05, 0.9999998807907104, 4.810860332327138e-07, 2.2806685251453018e-07, 0.9961042404174805, 1.0, 1.0, 1.0, 0.0006798318354412913, 0.9968224763870239, 0.9989668130874634, 1.0, 1.7486912895492424e-07, 0.0015910622896626592, 1.0, 0.9999998807907104, 0.9999608993530273, 1.0, 0.000650332891382277, 1.0, 1.0, 1.1945179721806198e-05, 1.3423494237940758e-05, 1.0, 1.0, 0.9940407872200012, 3.441875378484838e-05, 0.00029480078956112266, 1.0, 5.550176069846202e-07, 0.9999994039535522, 0.005781808402389288, 1.0, 0.9999998807907104, 0.9999997615814209, 0.9978926777839661, 1.0, 0.9999921321868896, 0.9999988079071045, 1.6606624964055072e-09, 0.26837748289108276, 1.0, 0.9999994039535522, 3.50888331013266e-06, 0.9999998807907104, 1.1131385235785274e-07, 1.0, 0.7665700912475586, 0.9999998807907104, 0.00025758787523955107, 1.0, 1.986039706025622e-06, 0.00045475715887732804, 2.1650840842379893e-11, 5.083299559061061e-09, 1.0, 6.505835699499585e-06, 1.635889503859289e-07, 1.0, 0.0004186942824162543, 0.0017456872155889869, 6.647297112749584e-08, 1.0, 1.3841786312696058e-06, 7.578679651487619e-05, 0.9999982118606567, 0.010824767872691154, 6.88232546508516e-08, 0.9998680353164673, 3.052085230592638e-05, 9.473829413764179e-05, 1.986494226002833e-06, 0.9999985694885254, 1.1643434163488564e-06, 1.0, 1.1716158496710705e-06, 3.406406904105097e-05, 1.107849278980666e-08, 0.9999620914459229, 3.0118139875412453e-06, 0.4561205208301544, 1.0, 0.9999995231628418, 1.0, 0.9999997615814209, 1.549581156723434e-06, 1.2450355825421866e-05, 4.80854905617889e-05, 3.2400674143673314e-08, 0.9999998807907104, 1.0, 1.0, 7.88349370850483e-06, 1.0, 1.0, 9.879192657535896e-05, 2.0188268479159888e-07, 1.0, 6.375553880388907e-07, 0.00011750769772334024, 0.999998927116394, 1.3207019264882547e-06, 8.933493944596194e-08, 1.0, 1.0, 1.0, 1.0, 0.00010167567961616442, 2.4277886041090824e-05, 6.644655513809994e-05, 3.9613983062736224e-06, 1.0, 0.00017134821973741055, 1.8303276192455087e-06, 1.0, 8.706765584065579e-06, 0.9999971389770508, 0.9999969005584717, 2.786526056297589e-05, 3.957931767217815e-06, 1.0, 1.8163663639825245e-07, 1.0, 3.0369474046665346e-08, 1.0, 9.702802827860069e-08, 0.9999997615814209, 2.0030713585583726e-06, 1.0, 0.0006345235160551965, 1.0, 5.706225536528109e-08, 3.091361213591881e-05, 0.002406654180958867, 0.9999910593032837, 1.0, 0.9999947547912598, 0.9999998807907104, 0.9999957084655762, 2.5870365789160132e-05, 0.00024340662639588118, 1.0, 1.0, 9.979319293051958e-05, 3.2793916915352384e-08, 0.9999899864196777, 3.902114542597701e-08, 1.0, 4.345938941696659e-05, 1.217027261191106e-07, 0.0006127397064119577, 1.0, 1.0, 1.0, 1.0, 4.3106112570967525e-05, 4.8134712216096887e-08, 5.018634965381352e-07, 0.004528016783297062, 2.9928030926384963e-05, 1.0421233582746936e-06, 1.0, 1.5556310245301574e-05, 2.3494976630900055e-06, 1.0, 1.0, 2.173420767803691e-07, 1.0, 0.9996309280395508, 7.710364791080337e-09, 1.712045013846364e-05, 2.3637414869881468e-07, 1.0, 0.00011979386181337759, 6.830838117366511e-08, 0.9999990463256836, 0.9999984502792358, 1.0, 1.0, 0.9964780211448669, 8.865897598298034e-07, 0.00030565474298782647, 9.590271474735346e-07, 0.39759281277656555, 1.0, 1.9987339783256175e-06, 2.1783978809253313e-05, 1.0, 0.9263063669204712, 1.1165039722982328e-05, 1.0, 2.2219595848582685e-05, 1.0, 1.0, 3.8800387902604427e-10, 5.038580184191233e-07, 2.2198497617864632e-07, 1.0336564628232736e-05, 3.1790464163350407e-06, 1.0, 1.0, 1.2932988013858449e-08, 7.373763821760804e-08, 5.090091690362897e-06, 2.024899892205667e-08, 0.0017543722642585635, 3.3533963232912356e-06, 1.0, 1.0, 1.0, 1.4119743152463116e-09, 0.9999738931655884, 0.005528174340724945, 1.0, 1.1311355407883639e-08, 2.0634033148780873e-07, 3.654740794445388e-05, 2.9733234896411886e-06, 9.577218129663834e-09]

#Metricas de validación
Best_acc4_7 = 0.9861111111111112
acc_aux4_7 = 0.9944444444444445
Recall4_7 = 1.0
Especificidad4_7 = 0.972972972972973
Tiempo4_7 = 1581.9494233131409

######## Base5 ########
#MODELO = VGG19
#Datos perdidas
xx_train5_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train5_7 = [0.06786603092411418, 0.044736854511278644, 0.05616302383534702, 0.043491363286236186, 0.03361849991811646, 0.023930540487722113, 0.03309503440503721, 0.025898195655993474, 0.030227240175008774, 0.023839085576710876, 0.02488792715249238, 0.020584277661494267, 0.023311771820356816, 0.024588252033716368, 0.025509389830224306, 0.026337828156020905, 0.01840192394124137, 0.027250674348554493, 0.027147034463691124, 0.023138396165989065, 0.026961817960312336, 0.020510574126685107, 0.026045136746984943, 0.02669800630322209, 0.03011090675437892]
xx_val5_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val5_7 = [0.0249071243736479, 0.0038230066498120626, 0.05180855393409729, 0.004960450530052185, 0.00175228218237559, 0.0011807627148098417, 0.0013710149460368687, 0.001355756488111284, 0.0015482795735200247, 0.00146692113743888, 0.0014758845170338949, 0.0014635504947768318, 0.0014638430542416043, 0.0014823039372762045, 0.0014627535310056475, 0.0014619825614823234, 0.0014616001811292437, 0.0014624659385946061, 0.001461878087785509, 0.0014617073039213816, 0.0014617165757550134, 0.0014617431494924756, 0.0014617587129275003, 0.0014617822236484951, 0.0014617975387308333]
epoch_min5_7 = 5
stop_val5_7 = 0.0011807627148098417

#Datos accuracy
acc_train5_7 = [0.9725308641975309, 0.9817901234567901, 0.9811728395061728, 0.9830246913580246, 0.9879629629629629, 0.9916666666666667, 0.9876543209876543, 0.9879629629629629, 0.9891975308641975, 0.990432098765432, 0.9910493827160494, 0.9925925925925926, 0.9895061728395061, 0.9919753086419753, 0.991358024691358, 0.9885802469135803, 0.9941358024691358, 0.9919753086419753, 0.9901234567901235, 0.9916666666666667, 0.9916666666666667, 0.9919753086419753, 0.9898148148148148, 0.9916666666666667, 0.9888888888888888]
acc_val5_7 = [0.9916666666666667, 1.0, 0.9805555555555556, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels5_7 = [1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
best_prob5_7 = [0.9992995262145996, 0.9999957084655762, 1.0, 0.9999580383300781, 2.447636006763787e-06, 0.9999995231628418, 0.9999972581863403, 0.9923571348190308, 1.5387207895400934e-05, 7.438245665980503e-05, 1.0, 0.00017071299953386188, 0.0021426822058856487, 0.0009653478628024459, 0.0033976021222770214, 1.0, 4.659305704990402e-05, 0.00023943331325426698, 1.0, 0.9999996423721313, 0.9999947547912598, 0.9999972581863403, 7.110160368029028e-05, 1.0, 1.0, 1.0, 0.0008126809843815863, 3.906408210241352e-07, 1.0, 1.0, 0.0005638389848172665, 3.691552410600707e-05, 0.9947304725646973, 0.9999998807907104, 1.4841298252576962e-05, 0.00022732497018296272, 1.0, 0.00015274385805241764, 0.9999949932098389, 1.0, 4.439313215698348e-06, 9.316630894318223e-06, 3.658367631942383e-06, 0.9999754428863525, 0.002448955550789833, 1.0, 1.0, 0.9999983310699463, 1.0, 0.03899112716317177, 0.9999996423721313, 0.9999991655349731, 0.00018208060646429658, 0.9999997615814209, 0.9999998807907104, 9.914999282045756e-06, 5.834307259533489e-08, 1.0, 1.037125457514776e-06, 0.002987002255395055, 3.364159056218341e-06, 3.0488652100757463e-06, 0.999998927116394, 0.9999152421951294, 5.039982625021366e-06, 0.0035103990230709314, 0.9999947547912598, 1.0, 4.863075810135342e-05, 1.0, 0.9999996423721313, 1.0, 6.959972779441159e-06, 0.8805415034294128, 1.0, 0.999998927116394, 0.9999997615814209, 0.9999419450759888, 1.0, 9.301624959334731e-05, 0.004030762240290642, 1.0, 1.0, 5.612905806628987e-05, 1.0, 4.113830436835997e-05, 6.632247732341057e-06, 6.699945515720174e-06, 0.00021500515867955983, 1.0, 0.0012393679935485125, 0.9999997615814209, 1.0, 1.0, 1.0, 1.0, 0.9999998807907104, 6.735346147479504e-08, 0.004414874594658613, 0.999997615814209, 2.207016450483934e-06, 0.9999915361404419, 0.9773095846176147, 9.335579420621798e-08, 7.807388101355173e-06, 0.00015860550047364086, 9.908391120916349e-07, 0.9999998807907104, 1.0, 0.9999997615814209, 0.00016659795073792338, 1.0, 0.9999998807907104, 0.0001248841726919636, 0.9999531507492065, 0.000738502771127969, 1.8302527848845784e-07, 1.1063351848861203e-05, 0.9997453093528748, 2.0910755438308115e-07, 1.0, 0.9999990463256836, 0.9999892711639404, 0.9999834299087524, 0.00010046000534202904, 1.0, 0.0002010572497965768, 3.1851097901380854e-06, 0.9997641444206238, 1.0, 0.9999960660934448, 5.963637158856727e-05, 0.9999998807907104, 0.9999998807907104, 3.4896398574346676e-05, 0.9643534421920776, 1.9993736088963487e-07, 0.001211390714161098, 4.103321771253832e-05, 8.90969374722772e-07, 0.0013787313364446163, 0.00026111488114111125, 8.600663568358868e-05, 4.210503902868368e-06, 0.9999996423721313, 1.0, 0.0002809741999953985, 1.0, 0.00034204323310405016, 0.00023244180192705244, 0.9998809099197388, 1.0, 0.9996793270111084, 2.1424928036140045e-06, 1.775591726982384e-06, 2.3367596440948546e-05, 0.9999692440032959, 2.2039669147488894e-06, 0.0005244623753242195, 9.859893907560036e-06, 0.9999986886978149, 0.0018560728058218956, 0.0022657315712422132, 0.9999998807907104, 1.0, 0.0005067028687335551, 1.0, 4.795995209860848e-06, 0.9999884366989136, 1.0, 0.9999927282333374, 2.053608841379173e-05, 0.9999995231628418, 0.9999988079071045, 0.0002425189159112051, 2.6898629101879123e-08, 1.0, 2.517790562706068e-05, 1.0, 1.0, 2.5160767108900473e-05, 7.069589628372341e-06, 0.00013433456479106098, 4.864019160777389e-07, 1.0, 0.9991878867149353, 6.53109309496358e-05, 0.9921608567237854, 1.0, 1.0, 0.9999823570251465, 3.08348376165668e-06, 0.9999998807907104, 0.9999997615814209, 4.495156463235617e-05, 4.096851171198068e-06, 3.545984873198904e-05, 9.225259418599308e-05, 0.0002217345463577658, 0.9999992847442627, 1.7924226995091885e-05, 0.0002204349875682965, 1.99378060727895e-08, 1.0, 2.9831251140421955e-06, 0.9999904632568359, 0.0006367188761942089, 1.0, 0.999998927116394, 1.0, 0.004376635886728764, 5.5630330280109774e-06, 0.00020064151613041759, 0.9999990463256836, 0.9999998807907104, 2.9300135793164372e-05, 1.0055310895040748e-06, 1.0, 7.131230813683942e-05, 0.011752162128686905, 0.9999996423721313, 4.919994012198003e-07, 2.876476173696574e-06, 5.906608930672519e-05, 9.151203812507447e-06, 1.0, 4.7847628593444824e-05, 1.0, 1.0, 0.00011711673141689971, 0.9999998807907104, 0.9999994039535522, 1.0, 0.9999839067459106, 0.9999994039535522, 0.0002254251594422385, 0.0005501701962202787, 1.0, 8.981501196103636e-06, 2.2518412151839584e-05, 0.000485160417156294, 7.780763411346925e-08, 1.3424868257061462e-06, 0.9999270439147949, 0.00032026131520979106, 0.9999996423721313, 0.003237195545807481, 0.9537485837936401, 0.004317278042435646, 0.00039822427788749337, 1.0529221071919892e-05, 4.054983946844004e-06, 0.9999998807907104, 1.0, 1.0, 2.0036239334331185e-08, 0.0048814984038472176, 0.9999998807907104, 0.00021212590218055993, 3.6295623431215063e-06, 0.9992697834968567, 0.00016805622726678848, 3.632163497968577e-05, 1.0, 1.0, 1.0, 1.3623214272229234e-06, 2.3933045667945407e-05, 0.00022312981309369206, 7.661254494450986e-05, 4.4771169882551476e-07, 1.0, 1.0458221844089621e-08, 2.570666310930392e-07, 2.253972525068093e-05, 1.7675986327958526e-06, 1.0, 1.0, 2.6907249761620733e-08, 7.445412484230474e-05, 0.9999549388885498, 2.53948911677071e-07, 1.0, 1.0, 1.0, 0.9999681711196899, 1.0, 0.9999971389770508, 0.010393520817160606, 1.0, 0.0004094359464943409, 1.0, 0.9999998807907104, 1.0, 0.9999998807907104, 1.781703031156212e-05, 5.930777115281671e-05, 8.31007055239752e-05, 0.9999494552612305, 2.830603079928551e-05, 2.741977368714288e-06, 0.0005926709272898734, 0.9999613761901855, 3.795688826357946e-06, 0.9999896287918091, 2.4210539777413942e-05, 0.9999997615814209, 0.00025791171356104314, 0.00063345767557621, 0.9999922513961792, 0.0005693429848179221, 3.706010465975851e-05, 2.9136435841792263e-05, 1.0, 0.9999833106994629, 1.6733562006265856e-05, 0.00010567938443273306, 0.005077154841274023, 0.002209437545388937, 2.7771102395490743e-06, 1.0, 1.9955816242145374e-05, 0.9999954700469971, 1.0, 3.1545962428936036e-06, 1.0, 1.0196976596432705e-08, 1.0, 0.0002184806508012116, 1.0, 0.013682269491255283, 0.9999998807907104, 0.9999978542327881, 0.9999997615814209, 1.0, 0.9999887943267822, 0.9895826578140259, 7.109146281436551e-06, 1.0, 1.0, 1.0, 0.007307166233658791, 3.361672042956343e-07, 1.0, 0.9999366998672485, 1.0, 1.0, 1.0, 0.00026274737319909036, 0.9999442100524902, 7.312914385693148e-05, 0.9999978542327881, 1.1454534615040757e-05, 0.00041389709804207087, 0.00011849495786009356, 1.0, 0.000333358853822574, 8.738642645766959e-05, 4.02261721319519e-05, 1.0]

#Metricas de validación
Best_acc5_7 = 1.0
acc_aux5_7 = 1.0
Recall5_7 = 1.0
Especificidad5_7 = 1.0
Tiempo5_7 = 1625.1757156848907

######## Base6 ########
#MODELO = VGG19
#Datos perdidas
xx_train6_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train6_7 = [0.049801967017076634, 0.03707890649452621, 0.044007162823353284, 0.039037208047545985, 0.02860789492174431, 0.019641732654453795, 0.023100669774008386, 0.028316092601528873, 0.024194479954463464, 0.015414616575947514, 0.022847876725373445, 0.023783125305249366, 0.02117204521725207, 0.017899334688245516, 0.017748945132817752, 0.026614048699905843, 0.026544968471115017, 0.0197539962184282, 0.0218038633097837, 0.023051458339632295, 0.02476467385336205, 0.021405969560146332, 0.02128131577262172, 0.017746042607375133, 0.016731624627186928]
xx_val6_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val6_7 = [0.013453989062044355, 0.013743239972326491, 0.01609376437134213, 0.00897052569521798, 0.014713004893726773, 0.013345342046684689, 0.012844162185986836, 0.011859219438499874, 0.011921166214678022, 0.011384085648589663, 0.011423356003231473, 0.011422897709740533, 0.01146911448902554, 0.011424548178911208, 0.011449755397107866, 0.011449078718821207, 0.01145047363307741, 0.01144798058602545, 0.011448232332865398, 0.01144808323846923, 0.011448050290346146, 0.011447993831502067, 0.011447939111126794, 0.011447925120592117, 0.011447911295625899]
epoch_min6_7 = 3
stop_val6_7 = 0.00897052569521798

#Datos accuracy
acc_train6_7 = [0.9799382716049383, 0.986111111111111, 0.9858024691358025, 0.9867283950617284, 0.9907407407407407, 0.9916666666666667, 0.9925925925925926, 0.9888888888888888, 0.991358024691358, 0.994753086419753, 0.9916666666666667, 0.9907407407407407, 0.9929012345679012, 0.9929012345679012, 0.9938271604938271, 0.990432098765432, 0.9901234567901235, 0.9925925925925926, 0.9925925925925926, 0.991358024691358, 0.991358024691358, 0.9919753086419753, 0.9941358024691358, 0.9925925925925926, 0.9932098765432098]
acc_val6_7 = [0.9916666666666667, 0.9944444444444445, 0.9916666666666667, 0.9944444444444445, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667, 0.9916666666666667]

#Datos curva ROC
best_labels6_7 = [1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0]
best_prob6_7 = [0.9999997615814209, 0.9999998807907104, 0.00015316902135964483, 1.4996827530922019e-06, 1.0, 0.999998927116394, 0.9999994039535522, 1.0, 0.9997875094413757, 0.002084241947159171, 1.0, 0.9999951124191284, 0.0012433051597326994, 0.9818416833877563, 1.0, 2.059686721622711e-06, 0.9969822764396667, 0.9999890327453613, 1.0, 1.0, 0.9999972581863403, 0.999931812286377, 0.0004160966200288385, 0.9999984502792358, 1.0074113561131526e-05, 4.98800327477511e-05, 5.4923773859627545e-05, 0.9999997615814209, 1.0, 0.00010399374878033996, 0.00014935398939996958, 0.9999998807907104, 1.0, 4.5539850361819845e-06, 0.9999998807907104, 1.0631791838022764e-06, 0.00015985200298018754, 9.342354587715818e-07, 0.999785840511322, 0.9999998807907104, 0.9418891668319702, 0.0003282792749814689, 4.299095962778665e-05, 0.0007516794721595943, 4.1829443944152445e-05, 0.03338255360722542, 5.038685912950314e-07, 0.9999996423721313, 0.9998935461044312, 1.6258067262242548e-05, 0.002058890648186207, 0.001718882005661726, 0.9999998807907104, 0.0012242932571098208, 7.662437383260112e-06, 0.0017216737614944577, 1.0, 0.00022737371909897774, 0.9979206919670105, 2.5165138595184544e-06, 0.9996330738067627, 9.318817319581285e-06, 1.0, 0.9999994039535522, 4.629855902749114e-05, 1.0, 0.9999949932098389, 0.00030306525877676904, 1.0, 1.0, 1.0, 0.004145869519561529, 3.425287650316022e-05, 0.0005617705173790455, 0.6724086999893188, 1.6763109670137055e-05, 1.0, 8.786676630734291e-08, 0.00022349860228132457, 1.654175434850913e-06, 1.0, 0.9999991655349731, 6.005940349496086e-07, 2.5810802981141023e-05, 5.445096576295327e-06, 0.0008493403438478708, 5.20313449214882e-07, 0.9999793767929077, 0.9960842132568359, 5.572782174567692e-05, 1.0, 0.9998915195465088, 0.9997842907905579, 0.00013651973858941346, 0.539776623249054, 4.4733525328410906e-07, 1.0, 0.9999972581863403, 0.0012883921153843403, 1.6417472579632886e-05, 2.5163502868963405e-05, 0.000995295005850494, 0.9999998807907104, 6.279102308326401e-06, 1.0, 0.9999998807907104, 1.0, 4.04746151616564e-06, 1.0, 0.0008745734812691808, 1.829556822485756e-05, 1.0, 0.999785840511322, 0.999976396560669, 1.23326935863588e-06, 7.789012670400552e-06, 4.2434643546584994e-05, 0.999998927116394, 0.000268139032414183, 1.0, 1.0, 7.855463081796188e-06, 0.00018602395721245557, 0.9999996423721313, 0.9998186230659485, 4.882627990809851e-07, 1.0, 0.9999843835830688, 3.460902462393278e-07, 1.0, 0.9999854564666748, 6.62523689243244e-06, 0.9999998807907104, 1.0, 7.728837226750329e-05, 1.5780911155616195e-07, 1.0, 3.5781875340035185e-05, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0011493897764012218, 0.9999961853027344, 4.2637300794012845e-05, 0.99801105260849, 3.0548753784387372e-06, 2.230771542599541e-06, 1.0, 3.677171844174154e-05, 1.696280924079474e-05, 0.9991604089736938, 1.573822373757139e-05, 0.9999998807907104, 0.9999998807907104, 2.8281641789362766e-05, 1.0251314961351454e-05, 4.171749878878472e-06, 0.0034373735543340445, 1.1777403869928094e-06, 1.0, 2.1618909613607684e-06, 0.001958357635885477, 1.2302572940825485e-05, 1.0, 1.0, 0.009512110613286495, 0.9999998807907104, 6.0948899772483855e-05, 4.0087237721309066e-05, 1.9053832147619687e-05, 5.9688994952011853e-05, 0.9999995231628418, 0.9999996423721313, 0.00034911843249574304, 6.072513656363299e-07, 4.6423640014836565e-05, 2.0241901438566856e-05, 0.9999995231628418, 1.0, 0.9998975992202759, 1.0, 0.9999923706054688, 5.4519357945537195e-05, 0.022973697632551193, 0.0006638022605329752, 1.0, 0.9999994039535522, 1.0, 1.0, 1.0, 0.0004024694499094039, 0.9999995231628418, 0.9999707937240601, 1.0, 7.736505358479917e-05, 0.9999998807907104, 1.0, 0.00031706830486655235, 0.001164112938567996, 6.842953098384896e-06, 1.3287318324728403e-05, 0.9999690055847168, 0.999985933303833, 2.059364533124608e-06, 0.9999762773513794, 0.00011344847735017538, 0.9996919631958008, 0.0003028307401109487, 0.9999997615814209, 1.0, 0.99998939037323, 0.00010552502499194816, 7.519680366385728e-06, 0.9999909400939941, 0.999998927116394, 0.0005125340539962053, 0.5487490296363831, 0.0033349809236824512, 9.598390170140192e-05, 4.746715603687335e-06, 0.996417760848999, 0.9999983310699463, 7.2663947321416344e-06, 0.9904369711875916, 0.9999929666519165, 1.8265041944687255e-05, 0.9999995231628418, 1.0, 3.1870360999164404e-06, 1.4529865438817069e-05, 0.9999774694442749, 1.0, 1.0, 1.7257669924219954e-06, 1.0, 1.0, 3.850082975986879e-06, 0.9999980926513672, 1.0, 0.999964714050293, 1.0, 0.0002964756276924163, 4.934437674819492e-05, 0.9999665021896362, 2.6831899958779104e-05, 0.9999172687530518, 3.7877147406106815e-05, 0.0004974296316504478, 0.9999998807907104, 1.0, 7.022671979939332e-06, 9.635402238927782e-05, 0.008081409148871899, 0.005329204257577658, 1.0, 0.0007782572647556663, 3.2393123547080904e-05, 1.0, 4.550310404738411e-05, 0.9997264742851257, 0.0012045466573908925, 1.0, 0.9998655319213867, 4.290045126253972e-06, 3.4227264222863596e-06, 7.596442446811125e-05, 1.0, 5.723219146602787e-06, 1.0, 0.00011020267993444577, 0.9998505115509033, 0.9999998807907104, 1.0, 1.0, 3.573352296371013e-05, 3.849198492389405e-06, 0.9828231930732727, 1.0, 0.9999996423721313, 0.003326548496261239, 0.00020021304953843355, 0.0004143625556025654, 0.9999444484710693, 0.003157271770760417, 3.025239857379347e-05, 2.9838192858733237e-06, 0.9999624490737915, 0.9999998807907104, 0.0024741969536989927, 8.285287913167849e-05, 0.9999887943267822, 1.0, 0.9999991655349731, 2.5844768970273435e-05, 0.00018061154696624726, 0.2561511695384979, 1.0, 0.9999997615814209, 3.6310439099906944e-07, 1.0, 4.903562512481585e-05, 0.0001469987037125975, 9.577897941426272e-08, 0.0006003172020427883, 0.00020855093316640705, 1.0, 0.9999203681945801, 0.47164252400398254, 0.001884604338556528, 0.9999998807907104, 0.9999995231628418, 0.000648955930955708, 0.9999163150787354, 0.9999998807907104, 0.0008866370772011578, 1.0, 8.83611119206762e-06, 0.0009003497543744743, 0.03723914548754692, 3.6809742596233264e-05, 0.0003191092691849917, 0.0006763518322259188, 3.3927728964044945e-06, 0.0006591806304641068, 1.0, 0.9999998807907104, 1.0, 8.413225600634178e-07, 0.007675340864807367, 1.0, 0.9999995231628418, 1.0, 1.0, 0.9999995231628418, 1.0, 0.00012855679960921407, 1.0, 0.9999984502792358, 5.554053586820373e-06, 1.0, 0.9999902248382568, 0.0042078010737895966, 0.0001358468143735081, 0.9999817609786987, 3.1576782930642366e-06, 0.999998927116394, 0.0005439622327685356, 0.9999998807907104, 0.9999980926513672, 6.433432281482965e-05, 8.630211141280597e-07, 0.9999998807907104, 5.955229880783008e-06, 0.008976524695754051, 1.0, 0.004613407421857119, 0.9994344115257263, 9.47412718232954e-06]

#Metricas de validación
Best_acc6_7 = 0.9944444444444445
acc_aux6_7 = 0.994753086419753
Recall6_7 = 0.989010989010989
Especificidad6_7 = 1.0
Tiempo6_7 = 1609.7312932014465

######## Base7 ########
#MODELO = VGG19
#Datos perdidas
xx_train7_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train7_7 = [0.048942129093187826, 0.04576720324011497, 0.037296725855565364, 0.028044738197400245, 0.03398319500280015, 0.023251829360738212, 0.022011358300108967, 0.019003784656524657, 0.015986195657356286, 0.02314807238218225, 0.020951825196360366, 0.019458719168180302, 0.0203802417733787, 0.0166497765591851, 0.019652864116209524, 0.02292533834223394, 0.02001654264735587, 0.016226090904739168, 0.021600997696320217, 0.018241629151650417, 0.020458903430420675, 0.019407126546641926, 0.021878554469641345, 0.017108651140221844, 0.0179405955252824]
xx_val7_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val7_7 = [0.2295679148700502, 0.01781033111943139, 0.03690697683228387, 0.03478419780731201, 0.04627699620193905, 0.04144570512904061, 0.03714131067196528, 0.034365793897045985, 0.03409562756617864, 0.032973297188679376, 0.03298356235027313, 0.03293562895721859, 0.03282865799135632, 0.03279442149731848, 0.03271669720609983, 0.032713469531801015, 0.032710488306151496, 0.03270768258306715, 0.03270066976547241, 0.03269528763161765, 0.032695082326730095, 0.03269503737489383, 0.03269490069813198, 0.032694732315010494, 0.032694687777095374]
epoch_min7_7 = 1
stop_val7_7 = 0.01781033111943139

#Datos accuracy
acc_train7_7 = [0.979320987654321, 0.983641975308642, 0.987037037037037, 0.9898148148148148, 0.9867283950617284, 0.9901234567901235, 0.9910493827160494, 0.9925925925925926, 0.9941358024691358, 0.9919753086419753, 0.9929012345679012, 0.9938271604938271, 0.9929012345679012, 0.994753086419753, 0.9932098765432098, 0.9922839506172839, 0.9929012345679012, 0.9935185185185185, 0.9919753086419753, 0.9938271604938271, 0.9922839506172839, 0.9925925925925926, 0.9925925925925926, 0.9922839506172839, 0.9932098765432098]
acc_val7_7 = [0.925, 0.9861111111111112, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112]

#Datos curva ROC
best_labels7_7 = [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0]
best_prob7_7 = [0.002176875015720725, 0.5446451902389526, 0.9978141784667969, 1.0, 0.0008213322726078331, 0.00034997568582184613, 0.9999997615814209, 0.0072581893764436245, 1.5183060895651579e-05, 0.011443152092397213, 0.004888394847512245, 0.0005846465937793255, 0.9999998807907104, 3.394020313862711e-05, 1.0, 0.00016573288303334266, 0.012485171668231487, 0.9999371767044067, 0.9999996423721313, 0.9998550415039062, 1.882072137959767e-05, 0.003872477915138006, 1.0, 0.00015717977657914162, 0.0006065735942684114, 0.9999985694885254, 0.9999969005584717, 0.002575292019173503, 0.9999977350234985, 0.002121486933901906, 1.0, 0.9999954700469971, 0.00149486493319273, 0.0023144588340073824, 0.9999998807907104, 0.0005899907555431128, 0.00033577653812244534, 0.9999943971633911, 0.00034469395177438855, 0.0008647243375889957, 1.0, 1.0, 1.0, 0.0003682952665258199, 1.4815398117207224e-06, 1.0, 3.2158636713575106e-06, 1.0, 0.9999995231628418, 1.0, 0.0005671845865435898, 2.8688533348031342e-05, 2.319729537703097e-05, 1.1468286174931563e-05, 1.0, 1.0, 0.0014452759642153978, 1.0, 0.641955554485321, 1.6490814687131206e-06, 3.0283517844509333e-05, 0.00021355823264457285, 0.9999998807907104, 0.9999994039535522, 1.0, 1.0, 1.0, 0.0019580975640565157, 0.0012590344995260239, 0.000140400166856125, 1.0, 0.00017880566883832216, 7.665909652132541e-05, 0.999996542930603, 0.9999955892562866, 0.9999958276748657, 0.9993866682052612, 0.00011009364970959723, 0.9999748468399048, 0.9999995231628418, 0.9999984502792358, 0.0006825425662100315, 0.9999996423721313, 0.00044604638242162764, 0.9999998807907104, 0.9999985694885254, 0.9999994039535522, 0.9999998807907104, 1.0, 0.00025612462195567787, 0.0032281503081321716, 0.9999984502792358, 0.07905168831348419, 0.9999947547912598, 2.317141479579732e-06, 0.9999964237213135, 0.9999985694885254, 0.9999954700469971, 0.00050889648264274, 0.9999983310699463, 7.141015885281377e-07, 6.691369890177157e-06, 3.214343450963497e-05, 5.701981717720628e-05, 1.0, 0.9999769926071167, 9.006584150483832e-05, 0.0006262045935727656, 0.0005898454110138118, 0.00023723230697214603, 0.9999986886978149, 0.00014438855578191578, 0.012807165272533894, 0.9955535531044006, 1.0, 0.9999942779541016, 0.9999920129776001, 1.0, 1.0, 0.9999468326568604, 7.589012966491282e-05, 1.0, 0.9999994039535522, 1.0, 0.0006582809728570282, 0.9999847412109375, 0.9999955892562866, 1.4108886716712732e-05, 8.28075353638269e-05, 1.0, 0.0003004744939971715, 0.9999997615814209, 1.0, 0.9999991655349731, 0.003974451683461666, 6.543001290992834e-06, 0.0003495633718557656, 1.0, 1.0, 0.9999849796295166, 0.9999732971191406, 0.0003775684745050967, 0.0002861389657482505, 0.00011195331171620637, 1.0, 0.00972998607903719, 5.066827384325734e-07, 0.9999921321868896, 0.9999983310699463, 0.9999997615814209, 0.00011994726810371503, 0.0013838255545124412, 0.9999814033508301, 0.0007826206274330616, 0.00016645471623633057, 5.671558028552681e-05, 0.999998927116394, 1.0, 0.9999958276748657, 1.0, 0.013023676350712776, 0.00027676468016579747, 2.649357520567719e-05, 1.0, 0.9999834299087524, 0.9997629523277283, 0.0001914672611746937, 3.773214757529786e-06, 1.0, 0.00024196576850954443, 0.9999998807907104, 0.9999966621398926, 1.0, 0.0001013778819469735, 0.006551748141646385, 1.0, 0.0002579498686827719, 0.0002723783836700022, 1.0, 0.0004939259961247444, 0.9999997615814209, 1.0, 0.9999997615814209, 0.7219197154045105, 0.0001787558721844107, 0.00015834739315323532, 0.9999867677688599, 0.0004533362516667694, 1.0, 0.0005583072197623551, 0.9999845027923584, 0.9999604225158691, 1.0, 0.9999972581863403, 0.00015475894906558096, 0.00014441428356803954, 0.999994158744812, 9.176914318231866e-05, 0.9924818873405457, 1.806321165531699e-06, 0.9999901056289673, 1.57937097355898e-06, 1.0, 1.0, 1.0, 0.9999994039535522, 0.0028949608094990253, 1.0, 1.0, 0.00016015789879020303, 0.9999971389770508, 0.0022760771680623293, 1.0, 0.9999994039535522, 0.0009244728134945035, 0.0009403219446539879, 3.8275073166005313e-05, 3.653796738944948e-05, 0.9999996423721313, 0.9999997615814209, 0.9999963045120239, 0.0003963820927310735, 0.0002197755966335535, 0.000669977453071624, 0.9999982118606567, 1.0, 0.0009608850814402103, 0.018781978636980057, 1.0, 0.9999991655349731, 0.08563042432069778, 0.0002901317784562707, 1.0, 0.0027085724286735058, 0.00032700487645342946, 0.9999958276748657, 0.0019695209339261055, 0.00046442923485301435, 0.0003124936774838716, 1.0, 0.00040077342418953776, 1.0, 0.9999996423721313, 0.0005875580245628953, 0.00038153224159032106, 1.0, 3.184798697475344e-05, 0.9883484244346619, 0.9999488592147827, 1.0, 1.0, 0.9998400211334229, 0.999985933303833, 0.060603782534599304, 0.999998927116394, 0.0022254029754549265, 0.001805677660740912, 4.604721368650644e-07, 0.9999960660934448, 3.364065196365118e-05, 0.0005510353366844356, 0.9999966621398926, 0.0036160459276288748, 0.9999768733978271, 4.552302198135294e-05, 0.9999996423721313, 0.9999998807907104, 0.006723199505358934, 0.999995231628418, 0.00015328121662605554, 0.00017749209655448794, 0.001170070841908455, 4.239471672917716e-05, 0.9999998807907104, 1.0, 0.001545144827105105, 0.9999996423721313, 0.9999995231628418, 0.013161806389689445, 1.0, 0.00041076590423472226, 6.746782310074195e-05, 0.0002920180559158325, 0.00012475193943828344, 0.0013827552320435643, 0.00037285921280272305, 1.0, 1.0, 0.0012717337813228369, 0.999991774559021, 1.0, 1.0, 0.00029466900741681457, 0.9999998807907104, 1.2873296327597927e-05, 0.000284458277747035, 0.00045401070383377373, 0.00022954259475227445, 0.9999985694885254, 0.9999850988388062, 0.9999922513961792, 0.0006593255093321204, 1.0, 0.9999996423721313, 9.93048888631165e-05, 0.4137542247772217, 0.9999991655349731, 1.0, 0.0020152872893959284, 0.00015632824215572327, 1.0, 0.7481820583343506, 0.0014320435002446175, 0.0001021201242110692, 0.00022616916976403445, 0.002686731517314911, 0.001251955283805728, 0.00043088983511552215, 0.9999998807907104, 7.267344335559756e-05, 1.0, 0.00018263825040776283, 0.9329585433006287, 1.0, 1.0, 1.0, 0.00010288164048688486, 0.9999998807907104, 0.9995142221450806, 0.5281912088394165, 0.00017046919674612582, 0.0002451510517857969, 0.9999997615814209, 1.0, 1.0, 0.9999984502792358, 0.0004775002889800817, 0.0002904376306105405, 0.9999933242797852, 0.005242204759269953, 0.9999994039535522, 0.00012060891458531842, 1.0, 0.09965768456459045, 0.0017790197161957622, 5.425763811217621e-05, 0.9999996423721313, 6.876995757920668e-05, 3.424154419917613e-05, 0.0008092753705568612, 0.9999996423721313, 0.0002627493522595614, 0.9999995231628418, 0.999991774559021, 0.0005595788825303316, 0.9999996423721313, 0.000803547038231045, 0.9999983310699463, 1.0, 0.00025762050063349307]

#Metricas de validación
Best_acc7_7 = 0.9861111111111112
acc_aux7_7 = 0.994753086419753
Recall7_7 = 0.972972972972973
Especificidad7_7 = 1.0
Tiempo7_7 = 1606.1813478469849

######## Base8 ########
#MODELO = VGG19
#Datos perdidas
xx_train8_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train8_7 = [0.041953208223905095, 0.03625634441405167, 0.03927902147909741, 0.032917633431929126, 0.028877533383575488, 0.02056120499784564, 0.02710895287217917, 0.019844800546581363, 0.0238358473704185, 0.016804436474670597, 0.014497222742180765, 0.018815458179623993, 0.019426095108559104, 0.024432063884573217, 0.01348172355397248, 0.021573148134313984, 0.01919980337958277, 0.020614425157323295, 0.024363730443112644, 0.014396997044483821, 0.019755141565829148, 0.019252200341886943, 0.015345067190535274, 0.017420422304191708, 0.019428561821028038]
xx_val8_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val8_7 = [0.0021356042888429428, 0.006334844728310903, 0.0031017912758721246, 0.0064528859323925445, 0.002703030241860284, 0.0017429620027542114, 0.0014873438411288791, 0.002050321466392941, 0.004008666674296061, 0.0026192506154378253, 0.002534904744890001, 0.002535829941431681, 0.0024959807594617206, 0.0024897303846147323, 0.00243263840675354, 0.0024312489562564425, 0.002432650824387868, 0.0024317983123991224, 0.0024314181672202215, 0.0024306558900409276, 0.0024306250943077934, 0.0024305918150477938, 0.0024305846956041125, 0.0024305709534221225, 0.0024305350250667995]
epoch_min8_7 = 6
stop_val8_7 = 0.0014873438411288791

#Datos accuracy
acc_train8_7 = [0.9827160493827161, 0.9864197530864197, 0.9854938271604938, 0.9876543209876543, 0.9907407407407407, 0.9929012345679012, 0.9885802469135803, 0.9919753086419753, 0.9898148148148148, 0.9935185185185185, 0.995679012345679, 0.9938271604938271, 0.9925925925925926, 0.991358024691358, 0.9953703703703703, 0.9910493827160494, 0.990432098765432, 0.9907407407407407, 0.9916666666666667, 0.9938271604938271, 0.9932098765432098, 0.9938271604938271, 0.9929012345679012, 0.9950617283950617, 0.994753086419753]
acc_val8_7 = [1.0, 1.0, 1.0, 0.9972222222222222, 0.9972222222222222, 1.0, 1.0, 1.0, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels8_7 = [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
best_prob8_7 = [1.0, 0.9999984502792358, 0.9999992847442627, 0.018655138090252876, 0.9999768733978271, 0.9999818801879883, 1.0, 6.560574661307328e-07, 1.4242789347918006e-06, 2.744261337284115e-06, 0.0004008693213108927, 0.9999985694885254, 0.9999982118606567, 8.307081770908553e-06, 4.680012716562487e-05, 0.00018765674030873924, 1.0, 0.9998471736907959, 0.0005765347741544247, 0.9999338388442993, 1.3053714610578027e-05, 1.4123476148597547e-06, 1.391345722367987e-05, 1.2555688954307698e-05, 1.0, 4.597740371536929e-06, 0.9999985694885254, 4.213833282307178e-09, 0.9999927282333374, 4.065079224346846e-07, 7.145158633647952e-07, 0.0009591111447662115, 1.0, 1.0451920218201849e-08, 0.9999997615814209, 5.515837528946577e-06, 1.0, 7.501730578951538e-06, 3.864717291435227e-05, 0.00022565505059901625, 0.9999998807907104, 0.9999990463256836, 4.2860210669459775e-05, 0.9999995231628418, 0.9999831914901733, 0.9996517896652222, 2.4146067516994663e-05, 4.2750158172566444e-05, 1.0, 1.0, 0.9999994039535522, 0.00025587540585547686, 0.9999836683273315, 0.9999337196350098, 0.9925202131271362, 1.0, 0.9995889067649841, 0.9999606609344482, 1.0, 6.522280455101281e-05, 0.9093061089515686, 0.9999998807907104, 9.622576868650867e-08, 1.0, 0.9999995231628418, 0.00018938540597446263, 6.275482883211225e-05, 0.00021466778707690537, 0.00039499413105659187, 2.827603964306036e-07, 0.00010207611921941862, 0.00020941793627571315, 1.0, 1.0, 0.001731904107145965, 1.0, 7.5515330536291e-05, 1.0, 6.680152750959678e-07, 0.0026970673352479935, 2.1077100882394006e-06, 1.8374688806943595e-05, 1.8220833226223476e-05, 2.9552575142588466e-07, 1.0, 1.0, 1.0, 1.8130552916773013e-06, 0.9999998807907104, 0.005501259118318558, 1.0, 1.895343615387901e-08, 3.649182644949178e-06, 1.0, 1.0, 0.9999971389770508, 6.594419119210215e-06, 1.0, 0.0014816223410889506, 0.9999982118606567, 1.0, 1.0, 0.9999550580978394, 0.0016872573178261518, 0.9999918937683105, 1.0, 2.40700610447675e-05, 0.0017131541389971972, 1.0, 0.0005568313645198941, 0.9999983310699463, 0.9998884201049805, 0.000888723589014262, 0.9999964237213135, 1.0, 1.0, 0.0011237870203331113, 0.999993085861206, 0.9999938011169434, 0.9999996423721313, 1.0, 4.042304226459237e-06, 0.995222270488739, 1.5272766177076846e-05, 4.085513793938844e-08, 0.9999986886978149, 2.674559596016479e-07, 0.9999923706054688, 0.0005855840863659978, 0.001536792959086597, 6.665832916041836e-05, 0.9999964237213135, 3.012882871189504e-06, 6.745997234247625e-05, 1.3474489605869167e-05, 1.0, 0.008191186934709549, 1.0, 0.9999995231628418, 0.9999955892562866, 2.935058728326112e-05, 0.00012714315380435437, 0.9999992847442627, 5.734993465011939e-06, 0.9999988079071045, 4.24790584929724e-07, 2.3254917778103845e-06, 1.9117728697892744e-06, 0.9999692440032959, 1.0, 1.2097688113499316e-06, 4.062036168761551e-06, 0.9999994039535522, 0.9999998807907104, 0.9999992847442627, 0.9999998807907104, 0.00022094673477113247, 1.0, 0.02865975722670555, 5.453038102132268e-05, 0.0001127263021771796, 2.0432420200222623e-09, 0.9999901056289673, 0.00017414802277926356, 0.9999998807907104, 1.0, 1.0, 0.0004986580461263657, 3.2715615816414356e-05, 0.9826081395149231, 1.0, 0.13375480473041534, 4.7300047299358994e-05, 1.0, 3.2903593738531356e-10, 0.9999995231628418, 0.9999992847442627, 0.9999998807907104, 0.9999990463256836, 0.9999984502792358, 0.00016455045260954648, 0.9999995231628418, 0.005091214552521706, 5.799147402285598e-05, 4.6056673454586416e-05, 5.096165114082396e-05, 3.589698494010918e-08, 1.0, 1.0, 0.9999967813491821, 1.0, 1.0, 0.9999299049377441, 0.9999996423721313, 0.9999994039535522, 6.114572897786275e-05, 2.0468784001081985e-08, 4.420246841618791e-05, 0.999956488609314, 0.9999983310699463, 1.5304463886423036e-05, 4.2773212527436044e-08, 1.0, 1.0, 0.9999998807907104, 1.0, 0.9999994039535522, 0.9999998807907104, 0.0024722444359213114, 0.9999992847442627, 1.7626580302021466e-06, 0.9999935626983643, 0.0001527373242424801, 1.0, 0.9999971389770508, 0.9997448325157166, 0.0008652436081320047, 1.0988480125817546e-09, 1.3833984269240318e-07, 0.9999997615814209, 1.0, 0.0007057858165353537, 3.078105874010362e-05, 0.9994959831237793, 0.9999998807907104, 0.0006355397053994238, 1.0, 1.8686607290874235e-05, 0.00031738256802782416, 1.0, 0.9999979734420776, 3.527927674440434e-07, 2.5169934815494344e-05, 8.239011890509573e-07, 3.285671823505254e-08, 0.897291362285614, 0.9999990463256836, 1.0, 1.0, 0.0016304198652505875, 0.0008384801330976188, 1.2497130228439346e-05, 6.191266584210098e-05, 0.9928323030471802, 3.758988142976705e-08, 0.9999969005584717, 1.0, 2.096335720125353e-06, 3.5098754125328924e-08, 0.9999978542327881, 0.9999988079071045, 0.9999905824661255, 0.0007376450812444091, 0.9999812841415405, 0.9999998807907104, 9.352390407002531e-06, 0.9999880790710449, 0.9999996423721313, 4.2981449951184914e-05, 6.714119081152603e-06, 0.0003620173956733197, 0.9999743700027466, 0.9999995231628418, 0.999970555305481, 1.0, 8.464517264883398e-08, 1.4693483535666019e-05, 7.227663900266634e-06, 3.2725038181524724e-05, 0.9999998807907104, 0.9999842643737793, 1.933074997850781e-07, 0.003059346228837967, 0.9999996423721313, 4.3844883634847065e-07, 0.003522841027006507, 1.6602287322697862e-09, 1.0, 0.0032200191635638475, 5.802567102364264e-07, 1.0, 0.9998778104782104, 0.0005334721645340323, 1.1786094944454817e-07, 0.00043582680518738925, 0.001824002480134368, 4.5517335820477456e-05, 0.9999995231628418, 6.7313080762687605e-06, 1.981467789846647e-07, 1.084334712686541e-06, 1.0, 5.626122856483562e-08, 0.0005737872561439872, 0.9999487400054932, 1.0, 1.3563203538069502e-07, 4.277655989426421e-07, 5.456730832520407e-06, 0.0008739824406802654, 0.000506024225614965, 0.9999997615814209, 0.0025124181993305683, 0.9999997615814209, 0.9999997615814209, 2.396451964159496e-05, 0.0002570998331066221, 2.2959220302709582e-08, 1.0, 1.0, 0.9707509279251099, 8.94346521818079e-06, 0.9999982118606567, 0.00045926630264148116, 0.9999978542327881, 1.7311125702690333e-05, 0.9999195337295532, 1.0, 0.0008874688064679503, 1.0, 1.0932417353615165e-05, 1.6296377225444303e-07, 0.00013420036702882499, 0.9999980926513672, 0.0004373173287604004, 0.00014580668357666582, 0.999980092048645, 7.676260429434478e-05, 1.1688587875369194e-07, 8.444989362033084e-05, 1.5775705151099828e-06, 0.9999942779541016, 0.00038266912451945245, 1.0, 6.4493256104469765e-06, 0.9999986886978149, 1.0, 0.9999994039535522, 0.9998942613601685, 1.0, 0.9999946355819702, 0.0015579061582684517, 1.0, 0.9999988079071045, 1.0, 4.537116546998732e-05, 0.9999979734420776, 6.415255484171212e-06, 1.0, 7.530747552664252e-06, 0.00011706303484970704, 5.112904545967467e-06, 1.0, 8.347471157321706e-05, 1.0, 2.821591351676034e-07, 0.9994679093360901, 2.190014492953196e-05, 5.3831623517908156e-05, 0.0003995519655290991]

#Metricas de validación
Best_acc8_7 = 1.0
acc_aux8_7 = 1.0
Recall8_7 = 1.0
Especificidad8_7 = 1.0
Tiempo8_7 = 1584.4005863666534

######## Base9 ########
#MODELO = VGG19
#Datos perdidas
xx_train9_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train9_7 = [0.04028597154919012, 0.03837016767557756, 0.025654143766488557, 0.03209924063196889, 0.024658735160842354, 0.017936490641699897, 0.014826860380025557, 0.022268369563935714, 0.016376766617651337, 0.01321789010623355, 0.022473560135673594, 0.016314093777794896, 0.01631728689427729, 0.017593365052232034, 0.01925845131462003, 0.01768762555755215, 0.017376200928364272, 0.01397297810073252, 0.01930861885164991, 0.013530896585664632, 0.021403554698199403, 0.014885089849984203, 0.021448173981021952, 0.015450876675270222, 0.019068626728322772]
xx_val9_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val9_7 = [0.00313486655553182, 0.002579872144593133, 0.004786311255560981, 0.008739193777243297, 0.003743819809622235, 0.0034091095129648846, 0.003154188063409593, 0.0028834864497184753, 0.002688320974508921, 0.0028160780668258665, 0.002807631757524278, 0.0028102195925182767, 0.0028007141417927214, 0.002809760222832362, 0.0027705657813284134, 0.0027711199389563665, 0.0027710818582110937, 0.0027713595165146723, 0.002771307362450494, 0.002772318571805954, 0.0027723103761672975, 0.0027723270157972973, 0.0027723195652167004, 0.0027723302443822223, 0.0027723362048467]
epoch_min9_7 = 1
stop_val9_7 = 0.002579872144593133

#Datos accuracy
acc_train9_7 = [0.9842592592592593, 0.9845679012345678, 0.9901234567901235, 0.9864197530864197, 0.9916666666666667, 0.9919753086419753, 0.9959876543209877, 0.9932098765432098, 0.994753086419753, 0.9950617283950617, 0.9907407407407407, 0.9944444444444445, 0.9925925925925926, 0.9935185185185185, 0.9922839506172839, 0.9944444444444445, 0.9935185185185185, 0.9941358024691358, 0.9929012345679012, 0.9950617283950617, 0.9929012345679012, 0.9953703703703703, 0.9922839506172839, 0.9944444444444445, 0.9932098765432098]
acc_val9_7 = [1.0, 1.0, 0.9972222222222222, 0.9944444444444445, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels9_7 = [1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]
best_prob9_7 = [1.0, 0.9999998807907104, 1.084558107322664e-06, 0.999922513961792, 1.0, 0.9997484087944031, 1.0, 0.999998927116394, 0.004187760408967733, 1.8785716747515835e-05, 0.0002828036085702479, 0.998671293258667, 0.0021299507934600115, 1.0, 0.003593283472582698, 0.86692214012146, 0.0029363809153437614, 1.0, 0.9999997615814209, 0.9999998807907104, 1.0, 0.999951958656311, 1.586067446623929e-05, 0.010339553467929363, 3.0131038784020348e-06, 1.0, 0.9999998807907104, 1.0, 1.2159106290710042e-06, 0.005784108769148588, 1.3618031289297505e-06, 1.0, 0.0010501515353098512, 2.7179548851563595e-06, 0.0019884572830051184, 1.0, 4.6860866859788075e-05, 5.892399030926754e-07, 1.0, 0.021314669400453568, 6.66369828650204e-07, 1.0, 0.9999998807907104, 0.9999974966049194, 6.205282261362299e-07, 1.6466599106479407e-07, 0.9999563694000244, 6.126902873937468e-10, 4.775876277562929e-06, 0.0008868960430845618, 0.9999901056289673, 0.9955331087112427, 1.0, 1.0, 4.27544506464983e-08, 1.4220464436220936e-05, 0.9999685287475586, 0.00020213040988892317, 0.9998337030410767, 7.51203770050779e-05, 1.0, 3.623128463914327e-08, 1.0, 2.773624373730854e-06, 2.7437003780050873e-08, 1.0, 0.9999997615814209, 1.0, 1.0, 0.00012105054338462651, 0.00327317975461483, 0.9999994039535522, 1.0, 0.9986849427223206, 0.0002038102102233097, 0.9999995231628418, 0.0005638062139041722, 0.9999980926513672, 0.00037453093682415783, 0.0002396866329945624, 0.9999998807907104, 1.0, 1.0, 2.3071298471677437e-07, 0.9999978542327881, 0.9999912977218628, 0.0013363956240937114, 1.0, 1.0, 0.9999997615814209, 1.0, 0.9998792409896851, 0.9999998807907104, 1.0, 6.202754593687132e-05, 0.9999998807907104, 3.506767097860575e-05, 0.9999991655349731, 0.9999901056289673, 1.2126877663831692e-05, 0.9999998807907104, 6.0646943893516436e-05, 1.0, 1.0, 2.9024727155047003e-06, 0.0005677227745763958, 1.0, 0.9999998807907104, 1.0, 1.0, 0.005559457000344992, 0.9999998807907104, 0.0017202849267050624, 1.0, 0.0005761345964856446, 0.9999961853027344, 0.0064753140322864056, 1.0, 1.0, 2.6871355203184066e-06, 1.0, 0.004664518404752016, 9.172872523777187e-06, 0.9951063394546509, 0.0029839067719876766, 0.9182683229446411, 5.192227035877295e-06, 0.9994126558303833, 0.9999966621398926, 3.8568119634874165e-05, 0.9999986886978149, 7.687176548643038e-06, 5.336470621841727e-06, 0.999998927116394, 0.9999996423721313, 0.005681374575942755, 0.9999998807907104, 1.0, 8.258621164713986e-06, 2.6658361093723215e-05, 0.9999966621398926, 3.4137673310397076e-07, 0.00016315824177581817, 0.9999998807907104, 1.0699264807456643e-09, 1.692051228019409e-05, 1.0, 1.0, 1.149489344243193e-05, 1.0, 9.02815954759717e-05, 0.002334549790248275, 4.922724838252179e-05, 0.0007189177558757365, 0.9998410940170288, 1.7085774743463844e-05, 0.9999997615814209, 0.9999995231628418, 0.0001503972162026912, 0.9999977350234985, 1.0, 1.0, 1.0, 0.999778687953949, 0.9999887943267822, 1.3043773833487649e-05, 8.867842780091451e-07, 5.792757656308822e-05, 9.22810201586799e-08, 7.807364568179764e-07, 2.2150489442651633e-08, 0.9830939769744873, 1.0, 1.0, 1.0, 0.999944806098938, 0.9999998807907104, 1.0, 1.0, 5.1583094318630174e-05, 3.973426373704569e-06, 1.108970809582388e-05, 3.513662886689417e-05, 3.873252829578178e-09, 2.0089753434149316e-06, 9.469692486163694e-06, 0.9999984502792358, 1.6900288756005466e-05, 4.3103732423332985e-06, 2.1137271488669285e-08, 0.049997713416814804, 1.0, 0.9999967813491821, 0.02872130274772644, 5.332461569196312e-06, 0.9999545812606812, 1.1616622108689967e-09, 5.185390818951419e-07, 3.596000851757708e-06, 0.999990701675415, 0.0002695608709473163, 0.9999971389770508, 1.0, 0.9999994039535522, 1.0, 0.0031162570230662823, 1.0, 0.008458519354462624, 0.000100671750260517, 1.0, 0.9999969005584717, 0.0006218510679900646, 0.9999990463256836, 0.9999998807907104, 1.0, 1.0, 0.00032095733331516385, 0.0002900756662711501, 0.0010860777692869306, 1.4162429579300806e-05, 1.0, 0.9986218214035034, 0.006527868565171957, 2.234524436062202e-05, 0.0004037062462884933, 7.224787259474397e-05, 0.9999996423721313, 3.60345052285993e-06, 0.9920308589935303, 0.9999996423721313, 1.0, 1.0, 1.0, 0.9999362230300903, 1.2066806220900617e-06, 1.0, 7.318059488170547e-06, 2.1072960407764185e-06, 1.0, 0.15394547581672668, 2.680869556570542e-06, 0.00261938339099288, 0.000697151233907789, 7.92040877328759e-11, 6.137437594588846e-05, 0.9999996423721313, 9.410786333319265e-06, 1.0, 2.853859041351825e-05, 0.006259417161345482, 2.29858051170595e-05, 0.9999984502792358, 0.035071324557065964, 1.0, 1.0, 1.1913871276192367e-05, 0.00019575066107790917, 0.00011557111429283395, 0.9999966621398926, 1.0, 0.9999988079071045, 1.6005121779016918e-06, 1.0, 2.8807201957192774e-08, 0.0001423828216502443, 8.969361715571722e-07, 1.2251491170900408e-05, 0.9532507061958313, 1.0, 0.9999949932098389, 0.9999973773956299, 0.0012927207862958312, 0.9999971389770508, 8.833094398141839e-06, 0.003331861225888133, 3.922647522358602e-07, 1.0, 1.0, 0.00129081925842911, 0.005914134904742241, 2.795492491713958e-06, 3.856481271213852e-05, 2.4872360882000066e-06, 0.9999996423721313, 0.9974452257156372, 0.0001258895790670067, 1.0, 3.362102143000811e-05, 0.0025710121262818575, 1.0, 0.9264493584632874, 4.513167173314514e-09, 0.9999995231628418, 5.479351239046082e-05, 0.00012039933062624186, 2.684686819520721e-07, 3.647940047812881e-06, 0.9999995231628418, 0.00031123653752729297, 2.6568264729576185e-05, 0.00041937315836548805, 0.9999998807907104, 0.027812860906124115, 0.9999983310699463, 0.00027878108085133135, 8.03955572337145e-06, 7.149621524149552e-05, 0.9999996423721313, 0.007129903417080641, 0.9999775886535645, 1.915393994522674e-07, 1.1319145869492786e-06, 0.9999994039535522, 0.9999992847442627, 0.0002031335316132754, 4.305272582882935e-08, 0.0008193093235604465, 0.9999991655349731, 0.00012138216698076576, 0.9977685213088989, 0.9999998807907104, 0.9999933242797852, 1.0, 0.01237249281257391, 0.9999998807907104, 1.8255985196447e-05, 0.008984014391899109, 1.0, 0.9999997615814209, 0.9999982118606567, 1.0, 0.9999997615814209, 1.0, 1.0, 0.000638398400042206, 1.4983104392740643e-06, 1.1525659715516667e-07, 0.9999998807907104, 1.1477781214352945e-07, 0.0002723040815908462, 0.999993085861206, 1.0027883412533356e-08, 0.9999964237213135, 0.01564992032945156, 5.265025902190246e-05, 1.9021734260604717e-05, 1.6529501725703888e-11, 1.2564721146191005e-05, 0.0276377871632576, 1.0, 0.0032495635095983744, 1.0, 1.0, 0.9999997615814209, 0.9999732971191406, 6.76855847814295e-07, 1.1810185052141264e-09, 0.00022648792946711183, 0.9999998807907104, 0.999666690826416]

#Metricas de validación
Best_acc9_7 = 1.0
acc_aux9_7 = 1.0
Recall9_7 = 1.0
Especificidad9_7 = 1.0
Tiempo9_7 = 1613.445363521576

######## Base10 ########
#MODELO = VGG19
#Datos perdidas
xx_train10_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train10_7 = [0.03395296593691096, 0.02459376557373706, 0.04583073303471377, 0.02198579274577859, 0.024070220716573575, 0.01873936583230525, 0.015969686946015298, 0.013482335862922079, 0.019473959928677405, 0.019122143503692415, 0.014141288482480578, 0.018211562738374428, 0.010662916597024895, 0.020003967649406856, 0.01216380375402945, 0.016856627184667704, 0.014661909382284424, 0.020278509089976183, 0.014844170737045782, 0.016918969476296578, 0.016586619284417896, 0.01762953469598735, 0.019246932422673262, 0.018935660695956077, 0.016933390498161317]
xx_val10_7 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val10_7 = [0.0011239929331673516, 0.02524483816491233, 0.005913426809840732, 0.0006126453479131063, 0.0002832767036226061, 0.00023752782079908584, 0.000235646300845676, 0.00018091400464375813, 0.0001834730307261149, 0.0001965483029683431, 0.0001959946420457628, 0.0001957310570610894, 0.00019444690810309516, 0.0001946621470981174, 0.00019410583708021376, 0.00019416014353434245, 0.00019419127040439183, 0.00019423895412021212, 0.0001942469014061822, 0.00019423233138190376, 0.00019423365592956542, 0.0001942369672987196, 0.0001942382918463813, 0.00019423961639404298, 0.0001942402786678738]
epoch_min10_7 = 7
stop_val10_7 = 0.00018091400464375813

#Datos accuracy
acc_train10_7 = [0.9876543209876543, 0.9925925925925926, 0.9830246913580246, 0.9925925925925926, 0.9910493827160494, 0.9932098765432098, 0.994753086419753, 0.9953703703703703, 0.9938271604938271, 0.9916666666666667, 0.9944444444444445, 0.9938271604938271, 0.9953703703703703, 0.991358024691358, 0.9950617283950617, 0.9929012345679012, 0.9935185185185185, 0.9910493827160494, 0.9941358024691358, 0.9944444444444445, 0.9944444444444445, 0.9925925925925926, 0.9922839506172839, 0.9916666666666667, 0.9932098765432098]
acc_val10_7 = [1.0, 0.9888888888888889, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels10_7 = [1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]
best_prob10_7 = [0.9999974966049194, 1.0, 1.0, 9.635585911382805e-07, 0.00027127537759952247, 1.0, 1.0, 1.0, 3.777441861529951e-06, 1.4699528492201352e-06, 1.2114748093594585e-09, 5.347133264876902e-05, 4.042180989927147e-06, 1.8858308976632543e-05, 6.153834419819759e-08, 1.0, 1.0, 3.307916267658584e-05, 5.878260253666667e-06, 0.00042391178430989385, 2.049289378192043e-07, 1.0, 0.00014995246601756662, 0.004064216278493404, 0.999976634979248, 1.850707849371247e-05, 8.14377490314655e-05, 4.4789274511458643e-07, 0.9999998807907104, 0.0007557487697340548, 1.0, 0.9999301433563232, 9.330558532383293e-05, 0.0014884623233228922, 1.0, 1.6195035641430877e-06, 7.7913144195918e-05, 3.837795361505414e-07, 7.620465476065874e-05, 2.4517135898349807e-05, 1.0, 1.9557268387870863e-05, 1.0, 1.0, 2.0397352272993885e-05, 1.0, 5.105985110276379e-06, 0.9999997615814209, 1.0, 1.0, 1.0, 1.0, 1.0, 1.5384212019853294e-06, 1.6338579371222295e-05, 0.9999998807907104, 0.9999589920043945, 1.3358136129681952e-05, 1.0, 6.012708126945654e-06, 0.9999998807907104, 2.590919905287592e-07, 1.2765797691827174e-05, 1.0, 2.9960272513562813e-05, 0.9999974966049194, 2.9644245387316914e-06, 1.0, 1.0, 0.0005330248386599123, 0.9996135830879211, 3.9658361856709234e-06, 4.938091535677813e-08, 0.9999997615814209, 5.579833668889478e-05, 0.00017219925939571112, 0.00035665478208102286, 1.0, 1.5318055091029237e-07, 3.8972945759496724e-08, 1.0, 1.0, 0.000445576966740191, 1.0, 2.2067841598527593e-07, 1.0, 2.1482921397364407e-07, 9.801792657526676e-07, 0.9999971389770508, 0.00040009088115766644, 1.0, 0.9999998807907104, 1.0, 2.018568245887309e-08, 0.9999812841415405, 1.0, 1.0, 6.653208117768372e-08, 0.0001768327783793211, 1.0, 0.9998956918716431, 1.0, 0.9999992847442627, 0.9999984502792358, 1.0, 0.0010413447162136436, 2.6935549612971954e-06, 1.0, 0.9999998807907104, 1.0, 5.846148383170657e-07, 0.00014884288248140365, 0.00021219105110503733, 1.0, 1.0, 0.9999998807907104, 0.9999841451644897, 7.793247277732007e-06, 1.0, 0.9999996423721313, 1.3881179938834975e-06, 1.0, 0.008655954152345657, 1.0, 0.0001100793742807582, 4.758308205055073e-05, 3.4360434142399754e-07, 6.520144779642578e-08, 0.9999998807907104, 2.8389708859322127e-06, 1.0, 0.9999992847442627, 0.9999988079071045, 0.9999998807907104, 1.0, 9.674279681348708e-06, 0.012034029699862003, 2.020318788709119e-06, 4.27512122769258e-06, 7.427345138921737e-08, 1.0, 4.67931640741881e-05, 1.2827347291022306e-06, 0.9999998807907104, 1.0, 1.1943925015689416e-10, 1.0, 1.0, 5.947925956206745e-07, 1.1942591982005979e-06, 3.2533710054849507e-06, 1.0543198186496738e-05, 8.964975677372422e-06, 1.0, 1.0, 6.0580536228371784e-05, 0.9999995231628418, 0.9999992847442627, 1.0, 1.0, 1.0, 6.90225419930357e-07, 1.1688225640682504e-06, 1.0, 5.739666448789649e-06, 1.0, 1.5637508113286458e-05, 2.1825765372796013e-07, 1.0, 1.0, 2.5311759799251377e-08, 8.036711278691655e-07, 0.9978922009468079, 1.0, 2.7887824671779526e-06, 4.502543743001297e-05, 5.155862936589983e-08, 0.0005635356646962464, 1.0, 0.00023552399943582714, 1.0, 0.9999972581863403, 0.999727189540863, 1.0525931202209904e-06, 1.0, 1.0, 7.053386525512906e-06, 1.0, 0.9999961853027344, 1.0, 3.097649710070982e-07, 0.9999998807907104, 1.0, 1.0, 0.00015398145478684455, 4.5405831770040095e-05, 8.632814996190064e-08, 1.0, 1.0, 0.9999998807907104, 8.243518095696345e-05, 1.0, 1.0, 0.9999998807907104, 1.0, 1.0, 7.965409167809412e-05, 0.9999980926513672, 0.0004354773263912648, 9.748890988703351e-06, 6.102364977778052e-07, 0.9978653788566589, 0.003257577773183584, 1.0, 1.5646789961465402e-06, 0.0002796910412143916, 1.0, 1.0, 0.9999986886978149, 0.0009622509824112058, 1.5459218047908507e-05, 1.0, 4.622492997441441e-05, 0.9999985694885254, 0.9913626313209534, 1.0, 8.512469840127324e-09, 2.2483643533632858e-06, 1.0, 0.00016206694999709725, 2.1375192318373593e-06, 7.4648341978900135e-06, 1.792465445760172e-05, 3.978781251134933e-07, 4.937402991345152e-05, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9999998807907104, 2.853715663775347e-08, 0.9999998807907104, 0.0001423012581653893, 0.999997615814209, 1.0, 6.873679467389593e-06, 1.6184927517315373e-05, 1.0, 0.00011186909978277981, 0.0002706517407204956, 1.0, 9.991383365104411e-08, 1.0629739335854538e-05, 4.785127849515902e-09, 1.0, 8.4997336671222e-05, 8.072714990703389e-06, 0.9999992847442627, 7.905154234322254e-06, 0.9999991655349731, 0.9999992847442627, 0.00018753814219962806, 1.0, 1.5680728893130436e-06, 1.3189875971875153e-05, 0.9999998807907104, 1.168303788290359e-05, 1.0, 1.0, 0.00016241133562289178, 0.9999998807907104, 0.9999802112579346, 1.0, 5.1343029383588146e-08, 0.999993085861206, 0.0001233518705703318, 0.9999985694885254, 0.0002342751540709287, 8.265668043350161e-07, 3.970055331592448e-05, 7.467488671863975e-08, 1.0, 2.0233585019013844e-05, 1.0, 1.0, 2.8077701585971226e-07, 1.0, 1.7155237230781495e-08, 1.0, 4.271932141364232e-07, 1.0, 1.0, 1.734508441586513e-05, 1.0, 1.0, 6.280907109612599e-05, 0.00012863775191362947, 1.0, 7.611865294165909e-05, 1.9826684738433187e-09, 9.785484422764057e-09, 1.0, 1.0, 0.00021320258383639157, 1.0, 0.9999971389770508, 1.0275721962216267e-07, 1.0, 2.798130260828202e-08, 0.9999998807907104, 9.146949014393613e-05, 1.0, 0.9999998807907104, 1.5176912171455115e-08, 6.825589935033349e-06, 9.028922249854077e-06, 0.9999806880950928, 4.5696877350565046e-05, 2.9727467335760593e-05, 6.833438419562299e-06, 0.0014062175760045648, 1.0, 6.706555723212659e-06, 0.9999995231628418, 3.4080716204698547e-07, 0.00010576538625173271, 1.908835292852018e-05, 1.0, 1.0, 0.9999942779541016, 0.9999997615814209, 0.9938260912895203, 1.0, 1.0, 0.9999963045120239, 1.7699465004739068e-08, 0.0002122448495356366, 1.326281903857307e-06, 1.0, 2.3737652554700617e-06, 1.0, 1.4902921975590289e-05, 1.0, 0.0018190103583037853, 1.0, 0.00014982254651840776, 2.5651232135714963e-05, 2.048214264505077e-05, 1.574762653611117e-09, 1.0, 1.0, 5.548797707888298e-05, 5.827883796882816e-06, 0.9999998807907104, 1.0, 3.3942990285140695e-07, 0.9999908208847046, 5.915994361060939e-09, 4.3165266106370836e-05]

#Metricas de validación
Best_acc10_7 = 1.0
acc_aux10_7 = 1.0
Recall10_7 = 1.0
Especificidad10_7 = 1.0
Tiempo10_7 = 1615.5301134586334
###################################################################

######## Base1 ########
#MODELO = VGG19_BN
#Datos perdidas
xx_train1_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train1_8 = [0.22045375633018988, 0.1162921729831048, 0.07485564901137058, 0.06885774420735277, 0.06681471414588115, 0.0597622379200694, 0.056734227729432374, 0.047702288029738416, 0.04647669599931917, 0.04795512976469817, 0.05183488658548873, 0.04485827614495784, 0.051362383871534724, 0.04636289776291376, 0.04514776035959338, 0.05284962851876094, 0.04991966344324159, 0.04351628409859575, 0.04541958504252964, 0.04190127264570307, 0.04091566761518702, 0.052939473101754245, 0.04904333998759588, 0.03952718171072595, 0.053599178699063665]
xx_val1_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val1_8 = [0.10595741040176815, 0.05994939257701238, 0.042161900798479715, 0.07642704745133717, 0.04253838029172685, 0.04169990039534039, 0.03803307778305477, 0.04479717628823386, 0.0369800489809778, 0.03972662463784218, 0.03725438631243176, 0.03972846468289693, 0.04191054453452428, 0.034034291985962126, 0.03917060908344057, 0.04261873381005393, 0.04350138621197806, 0.04192354670829243, 0.041628148489528234, 0.036182089315520395, 0.035952283028099274, 0.03826534334156248, 0.03641746838887532, 0.03752080433898502, 0.042553539574146274]
epoch_min1_8 = 2
stop_val1_8 = 0.042161900798479715

#Datos accuracy
acc_train1_8 = [0.9098765432098765, 0.9564814814814815, 0.9725308641975309, 0.9740740740740741, 0.9722222222222222, 0.9762345679012345, 0.9796296296296296, 0.9814814814814815, 0.9833333333333333, 0.9824074074074074, 0.9808641975308642, 0.9833333333333333, 0.9808641975308642, 0.983641975308642, 0.9858024691358025, 0.9777777777777777, 0.9802469135802468, 0.9830246913580246, 0.983641975308642, 0.9848765432098765, 0.9848765432098765, 0.9777777777777777, 0.9817901234567901, 0.9845679012345678, 0.9765432098765432]
acc_val1_8 = [0.9611111111111111, 0.9777777777777779, 0.9833333333333334, 0.9722222222222222, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9833333333333334, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9833333333333334, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556, 0.9805555555555556]

#Datos curva ROC
best_labels1_8 = [1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]
best_prob1_8 = [0.9999926090240479, 0.9999433755874634, 0.999316930770874, 0.00012627201795112342, 0.1549254208803177, 0.9999977350234985, 0.0004153019399382174, 1.8566755898064002e-05, 0.0029482219833880663, 0.9869194626808167, 0.9996780157089233, 0.9998527765274048, 0.0071908701211214066, 7.125860429368913e-05, 0.0002449548337608576, 0.9999995231628418, 0.9999994039535522, 0.009994897060096264, 0.9999910593032837, 0.0014262546319514513, 1.0507524166314397e-05, 0.0002537871478125453, 6.16701363469474e-05, 0.9986400008201599, 0.9999825954437256, 0.9999408721923828, 0.0007194723002612591, 0.6193602085113525, 0.003240673104301095, 2.456588663335424e-05, 0.9999958276748657, 0.9999423027038574, 0.0003408969787415117, 0.9997928738594055, 0.00018252857262268662, 0.9973516464233398, 0.9999203681945801, 0.9998348951339722, 1.228835026267916e-05, 0.0038144683931022882, 0.9999366998672485, 0.9999901056289673, 0.00012690779112745076, 0.9868655800819397, 0.9999939203262329, 0.9999629259109497, 9.040246368385851e-05, 0.003107393393293023, 0.9999865293502808, 0.9996227025985718, 1.0, 0.0003786620800383389, 0.0005575130926445127, 0.9999706745147705, 0.9999959468841553, 0.08437028527259827, 0.9999518394470215, 0.9998811483383179, 0.00013045570813119411, 0.9997789263725281, 7.918508345028386e-06, 6.181790377013385e-05, 0.9997870326042175, 1.0, 0.16697651147842407, 0.001032673753798008, 0.999552309513092, 0.999997615814209, 0.0004790925595443696, 0.997988224029541, 0.9950071573257446, 0.9999058246612549, 0.9999704360961914, 2.0540122932288796e-05, 0.9791207313537598, 0.00019401843019295484, 7.888615800766274e-05, 3.757189551834017e-05, 0.9999990463256836, 0.01005258597433567, 1.2288841389818117e-05, 0.9999895095825195, 3.710616738317185e-06, 0.9995085000991821, 0.9999948740005493, 0.23100386559963226, 1.1506782357173506e-05, 0.0016654623905196786, 0.9949067234992981, 0.9991045594215393, 3.4162694646511227e-06, 8.290550613310188e-05, 0.00010366700007580221, 0.9977243542671204, 0.998234748840332, 2.016837606788613e-05, 0.9999741315841675, 0.9999911785125732, 0.9999864101409912, 0.999997615814209, 0.9999430179595947, 1.321778108831495e-05, 0.000274518271908164, 0.9999914169311523, 0.19822175800800323, 0.9999960660934448, 9.543593478156254e-05, 0.9999752044677734, 4.10208986068028e-06, 3.782805652008392e-05, 0.9999997615814209, 3.282781472080387e-05, 0.8410655856132507, 0.00012173176946816966, 0.9999896287918091, 3.542287231539376e-05, 4.720298238680698e-05, 6.786082394683035e-06, 1.0, 0.00026719350717030466, 0.9999939203262329, 0.9999927282333374, 0.9995936751365662, 0.030199408531188965, 0.9999929666519165, 0.0013166957069188356, 0.0005573448725044727, 0.9999310970306396, 0.0002251201367471367, 2.6628495106706396e-06, 0.9999895095825195, 0.019111817702651024, 0.0001737690035952255, 0.9818458557128906, 0.9999823570251465, 0.9981744289398193, 0.9999606609344482, 0.3792147636413574, 0.07003072649240494, 0.9999974966049194, 0.1438470482826233, 0.0009904640028253198, 0.987532913684845, 0.9999990463256836, 0.0014233298134058714, 0.00036606291541829705, 1.517550481366925e-05, 0.00016880141629371792, 0.9999920129776001, 0.9952830672264099, 0.9999063014984131, 0.0002197257272200659, 0.9979678988456726, 0.9867059588432312, 0.9999969005584717, 0.9999850988388062, 0.0007901742355898023, 0.37398162484169006, 0.010881748050451279, 1.688704469415825e-05, 0.982233464717865, 1.4797852600167971e-05, 0.999993085861206, 0.9999998807907104, 0.00193531543482095, 0.9999197721481323, 0.9999997615814209, 0.9997780919075012, 2.2889078536536545e-05, 0.0016652055783197284, 0.9689448475837708, 0.9999994039535522, 0.000356366770574823, 0.0018829511245712638, 0.9999741315841675, 0.9959152340888977, 0.9999854564666748, 0.0004984132829122245, 0.001047311699949205, 0.04999753087759018, 2.3804324882803485e-05, 0.999985933303833, 6.208280683495104e-05, 0.9999995231628418, 0.9999263286590576, 0.9964086413383484, 0.9999886751174927, 0.0008083804859779775, 0.9999336004257202, 0.9991808533668518, 0.9981520771980286, 1.5445188182638958e-05, 0.004188436549156904, 1.292856177315116e-05, 0.9909026026725769, 0.6816748380661011, 3.653121166280471e-05, 0.9998168349266052, 0.9999525547027588, 0.9998014569282532, 0.9965718984603882, 0.00013601014507003129, 0.9998561143875122, 2.448961095069535e-05, 0.999915599822998, 0.9999139308929443, 0.0031029479578137398, 5.336344474926591e-05, 0.001820044475607574, 1.2288841389818117e-05, 0.001003372948616743, 0.9978173971176147, 5.664477157552028e-06, 1.9397637515794486e-05, 0.00026639457792043686, 3.775269942707382e-05, 1.0034284059656784e-05, 0.9999911785125732, 0.00010164620471186936, 0.00017658948490861803, 0.9999951124191284, 0.9990953207015991, 0.0008588748169131577, 0.9999037981033325, 9.177868196275085e-05, 0.00021836237283423543, 0.9999388456344604, 0.0024578759912401438, 0.9997461438179016, 1.317146029577998e-06, 4.980101039109286e-06, 0.9999973773956299, 5.395126572693698e-05, 0.9998341798782349, 0.001480128150433302, 2.1418396499939263e-05, 0.9994267225265503, 0.00010048453259514645, 0.9991353154182434, 0.09314166754484177, 1.0, 3.208524867659435e-05, 0.03241622447967529, 0.9999998807907104, 0.9951884746551514, 0.00028092548018321395, 0.9986748695373535, 0.9999991655349731, 0.00040917019941844046, 0.00257520261220634, 0.9999902248382568, 0.00010148079309146851, 0.0006836489192210138, 0.9999573230743408, 0.00015783548587933183, 0.9885832071304321, 0.029792044311761856, 3.405994357308373e-05, 0.9999504089355469, 0.00026446697302162647, 5.542587587115122e-06, 0.9999558925628662, 0.9999963045120239, 0.0006523503689095378, 8.550470374757424e-05, 0.00035751861287280917, 5.638544098474085e-05, 0.011939696967601776, 0.04324885830283165, 0.9999827146530151, 0.9999998807907104, 4.7694378736196086e-05, 0.02257612906396389, 0.9999920129776001, 0.00010128193389391527, 0.018675940111279488, 0.0009020052384585142, 0.0011645234189927578, 0.9969684481620789, 0.9999876022338867, 0.04786831513047218, 0.9999765157699585, 0.9999274015426636, 0.999990701675415, 0.00045746355317533016, 0.9999910593032837, 0.9999686479568481, 3.269933586125262e-05, 0.00014266619109548628, 0.00019181756942998618, 0.9579830169677734, 0.0010449024848639965, 0.002677425742149353, 3.6427816667128354e-05, 0.9999868869781494, 9.700322698336095e-05, 0.9999996423721313, 2.671970105438959e-05, 0.9999629259109497, 0.0006451906519941986, 1.628674363018945e-05, 0.00017409304564353079, 0.9818527102470398, 0.0005552420043386519, 0.0006370546179823577, 0.9970807433128357, 0.9993059635162354, 0.9957255125045776, 0.3242437541484833, 0.0004228362231515348, 0.003798347432166338, 0.999994158744812, 7.698684930801392e-05, 5.3960680816089734e-05, 0.9999508857727051, 0.9973872303962708, 6.120904345152667e-06, 0.999859094619751, 0.9964967370033264, 9.115548891713843e-05, 0.9991820454597473, 0.002029889263212681, 0.9999915361404419, 0.9997864365577698, 0.9999994039535522, 0.00017860630759969354, 1.1721092960215174e-05, 0.9999996423721313, 0.9998347759246826, 0.9999948740005493, 1.5997171658455045e-06, 0.0005057047819718719, 0.9999985694885254, 0.9999942779541016, 8.770132808422204e-06, 0.9986491799354553, 0.00010757274139905348, 0.9999791383743286, 0.9999970197677612, 0.0001264416496269405, 0.018313685432076454, 0.999803364276886, 0.9999927282333374, 0.0012468303320929408, 0.9998382329940796, 0.9984055161476135, 0.9999808073043823, 0.00018628561520017684, 0.9999115467071533, 0.001011827145703137, 0.00011911418550880626, 7.405632641166449e-05, 0.999982476234436, 0.0007738493150100112, 0.2602459192276001, 0.9999995231628418, 5.081868948764168e-05, 0.0009039100841619074, 0.9946147799491882, 2.3758469978929497e-05]

#Metricas de validación
Best_acc1_8 = 0.9833333333333334
acc_aux1_8 = 0.9858024691358025
Recall1_8 = 0.9943181818181818
Especificidad1_8 = 0.9728260869565217
Tiempo1_8 = 1672.875634431839

######## Base2 ########
#MODELO = VGG19_BN
#Datos perdidas
xx_train2_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train2_8 = [0.09757824867595861, 0.06884668119895605, 0.05899848291351471, 0.049248510120827475, 0.04768755258417424, 0.04864611262340605, 0.03309683290160732, 0.03094719880157047, 0.03000290118801741, 0.03797648338440024, 0.035276918757109, 0.03421748407461025, 0.03078580223667769, 0.03560233941232717, 0.027419399663254067, 0.03470359151194125, 0.039241202074436494, 0.03313849470865579, 0.02934653831668842, 0.04023239888526775, 0.034019078231520124, 0.028729029956423208, 0.03703009534029313, 0.03237312299969756, 0.030723899317744337]
xx_val2_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val2_8 = [0.046151672965950435, 0.07605710559421115, 0.09217181321647432, 0.029205001559522416, 0.04331650386253993, 0.04544864164458381, 0.04223474578724967, 0.04100478556421068, 0.03917442229059007, 0.03522053609291712, 0.03729553752475315, 0.03929034719864528, 0.041509611573484206, 0.04034000751045015, 0.03700179590119256, 0.036643721494409774, 0.037025598684946696, 0.039173672596613564, 0.039088041914833914, 0.03912261972824733, 0.03620523081885444, 0.037058688120709525, 0.040658319493134816, 0.03530640262696478, 0.038643787966834174]
epoch_min2_8 = 3
stop_val2_8 = 0.029205001559522416

#Datos accuracy
acc_train2_8 = [0.9669753086419753, 0.9765432098765432, 0.9753086419753086, 0.9824074074074074, 0.983641975308642, 0.9839506172839506, 0.9888888888888888, 0.990432098765432, 0.9895061728395061, 0.9873456790123457, 0.987037037037037, 0.9879629629629629, 0.9891975308641975, 0.986111111111111, 0.9910493827160494, 0.9854938271604938, 0.9879629629629629, 0.9879629629629629, 0.9907407407407407, 0.9867283950617284, 0.9882716049382716, 0.9895061728395061, 0.9882716049382716, 0.9901234567901235, 0.9895061728395061]
acc_val2_8 = [0.9777777777777779, 0.9694444444444444, 0.9666666666666667, 0.9944444444444445, 0.9777777777777779, 0.9777777777777779, 0.9805555555555556, 0.9777777777777779, 0.9833333333333334, 0.9861111111111112, 0.9833333333333334, 0.9833333333333334, 0.9805555555555556, 0.9805555555555556, 0.9833333333333334, 0.9833333333333334, 0.9805555555555556, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9861111111111112, 0.9805555555555556, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334]

#Datos curva ROC
best_labels2_8 = [1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0]
best_prob2_8 = [0.9999164342880249, 0.9944544434547424, 0.9999866485595703, 0.00013554714678321034, 0.9999285936355591, 8.975933451438323e-05, 0.0010885844239965081, 1.0, 0.9999929666519165, 0.9999964237213135, 6.913236575201154e-06, 1.3401763681031298e-05, 0.9999971389770508, 0.9998884201049805, 9.413874067831784e-06, 0.999992847442627, 0.9999896287918091, 0.9991995692253113, 0.00017956802912522107, 4.9975733418250456e-05, 0.00023340409097727388, 1.0, 0.9998983144760132, 0.00020017943461425602, 0.00012268371938262135, 8.219594019465148e-05, 7.142017420846969e-05, 0.9998648166656494, 0.99998939037323, 0.0001220175254275091, 7.908244879217818e-05, 5.135702303959988e-05, 0.0017222464084625244, 7.844806532375515e-05, 0.9999921321868896, 0.9998964071273804, 0.9999616146087646, 0.0001088144927052781, 0.9934929013252258, 0.9999969005584717, 3.1504674552707e-05, 7.021253986749798e-05, 0.888656497001648, 0.007234858814626932, 0.9923250675201416, 0.9999955892562866, 0.9997053742408752, 0.0043964688666164875, 1.0, 0.0005167446797713637, 0.003949998412281275, 0.9999829530715942, 0.9999774694442749, 0.0011737897293642163, 0.9999995231628418, 0.0003036338312085718, 0.04140256345272064, 0.0013514243764802814, 0.9999804496765137, 0.9999915361404419, 0.4009295701980591, 0.9999995231628418, 0.9999663829803467, 0.999998927116394, 1.0, 0.9999794960021973, 0.9999998807907104, 1.2601067282957956e-05, 0.9998093247413635, 0.0020697491709142923, 0.00014115122030489147, 0.9999927282333374, 0.9993736147880554, 0.28592798113822937, 0.0002281431370647624, 0.9999979734420776, 0.9999970197677612, 0.13228020071983337, 0.9999797344207764, 1.0, 0.00015997323498595506, 0.9999450445175171, 0.9999967813491821, 0.9955922961235046, 0.99998939037323, 0.9999475479125977, 0.0004988351138308644, 1.9760191207751632e-05, 0.9999998807907104, 3.246416099500493e-06, 0.9999971389770508, 0.998460054397583, 0.9997361302375793, 0.0011707028606906533, 0.9999899864196777, 0.0006295431521721184, 3.4970755223184824e-05, 0.9999991655349731, 2.5797444322961383e-05, 0.00043145069503225386, 0.0001464912056690082, 0.003493240103125572, 0.0055558993481099606, 0.0007628033054061234, 1.0, 0.8192571997642517, 0.9999704360961914, 0.9999898672103882, 0.9999996423721313, 9.268784197047353e-05, 5.75291778659448e-05, 0.6080012917518616, 0.9987456798553467, 0.9999996423721313, 1.0, 0.999988317489624, 3.76433708879631e-05, 0.00393541669473052, 5.6534408940933645e-06, 0.9996008276939392, 1.0, 0.0007285677711479366, 0.9970617890357971, 0.9987149238586426, 0.9999990463256836, 9.458267595618963e-05, 0.0017054179916158319, 0.9984691739082336, 0.0017152054933831096, 0.9997963309288025, 0.999995231628418, 3.361130802659318e-05, 0.9999994039535522, 0.00016704080917406827, 0.0001854084403021261, 4.918117156194057e-06, 0.997642457485199, 0.9997699856758118, 4.408241147757508e-05, 3.5436387406662107e-05, 2.3542694179923274e-05, 0.00032766483491286635, 0.00012414221419021487, 0.9998281002044678, 0.9998313188552856, 1.7876836864161305e-05, 0.9999998807907104, 1.0171054782404099e-05, 8.512077329214662e-06, 0.9998810291290283, 0.9998015761375427, 0.00038305361522361636, 0.4688681364059448, 1.0, 1.9902599888155237e-05, 1.5064663784869481e-05, 1.1484111382742412e-05, 0.9999974966049194, 0.9999983310699463, 1.33884268507245e-05, 6.568381650140509e-05, 3.965037831221707e-05, 0.99993896484375, 0.999994158744812, 8.588656055508181e-05, 0.9959653615951538, 2.8493937861640006e-05, 7.088275651767617e-06, 1.6926949683693238e-05, 0.9999986886978149, 0.005607954226434231, 7.880060729803517e-05, 0.9999799728393555, 0.9996997117996216, 4.46366575488355e-05, 4.250592246535234e-05, 0.9999529123306274, 0.9995371103286743, 0.9988569021224976, 0.999981164932251, 4.5555989345302805e-05, 0.0025897754821926355, 0.00304043130017817, 0.0004183174460195005, 1.6476320752190077e-06, 0.9999943971633911, 0.00011043706763302907, 9.871517249848694e-05, 1.886872837530973e-06, 0.011346112936735153, 0.9949603080749512, 0.999998927116394, 0.999944806098938, 0.0023530651815235615, 1.4375455066328868e-05, 0.001126027898862958, 0.002091366099193692, 0.9999920129776001, 0.0015263232635334134, 0.9999477863311768, 5.621149102807976e-05, 1.5884412277955562e-05, 0.0001008835606626235, 4.8918485845206305e-05, 0.9995228052139282, 0.9999969005584717, 5.175807382329367e-05, 2.1477649170265067e-06, 2.0388108623592416e-06, 5.228111604083097e-06, 0.00015122235345188528, 0.999974250793457, 9.99055482679978e-05, 4.176918082521297e-05, 0.9993966817855835, 0.00020895260968245566, 0.9999994039535522, 0.008272985927760601, 0.9999487400054932, 2.545125244068913e-05, 0.00044823202188126743, 0.9757056832313538, 5.088679699838394e-06, 0.9958601593971252, 2.5066527086892165e-05, 0.9995083808898926, 0.0013537219492718577, 0.9999338388442993, 1.0, 0.9949221014976501, 1.0, 0.00254022772423923, 2.4333727196790278e-05, 7.688334153499454e-05, 0.9999868869781494, 9.220022548106499e-06, 0.9999427795410156, 0.9999961853027344, 0.9999998807907104, 0.9959852695465088, 1.4385989288712153e-06, 0.9999872446060181, 4.327328861108981e-05, 6.147524254629388e-05, 0.9999994039535522, 0.9958237409591675, 0.00018062308663502336, 0.22756628692150116, 0.006396782584488392, 0.9999980926513672, 1.0, 0.0001699619897408411, 0.9999319314956665, 0.0015254863537847996, 2.4928000129875727e-05, 1.0, 0.9999998807907104, 1.0, 1.0, 0.00018180669576395303, 4.210788392811082e-05, 0.9638344645500183, 0.0002903885906562209, 0.9998862743377686, 0.0006110842805355787, 0.9999852180480957, 0.9999997615814209, 3.965472933487035e-05, 0.9999954700469971, 0.9998934268951416, 0.9999760389328003, 1.2116564676034614e-06, 0.999099612236023, 0.9999840259552002, 0.1072000190615654, 0.9999223947525024, 0.9999990463256836, 4.434810307429871e-06, 1.3558533282775898e-05, 3.703297261381522e-05, 0.9998936653137207, 0.9997851252555847, 0.00012657386832870543, 0.9988983869552612, 5.221949322731234e-05, 0.0006981125916354358, 5.5832192629168276e-06, 0.4478819668292999, 0.9986409544944763, 9.327220027444127e-07, 0.00266560772433877, 2.2480733605334535e-05, 0.9968164563179016, 0.0001862193748820573, 1.0, 7.825225111446343e-06, 6.382415449479595e-05, 1.0, 5.852670801687054e-05, 0.999958872795105, 1.3625367500935681e-05, 0.999975323677063, 0.9906240105628967, 9.666164260124788e-06, 0.9999532699584961, 0.0019806213676929474, 0.00018383364658802748, 3.5710763768292964e-05, 0.00027241575298830867, 9.41836042329669e-05, 0.0016387273790314794, 0.00026419476489536464, 0.9967103004455566, 1.0, 0.99989914894104, 0.00022492663993034512, 0.00010629549797158688, 0.9999920129776001, 0.9926958084106445, 1.0, 3.8478074202430435e-06, 0.9999997615814209, 0.9999998807907104, 4.9171041609952226e-05, 0.899608850479126, 0.9999632835388184, 0.9999912977218628, 1.0, 0.0161073487251997, 6.058750386728207e-06, 7.099949289113283e-05, 0.9969114661216736, 0.2697443664073944, 5.3352552640717477e-05, 7.573852599307429e-06, 4.776542846229859e-05, 0.00018551136599853635, 0.996967613697052, 0.9999797344207764, 2.193260661442764e-05, 2.1957699573249556e-05, 5.54377174921683e-06, 0.999998927116394, 4.0995044400915504e-05, 0.999963641166687, 0.999963641166687, 0.9999988079071045, 0.9999958276748657, 0.9999998807907104, 3.304365600342862e-05, 0.9981174468994141, 0.9999815225601196, 0.9999991655349731, 0.0001190180191770196, 0.9995399713516235, 0.9999985694885254, 0.002012613695114851, 0.9999847412109375, 7.133935923775425e-06, 0.9999635219573975]

#Metricas de validación
Best_acc2_8 = 0.9944444444444445
acc_aux2_8 = 0.9944444444444445
Recall2_8 = 1.0
Especificidad2_8 = 0.989010989010989
Tiempo2_8 = 1675.5125241279602

######## Base3 ########
#MODELO = VGG19_BN
#Datos perdidas
xx_train3_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train3_8 = [0.07048052889697345, 0.0449417742682092, 0.04626859260929955, 0.03153571158095642, 0.04863181148230294, 0.03331930298864105, 0.030241091742559716, 0.026551378693109678, 0.029149613014341873, 0.030994290233026318, 0.030373028684177517, 0.03192482488574805, 0.024637569744645812, 0.02705679072274102, 0.02969513209881606, 0.026432394714635096, 0.03350223800466384, 0.02828515582246545, 0.03236151049718445, 0.03214849918896769, 0.030474696924656996, 0.028061045402729953, 0.025378121343659767, 0.027243620138845327, 0.031408373284855004]
xx_val3_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val3_8 = [0.013712078332901001, 0.01599572127064069, 0.020909490353531308, 0.01719670279158486, 0.014181965920660232, 0.010710034602218204, 0.011430069969760048, 0.009499108294645946, 0.009734188351366255, 0.008818689319822524, 0.00919395453400082, 0.009470833010143703, 0.009528578321139018, 0.009112381935119629, 0.009376058479150136, 0.010137631164656744, 0.009095313979519739, 0.009274581240283119, 0.010086281928751203, 0.00965827935271793, 0.009062308073043823, 0.00926648626724879, 0.009062114026811388, 0.009277685980002085, 0.009740988910198211]
epoch_min3_8 = 7
stop_val3_8 = 0.009499108294645946

#Datos accuracy
acc_train3_8 = [0.975, 0.9845679012345678, 0.9814814814814815, 0.9888888888888888, 0.9814814814814815, 0.9879629629629629, 0.9901234567901235, 0.990432098765432, 0.9898148148148148, 0.9891975308641975, 0.9888888888888888, 0.9891975308641975, 0.991358024691358, 0.9910493827160494, 0.990432098765432, 0.9919753086419753, 0.9873456790123457, 0.990432098765432, 0.9901234567901235, 0.9879629629629629, 0.9895061728395061, 0.9898148148148148, 0.9910493827160494, 0.9895061728395061, 0.9891975308641975]
acc_val3_8 = [0.9944444444444445, 0.9944444444444445, 0.9944444444444445, 0.9916666666666667, 0.9944444444444445, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222]

#Datos curva ROC
best_labels3_8 = [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0]
best_prob3_8 = [0.0004400635079946369, 0.00010348468640586361, 0.9999940395355225, 0.9999864101409912, 1.7662489426584216e-06, 0.00010659066902007908, 0.00017079377721529454, 0.0006318772793747485, 1.1525319678185042e-05, 0.147514209151268, 0.9999997615814209, 0.9999561309814453, 4.639283724827692e-05, 6.717565702274442e-05, 1.0, 0.999994158744812, 0.999996542930603, 0.9999228715896606, 0.9870098829269409, 6.623506487812847e-05, 0.999976634979248, 0.9999994039535522, 0.9999904632568359, 0.9996674060821533, 3.368983379914425e-05, 0.005785340443253517, 0.9999985694885254, 0.9999998807907104, 0.0004057201440446079, 7.767330680508167e-05, 0.9999685287475586, 0.9999290704727173, 0.9999433755874634, 2.1863190795556875e-06, 0.9999852180480957, 3.8897967897355556e-05, 0.9993922710418701, 0.00023105900618247688, 1.746960697346367e-05, 1.067265998244693e-06, 0.9872120022773743, 0.00010760433360701427, 0.008448934182524681, 0.000493751373142004, 0.9976727366447449, 0.999998927116394, 5.24144616065314e-06, 0.00012840134149882942, 0.9999681711196899, 0.009886223822832108, 1.8545484635978937e-05, 0.00023999005497898906, 0.9999946355819702, 0.9999849796295166, 5.275793682812946e-06, 1.0, 5.054301436757669e-05, 0.9994179010391235, 4.934781100018881e-05, 0.9999959468841553, 3.488578295218758e-05, 5.3527455747826025e-05, 2.0351963030407205e-05, 0.9997578263282776, 0.9994864463806152, 1.0, 0.00021116573770996183, 0.00011956437083426863, 1.7330188711639494e-05, 0.8294339776039124, 2.7911535653402098e-05, 2.221023169113323e-05, 6.72131500323303e-05, 5.824761046824278e-06, 9.715134365251288e-05, 7.199340325314552e-05, 0.8688910007476807, 0.9999793767929077, 0.9987022876739502, 1.2636789961106842e-06, 0.9999998807907104, 7.205549627542496e-05, 0.9999076128005981, 1.0, 6.152444984763861e-05, 0.9997331500053406, 6.747052975697443e-05, 0.9999978542327881, 0.6964374780654907, 9.482844234298682e-07, 0.9999793767929077, 0.007975580170750618, 5.553352480092144e-07, 0.00022801068553235382, 3.220925009372877e-06, 1.1626730156422127e-05, 0.9999946355819702, 0.00021929416107013822, 0.8475525379180908, 0.9999678134918213, 5.099656118545681e-05, 0.0022984116803854704, 0.00025315204402431846, 0.9998648166656494, 0.9999856948852539, 0.016451237723231316, 0.99960857629776, 2.5467665182077326e-05, 0.9999809265136719, 0.9479665160179138, 0.00015388707106467336, 0.00011688923405017704, 1.3664839570992626e-06, 8.43640009406954e-06, 0.00048745924141258, 0.002017757622525096, 0.9999997615814209, 0.0003730880271177739, 0.9999545812606812, 0.013994055800139904, 3.7823294405825436e-05, 0.9996776580810547, 4.914975579595193e-05, 0.0002927020541392267, 0.006300820037722588, 4.081553197465837e-05, 0.19365455210208893, 0.9999963045120239, 2.836588646459859e-05, 2.1597728846245445e-05, 3.440221189521253e-05, 0.9999983310699463, 7.60652546887286e-05, 0.9993232488632202, 0.9999990463256836, 0.9999799728393555, 6.850029876659391e-07, 0.9943272471427917, 0.9999969005584717, 1.2460634479793953e-06, 0.06932473182678223, 0.00017984664009418339, 5.608820174529683e-06, 2.350740396650508e-05, 6.856042455183342e-05, 1.1843229913210962e-05, 0.0011482320260256529, 0.999600350856781, 0.9999995231628418, 0.0007139296503737569, 0.9999985694885254, 0.9999982118606567, 0.9999949932098389, 0.9999970197677612, 0.0005855979979969561, 0.07647865265607834, 0.999998927116394, 0.9973430037498474, 1.300418261962477e-07, 9.674401371739805e-05, 0.9999693632125854, 0.9999996423721313, 0.9997795224189758, 0.999985933303833, 0.99988853931427, 1.0, 0.9999979734420776, 0.9999985694885254, 6.0478425439214334e-05, 0.0005415040068328381, 0.00020496467186603695, 8.753639122005552e-06, 0.9999991655349731, 3.7669478842872195e-06, 0.0011765535455197096, 0.999984622001648, 0.00022980714857112616, 9.860590580501594e-06, 0.00034737741225399077, 0.9999946355819702, 0.9998201727867126, 0.9990938901901245, 0.9999940395355225, 0.012196947820484638, 9.503399633103982e-05, 1.0438131994305877e-06, 7.505022040277254e-06, 0.9964764714241028, 7.492083113902481e-07, 0.9999840259552002, 0.9999610185623169, 0.00967094674706459, 1.0, 0.9998914003372192, 0.9999827146530151, 3.6558494684868492e-06, 0.01927269622683525, 1.0, 0.9999727010726929, 0.0002345932589378208, 0.0022635844070464373, 0.0002724596706684679, 0.0001418346946593374, 0.00011059481767006218, 0.0015536824939772487, 5.39552784175612e-05, 0.982346773147583, 6.170129927340895e-05, 0.0006291811587288976, 0.9999353885650635, 0.9999979734420776, 0.9999834299087524, 7.603660196764395e-05, 2.421543285890948e-05, 2.7039061478717485e-06, 0.011780274100601673, 4.224713484290987e-05, 5.047606464358978e-06, 1.5661791621823795e-05, 1.0, 0.0004056497709825635, 3.82017606170848e-06, 0.9999988079071045, 2.890454743464943e-05, 0.007011376786977053, 0.9999998807907104, 0.9995300769805908, 8.587421325501055e-05, 0.9998034834861755, 9.990068065235391e-05, 0.99998939037323, 0.9999760389328003, 0.9977502226829529, 0.9999996423721313, 8.325553608301561e-06, 0.9999938011169434, 0.9999945163726807, 0.9999756813049316, 0.033757396042346954, 0.9999831914901733, 0.9999997615814209, 0.00011479711974970996, 0.9999980926513672, 0.9999998807907104, 0.9999245405197144, 0.9999504089355469, 1.3739841961069033e-05, 3.3991517511822167e-07, 0.9999990463256836, 3.4582437820063205e-06, 0.9999997615814209, 0.9989281296730042, 6.293246406130493e-05, 0.004497756715863943, 0.0001045026074280031, 0.9995786547660828, 0.9999839067459106, 0.0007727587944827974, 0.9999998807907104, 0.9999344348907471, 0.017779815942049026, 0.0006230936851352453, 0.9999994039535522, 0.9999979734420776, 0.9998251795768738, 0.9999996423721313, 3.2653373637003824e-05, 0.9999998807907104, 0.9999316930770874, 0.9999898672103882, 0.9986250400543213, 0.9999703168869019, 0.004235796630382538, 0.005257848650217056, 0.999996542930603, 0.999993085861206, 7.486807589884847e-05, 0.9999986886978149, 6.147151452751132e-06, 0.9999783039093018, 3.489446680760011e-05, 0.00020479747036006302, 0.9999970197677612, 0.9999988079071045, 0.999976634979248, 0.9996069073677063, 0.999536395072937, 0.9999958276748657, 0.0001447011309210211, 4.872283170698211e-06, 0.9999829530715942, 0.9999967813491821, 1.673590850259643e-05, 0.9999657869338989, 0.9999891519546509, 4.5871693146182224e-05, 0.9999939203262329, 0.999860405921936, 0.9999945163726807, 0.9999983310699463, 7.425155490636826e-05, 1.6238941498158965e-06, 1.0, 0.9870858788490295, 0.99997878074646, 0.9998683929443359, 0.9998608827590942, 0.9999902248382568, 7.772310345899314e-05, 9.997873712563887e-05, 2.7248501282883808e-05, 0.99988853931427, 0.9998916387557983, 0.9999853372573853, 9.554283315083012e-05, 0.9998260140419006, 0.05761878564953804, 0.9997616410255432, 5.620655792881735e-05, 0.0004364312917459756, 0.9998471736907959, 0.9999997615814209, 1.6546771348657785e-06, 3.2516804822080303e-06, 0.9999957084655762, 1.0, 1.9523613445926458e-05, 1.0, 0.0002941034908872098, 0.9999990463256836, 0.99974125623703, 0.9961888194084167, 0.9999995231628418, 1.3009073427383555e-06, 0.9999996423721313, 0.9998387098312378, 0.9999200105667114, 1.5524135960731655e-05, 0.025656718760728836, 1.363663704978535e-05, 5.246482396614738e-05, 0.9999967813491821, 0.9999990463256836, 2.026865513471421e-05, 0.9997307658195496, 4.251070640748367e-05, 1.0, 0.99994957447052, 2.2785628971178085e-05, 0.9990943670272827, 0.9999969005584717, 0.002114807488396764, 1.0, 0.9999812841415405, 0.0002867400471586734, 2.6987279397872044e-06, 1.0, 1.6464854297737475e-06, 0.9999994039535522, 6.775496331101749e-06]

#Metricas de validación
Best_acc3_8 = 0.9972222222222222
acc_aux3_8 = 0.9972222222222222
Recall3_8 = 0.994475138121547
Especificidad3_8 = 1.0
Tiempo3_8 = 1673.4980652332306

######## Base4 ########
#MODELO = VGG19_BN
#Datos perdidas
xx_train4_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train4_8 = [0.2132573033556526, 0.10401759960769136, 0.08301514626285177, 0.06349819333832941, 0.05757269999127329, 0.048609395132020665, 0.04510169559974729, 0.0461826221074587, 0.0460767400301533, 0.05001802742481232, 0.04007651067258399, 0.040232937801399346, 0.051046073271168606, 0.04438923842009203, 0.04391098165953601, 0.042820001540728556, 0.04871599643870636, 0.04155544793164288, 0.041762986687230476, 0.04594193168076468, 0.045360841675673, 0.04532817343686834, 0.045940926799803605, 0.0410353979854672, 0.03638329206057537]
xx_val4_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val4_8 = [0.07874939342339833, 0.049426123665438755, 0.06081389718585544, 0.03904822725388739, 0.041471482233868706, 0.03620698849360148, 0.03478853735658857, 0.03364083460635609, 0.03247349965903494, 0.038914034432835046, 0.031200181775622896, 0.03769263418184386, 0.042031012641059025, 0.03255541755093468, 0.03136820693810781, 0.03336190978686015, 0.040181171066231196, 0.033857663141356575, 0.04493188311656316, 0.03491415646341112, 0.03708352380328708, 0.03183322921395302, 0.034925288293096754, 0.03337658643722534, 0.03783727735280991]
epoch_min4_8 = 10
stop_val4_8 = 0.031200181775622896

#Datos accuracy
acc_train4_8 = [0.9098765432098765, 0.9589506172839506, 0.9700617283950617, 0.9746913580246913, 0.9799382716049383, 0.9811728395061728, 0.9867283950617284, 0.983641975308642, 0.9833333333333333, 0.9808641975308642, 0.9873456790123457, 0.9848765432098765, 0.9817901234567901, 0.9851851851851852, 0.9820987654320987, 0.983641975308642, 0.9802469135802468, 0.983641975308642, 0.9851851851851852, 0.9814814814814815, 0.9842592592592593, 0.983641975308642, 0.9830246913580246, 0.9867283950617284, 0.9864197530864197]
acc_val4_8 = [0.9666666666666667, 0.9805555555555556, 0.9722222222222222, 0.9861111111111112, 0.9805555555555556, 0.9805555555555556, 0.9833333333333334, 0.9861111111111112, 0.9861111111111112, 0.9805555555555556, 0.9861111111111112, 0.9833333333333334, 0.9805555555555556, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9833333333333334, 0.9861111111111112, 0.9805555555555556, 0.9861111111111112, 0.9805555555555556, 0.9833333333333334, 0.9833333333333334, 0.9861111111111112, 0.9833333333333334]

#Datos curva ROC
best_labels4_8 = [0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
best_prob4_8 = [4.627102498488966e-06, 0.9870995879173279, 7.607859515701421e-07, 0.9988570213317871, 0.9999988079071045, 0.245941162109375, 1.0, 0.9999986886978149, 0.9999992847442627, 0.9997424483299255, 0.002754688961431384, 0.9970786571502686, 0.0001481023064116016, 0.9964704513549805, 0.9999889135360718, 0.9999961853027344, 0.9999185800552368, 0.0480237640440464, 0.0010397518053650856, 6.841444815108844e-07, 0.9999994039535522, 0.9998925924301147, 1.0888361430261284e-06, 0.00042685188236646354, 0.9999653100967407, 0.030011510476469994, 0.0026393975131213665, 0.004352590534836054, 0.9999995231628418, 0.9993118047714233, 0.0006637137848883867, 0.0023336925078183413, 0.9999194145202637, 0.003756855148822069, 1.0, 0.00047565679415129125, 0.9999922513961792, 0.9999983310699463, 0.00015180371701717377, 0.0001006072634481825, 0.9984593391418457, 0.0007649102481082082, 0.9998617172241211, 0.9999910593032837, 0.0007964054238982499, 0.9998823404312134, 0.00010815367568284273, 0.0002214296837337315, 2.0218094505253248e-05, 0.9999955892562866, 0.9999802112579346, 0.9999234676361084, 4.603215586485021e-07, 0.0035124109126627445, 0.9999996423721313, 0.9999995231628418, 0.9988196492195129, 0.9985246062278748, 0.9999771118164062, 2.8837384888902307e-06, 0.00045358523493632674, 0.2621481716632843, 1.2914964599985979e-06, 5.113730003358796e-05, 0.9999997615814209, 6.19010825175792e-06, 0.0007993360050022602, 0.9999874830245972, 0.00011330976121826097, 1.2468130080378614e-05, 0.9999990463256836, 0.32305842638015747, 0.03961189463734627, 1.3056763918939396e-06, 0.9993589520454407, 0.0003949922393076122, 0.9999986886978149, 0.9998219609260559, 6.068536322345608e-07, 0.983512818813324, 7.81010530772619e-05, 0.00018722368986345828, 0.9999731779098511, 0.9995114803314209, 1.3183361261326354e-05, 0.9991944432258606, 1.3974396324556437e-06, 0.014031984843313694, 0.9998589754104614, 1.5314200936700217e-05, 0.999998927116394, 0.9999996423721313, 1.6426163711003028e-05, 0.9997063279151917, 0.9999998807907104, 0.9999333620071411, 0.4117235541343689, 5.466298716783058e-06, 0.9999980926513672, 0.000315744400722906, 0.0004917943151667714, 0.9998552799224854, 0.9999986886978149, 0.9999810457229614, 0.9999468326568604, 0.9999841451644897, 4.651320523407776e-06, 0.007378429174423218, 0.9999215602874756, 0.00020565804152283818, 0.9940965175628662, 0.9999933242797852, 0.9999995231628418, 0.000619762169662863, 7.196525257313624e-05, 0.9999843835830688, 0.008402792736887932, 6.795326044084504e-05, 7.896753231761977e-05, 0.9999988079071045, 1.9653541585284984e-06, 1.176763998955721e-05, 2.7273226805846207e-05, 0.0032008506823331118, 0.9999971389770508, 1.569427695358172e-05, 0.9998403787612915, 0.0023688178043812513, 0.9999935626983643, 0.0014007221907377243, 0.0005456801154650748, 0.004833178594708443, 7.071853178786114e-05, 0.9999990463256836, 1.6823604482851806e-06, 0.9999998807907104, 0.004586712922900915, 0.9995937943458557, 8.860015077516437e-05, 0.9999986886978149, 0.00011798948980867863, 2.438019464534591e-06, 0.9999994039535522, 9.527808288112283e-05, 9.317363583249971e-05, 5.679059904650785e-05, 0.07027389854192734, 0.9999046325683594, 6.987559686422173e-07, 5.460234751808457e-05, 0.016673825681209564, 0.9999938011169434, 0.9999983310699463, 0.9665700197219849, 0.0020480831153690815, 4.1078164940699935e-05, 0.77939373254776, 1.6437177691841498e-05, 0.9999996423721313, 8.422746873293363e-07, 6.54532072985603e-07, 0.00015147693920880556, 0.8160061240196228, 0.0005758631741628051, 0.9984831213951111, 2.195822389694513e-06, 0.0008016668725758791, 0.9999943971633911, 0.9999842643737793, 0.9999614953994751, 0.9999936819076538, 0.00020341764320619404, 1.6411163983320876e-07, 8.873366459738463e-05, 0.9999998807907104, 0.00019951404829043895, 0.9999791383743286, 0.9999985694885254, 0.9999507665634155, 5.264128503768006e-06, 1.4754872609046288e-05, 2.0505987777141854e-05, 0.9999370574951172, 2.0501254311966477e-06, 0.00018447947513777763, 0.9992806315422058, 0.00023209367645904422, 1.0, 0.9999957084655762, 3.036035195691511e-06, 0.0022319124545902014, 0.00032582232961431146, 0.024425840005278587, 0.9999973773956299, 0.9999997615814209, 0.31919151544570923, 0.9999305009841919, 5.408976903709117e-06, 0.9999983310699463, 0.9999814033508301, 5.6669787227292545e-06, 0.999998927116394, 0.9999964237213135, 0.9999983310699463, 0.9999709129333496, 0.9999022483825684, 1.7124433725257404e-05, 0.9999427795410156, 1.6232438611041289e-06, 0.9999884366989136, 0.9999780654907227, 0.9999709129333496, 0.9999984502792358, 0.999754011631012, 0.9993570446968079, 9.190716809825972e-05, 0.0031655547209084034, 0.9997255206108093, 0.9999957084655762, 0.40032488107681274, 9.420181186214904e-07, 0.9999991655349731, 0.9999961853027344, 0.9999616146087646, 0.00014282720803748816, 8.293941391457338e-06, 0.9999897480010986, 0.002254628110677004, 0.00038049108115956187, 0.9999984502792358, 0.9986819624900818, 7.657171954633668e-05, 0.9940233826637268, 5.306086450218572e-07, 0.9999897480010986, 1.2747730579576455e-05, 0.047869615256786346, 0.6252679824829102, 0.9999810457229614, 0.9999923706054688, 6.623594345001038e-06, 0.9999994039535522, 1.0, 7.917572474980261e-06, 0.9999990463256836, 0.00025563716189935803, 0.0138643654063344, 0.00016002859047148377, 2.7861344733537408e-06, 1.149521085608285e-05, 0.9995166063308716, 0.9999988079071045, 0.9999475479125977, 0.9999884366989136, 8.988544323074166e-06, 0.0011694194981828332, 0.9999808073043823, 0.9301335215568542, 0.9999998807907104, 0.9999812841415405, 2.2736745449947193e-05, 0.00946620013564825, 2.7007718728100372e-08, 1.605078432476148e-05, 0.9999969005584717, 0.030281949788331985, 0.9999330043792725, 0.9999998807907104, 0.999161958694458, 0.0036111732479184866, 0.0017157201655209064, 0.9999996423721313, 0.9999840259552002, 0.0008996751857921481, 1.2872731531388126e-05, 0.9999438524246216, 0.6028687953948975, 0.00011228815856156871, 0.9999771118164062, 0.07855097204446793, 5.9707330365199596e-05, 5.6227789173135534e-05, 0.9999991655349731, 4.5278184757080453e-07, 0.0012123488122597337, 3.936422217520885e-06, 0.0006162325153127313, 0.04012428969144821, 1.332059696323995e-06, 0.9921433925628662, 0.9999986886978149, 1.723296918498818e-05, 0.004722007550299168, 0.9999998807907104, 6.322962144622579e-05, 0.0030993891414254904, 1.0, 1.5449504644493572e-05, 0.9999994039535522, 0.9998688697814941, 0.9999977350234985, 5.7234949053963646e-05, 0.9999879598617554, 1.0924140951829031e-06, 0.0006674897158518434, 0.9999876022338867, 0.13537512719631195, 0.9999892711639404, 0.005277407355606556, 0.9999293088912964, 0.00025129408459179103, 0.9999998807907104, 0.9999994039535522, 0.999922513961792, 0.0003762171254493296, 0.9999744892120361, 0.9999945163726807, 0.9999959468841553, 0.0008778629126027226, 0.9999955892562866, 0.0001760833984008059, 0.895229160785675, 1.1810881005658302e-05, 0.0003684424445964396, 0.9999932050704956, 0.9989487528800964, 0.00013317062985152006, 0.00016197242075577378, 0.0008405356202274561, 0.0004869302501901984, 0.9999971389770508, 0.008375006727874279, 0.999583899974823, 0.0008846977143548429, 0.9999743700027466, 0.0002751534339040518, 0.9999721050262451, 0.0008296804153360426, 1.2399949810060207e-05, 0.9999996423721313, 0.9998636245727539, 0.9999152421951294, 0.0014798905467614532, 0.000993445166386664, 0.9999169111251831, 3.2347190426662564e-05, 0.9999990463256836, 0.9999973773956299, 0.002872020937502384, 1.3638211385114118e-05, 0.9986754059791565, 0.9999985694885254, 0.9999997615814209, 0.0003133516584057361, 0.0001804049825295806, 0.9991425275802612, 0.0017234290717169642, 3.1171279260888696e-05, 8.609592623542994e-05, 0.00038093511830084026]

#Metricas de validación
Best_acc4_8 = 0.9861111111111112
acc_aux4_8 = 0.9873456790123457
Recall4_8 = 1.0
Especificidad4_8 = 0.972972972972973
Tiempo4_8 = 1926

######## Base5 ########
#MODELO = VGG19_BN
#Datos perdidas
xx_train5_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train5_8 = [0.08028363079568486, 0.05627579519778122, 0.041425068198162836, 0.04294522803874663, 0.0451382843616568, 0.04125824518225811, 0.04470831408728788, 0.03500937308978151, 0.03970384611575692, 0.028765763802292905, 0.03525606885920336, 0.029006069273124508, 0.030330244801294655, 0.03776143662355564, 0.03910812041089858, 0.03510128804195074, 0.04194875767937413, 0.036296752969056, 0.030142704167483766, 0.032441910080335756, 0.046092806627720966, 0.037785209319841716, 0.0382137491747185, 0.032489684214930474, 0.027503941217322407]
xx_val5_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val5_8 = [0.022930879890918732, 0.0031482997867796157, 0.007673044006029765, 0.37314600315358903, 0.004739362498124441, 0.004661139845848083, 0.004267686771021949, 0.0043471061521106295, 0.0054550283484988745, 0.0033011886808607315, 0.0033601098590426977, 0.004935090574953291, 0.004104850027296278, 0.003743913190232383, 0.0031161942415767244, 0.003647618823581272, 0.003898645771874322, 0.003861571682824029, 0.004478560056951311, 0.004363509350352817, 0.0032447727190123665, 0.005349256594975789, 0.002896306746535831, 0.0039882310562663606, 0.004661911891566383]
epoch_min5_8 = 1
stop_val5_8 = 0.0031482997867796157

#Datos accuracy
acc_train5_8 = [0.9746913580246913, 0.9771604938271605, 0.9848765432098765, 0.983641975308642, 0.9833333333333333, 0.9845679012345678, 0.9833333333333333, 0.9882716049382716, 0.9882716049382716, 0.9888888888888888, 0.9879629629629629, 0.9882716049382716, 0.9891975308641975, 0.9848765432098765, 0.9827160493827161, 0.9882716049382716, 0.9845679012345678, 0.986111111111111, 0.9879629629629629, 0.9882716049382716, 0.9814814814814815, 0.9858024691358025, 0.9854938271604938, 0.9898148148148148, 0.991358024691358]
acc_val5_8 = [0.9916666666666667, 1.0, 1.0, 0.8777777777777778, 1.0, 1.0, 1.0, 1.0, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9972222222222222, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels5_8 = [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
best_prob5_8 = [0.9999960660934448, 0.9999003410339355, 0.9849340915679932, 0.9899952411651611, 9.529652743367478e-05, 0.0007889955886639655, 0.00014308035315480083, 0.9999942779541016, 0.9891586899757385, 0.999649167060852, 0.9999750852584839, 0.9999926090240479, 0.999799907207489, 0.9999969005584717, 0.9999356269836426, 8.243881893577054e-06, 2.3269712983164936e-06, 0.001101026777178049, 4.026024544145912e-05, 1.754687900756835e-06, 0.0014078021049499512, 0.9916449189186096, 0.999822199344635, 0.9999823570251465, 0.9998456239700317, 0.00019646945293061435, 0.00019329000497236848, 0.9962665438652039, 4.982108293916099e-05, 0.9999850988388062, 2.298405661349534e-06, 0.9993177652359009, 0.004213653039187193, 0.9953386783599854, 2.262625639559701e-05, 0.9993253946304321, 0.9981377124786377, 0.9996335506439209, 0.999707043170929, 0.0007471363642252982, 0.9997774958610535, 0.0001510515867266804, 4.050878942507552e-06, 0.9999983310699463, 0.9999217987060547, 0.002408689586445689, 0.9988479614257812, 0.9999958276748657, 0.9999879598617554, 0.0018345514545217156, 0.9999332427978516, 0.9870294332504272, 0.9999719858169556, 0.9920459985733032, 0.9996744394302368, 0.012460028752684593, 0.9999984502792358, 0.00016017607413232327, 0.0011348029365763068, 0.999534010887146, 0.9924853444099426, 2.1674275558325462e-05, 0.0056936112232506275, 0.9990429282188416, 0.0011773090809583664, 0.09153416752815247, 0.0025753737427294254, 0.9780842661857605, 8.680582141096238e-06, 0.9998698234558105, 0.0001637027453398332, 0.0004736375412903726, 0.0003384742303751409, 0.00027818555827252567, 4.185370471532224e-06, 0.0001851896959124133, 0.9999823570251465, 0.9972309470176697, 0.999981164932251, 1.0103072781930678e-05, 0.001272723893634975, 0.0007227492751553655, 6.23564628767781e-05, 0.9991627931594849, 0.0020112425554543734, 5.285533006826881e-06, 8.856391650624573e-05, 0.00034055899595841765, 0.998817503452301, 0.9997406601905823, 0.9999960660934448, 6.959745951462537e-05, 0.9999845027923584, 2.8898614345962415e-06, 0.006601938046514988, 0.9987345337867737, 1.459537452319637e-05, 1.2010918908345047e-05, 0.9999995231628418, 0.9963114857673645, 0.00022228894522413611, 0.9999043941497803, 0.9967526197433472, 0.00022626839927397668, 0.0003012475499417633, 2.2985388568486087e-05, 0.9999877214431763, 1.86573488463182e-05, 0.03360996022820473, 0.9999632835388184, 0.9999701976776123, 1.5074723705765791e-05, 0.0036930388305336237, 0.9986880421638489, 0.9997894167900085, 0.9999995231628418, 0.00018612634448800236, 0.0003493741387501359, 0.999850869178772, 0.9999961853027344, 0.9999232292175293, 0.9999819993972778, 0.0004761262971442193, 0.005572629626840353, 0.9999767541885376, 0.9999960660934448, 0.9999737739562988, 0.9998337030410767, 0.999962568283081, 0.0010480087948963046, 0.014544570818543434, 6.604533155041281e-06, 0.9999303817749023, 7.93727085692808e-05, 0.9808323383331299, 1.874777808552608e-05, 0.0014586449833586812, 0.008946187794208527, 2.279799446114339e-06, 0.9994733929634094, 0.9998942613601685, 0.00019479643378872424, 0.999991774559021, 0.0010836750734597445, 0.999913215637207, 0.99993896484375, 0.9999294281005859, 0.999875545501709, 0.03495631366968155, 5.090043487143703e-06, 0.00012081058230251074, 0.8528834581375122, 0.0010175015777349472, 0.9998776912689209, 0.9999973773956299, 0.995797872543335, 0.0003954937565140426, 0.9072030186653137, 0.000559392268769443, 3.841667057713494e-05, 8.925268048187718e-05, 0.9992528557777405, 0.9990847110748291, 0.9996023774147034, 0.9997560381889343, 0.9996100068092346, 0.0009995682630687952, 6.5662017732393e-05, 0.9995077848434448, 2.5055076548596844e-05, 0.9991577863693237, 0.008470901288092136, 0.9969447255134583, 0.00023390956630464643, 0.9962173104286194, 3.9671299418841954e-06, 0.0048787547275424, 0.0017309183022007346, 0.00043890433153137565, 3.102577693425701e-06, 0.9998672008514404, 0.9906236529350281, 0.9994937181472778, 0.0003559372271411121, 3.4762902942020446e-05, 7.282523438334465e-05, 0.9999980926513672, 0.9991410970687866, 0.0022409120574593544, 0.999956488609314, 0.9998877048492432, 1.0, 0.9998626708984375, 0.0002770835708361119, 7.9160017776303e-06, 0.0006707521388307214, 0.9996438026428223, 1.793960677787254e-06, 0.9998440742492676, 0.9984464049339294, 0.00011277661542408168, 1.5957639334374107e-05, 0.012906383723020554, 3.892268068739213e-05, 5.165368929738179e-05, 0.0009935015114024282, 0.9999580383300781, 0.9999877214431763, 0.0001293504174100235, 0.9999819993972778, 0.0005105097079649568, 0.9999446868896484, 0.9992669224739075, 0.998584508895874, 1.2025968317175284e-05, 0.00016145940753631294, 0.9999985694885254, 0.9999966621398926, 0.0001558857038617134, 1.6264953956124373e-05, 0.9998570680618286, 6.721301906509325e-05, 0.9999885559082031, 0.01711967960000038, 0.00024263547675218433, 3.5029518130613724e-06, 6.016732731950469e-05, 0.005060245748609304, 5.295526352711022e-05, 0.00015100478776730597, 1.1596565855143126e-05, 7.184425157902297e-06, 0.9999792575836182, 0.9998809099197388, 0.011005818843841553, 0.0005280206678435206, 0.007997476495802402, 0.0005494640208780766, 1.3767194104730152e-06, 0.9999957084655762, 0.9999983310699463, 0.0002525633026380092, 0.9999932050704956, 0.9996384382247925, 0.0002777226036414504, 0.00250940746627748, 0.008710314519703388, 0.9998842477798462, 0.05015455558896065, 0.9993575215339661, 0.9997523427009583, 0.0009697583736851811, 0.9992579817771912, 0.9999949932098389, 0.9999387264251709, 0.9998772144317627, 2.8642007237067446e-05, 0.002004347974434495, 0.9983672499656677, 0.9979037046432495, 0.9988446235656738, 0.9999479055404663, 0.02435886487364769, 0.9999958276748657, 0.9924092292785645, 2.3670922018936835e-05, 4.3350650230422616e-05, 0.9997997879981995, 0.00017862996901385486, 0.0035700954031199217, 0.9999499320983887, 0.9999986886978149, 3.494408883852884e-05, 0.0005860192468389869, 0.0013684793375432491, 5.324000994733069e-06, 0.0003269621229264885, 0.9999616146087646, 0.9954894185066223, 0.9997965693473816, 0.9907134771347046, 0.999996542930603, 0.9999232292175293, 0.9999945163726807, 0.9999996423721313, 9.707106073619798e-05, 0.9387909770011902, 5.895809590583667e-05, 6.894944817759097e-06, 0.00029556857771240175, 0.9999929666519165, 0.8808286786079407, 8.986572538560722e-06, 9.218932063959073e-06, 0.9999849796295166, 3.227483603041037e-06, 3.1317635148297995e-05, 0.9999970197677612, 0.9999231100082397, 3.259528602939099e-05, 0.00043926932266913354, 1.2614490287887747e-06, 0.9999901056289673, 0.00021712499437853694, 0.0013409587554633617, 0.0019476402085274458, 0.9999988079071045, 6.2249404436443e-05, 0.9993759989738464, 0.00014145438035484403, 0.00018871304928325117, 5.673765190294944e-05, 0.9999958276748657, 8.514509772794554e-07, 0.9999998807907104, 4.910796178592136e-06, 0.00029861394432373345, 0.9995927214622498, 0.9999819993972778, 0.9973140358924866, 4.926655674353242e-05, 0.9999833106994629, 0.9921948909759521, 0.0003511927789077163, 0.9997082352638245, 0.0018620347836986184, 0.00023655494442209601, 0.9993357062339783, 0.9991428852081299, 0.00014065248251426965, 2.477213456586469e-05, 0.9993867874145508, 0.9999991655349731, 2.1061483494122513e-05, 0.9994670748710632, 0.0004604735295288265, 0.004163957666605711, 0.9999908208847046, 0.9999765157699585, 7.0236360443232115e-06, 0.9999992847442627, 0.9991068243980408, 0.9999141693115234, 0.9897550940513611, 0.9997833371162415, 0.9999743700027466, 0.00011940759577555582, 0.9990028738975525, 0.999992847442627, 0.9995681643486023, 3.586140155675821e-05, 0.9999998807907104, 0.0004220863920636475, 0.0001355450804112479, 0.002157937502488494, 0.9999631643295288, 0.0022114505991339684, 0.0007400826434604824, 0.00020576556562446058, 0.9999326467514038]

#Metricas de validación
Best_acc5_8 = 1.0
acc_aux5_8 = 1.0
Recall5_8 = 1.0
Especificidad5_8 = 1.0
Tiempo5_8 = 1901

######## Base6 ########
#MODELO = VGG19_BN
#Datos perdidas
xx_train6_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train6_8 = [0.06184200747513477, 0.047385863849410306, 0.04191962856753373, 0.03839901022521066, 0.04481139605244001, 0.034822464679126384, 0.02863676346562527, 0.032280490105902705, 0.02847897743369326, 0.02653938079321826, 0.025756924323461675, 0.02433394029000659, 0.028597859017274997, 0.02342743946262348, 0.02415309962299135, 0.03513658293235449, 0.025384637731828807, 0.02376678811363232, 0.027578353109183135, 0.024596575923539973, 0.02932647238543004, 0.037229131042589375, 0.0332735665878396, 0.03327627522341999, 0.020213177423050375]
xx_val6_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val6_8 = [0.020702173974778918, 0.052961365547445086, 0.011775652567545572, 0.022107462916109297, 0.005669963525401222, 0.005912950469387902, 0.006090082973241806, 0.006428698284758462, 0.005942554275194804, 0.005821079346868727, 0.005913352800740136, 0.006161885791354709, 0.006775764707061979, 0.005513141221470303, 0.005702407326963213, 0.007017735060718324, 0.005575401584307352, 0.005535141626993815, 0.005966035856140984, 0.0062766205933358936, 0.005555464989609188, 0.005303296446800232, 0.0070875979132122464, 0.005480112300978767, 0.007360733052094778]
epoch_min6_8 = 4
stop_val6_8 = 0.005669963525401222


#Datos accuracy
acc_train6_8 = [0.9799382716049383, 0.9808641975308642, 0.9851851851851852, 0.9867283950617284, 0.9827160493827161, 0.986111111111111, 0.990432098765432, 0.9864197530864197, 0.9895061728395061, 0.9895061728395061, 0.9901234567901235, 0.9901234567901235, 0.9898148148148148, 0.9907407407407407, 0.9907407407407407, 0.9867283950617284, 0.9922839506172839, 0.9910493827160494, 0.991358024691358, 0.9910493827160494, 0.9901234567901235, 0.9879629629629629, 0.9879629629629629, 0.9879629629629629, 0.9944444444444445]
acc_val6_8 = [0.9944444444444445, 0.9805555555555556, 0.9972222222222222, 0.9916666666666667, 1.0, 1.0, 1.0, 1.0, 0.9972222222222222, 0.9972222222222222, 1.0, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 1.0, 1.0, 0.9972222222222222, 1.0, 1.0, 1.0, 0.9972222222222222, 1.0, 0.9972222222222222]

#Datos curva ROC
best_labels6_8 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0]
best_prob6_8 = [0.005606015678495169, 2.5551693397574127e-05, 0.00011296389129711315, 1.5370852679552627e-06, 0.0014694350538775325, 0.28635352849960327, 0.9999868869781494, 0.9999990463256836, 1.7350195093968068e-06, 0.9997715353965759, 0.0032142638228833675, 0.9999508857727051, 0.9999880790710449, 0.9999407529830933, 0.0009004368330352008, 7.389942766167223e-05, 0.9999603033065796, 0.9996815919876099, 0.9999940395355225, 5.921404522268858e-07, 0.9999884366989136, 0.00026006766711361706, 2.9669545256183483e-05, 0.0001442121429136023, 3.3334774343529716e-05, 2.8163914976175874e-05, 0.005935552529990673, 0.0007687451434321702, 0.0623919852077961, 0.9998462200164795, 0.9999986886978149, 0.31845688819885254, 0.001038211747072637, 2.471927018632414e-06, 0.003891000524163246, 0.9987612962722778, 2.0964027669378993e-07, 8.140826139424462e-06, 0.9999948740005493, 0.9999862909317017, 8.310275006806478e-06, 0.9995865225791931, 8.151586712301651e-07, 3.102844857494347e-05, 2.6273775688423484e-07, 3.7279380194377154e-05, 8.796178735792637e-05, 0.9999998807907104, 0.9999992847442627, 0.999991774559021, 0.005323605611920357, 0.9999849796295166, 4.0098020690493286e-05, 0.0019664892461150885, 0.9999942779541016, 1.0, 0.0021771362517029047, 1.991406679735519e-05, 0.9999940395355225, 0.9989883303642273, 0.0014777297619730234, 0.0021091997623443604, 0.9999979734420776, 3.351499617565423e-05, 0.007433820515871048, 0.9999356269836426, 0.9999831914901733, 0.0003077284200116992, 3.971452679252252e-05, 0.00010577033390291035, 0.9999982118606567, 0.9996716976165771, 9.425637836102396e-05, 0.9999912977218628, 0.9999804496765137, 6.104145882090961e-07, 2.0485931599978358e-05, 5.069748658570461e-05, 2.7531121304491535e-05, 2.002839755732566e-05, 0.9999794960021973, 0.0002248886739835143, 0.999993085861206, 0.9997125267982483, 0.9999788999557495, 8.730028639547527e-06, 7.89523767252831e-07, 3.7233780858514365e-06, 0.002573522739112377, 7.1798635872255545e-06, 1.882373726402875e-05, 0.0005359988426789641, 0.000273341458523646, 0.9999995231628418, 0.9999998807907104, 5.198053258936852e-06, 0.0008967637550085783, 0.9999411106109619, 0.9999995231628418, 6.232799933059141e-06, 0.9993625283241272, 0.9997730851173401, 7.704840754740871e-06, 0.0002172409585909918, 0.9999774694442749, 0.9999992847442627, 0.9999479055404663, 0.0006179063348099589, 1.486903420300223e-05, 0.00025835202541202307, 0.9999290704727173, 0.999996542930603, 0.9999967813491821, 0.9999982118606567, 1.0, 0.9999991655349731, 0.9999643564224243, 0.9999955892562866, 2.2607253413298167e-05, 2.5710858153615845e-06, 0.0008442114922218025, 0.00014459286467172205, 0.999993085861206, 0.05961582064628601, 0.000858169631101191, 0.9999700784683228, 0.9999997615814209, 3.4980661439476535e-05, 0.9969047904014587, 2.3405198589898646e-05, 0.9999706745147705, 0.9999982118606567, 0.002195824636146426, 9.9156886790297e-07, 9.011596375785302e-06, 0.9998947381973267, 0.9999861717224121, 8.352679287781939e-06, 0.9999998807907104, 0.9999986886978149, 0.9999406337738037, 0.9998154044151306, 0.9999979734420776, 1.0, 1.0, 0.999991774559021, 6.585051596630365e-05, 0.9996528625488281, 5.2792751375818625e-05, 0.9898055791854858, 0.9999974966049194, 0.9999891519546509, 0.011008954606950283, 2.5270092010032386e-06, 0.9999983310699463, 0.9999297857284546, 2.1228022887953557e-05, 0.9999828338623047, 6.39913123450242e-05, 2.8129959900979884e-05, 0.9988215565681458, 0.2925730347633362, 0.9999933242797852, 0.9683027267456055, 0.9999997615814209, 8.055579201027285e-06, 0.9999988079071045, 0.9998873472213745, 0.9997825026512146, 1.2099718560421024e-06, 0.9999998807907104, 0.9999991655349731, 1.0028362567027216e-06, 0.0012070219963788986, 6.578454485861585e-05, 0.002301101805642247, 0.9999765157699585, 0.9999955892562866, 0.9999831914901733, 1.7657588614383712e-05, 0.9999953508377075, 0.00012586041702888906, 0.00028878316516056657, 0.0020354241132736206, 0.9999985694885254, 0.0028009323868900537, 0.999998927116394, 0.9998779296875, 0.9999995231628418, 0.9994220733642578, 0.0009879040298983455, 0.997725784778595, 1.8724051642493578e-06, 0.00046904137707315385, 0.9854962229728699, 0.9999996423721313, 2.9295777494553477e-05, 0.9999997615814209, 0.00012853746011387557, 0.9995457530021667, 0.00015407706086989492, 0.9999959468841553, 0.9998100399971008, 5.5222611990757287e-05, 0.9999967813491821, 0.9999974966049194, 1.0, 0.99953293800354, 0.002197445370256901, 1.0, 1.0, 0.9998641014099121, 0.0007586551946587861, 0.007553665433079004, 0.9999274015426636, 0.9999970197677612, 0.9999964237213135, 0.9999874830245972, 0.9969047904014587, 0.9999858140945435, 0.998005211353302, 1.0, 3.336037480039522e-06, 5.4990545322652906e-05, 0.9999988079071045, 1.7864298342828988e-06, 0.9998154044151306, 1.1345600796630606e-05, 0.9999943971633911, 5.9770786720036995e-06, 0.0011169842910021544, 9.216163562086876e-06, 0.006189401727169752, 5.052669393990072e-07, 5.674609383277129e-06, 0.9999998807907104, 0.00011553861986612901, 0.9997678399085999, 8.137361146509647e-05, 5.209800747252302e-06, 0.9999421834945679, 0.9999039173126221, 0.9999994039535522, 0.002981041092425585, 4.267309486749582e-05, 0.9999754428863525, 0.00011142738367198035, 0.0002597509592305869, 0.0002100616693496704, 4.9757286433305126e-06, 2.4589748136349954e-05, 0.9999990463256836, 0.9999996423721313, 0.9999747276306152, 0.9982688426971436, 0.0004729812790174037, 0.9999943971633911, 0.9987806677818298, 0.9999897480010986, 0.0001167130540125072, 1.0, 3.9602313336217776e-05, 0.00023345038061961532, 0.00010637824016157538, 0.9999758005142212, 0.008276536129415035, 0.9999958276748657, 0.0006116065778769553, 0.9913740754127502, 1.056558630807558e-06, 0.9611291289329529, 0.9999988079071045, 0.9999995231628418, 0.9999978542327881, 1.4102733985055238e-06, 0.0003923797921743244, 0.9999997615814209, 0.9999881982803345, 6.972155097173527e-05, 0.999984860420227, 0.9999997615814209, 0.9999932050704956, 0.0013724967138841748, 0.9999992847442627, 0.006322117988020182, 8.565692041884176e-06, 0.9999998807907104, 0.0005520630511455238, 0.9999388456344604, 0.9999991655349731, 1.0, 0.9990884065628052, 0.9999957084655762, 0.999992847442627, 0.9999904632568359, 0.00010663009743439034, 0.019517693668603897, 0.9807459115982056, 0.003489261958748102, 0.998808741569519, 0.0004185338912066072, 0.9999620914459229, 0.9999998807907104, 0.9991757273674011, 0.002331951865926385, 5.494176548381802e-06, 7.374271808657795e-05, 0.0035272494424134493, 0.9999979734420776, 5.304862952471012e-06, 0.0010355267440900207, 0.999991774559021, 5.377412435336737e-06, 0.002835603430867195, 0.0014137404505163431, 6.936906720511615e-05, 4.6094453864498064e-05, 4.279208496882347e-06, 0.002309922594577074, 0.00012222377699799836, 0.9999997615814209, 1.1471810239527258e-06, 0.9999904632568359, 4.88897567265667e-05, 0.9999997615814209, 0.00010748107160907239, 0.7298282980918884, 0.999800980091095, 6.556209700647742e-05, 0.9999995231628418, 1.644155418034643e-05, 0.9999643564224243, 0.9999995231628418, 0.00011286440712865442, 5.141950623510638e-06, 5.6610864703543484e-05, 0.9999938011169434, 0.9997608065605164, 0.9999804496765137, 0.9999995231628418, 0.0339294895529747, 0.0001458947081118822, 3.999875843874179e-06, 0.9999945163726807, 0.9988687634468079, 1.1160143003507983e-05, 3.089616438956e-05, 0.9998878240585327, 0.0001799125166144222, 0.9999990463256836, 0.8277918696403503, 0.9999978542327881, 4.660736885853112e-05, 7.694867963436991e-05, 0.00017722169286571443, 2.428487050565309e-06, 0.9999992847442627, 3.841969828499714e-06, 6.102345651015639e-05, 0.9998476505279541]

#Metricas de validación
Best_acc6_8 = 1.0
acc_aux6_8 = 1.0
Recall6_8 = 1.0
Especificidad6_8 = 1.0
Tiempo6_8 = 1914

######## Base7 ########
#MODELO = VGG19_BN
#Datos perdidas
xx_train7_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train7_8 = [0.22370003872079613, 0.10865913283677749, 0.07549030245454223, 0.0652539849465276, 0.05824548125634959, 0.04877820758172023, 0.05825895118860551, 0.037577810846729044, 0.047476664774211835, 0.03851692864188441, 0.04435016649004854, 0.043681146130885605, 0.039095540970195956, 0.04102917734855487, 0.0424921754517673, 0.04678485141492184, 0.04736791716681586, 0.042566189960933025, 0.0459248118010568, 0.049901327518401324, 0.04761041903201445, 0.0522919194565879, 0.0335065194302135, 0.05210080216696233, 0.050654858461132754]
xx_val7_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val7_8 = [0.059553775356875525, 0.04770076158973906, 0.04563976294464535, 0.03678876045677397, 0.032461526658799915, 0.030309720171822443, 0.03171558429797491, 0.03342402643627591, 0.03512588143348694, 0.03378569566541248, 0.030805094871256086, 0.03436685966120826, 0.034479834967189364, 0.03589640971687105, 0.037154834965864816, 0.03273234483268526, 0.03483797096543842, 0.034522694845994316, 0.03398183460036914, 0.03597624037000868, 0.03466030011574427, 0.02932819194263882, 0.03233856343560749, 0.03129636562532849, 0.03372724089357588]
epoch_min7_8 = 4
stop_val7_8 = 0.032461526658799915

#Datos accuracy
acc_train7_8 = [0.9074074074074074, 0.9601851851851851, 0.9725308641975309, 0.9765432098765432, 0.9802469135802468, 0.9824074074074074, 0.9777777777777777, 0.9876543209876543, 0.9820987654320987, 0.9885802469135803, 0.9851851851851852, 0.9830246913580246, 0.9873456790123457, 0.9858024691358025, 0.9845679012345678, 0.9851851851851852, 0.9814814814814815, 0.986111111111111, 0.9854938271604938, 0.9802469135802468, 0.9830246913580246, 0.9814814814814815, 0.9882716049382716, 0.9802469135802468, 0.9839506172839506]
acc_val7_8 = [0.9805555555555556, 0.9833333333333334, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9833333333333334, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9833333333333334, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112, 0.9861111111111112]

#Datos curva ROC
best_labels7_8 = [1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]
best_prob7_8 = [0.999996542930603, 2.8922577257617377e-05, 0.9964293837547302, 0.004095432348549366, 0.9994699358940125, 0.4706190526485443, 0.9995032548904419, 0.9999668598175049, 1.4205038496584166e-05, 0.02093653194606304, 0.9984662532806396, 0.9999388456344604, 5.882169716642238e-05, 0.002137047704309225, 2.7536845664144494e-05, 4.474522484088084e-06, 1.6034929330999148e-06, 0.0014588616322726011, 0.00041560427052900195, 0.9970977306365967, 0.9998797178268433, 7.110607111826539e-05, 4.874130172538571e-05, 0.9991594552993774, 0.9985552430152893, 0.9995726943016052, 0.00011492746125441045, 0.9999940395355225, 0.8213254809379578, 0.8730376958847046, 5.769818017142825e-05, 0.8129175901412964, 9.451775258639827e-05, 1.0, 0.0018473881063982844, 0.9997170567512512, 0.00035398403997533023, 0.9999561309814453, 5.4669915698468685e-05, 0.00047915539471432567, 6.159615622891579e-06, 0.1963520646095276, 1.7701582066820265e-07, 0.9998912811279297, 0.9976898431777954, 0.9987425208091736, 7.024467322480632e-06, 0.9998669624328613, 0.000505313219036907, 0.9995030164718628, 1.277327464777045e-05, 0.9977097511291504, 0.9999864101409912, 9.413020961801521e-06, 0.9997865557670593, 0.9988588094711304, 1.2659897038247436e-05, 0.9999966621398926, 1.0, 6.603323981835274e-06, 0.0005028032464906573, 9.66463971963094e-07, 3.2324249332305044e-05, 1.3409204257186502e-05, 0.9999222755432129, 1.0, 0.9999997615814209, 0.00015745533164590597, 0.00010792795364977792, 3.09033725898189e-06, 0.9989515542984009, 0.9999246597290039, 0.9982991814613342, 0.9999091625213623, 0.00047556660138070583, 0.9965561628341675, 1.8355440261075273e-05, 1.7730028503137873e-06, 0.00022280860866885632, 0.9982494711875916, 0.0004947866545990109, 0.013756358996033669, 0.9999562501907349, 1.4993653394412831e-06, 3.371040293131955e-05, 0.9995286464691162, 0.009006495587527752, 0.9996077418327332, 0.9998811483383179, 0.9999701976776123, 0.0006626108661293983, 0.9999315738677979, 0.8967541456222534, 0.9998237490653992, 0.9993730187416077, 0.9982295632362366, 5.70335259908461e-07, 1.3221234439697582e-05, 0.9997465014457703, 0.9999997615814209, 1.2874612309587974e-07, 0.9999862909317017, 2.4408725948887877e-05, 0.0060922992415726185, 0.9999996423721313, 0.0013984683901071548, 1.0, 0.00014642627502325922, 9.026574844028801e-05, 0.9999030828475952, 0.998045802116394, 0.9883515238761902, 0.9999561309814453, 7.964383985381573e-05, 1.3444530850392766e-05, 0.9999654293060303, 0.9999833106994629, 0.26875245571136475, 0.9924112558364868, 0.00038032353040762246, 5.009047072235262e-06, 0.9992812275886536, 1.0419514637760585e-06, 0.0004474913002923131, 0.0004365074564702809, 0.9999997615814209, 0.00016585570119787008, 0.9992936849594116, 1.0, 0.9981849789619446, 0.0001173357231891714, 1.336759396508569e-06, 1.0321375157218426e-05, 0.9999796152114868, 0.9999995231628418, 0.0031720674596726894, 0.9966821074485779, 0.9999967813491821, 6.112392838986125e-06, 1.318451813858701e-05, 0.00018036046822089702, 0.9981458187103271, 0.9992033839225769, 0.00013573042815551162, 0.000671745277941227, 2.0637862689909525e-06, 0.9998337030410767, 0.00042279029730707407, 0.999996542930603, 0.9666069149971008, 0.00034165167016908526, 0.9998303651809692, 1.5478988188988296e-06, 0.9999955892562866, 0.0014575120294466615, 0.9893078207969666, 0.9975084066390991, 0.9999638795852661, 0.006644202396273613, 0.9999099969863892, 0.012917286716401577, 0.9999992847442627, 0.9984986782073975, 0.99921715259552, 0.9999998807907104, 0.00012651499127969146, 0.00016444145876448601, 0.00019546621479094028, 0.9995589852333069, 0.999996542930603, 1.1114345397800207e-05, 0.9999905824661255, 7.946331606945023e-06, 0.014708761125802994, 0.9997820258140564, 0.9999988079071045, 0.00016691265045665205, 0.9999548196792603, 0.0016070336569100618, 0.000771213963162154, 0.997591495513916, 1.0, 0.9999146461486816, 1.6720849089324474e-05, 0.0003214396128896624, 0.9992546439170837, 2.538738499424653e-07, 0.9999265670776367, 0.0008397954516112804, 0.0001627582823857665, 0.9975723624229431, 2.652774128364399e-06, 0.9994818568229675, 0.7941712141036987, 6.174437203299021e-06, 8.833322681311984e-06, 0.9918069243431091, 0.9999890327453613, 0.9999388456344604, 0.9999462366104126, 0.9998451471328735, 1.6554582543903962e-05, 0.9999680519104004, 0.5232516527175903, 1.4949212527426425e-05, 0.2994036376476288, 1.8486206272427808e-06, 2.2590445951209404e-05, 3.7362151488196105e-06, 0.04595056548714638, 0.9999426603317261, 0.9998700618743896, 1.0, 0.9999997615814209, 0.7343482971191406, 1.6115411654027412e-06, 0.9999971389770508, 0.00019140192307531834, 0.9997738003730774, 0.9999732971191406, 0.9998021721839905, 0.9999513626098633, 0.9999973773956299, 0.00842189695686102, 0.9999939203262329, 0.9999979734420776, 0.9999856948852539, 0.00011935534712392837, 0.9998212456703186, 3.388539880688768e-06, 7.207600901892874e-06, 0.9957602620124817, 0.00023623983724974096, 2.6593575057631824e-06, 0.9999994039535522, 0.9999998807907104, 2.0386247342685238e-05, 8.811379302642308e-06, 0.9646745324134827, 5.325118308974197e-06, 1.2852143527197768e-06, 4.0275740502693225e-06, 4.405576873978134e-06, 0.9316124320030212, 0.0005562319420278072, 0.0009110884857363999, 8.106119821604807e-06, 0.9998262524604797, 3.1872900763119105e-07, 0.9998916387557983, 0.13825920224189758, 0.9993519186973572, 0.9999542236328125, 7.126009586500004e-05, 0.0006460770382545888, 2.4019180273171514e-05, 2.4763417059148196e-06, 0.9999514818191528, 2.466495061526075e-05, 0.9998760223388672, 0.999998927116394, 3.7586119105981197e-06, 0.9990781545639038, 0.9997072815895081, 0.9999890327453613, 0.9997736811637878, 1.3991802916279994e-05, 0.9999594688415527, 0.017862001433968544, 0.0024915910325944424, 0.9999954700469971, 1.1757047104765661e-06, 0.9999980926513672, 0.0002918699465226382, 0.9999948740005493, 2.790599637592095e-06, 0.9999855756759644, 7.739961802144535e-06, 6.889743235660717e-05, 6.028802772561903e-07, 0.9999316930770874, 0.9990440011024475, 0.9999479055404663, 0.00041631405474618077, 0.9999997615814209, 6.981138085393468e-06, 0.9858437776565552, 0.0004127988941036165, 3.2933312468230724e-06, 0.9976617097854614, 0.9999982118606567, 0.9969064593315125, 0.9683590531349182, 0.004445496015250683, 0.0004440253251232207, 4.1501683881506324e-05, 0.999757707118988, 0.9999816417694092, 0.9997594952583313, 0.002669600071385503, 0.9999362230300903, 6.788784230593592e-05, 0.002141582779586315, 0.9997653365135193, 0.00010599992674542591, 0.9999887943267822, 0.9993860721588135, 0.0004295069957152009, 1.0731647307693493e-05, 3.703445690916851e-05, 0.9997350573539734, 3.404153176234104e-05, 0.9999997615814209, 0.0007497961632907391, 0.9999998807907104, 4.293933216104051e-06, 0.9996053576469421, 0.9999862909317017, 0.0001788817171473056, 0.9999972581863403, 0.9996106028556824, 6.087919246056117e-05, 1.931066208271659e-06, 0.006259391084313393, 1.9727549727122096e-07, 0.9999876022338867, 0.00013837053847964853, 6.657778328644781e-08, 0.9993589520454407, 0.9999876022338867, 0.9999037981033325, 0.9610645771026611, 0.9999169111251831, 0.9996404647827148, 0.00534970685839653, 0.9999994039535522, 0.9996707439422607, 0.9999986886978149, 8.434020128333941e-05, 4.494083896133816e-06, 1.0, 0.0006444141617976129, 0.001475167227908969, 5.6545366533100605e-05, 9.076776768779382e-05, 0.0005611585220322013, 0.9999977350234985, 0.9991804957389832, 0.9999967813491821, 3.3678334148135036e-05, 0.9980746507644653, 0.9999885559082031, 1.4201650628820062e-06, 8.722353754819778e-07, 0.001435658661648631, 0.0016200675163418055, 0.9036950469017029, 0.9997859597206116, 0.0005795722245238721, 0.003297076327726245]

#Metricas de validación
Best_acc7_8 = 0.9861111111111112
acc_aux7_8 = 0.9885802469135803
Recall7_8 = 0.9781420765027322
Especificidad7_8 = 0.9943502824858758
Tiempo7_8 = 1907.9098238945007

######## Base8 ########
#MODELO = VGG19_BN
#Datos perdidas
xx_train8_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train8_8 = [0.08782183229003424, 0.04663191874638016, 0.05008712648241608, 0.04784102293628233, 0.05189311938521303, 0.03989086522732252, 0.0311319026498147, 0.03330825135847668, 0.0304474738277035, 0.03268041675105507, 0.0347410286742228, 0.034398667955840075, 0.03572075753300278, 0.025692073264975607, 0.032655222447198116, 0.03153798153370987, 0.04082396291656259, 0.024569239596157897, 0.03261960995050124, 0.0377164125074575, 0.03393811486017557, 0.039852968566579584, 0.03653290968986205, 0.04570074529375559, 0.03493956532099365]
xx_val8_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val8_8 = [0.010637996262974209, 0.025709745619032116, 0.007123994496133592, 0.0096400100323889, 0.0032906575335396662, 0.003373531334929996, 0.0043895994623502094, 0.003960413071844313, 0.0031698183053069646, 0.005153636965486739, 0.0052260536286565995, 0.004394561880164676, 0.004547597799036238, 0.0051403330432044135, 0.003876339395840963, 0.006175579461786482, 0.004168085091643863, 0.004813991155889299, 0.004444195164574517, 0.004258705841170417, 0.00389872541030248, 0.004285250935289595, 0.0038993000984191895, 0.0040080164869626365, 0.005783817834324307]
epoch_min8_8 = 4
stop_val8_8 = 0.0032906575335396662

#Datos accuracy
acc_train8_8 = [0.9682098765432099, 0.9839506172839506, 0.9805555555555555, 0.9830246913580246, 0.9799382716049383, 0.9830246913580246, 0.9873456790123457, 0.9895061728395061, 0.9882716049382716, 0.9888888888888888, 0.9876543209876543, 0.9864197530864197, 0.9873456790123457, 0.991358024691358, 0.9873456790123457, 0.9901234567901235, 0.9845679012345678, 0.9922839506172839, 0.9885802469135803, 0.9851851851851852, 0.9879629629629629, 0.9879629629629629, 0.9867283950617284, 0.9827160493827161, 0.9891975308641975]
acc_val8_8 = [0.9972222222222222, 0.9916666666666667, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 1.0, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9972222222222222]

#Datos curva ROC
best_labels8_8 = [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0]
best_prob8_8 = [0.9999663829803467, 0.00013372913235798478, 7.277172244357644e-06, 1.286110659748374e-06, 3.1400268198922276e-06, 2.9716722565353848e-05, 0.999962568283081, 0.00015051414084155113, 4.044372690259479e-05, 1.0, 0.9999940395355225, 0.9999103546142578, 1.1402525160519872e-05, 1.608761158422567e-05, 0.9999915361404419, 0.0008027591975405812, 0.9999716281890869, 5.8871915825875476e-05, 0.0013202845584601164, 0.0003552327980287373, 0.9995760321617126, 0.9970771074295044, 0.9999483823776245, 0.999931812286377, 0.00017269005184061825, 0.9999743700027466, 0.9906730055809021, 0.9999951124191284, 0.999994158744812, 0.001511761685833335, 0.9997245669364929, 0.009603754617273808, 1.0, 0.9999970197677612, 0.000112083725980483, 0.0011140805436298251, 0.9999750852584839, 1.2108918781450484e-05, 1.0, 0.9999992847442627, 0.99978107213974, 5.1247239753138274e-05, 0.9975033402442932, 0.9998916387557983, 0.9999563694000244, 3.386067692190409e-05, 0.00011680968600558117, 4.594028268911643e-06, 0.9999799728393555, 3.9899416037769697e-07, 0.9996219873428345, 7.944797835079953e-05, 1.0, 0.9999854564666748, 0.00019899690232705325, 0.9982640147209167, 6.614228914259002e-05, 0.9999923706054688, 1.858507800989173e-07, 0.9999957084655762, 0.9999998807907104, 0.9999294281005859, 0.999403715133667, 0.006923337001353502, 1.770169365045149e-05, 0.9998708963394165, 0.9999984502792358, 0.0006845231982879341, 0.9999325275421143, 0.9999961853027344, 0.9999934434890747, 0.0005306233069859445, 0.9999463558197021, 5.376515218813438e-06, 0.00021838278917130083, 0.00022034882567822933, 0.9999713897705078, 2.2623728455073433e-06, 5.5029151553753763e-05, 0.00044530967716127634, 0.009099326096475124, 0.00020300960750319064, 0.9998175501823425, 9.353096538688987e-05, 0.00010499614290893078, 7.079223451000871e-06, 0.999872088432312, 8.547485776944086e-06, 0.001568712992593646, 1.0, 0.9999988079071045, 0.9999998807907104, 0.00024145162024069577, 0.012171110138297081, 4.408543827594258e-05, 0.9999828338623047, 0.9999932050704956, 0.9998798370361328, 0.9528428316116333, 0.9999998807907104, 0.9999949932098389, 0.9999343156814575, 0.9999291896820068, 5.463979687192477e-05, 0.9997614026069641, 0.0007184733403846622, 0.999998927116394, 0.9996013045310974, 0.9768862128257751, 1.5932851965771988e-05, 0.9999791383743286, 0.9999877214431763, 7.477981125703081e-05, 0.0001044599775923416, 5.462885746965185e-05, 1.4242818906495813e-05, 0.9999998807907104, 0.00021868613839615136, 0.9999980926513672, 0.0009488345240242779, 0.9579556584358215, 0.9998619556427002, 0.004336335230618715, 0.9999827146530151, 0.00011584574531298131, 0.9993777275085449, 0.0101633220911026, 0.0004249591438565403, 9.266866072721314e-06, 9.984705684473738e-05, 5.248378874966875e-05, 0.12912367284297943, 0.00018496964185032994, 2.448056193316006e-06, 1.376494878968515e-06, 0.9988223910331726, 0.9999979734420776, 0.9994065761566162, 0.9999998807907104, 0.9999935626983643, 0.0022611485328525305, 1.3134595064911991e-06, 3.501550600049086e-05, 0.00015822335262782872, 0.9999992847442627, 0.9999988079071045, 0.9944778680801392, 0.0010956398909911513, 0.9999904632568359, 0.002001074142754078, 2.2105259631644003e-06, 0.00018974111299030483, 0.9996572732925415, 0.9999988079071045, 4.423152131494135e-05, 0.0004296452389098704, 0.999976396560669, 0.9999942779541016, 0.9999703168869019, 0.9999970197677612, 0.9999963045120239, 0.9970971345901489, 0.999998927116394, 6.81821329635568e-05, 0.00021888785704504699, 1.8486120097804815e-05, 6.376240344252437e-05, 0.9999978542327881, 0.9999945163726807, 0.9999698400497437, 0.9999926090240479, 6.036699051037431e-05, 0.9999701976776123, 0.9999843835830688, 7.74479815390805e-07, 1.1850211194541771e-05, 0.9966885447502136, 6.271059191931272e-06, 1.0, 0.0029758657328784466, 0.9999973773956299, 0.9990239143371582, 6.43169623799622e-05, 0.999983549118042, 5.959220288787037e-05, 0.9999988079071045, 0.0004893498262390494, 0.9999970197677612, 7.184851256170077e-07, 0.0014302984345704317, 0.9992700219154358, 0.9994902610778809, 0.00010575459600659087, 0.9999983310699463, 0.0010224456200376153, 0.21407535672187805, 0.9999990463256836, 3.189699782524258e-06, 0.03034425713121891, 6.201951237017056e-07, 0.03012215904891491, 1.0, 0.001010415842756629, 0.9999556541442871, 3.465549889369868e-05, 0.999945878982544, 2.942926357718534e-06, 0.0040083397179841995, 0.0004611500189639628, 0.9999911785125732, 9.093420885619707e-06, 2.4160310658771778e-06, 0.9999991655349731, 2.8669412131421268e-05, 0.004912808537483215, 0.9999974966049194, 0.9999163150787354, 0.9780072569847107, 0.9960731267929077, 0.9870532155036926, 0.05885624140501022, 0.9997757077217102, 0.002294014673680067, 0.9999939203262329, 0.00010608446609694511, 0.007887699641287327, 0.9999637603759766, 0.9993383288383484, 0.000134811649331823, 0.9999998807907104, 0.9997182488441467, 0.00012341341061983258, 0.0010519034694880247, 0.0011785654351115227, 0.9999866485595703, 0.00044645063462667167, 0.9999942779541016, 0.9998900890350342, 1.0, 0.9999969005584717, 9.237184713128954e-06, 0.004817658569663763, 3.430020296946168e-05, 1.9214849089621566e-05, 0.9999990463256836, 0.00016564976249355823, 1.0, 0.9999756813049316, 0.0022265720181167126, 0.9999997615814209, 0.0015039328718557954, 0.0002281333290738985, 1.1510634976730216e-05, 0.9947840571403503, 8.80794175373012e-07, 0.9995648264884949, 0.9999957084655762, 4.002901550848037e-05, 7.161428356994293e-07, 1.0, 0.9999849796295166, 0.9997228980064392, 0.9999990463256836, 0.9994065761566162, 0.9999992847442627, 0.9999830722808838, 0.08967439085245132, 0.9993858337402344, 0.93581223487854, 0.9999707937240601, 0.9999998807907104, 0.9999972581863403, 0.0005995880928821862, 1.0, 0.9982377290725708, 0.9999980926513672, 4.44643956143409e-06, 0.0028610918670892715, 1.7596314137335867e-05, 0.0012018019333481789, 0.9999988079071045, 1.110093421630154e-06, 0.9996129870414734, 0.00031953578582033515, 0.011902072466909885, 1.65296842880025e-08, 0.9725375175476074, 0.00028564888634718955, 1.8706738046603277e-05, 0.999975323677063, 0.9997337460517883, 2.100628535117721e-06, 0.0034457712899893522, 0.0006080130697228014, 0.9999986886978149, 8.027052899706177e-06, 0.00029871019069105387, 6.314929578365991e-06, 1.984509617614094e-05, 1.0, 0.9998087286949158, 3.926939825760201e-06, 0.999599277973175, 0.9998923540115356, 1.813604285416659e-05, 5.7179704526788555e-06, 0.0026692687533795834, 3.741470777640643e-07, 0.9999810457229614, 0.000129710475448519, 0.008363882079720497, 4.622889537131414e-05, 0.00032651665969751775, 0.9993001222610474, 0.9987733960151672, 0.9999998807907104, 0.9999979734420776, 1.9038584468944464e-06, 0.9999667406082153, 0.9933297038078308, 0.0002298382605658844, 7.5305533755454235e-06, 2.273767677252181e-05, 5.4647280478548055e-08, 1.3235692222224316e-06, 9.718414366943762e-05, 0.9999986886978149, 0.0007120834197849035, 0.9998676776885986, 7.269423076650128e-05, 1.0320668479835149e-06, 0.00013946254330221564, 0.9983206391334534, 0.002595462603494525, 0.9999991655349731, 0.9999693632125854, 8.71364536578767e-05, 0.9986407160758972, 4.449827247299254e-05, 7.983134128153324e-05, 4.3597741751000285e-05, 0.9999988079071045, 0.9991015195846558, 0.9999773502349854, 0.9995766282081604, 7.829962851246819e-05, 0.9999693632125854, 9.18048681342043e-06, 0.001330795232206583, 0.9998974800109863, 0.9999979734420776, 0.051825158298015594, 1.013442556541122e-06, 0.9999998807907104, 0.07133104652166367, 0.9999232292175293, 0.9999814033508301, 0.9998961687088013, 0.999981164932251, 3.2224058941210387e-06]

#Metricas de validación
Best_acc8_8 = 1.0
acc_aux8_8 = 1.0
Recall8_8 = 1.0
Especificidad8_8 = 1.0
Tiempo8_8 = 1932.1325089931488

######## Base9 ########
#MODELO = VGG19_BN
#Datos perdidas
xx_train9_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train9_8 = [0.06138350045607414, 0.03738947154747115, 0.03905362473961748, 0.040390426803518224, 0.029954984342610395, 0.034031276680805064, 0.029573311316378324, 0.035206816566211205, 0.021597633253277085, 0.03180237059239988, 0.029285136440101965, 0.029340025460646475, 0.030452519563245184, 0.029906674925191903, 0.024723322174431364, 0.021135664344937712, 0.02245088796372767, 0.02741747065274804, 0.019241895830189742, 0.033136445109123065, 0.029377873609831302, 0.027283717461942154, 0.03028948055005368, 0.023587440377400244, 0.02377466322095306]
xx_val9_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val9_8 = [0.014763803283373515, 0.01030918624666002, 0.006215802994039323, 0.0071166202425956724, 0.0035756609506077237, 0.002830554296573003, 0.006359629498587714, 0.0031281426548957826, 0.0032475294338332283, 0.0037308916449546813, 0.002722227366434203, 0.002207391295168135, 0.0023648829923735726, 0.00320115081138081, 0.003086863375372357, 0.003186013383997811, 0.003542958117193646, 0.001966406073835161, 0.0030451168616612754, 0.0031609687126345105, 0.0030069390932718912, 0.0033297888934612275, 0.0029491684503025477, 0.0032803506900866827, 0.0036112767954667407]
epoch_min9_8 = 5
stop_val9_8 = 0.002830554296573003

#Datos accuracy
acc_train9_8 = [0.9780864197530864, 0.9876543209876543, 0.9845679012345678, 0.9867283950617284, 0.990432098765432, 0.9882716049382716, 0.9876543209876543, 0.9901234567901235, 0.9916666666666667, 0.9882716049382716, 0.9895061728395061, 0.9907407407407407, 0.9876543209876543, 0.990432098765432, 0.9907407407407407, 0.9916666666666667, 0.991358024691358, 0.9919753086419753, 0.9932098765432098, 0.9898148148148148, 0.9895061728395061, 0.991358024691358, 0.9910493827160494, 0.9919753086419753, 0.9922839506172839]
acc_val9_8 = [0.9944444444444445, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 0.9972222222222222, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels9_8 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
best_prob9_8 = [4.573262413032353e-05, 3.352650674059987e-05, 9.104475793719757e-06, 3.798205625571427e-07, 9.200443855661433e-06, 0.002355474978685379, 8.303098729811609e-05, 0.9999992847442627, 3.574797347027925e-07, 0.03212481364607811, 0.9999774694442749, 0.9999978542327881, 0.008618778549134731, 8.40961183712352e-06, 0.9999982118606567, 3.466799171292223e-05, 0.01555473543703556, 0.9999576807022095, 2.0686384516466205e-07, 0.999745786190033, 7.300379365915433e-05, 1.417130420122703e-06, 0.9999399185180664, 1.0, 6.785800360376015e-05, 0.9998024106025696, 0.9999881982803345, 0.999943733215332, 0.00045455459621734917, 0.9947977066040039, 0.9999969005584717, 0.9999974966049194, 7.454580952526157e-08, 0.9999934434890747, 4.980428457201924e-06, 0.9999974966049194, 0.999992847442627, 1.0, 3.02339976769872e-05, 0.9999998807907104, 2.9450457077473402e-05, 6.361232954077423e-05, 0.003670864272862673, 0.9999915361404419, 0.9999909400939941, 0.9999997615814209, 0.9999932050704956, 1.0, 0.0008295243605971336, 2.2349622668116353e-06, 0.00015124658239074051, 0.9999943971633911, 4.157800958637381e-06, 7.948973507154733e-05, 0.9999966621398926, 0.9999847412109375, 0.9924848675727844, 0.9999948740005493, 0.999901533126831, 0.9999665021896362, 0.9999254941940308, 0.0006835883832536638, 1.9404983220283611e-07, 0.9999828338623047, 4.3513400669326074e-06, 0.0031429599039256573, 0.9999909400939941, 0.9999639987945557, 0.0011968542821705341, 3.801919228862971e-05, 6.279281933530001e-06, 6.941365882084938e-07, 3.4010899980785325e-05, 0.9992203712463379, 0.9999179840087891, 0.9999390840530396, 0.9998817443847656, 0.9999547004699707, 4.821672519028652e-07, 0.9999921321868896, 0.014392341487109661, 0.00016348059580195695, 0.9998878240585327, 0.999956488609314, 0.003588630584999919, 0.9999966621398926, 0.9999979734420776, 0.0001447573013138026, 1.994630292756483e-05, 0.9999940395355225, 0.9999980926513672, 0.9999866485595703, 0.0008239538874477148, 0.999934196472168, 9.936894457496237e-06, 0.9985427856445312, 4.122466270928271e-05, 0.9997413754463196, 3.073459447477944e-05, 7.068926788633689e-05, 1.1834332326543517e-05, 1.803766735974932e-06, 9.013918315758929e-06, 0.0009964139899238944, 1.3235401183919748e-06, 5.8332079788669944e-05, 0.9998977184295654, 0.00022112630540505052, 0.00020487989240791649, 4.474908564588986e-05, 0.0336720272898674, 2.6879633878706954e-06, 0.9999741315841675, 0.9999998807907104, 0.9999985694885254, 2.52223399002105e-05, 9.040271834237501e-05, 1.0921453394985292e-06, 0.9999946355819702, 3.946361903217621e-05, 0.9999992847442627, 0.9993249177932739, 1.9676733700180193e-06, 1.506819216956501e-06, 0.0002886821166612208, 0.9999997615814209, 0.9821458458900452, 2.2962276489124633e-05, 0.9999881982803345, 0.9999979734420776, 0.00011788019037339836, 8.238678856287152e-05, 8.718932076590136e-05, 0.9997389912605286, 0.0005877799121662974, 1.0, 1.0, 0.0020525322761386633, 0.9999662637710571, 0.9999998807907104, 0.9879997372627258, 0.9999581575393677, 7.390379323624074e-05, 0.9994843006134033, 0.9474512338638306, 0.9998829364776611, 9.547849913360551e-06, 1.2063539998052875e-06, 1.2012588967991178e-06, 0.0002468543534632772, 0.0009584046201780438, 0.9999984502792358, 0.9999982118606567, 0.9999890327453613, 1.8224704945168924e-07, 8.578433335060254e-05, 0.9999837875366211, 6.004748684063088e-06, 0.9979196190834045, 0.00012993230484426022, 0.9999890327453613, 2.640626007632818e-06, 0.9999823570251465, 0.000346112996339798, 0.001297004520893097, 0.018704252317547798, 1.3167782526579686e-05, 0.0007737110718153417, 0.00044492268352769315, 1.0, 0.9837910532951355, 0.9420217871665955, 0.9999997615814209, 2.6958527087117545e-05, 0.9999483823776245, 1.5006117791926954e-05, 0.9954631924629211, 0.9999991655349731, 0.9999845027923584, 0.0031980311032384634, 0.9999494552612305, 0.999998927116394, 0.9999898672103882, 0.9999998807907104, 0.9999996423721313, 0.9999232292175293, 3.912744432454929e-05, 0.0001433669967809692, 3.1805689104658086e-06, 0.9974024891853333, 6.740133540006354e-05, 0.9999517202377319, 0.9999868869781494, 0.0002873057092074305, 0.0023379481863230467, 5.451025936054066e-05, 0.000807013304438442, 9.168841643258929e-05, 2.9992315830895677e-05, 0.008612697944045067, 1.0628857438632622e-07, 0.00010183393897023052, 0.9999334812164307, 0.999970555305481, 0.9999910593032837, 2.092085651383968e-06, 1.3623754966829438e-05, 0.9999997615814209, 0.9996198415756226, 0.9998292922973633, 0.9999845027923584, 0.99950110912323, 0.9999418258666992, 0.9999997615814209, 0.9999381303787231, 0.09445597231388092, 1.0, 0.9343313574790955, 0.9999686479568481, 0.001246918924152851, 0.9999940395355225, 6.877979467390105e-05, 0.00041224516462534666, 0.9992942810058594, 0.9999966621398926, 0.9603868126869202, 3.881481916323537e-06, 0.9999992847442627, 1.0, 2.782861156447325e-05, 0.0017132244538515806, 0.9999220371246338, 0.999994158744812, 1.0, 0.9999996423721313, 0.00011982711293967441, 2.9786744562443346e-05, 0.9999921321868896, 0.00016256951494142413, 0.0003984025970567018, 0.0002701649791561067, 8.735291885386687e-06, 0.9999998807907104, 0.001008456340059638, 0.9882951974868774, 0.9999994039535522, 0.999987006187439, 6.091118848416954e-05, 0.9999972581863403, 0.0021420675329864025, 0.9999841451644897, 2.946029053418897e-05, 0.9999974966049194, 0.9999979734420776, 7.740380533505231e-05, 2.257724554510787e-05, 1.6289816358039388e-06, 7.551646376668941e-06, 0.9999963045120239, 0.00012022012379020452, 0.9999997615814209, 3.1377428967971355e-05, 0.9997978806495667, 0.00013734601088799536, 0.9981198906898499, 0.06328503042459488, 0.9999511241912842, 9.579580364516005e-05, 0.9999969005584717, 0.00028315759846009314, 0.01183914951980114, 0.0002496161323506385, 0.9997969269752502, 0.00011409975559217855, 0.9998047947883606, 2.26269435188442e-06, 0.00022364736651070416, 0.9999997615814209, 7.434488935587069e-08, 0.9999958276748657, 0.9999991655349731, 0.7279645800590515, 0.9999991655349731, 0.9999974966049194, 0.9825090169906616, 0.999984622001648, 0.9999637603759766, 0.999997615814209, 5.147076808498241e-05, 1.0, 2.0018067516502924e-05, 2.259620686118069e-07, 0.9999998807907104, 0.9999998807907104, 1.0, 1.1007469765900169e-05, 0.9999942779541016, 4.645549324777676e-06, 7.241258572321385e-05, 5.963784133200534e-08, 0.9999680519104004, 4.634361539501697e-05, 0.9999380111694336, 0.00019664934370666742, 0.9999022483825684, 0.9993413090705872, 0.00011287022061878815, 0.012390592135488987, 0.9999995231628418, 4.336220627010334e-06, 6.861328074592166e-07, 0.9999998807907104, 0.00022075933520682156, 0.9999887943267822, 1.5185040638243663e-06, 4.1644557313702535e-06, 2.3507861897087423e-06, 0.9999996423721313, 0.9998425245285034, 2.0996130842831917e-06, 5.0757174903992563e-05, 1.3355615010368638e-05, 0.0002197701542172581, 1.554959271743428e-05, 0.0001653165090829134, 0.9999881982803345, 4.1394787331228144e-06, 0.9999986886978149, 0.9999992847442627, 6.187736289575696e-05, 0.9999986886978149, 0.9999798536300659, 0.999998927116394, 0.00048008415615186095, 0.00014636051491834223, 0.9998075366020203, 4.723414349427912e-06, 0.0002619269653223455, 0.999991774559021, 2.7521580250322586e-06, 0.00047361949691548944, 0.9999971389770508, 0.9999791383743286, 0.999997615814209, 1.0, 1.7910744645632803e-05, 1.0, 0.0001412521960446611, 0.999993085861206, 0.9999992847442627, 1.630755650694482e-05, 0.9999990463256836, 1.0, 9.206680988427252e-05, 0.9999995231628418, 0.00018418666149955243, 3.316257425467484e-05, 0.00016369119111914188, 0.9998760223388672, 2.0050592866027728e-06]

#Metricas de validación
Best_acc9_8 = 1.0
acc_aux9_8 = 1.0
Recall9_8 = 1.0
Especificidad9_8 = 1.0
Tiempo9_8 = 1878.9727585315704

######## Base10 ########
#MODELO = VGG19_BN
#Datos perdidas
xx_train10_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_train10_8 = [0.048196703959026456, 0.02988163595214302, 0.0299843203506352, 0.028088774670053412, 0.038747589418917525, 0.02799238657325874, 0.020746912411701533, 0.0292345363324807, 0.023096676373187408, 0.022977431228867282, 0.027505810117280042, 0.02440055955523326, 0.02777037995832938, 0.03036753191256229, 0.03008701827055142, 0.019237112566644763, 0.020864996479617225, 0.025605305560209134, 0.01808986908491747, 0.026845168984598582, 0.030431005275911756, 0.02226863016868815, 0.020701477244312382, 0.02569222018858533, 0.030360811599242834]
xx_val10_8 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
yy_val10_8 = [0.0027205728822284276, 0.0024769557846917047, 0.01587151520782047, 0.0011939810381995308, 0.0016154832310146755, 0.0007528016964594523, 0.0010338395833969117, 0.0010166860289043851, 0.0009893728627098932, 0.0008220076560974121, 0.0010983486970265707, 0.0010730475187301636, 0.0019431814551353454, 0.0007884717649883694, 0.0008673267232047187, 0.000767913129594591, 0.0010653310351901584, 0.0021721988916397093, 0.0017862823274400498, 0.001244855092631446, 0.0005916386842727661, 0.0006814503007464939, 0.0008185174730088976, 0.00139516724480523, 0.0015526834461424086]
epoch_min10_8 = 5
stop_val10_8 = 0.0007528016964594523

#Datos accuracy
acc_train10_8 = [0.9805555555555555, 0.9895061728395061, 0.990432098765432, 0.9895061728395061, 0.9833333333333333, 0.9876543209876543, 0.9935185185185185, 0.9901234567901235, 0.991358024691358, 0.9938271604938271, 0.9910493827160494, 0.9919753086419753, 0.9882716049382716, 0.9895061728395061, 0.9882716049382716, 0.9932098765432098, 0.9922839506172839, 0.9901234567901235, 0.9938271604938271, 0.9907407407407407, 0.9898148148148148, 0.9929012345679012, 0.9941358024691358, 0.991358024691358, 0.9882716049382716]
acc_val10_8 = [1.0, 1.0, 0.9944444444444445, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

#Datos curva ROC
best_labels10_8 = [1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
best_prob10_8 = [0.9999972581863403, 0.9999681711196899, 1.0, 0.0014324873918667436, 0.9999982118606567, 1.121563627748401e-06, 0.00046340771950781345, 0.0004870179982390255, 0.9999933242797852, 0.9999998807907104, 0.9999994039535522, 0.9999430179595947, 0.0010331370867788792, 0.9999996423721313, 9.576183947501704e-05, 0.9999990463256836, 0.0003654526954051107, 0.9999783039093018, 1.9191975297871977e-05, 3.8613976357737556e-05, 0.9999969005584717, 0.9999895095825195, 2.9707016437896527e-06, 0.9999635219573975, 5.0734332035062835e-05, 4.102344064449426e-06, 0.00011813978926511481, 0.9999785423278809, 0.00022583130339626223, 0.9999991655349731, 0.0013716708635911345, 0.9999985694885254, 9.704625699669123e-05, 0.0018366641597822309, 0.9999947547912598, 0.9999992847442627, 0.0006599283660762012, 0.9755967259407043, 0.9999922513961792, 0.9999994039535522, 0.9998075366020203, 0.0003552609123289585, 1.1609756256802939e-05, 4.37355374742765e-05, 0.0024467024486511946, 1.0, 0.9999994039535522, 0.9999984502792358, 0.999997615814209, 0.9999995231628418, 0.9996533393859863, 0.9999847412109375, 0.0002310171548742801, 0.0017911026952788234, 0.0003829417109955102, 0.0012148525565862656, 3.392473445273936e-05, 6.15773897152394e-05, 1.0, 9.54087227000855e-07, 0.00017279942403547466, 0.9999973773956299, 5.189289731788449e-05, 0.004712188150733709, 1.0, 0.9999665021896362, 0.9993791580200195, 0.9999451637268066, 0.9999991655349731, 0.9999862909317017, 3.903591277776286e-05, 0.9999655485153198, 0.00010814893175847828, 0.9999997615814209, 1.54276149260113e-05, 0.9999945163726807, 2.8874508188891923e-06, 0.0006904519395902753, 0.9999841451644897, 0.9999997615814209, 1.3762205526290927e-06, 7.395230932161212e-05, 0.004004545509815216, 1.0, 0.9999996423721313, 0.9934735894203186, 0.006971070077270269, 0.9999983310699463, 0.0009500178275629878, 0.9999960660934448, 0.9999821186065674, 0.9999998807907104, 0.0001223014114657417, 0.00018520629964768887, 2.8679198749159696e-07, 0.9999958276748657, 1.0, 0.9999927282333374, 0.9999963045120239, 0.00013561244122684002, 0.999958872795105, 0.000266103888861835, 0.00024689530255272985, 1.4044770068721846e-05, 0.9999867677688599, 0.9999874830245972, 0.9999860525131226, 0.0005598622374236584, 9.719053923618048e-05, 0.9999982118606567, 0.007011097855865955, 6.384303219419962e-07, 0.9996459484100342, 6.751178602826258e-07, 1.0, 0.0006523639895021915, 8.395870281674433e-06, 0.9999997615814209, 5.937290552537888e-05, 0.0010420255130156875, 4.495880148169817e-06, 0.0040664514526724815, 4.7006058593979105e-05, 0.9999548196792603, 1.1037184776796494e-05, 0.9999366998672485, 0.00012320034147705883, 0.9999961853027344, 7.889691914897412e-06, 6.857572589069605e-05, 7.475449820049107e-05, 0.9622516632080078, 3.716040009749122e-05, 0.9999997615814209, 2.0244642655598e-05, 1.0, 0.999997615814209, 0.9999998807907104, 2.647989504112047e-06, 1.0, 0.9999998807907104, 5.7902121625375e-05, 0.9999991655349731, 0.9999887943267822, 1.2846542176703224e-06, 0.9999998807907104, 0.00012874648382421583, 0.00024214645964093506, 0.0001032953878166154, 0.999996542930603, 5.867199433851056e-06, 0.00011706080113071948, 0.9994457364082336, 0.0002133701927959919, 0.9999996423721313, 3.3621981856413186e-05, 0.9999942779541016, 8.265578799182549e-06, 7.4117115218541585e-06, 0.9999688863754272, 0.003141609486192465, 0.00985447596758604, 0.9999876022338867, 1.0, 0.0012876702239736915, 1.0, 0.00012030293146381155, 4.2298224798287265e-06, 5.3400402975967154e-05, 0.9978812336921692, 0.9998364448547363, 0.9999949932098389, 0.9999501705169678, 0.00015132837870623916, 1.0, 0.99997878074646, 9.83141489996342e-07, 0.9999963045120239, 0.9999972581863403, 1.119796934290207e-06, 0.9999425411224365, 1.0, 0.9999998807907104, 1.8617360183270648e-05, 0.9999983310699463, 0.9999995231628418, 0.9999995231628418, 0.9999895095825195, 0.001049890648573637, 0.9999890327453613, 0.00016253635112661868, 0.000979900942184031, 0.0002372273593209684, 0.999998927116394, 0.00012446215259842575, 0.0011427346616983414, 0.9999814033508301, 1.0, 1.1518807241372997e-06, 2.518532710382715e-05, 0.9999986886978149, 7.249754617077997e-06, 0.9981426000595093, 9.991373372031376e-05, 9.739845700096339e-05, 0.9999994039535522, 8.999356214189902e-05, 1.0, 0.9999653100967407, 0.00021458861010614783, 0.005536298733204603, 4.249441553838551e-05, 1.5819456393728615e-06, 5.060348485130817e-05, 0.0016952389851212502, 0.9967106580734253, 0.9999918937683105, 4.820499725610716e-06, 0.9999998807907104, 0.9999644756317139, 3.517444565659389e-05, 0.999832272529602, 6.364812725223601e-05, 2.8282289349590428e-05, 0.9994365572929382, 0.9997625946998596, 0.001231093192473054, 0.0001380198955303058, 2.3638817765458953e-06, 0.0003420135471969843, 0.9999969005584717, 0.9997748732566833, 0.00042218618909828365, 0.0017185186734423041, 0.9999945163726807, 7.613905472680926e-05, 0.9999886751174927, 2.6574522053124383e-05, 0.9999909400939941, 7.776176857987593e-07, 8.006349162315018e-06, 3.1466936434298987e-06, 0.9999991655349731, 1.73471198650077e-05, 3.818277036771178e-05, 4.043346780235879e-05, 1.9771914594457485e-05, 0.999998927116394, 0.9999939203262329, 1.0, 0.9999963045120239, 0.9999943971633911, 2.9268301204865566e-06, 0.999998927116394, 3.6698218082165113e-06, 5.782487278338522e-05, 0.9999970197677612, 0.002223258838057518, 0.9999991655349731, 0.9999953508377075, 0.9998519420623779, 0.9999638795852661, 1.799182064132765e-05, 1.0, 8.168771455530077e-05, 0.9999885559082031, 0.9999960660934448, 0.0026118385139852762, 0.00014339968038257211, 0.9998825788497925, 1.0, 8.538784641132224e-06, 8.762934157857671e-05, 9.156407827504154e-07, 0.9999977350234985, 0.9999998807907104, 0.9998176693916321, 0.9996108412742615, 0.9999802112579346, 0.9999567270278931, 0.00016426751972176135, 1.0191560022576596e-06, 7.150337478378788e-05, 0.9999780654907227, 0.9999997615814209, 0.9999866485595703, 3.510204749090917e-07, 2.7492185836308636e-05, 0.0005424695555120707, 2.468507045705337e-05, 0.039939507842063904, 6.787257007090375e-05, 1.2314957530179527e-05, 0.9999977350234985, 0.0006743842386640608, 0.9999996423721313, 0.0001192579438793473, 0.9999988079071045, 1.00848219517502e-05, 1.0, 3.1591443985234946e-05, 0.9999712705612183, 0.9999910593032837, 1.2812296290576342e-06, 4.1211060306522995e-05, 1.0, 0.999990701675415, 0.9999722242355347, 2.241826405224856e-05, 0.9999970197677612, 5.936970410402864e-06, 0.00044976407662034035, 0.0016412509139627218, 0.999992847442627, 5.620310730591882e-06, 0.9999933242797852, 1.0, 0.9999997615814209, 0.9998748302459717, 0.9947296380996704, 0.9999996423721313, 0.9999998807907104, 0.9999692440032959, 0.0029602604918181896, 1.2869030570072937e-06, 0.9999921321868896, 0.999980092048645, 0.0006672827294096351, 0.9999725818634033, 0.9995057582855225, 0.024401037022471428, 0.9999997615814209, 0.9999994039535522, 2.103081533277873e-05, 0.00011088913743151352, 2.4722859961912036e-05, 0.9999953508377075, 9.849900379776955e-05, 0.9956156015396118, 0.0004545875417534262, 0.9999997615814209, 0.9999996423721313, 0.9999980926513672, 3.883210865751607e-06, 0.9998947381973267, 4.157211151323281e-05, 4.900131170870736e-05, 0.001452479511499405, 1.7119436961365864e-05, 0.9999310970306396, 0.0010519424686208367, 0.00012336975487414747, 1.152378445112845e-06, 0.9999986886978149, 0.0008147869957610965, 0.014464999549090862, 3.211733201169409e-05, 0.9999169111251831, 0.9999760389328003, 0.0016574328765273094]

#Metricas de validación
Best_acc10_8 = 1.0
acc_aux10_8 = 1.0
Recall10_8 = 1.0
Especificidad10_8 = 1.0
Tiempo10_8 = 1871.1891658306122

In [ ]:
prom1=(Best_acc1_1+Best_acc2_1+Best_acc3_1+Best_acc4_1+Best_acc5_1+Best_acc6_1+Best_acc7_1+Best_acc8_1+Best_acc9_1+Best_acc10_1)/10
prom2=(Best_acc1_2+Best_acc2_2+Best_acc3_2+Best_acc4_2+Best_acc5_2+Best_acc6_2+Best_acc7_2+Best_acc8_2+Best_acc9_2+Best_acc10_2)/10
prom3=(Best_acc1_3+Best_acc2_3+Best_acc3_3+Best_acc4_3+Best_acc5_3+Best_acc6_3+Best_acc7_3+Best_acc8_3+Best_acc9_3+Best_acc10_3)/10
prom4=(Best_acc1_4+Best_acc2_4+Best_acc3_4+Best_acc4_4+Best_acc5_4+Best_acc6_4+Best_acc7_4+Best_acc8_4+Best_acc9_4+Best_acc10_4)/10
prom5=(Best_acc1_5+Best_acc2_5+Best_acc3_5+Best_acc4_5+Best_acc5_5+Best_acc6_5+Best_acc7_5+Best_acc8_5+Best_acc9_5+Best_acc10_5)/10
prom6=(Best_acc1_6+Best_acc2_6+Best_acc3_6+Best_acc4_6+Best_acc5_6+Best_acc6_6+Best_acc7_6+Best_acc8_6+Best_acc9_6+Best_acc10_6)/10
prom7=(Best_acc1_7+Best_acc2_7+Best_acc3_7+Best_acc4_7+Best_acc5_7+Best_acc6_7+Best_acc7_7+Best_acc8_7+Best_acc9_7+Best_acc10_7)/10
prom8=(Best_acc1_8+Best_acc2_8+Best_acc3_8+Best_acc4_8+Best_acc5_8+Best_acc6_8+Best_acc7_8+Best_acc8_8+Best_acc9_8+Best_acc10_8)/10

print("\nAccuracy\n")
print("{:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f}" .format(prom1, Best_acc1_1, Best_acc2_1, Best_acc3_1, Best_acc4_1, Best_acc5_1, Best_acc6_1, Best_acc7_1, Best_acc8_1, Best_acc9_1, Best_acc10_1))
print("{:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f}" .format(prom2, Best_acc1_2, Best_acc2_2, Best_acc3_2, Best_acc4_2, Best_acc5_2, Best_acc6_2, Best_acc7_2, Best_acc8_2, Best_acc9_2, Best_acc10_2))
print("{:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f}" .format(prom3, Best_acc1_3, Best_acc2_3, Best_acc3_3, Best_acc4_3, Best_acc5_3, Best_acc6_3, Best_acc7_3, Best_acc8_3, Best_acc9_3, Best_acc10_3))
print("{:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f}" .format(prom4, Best_acc1_4, Best_acc2_4, Best_acc3_4, Best_acc4_4, Best_acc5_4, Best_acc6_4, Best_acc7_4, Best_acc8_4, Best_acc9_4, Best_acc10_4))
print("{:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f}" .format(prom5, Best_acc1_5, Best_acc2_5, Best_acc3_5, Best_acc4_5, Best_acc5_5, Best_acc6_5, Best_acc7_5, Best_acc8_5, Best_acc9_5, Best_acc10_5))
print("{:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f}" .format(prom6, Best_acc1_6, Best_acc2_6, Best_acc3_6, Best_acc4_6, Best_acc5_6, Best_acc6_6, Best_acc7_6, Best_acc8_6, Best_acc9_6, Best_acc10_6))
print("{:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f}" .format(prom7, Best_acc1_7, Best_acc2_7, Best_acc3_7, Best_acc4_7, Best_acc5_7, Best_acc6_7, Best_acc7_7, Best_acc8_7, Best_acc9_7, Best_acc10_7))
print("{:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f} {:.4f}" .format(prom8, Best_acc1_8, Best_acc2_8, Best_acc3_8, Best_acc4_8, Best_acc5_8, Best_acc6_8, Best_acc7_8, Best_acc8_8, Best_acc9_8, Best_acc10_8))


promrec1=(Recall1_1+Recall2_1+Recall3_1+Recall4_1+Recall5_1+Recall6_1+Recall7_1+Recall8_1+Recall9_1+Recall10_1)/10
promrec2=(Recall1_2+Recall2_2+Recall3_2+Recall4_2+Recall5_2+Recall6_2+Recall7_2+Recall8_2+Recall9_2+Recall10_2)/10
promrec3=(Recall1_3+Recall2_3+Recall3_3+Recall4_3+Recall5_3+Recall6_3+Recall7_3+Recall8_3+Recall9_3+Recall10_3)/10
promrec4=(Recall1_4+Recall2_4+Recall3_4+Recall4_4+Recall5_4+Recall6_4+Recall7_4+Recall8_4+Recall9_4+Recall10_4)/10
promrec5=(Recall1_5+Recall2_5+Recall3_5+Recall4_5+Recall5_5+Recall6_5+Recall7_5+Recall8_5+Recall9_5+Recall10_5)/10
promrec6=(Recall1_6+Recall2_6+Recall3_6+Recall4_6+Recall5_6+Recall6_6+Recall7_6+Recall8_6+Recall9_6+Recall10_6)/10
promrec7=(Recall1_7+Recall2_7+Recall3_7+Recall4_7+Recall5_7+Recall6_7+Recall7_7+Recall8_7+Recall9_7+Recall10_7)/10
promrec8=(Recall1_8+Recall2_8+Recall3_8+Recall4_8+Recall5_8+Recall6_8+Recall7_8+Recall8_8+Recall9_8+Recall10_8)/10

promesp1=(Especificidad1_1+Especificidad2_1+Especificidad3_1+Especificidad4_1+Especificidad5_1+Especificidad6_1+Especificidad7_1+Especificidad8_1+Especificidad9_1+Especificidad10_1)/10
promesp2=(Especificidad1_2+Especificidad2_2+Especificidad3_2+Especificidad4_2+Especificidad5_2+Especificidad6_2+Especificidad7_2+Especificidad8_2+Especificidad9_2+Especificidad10_2)/10
promesp3=(Especificidad1_3+Especificidad2_3+Especificidad3_3+Especificidad4_3+Especificidad5_3+Especificidad6_3+Especificidad7_3+Especificidad8_3+Especificidad9_3+Especificidad10_3)/10
promesp4=(Especificidad1_4+Especificidad2_4+Especificidad3_4+Especificidad4_4+Especificidad5_4+Especificidad6_4+Especificidad7_4+Especificidad8_4+Especificidad9_4+Especificidad10_4)/10
promesp5=(Especificidad1_5+Especificidad2_5+Especificidad3_5+Especificidad4_5+Especificidad5_5+Especificidad6_5+Especificidad7_5+Especificidad8_5+Especificidad9_5+Especificidad10_5)/10
promesp6=(Especificidad1_6+Especificidad2_6+Especificidad3_6+Especificidad4_6+Especificidad5_6+Especificidad6_6+Especificidad7_6+Especificidad8_6+Especificidad9_6+Especificidad10_6)/10
promesp7=(Especificidad1_7+Especificidad2_7+Especificidad3_7+Especificidad4_7+Especificidad5_7+Especificidad6_7+Especificidad7_7+Especificidad8_7+Especificidad9_7+Especificidad10_7)/10
promesp8=(Especificidad1_8+Especificidad2_8+Especificidad3_8+Especificidad4_8+Especificidad5_8+Especificidad6_8+Especificidad7_8+Especificidad8_8+Especificidad9_8+Especificidad10_8)/10

#print("\n")
print("\nMetricas\n")
print("{:.4f} {:.4f} {:.4f}" .format(prom1, promrec1, promesp1))
print("{:.4f} {:.4f} {:.4f}" .format(prom2, promrec2, promesp2))
print("{:.4f} {:.4f} {:.4f}" .format(prom3, promrec3, promesp3))
print("{:.4f} {:.4f} {:.4f}" .format(prom4, promrec4, promesp4))
print("{:.4f} {:.4f} {:.4f}" .format(prom5, promrec5, promesp5))
print("{:.4f} {:.4f} {:.4f}" .format(prom6, promrec6, promesp6))
print("{:.4f} {:.4f} {:.4f}" .format(prom7, promrec7, promesp7))
print("{:.4f} {:.4f} {:.4f}" .format(prom8, promrec8, promesp8))


promearlyStop1=(epoch_min1_1+epoch_min2_1+epoch_min3_1+epoch_min4_1+epoch_min5_1+epoch_min6_1+epoch_min7_1+epoch_min8_1+epoch_min9_1+epoch_min10_1)/10
promearlyStop2=(epoch_min1_2+epoch_min2_2+epoch_min3_2+epoch_min4_2+epoch_min5_2+epoch_min6_2+epoch_min7_2+epoch_min8_2+epoch_min9_2+epoch_min10_2)/10
promearlyStop3=(epoch_min1_3+epoch_min2_3+epoch_min3_3+epoch_min4_3+epoch_min5_3+epoch_min6_3+epoch_min7_3+epoch_min8_3+epoch_min9_3+epoch_min10_3)/10
promearlyStop4=(epoch_min1_4+epoch_min2_4+epoch_min3_4+epoch_min4_4+epoch_min5_4+epoch_min6_4+epoch_min7_4+epoch_min8_4+epoch_min9_4+epoch_min10_4)/10
promearlyStop5=(epoch_min1_5+epoch_min2_5+epoch_min3_5+epoch_min4_5+epoch_min5_5+epoch_min6_5+epoch_min7_5+epoch_min8_5+epoch_min9_5+epoch_min10_5)/10
promearlyStop6=(epoch_min1_6+epoch_min2_6+epoch_min3_6+epoch_min4_6+epoch_min5_6+epoch_min6_6+epoch_min7_6+epoch_min8_6+epoch_min9_6+epoch_min10_6)/10
promearlyStop7=(epoch_min1_7+epoch_min2_7+epoch_min3_7+epoch_min4_7+epoch_min5_7+epoch_min6_7+epoch_min7_7+epoch_min8_7+epoch_min9_7+epoch_min10_7)/10
promearlyStop8=(epoch_min1_8+epoch_min2_8+epoch_min3_8+epoch_min4_8+epoch_min5_8+epoch_min6_8+epoch_min7_8+epoch_min8_8+epoch_min9_8+epoch_min10_8)/10


print("\nEpocas {:.0f} {:.0f} {:.0f} {:.0f} {:.0f} {:.0f} {:.0f} {:.0f}" .format(promearlyStop1, promearlyStop2, promearlyStop3, promearlyStop4, promearlyStop5, promearlyStop6, promearlyStop7, promearlyStop8))

tiempo1=(Tiempo1_1+Tiempo2_1+Tiempo3_1+Tiempo4_1+Tiempo5_1+Tiempo6_1+Tiempo7_1+Tiempo8_1+Tiempo9_1+Tiempo10_1)/10
tiempo2=(Tiempo1_2+Tiempo2_2+Tiempo3_2+Tiempo4_2+Tiempo5_2+Tiempo6_2+Tiempo7_2+Tiempo8_2+Tiempo9_2+Tiempo10_2)/10
tiempo3=(Tiempo1_3+Tiempo2_3+Tiempo3_3+Tiempo4_3+Tiempo5_3+Tiempo6_3+Tiempo7_3+Tiempo8_3+Tiempo9_3+Tiempo10_3)/10
tiempo4=(Tiempo1_4+Tiempo2_4+Tiempo3_4+Tiempo4_4+Tiempo5_4+Tiempo6_4+Tiempo7_4+Tiempo8_4+Tiempo9_4+Tiempo10_4)/10
tiempo5=(Tiempo1_5+Tiempo2_5+Tiempo3_5+Tiempo4_5+Tiempo5_5+Tiempo6_5+Tiempo7_5+Tiempo8_5+Tiempo9_5+Tiempo10_5)/10
tiempo6=(Tiempo1_6+Tiempo2_6+Tiempo3_6+Tiempo4_6+Tiempo5_6+Tiempo6_6+Tiempo7_6+Tiempo8_6+Tiempo9_6+Tiempo10_6)/10
tiempo7=(Tiempo1_7+Tiempo2_7+Tiempo3_7+Tiempo4_7+Tiempo5_7+Tiempo6_7+Tiempo7_7+Tiempo8_7+Tiempo9_7+Tiempo10_7)/10
tiempo8=(Tiempo1_8+Tiempo2_8+Tiempo3_8+Tiempo4_8+Tiempo5_8+Tiempo6_8+Tiempo7_8+Tiempo8_8+Tiempo9_8+Tiempo10_8)/10

print("\nTiempo {:.0f} {:.0f} {:.0f} {:.0f} {:.0f} {:.0f} {:.0f} {:.0f}" .format(tiempo1, tiempo2, tiempo3, tiempo4, tiempo5, tiempo6, tiempo7, tiempo8))
print("\nTiempo {:.0f} {:.0f} {:.0f} {:.0f} {:.0f} {:.0f} {:.0f} {:.0f}" .format(tiempo1/25*promearlyStop1, tiempo2/25*promearlyStop2, tiempo3/25*promearlyStop3, tiempo4/25*promearlyStop4, tiempo5/25*promearlyStop5, tiempo6/25*promearlyStop6, tiempo7/25*promearlyStop7, tiempo8/25*promearlyStop8))



Accuracy

0.9922 0.9806 0.9750 0.9889 1.0000 1.0000 0.9889 0.9917 1.0000 0.9972 1.0000
0.9964 0.9861 0.9833 0.9944 1.0000 1.0000 1.0000 1.0000 1.0000 1.0000 1.0000
0.9942 0.9833 0.9861 0.9972 0.9917 0.9972 0.9944 0.9944 1.0000 0.9972 1.0000
0.9950 0.9806 0.9778 1.0000 1.0000 1.0000 1.0000 0.9944 1.0000 0.9972 1.0000
0.9939 0.9806 0.9806 0.9944 1.0000 1.0000 0.9972 0.9917 0.9972 0.9972 1.0000
0.9944 0.9889 0.9889 0.9972 0.9972 1.0000 0.9917 0.9917 1.0000 0.9917 0.9972
0.9931 0.9861 0.9833 0.9944 0.9861 1.0000 0.9944 0.9861 1.0000 1.0000 1.0000
0.9947 0.9833 0.9944 0.9972 0.9861 1.0000 1.0000 0.9861 1.0000 1.0000 1.0000

Metricas

0.9922 0.9885 0.9961
0.9964 0.9972 0.9956
0.9942 0.9950 0.9934
0.9950 0.9951 0.9951
0.9939 0.9917 0.9961
0.9944 0.9950 0.9939
0.9931 0.9923 0.9940
0.9947 0.9967 0.9929

Epocas 4 3 3 5 4 4 3 4

Tiempo 875 672 1234 1284 1497 1582 1609 1835

Tiempo 123 86 158 267 234 228 206 330


In [ ]:
      #vgg16 = models.vgg11(pretrained=True)
      #vgg16 = models.vgg11_bn(pretrained=True)
      #vgg16 = models.vgg13(pretrained=True)
      #vgg16 = models.vgg13_bn(pretrained=True)
      #vgg16 = models.vgg16(pretrained=True)
      #vgg16 = models.vgg16_bn(pretrained=True)
      #vgg16 = models.vgg19(pretrained=True)
      #vgg16 = models.vgg19_bn(pretrained=True)

      #model_ft = models.resnet34(pretrained=True)
print(torch.__version__)

#model_full = models.resnet152(pretrained=True)
#torch.save(model_full,'/content/drive/MyDrive/modelos guardados/modelo_Resnet152_full.pth')
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from PIL import Image
import numpy as np
from torch.autograd import Variable
# TODO: Write a function that loads a checkpoint and rebuilds the model
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = torch.load('/content/drive/MyDrive/modelos guardados/modelo_Vgg11_full.pth')
    #num_ftrs = model.fc.in_features  #numero de neuronas en la capa de entrada 
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs,2)
    #model.fc = nn.Linear(num_ftrs, 2) 
    model.load_state_dict(checkpoint['state_dict'])
    
    return model, checkpoint['class_to_idx']


def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch    
    model, returns an Numpy array '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    size = 256, 256
    image.thumbnail(size, Image.ANTIALIAS)
    image = image.crop((128 - 112, 128 - 112, 128 + 112, 128 + 112))
    npImage = np.array(image)
    npImage = npImage/255.
        
    imgA = npImage[:,:,0]
    imgB = npImage[:,:,1]
    imgC = npImage[:,:,2]
    
    imgA = (imgA - 0.485)/(0.229) 
    imgB = (imgB - 0.456)/(0.224)
    imgC = (imgC - 0.406)/(0.225)
        
    npImage[:,:,0] = imgA
    npImage[:,:,1] = imgB
    npImage[:,:,2] = imgC
    
    npImage = np.transpose(npImage, (2,0,1))
    
    return npImage



def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained 
    deep learning model.'''
    
    # TODO: Implement the code to predict the class from an image file
    
    image = torch.FloatTensor([process_image(Image.open(image_path))])
    model.eval()
    output = model.forward(Variable(image))
    pobabilities = torch.exp(output).data.numpy()[0]
    top_idx = np.argsort(pobabilities)[-topk:][::-1] 
    top_class = [idx_to_class[x] for x in top_idx]
    top_probability = pobabilities[top_idx]
    return top_probability, top_class

# get index to class mapping
loaded_model, class_to_idx = load_checkpoint('/content/drive/MyDrive/modelos guardados/VGG11 model_It4.pth')
idx_to_class = { v : k for k,v in class_to_idx.items()}

top_probability, top_class=predict('/content/drive/MyDrive/img_prueba/Color6.jpg', loaded_model)
print(top_class)
print(top_probability)
     

1.3.0+cu100
['Incendio', 'No incendio']
[2.6737301e+01 2.6482413e-02]


In [ ]:
print(torch.__version__)

1.3.0+cu100


In [ ]:
print("asdasdadasd")

asdasdadasd
